<a id='home'></a>

# Performance Analysis of VersionedHDF5 Files

For these tests, we have generated `.h5` data files using the `generate_data_deterministic.py` script from the [VersionedHDF5 repository](https://github.com/Quansight/versioned-hdf5), using the standard options ([see details here](#standard))

We performed the following tests:
1. [Test Large Fraction Changes Sparse](#test1)
2. [Test Mostly Appends Sparse](#test2)
3. [Test Small Fraction Changes Sparse](#test3)
4. [Test Mostly Appends Dense](#test4)

**These tests were last run on**

In [ ]:
from datetime import datetime
print(datetime.utcnow(), "UTC")

## Setup

The path to the generated test files is

In [ ]:
path = "/home/melissa/projects/versioned-hdf5/analysis" # change this as necessary

In [ ]:
%matplotlib inline
import h5py
import json
import numpy as np
import performance_tests
import matplotlib.pyplot as plt

The information from the generated test files are stored in either
- `testcase.tests`, a dictionary containing all the info related to a testcase that was run recently;
- a `.json` file named after the test name and options, containing a summary of the results. This file can be read with
    ```python
    with open(f"{testname}.json", "r") as json_in:
        test = json.load(json_in)
    ```

<a id='test1'></a>

# Test 1: Large fraction changes (sparse)

In [ ]:
testname = "test_large_fraction_changes_sparse"

We have tested the following numbers of versions (or transactions):

```python
num_transactions_1 = [50, 100, 500, 1000, 5000, 10000]
```

### Generating new tests 

If you want to generate the files now, modify the following constants for the desired tests. **Please keep in mind that file sizes can become very large for large numbers of transactions (above 5000 transactions).**

In [ ]:
num_transactions_1 = [50, 100, 500]

For the chunk size parameter, we have tested chunk sizes of $2^8, 2^{10}, 2^{12}$ and $2^{14}$.

In [ ]:
exponents_1 = [12, 14]

Choose desired compression algorithm.

In [ ]:
compression_1 = [None, "gzip", "lzf"]

Create files:

In [ ]:
testcase = performance_tests.test_large_fraction_changes_sparse(path=path,
                                                                num_transactions=num_transactions_1, 
                                                                exponents=exponents_1, 
                                                                compression=compression_1)
testcase_1, msg = testcase.create_files()
if msg:
    print(msg)
t_sizes_1 = [test['theoretical_sizes'] for test in testcase.tests[-len(num_transactions_1):]]

If you wish to save the results for later, execute the cell below: **CAUTION: This may erase previous versions of saved tests.**

In [ ]:
#testcase.save(testcase_1)

The file will be saved as

In [ ]:
f"{testname}.json"

### Reading previously computed tests

If you prefer, you can read an existing `.json` file by uncommenting the lines below.

In [ ]:
#with open(f"{testname}.json", "r") as json_in:
#    testcase_1 = json.load(json_in)
#t_sizes_1 = None

## Analysis

We'll start by analyzing how the `.h5` file sizes grow as the number of versions grows. 

Note that the array size also grows as the number of versions grows, since each transaction is changing the original arrays by adding, deleting and changing values in the original arrays. In order to compute a (naive) theoretical lower bound on the file size, we'll compute how much space each version should take. Keep in mind there is redundant data as some of it is not changed during the staging of a new version but it is still being stored. In this example, we start with three arrays with 5000 elements (2 integer arrays and one float), and in the end we have the following array sizes:

```
Maximum array size for file with 50 transactions: 5500
Maximum array size for file with 100 transactions: 6000
Maximum array size for file with 500 transactions: 10000
Maximum array size for file with 1000 transactions: 15000
Maximum array size for file with 5000 transactions: 55000
Maximum array size for file with 10000 transactions: 105000
```

Let's show the size information in a plot:

In [ ]:
num_transactions_1 = [test['num_transactions'] for test in testcase_1]
chunk_sizes_1 = [test['chunk_size'] for test in testcase_1]
compression_1 = [test['compression'] for test in testcase_1]

In [ ]:
filesizes_1 = np.array([test['size'] for test in testcase_1])
sizelabels_1 = np.array([test['size_label'] for test in testcase_1])

n = len(set(num_transactions_1))
ncs = len(set(chunk_sizes_1))
ncomp = len(set(compression_1))

fig_large_fraction_changes = plt.figure(figsize=(14,10))

if t_sizes_1:
    plt.plot(num_transactions_1[:n], t_sizes_1, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_1[:n],
                 filesizes_1[start+j*n:start+(j+1)*n],
                 '*--', ms=12, 
                 label=f"Chunk size {chunk_sizes_1[start+j*n]}, {compression_1[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
selected = [7, 9, 12, 14, 15]
plt.yticks(filesizes_1[selected], sizelabels_1[selected])
plt.xticks(num_transactions_1[:n])
plt.grid(True)
plt.show()

Changing the view to a logarithmic scale, we have the following:

In [ ]:
fig_large_fraction_changes_log = plt.figure(figsize=(14,10))

if t_sizes_1:
    plt.loglog(num_transactions_1[:n], t_sizes_1, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.loglog(num_transactions_1[:n],
                 filesizes_1[start+j*n:start+(j+1)*n],
                 '*--', ms=12, 
                 label=f"Chunk size {chunk_sizes_1[start+j*n]}, {compression_1[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
plt.grid(True)
plt.xticks(num_transactions_1[:n], num_transactions_1[:n])
plt.yticks(filesizes_1[selected], sizelabels_1[selected])
plt.minorticks_off()
plt.show()

### Comparing compression algorithms

For each chunk size that we chose to test, let's compare the file sizes corresponding to each compression algorithm that we used.

In [ ]:
fig_comp, ax_comp = plt.subplots(ncs, figsize=(10,8), sharey=True)
fig_comp.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_comp[j].loglog(num_transactions_1[:n],
                            filesizes_1[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"{compression_1[start]}")
        ax_comp[j].legend()
        ax_comp[j].set_title(f"Chunk Size {chunk_sizes_1[start+j*n]}")
        ax_comp[j].set_xticks(num_transactions_1[:n])
        ax_comp[j].set_xticklabels(num_transactions_1[:n])
        ax_comp[j].set_yticks(filesizes_1[selected])
        ax_comp[j].set_yticklabels(sizelabels_1[selected])
        ax_comp[j].grid()
        ax_comp[j].minorticks_off()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

### Comparing chunk sizes

Now, for each choice of compression algorithm, we compare different chunk sizes.

In [ ]:
fig_chunks, ax_chunks = plt.subplots(ncomp, figsize=(10,10), sharey=True)
fig_chunks.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_chunks[i].loglog(num_transactions_1[:n],
                            filesizes_1[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"Chunk size {chunk_sizes_1[start+j*n]}")
        ax_chunks[i].legend()
        ax_chunks[i].set_title(f"Compression: {compression_1[start]}")
        ax_chunks[i].set_xticks(num_transactions_1[:n])
        ax_chunks[i].set_xticklabels(num_transactions_1[:n])
        ax_chunks[i].set_yticks(filesizes_1[selected])
        ax_chunks[i].set_yticklabels(sizelabels_1[selected])
        ax_chunks[i].minorticks_off()
        ax_chunks[i].grid()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

## Creation times

If we look at the creation times for these files, we have something like this:

In [ ]:
t_write_1 = np.array([test['t_write'][-1] for test in testcase_1])

fig_large_fraction_changes_times = plt.figure(figsize=(10,8))
for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_1[:n], 
                 t_write_1[start+j*n:start+(j+1)*n], 
                 'o--', ms=8, 
                 label=f"Chunk size {chunk_sizes_1[start+j*n]}, {compression_1[start]}")
plt.plot(5000, 551.6725962162018, 'k*', ms=10, label=f"Chunk size 4096, None, No versioning")
plt.plot(5000, 526.3601763248444, '*', ms=10, label=f"Chunk size 16384, None, No versioning")
plt.plot(5000, 528.1062672138214, '*', ms=10, label=f"Chunk size 4096, gzip, No versioning")
plt.plot(5000, 540.9422471523285, '*', ms=10, label=f"Chunk size 16384, gzip, No versioning")
plt.plot(5000, 569.4801971912384, '*', ms=10, label=f"Chunk size 4096, lzf, No versioning")
plt.plot(5000, 537.3106207847595, '*', ms=10, label=f"Chunk size 16384, lzf, No versioning")
plt.xlabel("Transactions")
plt.title(f"{testname} - creation times in seconds")
plt.legend()
plt.xticks(num_transactions_1[:n])
plt.show()

Now, we can look at the time required to stage a new version in the file, that is, to add a new transaction. The graphs below show, for each fixed number of transactions, the time required to add new versions as the file is created.

In [ ]:
fig_times, ax = plt.subplots(n+1, figsize=(14,20))
fig_times.suptitle(f"{testname}: time to write each new version")

for i in range(n):
    for test in testcase_1:
        if test['num_transactions'] == num_transactions_1[i]:
            t_write = np.array(test['t_write'][:-1])
            ax[i].plot(t_write, 
                       label=f"chunk size {test['chunk_size']}, {test['compression']}")
            ax[i].legend(loc='center left')
            
t_12_none = [0.13626360893249512, 0.10637974739074707, 0.10327792167663574, 0.10448169708251953, 0.10272550582885742, 0.10271954536437988, 0.14571070671081543, 0.11351227760314941, 0.10247993469238281, 0.10403680801391602, 0.10609984397888184, 0.10461068153381348, 0.10340666770935059, 0.10634231567382812, 0.11178827285766602, 0.18336200714111328, 0.19301104545593262, 0.26909804344177246, 0.20513677597045898, 0.1351337432861328, 0.10628437995910645, 0.10613322257995605, 0.10387039184570312, 0.1156771183013916, 0.10837006568908691, 0.10351014137268066, 0.10378479957580566, 0.10333061218261719, 0.10912418365478516, 0.10658478736877441, 0.10568642616271973, 0.11689424514770508, 0.11127734184265137, 0.10451626777648926, 0.11055397987365723, 0.1154627799987793, 0.10529494285583496, 0.10359668731689453, 0.10446286201477051, 0.10953712463378906, 0.1274571418762207, 0.1039736270904541, 0.10523247718811035, 0.10434246063232422, 0.10897254943847656, 0.10372591018676758, 0.10596466064453125, 0.10621285438537598, 0.1053769588470459, 0.10953736305236816, 0.10416555404663086, 0.10305547714233398, 0.10362482070922852, 0.10366082191467285, 0.11280322074890137, 0.10446667671203613, 0.10582971572875977, 0.10350728034973145, 0.10480737686157227, 0.10952877998352051, 0.10376739501953125, 0.10343432426452637, 0.10410642623901367, 0.10702848434448242, 0.11842536926269531, 0.1035776138305664, 0.103363037109375, 0.11658573150634766, 0.10438132286071777, 0.10349321365356445, 0.10321736335754395, 0.10387086868286133, 0.11122536659240723, 0.10389971733093262, 0.11880850791931152, 0.10331320762634277, 0.10366368293762207, 0.11155867576599121, 0.10400152206420898, 0.10350441932678223, 0.10370993614196777, 0.10338807106018066, 0.10762453079223633, 0.10352134704589844, 0.10411238670349121, 0.10401439666748047, 0.10936260223388672, 0.10740303993225098, 0.10379815101623535, 0.10336756706237793, 0.10400271415710449, 0.10866451263427734, 0.10428881645202637, 0.10350441932678223, 0.1032872200012207, 0.10416865348815918, 0.10793066024780273, 0.10347294807434082, 0.10314130783081055, 0.10373091697692871, 0.10344099998474121, 0.11171603202819824, 0.10361385345458984, 0.10368990898132324, 0.10361170768737793, 0.10823512077331543, 0.10778498649597168, 0.10394859313964844, 0.10353326797485352, 0.10407471656799316, 0.12680292129516602, 0.10419869422912598, 0.10318374633789062, 0.10580158233642578, 0.14892053604125977, 0.18776655197143555, 0.16259121894836426, 0.10309290885925293, 0.10490202903747559, 0.11416769027709961, 0.10599613189697266, 0.10369610786437988, 0.11047625541687012, 0.21727418899536133, 0.12146210670471191, 0.10432887077331543, 0.10411906242370605, 0.10404467582702637, 0.1038825511932373, 0.10814666748046875, 0.10404849052429199, 0.10358905792236328, 0.10376572608947754, 0.10371136665344238, 0.11249828338623047, 0.10389208793640137, 0.10387969017028809, 0.10358071327209473, 0.10904908180236816, 0.1062169075012207, 0.10538554191589355, 0.11130547523498535, 0.10350513458251953, 0.10454654693603516, 0.10565733909606934, 0.10397958755493164, 0.1191256046295166, 0.10515785217285156, 0.10533738136291504, 0.10395264625549316, 0.11067938804626465, 0.10522913932800293, 0.1049504280090332, 0.10530233383178711, 0.11179637908935547, 0.10414576530456543, 0.10489296913146973, 0.11058640480041504, 0.10364627838134766, 0.1043241024017334, 0.10938525199890137, 0.10572242736816406, 0.10358715057373047, 0.10348248481750488, 0.11518216133117676, 0.10326433181762695, 0.1035919189453125, 0.10373210906982422, 0.10389161109924316, 0.11124944686889648, 0.10381960868835449, 0.11735820770263672, 0.10383987426757812, 0.1038823127746582, 0.10390424728393555, 0.11577010154724121, 0.10499119758605957, 0.10492348670959473, 0.10395073890686035, 0.10343241691589355, 0.10881924629211426, 0.10565185546875, 0.1044318675994873, 0.1038217544555664, 0.10452985763549805, 0.11030936241149902, 0.10638046264648438, 0.10498762130737305, 0.10496091842651367, 0.10441923141479492, 0.11180710792541504, 0.10521888732910156, 0.10354161262512207, 0.11166262626647949, 0.10513567924499512, 0.10379767417907715, 0.10536050796508789, 0.10377740859985352, 0.11818170547485352, 0.10358476638793945, 0.10426139831542969, 0.10630440711975098, 0.10664534568786621, 0.10877013206481934, 0.10395503044128418, 0.1056680679321289, 0.10455608367919922, 0.10400843620300293, 0.10852503776550293, 0.1039741039276123, 0.10617280006408691, 0.10719418525695801, 0.113494873046875, 0.10649538040161133, 0.10907864570617676, 0.10532212257385254, 0.1045689582824707, 0.11201119422912598, 0.1075751781463623, 0.10582399368286133, 0.10381102561950684, 0.10508251190185547, 0.11302590370178223, 0.10380077362060547, 0.10396695137023926, 0.10367894172668457, 0.1120612621307373, 0.10344076156616211, 0.10505962371826172, 0.10573816299438477, 0.10559940338134766, 0.10718226432800293, 0.10501742362976074, 0.10349488258361816, 0.10350584983825684, 0.11532306671142578, 0.10355472564697266, 0.1031954288482666, 0.10386848449707031, 0.10370230674743652, 0.10838437080383301, 0.10514044761657715, 0.10351967811584473, 0.10341215133666992, 0.10889267921447754, 0.10588192939758301, 0.10335326194763184, 0.10424637794494629, 0.10416984558105469, 0.11416244506835938, 0.11040091514587402, 0.1063849925994873, 0.10659170150756836, 0.10471200942993164, 0.10676741600036621, 0.11026954650878906, 0.10607028007507324, 0.10387825965881348, 0.10407447814941406, 0.10403752326965332, 0.11361098289489746, 0.10378360748291016, 0.10410642623901367, 0.10562825202941895, 0.11172723770141602, 0.10585331916809082, 0.10402059555053711, 0.10390591621398926, 0.10557985305786133, 0.10794401168823242, 0.10500860214233398, 0.10398674011230469, 0.11433148384094238, 0.10520815849304199, 0.10538220405578613, 0.10629391670227051, 0.10926413536071777, 0.10461258888244629, 0.10376715660095215, 0.10410189628601074, 0.10415887832641602, 0.10832738876342773, 0.10382199287414551, 0.10422039031982422, 0.10513567924499512, 0.10523509979248047, 0.11338114738464355, 0.10376358032226562, 0.10387706756591797, 0.10616850852966309, 0.11013269424438477, 0.10439634323120117, 0.10375118255615234, 0.10339522361755371, 0.10757756233215332, 0.10381460189819336, 0.10380005836486816, 0.11405014991760254, 0.10466885566711426, 0.10381913185119629, 0.1048123836517334, 0.10361695289611816, 0.10890626907348633, 0.1036379337310791, 0.10531949996948242, 0.10361528396606445, 0.10902571678161621, 0.10426139831542969, 0.1213843822479248, 0.10389590263366699, 0.10350489616394043, 0.11946535110473633, 0.10377907752990723, 0.10419797897338867, 0.11728882789611816, 0.10381436347961426, 0.1042780876159668, 0.10403728485107422, 0.11659359931945801, 0.11708831787109375, 0.10434317588806152, 0.10535812377929688, 0.11981081962585449, 0.10436773300170898, 0.10394096374511719, 0.12346434593200684, 0.11560511589050293, 0.10511374473571777, 0.10535502433776855, 0.10414624214172363, 0.11402726173400879, 0.1035463809967041, 0.12018799781799316, 0.10401391983032227, 0.10422754287719727, 0.10389113426208496, 0.12095212936401367, 0.10404491424560547, 0.10378503799438477, 0.10534048080444336, 0.10562872886657715, 0.13049912452697754, 0.10392403602600098, 0.10446357727050781, 0.10442328453063965, 0.11411666870117188, 0.11713480949401855, 0.10564589500427246, 0.10418295860290527, 0.10552263259887695, 0.1141972541809082, 0.10399818420410156, 0.10525918006896973, 0.11072230339050293, 0.10832834243774414, 0.10450458526611328, 0.1043386459350586, 0.11482763290405273, 0.10325789451599121, 0.10399794578552246, 0.11467695236206055, 0.10668349266052246, 0.10438060760498047, 0.1040048599243164, 0.10350370407104492, 0.11163973808288574, 0.1044163703918457, 0.10439705848693848, 0.10459327697753906, 0.11267352104187012, 0.10492300987243652, 0.10392260551452637, 0.10462498664855957, 0.11145257949829102, 0.10597944259643555, 0.1040201187133789, 0.11596155166625977, 0.10538411140441895, 0.1055300235748291, 0.12022829055786133, 0.10514450073242188, 0.10457563400268555, 0.1046297550201416, 0.10503697395324707, 0.1147925853729248, 0.10576200485229492, 0.10464096069335938, 0.10515522956848145, 0.10568547248840332, 0.12272357940673828, 0.10529541969299316, 0.1055147647857666, 0.10684514045715332, 0.10496902465820312, 0.10788512229919434, 0.10506057739257812, 0.10424280166625977, 0.10477852821350098, 0.10415911674499512, 0.11361241340637207, 0.1042182445526123, 0.10457730293273926, 0.10569643974304199, 0.11180543899536133, 0.10542130470275879, 0.10506582260131836, 0.10463404655456543, 0.10443425178527832, 0.10829710960388184, 0.10601544380187988, 0.10403823852539062, 0.1043548583984375, 0.1038973331451416, 0.11420631408691406, 0.10438847541809082, 0.1043848991394043, 0.10391092300415039, 0.11489105224609375, 0.11235952377319336, 0.10613179206848145, 0.10381364822387695, 0.10558605194091797, 0.10538196563720703, 0.1124875545501709, 0.1189577579498291, 0.10501432418823242, 0.10400795936584473, 0.11423635482788086, 0.10555839538574219, 0.11807703971862793, 0.10508608818054199, 0.1042487621307373, 0.11733508110046387, 0.10664987564086914, 0.10459208488464355, 0.10492134094238281, 0.10386943817138672, 0.11942005157470703, 0.1056978702545166, 0.10586428642272949, 0.10594606399536133, 0.10988473892211914, 0.10553622245788574, 0.10523200035095215, 0.1044924259185791, 0.10616469383239746, 0.10788178443908691, 0.10465455055236816, 0.10424613952636719, 0.10409140586853027, 0.10457086563110352, 0.11214470863342285, 0.10475778579711914, 0.10640358924865723, 0.10672569274902344, 0.11304545402526855, 0.10788440704345703, 0.10648441314697266, 0.10680460929870605, 0.10630965232849121, 0.10886144638061523, 0.10677099227905273, 0.10547494888305664, 0.10786104202270508, 0.11349105834960938, 0.11266589164733887, 0.10784053802490234, 0.10560417175292969, 0.10434699058532715, 0.10569143295288086, 0.10761523246765137, 0.10402297973632812, 0.10436439514160156, 0.12005615234375, 0.10522794723510742, 0.10490703582763672, 0.11747860908508301, 0.10475873947143555, 0.10463380813598633, 0.10582923889160156, 0.11179947853088379, 0.10571432113647461, 0.11599040031433105, 0.1111609935760498, 0.11083245277404785, 0.11427640914916992, 0.11227583885192871, 0.12032151222229004, 0.11228561401367188, 0.10470056533813477, 0.10414338111877441, 0.10400557518005371, 0.10430097579956055, 0.11732649803161621, 0.1059732437133789, 0.10428237915039062, 0.11368894577026367, 0.11164522171020508, 0.11446189880371094, 0.11158871650695801, 0.10477995872497559, 0.10527729988098145, 0.10389471054077148, 0.10853862762451172, 0.10499906539916992, 0.10584712028503418, 0.10851335525512695, 0.10598206520080566, 0.11881184577941895, 0.1059122085571289, 0.10396122932434082, 0.11225771903991699, 0.10402393341064453, 0.10653233528137207, 0.10601305961608887, 0.11015701293945312, 0.10473418235778809, 0.10593295097351074, 0.10570693016052246, 0.11173343658447266, 0.10649394989013672, 0.11084222793579102, 0.10562348365783691, 0.10682535171508789, 0.10401654243469238, 0.10967779159545898, 0.1162569522857666, 0.10443425178527832, 0.1045684814453125, 0.10466980934143066, 0.10586428642272949, 0.11209940910339355, 0.11014032363891602, 0.10677123069763184, 0.10593676567077637, 0.10648655891418457, 0.10988163948059082, 0.10495781898498535, 0.11315345764160156, 0.114044189453125, 0.10593509674072266, 0.1056058406829834, 0.10432648658752441, 0.11010527610778809, 0.10536789894104004, 0.11269402503967285, 0.11233973503112793, 0.1210012435913086, 0.11793875694274902, 0.11894488334655762, 0.1330561637878418, 0.11866641044616699, 0.10578465461730957, 0.10616660118103027, 0.11981725692749023, 0.10492062568664551, 0.10400724411010742, 0.1054844856262207, 0.10500359535217285, 0.10927391052246094, 0.10539913177490234, 0.1042182445526123, 0.10434293746948242, 0.11625456809997559, 0.11024308204650879, 0.10465121269226074, 0.10418200492858887, 0.11875271797180176, 0.1353464126586914, 0.10607194900512695, 0.10653519630432129, 0.12119269371032715, 0.10644245147705078, 0.10777997970581055, 0.10507869720458984, 0.1049501895904541, 0.10924434661865234, 0.10400724411010742, 0.10433030128479004, 0.10527539253234863, 0.10393810272216797, 0.1076667308807373, 0.10432100296020508, 0.11244893074035645, 0.10480570793151855, 0.10474276542663574, 0.1049809455871582, 0.10855841636657715, 0.10434365272521973, 0.11192655563354492, 0.10498785972595215, 0.10718631744384766, 0.10648536682128906, 0.11437058448791504, 0.10751557350158691, 0.10643720626831055, 0.10435986518859863, 0.11811542510986328, 0.11096310615539551, 0.10549283027648926, 0.10563373565673828, 0.10432672500610352, 0.10570549964904785, 0.11230349540710449, 0.10416841506958008, 0.11015105247497559, 0.10466361045837402, 0.10378575325012207, 0.1089935302734375, 0.10611152648925781, 0.1054830551147461, 0.11404943466186523, 0.1112356185913086, 0.11197733879089355, 0.11623334884643555, 0.11165142059326172, 0.12244248390197754, 0.1073770523071289, 0.10450148582458496, 0.10429024696350098, 0.10440230369567871, 0.10781574249267578, 0.1142737865447998, 0.10575532913208008, 0.10482549667358398, 0.11105084419250488, 0.10547137260437012, 0.11002635955810547, 0.12189579010009766, 0.10476374626159668, 0.103973388671875, 0.10482001304626465, 0.10426735877990723, 0.10870933532714844, 0.10687518119812012, 0.10915184020996094, 0.10671138763427734, 0.1041414737701416, 0.11370515823364258, 0.1147146224975586, 0.10546374320983887, 0.10465335845947266, 0.10953378677368164, 0.10738420486450195, 0.10536623001098633, 0.10683846473693848, 0.10550308227539062, 0.1151280403137207, 0.10475802421569824, 0.10401320457458496, 0.10618996620178223, 0.11009621620178223, 0.10500478744506836, 0.11789083480834961, 0.10477328300476074, 0.10455107688903809, 0.12128639221191406, 0.10566258430480957, 0.10464835166931152, 0.1070711612701416, 0.10596323013305664, 0.1322154998779297, 0.10705018043518066, 0.10751771926879883, 0.10673713684082031, 0.1198732852935791, 0.11404895782470703, 0.10413646697998047, 0.10557389259338379, 0.1064310073852539, 0.10482406616210938, 0.10989642143249512, 0.10397815704345703, 0.13362741470336914, 0.11555695533752441, 0.1197669506072998, 0.12131333351135254, 0.13215136528015137, 0.12691283226013184, 0.12063455581665039, 0.10604429244995117, 0.10394787788391113, 0.10452008247375488, 0.10429143905639648, 0.11098027229309082, 0.10436320304870605, 0.10486412048339844, 0.10387945175170898, 0.11104583740234375, 0.10564041137695312, 0.10575437545776367, 0.10429000854492188, 0.11582159996032715, 0.10426855087280273, 0.10526752471923828, 0.11303162574768066, 0.10584878921508789, 0.10406756401062012, 0.10560989379882812, 0.12052726745605469, 0.10593461990356445, 0.10432767868041992, 0.10419869422912598, 0.10437607765197754, 0.11882996559143066, 0.10391879081726074, 0.10567736625671387, 0.10880684852600098, 0.11058259010314941, 0.1071779727935791, 0.10733938217163086, 0.10640430450439453, 0.10408210754394531, 0.1092381477355957, 0.10575723648071289, 0.10428571701049805, 0.10576486587524414, 0.10419082641601562, 0.11329388618469238, 0.11847972869873047, 0.10565972328186035, 0.10537075996398926, 0.10582590103149414, 0.1054680347442627, 0.1159524917602539, 0.11662864685058594, 0.10736775398254395, 0.1045224666595459, 0.11403226852416992, 0.11696934700012207, 0.10629749298095703, 0.10474205017089844, 0.10481071472167969, 0.1059720516204834, 0.10887670516967773, 0.10576558113098145, 0.10557937622070312, 0.10746407508850098, 0.11065030097961426, 0.11821269989013672, 0.1058499813079834, 0.10555100440979004, 0.10781574249267578, 0.1170506477355957, 0.10433769226074219, 0.10576605796813965, 0.10505270957946777, 0.10426020622253418, 0.10959553718566895, 0.10576486587524414, 0.1060483455657959, 0.10572195053100586, 0.11368894577026367, 0.10722494125366211, 0.10488557815551758, 0.10476946830749512, 0.10437774658203125, 0.11020898818969727, 0.10595965385437012, 0.10581302642822266, 0.11209416389465332, 0.1056966781616211, 0.10447573661804199, 0.1044309139251709, 0.11336898803710938, 0.10489654541015625, 0.1052405834197998, 0.10629630088806152, 0.10511255264282227, 0.11217164993286133, 0.10449838638305664, 0.10500669479370117, 0.10559582710266113, 0.11883330345153809, 0.10499238967895508, 0.10508251190185547, 0.10411524772644043, 0.10427188873291016, 0.11897945404052734, 0.10651659965515137, 0.1059424877166748, 0.1154322624206543, 0.10690593719482422, 0.10523271560668945, 0.10600829124450684, 0.10636544227600098, 0.10908937454223633, 0.10628366470336914, 0.10606527328491211, 0.1220865249633789, 0.10586166381835938, 0.1049652099609375, 0.10575652122497559, 0.1149451732635498, 0.10630321502685547, 0.10471987724304199, 0.1039891242980957, 0.1050114631652832, 0.11041808128356934, 0.10527729988098145, 0.1051783561706543, 0.10506892204284668, 0.10547041893005371, 0.11770820617675781, 0.10408139228820801, 0.10608315467834473, 0.10481691360473633, 0.11706995964050293, 0.10575151443481445, 0.10433626174926758, 0.10413408279418945, 0.10619354248046875, 0.11113214492797852, 0.10878419876098633, 0.12417936325073242, 0.1184079647064209, 0.10796737670898438, 0.10441851615905762, 0.12031674385070801, 0.10664892196655273, 0.10591340065002441, 0.10498332977294922, 0.10480475425720215, 0.11469221115112305, 0.11839413642883301, 0.10511946678161621, 0.10445952415466309, 0.10405468940734863, 0.11779046058654785, 0.1043386459350586, 0.10377168655395508, 0.10613822937011719, 0.11882424354553223, 0.1047983169555664, 0.10548281669616699, 0.10541653633117676, 0.10416126251220703, 0.11019754409790039, 0.11165213584899902, 0.10670351982116699, 0.10621285438537598, 0.10528445243835449, 0.10509443283081055, 0.11415386199951172, 0.10644721984863281, 0.10681843757629395, 0.10407376289367676, 0.1045832633972168, 0.11052298545837402, 0.10452103614807129, 0.10530805587768555, 0.10535216331481934, 0.11101293563842773, 0.10561466217041016, 0.106048583984375, 0.10595059394836426, 0.10485196113586426, 0.11302518844604492, 0.10473132133483887, 0.10951542854309082, 0.10725188255310059, 0.10655331611633301, 0.11260032653808594, 0.10588836669921875, 0.10785984992980957, 0.10600543022155762, 0.10942816734313965, 0.10468697547912598, 0.10500264167785645, 0.10583138465881348, 0.10718107223510742, 0.11231374740600586, 0.10866451263427734, 0.10672712326049805, 0.10555458068847656, 0.10549473762512207, 0.10961008071899414, 0.1053314208984375, 0.10587334632873535, 0.10527420043945312, 0.10936141014099121, 0.10750007629394531, 0.10549259185791016, 0.10589003562927246, 0.10547876358032227, 0.11289167404174805, 0.10523676872253418, 0.10604047775268555, 0.10614991188049316, 0.10592341423034668, 0.1129598617553711, 0.10789084434509277, 0.1067202091217041, 0.10583114624023438, 0.11202001571655273, 0.10773754119873047, 0.10566377639770508, 0.10484457015991211, 0.1075129508972168, 0.11509084701538086, 0.10524272918701172, 0.10524559020996094, 0.10582256317138672, 0.10712623596191406, 0.11168575286865234, 0.10762214660644531, 0.10601615905761719, 0.10731959342956543, 0.11213064193725586, 0.10645389556884766, 0.1050271987915039, 0.10639238357543945, 0.10574793815612793, 0.11612272262573242, 0.10521793365478516, 0.10453391075134277, 0.10849881172180176, 0.10651040077209473, 0.11036038398742676, 0.1056358814239502, 0.10470795631408691, 0.10469675064086914, 0.10994362831115723, 0.10500884056091309, 0.10774421691894531, 0.10452008247375488, 0.10633993148803711, 0.11501479148864746, 0.10760331153869629, 0.10517358779907227, 0.1058356761932373, 0.10599660873413086, 0.11323189735412598, 0.10830569267272949, 0.10635757446289062, 0.10660314559936523, 0.11890196800231934, 0.11696267127990723, 0.10568618774414062, 0.10526227951049805, 0.10596585273742676, 0.10661864280700684, 0.10953140258789062, 0.11119985580444336, 0.10560202598571777, 0.10585236549377441, 0.10994791984558105, 0.10608434677124023, 0.11920762062072754, 0.11188602447509766, 0.12009763717651367, 0.10599327087402344, 0.1318066120147705, 0.1321086883544922, 0.13318943977355957, 0.11731171607971191, 0.10539984703063965, 0.10587573051452637, 0.10524702072143555, 0.11751127243041992, 0.1184854507446289, 0.1055147647857666, 0.10510444641113281, 0.1044626235961914, 0.11368012428283691, 0.11193728446960449, 0.10432577133178711, 0.1049354076385498, 0.10410499572753906, 0.10962939262390137, 0.10569596290588379, 0.10403108596801758, 0.10436582565307617, 0.1049184799194336, 0.10890531539916992, 0.10667276382446289, 0.10399413108825684, 0.10587096214294434, 0.10600686073303223, 0.11393260955810547, 0.10600662231445312, 0.10457420349121094, 0.10575079917907715, 0.11067342758178711, 0.10458588600158691, 0.10489416122436523, 0.10646843910217285, 0.10522985458374023, 0.10935664176940918, 0.10538482666015625, 0.10622429847717285, 0.10470414161682129, 0.10432291030883789, 0.11232829093933105, 0.10501313209533691, 0.10491657257080078, 0.10554051399230957, 0.11157536506652832, 0.10609149932861328, 0.10596966743469238, 0.10459375381469727, 0.1061704158782959, 0.10916471481323242, 0.12151312828063965, 0.10378789901733398, 0.1052703857421875, 0.10418176651000977, 0.1055760383605957, 0.11679792404174805, 0.10393476486206055, 0.10435795783996582, 0.10551595687866211, 0.10636138916015625, 0.10843658447265625, 0.10512208938598633, 0.1046133041381836, 0.10602736473083496, 0.11324644088745117, 0.10616493225097656, 0.10575056076049805, 0.10634779930114746, 0.10613203048706055, 0.11276721954345703, 0.1059722900390625, 0.10596513748168945, 0.10645413398742676, 0.10667061805725098, 0.10705184936523438, 0.11511468887329102, 0.10612010955810547, 0.10809087753295898, 0.10791754722595215, 0.1081380844116211, 0.10953569412231445, 0.1107790470123291, 0.11676740646362305, 0.1049196720123291, 0.10383009910583496, 0.10443401336669922, 0.10559487342834473, 0.10954451560974121, 0.10891556739807129, 0.10415911674499512, 0.10532569885253906, 0.10494709014892578, 0.10454916954040527, 0.11211609840393066, 0.1041419506072998, 0.10468101501464844, 0.11188888549804688, 0.1096494197845459, 0.11531615257263184, 0.11153745651245117, 0.11889314651489258, 0.11534476280212402, 0.10955119132995605, 0.12398171424865723, 0.10822844505310059, 0.10574603080749512, 0.10480904579162598, 0.10428142547607422, 0.11204385757446289, 0.1046607494354248, 0.11188530921936035, 0.10527157783508301, 0.1038365364074707, 0.10500693321228027, 0.10751581192016602, 0.10506844520568848, 0.11372542381286621, 0.11150574684143066, 0.10963177680969238, 0.11933708190917969, 0.10886192321777344, 0.11303567886352539, 0.11027240753173828, 0.1097722053527832, 0.11384725570678711, 0.11197376251220703, 0.10881567001342773, 0.11995792388916016, 0.10715818405151367, 0.11540889739990234, 0.11304664611816406, 0.10978388786315918, 0.11348485946655273, 0.1103203296661377, 0.11191678047180176, 0.1138620376586914, 0.11063361167907715, 0.10966205596923828, 0.1145322322845459, 0.11079549789428711, 0.11104130744934082, 0.11349296569824219, 0.11200141906738281, 0.11006593704223633, 0.11296677589416504, 0.14217448234558105, 0.14243841171264648, 0.11818957328796387, 0.10964083671569824, 0.10441756248474121, 0.11384892463684082, 0.10484933853149414, 0.10428738594055176, 0.10558128356933594, 0.11093258857727051, 0.10463070869445801, 0.1106557846069336, 0.10662984848022461, 0.10386085510253906, 0.10552501678466797, 0.10701107978820801, 0.11700630187988281, 0.1166071891784668, 0.12008547782897949, 0.11999654769897461, 0.1041560173034668, 0.12630844116210938, 0.103912353515625, 0.11911368370056152, 0.11841011047363281, 0.10612249374389648, 0.10490894317626953, 0.1056208610534668, 0.11071157455444336, 0.10535192489624023, 0.10573196411132812, 0.10503625869750977, 0.11498427391052246, 0.1054234504699707, 0.105804443359375, 0.10500049591064453, 0.10443401336669922, 0.1112205982208252, 0.10830950736999512, 0.12305712699890137, 0.10903692245483398, 0.10778975486755371, 0.11116361618041992, 0.10622239112854004, 0.10476469993591309, 0.1191864013671875, 0.1060786247253418, 0.10669994354248047, 0.10512113571166992, 0.10697126388549805, 0.12195372581481934, 0.10649418830871582, 0.10868382453918457, 0.10434865951538086, 0.11225056648254395, 0.10672807693481445, 0.11496376991271973, 0.11604881286621094, 0.1041417121887207, 0.10483288764953613, 0.10502958297729492, 0.11124610900878906, 0.10735392570495605, 0.10415267944335938, 0.10842347145080566, 0.1077888011932373, 0.12354540824890137, 0.11250734329223633, 0.10486078262329102, 0.10612726211547852, 0.10501289367675781, 0.10546994209289551, 0.11319780349731445, 0.1301734447479248, 0.1324164867401123, 0.14101457595825195, 0.10778546333312988, 0.11205697059631348, 0.1039886474609375, 0.1058034896850586, 0.10420775413513184, 0.11449027061462402, 0.10545825958251953, 0.10568070411682129, 0.10545134544372559, 0.10440373420715332, 0.12106561660766602, 0.10709929466247559, 0.1064765453338623, 0.1060800552368164, 0.1046748161315918, 0.11471009254455566, 0.10600972175598145, 0.11046934127807617, 0.12053966522216797, 0.11411333084106445, 0.11147904396057129, 0.10628938674926758, 0.10555601119995117, 0.10568857192993164, 0.11182188987731934, 0.10534286499023438, 0.1126554012298584, 0.11937856674194336, 0.10613250732421875, 0.10650062561035156, 0.10709500312805176, 0.11224937438964844, 0.10556530952453613, 0.10667729377746582, 0.13519072532653809, 0.10623955726623535, 0.10492992401123047, 0.1066749095916748, 0.10701775550842285, 0.1150205135345459, 0.10716962814331055, 0.1064453125, 0.10942292213439941, 0.11231851577758789, 0.1088566780090332, 0.10901188850402832, 0.1061563491821289, 0.10699295997619629, 0.12175822257995605, 0.10726189613342285, 0.10586309432983398, 0.11544227600097656, 0.10719752311706543, 0.10646557807922363, 0.10597109794616699, 0.11335110664367676, 0.1116480827331543, 0.1206216812133789, 0.10731887817382812, 0.11701846122741699, 0.12539267539978027, 0.10777473449707031, 0.12006163597106934, 0.11736440658569336, 0.1140592098236084, 0.11239385604858398, 0.11779260635375977, 0.11147713661193848, 0.11129927635192871, 0.12055039405822754, 0.10958027839660645, 0.11498045921325684, 0.10645794868469238, 0.11203336715698242, 0.1045081615447998, 0.10645484924316406, 0.10627913475036621, 0.106292724609375, 0.11930227279663086, 0.12400388717651367, 0.1301431655883789, 0.10701608657836914, 0.118896484375, 0.11499404907226562, 0.10537147521972656, 0.11962676048278809, 0.10648918151855469, 0.11757683753967285, 0.10531115531921387, 0.1045236587524414, 0.11113643646240234, 0.10620856285095215, 0.10547971725463867, 0.10690188407897949, 0.11271309852600098, 0.11362862586975098, 0.12771129608154297, 0.12519097328186035, 0.12360978126525879, 0.12402963638305664, 0.12287068367004395, 0.11374425888061523, 0.10600948333740234, 0.11077117919921875, 0.10594964027404785, 0.10567522048950195, 0.10454344749450684, 0.10929179191589355, 0.12251543998718262, 0.1069021224975586, 0.10421347618103027, 0.10609579086303711, 0.10429191589355469, 0.11627602577209473, 0.10548925399780273, 0.11451363563537598, 0.10578727722167969, 0.1047365665435791, 0.10441207885742188, 0.11366510391235352, 0.10411524772644043, 0.10580253601074219, 0.1054985523223877, 0.10602760314941406, 0.11197447776794434, 0.1048429012298584, 0.11159396171569824, 0.1057889461517334, 0.10600686073303223, 0.10431981086730957, 0.11242794990539551, 0.10577774047851562, 0.10475540161132812, 0.10468029975891113, 0.1042640209197998, 0.11110758781433105, 0.10545635223388672, 0.10627222061157227, 0.10423827171325684, 0.10450863838195801, 0.10775375366210938, 0.10605573654174805, 0.10509490966796875, 0.10532760620117188, 0.12329769134521484, 0.10939812660217285, 0.10790157318115234, 0.10917878150939941, 0.1084890365600586, 0.11060905456542969, 0.10719013214111328, 0.10562562942504883, 0.1058664321899414, 0.10468411445617676, 0.14473247528076172, 0.10617947578430176, 0.10420775413513184, 0.1048285961151123, 0.10483884811401367, 0.11985015869140625, 0.10407900810241699, 0.11802959442138672, 0.10631108283996582, 0.10425448417663574, 0.1041867733001709, 0.10570693016052246, 0.1145467758178711, 0.10414600372314453, 0.10648441314697266, 0.10628271102905273, 0.11343622207641602, 0.10633587837219238, 0.10749340057373047, 0.10861730575561523, 0.10776138305664062, 0.11227774620056152, 0.10719776153564453, 0.10553169250488281, 0.10535526275634766, 0.10749697685241699, 0.1096658706665039, 0.11793923377990723, 0.11132526397705078, 0.10602664947509766, 0.10997772216796875, 0.10729217529296875, 0.10887765884399414, 0.10633134841918945, 0.10574078559875488, 0.10647797584533691, 0.10620927810668945, 0.1135246753692627, 0.10444164276123047, 0.11195039749145508, 0.10440778732299805, 0.1058356761932373, 0.10621237754821777, 0.1058645248413086, 0.1116645336151123, 0.10553145408630371, 0.10573649406433105, 0.10614943504333496, 0.11534309387207031, 0.10857748985290527, 0.10653829574584961, 0.10754895210266113, 0.10552787780761719, 0.11048722267150879, 0.10457706451416016, 0.10482311248779297, 0.11087203025817871, 0.11573004722595215, 0.10556697845458984, 0.10683560371398926, 0.10545158386230469, 0.12016940116882324, 0.11583542823791504, 0.10631346702575684, 0.10651969909667969, 0.10484766960144043, 0.11618828773498535, 0.10399889945983887, 0.10587239265441895, 0.10600161552429199, 0.10687804222106934, 0.1135401725769043, 0.10529422760009766, 0.10817599296569824, 0.10751962661743164, 0.11280512809753418, 0.10822725296020508, 0.10708212852478027, 0.10593390464782715, 0.1062479019165039, 0.1088259220123291, 0.10622191429138184, 0.10565423965454102, 0.10504674911499023, 0.13032817840576172, 0.10821247100830078, 0.10609722137451172, 0.10442161560058594, 0.10622239112854004, 0.10897445678710938, 0.10563802719116211, 0.1045839786529541, 0.1060938835144043, 0.10650825500488281, 0.11212682723999023, 0.1057131290435791, 0.11688542366027832, 0.10553312301635742, 0.11657071113586426, 0.11322927474975586, 0.11213445663452148, 0.1074059009552002, 0.11569070816040039, 0.10523772239685059, 0.11413240432739258, 0.11742377281188965, 0.11302757263183594, 0.10582852363586426, 0.10427999496459961, 0.10517430305480957, 0.11186671257019043, 0.11648035049438477, 0.11292123794555664, 0.10585474967956543, 0.1101524829864502, 0.11500716209411621, 0.10590004920959473, 0.11141753196716309, 0.11530351638793945, 0.10551285743713379, 0.12150311470031738, 0.10455989837646484, 0.10406684875488281, 0.11818385124206543, 0.10506892204284668, 0.11469697952270508, 0.10683393478393555, 0.11778092384338379, 0.10638022422790527, 0.1056978702545166, 0.10633420944213867, 0.10727453231811523, 0.10953617095947266, 0.10603094100952148, 0.10539889335632324, 0.10610508918762207, 0.10926580429077148, 0.10442137718200684, 0.10626649856567383, 0.1055915355682373, 0.1050872802734375, 0.11487412452697754, 0.10468721389770508, 0.10560369491577148, 0.10677480697631836, 0.10726737976074219, 0.10898876190185547, 0.10539650917053223, 0.10605144500732422, 0.10463953018188477, 0.1093442440032959, 0.11308073997497559, 0.10486507415771484, 0.10612177848815918, 0.1041402816772461, 0.11092829704284668, 0.10487651824951172, 0.11157894134521484, 0.10600900650024414, 0.10434365272521973, 0.10665106773376465, 0.10581040382385254, 0.11358928680419922, 0.10536980628967285, 0.1043851375579834, 0.1057131290435791, 0.10665440559387207, 0.11148643493652344, 0.10437798500061035, 0.1101388931274414, 0.10549402236938477, 0.11188602447509766, 0.10723090171813965, 0.10442471504211426, 0.10453319549560547, 0.10524225234985352, 0.11662936210632324, 0.10683155059814453, 0.11364626884460449, 0.11521124839782715, 0.105499267578125, 0.10618185997009277, 0.10422086715698242, 0.10595297813415527, 0.11340117454528809, 0.11961126327514648, 0.10754752159118652, 0.10413002967834473, 0.10606074333190918, 0.10619640350341797, 0.11357355117797852, 0.10657501220703125, 0.10702729225158691, 0.10467839241027832, 0.10472750663757324, 0.11277580261230469, 0.10544681549072266, 0.10588693618774414, 0.10467338562011719, 0.11168241500854492, 0.10561060905456543, 0.10617780685424805, 0.10521650314331055, 0.10461211204528809, 0.10979962348937988, 0.1051948070526123, 0.10537886619567871, 0.1061851978302002, 0.1065526008605957, 0.1135709285736084, 0.10604262351989746, 0.1058046817779541, 0.10648560523986816, 0.10922741889953613, 0.10608983039855957, 0.1075599193572998, 0.10834121704101562, 0.10673069953918457, 0.10933184623718262, 0.10691022872924805, 0.10623335838317871, 0.10653018951416016, 0.10615134239196777, 0.11352276802062988, 0.12624478340148926, 0.10645532608032227, 0.10605430603027344, 0.10548758506774902, 0.10558462142944336, 0.11277437210083008, 0.10578250885009766, 0.10520458221435547, 0.10570716857910156, 0.1058802604675293, 0.1104440689086914, 0.11162447929382324, 0.10548686981201172, 0.11487412452697754, 0.10566473007202148, 0.10617208480834961, 0.11123847961425781, 0.1102304458618164, 0.11515069007873535, 0.11258435249328613, 0.11104679107666016, 0.11630964279174805, 0.11283349990844727, 0.11439633369445801, 0.12067747116088867, 0.12075495719909668, 0.11544585227966309, 0.10534381866455078, 0.12170648574829102, 0.10625100135803223, 0.10556554794311523, 0.1046595573425293, 0.10714268684387207, 0.12251710891723633, 0.12877821922302246, 0.12460923194885254, 0.11307144165039062, 0.11057448387145996, 0.1164093017578125, 0.1140286922454834, 0.11973214149475098, 0.10566449165344238, 0.10636091232299805, 0.10560727119445801, 0.10588598251342773, 0.11092758178710938, 0.10575127601623535, 0.10593414306640625, 0.1077725887298584, 0.10768985748291016, 0.10880494117736816, 0.10579633712768555, 0.10731768608093262, 0.10690927505493164, 0.11345577239990234, 0.10560393333435059, 0.10572218894958496, 0.10587286949157715, 0.10537338256835938, 0.1119840145111084, 0.10491347312927246, 0.10448408126831055, 0.10584545135498047, 0.11653566360473633, 0.10550689697265625, 0.10427212715148926, 0.10774993896484375, 0.11772370338439941, 0.10575699806213379, 0.10625457763671875, 0.10609984397888184, 0.11483144760131836, 0.10413098335266113, 0.10544705390930176, 0.10721397399902344, 0.10575747489929199, 0.10857391357421875, 0.10586977005004883, 0.10632920265197754, 0.10735058784484863, 0.11508512496948242, 0.1074376106262207, 0.10599470138549805, 0.10505032539367676, 0.10582661628723145, 0.10991811752319336, 0.10454726219177246, 0.1113739013671875, 0.10444211959838867, 0.10471844673156738, 0.1227717399597168, 0.10465574264526367, 0.10591912269592285, 0.10463309288024902, 0.11651206016540527, 0.10565996170043945, 0.10416555404663086, 0.10577726364135742, 0.10454106330871582, 0.10881567001342773, 0.12090253829956055, 0.10440754890441895, 0.12067198753356934, 0.10596609115600586, 0.10465764999389648, 0.10458540916442871, 0.11376142501831055, 0.10501885414123535, 0.10587406158447266, 0.10409903526306152, 0.10494232177734375, 0.11604809761047363, 0.11333084106445312, 0.10601043701171875, 0.11072897911071777, 0.11911630630493164, 0.10983681678771973, 0.10626101493835449, 0.1038811206817627, 0.10564470291137695, 0.10897469520568848, 0.10527324676513672, 0.11495089530944824, 0.1054239273071289, 0.1054685115814209, 0.1073758602142334, 0.10601973533630371, 0.1108558177947998, 0.1048588752746582, 0.10552024841308594, 0.11922097206115723, 0.10490155220031738, 0.10442709922790527, 0.10637617111206055, 0.12096261978149414, 0.10429263114929199, 0.10998845100402832, 0.11702466011047363, 0.10581135749816895, 0.10511517524719238, 0.10567021369934082, 0.11243438720703125, 0.10513877868652344, 0.10576295852661133, 0.10505127906799316, 0.10494160652160645, 0.11301493644714355, 0.10568809509277344, 0.10514020919799805, 0.10451579093933105, 0.10576510429382324, 0.10962224006652832, 0.10579538345336914, 0.11179256439208984, 0.10670089721679688, 0.1052861213684082, 0.1227724552154541, 0.10571098327636719, 0.10597109794616699, 0.11912226676940918, 0.10389137268066406, 0.10484027862548828, 0.10545134544372559, 0.11430954933166504, 0.10705018043518066, 0.1053309440612793, 0.10494351387023926, 0.10471606254577637, 0.11431121826171875, 0.10596537590026855, 0.10473847389221191, 0.10413789749145508, 0.10444045066833496, 0.10983514785766602, 0.10426807403564453, 0.10545206069946289, 0.10459494590759277, 0.11098980903625488, 0.10562348365783691, 0.1159360408782959, 0.10573363304138184, 0.10418581962585449, 0.10449504852294922, 0.1040658950805664, 0.11247038841247559, 0.11189746856689453, 0.12278962135314941, 0.10560202598571777, 0.10422515869140625, 0.1059107780456543, 0.10534119606018066, 0.13297367095947266, 0.10438704490661621, 0.10565638542175293, 0.12335896492004395, 0.10466861724853516, 0.1049489974975586, 0.10452461242675781, 0.10616540908813477, 0.11542129516601562, 0.10496330261230469, 0.10556244850158691, 0.1058199405670166, 0.12036275863647461, 0.10551953315734863, 0.12214398384094238, 0.10733318328857422, 0.10612368583679199, 0.10575699806213379, 0.12064194679260254, 0.10545039176940918, 0.10448288917541504, 0.10756826400756836, 0.10989022254943848, 0.10582661628723145, 0.10442757606506348, 0.10544991493225098, 0.10669827461242676, 0.1145775318145752, 0.11435770988464355, 0.10559678077697754, 0.10429596900939941, 0.10467958450317383, 0.10797810554504395, 0.11235713958740234, 0.10415005683898926, 0.10572075843811035, 0.10553193092346191, 0.10432100296020508, 0.11063432693481445, 0.10536599159240723, 0.11047887802124023, 0.10533547401428223, 0.10558843612670898, 0.10440349578857422, 0.10427618026733398, 0.11258530616760254, 0.10546517372131348, 0.10516142845153809, 0.10425376892089844, 0.11110854148864746, 0.10573434829711914, 0.10411214828491211, 0.10517430305480957, 0.1053318977355957, 0.1104276180267334, 0.10568356513977051, 0.10511469841003418, 0.10522055625915527, 0.10602569580078125, 0.11362934112548828, 0.10585522651672363, 0.10536861419677734, 0.10432147979736328, 0.10764789581298828, 0.1079871654510498, 0.10518264770507812, 0.10459685325622559, 0.10399961471557617, 0.1096043586730957, 0.10531735420227051, 0.10500335693359375, 0.10570335388183594, 0.10530495643615723, 0.11262750625610352, 0.10449051856994629, 0.10448670387268066, 0.10485696792602539, 0.10570955276489258, 0.1095132827758789, 0.10559916496276855, 0.10544109344482422, 0.1042633056640625, 0.12965130805969238, 0.11311769485473633, 0.1203615665435791, 0.10575199127197266, 0.1200401782989502, 0.10418534278869629, 0.10441756248474121, 0.11795353889465332, 0.11589407920837402, 0.1056358814239502, 0.10431194305419922, 0.10461592674255371, 0.11832642555236816, 0.10505867004394531, 0.10450267791748047, 0.12094736099243164, 0.10547971725463867, 0.10886597633361816, 0.11000347137451172, 0.10827183723449707, 0.10560989379882812, 0.10413122177124023, 0.10913515090942383, 0.11529827117919922, 0.10541653633117676, 0.10426044464111328, 0.12049031257629395, 0.1063241958618164, 0.10577678680419922, 0.11794233322143555, 0.10603785514831543, 0.1056203842163086, 0.10560798645019531, 0.10508894920349121, 0.11640763282775879, 0.10594987869262695, 0.10521221160888672, 0.10553789138793945, 0.10463380813598633, 0.12119007110595703, 0.10451316833496094, 0.10420608520507812, 0.1047067642211914, 0.11773300170898438, 0.10793948173522949, 0.10471391677856445, 0.10624885559082031, 0.10604381561279297, 0.11236715316772461, 0.10549449920654297, 0.10546469688415527, 0.10543107986450195, 0.10574865341186523, 0.10978817939758301, 0.1054680347442627, 0.10479569435119629, 0.1054539680480957, 0.11466717720031738, 0.10525178909301758, 0.10483694076538086, 0.10392355918884277, 0.10532975196838379, 0.10976076126098633, 0.10587620735168457, 0.10494089126586914, 0.10591650009155273, 0.10560345649719238, 0.10981392860412598, 0.10555624961853027, 0.10393500328063965, 0.10485124588012695, 0.10513925552368164, 0.11373448371887207, 0.10441422462463379, 0.10573720932006836, 0.10590553283691406, 0.1099691390991211, 0.10455322265625, 0.10703110694885254, 0.10604524612426758, 0.10608124732971191, 0.10868668556213379, 0.1054375171661377, 0.10741043090820312, 0.106658935546875, 0.1065835952758789, 0.1121211051940918, 0.10469341278076172, 0.10443615913391113, 0.10533738136291504, 0.11085653305053711, 0.10478758811950684, 0.10530328750610352, 0.10582995414733887, 0.10442090034484863, 0.1085500717163086, 0.10538887977600098, 0.10486650466918945, 0.10443782806396484, 0.10468554496765137, 0.11289215087890625, 0.10466146469116211, 0.10514402389526367, 0.10586237907409668, 0.11005234718322754, 0.10572004318237305, 0.10498380661010742, 0.10569119453430176, 0.1045064926147461, 0.11087822914123535, 0.10432577133178711, 0.10544896125793457, 0.10597634315490723, 0.10578656196594238, 0.11379384994506836, 0.10433220863342285, 0.10445928573608398, 0.10573887825012207, 0.11330628395080566, 0.10580658912658691, 0.10568404197692871, 0.10517120361328125, 0.10554718971252441, 0.11007571220397949, 0.10439538955688477, 0.10660982131958008, 0.1042640209197998, 0.1052846908569336, 0.11437511444091797, 0.10533356666564941, 0.10455727577209473, 0.10567069053649902, 0.10694098472595215, 0.10754656791687012, 0.10462594032287598, 0.10439634323120117, 0.10438060760498047, 0.10446548461914062, 0.10502266883850098, 0.10646510124206543, 0.10581231117248535, 0.10410737991333008, 0.105377197265625, 0.10632610321044922, 0.10708093643188477, 0.10459136962890625, 0.10524773597717285, 0.10655903816223145, 0.10618734359741211, 0.10581302642822266, 0.10414409637451172, 0.10511016845703125, 0.10470795631408691, 0.1059420108795166, 0.10557937622070312, 0.10581541061401367, 0.10548830032348633, 0.10471367835998535, 0.10553836822509766, 0.10607719421386719, 0.10419821739196777, 0.10650038719177246, 0.10413169860839844, 0.10466957092285156, 0.10585236549377441, 0.10532975196838379, 0.10429120063781738, 0.10409331321716309, 0.10586857795715332, 0.10567760467529297, 0.10677647590637207, 0.10526442527770996, 0.10559678077697754, 0.10523581504821777, 0.10573339462280273, 0.10459327697753906, 0.10421347618103027, 0.1051797866821289, 0.10627341270446777, 0.10570287704467773, 0.10565638542175293, 0.10535883903503418, 0.10529971122741699, 0.10819506645202637, 0.10598087310791016, 0.10554933547973633, 0.10544419288635254, 0.10463571548461914, 0.10445642471313477, 0.1083986759185791, 0.10606551170349121, 0.10563969612121582, 0.10650300979614258, 0.10733890533447266, 0.10617995262145996, 0.1140286922454834, 0.10724830627441406, 0.10653376579284668, 0.12267470359802246, 0.10597658157348633, 0.10599946975708008, 0.12009549140930176, 0.106231689453125, 0.10671067237854004, 0.11677289009094238, 0.10488605499267578, 0.10632085800170898, 0.1067955493927002, 0.10552096366882324, 0.1134333610534668, 0.11207890510559082, 0.1052556037902832, 0.10508298873901367, 0.1092686653137207, 0.10480165481567383, 0.11380410194396973, 0.10611772537231445, 0.10489916801452637, 0.11291098594665527, 0.11494731903076172, 0.10544919967651367, 0.10708498954772949, 0.10545730590820312, 0.11514639854431152, 0.12618207931518555, 0.10638165473937988, 0.1053617000579834, 0.11890602111816406, 0.10514497756958008, 0.11871480941772461, 0.10582542419433594, 0.10616683959960938, 0.10465121269226074, 0.10700106620788574, 0.12012910842895508, 0.1060953140258789, 0.10507464408874512, 0.11810040473937988, 0.10618734359741211, 0.10733962059020996, 0.11565184593200684, 0.13392162322998047, 0.10855889320373535, 0.1061098575592041, 0.1070256233215332, 0.10592436790466309, 0.1094048023223877, 0.10486459732055664, 0.10572171211242676, 0.10584473609924316, 0.10466241836547852, 0.11365199089050293, 0.1046912670135498, 0.10514354705810547, 0.10465884208679199, 0.1120002269744873, 0.10563182830810547, 0.10514640808105469, 0.10594487190246582, 0.10516524314880371, 0.1103522777557373, 0.10627317428588867, 0.10695004463195801, 0.10622620582580566, 0.10629510879516602, 0.11480450630187988, 0.1050870418548584, 0.10523009300231934, 0.10720658302307129, 0.1105337142944336, 0.10632777214050293, 0.10593318939208984, 0.10524272918701172, 0.10639667510986328, 0.10962033271789551, 0.10478639602661133, 0.1063990592956543, 0.10481715202331543, 0.10483288764953613, 0.11408233642578125, 0.10536026954650879, 0.10554122924804688, 0.10584735870361328, 0.11258840560913086, 0.10662198066711426, 0.10688614845275879, 0.10570549964904785, 0.10619449615478516, 0.10921049118041992, 0.1060187816619873, 0.10702729225158691, 0.1070241928100586, 0.1059107780456543, 0.11464738845825195, 0.1060633659362793, 0.10775184631347656, 0.10699343681335449, 0.11232972145080566, 0.10742068290710449, 0.1057896614074707, 0.10670280456542969, 0.10697174072265625, 0.11060857772827148, 0.10646247863769531, 0.10749268531799316, 0.1074824333190918, 0.1064462661743164, 0.11591958999633789, 0.12301421165466309, 0.14011263847351074, 0.18356680870056152, 0.17609262466430664, 0.1753699779510498, 0.18131470680236816, 0.17722558975219727, 0.1757984161376953, 0.18592143058776855, 0.1760244369506836, 0.17516875267028809, 0.183182954788208, 0.1769397258758545, 0.1818530559539795, 0.1772000789642334, 0.1764087677001953, 0.1854846477508545, 0.17676734924316406, 0.17640972137451172, 0.17580676078796387, 0.17728543281555176, 0.17556238174438477, 0.1761455535888672, 0.17676377296447754, 0.1755373477935791, 0.17676067352294922, 0.17606234550476074, 0.17505884170532227, 0.17540216445922852, 0.17737150192260742, 0.17785882949829102, 0.1779630184173584, 0.17567658424377441, 0.17914795875549316, 0.1793231964111328, 0.17691850662231445, 0.17525577545166016, 0.1774311065673828, 0.17519807815551758, 0.17557358741760254, 0.17535758018493652, 0.17546939849853516, 0.17628240585327148, 0.17628979682922363, 0.17643260955810547, 0.17558670043945312, 0.17651987075805664, 0.17608380317687988, 0.17618489265441895, 0.17598581314086914, 0.17694997787475586, 0.17599725723266602, 0.17539572715759277, 0.1770780086517334, 0.17581796646118164, 0.17531061172485352, 0.18336868286132812, 0.17790961265563965, 0.17601847648620605, 0.17704987525939941, 0.17799115180969238, 0.17653799057006836, 0.17620444297790527, 0.17583703994750977, 0.17576193809509277, 0.17645978927612305, 0.17603230476379395, 0.17725467681884766, 0.176469087600708, 0.17637991905212402, 0.17586183547973633, 0.1764535903930664, 0.1757504940032959, 0.17594242095947266, 0.1758253574371338, 0.17506694793701172, 0.17583942413330078, 0.17643117904663086, 0.17582082748413086, 0.17598342895507812, 0.17606782913208008, 0.17626142501831055, 0.1753559112548828, 0.17705345153808594, 0.17562055587768555, 0.1776266098022461, 0.17502689361572266, 0.17592859268188477, 0.17512035369873047, 0.17525124549865723, 0.17630887031555176, 0.17661762237548828, 0.1758885383605957, 0.17543864250183105, 0.17652034759521484, 0.17644524574279785, 0.17633771896362305, 0.17672014236450195, 0.17518281936645508, 0.17638945579528809, 0.17606878280639648, 0.17600226402282715, 0.17634224891662598, 0.17498278617858887, 0.17689871788024902, 0.1765127182006836, 0.17682266235351562, 0.17673349380493164, 0.17625832557678223, 0.17507672309875488, 0.17581748962402344, 0.17536211013793945, 0.1748950481414795, 0.17608284950256348, 0.1750500202178955, 0.1754775047302246, 0.17626428604125977, 0.17507052421569824, 0.17493081092834473, 0.17746210098266602, 0.17591142654418945, 0.17484116554260254, 0.17504358291625977, 0.17413330078125, 0.17876863479614258, 0.17945265769958496, 0.21276617050170898, 0.1972520351409912, 0.10478949546813965, 0.10599303245544434, 0.10472989082336426, 0.1219489574432373, 0.11881136894226074, 0.10928630828857422, 0.1059274673461914, 0.1061394214630127, 0.10596227645874023, 0.10634946823120117, 0.10567665100097656, 0.10617637634277344, 0.1047830581665039, 0.1046142578125, 0.10433745384216309, 0.10631585121154785, 0.10588192939758301, 0.10672807693481445, 0.10582232475280762, 0.10560774803161621, 0.10482048988342285, 0.1041712760925293, 0.10587811470031738, 0.10641336441040039, 0.10495829582214355, 0.10589337348937988, 0.1058046817779541, 0.10599875450134277, 0.10480117797851562, 0.10479354858398438, 0.10567402839660645, 0.10501384735107422, 0.10616207122802734, 0.10591602325439453, 0.10577607154846191, 0.10573148727416992, 0.1043710708618164, 0.10669183731079102, 0.10486865043640137, 0.10468101501464844, 0.10441398620605469, 0.10557389259338379, 0.10576319694519043, 0.10506987571716309, 0.10468101501464844, 0.10521459579467773, 0.1040046215057373, 0.10527682304382324, 0.10442304611206055, 0.10472464561462402, 0.1046133041381836, 0.10478496551513672, 0.10552549362182617, 0.10509753227233887, 0.1054830551147461, 0.10501885414123535, 0.10585451126098633, 0.10622334480285645, 0.10508251190185547, 0.10615944862365723, 0.10488271713256836, 0.10541987419128418, 0.10564970970153809, 0.10442614555358887, 0.10419845581054688, 0.10457253456115723, 0.1046135425567627, 0.10564374923706055, 0.10495948791503906, 0.10609698295593262, 0.10537910461425781, 0.10550642013549805, 0.10441112518310547, 0.10477137565612793, 0.10588479042053223, 0.10495591163635254, 0.10530591011047363, 0.10515046119689941, 0.10552096366882324, 0.10578346252441406, 0.10549736022949219, 0.10520601272583008, 0.10450410842895508, 0.10577797889709473, 0.1047515869140625, 0.10556459426879883, 0.10443234443664551, 0.10542011260986328, 0.10605907440185547, 0.10432291030883789, 0.10574054718017578, 0.1056668758392334, 0.10432887077331543, 0.10449671745300293, 0.1056675910949707, 0.10426211357116699, 0.10465002059936523, 0.1055765151977539, 0.10444068908691406, 0.10419797897338867, 0.10558485984802246, 0.10497570037841797, 0.10562562942504883, 0.10552620887756348, 0.10484910011291504, 0.10474824905395508, 0.10550141334533691, 0.1048421859741211, 0.10506772994995117, 0.10447216033935547, 0.10550689697265625, 0.10486197471618652, 0.10415387153625488, 0.10404658317565918, 0.10535287857055664, 0.1053018569946289, 0.10585141181945801, 0.10475540161132812, 0.1054384708404541, 0.1055898666381836, 0.10514378547668457, 0.10650062561035156, 0.1042630672454834, 0.10491180419921875, 0.10578775405883789, 0.10491561889648438, 0.10735392570495605, 0.10672593116760254, 0.10609245300292969, 0.10534286499023438, 0.10466718673706055, 0.10556197166442871, 0.10470151901245117, 0.10422921180725098, 0.10566973686218262, 0.1052250862121582, 0.10444998741149902, 0.10507559776306152, 0.10484933853149414, 0.10481834411621094, 0.1048116683959961, 0.10388898849487305, 0.1049647331237793, 0.10411405563354492, 0.10509538650512695, 0.10539388656616211, 0.10567069053649902, 0.1043081283569336, 0.10591435432434082, 0.10478806495666504, 0.10571026802062988, 0.10541105270385742, 0.10664176940917969, 0.1055760383605957, 0.10452866554260254, 0.10532546043395996, 0.10595083236694336, 0.10569477081298828, 0.10410451889038086, 0.1040964126586914, 0.10474324226379395, 0.10570597648620605, 0.1055610179901123, 0.10591554641723633, 0.10451912879943848, 0.10612344741821289, 0.10675406455993652, 0.10596632957458496, 0.10515737533569336, 0.10683679580688477, 0.10558605194091797, 0.10659265518188477, 0.10444426536560059, 0.10432052612304688, 0.10462760925292969, 0.10422730445861816, 0.10447859764099121, 0.10410928726196289, 0.10489749908447266, 0.10503149032592773, 0.10455536842346191, 0.1054384708404541, 0.10562729835510254, 0.10492682456970215, 0.10561609268188477, 0.10512661933898926, 0.10492467880249023, 0.1047828197479248, 0.1045372486114502, 0.1045534610748291, 0.10643887519836426, 0.10412716865539551, 0.10604357719421387, 0.10409402847290039, 0.1056523323059082, 0.1048593521118164, 0.1046292781829834, 0.10594034194946289, 0.10503435134887695, 0.10573554039001465, 0.10506272315979004, 0.10398149490356445, 0.10540103912353516, 0.1055135726928711, 0.10457110404968262, 0.10536003112792969, 0.10420870780944824, 0.10458493232727051, 0.10452127456665039, 0.10497736930847168, 0.10560393333435059, 0.10524320602416992, 0.10491585731506348, 0.10565590858459473, 0.1041254997253418, 0.10418128967285156, 0.10604214668273926, 0.10553598403930664, 0.10460495948791504, 0.10544991493225098, 0.10471868515014648, 0.10514593124389648, 0.1046450138092041, 0.10601544380187988, 0.1053464412689209, 0.10519862174987793, 0.10528373718261719, 0.1047208309173584, 0.1059412956237793, 0.10427522659301758, 0.1057577133178711, 0.10525965690612793, 0.10541439056396484, 0.10488271713256836, 0.10578036308288574, 0.10459685325622559, 0.10537576675415039, 0.10443830490112305, 0.10449934005737305, 0.10535335540771484, 0.10535240173339844, 0.10502076148986816, 0.1055295467376709, 0.10613870620727539, 0.10438871383666992, 0.10510373115539551, 0.10488033294677734, 0.10500550270080566, 0.10464358329772949, 0.10485482215881348, 0.10576963424682617, 0.10495996475219727, 0.10493302345275879, 0.10569405555725098, 0.10583043098449707, 0.10457968711853027, 0.10448741912841797, 0.10438394546508789, 0.1042327880859375, 0.10461974143981934, 0.10521316528320312, 0.10563969612121582, 0.10431456565856934, 0.10566592216491699, 0.10684418678283691, 0.10483098030090332, 0.10637497901916504, 0.10550546646118164, 0.10576868057250977, 0.10745978355407715, 0.10528731346130371, 0.10524415969848633, 0.10536980628967285, 0.10530638694763184, 0.10547065734863281, 0.10498690605163574, 0.10570096969604492, 0.10570907592773438, 0.10499405860900879, 0.10436248779296875, 0.1049492359161377, 0.1048135757446289, 0.10468149185180664, 0.10662627220153809, 0.10512042045593262, 0.10465431213378906, 0.10410094261169434, 0.1058189868927002, 0.10455203056335449, 0.10445785522460938, 0.10600757598876953, 0.10590744018554688, 0.10499358177185059, 0.10437607765197754, 0.10586762428283691, 0.10590481758117676, 0.10442852973937988, 0.10588359832763672, 0.10440826416015625, 0.10530447959899902, 0.10551667213439941, 0.10451889038085938, 0.10580253601074219, 0.10544872283935547, 0.10601472854614258, 0.10543966293334961, 0.10467386245727539, 0.10558366775512695, 0.10521912574768066, 0.10547780990600586, 0.10553908348083496, 0.10572195053100586, 0.10560131072998047, 0.14508819580078125, 0.1159205436706543, 0.10568904876708984, 0.10544085502624512, 0.10555696487426758, 0.10569143295288086, 0.10621380805969238, 0.10492706298828125, 0.1057429313659668, 0.10554981231689453, 0.10558152198791504, 0.10472249984741211, 0.10553884506225586, 0.10456061363220215, 0.10527420043945312, 0.1048429012298584, 0.10569643974304199, 0.10497021675109863, 0.10524940490722656, 0.10507822036743164, 0.10503172874450684, 0.10443925857543945, 0.10549139976501465, 0.10417413711547852, 0.1058042049407959, 0.10437774658203125, 0.10621905326843262, 0.10462474822998047, 0.10574603080749512, 0.10483098030090332, 0.10498929023742676, 0.1060185432434082, 0.10643815994262695, 0.10495638847351074, 0.10539078712463379, 0.10449457168579102, 0.10530209541320801, 0.10448122024536133, 0.1044468879699707, 0.10783958435058594, 0.10910391807556152, 0.10577178001403809, 0.10631322860717773, 0.10471868515014648, 0.10489416122436523, 0.10562443733215332, 0.105438232421875, 0.10560417175292969, 0.10590076446533203, 0.10397124290466309, 0.10536336898803711, 0.10538816452026367, 0.10461759567260742, 0.10485410690307617, 0.10570621490478516, 0.10514354705810547, 0.10406184196472168, 0.10423970222473145, 0.10448718070983887, 0.10607481002807617, 0.10561323165893555, 0.10572409629821777, 0.10529661178588867, 0.10485696792602539, 0.10616087913513184, 0.10557889938354492, 0.10478854179382324, 0.10608172416687012, 0.10546326637268066, 0.10491609573364258, 0.10530281066894531, 0.10474705696105957, 0.10450983047485352, 0.10442018508911133, 0.10481929779052734, 0.10637140274047852, 0.10595941543579102, 0.10532093048095703, 0.10502862930297852, 0.10409212112426758, 0.10563158988952637, 0.10416555404663086, 0.10549759864807129, 0.10552096366882324, 0.10490131378173828, 0.10420870780944824, 0.10481548309326172, 0.10453343391418457, 0.10546374320983887, 0.10530662536621094, 0.10493183135986328, 0.10554313659667969, 0.10580039024353027, 0.10592460632324219, 0.10575413703918457, 0.10675191879272461, 0.10435199737548828, 0.10443425178527832, 0.10742950439453125, 0.10606551170349121, 0.10610127449035645, 0.10428118705749512, 0.10570359230041504, 0.10566592216491699, 0.10496830940246582, 0.10427188873291016, 0.10567688941955566, 0.10446405410766602, 0.1053171157836914, 0.10599851608276367, 0.10422945022583008, 0.1048274040222168, 0.1057901382446289, 0.10439038276672363, 0.10571050643920898, 0.10593509674072266, 0.10486364364624023, 0.10530638694763184, 0.10568904876708984, 0.10469508171081543, 0.10468435287475586, 0.10453248023986816, 0.10422468185424805, 0.10578489303588867, 0.106292724609375, 0.10477638244628906, 0.10540413856506348, 0.10571050643920898, 0.10541582107543945, 0.1053311824798584, 0.10478401184082031, 0.10437774658203125, 0.1042470932006836, 0.10417675971984863, 0.10494542121887207, 0.10431718826293945, 0.11551117897033691, 0.10589146614074707, 0.10453534126281738, 0.10542464256286621, 0.1070249080657959, 0.10636210441589355, 0.10645413398742676, 0.10488176345825195, 0.10435056686401367, 0.1048426628112793, 0.10558271408081055, 0.10627317428588867, 0.10441374778747559, 0.10493326187133789, 0.10580945014953613, 0.1048591136932373, 0.1058650016784668, 0.10440969467163086, 0.10622167587280273, 0.10593080520629883, 0.10438084602355957, 0.10429120063781738, 0.10527849197387695, 0.1047515869140625, 0.10574674606323242, 0.10524439811706543, 0.10578632354736328, 0.10538721084594727, 0.10602641105651855, 0.10556244850158691, 0.10665464401245117, 0.10592222213745117, 0.10515022277832031, 0.10618901252746582, 0.10568928718566895, 0.10618019104003906, 0.1054832935333252, 0.10542750358581543, 0.10415506362915039, 0.10589599609375, 0.10442519187927246, 0.105987548828125, 0.10490107536315918, 0.1057271957397461, 0.10463786125183105, 0.10499000549316406, 0.10443282127380371, 0.10494017601013184, 0.10563421249389648, 0.10505509376525879, 0.10519194602966309, 0.10565543174743652, 0.10596847534179688, 0.10465812683105469, 0.10613298416137695, 0.10559678077697754, 0.10551571846008301, 0.10612034797668457, 0.10580587387084961, 0.10589766502380371, 0.1053473949432373, 0.10544753074645996, 0.10522031784057617, 0.10561537742614746, 0.10513997077941895, 0.10515713691711426, 0.10453581809997559, 0.10550045967102051, 0.1048738956451416, 0.10563516616821289, 0.10535836219787598, 0.10487866401672363, 0.1044163703918457, 0.10474324226379395, 0.10468196868896484, 0.10497927665710449, 0.10594034194946289, 0.10625815391540527, 0.10471129417419434, 0.10640406608581543, 0.10610413551330566, 0.1045677661895752, 0.10472846031188965, 0.10463786125183105, 0.10425758361816406, 0.10503196716308594, 0.1046745777130127, 0.1056666374206543, 0.10436129570007324, 0.10456967353820801, 0.10428857803344727, 0.10452032089233398, 0.10469174385070801, 0.10612058639526367, 0.10486888885498047, 0.10487246513366699, 0.10488629341125488, 0.10437321662902832, 0.10488009452819824, 0.10563230514526367, 0.10525703430175781, 0.10483741760253906, 0.10662078857421875, 0.10477423667907715, 0.1061086654663086, 0.10506343841552734, 0.10550284385681152, 0.10438227653503418, 0.10503935813903809, 0.10444498062133789, 0.10594391822814941, 0.1044764518737793, 0.10444116592407227, 0.10482168197631836, 0.10482096672058105, 0.10438799858093262, 0.10454678535461426, 0.10587167739868164, 0.10466623306274414, 0.10451555252075195, 0.10463929176330566, 0.10456156730651855, 0.10448861122131348, 0.1046140193939209, 0.10451960563659668, 0.10490870475769043, 0.10587525367736816, 0.10432314872741699, 0.10577583312988281, 0.10447525978088379, 0.10599780082702637, 0.10457944869995117, 0.1048743724822998, 0.104461669921875, 0.10431742668151855, 0.10453653335571289, 0.10547685623168945, 0.10696029663085938, 0.10494041442871094, 0.10589933395385742, 0.10450601577758789, 0.1060173511505127, 0.10518193244934082, 0.10446405410766602, 0.1050417423248291, 0.10567498207092285, 0.10460996627807617, 0.10701847076416016, 0.1049659252166748, 0.10562944412231445, 0.10470318794250488, 0.10457038879394531, 0.1045985221862793, 0.10683012008666992, 0.10541510581970215, 0.10621833801269531, 0.10626864433288574, 0.10553550720214844, 0.10475468635559082, 0.10482263565063477, 0.10435938835144043, 0.10473847389221191, 0.10429906845092773, 0.10457468032836914, 0.10433006286621094, 0.10476136207580566, 0.10485053062438965, 0.10406851768493652, 0.10427665710449219, 0.1044774055480957, 0.1039888858795166, 0.10413432121276855, 0.1041250228881836, 0.10484123229980469, 0.10470223426818848, 0.10505890846252441, 0.1052248477935791, 0.10567665100097656, 0.10668110847473145, 0.10660052299499512, 0.10693955421447754, 0.10801482200622559, 0.1046745777130127, 0.10437393188476562, 0.10454344749450684, 0.10462832450866699, 0.10464668273925781, 0.10478782653808594, 0.1060938835144043, 0.10515213012695312, 0.10483336448669434, 0.10592889785766602, 0.1060481071472168, 0.10592484474182129, 0.10556554794311523, 0.10491633415222168, 0.1059255599975586, 0.10573077201843262, 0.10480475425720215, 0.10506868362426758, 0.10607075691223145, 0.10491204261779785, 0.1043694019317627, 0.10619258880615234, 0.1045832633972168, 0.10501837730407715, 0.10446405410766602, 0.10479903221130371, 0.10564184188842773, 0.1063992977142334, 0.1042325496673584, 0.1062617301940918, 0.10456681251525879, 0.10620832443237305, 0.10445499420166016, 0.10578083992004395, 0.10453248023986816, 0.10495781898498535, 0.10483765602111816, 0.10525894165039062, 0.1049196720123291, 0.10493040084838867, 0.10451436042785645, 0.10585284233093262, 0.10458922386169434, 0.10492396354675293, 0.10571050643920898, 0.10458946228027344, 0.10473108291625977, 0.10437321662902832, 0.10489630699157715, 0.10547327995300293, 0.10479187965393066, 0.10520148277282715, 0.10476994514465332, 0.10534811019897461, 0.10579299926757812, 0.10455846786499023, 0.1044611930847168, 0.10659122467041016, 0.10486674308776855, 0.10570478439331055, 0.10601687431335449, 0.10458970069885254, 0.10461854934692383, 0.10476970672607422, 0.10513663291931152, 0.1060478687286377, 0.10528802871704102, 0.10476160049438477, 0.10457825660705566, 0.10483169555664062, 0.10559296607971191, 0.10642242431640625, 0.10454797744750977, 0.10455942153930664, 0.10474658012390137, 0.10460305213928223, 0.10492491722106934, 0.10597681999206543, 0.10596299171447754, 0.10557126998901367, 0.10471391677856445, 0.10441374778747559, 0.1046743392944336, 0.10547184944152832, 0.1049044132232666, 0.1046743392944336, 0.10608315467834473, 0.10571694374084473, 0.10502076148986816, 0.1046609878540039, 0.10525202751159668, 0.10633325576782227, 0.10467410087585449, 0.10587954521179199, 0.10665130615234375, 0.10459709167480469, 0.10432314872741699, 0.10464119911193848, 0.1047065258026123, 0.10450601577758789, 0.10569310188293457, 0.1045534610748291, 0.10444021224975586, 0.10572171211242676, 0.10535430908203125, 0.10645508766174316, 0.10435175895690918, 0.10443830490112305, 0.1055290699005127, 0.1064913272857666, 0.10523390769958496, 0.10553622245788574, 0.10429596900939941, 0.10479521751403809, 0.10476922988891602, 0.1044769287109375, 0.1047663688659668, 0.10475611686706543, 0.10473012924194336, 0.10491681098937988, 0.10483574867248535, 0.10459661483764648, 0.10535049438476562, 0.10550618171691895, 0.10618305206298828, 0.10416603088378906, 0.10467886924743652, 0.10449600219726562, 0.10452866554260254, 0.10450029373168945, 0.10467410087585449, 0.10587358474731445, 0.10456657409667969, 0.10606956481933594, 0.10587573051452637, 0.10474753379821777, 0.10458803176879883, 0.10471153259277344, 0.10457229614257812, 0.1045081615447998, 0.10421419143676758, 0.10463690757751465, 0.10413217544555664, 0.10473036766052246, 0.10447978973388672, 0.10430169105529785, 0.10442566871643066, 0.10446953773498535, 0.10452723503112793, 0.10452866554260254, 0.10515999794006348, 0.1049189567565918, 0.10460829734802246, 0.10478472709655762, 0.10438370704650879, 0.10579562187194824, 0.10439062118530273, 0.10497307777404785, 0.10473895072937012, 0.1045069694519043, 0.10481905937194824, 0.10531306266784668, 0.10515856742858887, 0.10452914237976074, 0.1048882007598877, 0.10475945472717285, 0.1046288013458252, 0.10483288764953613, 0.10439300537109375, 0.10576963424682617, 0.10477399826049805, 0.10537147521972656, 0.10473394393920898, 0.1043844223022461, 0.1067054271697998, 0.10440468788146973, 0.10554909706115723, 0.10602164268493652, 0.10470390319824219, 0.10480976104736328, 0.10573196411132812, 0.10477471351623535, 0.10438394546508789, 0.10488438606262207, 0.10606002807617188, 0.1044931411743164, 0.1065378189086914, 0.10507774353027344, 0.10614156723022461, 0.10534143447875977, 0.10447311401367188, 0.10602211952209473, 0.10547280311584473, 0.10452055931091309, 0.10400915145874023, 0.10469818115234375, 0.1060330867767334, 0.10584473609924316, 0.10648608207702637, 0.10607051849365234, 0.10647869110107422, 0.10590577125549316, 0.10536861419677734, 0.1080789566040039, 0.10489416122436523, 0.10527777671813965, 0.10578274726867676, 0.10471463203430176, 0.10649776458740234, 0.1061558723449707, 0.10623955726623535, 0.10450887680053711, 0.10469555854797363, 0.10460591316223145, 0.10486602783203125, 0.10589313507080078, 0.1045234203338623, 0.10468482971191406, 0.10478878021240234, 0.1048133373260498, 0.10584378242492676, 0.10460782051086426, 0.10632967948913574, 0.10463714599609375, 0.10500073432922363, 0.1045229434967041, 0.10459041595458984, 0.1051020622253418, 0.10432815551757812, 0.10612154006958008, 0.1045384407043457, 0.1046285629272461, 0.10426092147827148, 0.10476922988891602, 0.10527825355529785, 0.10455942153930664, 0.10613226890563965, 0.10463643074035645, 0.10464859008789062, 0.10416960716247559, 0.10595154762268066, 0.10618877410888672, 0.10476899147033691, 0.10454559326171875, 0.10601949691772461, 0.10552382469177246, 0.10574555397033691, 0.1067967414855957, 0.10490059852600098, 0.10436224937438965, 0.10588622093200684, 0.10619568824768066, 0.10525226593017578, 0.11115145683288574, 0.10523414611816406, 0.10472726821899414, 0.10640406608581543, 0.10495901107788086, 0.10703706741333008, 0.10533714294433594, 0.10447216033935547, 0.10472750663757324, 0.10480999946594238, 0.10598897933959961, 0.10455703735351562, 0.10592079162597656, 0.10468363761901855, 0.10507392883300781, 0.10626602172851562, 0.10502815246582031, 0.10604548454284668, 0.10616755485534668, 0.10487604141235352, 0.10477089881896973, 0.10648274421691895, 0.10468745231628418, 0.10510659217834473, 0.10607481002807617, 0.1052250862121582, 0.10516595840454102, 0.10476946830749512, 0.10664939880371094, 0.10480332374572754, 0.10481834411621094, 0.10644793510437012, 0.10593008995056152, 0.10502767562866211, 0.1061258316040039, 0.10607361793518066, 0.10575056076049805, 0.10484051704406738, 0.10587692260742188, 0.10669493675231934, 0.1106407642364502, 0.10442423820495605, 0.10578227043151855, 0.10467171669006348, 0.10514426231384277, 0.10583877563476562, 0.10488462448120117, 0.10525274276733398, 0.1058342456817627, 0.10624408721923828, 0.10526800155639648, 0.10587167739868164, 0.10597586631774902, 0.1048133373260498, 0.10536479949951172, 0.10586237907409668, 0.1049349308013916, 0.10524964332580566, 0.10474491119384766, 0.10499382019042969, 0.10599541664123535, 0.1049952507019043, 0.10604619979858398, 0.10646820068359375, 0.10511231422424316, 0.10528063774108887, 0.10582685470581055, 0.1047065258026123, 0.10576939582824707, 0.10644149780273438, 0.10446786880493164, 0.10688972473144531, 0.10514235496520996, 0.10562252998352051, 0.10510587692260742, 0.10570883750915527, 0.10627436637878418, 0.10456490516662598, 0.10521745681762695, 0.10554957389831543, 0.1047055721282959, 0.10606622695922852, 0.10591912269592285, 0.10530543327331543, 0.10599899291992188, 0.10540652275085449, 0.10599446296691895, 0.10589361190795898, 0.10643863677978516, 0.10674476623535156, 0.1053922176361084, 0.10456228256225586, 0.10710692405700684, 0.10538792610168457, 0.10520577430725098, 0.10601449012756348, 0.10674810409545898, 0.1075594425201416, 0.10534262657165527, 0.10659146308898926, 0.10551095008850098, 0.10500717163085938, 0.1057131290435791, 0.10577797889709473, 0.10588622093200684, 0.10537314414978027, 0.10695052146911621, 0.10598278045654297, 0.10667538642883301, 0.10491466522216797, 0.10600852966308594, 0.10636305809020996, 0.10574793815612793, 0.10606932640075684, 0.10566043853759766, 0.10568046569824219, 0.10570788383483887, 0.10508179664611816, 0.10644102096557617, 0.10613083839416504, 0.10627388954162598, 0.1045541763305664, 0.10519027709960938, 0.1059117317199707, 0.10502266883850098, 0.10678362846374512, 0.10573244094848633, 0.10481786727905273, 0.10571599006652832, 0.10487246513366699, 0.10496830940246582, 0.10471987724304199, 0.10589361190795898, 0.10534167289733887, 0.11137890815734863, 0.10514521598815918, 0.10666131973266602, 0.10519170761108398, 0.10579657554626465, 0.10757613182067871, 0.10628747940063477, 0.10602712631225586, 0.1056525707244873, 0.10502243041992188, 0.10610342025756836, 0.10471391677856445, 0.10484886169433594, 0.10467910766601562, 0.10622000694274902, 0.10631608963012695, 0.10637736320495605, 0.10613059997558594, 0.10558843612670898, 0.10704994201660156, 0.10624980926513672, 0.10585427284240723, 0.1062157154083252, 0.10513544082641602, 0.10614395141601562, 0.10610008239746094, 0.10592508316040039, 0.10459327697753906, 0.10473108291625977, 0.10468411445617676, 0.10596656799316406, 0.10474491119384766, 0.10468411445617676, 0.10607671737670898, 0.10603737831115723, 0.10580182075500488, 0.10478019714355469, 0.10535931587219238, 0.10563349723815918, 0.10613083839416504, 0.10433077812194824, 0.1048283576965332, 0.10485172271728516, 0.10568523406982422, 0.10544085502624512, 0.10488080978393555, 0.10588860511779785, 0.10600709915161133, 0.10541939735412598, 0.10478401184082031, 0.10704207420349121, 0.1058344841003418, 0.10613489151000977, 0.1069040298461914, 0.10519075393676758, 0.10621905326843262, 0.10654425621032715, 0.10594511032104492, 0.10604977607727051, 0.1061563491821289, 0.10491108894348145, 0.10476160049438477, 0.10599565505981445, 0.1056976318359375, 0.10607647895812988, 0.10600709915161133, 0.10596084594726562, 0.10599875450134277, 0.10545802116394043, 0.10494327545166016, 0.10591387748718262, 0.10629725456237793, 0.10565996170043945, 0.10487103462219238, 0.10576629638671875, 0.10591602325439453, 0.10646533966064453, 0.10515046119689941, 0.10621070861816406, 0.10471391677856445, 0.10597944259643555, 0.10544943809509277, 0.10481929779052734, 0.10559654235839844, 0.10608148574829102, 0.10510110855102539, 0.10660505294799805, 0.10508012771606445, 0.1063377857208252, 0.10511541366577148, 0.10641598701477051, 0.1047964096069336, 0.10564589500427246, 0.10543942451477051, 0.10577797889709473, 0.10532784461975098, 0.10609316825866699, 0.12406682968139648, 0.10738301277160645, 0.1051168441772461, 0.10612821578979492, 0.10561370849609375, 0.10628032684326172, 0.10497593879699707, 0.10458946228027344, 0.10795283317565918, 0.10522007942199707, 0.10669946670532227, 0.10614562034606934, 0.10608530044555664, 0.10463595390319824, 0.10483241081237793, 0.10600590705871582, 0.10523581504821777, 0.1049952507019043, 0.10574150085449219, 0.10590243339538574, 0.10450243949890137, 0.10471582412719727, 0.10612845420837402, 0.10566973686218262, 0.10556459426879883, 0.10560011863708496, 0.10454702377319336, 0.10519003868103027, 0.10610222816467285, 0.10492491722106934, 0.10607194900512695, 0.10454535484313965, 0.10638308525085449, 0.10626769065856934, 0.10474944114685059, 0.1064596176147461, 0.10493659973144531, 0.10604310035705566, 0.10627603530883789, 0.10578060150146484, 0.10555100440979004, 0.10648918151855469, 0.1062929630279541, 0.10529255867004395, 0.10456323623657227, 0.1047067642211914, 0.1042323112487793, 0.10714244842529297, 0.10689401626586914, 0.10561299324035645, 0.10578417778015137, 0.10511636734008789, 0.10596394538879395, 0.10626530647277832, 0.10496640205383301, 0.10492110252380371, 0.10503244400024414, 0.10474133491516113, 0.10513782501220703, 0.10544061660766602, 0.10610008239746094, 0.10599374771118164, 0.10593652725219727, 0.10611271858215332, 0.10628724098205566, 0.10492968559265137, 0.10580682754516602, 0.10470461845397949, 0.10638093948364258, 0.10619473457336426, 0.10531115531921387, 0.10627198219299316, 0.10537290573120117, 0.10564565658569336, 0.10565686225891113, 0.10451507568359375, 0.10459375381469727, 0.10487842559814453, 0.10619187355041504, 0.10590362548828125, 0.10627031326293945, 0.10659122467041016, 0.10547494888305664, 0.1063845157623291, 0.10633420944213867, 0.10607743263244629, 0.10449051856994629, 0.10451292991638184, 0.1060492992401123, 0.10587358474731445, 0.10527181625366211, 0.10555577278137207, 0.10582518577575684, 0.10601592063903809, 0.10634708404541016, 0.10595536231994629, 0.10451316833496094, 0.10618185997009277, 0.10629940032958984, 0.106689453125, 0.10496234893798828, 0.10484147071838379, 0.10500073432922363, 0.10470080375671387, 0.10603451728820801, 0.1046602725982666, 0.10609650611877441, 0.10519123077392578, 0.10584402084350586, 0.11274933815002441, 0.10605907440185547, 0.10570168495178223, 0.10499143600463867, 0.10583686828613281, 0.10581088066101074, 0.1059424877166748, 0.1058042049407959, 0.10557246208190918, 0.1064596176147461, 0.10576844215393066, 0.10522603988647461, 0.10663294792175293, 0.10595321655273438, 0.10483169555664062, 0.10598874092102051, 0.1045839786529541, 0.10536026954650879, 0.10491752624511719, 0.10510611534118652, 0.10562372207641602, 0.1118478775024414, 0.10562992095947266, 0.10552859306335449, 0.10588741302490234, 0.10464787483215332, 0.10798072814941406, 0.10583639144897461, 0.1101679801940918, 0.10700273513793945, 0.10633015632629395, 0.10864949226379395, 0.10631179809570312, 0.1271522045135498, 0.10626912117004395, 0.10985803604125977, 0.10855269432067871, 0.10681676864624023, 0.10591530799865723, 0.11055970191955566, 0.10862565040588379, 0.10746574401855469, 0.10878252983093262, 0.10711860656738281, 0.1145632266998291, 0.10614681243896484, 0.10759305953979492, 0.11011052131652832, 0.10894584655761719, 0.11271810531616211, 0.10913848876953125, 0.10859894752502441, 0.10542440414428711, 0.10945248603820801, 0.10640192031860352, 0.1054544448852539, 0.10669732093811035, 0.10526466369628906, 0.11747479438781738, 0.10553431510925293, 0.1066598892211914, 0.1062772274017334, 0.11268472671508789, 0.10744690895080566, 0.10787320137023926, 0.10584545135498047, 0.10596179962158203, 0.10973811149597168, 0.10603880882263184, 0.10707569122314453, 0.10553932189941406, 0.10595846176147461, 0.12828636169433594, 0.10637712478637695, 0.1060488224029541, 0.10574483871459961, 0.10614705085754395, 0.13245439529418945, 0.10627317428588867, 0.10597729682922363, 0.10629582405090332, 0.10637044906616211, 0.11736273765563965, 0.11994194984436035, 0.1057734489440918, 0.10502791404724121, 0.10488224029541016, 0.10609173774719238, 0.11844515800476074, 0.1052541732788086, 0.10486292839050293, 0.12088155746459961, 0.10513710975646973, 0.10597419738769531, 0.11631202697753906, 0.12259364128112793, 0.10750699043273926, 0.10696744918823242, 0.10676264762878418, 0.10530662536621094, 0.1113588809967041, 0.12796854972839355, 0.10852432250976562, 0.10688591003417969, 0.10644268989562988, 0.10536360740661621, 0.11171150207519531, 0.10482597351074219, 0.10657572746276855, 0.10492873191833496, 0.11044788360595703, 0.10622048377990723, 0.10470843315124512, 0.10646247863769531, 0.10607051849365234, 0.11475896835327148, 0.10598278045654297, 0.10646677017211914, 0.11194300651550293, 0.10626935958862305, 0.10980796813964844, 0.10710287094116211, 0.10684776306152344, 0.10614132881164551, 0.10700535774230957, 0.11518049240112305, 0.10614824295043945, 0.10683917999267578, 0.10564661026000977, 0.10793352127075195, 0.11034512519836426, 0.10496306419372559, 0.10482382774353027, 0.10685086250305176, 0.11091089248657227, 0.10648655891418457, 0.11353445053100586, 0.1065821647644043, 0.11475229263305664, 0.1077723503112793, 0.10543155670166016, 0.10499310493469238, 0.10483193397521973, 0.11145639419555664, 0.10621023178100586, 0.10464930534362793, 0.10532402992248535, 0.10446715354919434, 0.11374783515930176, 0.11222553253173828, 0.1069478988647461, 0.10607075691223145, 0.10721421241760254, 0.10716414451599121, 0.11795735359191895, 0.12324380874633789, 0.12409567832946777, 0.12265253067016602, 0.12429332733154297, 0.10691666603088379, 0.11026453971862793, 0.10628414154052734, 0.11122941970825195, 0.10654997825622559, 0.10455179214477539, 0.10501956939697266, 0.1064295768737793, 0.11439204216003418, 0.1053469181060791, 0.11163139343261719, 0.10573887825012207, 0.10662078857421875, 0.10546016693115234, 0.1152031421661377, 0.12385153770446777, 0.1553056240081787, 0.15522074699401855, 0.158705472946167, 0.1435534954071045, 0.10616326332092285, 0.10602450370788574, 0.10597395896911621, 0.12464404106140137, 0.10607600212097168, 0.10464620590209961, 0.10513830184936523, 0.11411285400390625, 0.10634589195251465, 0.10549712181091309, 0.10580706596374512, 0.10641622543334961, 0.1107490062713623, 0.12344145774841309, 0.12086296081542969, 0.10563969612121582, 0.11261558532714844, 0.11045479774475098, 0.10565352439880371, 0.1066427230834961, 0.11606121063232422, 0.10684084892272949, 0.11278438568115234, 0.11094832420349121, 0.10547828674316406, 0.11349034309387207, 0.11330485343933105, 0.11029553413391113, 0.1064920425415039, 0.11586666107177734, 0.11408042907714844, 0.10621070861816406, 0.10551667213439941, 0.10662055015563965, 0.1138906478881836, 0.11729240417480469, 0.11320924758911133, 0.1072542667388916, 0.12225747108459473, 0.10727596282958984, 0.11323189735412598, 0.10829544067382812, 0.11771798133850098, 0.13350868225097656, 0.11865973472595215, 0.10565590858459473, 0.10615897178649902, 0.10629677772521973, 0.10570549964904785, 0.13036537170410156, 0.10508418083190918, 0.10626840591430664, 0.10601568222045898, 0.10946393013000488, 0.10594058036804199, 0.11507344245910645, 0.10598087310791016, 0.10627102851867676, 0.10591411590576172, 0.110748291015625, 0.11264395713806152, 0.10792303085327148, 0.10606002807617188, 0.10454916954040527, 0.10516810417175293, 0.11316823959350586, 0.11214995384216309, 0.10636425018310547, 0.10603213310241699, 0.10589790344238281, 0.10962653160095215, 0.12041330337524414, 0.11485719680786133, 0.11414003372192383, 0.1058809757232666, 0.11203455924987793, 0.1058950424194336, 0.10456705093383789, 0.11129426956176758, 0.10675954818725586, 0.10604381561279297, 0.1048731803894043, 0.10650086402893066, 0.11714339256286621, 0.10507869720458984, 0.10708737373352051, 0.10486173629760742, 0.10560488700866699, 0.11099076271057129, 0.10525155067443848, 0.10467910766601562, 0.10464859008789062, 0.11352872848510742, 0.11609554290771484, 0.11426568031311035, 0.10714888572692871, 0.10571146011352539, 0.12260770797729492, 0.11569523811340332, 0.12060070037841797, 0.11321020126342773, 0.11392521858215332, 0.11617398262023926, 0.12300658226013184, 0.11920952796936035, 0.10564947128295898, 0.11440777778625488, 0.10666656494140625, 0.1135416030883789, 0.11648821830749512, 0.11227989196777344, 0.12134981155395508, 0.11375999450683594, 0.11258149147033691, 0.12042808532714844, 0.147538423538208, 0.10599040985107422, 0.10684657096862793, 0.10638761520385742, 0.10526347160339355, 0.11724710464477539, 0.10638117790222168, 0.10473227500915527, 0.10654044151306152, 0.10618019104003906, 0.10973930358886719, 0.10878586769104004, 0.11928820610046387, 0.10740208625793457, 0.10543060302734375, 0.1064307689666748, 0.10588598251342773, 0.12131214141845703, 0.10717368125915527, 0.11301565170288086, 0.10994911193847656, 0.11505770683288574, 0.10681772232055664, 0.11280632019042969, 0.11036014556884766, 0.11434054374694824, 0.10700702667236328, 0.10926532745361328, 0.1062004566192627, 0.11038041114807129, 0.10666990280151367, 0.12417340278625488, 0.10697364807128906, 0.10542583465576172, 0.10515093803405762, 0.10664677619934082, 0.12772035598754883, 0.10614395141601562, 0.10466337203979492, 0.1061561107635498, 0.10604739189147949, 0.11326837539672852, 0.10683107376098633, 0.10474443435668945, 0.10599637031555176, 0.11221098899841309, 0.10620808601379395, 0.10620403289794922, 0.10496902465820312, 0.11079168319702148, 0.10721135139465332, 0.10649490356445312, 0.10643839836120605, 0.12146902084350586, 0.11403465270996094, 0.11526656150817871, 0.10622668266296387, 0.11379623413085938, 0.1071019172668457, 0.10495638847351074, 0.10706043243408203, 0.10846757888793945, 0.11017584800720215, 0.11439132690429688, 0.1054832935333252, 0.10634207725524902, 0.10558056831359863, 0.10553789138793945, 0.11266040802001953, 0.1217813491821289, 0.12752389907836914, 0.1300196647644043, 0.14586210250854492, 0.15905070304870605, 0.1556541919708252, 0.13164806365966797, 0.10622048377990723, 0.10628056526184082, 0.11774539947509766, 0.11896371841430664, 0.10491132736206055, 0.10528755187988281, 0.10570764541625977, 0.10573482513427734, 0.12001228332519531, 0.10479426383972168, 0.10532546043395996, 0.10637164115905762, 0.11210489273071289, 0.11630940437316895, 0.12862920761108398, 0.12695097923278809, 0.13429021835327148, 0.12828826904296875, 0.12372469902038574, 0.11057329177856445, 0.10698437690734863, 0.10519123077392578, 0.1054680347442627, 0.11378765106201172, 0.11241888999938965, 0.1134653091430664, 0.10583734512329102, 0.10583066940307617, 0.11318016052246094, 0.10599184036254883, 0.10584402084350586, 0.10550117492675781, 0.11174297332763672, 0.11035466194152832, 0.12381291389465332, 0.1231689453125, 0.10503959655761719, 0.10641598701477051, 0.10849356651306152, 0.11334657669067383, 0.11224222183227539, 0.1108849048614502, 0.11681270599365234, 0.11206889152526855, 0.10709810256958008, 0.11261749267578125, 0.11606431007385254, 0.11237668991088867, 0.10677385330200195, 0.10625767707824707, 0.11265230178833008, 0.11765003204345703, 0.10544300079345703, 0.10647082328796387, 0.10650825500488281, 0.10520434379577637, 0.11934781074523926, 0.10708880424499512, 0.10638785362243652, 0.10596275329589844, 0.10469985008239746, 0.11799979209899902, 0.11498308181762695, 0.10683059692382812, 0.12572336196899414, 0.10698652267456055, 0.11429095268249512, 0.11320042610168457, 0.11734414100646973, 0.11057567596435547, 0.11270689964294434, 0.10506176948547363, 0.1068260669708252, 0.10497093200683594, 0.11486101150512695, 0.11582231521606445, 0.10675525665283203, 0.1064445972442627, 0.10922122001647949, 0.10714530944824219, 0.10607790946960449, 0.10690665245056152, 0.11292052268981934, 0.10522961616516113, 0.10749292373657227, 0.10727787017822266, 0.11233067512512207, 0.10627055168151855, 0.11044144630432129, 0.10698604583740234, 0.10613536834716797, 0.106719970703125, 0.10576152801513672, 0.11420893669128418, 0.10622000694274902, 0.10576868057250977, 0.1122293472290039, 0.10664224624633789, 0.11183714866638184, 0.10640096664428711, 0.12360954284667969, 0.11282968521118164, 0.12407469749450684, 0.11606645584106445, 0.12122106552124023, 0.11972832679748535, 0.15127277374267578, 0.10620951652526855, 0.11677265167236328, 0.10700750350952148, 0.11970758438110352, 0.12144994735717773, 0.13135981559753418, 0.1267223358154297, 0.10714077949523926, 0.1339571475982666, 0.12929439544677734, 0.11382412910461426, 0.10513877868652344, 0.10673165321350098, 0.10632586479187012, 0.11356449127197266, 0.12400412559509277, 0.10650897026062012, 0.10611915588378906, 0.10608053207397461, 0.11641979217529297, 0.10627126693725586, 0.10639834403991699, 0.10467100143432617, 0.10522794723510742, 0.11039209365844727, 0.10552763938903809, 0.10469865798950195, 0.10643434524536133, 0.10768461227416992, 0.11855053901672363, 0.10504746437072754, 0.10602355003356934, 0.10533022880554199, 0.10479235649108887, 0.1108248233795166, 0.11541628837585449, 0.11372923851013184, 0.10990428924560547, 0.11153078079223633, 0.11603951454162598, 0.11307168006896973, 0.10673260688781738, 0.10897064208984375, 0.11504411697387695, 0.10712027549743652, 0.10655474662780762, 0.1050252914428711, 0.10611438751220703, 0.11178994178771973, 0.10480594635009766, 0.10627031326293945, 0.10665059089660645, 0.16060113906860352, 0.11077308654785156, 0.10663294792175293, 0.10689949989318848, 0.1170499324798584, 0.1067349910736084, 0.10539531707763672, 0.10685849189758301, 0.10664844512939453, 0.11145305633544922, 0.10665607452392578, 0.10622310638427734, 0.10649847984313965, 0.10543012619018555, 0.11934852600097656, 0.10714340209960938, 0.10883045196533203, 0.10854911804199219, 0.11207270622253418, 0.10644364356994629, 0.10694169998168945, 0.10898351669311523, 0.10724067687988281, 0.1122133731842041, 0.10652613639831543, 0.10497760772705078, 0.10682868957519531, 0.11818289756774902, 0.10564756393432617, 0.12193608283996582, 0.1062154769897461, 0.1050405502319336, 0.10502123832702637, 0.11469459533691406, 0.1080629825592041, 0.10632586479187012, 0.12079811096191406, 0.10567831993103027, 0.10631895065307617, 0.1067190170288086, 0.1135549545288086, 0.11075329780578613, 0.10736751556396484, 0.1075899600982666, 0.10642838478088379, 0.11332249641418457, 0.10675668716430664, 0.10842227935791016, 0.11011552810668945, 0.12220120429992676, 0.10862040519714355, 0.10935568809509277, 0.11038708686828613, 0.11016654968261719, 0.11032748222351074, 0.10701990127563477, 0.12176108360290527, 0.10576367378234863, 0.10582613945007324, 0.10630488395690918, 0.10666990280151367, 0.11025333404541016, 0.10580062866210938, 0.11177873611450195, 0.12189245223999023, 0.10569381713867188, 0.10609006881713867, 0.10512876510620117, 0.11116886138916016, 0.12314081192016602, 0.11196660995483398, 0.11575722694396973, 0.11895418167114258, 0.10843920707702637, 0.10605955123901367, 0.10556983947753906, 0.10737895965576172, 0.1145932674407959, 0.10588836669921875, 0.11244797706604004, 0.10745859146118164, 0.10604071617126465, 0.10517048835754395, 0.1044459342956543, 0.11450386047363281, 0.10684442520141602, 0.10658407211303711, 0.11513352394104004, 0.1117091178894043, 0.11199831962585449, 0.12445330619812012, 0.11499381065368652, 0.1122434139251709, 0.1264193058013916, 0.12730932235717773, 0.12529349327087402, 0.11873507499694824, 0.1174461841583252, 0.1393451690673828, 0.1193990707397461, 0.12043285369873047, 0.13120126724243164, 0.12480878829956055, 0.12083244323730469, 0.11831831932067871, 0.12603163719177246, 0.1232457160949707, 0.11862611770629883, 0.1176145076751709, 0.11559844017028809, 0.1060330867767334, 0.10819864273071289, 0.12186241149902344, 0.10645508766174316, 0.10672760009765625, 0.10765242576599121, 0.10945749282836914, 0.11222267150878906, 0.10533809661865234, 0.10638833045959473, 0.10816478729248047, 0.11338949203491211, 0.10587430000305176, 0.1074988842010498, 0.10922718048095703, 0.11469507217407227, 0.11472368240356445, 0.12709403038024902, 0.11568975448608398, 0.10645627975463867, 0.107940673828125, 0.11896848678588867, 0.10715961456298828, 0.10719513893127441, 0.10765886306762695, 0.11862468719482422, 0.11249876022338867, 0.10681605339050293, 0.10918879508972168, 0.1060335636138916, 0.10721445083618164, 0.11261487007141113, 0.10999298095703125, 0.10688042640686035, 0.10672211647033691, 0.10867786407470703, 0.1185152530670166, 0.11048531532287598, 0.11000490188598633, 0.10956406593322754, 0.13668370246887207, 0.10560369491577148, 0.10818791389465332, 0.11121225357055664, 0.11641979217529297, 0.10963225364685059, 0.12767267227172852, 0.10701489448547363, 0.10628652572631836, 0.11038494110107422, 0.10541963577270508, 0.10625815391540527, 0.1305828094482422, 0.12189221382141113, 0.11304354667663574, 0.1100916862487793, 0.11210393905639648, 0.10540342330932617, 0.10632467269897461, 0.1071467399597168, 0.11079216003417969, 0.1160280704498291, 0.10789060592651367, 0.1082758903503418, 0.10880494117736816, 0.11653566360473633, 0.11729264259338379, 0.11710762977600098, 0.10473346710205078, 0.10713076591491699, 0.10749220848083496, 0.11327075958251953, 0.10550713539123535, 0.1054525375366211, 0.11412286758422852, 0.10777831077575684, 0.10803771018981934, 0.11300778388977051, 0.11233901977539062, 0.1053779125213623, 0.10721468925476074, 0.11255860328674316, 0.11711907386779785, 0.10734081268310547, 0.10557842254638672, 0.1070857048034668, 0.11792325973510742, 0.11961102485656738, 0.10739398002624512, 0.10803699493408203, 0.11563968658447266, 0.11114358901977539, 0.1068112850189209, 0.10760140419006348, 0.1313621997833252, 0.10744762420654297, 0.1254889965057373, 0.12391304969787598, 0.11114978790283203, 0.11931514739990234, 0.12293219566345215, 0.12149167060852051, 0.12079119682312012, 0.1134331226348877, 0.11901092529296875, 0.10654163360595703, 0.1074378490447998, 0.10802912712097168, 0.11419081687927246, 0.10747432708740234, 0.10733819007873535, 0.1079246997833252, 0.11101341247558594, 0.11515498161315918, 0.1239619255065918, 0.1098780632019043, 0.11065244674682617, 0.10864877700805664, 0.10815048217773438, 0.11495304107666016, 0.10913825035095215, 0.11362123489379883, 0.10967493057250977, 0.1218729019165039, 0.10994744300842285, 0.10838818550109863, 0.11128783226013184, 0.10933089256286621, 0.1168818473815918, 0.11636948585510254, 0.11617422103881836, 0.10721731185913086, 0.10915851593017578, 0.10710644721984863, 0.11420893669128418, 0.10768890380859375, 0.1322014331817627, 0.11560726165771484, 0.10821962356567383, 0.10685420036315918, 0.11032509803771973, 0.12002396583557129, 0.10799551010131836, 0.11792874336242676, 0.10890579223632812, 0.10764026641845703, 0.10804128646850586, 0.10640549659729004, 0.1239938735961914, 0.13056135177612305, 0.12096309661865234, 0.110198974609375, 0.11455798149108887, 0.10753083229064941, 0.1158902645111084, 0.10842633247375488, 0.1172325611114502, 0.1144101619720459, 0.1157064437866211, 0.12257647514343262, 0.13672780990600586, 0.12660884857177734, 0.11906719207763672, 0.13137245178222656, 0.11115431785583496, 0.1304638385772705, 0.13650822639465332, 0.1102454662322998, 0.13528227806091309, 0.10953807830810547, 0.10642600059509277, 0.10959148406982422, 0.11979317665100098, 0.10614800453186035, 0.10827398300170898, 0.10983514785766602, 0.11116576194763184, 0.10832476615905762, 0.1074676513671875, 0.10876965522766113, 0.10787010192871094, 0.11562180519104004, 0.10993289947509766, 0.10845470428466797, 0.10752487182617188, 0.11443877220153809, 0.11186957359313965, 0.1084139347076416, 0.11197161674499512, 0.11063337326049805, 0.1075594425201416, 0.10846257209777832, 0.11847901344299316, 0.10891890525817871, 0.10860610008239746, 0.10556435585021973, 0.10876703262329102, 0.11238980293273926, 0.10806846618652344, 0.1079244613647461, 0.11084556579589844, 0.10771894454956055, 0.11108994483947754, 0.10794782638549805, 0.10783219337463379, 0.10630965232849121, 0.11569952964782715, 0.10991692543029785, 0.10766482353210449, 0.10891175270080566, 0.1079254150390625, 0.1143341064453125, 0.10546541213989258, 0.10733151435852051, 0.10767245292663574, 0.11240768432617188, 0.10770678520202637, 0.10734415054321289, 0.10843634605407715, 0.10716795921325684, 0.1165776252746582, 0.10836672782897949, 0.10911107063293457, 0.10609245300292969, 0.11840581893920898, 0.1124880313873291, 0.11107635498046875, 0.11166858673095703, 0.11137819290161133, 0.11272335052490234, 0.10935616493225098, 0.11048674583435059, 0.11068248748779297, 0.13133955001831055, 0.11212372779846191, 0.10676765441894531, 0.1064612865447998, 0.10923957824707031, 0.11320972442626953, 0.10922646522521973, 0.10890817642211914, 0.10662722587585449, 0.12362170219421387, 0.11983251571655273, 0.10802912712097168, 0.11069607734680176, 0.10831856727600098, 0.11886739730834961, 0.10873103141784668, 0.11725091934204102, 0.11462521553039551, 0.11383771896362305, 0.11801695823669434, 0.1150822639465332, 0.12459969520568848, 0.11460518836975098, 0.11482405662536621, 0.11613845825195312, 0.11437845230102539, 0.11279559135437012, 0.10796546936035156, 0.12047076225280762, 0.11396145820617676, 0.11291122436523438, 0.10715055465698242, 0.10938739776611328, 0.1074528694152832, 0.10854792594909668, 0.12188315391540527, 0.10576605796813965, 0.1078805923461914, 0.1075139045715332, 0.11864042282104492, 0.10700821876525879, 0.10736727714538574, 0.15187644958496094, 0.12033486366271973, 0.10785627365112305, 0.10921049118041992, 0.11268925666809082, 0.1209559440612793, 0.10620951652526855, 0.10831785202026367, 0.1080927848815918, 0.11817431449890137, 0.10728979110717773, 0.10899615287780762, 0.10946774482727051, 0.1098482608795166, 0.11464738845825195, 0.10784578323364258, 0.10817337036132812, 0.10889458656311035, 0.11301445960998535, 0.10932183265686035, 0.10854434967041016, 0.10918378829956055, 0.10802817344665527, 0.11618685722351074, 0.1084299087524414, 0.10890984535217285, 0.10809564590454102, 0.1127786636352539, 0.10732269287109375, 0.10973620414733887, 0.10726213455200195, 0.1079874038696289, 0.11291384696960449, 0.10804176330566406, 0.10608148574829102, 0.1092994213104248, 0.11028194427490234, 0.11611723899841309, 0.10841798782348633, 0.10877370834350586, 0.11640691757202148, 0.1063835620880127, 0.10736584663391113, 0.10663199424743652, 0.1119832992553711, 0.10701704025268555, 0.10808181762695312, 0.10900497436523438, 0.10824918746948242, 0.11659526824951172, 0.10868215560913086, 0.10688018798828125, 0.10816192626953125, 0.11387038230895996, 0.10800862312316895, 0.10854506492614746, 0.10767960548400879, 0.10770583152770996, 0.12213516235351562, 0.10832762718200684, 0.1088094711303711, 0.1070864200592041, 0.12161660194396973, 0.1085047721862793, 0.10961246490478516, 0.10889077186584473, 0.10889434814453125, 0.11555218696594238, 0.10738730430603027, 0.10758066177368164, 0.10713839530944824, 0.11147832870483398, 0.10873770713806152, 0.10811471939086914, 0.1070394515991211, 0.10717439651489258, 0.11657214164733887, 0.1074361801147461, 0.10787653923034668, 0.108306884765625, 0.11648297309875488, 0.10840892791748047, 0.10909891128540039, 0.10666584968566895, 0.1070108413696289, 0.11199760437011719, 0.10667681694030762, 0.10858559608459473, 0.10584115982055664, 0.10595893859863281, 0.11534500122070312, 0.10747194290161133, 0.10807204246520996, 0.10858631134033203, 0.11464548110961914, 0.10763049125671387, 0.10784149169921875, 0.1078941822052002, 0.10905909538269043, 0.11163997650146484, 0.1261272430419922, 0.10796666145324707, 0.10618782043457031, 0.10774874687194824, 0.10704779624938965, 0.11948060989379883, 0.10553979873657227, 0.10587263107299805, 0.10742831230163574, 0.11025834083557129, 0.1088097095489502, 0.10573983192443848, 0.10577178001403809, 0.10617423057556152, 0.11562657356262207, 0.10705924034118652, 0.10726141929626465, 0.16536617279052734, 0.16063976287841797, 0.13835716247558594, 0.1301860809326172, 0.1210169792175293, 0.12018394470214844, 0.13150477409362793, 0.13352394104003906, 0.12186574935913086, 0.1177818775177002, 0.12333893775939941, 0.13026738166809082, 0.12846803665161133, 0.11663126945495605, 0.12165069580078125, 0.12369227409362793, 0.12218952178955078, 0.1253185272216797, 0.12572431564331055, 0.12923979759216309, 0.13138890266418457, 0.12881183624267578, 0.15955591201782227, 0.148942232131958, 0.14244794845581055, 0.12487006187438965, 0.1341707706451416, 0.14045000076293945, 0.12925148010253906, 0.12439632415771484, 0.1477358341217041, 0.12650394439697266, 0.1253516674041748, 0.14292407035827637, 0.13779449462890625, 0.12422060966491699, 0.1259448528289795, 0.12317872047424316, 0.1326279640197754, 0.12207603454589844, 0.11808252334594727, 0.11799263954162598, 0.1240231990814209, 0.11504173278808594, 0.1180262565612793, 0.1180410385131836, 0.12654352188110352, 0.15258383750915527, 0.18613672256469727, 0.2227492332458496, 0.15734219551086426, 0.13495707511901855, 0.11619687080383301, 0.10671091079711914, 0.10840225219726562, 0.10713386535644531, 0.10815906524658203, 0.12284111976623535, 0.10799312591552734, 0.10558700561523438, 0.10652422904968262, 0.11102104187011719, 0.10894632339477539, 0.12168025970458984, 0.10704994201660156, 0.10772824287414551, 0.11408877372741699, 0.1077125072479248, 0.10773277282714844, 0.10708212852478027, 0.11310243606567383, 0.10882139205932617, 0.11733603477478027, 0.10861349105834961, 0.11932992935180664, 0.11074233055114746, 0.11942720413208008, 0.10933589935302734, 0.12028241157531738, 0.17216992378234863, 0.20332956314086914, 0.19658160209655762, 0.2577245235443115, 0.10950160026550293, 0.21419906616210938, 0.18052363395690918, 0.10595488548278809, 0.10842752456665039, 0.10601520538330078, 0.10495615005493164, 0.10628104209899902, 0.10529232025146484, 0.10570812225341797, 0.10559296607971191, 0.10688304901123047, 0.10541200637817383, 0.10514378547668457, 0.10682272911071777, 0.1068418025970459, 0.1066901683807373, 0.10518789291381836, 0.10624003410339355, 0.10541105270385742, 0.10575532913208008, 0.10534143447875977, 0.10601162910461426, 0.10639524459838867, 0.10528802871704102, 0.10525393486022949, 0.10567378997802734, 0.10512995719909668, 0.10687470436096191, 0.10732507705688477, 0.10548233985900879, 0.10588645935058594, 0.10599064826965332, 0.10704588890075684, 0.10553312301635742, 0.1065206527709961, 0.10507535934448242, 0.10642075538635254, 0.10647845268249512, 0.10650634765625, 0.10600423812866211, 0.10452914237976074, 0.10565042495727539, 0.10605525970458984, 0.1047677993774414, 0.10478711128234863, 0.1051795482635498, 0.1063683032989502, 0.10558700561523438, 0.1052558422088623, 0.10623979568481445, 0.10630321502685547, 0.10566449165344238, 0.10604023933410645, 0.10554671287536621, 0.10650229454040527, 0.10533428192138672, 0.10480928421020508, 0.10487818717956543, 0.10660171508789062, 0.10649800300598145, 0.10612154006958008, 0.10530900955200195, 0.1058967113494873, 0.1063985824584961, 0.1053164005279541, 0.1065359115600586, 0.10539746284484863, 0.10604071617126465, 0.10635614395141602, 0.10584664344787598, 0.10634922981262207, 0.10632085800170898, 0.10543489456176758, 0.10461831092834473, 0.10568022727966309, 0.10574960708618164, 0.10572266578674316, 0.10599827766418457, 0.10577559471130371, 0.10630011558532715, 0.10627913475036621, 0.10567641258239746, 0.10520815849304199, 0.10523033142089844, 0.10609316825866699, 0.10493922233581543, 0.10596966743469238, 0.10626554489135742, 0.10550117492675781, 0.106658935546875, 0.10613870620727539, 0.10530638694763184, 0.10492634773254395, 0.10634469985961914, 0.10685968399047852, 0.10530591011047363, 0.1049966812133789, 0.10488200187683105, 0.10584831237792969, 0.10618758201599121, 0.1050872802734375, 0.10460972785949707, 0.10497069358825684, 0.10606718063354492, 0.1057281494140625, 0.10564422607421875, 0.10500311851501465, 0.1055457592010498, 0.10692834854125977, 0.10602593421936035, 0.10639691352844238, 0.10608100891113281, 0.10613489151000977, 0.10631370544433594, 0.10609722137451172, 0.10613465309143066, 0.10574507713317871, 0.10654759407043457, 0.10628890991210938, 0.10647296905517578, 0.10567927360534668, 0.10477948188781738, 0.10703492164611816, 0.1062173843383789, 0.10582852363586426, 0.10510373115539551, 0.1051931381225586, 0.10693359375, 0.10515522956848145, 0.10679292678833008, 0.10614824295043945, 0.10486340522766113, 0.10487174987792969, 0.10593271255493164, 0.10569357872009277, 0.10597014427185059, 0.10492253303527832, 0.1049339771270752, 0.1054222583770752, 0.1059417724609375, 0.1064293384552002, 0.10655546188354492, 0.10631108283996582, 0.10557031631469727, 0.10573434829711914, 0.10544562339782715, 0.10639071464538574, 0.10514616966247559, 0.10582900047302246, 0.10567808151245117, 0.10527944564819336, 0.10595941543579102, 0.10521531105041504, 0.10489296913146973, 0.10460305213928223, 0.10559415817260742, 0.10593390464782715, 0.10589313507080078, 0.10613656044006348, 0.1053769588470459, 0.10660886764526367, 0.10470151901245117, 0.10523486137390137, 0.10468459129333496, 0.10493779182434082, 0.1058809757232666, 0.10522341728210449, 0.10553884506225586, 0.10586905479431152, 0.10719442367553711, 0.10560059547424316, 0.10604381561279297, 0.10539984703063965, 0.10548090934753418, 0.10640501976013184, 0.10593676567077637, 0.10560846328735352, 0.10540390014648438, 0.10564446449279785, 0.10605740547180176, 0.10526657104492188, 0.10590386390686035, 0.10628628730773926, 0.10528922080993652, 0.10615015029907227, 0.10605716705322266, 0.10612225532531738, 0.10611248016357422, 0.10609745979309082, 0.10517334938049316, 0.10590171813964844, 0.1065518856048584, 0.10607600212097168, 0.10604548454284668, 0.1054983139038086, 0.10514140129089355, 0.10472702980041504, 0.10529708862304688, 0.10630178451538086, 0.10655474662780762, 0.10565328598022461, 0.10561752319335938, 0.10577106475830078, 0.10649251937866211, 0.10650110244750977, 0.10492563247680664, 0.10642695426940918, 0.1062161922454834, 0.105316162109375, 0.10540199279785156, 0.1066286563873291, 0.10668826103210449, 0.10560774803161621, 0.10565447807312012, 0.10706329345703125, 0.1047217845916748, 0.10630488395690918, 0.10721635818481445, 0.10640549659729004, 0.10772442817687988, 0.10497188568115234, 0.10614180564880371, 0.10600709915161133, 0.10648131370544434, 0.10445880889892578, 0.10657310485839844, 0.10492396354675293, 0.10499429702758789, 0.10613608360290527, 0.1056966781616211, 0.10524415969848633, 0.10633635520935059, 0.10625529289245605, 0.10576963424682617, 0.10628032684326172, 0.10526180267333984, 0.10544586181640625, 0.11056232452392578, 0.10689043998718262, 0.10558772087097168, 0.10602474212646484, 0.1059565544128418, 0.1053156852722168, 0.10606932640075684, 0.10629129409790039, 0.10628604888916016, 0.10615706443786621, 0.1050882339477539, 0.1060326099395752, 0.10613870620727539, 0.10639691352844238, 0.10518598556518555, 0.10581445693969727, 0.10479426383972168, 0.10574173927307129, 0.1049342155456543, 0.10596990585327148, 0.10652399063110352, 0.10600614547729492, 0.10670924186706543, 0.10508441925048828, 0.10707688331604004, 0.10558772087097168, 0.10653185844421387, 0.10634875297546387, 0.10609793663024902, 0.10614156723022461, 0.10521554946899414, 0.1064915657043457, 0.10718560218811035, 0.10579228401184082, 0.10632610321044922, 0.10625243186950684, 0.10630941390991211, 0.10538387298583984, 0.10594558715820312, 0.10607481002807617, 0.10605430603027344, 0.13513612747192383, 0.12302994728088379, 0.10854864120483398, 0.10640358924865723, 0.10642790794372559, 0.10657668113708496, 0.11163091659545898, 0.10679888725280762, 0.10685324668884277, 0.1151893138885498, 0.10572648048400879, 0.10569620132446289, 0.10625123977661133, 0.10564732551574707, 0.10601282119750977, 0.10664081573486328, 0.10711002349853516, 0.10645461082458496, 0.10627532005310059, 0.10550427436828613, 0.10568761825561523, 0.10526895523071289, 0.10682058334350586, 0.1049501895904541, 0.10651493072509766, 0.1067514419555664, 0.10639762878417969, 0.11741113662719727, 0.10570621490478516, 0.10812115669250488, 0.10640692710876465, 0.10724020004272461, 0.1080162525177002, 0.10634374618530273, 0.10698485374450684, 0.10518288612365723, 0.10561275482177734, 0.10550332069396973, 0.10494446754455566, 0.1055145263671875, 0.10590314865112305, 0.10665512084960938, 0.10607409477233887, 0.10633206367492676, 0.10666728019714355, 0.10559773445129395, 0.10704398155212402, 0.10602569580078125, 0.10642147064208984, 0.10590600967407227, 0.1054682731628418, 0.10682010650634766, 0.10578155517578125, 0.10561275482177734, 0.10618185997009277, 0.10625004768371582, 0.10556149482727051, 0.10611319541931152, 0.105743408203125, 0.10553669929504395, 0.1063385009765625, 0.10619187355041504, 0.10603976249694824, 0.10562729835510254, 0.10604429244995117, 0.10586857795715332, 0.1061398983001709, 0.10514307022094727, 0.10606789588928223, 0.1049051284790039, 0.10556244850158691, 0.10634326934814453, 0.1063997745513916, 0.1047666072845459, 0.10470962524414062, 0.10649561882019043, 0.1069645881652832, 0.1069173812866211, 0.10524940490722656, 0.10693740844726562, 0.10619497299194336, 0.10526084899902344, 0.10565996170043945, 0.10453557968139648, 0.10631513595581055, 0.10563778877258301, 0.10657525062561035, 0.10596704483032227, 0.10628247261047363, 0.10597419738769531, 0.1052865982055664, 0.10661602020263672, 0.10601639747619629, 0.10633420944213867, 0.10523080825805664, 0.10525131225585938, 0.10487651824951172, 0.10621500015258789, 0.10624980926513672, 0.1063230037689209, 0.10625743865966797, 0.10630917549133301, 0.10483813285827637, 0.10501241683959961, 0.10443568229675293, 0.10565590858459473, 0.10506534576416016, 0.1064901351928711, 0.10467648506164551, 0.10501527786254883, 0.10667943954467773, 0.1053316593170166, 0.10688591003417969, 0.10664534568786621, 0.10518383979797363, 0.10623431205749512, 0.10479426383972168, 0.1061851978302002, 0.10634779930114746, 0.10550904273986816, 0.10523414611816406, 0.1053164005279541, 551.6725962162018]
t_14_none = [0.11796903610229492, 0.12370467185974121, 0.10464310646057129, 0.103729248046875, 0.1046607494354248, 0.10493659973144531, 0.14478111267089844, 0.1044759750366211, 0.10430240631103516, 0.10653495788574219, 0.1045377254486084, 0.1037590503692627, 0.1035764217376709, 0.1039888858795166, 0.10408401489257812, 0.10368490219116211, 0.1043236255645752, 0.10356712341308594, 0.10416460037231445, 0.1034390926361084, 0.10392427444458008, 0.10356783866882324, 0.10402894020080566, 0.10452079772949219, 0.10390210151672363, 0.10367703437805176, 0.10471510887145996, 0.10487771034240723, 0.10502362251281738, 0.10397601127624512, 0.10425925254821777, 0.10419607162475586, 0.10450959205627441, 0.10430669784545898, 0.10437226295471191, 0.1043548583984375, 0.10431957244873047, 0.1049191951751709, 0.10452723503112793, 0.10390806198120117, 0.10443758964538574, 0.10492300987243652, 0.10450148582458496, 0.1043403148651123, 0.10411286354064941, 0.10440516471862793, 0.1040964126586914, 0.1045694351196289, 0.1040334701538086, 0.10508441925048828, 0.10409879684448242, 0.10441923141479492, 0.10527992248535156, 0.10448765754699707, 0.10425519943237305, 0.10398411750793457, 0.10433721542358398, 0.1046750545501709, 0.10443115234375, 0.10441446304321289, 0.10431885719299316, 0.10413813591003418, 0.10417318344116211, 0.10415935516357422, 0.10434603691101074, 0.10392069816589355, 0.10402226448059082, 0.10395097732543945, 0.10446858406066895, 0.10369062423706055, 0.10426807403564453, 0.10428547859191895, 0.10421419143676758, 0.10391688346862793, 0.10425662994384766, 0.10393190383911133, 0.10399222373962402, 0.10427594184875488, 0.10427188873291016, 0.10399651527404785, 0.1043097972869873, 0.10395956039428711, 0.10442733764648438, 0.10456109046936035, 0.10462808609008789, 0.10549330711364746, 0.10503602027893066, 0.10441350936889648, 0.10412001609802246, 0.10378217697143555, 0.10398292541503906, 0.1041264533996582, 0.1041707992553711, 0.10439252853393555, 0.1037912368774414, 0.10510063171386719, 0.1040656566619873, 0.10475969314575195, 0.10425209999084473, 0.10437560081481934, 0.10504269599914551, 0.10405158996582031, 0.10478377342224121, 0.10451817512512207, 0.10402584075927734, 0.10456514358520508, 0.10412430763244629, 0.1042637825012207, 0.10416054725646973, 0.10422086715698242, 0.10419464111328125, 0.10401535034179688, 0.10471701622009277, 0.10403585433959961, 0.1132969856262207, 0.10431718826293945, 0.10460686683654785, 0.10399293899536133, 0.10446715354919434, 0.10460495948791504, 0.10431694984436035, 0.10440826416015625, 0.10440635681152344, 0.10428667068481445, 0.10456466674804688, 0.10423064231872559, 0.10423517227172852, 0.10403132438659668, 0.10453486442565918, 0.10427737236022949, 0.1041707992553711, 0.1044764518737793, 0.10457587242126465, 0.10436677932739258, 0.10391807556152344, 0.10432815551757812, 0.10414648056030273, 0.10392951965332031, 0.10390210151672363, 0.10380196571350098, 0.10418057441711426, 0.10403966903686523, 0.10428261756896973, 0.10427331924438477, 0.10389971733093262, 0.10511612892150879, 0.10377788543701172, 0.10457086563110352, 0.10541415214538574, 0.1046304702758789, 0.1041259765625, 0.104248046875, 0.10480380058288574, 0.10663843154907227, 0.10484504699707031, 0.10401129722595215, 0.10399127006530762, 0.10457587242126465, 0.10449695587158203, 0.10412073135375977, 0.10437321662902832, 0.10415267944335938, 0.10451960563659668, 0.10384798049926758, 0.10454130172729492, 0.10390400886535645, 0.10418272018432617, 0.10461997985839844, 0.10446000099182129, 0.10420083999633789, 0.10424113273620605, 0.10404658317565918, 0.1043391227722168, 0.10418057441711426, 0.10437440872192383, 0.10396528244018555, 0.10416388511657715, 0.1041266918182373, 0.10485696792602539, 0.10459327697753906, 0.10410475730895996, 0.10453319549560547, 0.10454726219177246, 0.10425353050231934, 0.1040496826171875, 0.10421562194824219, 0.10367798805236816, 0.10426187515258789, 0.10404515266418457, 0.10461568832397461, 0.10421872138977051, 0.1044764518737793, 0.1041102409362793, 0.10521197319030762, 0.10413813591003418, 0.1047520637512207, 0.10519957542419434, 0.10455918312072754, 0.10491943359375, 0.10445785522460938, 0.10428261756896973, 0.10413837432861328, 0.10450124740600586, 0.10414886474609375, 0.1043694019317627, 0.10442423820495605, 0.10416769981384277, 0.10394787788391113, 0.10444259643554688, 0.10435152053833008, 0.10440444946289062, 0.10375809669494629, 0.10428285598754883, 0.1040184497833252, 0.10427522659301758, 0.10450196266174316, 0.10411453247070312, 0.10431718826293945, 0.10447812080383301, 0.10416555404663086, 0.10409927368164062, 0.10428786277770996, 0.1043553352355957, 0.1038668155670166, 0.10403990745544434, 0.10420012474060059, 0.1038050651550293, 0.10444784164428711, 0.10422778129577637, 0.10439276695251465, 0.10401558876037598, 0.10407447814941406, 0.10393357276916504, 0.10415291786193848, 0.10382938385009766, 0.10418295860290527, 0.1039578914642334, 0.10454106330871582, 0.10460996627807617, 0.10441136360168457, 0.10401773452758789, 0.10498261451721191, 0.10396647453308105, 0.10451912879943848, 0.10491037368774414, 0.10447001457214355, 0.10414505004882812, 0.10404586791992188, 0.10397100448608398, 0.10433459281921387, 0.10421085357666016, 0.10397744178771973, 0.10406661033630371, 0.1040956974029541, 0.10402607917785645, 0.10422658920288086, 0.10439300537109375, 0.10407447814941406, 0.10427498817443848, 0.1035456657409668, 0.10454654693603516, 0.10384011268615723, 0.1042022705078125, 0.10436892509460449, 0.1035456657409668, 0.10414266586303711, 0.10424327850341797, 0.10461711883544922, 0.10405373573303223, 0.10375070571899414, 0.10350227355957031, 0.10356688499450684, 0.10438704490661621, 0.10414648056030273, 0.10421919822692871, 0.10408139228820801, 0.10428524017333984, 0.10478639602661133, 0.10476875305175781, 0.10439872741699219, 0.10390329360961914, 0.1043393611907959, 0.10376238822937012, 0.10403680801391602, 0.10412812232971191, 0.10438942909240723, 0.10431694984436035, 0.10432910919189453, 0.10396456718444824, 0.1049203872680664, 0.10389208793640137, 0.1041100025177002, 0.10531425476074219, 0.10456418991088867, 0.10468721389770508, 0.1040506362915039, 0.10498976707458496, 0.10513639450073242, 0.1044013500213623, 0.10381221771240234, 0.10408759117126465, 0.10413789749145508, 0.10413837432861328, 0.10399651527404785, 0.10425829887390137, 0.10376954078674316, 0.10395622253417969, 0.10385441780090332, 0.1040656566619873, 0.10353827476501465, 0.10427045822143555, 0.10422992706298828, 0.10403895378112793, 0.10405302047729492, 0.10404396057128906, 0.10397672653198242, 0.1043100357055664, 0.10396838188171387, 0.10415363311767578, 0.10358548164367676, 0.10422801971435547, 0.10421133041381836, 0.10433340072631836, 0.10433292388916016, 0.11036014556884766, 0.10469532012939453, 0.10426902770996094, 0.10420918464660645, 0.10419845581054688, 0.10440325736999512, 0.10447192192077637, 0.10422086715698242, 0.10460090637207031, 0.10480451583862305, 0.10430264472961426, 0.10449790954589844, 0.10406970977783203, 0.10514283180236816, 0.1039576530456543, 0.1047508716583252, 0.10493612289428711, 0.10457086563110352, 0.1046452522277832, 0.10422158241271973, 0.10425329208374023, 0.1044621467590332, 0.10468316078186035, 0.10491466522216797, 0.10435962677001953, 0.10458493232727051, 0.10437750816345215, 0.10442066192626953, 0.10471248626708984, 0.10415434837341309, 0.10420393943786621, 0.10407328605651855, 0.10494804382324219, 0.10416746139526367, 0.10463452339172363, 0.10424423217773438, 0.10428929328918457, 0.1043999195098877, 0.10428786277770996, 0.10454630851745605, 0.10415458679199219, 0.10467720031738281, 0.10413503646850586, 0.1041100025177002, 0.10446834564208984, 0.10415005683898926, 0.10478806495666504, 0.10438346862792969, 0.10493969917297363, 0.10465097427368164, 0.10422921180725098, 0.10437917709350586, 0.10418152809143066, 0.10423946380615234, 0.10462141036987305, 0.10448861122131348, 0.1043558120727539, 0.10445690155029297, 0.10503458976745605, 0.10468649864196777, 0.10489296913146973, 0.10524296760559082, 0.10468220710754395, 0.10554361343383789, 0.10407114028930664, 0.10508370399475098, 0.10860252380371094, 0.10456371307373047, 0.10452485084533691, 0.10416293144226074, 0.10406351089477539, 0.104034423828125, 0.10403060913085938, 0.10405731201171875, 0.1045691967010498, 0.10428404808044434, 0.10432887077331543, 0.10464787483215332, 0.10450458526611328, 0.10457754135131836, 0.1047201156616211, 0.10434603691101074, 0.10458731651306152, 0.1045222282409668, 0.10459375381469727, 0.10472822189331055, 0.10422778129577637, 0.10458588600158691, 0.10450267791748047, 0.1044776439666748, 0.10441994667053223, 0.10436773300170898, 0.10458636283874512, 0.10449051856994629, 0.10451841354370117, 0.1045522689819336, 0.1044149398803711, 0.10428047180175781, 0.10420536994934082, 0.10451006889343262, 0.10433483123779297, 0.10466432571411133, 0.10416531562805176, 0.10461926460266113, 0.10459685325622559, 0.10447311401367188, 0.10431385040283203, 0.10459589958190918, 0.10514020919799805, 0.10506391525268555, 0.1040036678314209, 0.10582828521728516, 0.1042625904083252, 0.10453295707702637, 0.10462617874145508, 0.10539078712463379, 0.10607409477233887, 0.10494852066040039, 0.10458827018737793, 0.10452747344970703, 0.10439276695251465, 0.1041250228881836, 0.10451912879943848, 0.1046290397644043, 0.10430502891540527, 0.10423803329467773, 0.104278564453125, 0.1043398380279541, 0.10455632209777832, 0.1044163703918457, 0.1042485237121582, 0.10426568984985352, 0.10410070419311523, 0.10463571548461914, 0.10973095893859863, 0.10439491271972656, 0.1045534610748291, 0.1042628288269043, 0.10443401336669922, 0.10409188270568848, 0.10451984405517578, 0.10491108894348145, 0.10473299026489258, 0.10460829734802246, 0.10467195510864258, 0.1047050952911377, 0.10428261756896973, 0.10413932800292969, 0.1042490005493164, 0.1046750545501709, 0.1041712760925293, 0.10437178611755371, 0.10466670989990234, 0.10444998741149902, 0.11060166358947754, 0.10450506210327148, 0.10545635223388672, 0.10563516616821289, 0.10464215278625488, 0.10601806640625, 0.10463404655456543, 0.1050257682800293, 0.10488581657409668, 0.10452723503112793, 0.10470128059387207, 0.10451889038085938, 0.10466599464416504, 0.10430693626403809, 0.10468053817749023, 0.10451126098632812, 0.10440683364868164, 0.1047525405883789, 0.10416054725646973, 0.10474467277526855, 0.10457777976989746, 0.10433101654052734, 0.10450506210327148, 0.10434389114379883, 0.10508418083190918, 0.10458827018737793, 0.10447120666503906, 0.10448122024536133, 0.10450387001037598, 0.1045827865600586, 0.10445070266723633, 0.10436105728149414, 0.10420894622802734, 0.10446763038635254, 0.10443687438964844, 0.10427212715148926, 0.10447120666503906, 0.10427260398864746, 0.10456204414367676, 0.10426497459411621, 0.10415315628051758, 0.10441780090332031, 0.10421013832092285, 0.10420036315917969, 0.1046590805053711, 0.10435366630554199, 0.10453104972839355, 0.10430908203125, 0.10411214828491211, 0.10430479049682617, 0.10574841499328613, 0.10431194305419922, 0.10461020469665527, 0.10535907745361328, 0.10483646392822266, 0.10438799858093262, 0.10452890396118164, 0.10401368141174316, 0.10442638397216797, 0.1044304370880127, 0.10408854484558105, 0.10423064231872559, 0.10444927215576172, 0.10421609878540039, 0.10428524017333984, 0.10453176498413086, 0.10431814193725586, 0.10459256172180176, 0.10446691513061523, 0.10454392433166504, 0.10434603691101074, 0.1041421890258789, 0.10476469993591309, 0.10463666915893555, 0.10466861724853516, 0.10441923141479492, 0.10500025749206543, 0.10418033599853516, 0.10451245307922363, 0.10427021980285645, 0.10420584678649902, 0.1043248176574707, 0.10455965995788574, 0.10482454299926758, 0.10431265830993652, 0.10436058044433594, 0.10484743118286133, 0.10471224784851074, 0.10444474220275879, 0.10425162315368652, 0.1043403148651123, 0.10446929931640625, 0.10416197776794434, 0.10413956642150879, 0.10489535331726074, 0.10463690757751465, 0.10434246063232422, 0.10448813438415527, 0.10553693771362305, 0.1048891544342041, 0.10439538955688477, 0.1058499813079834, 0.10452008247375488, 0.10480856895446777, 0.10425972938537598, 0.10693073272705078, 0.1050727367401123, 0.10454416275024414, 0.10424470901489258, 0.10443639755249023, 0.10445761680603027, 0.10434603691101074, 0.10436153411865234, 0.10418176651000977, 0.10444211959838867, 0.10428762435913086, 0.1045076847076416, 0.1042013168334961, 0.10444927215576172, 0.10458850860595703, 0.10488319396972656, 0.10450220108032227, 0.10449075698852539, 0.10409879684448242, 0.10462689399719238, 0.1043233871459961, 0.10428214073181152, 0.10406994819641113, 0.10431909561157227, 0.1044304370880127, 0.10442972183227539, 0.10429549217224121, 0.10438704490661621, 0.10489392280578613, 0.10426998138427734, 0.10431289672851562, 0.10432171821594238, 0.10413765907287598, 0.10519814491271973, 0.10465526580810547, 0.10456514358520508, 0.10449433326721191, 0.10437917709350586, 0.1043553352355957, 0.10442638397216797, 0.10541820526123047, 0.1045529842376709, 0.1043248176574707, 0.10507369041442871, 0.10477018356323242, 0.1042938232421875, 0.10458636283874512, 0.1041269302368164, 0.10420513153076172, 0.10458111763000488, 0.10416936874389648, 0.1047048568725586, 0.10471272468566895, 0.10438942909240723, 0.1042168140411377, 0.10427641868591309, 0.10422301292419434, 0.10431098937988281, 0.10405230522155762, 0.10430335998535156, 0.1043558120727539, 0.10431957244873047, 0.10410094261169434, 0.10451078414916992, 0.10422563552856445, 0.10474348068237305, 0.10428619384765625, 0.10415887832641602, 0.10424518585205078, 0.10438680648803711, 0.10424590110778809, 0.1043555736541748, 0.10427165031433105, 0.10437583923339844, 0.10535573959350586, 0.10436868667602539, 0.10482430458068848, 0.10477375984191895, 0.10421156883239746, 0.10438299179077148, 0.10434198379516602, 0.10446524620056152, 0.10451340675354004, 0.10424518585205078, 0.10435199737548828, 0.1042633056640625, 0.10521197319030762, 0.10415339469909668, 0.10513162612915039, 0.10447406768798828, 0.10438227653503418, 0.1056511402130127, 0.10434985160827637, 0.10460710525512695, 0.10469794273376465, 0.10455513000488281, 0.10417532920837402, 0.10411286354064941, 0.10425996780395508, 0.1042931079864502, 0.10432147979736328, 0.10451269149780273, 0.10457015037536621, 0.10444474220275879, 0.10464215278625488, 0.10483169555664062, 0.10415101051330566, 0.10445976257324219, 0.10445928573608398, 0.10445690155029297, 0.10425877571105957, 0.1045842170715332, 0.10437178611755371, 0.10454225540161133, 0.10436129570007324, 0.10454344749450684, 0.10435056686401367, 0.10458850860595703, 0.10416674613952637, 0.10418486595153809, 0.10447192192077637, 0.10412454605102539, 0.10412025451660156, 0.10456037521362305, 0.10450005531311035, 0.10408544540405273, 0.10437369346618652, 0.10425448417663574, 0.10418176651000977, 0.10433816909790039, 0.10417699813842773, 0.10429215431213379, 0.10457754135131836, 0.10393595695495605, 0.10450506210327148, 0.10438275337219238, 0.10479187965393066, 0.10404300689697266, 0.10419344902038574, 0.10522055625915527, 0.10432600975036621, 0.10441875457763672, 0.10465168952941895, 0.10450625419616699, 0.10566449165344238, 0.10510563850402832, 0.10431051254272461, 0.10440278053283691, 0.10432982444763184, 0.1045999526977539, 0.10441446304321289, 0.10416936874389648, 0.10426831245422363, 0.10428333282470703, 0.10424017906188965, 0.10462641716003418, 0.10401129722595215, 0.10454678535461426, 0.10430216789245605, 0.10478496551513672, 0.10422205924987793, 0.10426831245422363, 0.10588192939758301, 0.1043391227722168, 0.10457539558410645, 0.10436630249023438, 0.10448622703552246, 0.10453033447265625, 0.1044623851776123, 0.10473179817199707, 0.10449075698852539, 0.1042017936706543, 0.10465455055236816, 0.10478401184082031, 0.10461091995239258, 0.10438275337219238, 0.10433316230773926, 0.1040499210357666, 0.10432791709899902, 0.10429668426513672, 0.104766845703125, 0.10409212112426758, 0.10419702529907227, 0.10430335998535156, 0.10508227348327637, 0.1047666072845459, 0.10399127006530762, 0.10527825355529785, 0.10429191589355469, 0.10500335693359375, 0.10432815551757812, 0.10466814041137695, 0.10457110404968262, 0.10443377494812012, 0.104522705078125, 0.1045069694519043, 0.10447359085083008, 0.10447072982788086, 0.10446882247924805, 0.1045989990234375, 0.10442614555358887, 0.1047053337097168, 0.1043100357055664, 0.10451698303222656, 0.10400128364562988, 0.10436487197875977, 0.10460209846496582, 0.10440993309020996, 0.10426998138427734, 0.10439920425415039, 0.10457754135131836, 0.10433769226074219, 0.10454416275024414, 0.10447168350219727, 0.10408592224121094, 0.10410189628601074, 0.10433125495910645, 0.1041719913482666, 0.10441446304321289, 0.1043252944946289, 0.10461735725402832, 0.10433578491210938, 0.10431432723999023, 0.10426998138427734, 0.10461616516113281, 0.10409045219421387, 0.10446715354919434, 0.10426688194274902, 0.10445189476013184, 0.1051628589630127, 0.1041719913482666, 0.10407352447509766, 0.10505342483520508, 0.10427546501159668, 0.10411500930786133, 0.10554194450378418, 0.10424280166625977, 0.10419678688049316, 0.10472607612609863, 0.1043403148651123, 0.10455489158630371, 0.10450220108032227, 0.10436058044433594, 0.1042482852935791, 0.10448431968688965, 0.10465526580810547, 0.10414648056030273, 0.10462594032287598, 0.10426783561706543, 0.10459613800048828, 0.1043233871459961, 0.10456585884094238, 0.10412716865539551, 0.10436487197875977, 0.10477256774902344, 0.10429763793945312, 0.10418438911437988, 0.1045374870300293, 0.10471558570861816, 0.10440278053283691, 0.10477495193481445, 0.10414528846740723, 0.1041707992553711, 0.10406613349914551, 0.10418510437011719, 0.10469651222229004, 0.10468077659606934, 0.10391783714294434, 0.10477256774902344, 0.10462403297424316, 0.10459661483764648, 0.10416340827941895, 0.10411763191223145, 0.10388517379760742, 0.10426855087280273, 0.10416197776794434, 0.10445499420166016, 0.10443878173828125, 0.10423016548156738, 0.10467123985290527, 0.10556674003601074, 0.10460233688354492, 0.10417294502258301, 0.10521531105041504, 0.10481142997741699, 0.10474705696105957, 0.10425710678100586, 0.10450148582458496, 0.10583972930908203, 0.10482645034790039, 0.10423421859741211, 0.1044614315032959, 0.1042184829711914, 0.1043100357055664, 0.10404610633850098, 0.1045231819152832, 0.10407590866088867, 0.1043088436126709, 0.10416984558105469, 0.10473465919494629, 0.10423421859741211, 0.10429716110229492, 0.1045682430267334, 0.10425925254821777, 0.10431432723999023, 0.10465741157531738, 0.10433721542358398, 0.10469388961791992, 0.10413646697998047, 0.10456204414367676, 0.10405826568603516, 0.10429835319519043, 0.1042013168334961, 0.1039590835571289, 0.10423421859741211, 0.10437607765197754, 0.10460329055786133, 0.10388851165771484, 0.10423493385314941, 0.10431766510009766, 0.1042940616607666, 0.10432720184326172, 0.1040647029876709, 0.10409021377563477, 0.10460782051086426, 0.10455656051635742, 0.1042184829711914, 0.10396194458007812, 0.10488510131835938, 0.10431623458862305, 0.10438966751098633, 0.10565853118896484, 0.10436606407165527, 0.1045374870300293, 0.10397052764892578, 0.10438895225524902, 0.10426902770996094, 0.10449361801147461, 0.10466194152832031, 0.10448050498962402, 0.10441422462463379, 0.1044778823852539, 0.10396766662597656, 0.10446286201477051, 0.10426545143127441, 0.10438370704650879, 0.10405230522155762, 0.10463738441467285, 0.10426139831542969, 0.10416531562805176, 0.10482263565063477, 0.10449743270874023, 0.1040194034576416, 0.10423517227172852, 0.10441136360168457, 0.10409188270568848, 0.10462117195129395, 0.10409021377563477, 0.10420083999633789, 0.1044778823852539, 0.10444164276123047, 0.10482072830200195, 0.10469579696655273, 0.10412931442260742, 0.10492920875549316, 0.10416984558105469, 0.10434937477111816, 0.10413146018981934, 0.1041715145111084, 0.10405206680297852, 0.10452008247375488, 0.10408639907836914, 0.10436606407165527, 0.10503649711608887, 0.10419917106628418, 0.10414671897888184, 0.10501861572265625, 0.10420632362365723, 0.10435938835144043, 0.10606884956359863, 0.10469174385070801, 0.10446381568908691, 0.10390019416809082, 0.10456705093383789, 0.10411238670349121, 0.10461091995239258, 0.10391497611999512, 0.10439133644104004, 0.10425186157226562, 0.1045539379119873, 0.10422992706298828, 0.10454487800598145, 0.1040496826171875, 0.10429573059082031, 0.10416030883789062, 0.10463762283325195, 0.10423111915588379, 0.10413432121276855, 0.1045985221862793, 0.10442638397216797, 0.1046133041381836, 0.10429143905639648, 0.10424137115478516, 0.1042029857635498, 0.10456418991088867, 0.10428380966186523, 0.1044461727142334, 0.10425615310668945, 0.10420632362365723, 0.10451793670654297, 0.10463595390319824, 0.1040184497833252, 0.10466551780700684, 0.10416245460510254, 0.1045680046081543, 0.10404157638549805, 0.10452151298522949, 0.10457491874694824, 0.1044156551361084, 0.1045529842376709, 0.1041557788848877, 0.10423469543457031, 0.10429525375366211, 0.10429048538208008, 0.10481715202331543, 0.10440850257873535, 0.1048135757446289, 0.1058511734008789, 0.10430240631103516, 0.10429048538208008, 0.10440325736999512, 0.10621023178100586, 0.10452556610107422, 0.10475873947143555, 0.10403037071228027, 0.10458850860595703, 0.10427260398864746, 0.10442566871643066, 0.10333800315856934, 0.1043548583984375, 0.1055147647857666, 0.1040191650390625, 0.10439777374267578, 0.1043398380279541, 0.1045997142791748, 0.10477066040039062, 0.10478019714355469, 0.10451245307922363, 0.10472917556762695, 0.10438990592956543, 0.10452842712402344, 0.10442733764648438, 0.10482645034790039, 0.1039881706237793, 0.1042940616607666, 0.10457849502563477, 0.10454607009887695, 0.10497879981994629, 0.10448956489562988, 0.10467314720153809, 0.10477209091186523, 0.10429906845092773, 0.10430526733398438, 0.10414981842041016, 0.10459733009338379, 0.1044473648071289, 0.10426974296569824, 0.10442209243774414, 0.10434508323669434, 0.10987162590026855, 0.10486054420471191, 0.10522294044494629, 0.10455679893493652, 0.10534882545471191, 0.10555553436279297, 0.10493659973144531, 0.10502028465270996, 0.10463500022888184, 0.1047215461730957, 0.10490131378173828, 0.1045224666595459, 0.10468411445617676, 0.10406875610351562, 0.10451173782348633, 0.10464668273925781, 0.10473871231079102, 0.10447907447814941, 0.10460686683654785, 0.10560941696166992, 0.10467767715454102, 0.10472607612609863, 0.10430073738098145, 0.10454678535461426, 0.10478067398071289, 0.10444951057434082, 0.10473895072937012, 0.10454607009887695, 0.10405182838439941, 0.10446834564208984, 0.10456705093383789, 0.10484719276428223, 0.10419845581054688, 0.10450863838195801, 0.10460877418518066, 0.10454940795898438, 0.10471892356872559, 0.10479021072387695, 0.10444164276123047, 0.10436606407165527, 0.1044461727142334, 0.10440993309020996, 0.10430240631103516, 0.10420107841491699, 0.10461282730102539, 0.10461306571960449, 0.10440826416015625, 0.10402369499206543, 0.10453462600708008, 0.10408496856689453, 0.10517621040344238, 0.10463070869445801, 0.10430765151977539, 0.10551261901855469, 0.10430407524108887, 0.1043403148651123, 0.10478973388671875, 0.10409235954284668, 0.10418581962585449, 0.1042625904083252, 0.10452103614807129, 0.10429883003234863, 0.10445475578308105, 0.1046140193939209, 0.10441184043884277, 0.10457444190979004, 0.10422992706298828, 0.10457134246826172, 0.1041712760925293, 0.10456252098083496, 0.10432004928588867, 0.10434937477111816, 0.10436749458312988, 0.1042177677154541, 0.10421466827392578, 0.10479998588562012, 0.1047661304473877, 0.1043691635131836, 0.10428071022033691, 0.1044776439666748, 0.10433387756347656, 0.1045074462890625, 0.10468935966491699, 0.10489344596862793, 0.10468363761901855, 0.1049351692199707, 0.10489201545715332, 0.10487222671508789, 0.10521650314331055, 0.10452961921691895, 0.10477113723754883, 0.1046299934387207, 0.10459637641906738, 0.10435867309570312, 0.10501503944396973, 0.10496330261230469, 0.10512328147888184, 0.10456204414367676, 0.13446426391601562, 0.10471725463867188, 0.10482025146484375, 0.10603189468383789, 0.10474371910095215, 0.10535907745361328, 0.10444021224975586, 0.10597872734069824, 0.10566902160644531, 0.1058354377746582, 0.10486960411071777, 0.10507702827453613, 0.10458207130432129, 0.10496878623962402, 0.1048581600189209, 0.10529613494873047, 0.10454988479614258, 0.10485363006591797, 0.10481715202331543, 0.10513901710510254, 0.10448408126831055, 0.1049947738647461, 0.10519886016845703, 0.10492515563964844, 0.10495662689208984, 0.10454559326171875, 0.10484862327575684, 0.10454964637756348, 0.10485172271728516, 0.10461759567260742, 0.10488462448120117, 0.10475659370422363, 0.10498285293579102, 0.10500407218933105, 0.10515356063842773, 0.10481381416320801, 0.10511636734008789, 0.10482549667358398, 0.10526037216186523, 0.10532760620117188, 0.1049494743347168, 0.1047203540802002, 0.10524630546569824, 0.10485291481018066, 0.10461878776550293, 0.10470128059387207, 0.10444760322570801, 0.10514140129089355, 0.10535502433776855, 0.10484004020690918, 0.10555505752563477, 0.10488128662109375, 0.10478568077087402, 0.10487198829650879, 0.10456037521362305, 0.10477828979492188, 0.10494875907897949, 0.10466742515563965, 0.10509681701660156, 0.10483670234680176, 0.10462498664855957, 0.10493755340576172, 0.10439753532409668, 0.10483312606811523, 0.10491609573364258, 0.10472297668457031, 0.10471844673156738, 0.1045982837677002, 0.10478019714355469, 0.10472607612609863, 0.10506081581115723, 0.10442066192626953, 0.10518407821655273, 0.10457062721252441, 0.10465073585510254, 0.10445308685302734, 0.10499858856201172, 0.10467529296875, 0.1048424243927002, 0.10482001304626465, 0.1049184799194336, 0.1048588752746582, 0.10521554946899414, 0.10500073432922363, 0.10709524154663086, 0.10460329055786133, 0.10476064682006836, 0.10490989685058594, 0.10461735725402832, 0.10499906539916992, 0.10445570945739746, 0.10479903221130371, 0.10523724555969238, 0.1050112247467041, 0.10453963279724121, 0.10552644729614258, 0.1047356128692627, 0.1051478385925293, 0.10583925247192383, 0.10529017448425293, 0.10466504096984863, 0.10500454902648926, 0.10481595993041992, 0.10493230819702148, 0.10509848594665527, 0.10464692115783691, 0.10466170310974121, 0.10474109649658203, 0.10471034049987793, 0.10568690299987793, 0.10510706901550293, 0.10472297668457031, 0.10549688339233398, 0.10460662841796875, 0.10530924797058105, 0.10457062721252441, 0.10496711730957031, 0.1051483154296875, 0.10507559776306152, 0.10468816757202148, 0.10470724105834961, 0.10467529296875, 0.10545897483825684, 0.10445594787597656, 0.10497236251831055, 0.10463595390319824, 0.10472965240478516, 0.10486173629760742, 0.10509896278381348, 0.10491657257080078, 0.10434722900390625, 0.10475873947143555, 0.10459613800048828, 0.10500502586364746, 0.10478830337524414, 0.10473108291625977, 0.10464668273925781, 0.10541367530822754, 0.10524129867553711, 0.10495305061340332, 0.10510063171386719, 0.1047677993774414, 0.10475730895996094, 0.10579371452331543, 0.10465669631958008, 0.10487580299377441, 0.10556554794311523, 0.10462045669555664, 0.10475540161132812, 0.10448622703552246, 0.10648369789123535, 0.10562658309936523, 0.10535526275634766, 0.10466122627258301, 0.1050419807434082, 0.1046910285949707, 0.10457229614257812, 0.10560798645019531, 0.10497164726257324, 0.10437130928039551, 0.10456228256225586, 0.10457968711853027, 0.10544729232788086, 0.10453248023986816, 0.1052238941192627, 0.10476350784301758, 0.10472869873046875, 0.10495877265930176, 0.10476946830749512, 0.10464715957641602, 0.10472226142883301, 0.10481882095336914, 0.1046898365020752, 0.10459542274475098, 0.10448575019836426, 0.10484528541564941, 0.10508298873901367, 0.10485124588012695, 0.10523414611816406, 0.10540199279785156, 0.10488700866699219, 0.10517024993896484, 0.10465335845947266, 0.10482478141784668, 0.10454106330871582, 0.10478901863098145, 0.10488677024841309, 0.10490751266479492, 0.10491752624511719, 0.10476064682006836, 0.10586142539978027, 0.10525012016296387, 0.10484576225280762, 0.10575127601623535, 0.10479187965393066, 0.10545730590820312, 0.10486650466918945, 0.10478711128234863, 0.10495138168334961, 0.10482358932495117, 0.10491180419921875, 0.10460686683654785, 0.1047215461730957, 0.10495352745056152, 0.10485386848449707, 0.10471248626708984, 0.10473918914794922, 0.10513997077941895, 0.10502266883850098, 0.10452985763549805, 0.10459733009338379, 0.10478568077087402, 0.1052546501159668, 0.10495781898498535, 0.10457348823547363, 0.10493731498718262, 0.10462379455566406, 0.1048424243927002, 0.10469675064086914, 0.10485625267028809, 0.10433340072631836, 0.10489606857299805, 0.10499382019042969, 0.10469484329223633, 0.10483860969543457, 0.10480570793151855, 0.10458064079284668, 0.10494518280029297, 0.10478520393371582, 0.10419321060180664, 0.10445046424865723, 0.10464072227478027, 0.10468339920043945, 0.10482597351074219, 0.10467529296875, 0.10517334938049316, 0.10484457015991211, 0.10427451133728027, 0.10583162307739258, 0.10441470146179199, 0.1045219898223877, 0.10572481155395508, 0.10498833656311035, 0.10456633567810059, 0.10497164726257324, 0.10418581962585449, 0.10428738594055176, 0.10485696792602539, 0.10461688041687012, 0.1043398380279541, 0.10464191436767578, 0.10447502136230469, 0.10483860969543457, 0.1049191951751709, 0.10470747947692871, 0.10460400581359863, 0.10460138320922852, 0.10516858100891113, 0.10450983047485352, 0.10451245307922363, 0.10511970520019531, 0.10507893562316895, 0.10507392883300781, 0.10544419288635254, 0.10481595993041992, 0.10453486442565918, 0.10464620590209961, 0.1045987606048584, 0.10465478897094727, 0.10465049743652344, 0.10495924949645996, 0.10524177551269531, 0.1052560806274414, 0.10445952415466309, 0.10515761375427246, 0.10470819473266602, 0.10459470748901367, 0.10468101501464844, 0.1047358512878418, 0.10444498062133789, 0.10491323471069336, 0.10449481010437012, 0.10468220710754395, 0.1048893928527832, 0.1047813892364502, 0.10438323020935059, 0.10599184036254883, 0.10445213317871094, 0.10483098030090332, 0.1054837703704834, 0.10519742965698242, 0.10452461242675781, 0.10472440719604492, 0.10578751564025879, 0.10529804229736328, 0.10491275787353516, 0.10466146469116211, 0.10463976860046387, 0.10453391075134277, 0.10480046272277832, 0.10457921028137207, 0.10480618476867676, 0.10486412048339844, 0.10503363609313965, 0.1043853759765625, 0.10528016090393066, 0.10457158088684082, 0.10493946075439453, 0.10505485534667969, 0.10489892959594727, 0.10466885566711426, 0.10467910766601562, 0.10512185096740723, 0.10499715805053711, 0.10461044311523438, 0.10453057289123535, 0.10466313362121582, 0.10457015037536621, 0.10463070869445801, 0.10461664199829102, 0.10479426383972168, 0.10499024391174316, 0.10468268394470215, 0.10460305213928223, 0.10469937324523926, 0.10448098182678223, 0.10455918312072754, 0.10510396957397461, 0.10479211807250977, 0.10462021827697754, 0.10497856140136719, 0.10456252098083496, 0.10468411445617676, 0.10513544082641602, 0.10478019714355469, 0.10437583923339844, 0.10562825202941895, 0.10494875907897949, 0.10474109649658203, 0.10485029220581055, 0.1046910285949707, 0.10435676574707031, 0.1049354076385498, 0.1048583984375, 0.1051023006439209, 0.10464358329772949, 0.1046748161315918, 0.10470104217529297, 0.10438895225524902, 0.10475444793701172, 0.10443663597106934, 0.10492372512817383, 0.10500884056091309, 0.10461258888244629, 0.10470938682556152, 0.10483741760253906, 0.10466361045837402, 0.10481810569763184, 0.10481500625610352, 0.10507321357727051, 0.10443401336669922, 0.104705810546875, 0.10488700866699219, 0.10450506210327148, 0.10449790954589844, 0.10475301742553711, 0.10454368591308594, 0.10514998435974121, 0.1051187515258789, 0.1046445369720459, 0.10504651069641113, 0.1042170524597168, 0.10454535484313965, 0.1045989990234375, 0.10465407371520996, 0.10406064987182617, 0.10417747497558594, 0.10464286804199219, 0.10495972633361816, 0.10487532615661621, 0.10475754737854004, 0.10481691360473633, 0.10469961166381836, 0.10463237762451172, 0.1060645580291748, 0.10455489158630371, 0.10540056228637695, 0.10503172874450684, 0.1052253246307373, 0.10513138771057129, 0.10446429252624512, 0.10492229461669922, 0.10448741912841797, 0.1047067642211914, 0.10487699508666992, 0.10477113723754883, 0.10564184188842773, 0.10466957092285156, 0.1050105094909668, 0.10497498512268066, 0.1051640510559082, 0.10466313362121582, 0.10447835922241211, 0.10487055778503418, 0.10466980934143066, 0.10474991798400879, 0.1047663688659668, 0.10482048988342285, 0.10505461692810059, 0.10457634925842285, 0.10483002662658691, 0.10439395904541016, 0.10471630096435547, 0.1047360897064209, 0.10518717765808105, 0.10483884811401367, 0.1045992374420166, 0.10440969467163086, 0.10474538803100586, 0.10441827774047852, 0.10425209999084473, 0.10443735122680664, 0.10456109046936035, 0.10471391677856445, 0.10466623306274414, 0.10486841201782227, 0.1045689582824707, 0.10463404655456543, 0.1046602725982666, 0.10526371002197266, 0.10464882850646973, 0.10485148429870605, 0.10646629333496094, 0.1049201488494873, 0.10454940795898438, 0.1055760383605957, 0.10582780838012695, 0.10606932640075684, 0.10540223121643066, 0.10470461845397949, 0.10492825508117676, 0.10510087013244629, 0.10500693321228027, 0.10493063926696777, 0.10507512092590332, 0.1049964427947998, 0.10461759567260742, 0.10511541366577148, 0.10507917404174805, 0.10483121871948242, 0.10473942756652832, 0.10483431816101074, 0.10491347312927246, 0.10458111763000488, 0.10508275032043457, 0.1047053337097168, 0.10480880737304688, 0.10473179817199707, 0.10482311248779297, 0.10445785522460938, 0.10454320907592773, 0.10464787483215332, 0.1052711009979248, 0.10507369041442871, 0.10483789443969727, 0.10511612892150879, 0.10471701622009277, 0.10481858253479004, 0.1046912670135498, 0.10450220108032227, 0.10472297668457031, 0.10456109046936035, 0.10480260848999023, 0.10488128662109375, 0.11932826042175293, 0.10494565963745117, 0.10529446601867676, 0.10568881034851074, 0.10499238967895508, 0.10570240020751953, 0.10485458374023438, 0.10540032386779785, 0.10509109497070312, 0.10494828224182129, 0.10490727424621582, 0.10481500625610352, 0.10470080375671387, 0.10499763488769531, 0.10467958450317383, 0.10520195960998535, 0.10488700866699219, 0.10480427742004395, 0.10499739646911621, 0.10521721839904785, 0.10495638847351074, 0.10467052459716797, 0.1047675609588623, 0.10478758811950684, 0.1051785945892334, 0.10443401336669922, 0.10487008094787598, 0.10486698150634766, 0.10516667366027832, 0.10459160804748535, 0.10462665557861328, 0.10497856140136719, 0.10454702377319336, 0.1048429012298584, 0.10522603988647461, 0.10464596748352051, 0.10447883605957031, 0.10473227500915527, 0.10504364967346191, 0.10489392280578613, 0.10446596145629883, 0.10489869117736816, 0.10434842109680176, 0.10450029373168945, 0.10453915596008301, 0.10480380058288574, 0.1050107479095459, 0.10480952262878418, 0.1047511100769043, 0.10474944114685059, 0.10520577430725098, 0.10444927215576172, 0.10456299781799316, 0.10603904724121094, 0.10479855537414551, 0.10466766357421875, 0.10483336448669434, 0.1045083999633789, 0.10476207733154297, 0.10464763641357422, 0.10462760925292969, 0.10437250137329102, 0.10462617874145508, 0.10491418838500977, 0.10461974143981934, 0.10475516319274902, 0.10445666313171387, 0.10463118553161621, 0.10455036163330078, 0.10513901710510254, 0.1045987606048584, 0.10466957092285156, 0.10485410690307617, 0.10476469993591309, 0.10457515716552734, 0.10465240478515625, 0.10492491722106934, 0.10430550575256348, 0.10478472709655762, 0.10483598709106445, 0.10462546348571777, 0.10451698303222656, 0.10489511489868164, 0.10506677627563477, 0.1050870418548584, 0.10468602180480957, 0.10498380661010742, 0.10534381866455078, 0.10456371307373047, 0.1042637825012207, 0.10435891151428223, 0.10453248023986816, 0.10481977462768555, 0.1045694351196289, 0.10488057136535645, 0.10470342636108398, 0.10460162162780762, 0.10488772392272949, 0.10563087463378906, 0.10450315475463867, 0.10424566268920898, 0.1056969165802002, 0.10485959053039551, 0.10485696792602539, 0.10481810569763184, 0.10548019409179688, 0.1067361831665039, 0.10511064529418945, 0.10474181175231934, 0.10468745231628418, 0.10476112365722656, 0.10454058647155762, 0.1050865650177002, 0.10465645790100098, 0.1047825813293457, 0.10471653938293457, 0.10453271865844727, 0.1050264835357666, 0.10468053817749023, 0.10483121871948242, 0.10495495796203613, 0.10436892509460449, 0.10471343994140625, 0.10500645637512207, 0.10547423362731934, 0.10476922988891602, 0.10506105422973633, 0.10437989234924316, 0.1047525405883789, 0.1046907901763916, 0.1047067642211914, 0.10440802574157715, 0.10489892959594727, 0.10517525672912598, 0.10448312759399414, 0.10448789596557617, 0.10476851463317871, 0.10446906089782715, 0.10863280296325684, 0.1058492660522461, 0.10473799705505371, 0.10636782646179199, 0.1048424243927002, 0.10517096519470215, 0.10451650619506836, 0.10549545288085938, 0.1051177978515625, 0.10484194755554199, 0.10553383827209473, 0.105224609375, 0.10474014282226562, 0.10449099540710449, 0.1045987606048584, 0.10428833961486816, 0.10478353500366211, 0.1046440601348877, 0.10511016845703125, 0.10501456260681152, 0.10452461242675781, 0.10474324226379395, 0.10471367835998535, 0.1045987606048584, 0.10468411445617676, 0.10475587844848633, 0.10462689399719238, 0.10509586334228516, 0.10484528541564941, 0.1050267219543457, 0.10463666915893555, 0.10463762283325195, 0.10477185249328613, 0.10450363159179688, 0.10453367233276367, 0.10460233688354492, 0.10484027862548828, 0.10458016395568848, 0.10459351539611816, 0.10446286201477051, 0.10464954376220703, 0.1052863597869873, 0.10484528541564941, 0.10499978065490723, 0.10499429702758789, 0.10471034049987793, 0.1050407886505127, 0.10474228858947754, 0.10449433326721191, 0.1049048900604248, 0.1046595573425293, 0.1050882339477539, 0.10456180572509766, 0.1054544448852539, 0.10463190078735352, 0.1055440902709961, 0.10461974143981934, 0.10463213920593262, 0.10594558715820312, 0.10468506813049316, 0.10534477233886719, 0.10475993156433105, 0.10507607460021973, 0.1045229434967041, 0.10478019714355469, 0.10488462448120117, 0.10472512245178223, 0.10437917709350586, 0.10493659973144531, 0.10436224937438965, 0.10540962219238281, 0.10454869270324707, 0.10504817962646484, 0.10475969314575195, 0.10471320152282715, 0.10460186004638672, 0.10461950302124023, 0.10478615760803223, 0.10456514358520508, 0.10472917556762695, 0.10502076148986816, 0.10513806343078613, 0.10491180419921875, 0.10440278053283691, 0.10507845878601074, 0.10510444641113281, 0.10462474822998047, 0.10472393035888672, 0.10461306571960449, 0.10462546348571777, 0.10460782051086426, 0.10506844520568848, 0.10480713844299316, 0.10458135604858398, 0.10464239120483398, 0.10445094108581543, 0.10449743270874023, 0.10480213165283203, 0.10457086563110352, 0.10543346405029297, 0.10507583618164062, 0.10449838638305664, 0.10471343994140625, 0.10602045059204102, 0.10470104217529297, 0.10471320152282715, 0.10626721382141113, 0.10506248474121094, 0.1052546501159668, 0.10511469841003418, 0.10632181167602539, 0.10601139068603516, 0.10498476028442383, 0.10454154014587402, 0.10463547706604004, 0.10440564155578613, 0.10482096672058105, 0.1045694351196289, 0.10459423065185547, 0.10464930534362793, 0.10467410087585449, 0.10465860366821289, 0.10507750511169434, 0.10435271263122559, 0.10461854934692383, 0.1049797534942627, 0.10498166084289551, 0.10467243194580078, 0.10460162162780762, 0.1044766902923584, 0.10471940040588379, 0.10487604141235352, 0.10432219505310059, 0.10486268997192383, 0.10446906089782715, 0.10457730293273926, 0.10517024993896484, 0.10483765602111816, 0.1049346923828125, 0.10488343238830566, 0.10511469841003418, 0.10434937477111816, 0.10452389717102051, 0.1045832633972168, 0.10419464111328125, 0.10482907295227051, 0.10440206527709961, 0.10475564002990723, 0.10490059852600098, 0.1045691967010498, 0.10492920875549316, 0.1052093505859375, 0.10489940643310547, 0.10521268844604492, 0.10568594932556152, 0.10558938980102539, 0.10510110855102539, 0.10433030128479004, 0.10478377342224121, 0.1043550968170166, 0.10479998588562012, 0.10440325736999512, 0.10440993309020996, 0.10491681098937988, 0.10477876663208008, 0.10486531257629395, 0.10464048385620117, 0.10549592971801758, 0.10451579093933105, 0.10436177253723145, 0.10460925102233887, 0.10496664047241211, 0.10449910163879395, 0.10493993759155273, 0.10456109046936035, 0.10507559776306152, 0.10470390319824219, 0.10461544990539551, 0.1049187183380127, 0.10476469993591309, 0.10488271713256836, 0.10448741912841797, 0.10468912124633789, 0.10495972633361816, 0.10468292236328125, 0.10494494438171387, 0.10469174385070801, 0.10451817512512207, 0.10450935363769531, 0.10449004173278809, 0.10457563400268555, 0.1045384407043457, 0.10468363761901855, 0.10442566871643066, 0.10466337203979492, 0.10495543479919434, 0.10549020767211914, 0.1044611930847168, 0.1051483154296875, 0.10490298271179199, 0.10481762886047363, 0.10582733154296875, 0.10454607009887695, 0.10461020469665527, 0.10452461242675781, 0.10455465316772461, 0.10491323471069336, 0.10502171516418457, 0.10469198226928711, 0.10477995872497559, 0.10487151145935059, 0.10483026504516602, 0.10474205017089844, 0.10517644882202148, 0.10486030578613281, 0.10513806343078613, 0.10474348068237305, 0.10504674911499023, 0.10423660278320312, 0.1045677661895752, 0.10521531105041504, 0.10474371910095215, 0.1046607494354248, 0.10495829582214355, 0.1048576831817627, 0.10474085807800293, 0.10463142395019531, 0.1045827865600586, 0.10447192192077637, 0.10462117195129395, 0.10473227500915527, 0.10480117797851562, 0.10480046272277832, 0.10455632209777832, 0.10511231422424316, 0.10508513450622559, 0.10505437850952148, 0.1044762134552002, 0.10476255416870117, 0.10455155372619629, 0.10482048988342285, 0.10462141036987305, 0.10500192642211914, 0.10481071472167969, 0.10462427139282227, 0.10432910919189453, 0.10538911819458008, 0.10441923141479492, 0.10458827018737793, 0.10550951957702637, 0.10530304908752441, 0.10493946075439453, 0.10505199432373047, 0.10473394393920898, 0.10637617111206055, 0.10508441925048828, 0.1045389175415039, 0.1045083999633789, 0.10445690155029297, 0.10453033447265625, 0.10454750061035156, 0.10484695434570312, 0.1045372486114502, 0.10445141792297363, 0.10454893112182617, 0.10489916801452637, 0.10380220413208008, 0.10423588752746582, 0.10447454452514648, 0.10432267189025879, 0.10442280769348145, 0.10461688041687012, 0.10522103309631348, 0.1049034595489502, 0.10462355613708496, 0.10479354858398438, 0.10446977615356445, 0.10449481010437012, 0.10478639602661133, 0.10467886924743652, 0.10554003715515137, 0.10484051704406738, 0.1049494743347168, 0.1043086051940918, 0.1047525405883789, 0.10458254814147949, 0.10509419441223145, 0.10567927360534668, 0.10611295700073242, 0.10545039176940918, 0.10457491874694824, 0.10548996925354004, 0.10466647148132324, 0.1047813892364502, 0.10585665702819824, 0.10520148277282715, 0.10457777976989746, 0.10576939582824707, 0.1044919490814209, 0.10487651824951172, 0.10481405258178711, 0.10520434379577637, 0.10455775260925293, 0.10480999946594238, 0.10464978218078613, 0.10499000549316406, 0.10451555252075195, 0.10533380508422852, 0.10513949394226074, 0.10505175590515137, 0.10485219955444336, 0.10458087921142578, 0.10455107688903809, 0.1047205924987793, 0.10435152053833008, 0.10473299026489258, 0.10507369041442871, 0.10451984405517578, 0.10485267639160156, 0.10456013679504395, 0.10436439514160156, 0.10438680648803711, 0.10485196113586426, 0.1041555404663086, 0.10463881492614746, 0.1049051284790039, 0.10485363006591797, 0.10508298873901367, 0.10462331771850586, 0.10480284690856934, 0.10488033294677734, 0.1042778491973877, 0.1046600341796875, 0.10438871383666992, 0.1049509048461914, 0.10486078262329102, 0.10451316833496094, 0.10476565361022949, 0.10488414764404297, 0.10539102554321289, 0.10483598709106445, 0.10464978218078613, 0.10500645637512207, 0.10478353500366211, 0.10547566413879395, 0.1048574447631836, 0.10523295402526855, 0.10473823547363281, 0.10439729690551758, 0.10501480102539062, 0.10474014282226562, 0.10474395751953125, 0.10463142395019531, 0.10441851615905762, 0.10457706451416016, 0.10462760925292969, 0.10490655899047852, 0.10485434532165527, 0.10474753379821777, 0.10459733009338379, 0.10499286651611328, 0.10434579849243164, 0.10470795631408691, 0.1047518253326416, 0.10475897789001465, 0.10439634323120117, 0.10513925552368164, 0.1047065258026123, 0.10466265678405762, 0.10451412200927734, 0.10478687286376953, 0.10447001457214355, 0.10467243194580078, 0.10493302345275879, 0.1048727035522461, 0.10432219505310059, 0.10522675514221191, 0.10435748100280762, 0.10457158088684082, 0.1049492359161377, 0.10458517074584961, 0.10480046272277832, 0.10430455207824707, 0.10463547706604004, 0.10435080528259277, 0.10505056381225586, 0.10446453094482422, 0.10433316230773926, 0.10424041748046875, 0.10513687133789062, 0.10427474975585938, 0.10457420349121094, 0.1053318977355957, 0.10416126251220703, 0.10412740707397461, 0.10462641716003418, 0.10443353652954102, 0.10674023628234863, 0.10457062721252441, 0.10451960563659668, 0.1046903133392334, 0.10514068603515625, 0.10442614555358887, 0.10474228858947754, 0.10477447509765625, 0.10497713088989258, 0.10498809814453125, 0.1045072078704834, 0.10499310493469238, 0.10440754890441895, 0.1047976016998291, 0.10477042198181152, 0.10471868515014648, 0.10465264320373535, 0.10477805137634277, 0.10455870628356934, 0.10470318794250488, 0.10436534881591797, 0.10465550422668457, 0.10435080528259277, 0.10466551780700684, 0.10478568077087402, 0.10477399826049805, 0.10526323318481445, 0.10457587242126465, 0.10612845420837402, 0.10478544235229492, 0.1049504280090332, 0.10441946983337402, 0.10475778579711914, 0.10458779335021973, 0.1047067642211914, 0.10449099540710449, 0.1048746109008789, 0.10977983474731445, 0.1134796142578125, 0.10495567321777344, 0.10570693016052246, 0.10468864440917969, 0.1047508716583252, 0.10641098022460938, 0.10520553588867188, 0.10497498512268066, 0.10460567474365234, 0.10492467880249023, 0.10474610328674316, 0.10503625869750977, 0.10450363159179688, 0.10502123832702637, 0.1049196720123291, 0.10555624961853027, 0.1048424243927002, 0.10522580146789551, 0.10476565361022949, 0.10478615760803223, 0.10456705093383789, 0.10493946075439453, 0.10431051254272461, 0.10501623153686523, 0.10449409484863281, 0.10479044914245605, 0.10467410087585449, 0.10455703735351562, 0.10493659973144531, 0.10430526733398438, 0.10492157936096191, 0.10444474220275879, 0.10443782806396484, 0.10444903373718262, 0.10465550422668457, 0.10447812080383301, 0.10493588447570801, 0.10427451133728027, 0.10452675819396973, 0.10437917709350586, 0.10476183891296387, 0.1043083667755127, 0.1045684814453125, 0.10496330261230469, 0.10469198226928711, 0.10452795028686523, 0.10496640205383301, 0.10502505302429199, 0.10424280166625977, 0.10482048988342285, 0.1050715446472168, 0.10460209846496582, 0.10490226745605469, 0.10489940643310547, 0.10422348976135254, 0.10502934455871582, 0.10441446304321289, 0.10463643074035645, 0.10477948188781738, 0.10482478141784668, 0.10451006889343262, 0.10451507568359375, 0.10462093353271484, 0.10478496551513672, 0.1045231819152832, 0.10516929626464844, 0.10459733009338379, 0.10471177101135254, 0.10475707054138184, 0.1048734188079834, 0.10474157333374023, 0.10468316078186035, 0.105133056640625, 0.10454082489013672, 0.10482096672058105, 0.10541510581970215, 0.1049642562866211, 0.10441255569458008, 0.10476088523864746, 0.10430335998535156, 0.10473799705505371, 0.10469198226928711, 0.10498762130737305, 0.10500311851501465, 0.10471630096435547, 0.10516524314880371, 0.10544514656066895, 0.10454177856445312, 0.1046898365020752, 0.10476350784301758, 0.10451054573059082, 0.10477614402770996, 0.10451102256774902, 0.1047976016998291, 0.10442352294921875, 0.10525083541870117, 0.10504674911499023, 0.10552644729614258, 0.10469532012939453, 0.1046442985534668, 0.10585856437683105, 0.10487103462219238, 0.1050717830657959, 0.10485386848449707, 0.10445451736450195, 0.10637521743774414, 0.10520339012145996, 0.10513925552368164, 0.10467314720153809, 0.1049034595489502, 0.10479497909545898, 0.10447049140930176, 0.1044619083404541, 0.10440349578857422, 0.10505938529968262, 0.10499000549316406, 0.10456109046936035, 0.1044158935546875, 0.10456347465515137, 0.10480952262878418, 0.10491085052490234, 0.10461783409118652, 0.10472297668457031, 0.10461282730102539, 0.10544943809509277, 0.10492539405822754, 0.10472345352172852, 0.10449075698852539, 0.10430169105529785, 0.10437583923339844, 0.10494256019592285, 0.1047978401184082, 0.10459399223327637, 0.10543489456176758, 0.10436367988586426, 0.10467696189880371, 0.10450887680053711, 0.10553121566772461, 0.10437536239624023, 0.1045682430267334, 0.10458803176879883, 0.10467052459716797, 0.10497450828552246, 0.10501313209533691, 0.10417819023132324, 0.10595822334289551, 0.10470819473266602, 0.10465359687805176, 0.10583734512329102, 0.10490036010742188, 0.10515594482421875, 0.10486054420471191, 0.10469460487365723, 0.10473871231079102, 0.10470700263977051, 0.10537981986999512, 0.10614180564880371, 0.1046600341796875, 0.10462784767150879, 0.10486268997192383, 0.10501551628112793, 0.10483598709106445, 0.10466289520263672, 0.10479307174682617, 0.10481095314025879, 0.10440611839294434, 0.10467386245727539, 0.10538196563720703, 0.10487771034240723, 0.10459327697753906, 0.10506868362426758, 0.10441350936889648, 0.10457754135131836, 0.10465383529663086, 0.1047525405883789, 0.10449695587158203, 0.10472226142883301, 0.10478711128234863, 0.10508131980895996, 0.1050422191619873, 0.10485219955444336, 0.1062462329864502, 0.1051483154296875, 0.10482478141784668, 0.10476374626159668, 0.10480642318725586, 0.10451340675354004, 0.10487842559814453, 0.10418128967285156, 0.10487747192382812, 0.10508513450622559, 0.1048729419708252, 0.1045072078704834, 0.10536789894104004, 0.10453104972839355, 0.10537147521972656, 0.10530209541320801, 0.10495781898498535, 0.10513639450073242, 0.10473918914794922, 0.10471606254577637, 0.10440969467163086, 0.10457634925842285, 0.1042933464050293, 0.10450077056884766, 0.1044011116027832, 0.1046900749206543, 0.1043233871459961, 0.10470986366271973, 0.10450339317321777, 0.10465788841247559, 0.10460162162780762, 0.10458254814147949, 0.10384654998779297, 0.10434389114379883, 0.10487818717956543, 0.10484695434570312, 0.10452008247375488, 0.10449433326721191, 0.10477900505065918, 0.10483813285827637, 0.1047971248626709, 0.10466122627258301, 0.10514283180236816, 0.1047518253326416, 0.10501432418823242, 0.10443115234375, 0.10503721237182617, 0.10444498062133789, 0.10493874549865723, 0.10448718070983887, 0.10483288764953613, 0.10450458526611328, 0.10463523864746094, 0.10466504096984863, 0.104888916015625, 0.10491466522216797, 0.10475564002990723, 0.10478067398071289, 0.10457706451416016, 0.1048285961151123, 0.10513925552368164, 0.10443472862243652, 0.10567712783813477, 0.1046903133392334, 0.10471606254577637, 0.10514092445373535, 0.10491681098937988, 0.10648107528686523, 0.10519123077392578, 0.10495924949645996, 0.1049799919128418, 0.10461616516113281, 0.10488438606262207, 0.10469317436218262, 0.10494470596313477, 0.1047201156616211, 0.10487556457519531, 0.10494875907897949, 0.10490918159484863, 0.10474300384521484, 0.10453009605407715, 0.10493683815002441, 0.10464620590209961, 0.10445618629455566, 0.10506463050842285, 0.10503959655761719, 0.10499095916748047, 0.1046295166015625, 0.10485672950744629, 0.1046149730682373, 0.10564661026000977, 0.10468721389770508, 0.10508847236633301, 0.1047210693359375, 0.10473251342773438, 0.1053762435913086, 0.10508275032043457, 0.1046748161315918, 0.10454273223876953, 0.10468411445617676, 0.10492372512817383, 0.10447096824645996, 0.10450196266174316, 0.10495758056640625, 0.10471844673156738, 0.1046748161315918, 0.10462570190429688, 0.1055750846862793, 0.10441017150878906, 0.10498237609863281, 0.10559701919555664, 0.1050262451171875, 0.10474705696105957, 0.10472273826599121, 0.10493183135986328, 0.1047811508178711, 0.1046295166015625, 0.10451459884643555, 0.10454988479614258, 0.10479736328125, 0.10483765602111816, 0.10460090637207031, 0.10464763641357422, 0.10511946678161621, 0.10485696792602539, 0.10442161560058594, 0.1046745777130127, 0.10450267791748047, 0.10469555854797363, 0.1048734188079834, 0.10478711128234863, 0.10448431968688965, 0.10501646995544434, 0.10501766204833984, 0.10473394393920898, 0.1048886775970459, 0.10496735572814941, 0.1044313907623291, 0.10459613800048828, 0.10457086563110352, 0.10514163970947266, 0.10491800308227539, 0.1046454906463623, 0.10421276092529297, 0.10467004776000977, 0.10462021827697754, 0.10439705848693848, 0.1045224666595459, 0.10469889640808105, 0.1047360897064209, 0.1045083999633789, 0.1051032543182373, 0.10489320755004883, 0.10454297065734863, 0.1044166088104248, 0.10573482513427734, 0.10454535484313965, 0.10457563400268555, 0.10551595687866211, 0.10478854179382324, 0.10461997985839844, 0.10463094711303711, 0.1045980453491211, 0.1046910285949707, 0.1047966480255127, 0.10463905334472656, 0.10443973541259766, 0.10476851463317871, 0.10472607612609863, 0.10496401786804199, 0.10468935966491699, 0.10446548461914062, 0.1045684814453125, 0.10442566871643066, 0.10491371154785156, 0.1045534610748291, 0.1043848991394043, 0.10500717163085938, 0.10455179214477539, 0.10461258888244629, 0.10467529296875, 0.10520195960998535, 0.10441327095031738, 0.10438704490661621, 0.10450315475463867, 0.10476326942443848, 0.10468006134033203, 0.10440182685852051, 0.10524868965148926, 0.10463094711303711, 0.10498452186584473, 0.10484552383422852, 0.10429525375366211, 0.1046295166015625, 0.10467314720153809, 0.1046593189239502, 0.10432553291320801, 0.10462164878845215, 0.10446739196777344, 0.10517144203186035, 0.10499119758605957, 0.1046149730682373, 0.10481071472167969, 0.10554814338684082, 0.10491633415222168, 0.10528707504272461, 0.10512709617614746, 0.10529923439025879, 0.10477995872497559, 0.10464596748352051, 0.10632562637329102, 0.1050119400024414, 0.10483431816101074, 0.10457777976989746, 0.10472798347473145, 0.10426092147827148, 0.10475277900695801, 0.10419678688049316, 0.10482001304626465, 0.10448956489562988, 0.1047663688659668, 0.1047670841217041, 0.10499858856201172, 0.1044924259185791, 0.10473060607910156, 0.10492801666259766, 0.10469341278076172, 0.10472583770751953, 0.10485029220581055, 0.1050271987915039, 0.10488033294677734, 0.10468387603759766, 0.10453295707702637, 0.10462689399719238, 0.10424613952636719, 0.10503888130187988, 0.10455918312072754, 0.10492253303527832, 0.10480260848999023, 0.10476517677307129, 0.1046915054321289, 0.1047050952911377, 0.10427975654602051, 0.10499691963195801, 0.10454392433166504, 0.10453152656555176, 0.1044321060180664, 0.10458016395568848, 0.10474514961242676, 0.10458874702453613, 0.10544300079345703, 0.10437488555908203, 0.10462808609008789, 0.10560154914855957, 0.10469746589660645, 0.10483837127685547, 0.10498309135437012, 0.10433173179626465, 0.10465717315673828, 0.10477328300476074, 0.10467267036437988, 0.10474109649658203, 0.10486817359924316, 0.10451197624206543, 0.10488700866699219, 0.10495758056640625, 0.10473108291625977, 0.10455918312072754, 0.10466909408569336, 0.10485506057739258, 0.10443949699401855, 0.10450267791748047, 0.10500049591064453, 0.10436105728149414, 0.10457491874694824, 0.10501718521118164, 0.10451698303222656, 0.10459184646606445, 0.10481691360473633, 0.10512781143188477, 0.10430169105529785, 0.10471296310424805, 0.10428524017333984, 0.10472655296325684, 0.10457229614257812, 0.10416078567504883, 0.1050560474395752, 0.10483837127685547, 0.1045992374420166, 0.10462260246276855, 0.10473299026489258, 0.10472631454467773, 0.10462617874145508, 0.10502743721008301, 0.10495948791503906, 0.10567569732666016, 0.10466718673706055, 0.10464072227478027, 0.10537219047546387, 0.10455989837646484, 0.10489845275878906, 0.10588669776916504, 0.10485291481018066, 0.10457468032836914, 0.10504007339477539, 0.10472798347473145, 0.10436224937438965, 0.10468363761901855, 0.10472774505615234, 0.10486555099487305, 0.10448288917541504, 0.10448408126831055, 0.10495829582214355, 0.1044769287109375, 0.10495352745056152, 0.10486221313476562, 0.10463237762451172, 0.10468840599060059, 0.10464620590209961, 0.10450220108032227, 0.10550260543823242, 0.10487079620361328, 0.1045219898223877, 0.10502195358276367, 0.10466837882995605, 0.10509824752807617, 0.10453081130981445, 0.10497426986694336, 0.10453534126281738, 0.10477256774902344, 0.10488200187683105, 0.10473346710205078, 0.10435605049133301, 0.10449695587158203, 0.10483169555664062, 0.10451936721801758, 0.10473489761352539, 0.10472679138183594, 0.10448527336120605, 0.1047511100769043, 0.10455918312072754, 0.10484933853149414, 0.10453271865844727, 0.10457968711853027, 0.10465312004089355, 0.10429024696350098, 0.10537385940551758, 0.10445928573608398, 0.10484623908996582, 0.10587167739868164, 0.10457777976989746, 0.1045382022857666, 0.10535407066345215, 0.10649681091308594, 0.1058802604675293, 0.1050565242767334, 0.10474014282226562, 0.10528063774108887, 0.10493826866149902, 0.10467958450317383, 0.10489130020141602, 0.10468935966491699, 0.10468316078186035, 0.10487580299377441, 0.10500144958496094, 0.1052548885345459, 0.10454368591308594, 0.10484647750854492, 0.10495328903198242, 0.10447478294372559, 0.10469818115234375, 0.1049494743347168, 0.10490179061889648, 0.10480999946594238, 0.10481548309326172, 0.1048426628112793, 0.10456013679504395, 0.10514092445373535, 0.10452580451965332, 0.10527873039245605, 0.10479378700256348, 0.10498404502868652, 0.10512852668762207, 0.10468626022338867, 0.10473775863647461, 0.10451102256774902, 0.10499024391174316, 0.10471224784851074, 0.10441803932189941, 0.10491251945495605, 0.10464310646057129, 0.11101627349853516, 0.1048898696899414, 0.10548567771911621, 0.10486412048339844, 0.10489511489868164, 0.10573029518127441, 0.10452485084533691, 0.10580921173095703, 0.10566926002502441, 0.10565900802612305, 0.10505104064941406, 0.10494637489318848, 0.10495233535766602, 0.10513591766357422, 0.10499906539916992, 0.10524702072143555, 0.10509181022644043, 0.10491013526916504, 0.10532188415527344, 0.1054227352142334, 0.10503292083740234, 0.10484504699707031, 0.10494852066040039, 0.1051797866821289, 0.10552573204040527, 0.10522222518920898, 0.10499763488769531, 0.10551857948303223, 0.10531282424926758, 0.10532426834106445, 0.10548233985900879, 0.10521149635314941, 0.10497117042541504, 0.10504722595214844, 0.10508370399475098, 0.10499143600463867, 0.10483431816101074, 0.10517191886901855, 0.10464906692504883, 0.10495400428771973, 0.10467314720153809, 0.10483813285827637, 0.10488414764404297, 0.10480356216430664, 0.1047978401184082, 0.10486674308776855, 0.10477948188781738, 0.10492181777954102, 0.1051788330078125, 0.10464954376220703, 0.10570216178894043, 0.10473942756652832, 0.10480737686157227, 0.10591745376586914, 0.10475397109985352, 0.1050114631652832, 0.10498738288879395, 0.10476064682006836, 0.1051323413848877, 0.10495233535766602, 0.10494160652160645, 0.1049501895904541, 0.10471248626708984, 0.10524535179138184, 0.1049203872680664, 0.10538101196289062, 0.10471606254577637, 0.10456085205078125, 0.10488533973693848, 0.1049959659576416, 0.10479974746704102, 0.10458588600158691, 0.10502052307128906, 0.10502481460571289, 0.10477304458618164, 0.10475325584411621, 0.10533404350280762, 0.10505175590515137, 0.10458636283874512, 0.10495233535766602, 0.10484552383422852, 0.10480356216430664, 0.1046457290649414, 0.1052560806274414, 0.1048586368560791, 0.10533475875854492, 0.1049947738647461, 0.10475611686706543, 0.10471200942993164, 0.10484910011291504, 0.10478544235229492, 0.10452747344970703, 0.1047356128692627, 0.10539889335632324, 0.1049034595489502, 0.10544371604919434, 0.10458159446716309, 0.10491418838500977, 0.10541391372680664, 0.10473465919494629, 0.10538315773010254, 0.1054682731628418, 0.10514998435974121, 0.10479021072387695, 0.10550189018249512, 0.10630393028259277, 0.10532736778259277, 0.10504603385925293, 0.10483217239379883, 0.10483908653259277, 0.10498929023742676, 0.10474848747253418, 0.10491776466369629, 0.10496139526367188, 0.10503888130187988, 0.10481119155883789, 0.10515785217285156, 0.10499787330627441, 0.10482048988342285, 0.10500597953796387, 0.10475683212280273, 0.10485219955444336, 0.10493993759155273, 0.10539460182189941, 0.10509705543518066, 0.10489225387573242, 0.10479068756103516, 0.10461068153381348, 0.10481905937194824, 0.10471439361572266, 0.10453677177429199, 0.10499978065490723, 0.10486626625061035, 0.10553622245788574, 0.10474109649658203, 0.10470461845397949, 0.1046750545501709, 0.10552573204040527, 0.10505318641662598, 0.10481739044189453, 0.10450053215026855, 0.10439562797546387, 0.10490655899047852, 0.10478901863098145, 0.10487914085388184, 0.10553693771362305, 0.10462474822998047, 0.10479855537414551, 0.10581517219543457, 0.10478401184082031, 0.10524988174438477, 0.10476231575012207, 0.10493135452270508, 0.10514163970947266, 0.10503578186035156, 0.10557055473327637, 0.10504937171936035, 0.10451722145080566, 0.1051483154296875, 0.1049964427947998, 0.10486459732055664, 0.10535049438476562, 0.10457849502563477, 0.10494208335876465, 0.10490655899047852, 0.10476994514465332, 0.10454463958740234, 0.1050255298614502, 0.10479068756103516, 0.10475468635559082, 0.10513973236083984, 0.10486221313476562, 0.1050264835357666, 0.10492515563964844, 0.10496068000793457, 0.1047201156616211, 0.10467267036437988, 0.10534119606018066, 0.10508084297180176, 0.10532307624816895, 0.1049346923828125, 0.10517096519470215, 0.10528683662414551, 0.10495448112487793, 0.10493302345275879, 0.10476016998291016, 0.10478949546813965, 0.10461115837097168, 0.10477352142333984, 0.10504364967346191, 0.10543179512023926, 0.10482287406921387, 0.10457611083984375, 0.10581231117248535, 0.10523605346679688, 0.10513710975646973, 0.10603547096252441, 0.10571885108947754, 0.10532951354980469, 0.1049184799194336, 0.10507059097290039, 0.10476565361022949, 0.10479497909545898, 0.10469841957092285, 0.10462164878845215, 0.10474777221679688, 0.10471439361572266, 0.10509610176086426, 0.10518670082092285, 0.10535860061645508, 0.10495710372924805, 0.10474967956542969, 0.10495853424072266, 0.10472440719604492, 0.10495805740356445, 0.10509753227233887, 0.10486459732055664, 0.10509538650512695, 0.10482311248779297, 0.10504412651062012, 0.10508275032043457, 0.1051778793334961, 0.10484623908996582, 0.10480785369873047, 0.1049191951751709, 0.10479617118835449, 0.10476160049438477, 0.10504579544067383, 0.10495710372924805, 0.1046609878540039, 0.10465359687805176, 0.10479092597961426, 0.10504412651062012, 0.10464739799499512, 0.10473108291625977, 0.10495495796203613, 0.10501980781555176, 0.10503220558166504, 0.10497665405273438, 0.10460543632507324, 0.10539031028747559, 0.10546231269836426, 0.10504674911499023, 0.10568356513977051, 0.10515379905700684, 0.10506439208984375, 0.1048283576965332, 0.10548281669616699, 0.1066293716430664, 0.10551857948303223, 0.10511946678161621, 0.10549378395080566, 0.1049337387084961, 0.10521435737609863, 0.10461544990539551, 0.10509920120239258, 0.10483098030090332, 0.10523366928100586, 0.10459017753601074, 0.10519909858703613, 0.10489916801452637, 0.10507941246032715, 0.10471081733703613, 0.10519170761108398, 0.10475039482116699, 0.10464954376220703, 0.10448479652404785, 0.10487031936645508, 0.10493326187133789, 0.10485005378723145, 0.1048283576965332, 0.10502195358276367, 0.1051478385925293, 0.10516929626464844, 0.10532164573669434, 0.10512614250183105, 0.10531473159790039, 0.10525989532470703, 0.1053016185760498, 0.1051640510559082, 0.10539722442626953, 0.10497665405273438, 0.10526299476623535, 0.10466265678405762, 0.10510087013244629, 0.10509920120239258, 0.10477066040039062, 0.10478949546813965, 0.10585284233093262, 0.10476922988891602, 0.10475635528564453, 0.10617566108703613, 0.10494685173034668, 0.1055765151977539, 0.10500121116638184, 0.10534310340881348, 0.10446023941040039, 0.10505890846252441, 0.10457110404968262, 0.10527873039245605, 0.10459661483764648, 0.10473871231079102, 0.1049964427947998, 0.1053471565246582, 0.10552668571472168, 0.1050264835357666, 0.10481572151184082, 0.10505938529968262, 0.10495996475219727, 0.10506892204284668, 0.10512852668762207, 0.10500216484069824, 0.10478425025939941, 0.10512113571166992, 0.10487198829650879, 0.1050264835357666, 0.10497355461120605, 0.10476207733154297, 0.10503792762756348, 0.10471391677856445, 0.10513424873352051, 0.10461854934692383, 0.1050715446472168, 0.10527944564819336, 0.10505414009094238, 0.10474491119384766, 0.1049196720123291, 0.10480046272277832, 0.1048431396484375, 0.10495877265930176, 0.10485672950744629, 0.1051332950592041, 0.10486268997192383, 0.10572004318237305, 0.10504317283630371, 0.1049649715423584, 0.10584330558776855, 0.10483264923095703, 0.10463380813598633, 0.10605406761169434, 0.1047663688659668, 0.10488486289978027, 0.10478019714355469, 0.10478734970092773, 0.10517549514770508, 0.1051034927368164, 0.10478520393371582, 0.10463476181030273, 0.10526633262634277, 0.10496711730957031, 0.10525870323181152, 0.1053168773651123, 0.10513758659362793, 0.10477375984191895, 0.10521841049194336, 0.10503339767456055, 0.10504674911499023, 0.10466313362121582, 0.10530257225036621, 0.10493135452270508, 0.10530304908752441, 0.1053612232208252, 0.10518097877502441, 0.10524201393127441, 0.10514521598815918, 0.10488009452819824, 0.10505461692810059, 0.10492873191833496, 0.10490798950195312, 0.10578703880310059, 0.10501456260681152, 0.10548257827758789, 0.10549306869506836, 0.10525751113891602, 0.10488533973693848, 0.1050574779510498, 0.10456323623657227, 0.10491037368774414, 0.10507512092590332, 0.10517072677612305, 0.10495448112487793, 0.10545086860656738, 0.10478568077087402, 0.10577154159545898, 0.1047513484954834, 0.10502743721008301, 0.10584759712219238, 0.10505843162536621, 0.10583305358886719, 0.10519099235534668, 0.10511231422424316, 0.10611152648925781, 0.10525107383728027, 0.10507965087890625, 0.1050710678100586, 0.10486984252929688, 0.10492491722106934, 0.10461091995239258, 0.10522770881652832, 0.1049492359161377, 0.1047811508178711, 0.1046597957611084, 0.10493612289428711, 0.10463380813598633, 0.10470795631408691, 0.1051030158996582, 0.10501861572265625, 0.10540318489074707, 0.10518813133239746, 0.10525870323181152, 0.10519552230834961, 0.10494446754455566, 0.10492253303527832, 0.10500168800354004, 0.10467147827148438, 0.10489726066589355, 0.10498428344726562, 0.10511422157287598, 0.1048727035522461, 0.10562348365783691, 0.10485100746154785, 0.10490989685058594, 0.10470366477966309, 0.10518932342529297, 0.10489940643310547, 0.10493779182434082, 0.10454010963439941, 0.10497307777404785, 0.10512638092041016, 0.10499215126037598, 0.1050419807434082, 0.10570120811462402, 0.10493278503417969, 0.10487937927246094, 0.1060950756072998, 0.10462069511413574, 0.10515308380126953, 0.10504770278930664, 0.10491800308227539, 0.1051034927368164, 0.10500550270080566, 0.10538673400878906, 0.10463905334472656, 0.10498499870300293, 0.10501241683959961, 0.10478639602661133, 0.1051797866821289, 0.1050405502319336, 0.10481643676757812, 0.1050119400024414, 0.10515689849853516, 0.10490632057189941, 0.10475468635559082, 0.1054527759552002, 0.10477972030639648, 0.10498666763305664, 0.10503721237182617, 0.10513496398925781, 0.10482192039489746, 0.10515236854553223, 0.10451626777648926, 0.10496068000793457, 0.10464906692504883, 0.10497260093688965, 0.10544848442077637, 0.10518789291381836, 0.10560464859008789, 0.10534381866455078, 0.10495233535766602, 0.10487723350524902, 0.1048440933227539, 0.10474681854248047, 0.10460376739501953, 0.10516023635864258, 0.10534167289733887, 0.10500741004943848, 0.10567760467529297, 0.10482978820800781, 0.10547614097595215, 0.10533905029296875, 0.10477161407470703, 0.10554337501525879, 0.10528135299682617, 0.10535573959350586, 0.10521197319030762, 0.10501670837402344, 0.10502076148986816, 0.10469603538513184, 0.10509276390075684, 0.10508322715759277, 0.10477662086486816, 0.10502338409423828, 0.10457611083984375, 0.10481715202331543, 0.10482954978942871, 0.10501503944396973, 0.10459709167480469, 0.10485577583312988, 0.10486960411071777, 0.10480523109436035, 0.10505437850952148, 0.10507798194885254, 0.10455775260925293, 0.10482621192932129, 0.10511136054992676, 0.10504150390625, 0.10516548156738281, 0.10493206977844238, 0.10508084297180176, 0.1052999496459961, 0.10462450981140137, 0.10508918762207031, 0.10523629188537598, 0.1049957275390625, 0.10507798194885254, 0.10474681854248047, 0.1050117015838623, 0.10442709922790527, 0.10485649108886719, 0.10495519638061523, 0.10464262962341309, 0.10469841957092285, 0.1044917106628418, 0.10532784461975098, 0.10495305061340332, 0.10406899452209473, 0.1055150032043457, 0.10474395751953125, 0.10529732704162598, 0.10633468627929688, 0.10486555099487305, 0.10498356819152832, 0.10512685775756836, 0.10585784912109375, 0.10716867446899414, 0.10534048080444336, 0.10561537742614746, 0.10493016242980957, 0.10492348670959473, 0.10521650314331055, 0.10471248626708984, 0.10543179512023926, 0.1052389144897461, 0.10505557060241699, 0.10474324226379395, 0.10509896278381348, 0.10462117195129395, 0.10516095161437988, 0.10498356819152832, 0.10492467880249023, 0.10467123985290527, 0.10500693321228027, 0.10485124588012695, 0.10498809814453125, 0.10534048080444336, 0.10478639602661133, 0.10483551025390625, 0.10469937324523926, 0.10508108139038086, 0.10576510429382324, 0.10610485076904297, 0.10525035858154297, 0.10539007186889648, 0.10503554344177246, 0.1048896312713623, 0.10460376739501953, 0.10485458374023438, 0.10463905334472656, 0.10515451431274414, 0.10456657409667969, 0.10514473915100098, 0.10955595970153809, 0.10503530502319336, 0.1053457260131836, 0.10597085952758789, 0.10533452033996582, 0.10533523559570312, 0.10581707954406738, 0.10561418533325195, 0.1049954891204834, 0.10538220405578613, 0.10499978065490723, 0.10471439361572266, 0.1051180362701416, 0.1047518253326416, 0.10521340370178223, 0.10479354858398438, 0.10515618324279785, 0.10521459579467773, 0.10543060302734375, 0.10538816452026367, 0.10475683212280273, 0.10509443283081055, 0.10521268844604492, 0.10497140884399414, 0.10476088523864746, 0.10564088821411133, 0.10496973991394043, 0.1052706241607666, 0.1051182746887207, 0.10481786727905273, 0.1050114631652832, 0.10548734664916992, 0.104766845703125, 0.10511016845703125, 0.10475802421569824, 0.10522246360778809, 0.10467648506164551, 0.10503530502319336, 0.10501670837402344, 0.10490179061889648, 0.10490202903747559, 0.10452151298522949, 0.10480833053588867, 0.10476016998291016, 0.10478615760803223, 0.1045680046081543, 0.10474181175231934, 0.10510563850402832, 0.10524797439575195, 0.10474896430969238, 0.10581016540527344, 0.10464215278625488, 0.10505843162536621, 0.10621166229248047, 0.10483622550964355, 0.10458898544311523, 0.10508608818054199, 0.10492444038391113, 0.10521507263183594, 0.10496950149536133, 0.10485386848449707, 0.10489559173583984, 0.10474252700805664, 0.10480427742004395, 0.10466575622558594, 0.1050865650177002, 0.10503506660461426, 0.10488152503967285, 0.10474228858947754, 0.10484647750854492, 0.10470008850097656, 0.10506224632263184, 0.10527253150939941, 0.10513734817504883, 0.1046743392944336, 0.10474061965942383, 0.1048734188079834, 0.10473155975341797, 0.10480403900146484, 0.1047978401184082, 0.10467672348022461, 0.10474181175231934, 0.1047523021697998, 0.10492515563964844, 0.10537981986999512, 0.10514569282531738, 0.10551762580871582, 0.10561013221740723, 0.10503840446472168, 0.10496330261230469, 0.10490632057189941, 0.10461115837097168, 0.10497879981994629, 0.1049661636352539, 0.10509276390075684, 0.10529780387878418, 0.10477352142333984, 0.10473942756652832, 0.10573983192443848, 0.10482668876647949, 0.1049356460571289, 0.10605859756469727, 0.10506963729858398, 0.10512042045593262, 0.10502195358276367, 0.10648775100708008, 0.1054677963256836, 0.10560464859008789, 0.10471439361572266, 0.10483336448669434, 0.10449600219726562, 0.1049492359161377, 0.10480880737304688, 0.1049337387084961, 0.10506439208984375, 0.10612273216247559, 0.10489821434020996, 0.10516762733459473, 0.10509920120239258, 0.10474562644958496, 0.10525894165039062, 0.10459566116333008, 0.10490107536315918, 0.10488677024841309, 0.1053154468536377, 0.10463309288024902, 0.10509562492370605, 0.10457301139831543, 0.1048116683959961, 0.10476541519165039, 0.10479307174682617, 0.10463333129882812, 0.10492777824401855, 0.10541200637817383, 0.1048879623413086, 0.10471844673156738, 0.10483407974243164, 0.10582971572875977, 0.10491180419921875, 0.10500383377075195, 0.10500335693359375, 0.10506391525268555, 0.10499691963195801, 0.10505366325378418, 0.10484099388122559, 0.1054689884185791, 0.10497164726257324, 0.10522079467773438, 0.10587930679321289, 0.10519719123840332, 0.10484838485717773, 0.1049494743347168, 0.10490584373474121, 0.10474634170532227, 0.10484743118286133, 0.10495710372924805, 0.10518980026245117, 0.10516190528869629, 0.10519075393676758, 0.10485696792602539, 0.10477209091186523, 0.10482954978942871, 0.10486149787902832, 0.10509753227233887, 0.10466909408569336, 0.10487484931945801, 0.10488080978393555, 0.10494089126586914, 0.1049964427947998, 0.10461854934692383, 0.10498309135437012, 0.10485601425170898, 0.10500669479370117, 0.10491108894348145, 0.10494709014892578, 0.10426497459411621, 0.10491394996643066, 0.10479187965393066, 0.10522031784057617, 0.1050715446472168, 0.10486841201782227, 0.10510993003845215, 0.10619163513183594, 0.105010986328125, 0.1048896312713623, 0.10487699508666992, 0.10457754135131836, 0.1050727367401123, 0.10480713844299316, 0.10488772392272949, 0.10571885108947754, 0.10510683059692383, 0.10451865196228027, 0.10567331314086914, 0.10457253456115723, 0.10497331619262695, 0.10638236999511719, 0.10516929626464844, 0.10521316528320312, 0.10484194755554199, 0.10477614402770996, 0.1047675609588623, 0.10456347465515137, 0.10464763641357422, 0.10429739952087402, 0.10469818115234375, 0.10482335090637207, 0.10462760925292969, 0.10492968559265137, 0.10509467124938965, 0.10522150993347168, 0.10458111763000488, 0.10528802871704102, 0.10474920272827148, 0.10523414611816406, 0.10522794723510742, 0.10487031936645508, 0.10492682456970215, 0.10510587692260742, 0.10510540008544922, 0.10511016845703125, 0.10509729385375977, 0.1051638126373291, 0.1053004264831543, 0.1048588752746582, 0.1046898365020752, 0.10474777221679688, 0.10487198829650879, 0.10505414009094238, 0.10496211051940918, 0.10468602180480957, 0.1050722599029541, 0.1046438217163086, 0.10522747039794922, 0.10517287254333496, 0.10471725463867188, 0.10493326187133789, 0.10514640808105469, 0.10517740249633789, 0.10498929023742676, 0.10494112968444824, 0.1056818962097168, 0.10488390922546387, 0.105316162109375, 0.10631775856018066, 0.10500931739807129, 0.10496783256530762, 0.10588574409484863, 0.10624933242797852, 0.1054224967956543, 0.10571885108947754, 0.10501623153686523, 0.10527467727661133, 0.10519695281982422, 0.10511279106140137, 0.1051177978515625, 0.10526728630065918, 0.10495519638061523, 0.10492181777954102, 0.1048133373260498, 0.10505795478820801, 0.10481786727905273, 0.10505127906799316, 0.1053619384765625, 0.10491061210632324, 0.1050724983215332, 0.10625362396240234, 0.10510993003845215, 0.10483169555664062, 0.1052708625793457, 0.10535550117492676, 0.10498762130737305, 0.10497355461120605, 0.10520124435424805, 0.10491228103637695, 0.1051340103149414, 0.10514521598815918, 0.10542082786560059, 0.10477113723754883, 0.10483336448669434, 0.1047663688659668, 0.10475373268127441, 0.10473489761352539, 0.10483670234680176, 0.10497927665710449, 0.10525012016296387, 0.10520124435424805, 0.10489630699157715, 0.10611414909362793, 0.10450243949890137, 0.10488677024841309, 0.10612630844116211, 0.10552096366882324, 0.10525321960449219, 0.10524606704711914, 0.10510993003845215, 0.10481452941894531, 0.10469698905944824, 0.10503816604614258, 0.10491037368774414, 0.10501742362976074, 0.10491609573364258, 0.10495805740356445, 0.10499882698059082, 0.10578298568725586, 0.1045987606048584, 0.10489988327026367, 0.1049344539642334, 0.10498356819152832, 0.1049811840057373, 0.1052238941192627, 0.10489034652709961, 0.10480332374572754, 0.10492491722106934, 0.10494017601013184, 0.10516715049743652, 0.10536026954650879, 0.10520172119140625, 0.10495615005493164, 0.10487818717956543, 0.10484766960144043, 0.10478806495666504, 0.10505914688110352, 0.10505867004394531, 0.10486221313476562, 0.10492539405822754, 0.10485363006591797, 0.10484051704406738, 0.10496377944946289, 0.10487127304077148, 0.1050252914428711, 0.10441088676452637, 0.10483169555664062, 0.10552668571472168, 0.10471534729003906, 0.1046285629272461, 0.10545945167541504, 0.10470390319824219, 0.1051182746887207, 0.10618472099304199, 0.1049952507019043, 0.10472488403320312, 0.10503935813903809, 0.10478639602661133, 0.1051788330078125, 0.10463929176330566, 0.10483074188232422, 0.10483837127685547, 0.10485196113586426, 0.10497474670410156, 0.10475897789001465, 0.10493969917297363, 0.10488343238830566, 0.1047661304473877, 0.10428500175476074, 0.1052391529083252, 0.10457324981689453, 0.1049795150756836, 0.10513520240783691, 0.1049957275390625, 0.10469985008239746, 0.10479998588562012, 0.10535836219787598, 0.10487723350524902, 0.10507988929748535, 0.10466766357421875, 0.1044619083404541, 0.1048440933227539, 0.1049814224243164, 0.10532259941101074, 0.10471606254577637, 0.10535073280334473, 0.1054677963256836, 0.10511350631713867, 0.10494804382324219, 0.10474562644958496, 0.1051633358001709, 0.10449528694152832, 0.10483813285827637, 0.10482239723205566, 0.10492062568664551, 0.10481977462768555, 0.10507798194885254, 0.10472965240478516, 0.1051020622253418, 0.10486888885498047, 0.10553669929504395, 0.10510611534118652, 0.10554838180541992, 0.10480260848999023, 0.10520744323730469, 0.10630512237548828, 0.10514116287231445, 0.10528445243835449, 0.10460209846496582, 0.10505437850952148, 0.10442948341369629, 0.10520172119140625, 0.10491585731506348, 0.10476255416870117, 0.10475826263427734, 0.10479474067687988, 0.10470414161682129, 0.10495996475219727, 0.1046450138092041, 0.10494661331176758, 0.10480308532714844, 0.10488438606262207, 0.1049661636352539, 0.10509037971496582, 0.10504317283630371, 0.10473203659057617, 0.10521769523620605, 0.10503911972045898, 0.10487723350524902, 0.10463857650756836, 0.10485410690307617, 0.10446834564208984, 0.10501265525817871, 0.10518765449523926, 0.10444307327270508, 0.10456228256225586, 0.10461235046386719, 0.10465741157531738, 0.10512757301330566, 0.10443449020385742, 0.10474348068237305, 0.10482978820800781, 0.10520124435424805, 0.10483646392822266, 0.10550522804260254, 0.10563826560974121, 0.1048123836517334, 0.10547971725463867, 0.10614991188049316, 0.10478329658508301, 0.10502839088439941, 0.10487651824951172, 0.10500764846801758, 0.1047358512878418, 0.10484981536865234, 0.10561060905456543, 0.10492491722106934, 0.10505151748657227, 0.10465598106384277, 0.10506057739257812, 0.10483837127685547, 0.10540080070495605, 0.10473251342773438, 0.10500288009643555, 0.10479903221130371, 0.10470294952392578, 0.10500216484069824, 0.10537385940551758, 0.10478544235229492, 0.10465216636657715, 0.10497879981994629, 0.10482525825500488, 0.1050875186920166, 0.10491228103637695, 0.1055302619934082, 0.1045522689819336, 0.10506606101989746, 0.10468649864196777, 0.10528206825256348, 0.10514235496520996, 0.10561871528625488, 0.10566306114196777, 0.10504364967346191, 0.10469222068786621, 0.10456085205078125, 0.10477089881896973, 0.10494637489318848, 0.10485219955444336, 0.1049196720123291, 0.10497736930847168, 0.10551834106445312, 0.10506677627563477, 0.10490942001342773, 0.10558366775512695, 0.1050567626953125, 0.1049957275390625, 0.10596084594726562, 0.10532736778259277, 0.10483670234680176, 0.10511374473571777, 0.10494804382324219, 0.10499930381774902, 0.10520672798156738, 0.10473060607910156, 0.10472440719604492, 0.10476279258728027, 0.10484886169433594, 0.10483241081237793, 0.1048593521118164, 0.10502815246582031, 0.1051492691040039, 0.10464143753051758, 0.10504865646362305, 0.1045529842376709, 0.10479402542114258, 0.1052238941192627, 0.10501313209533691, 0.1047677993774414, 0.10520267486572266, 0.10494422912597656, 0.10489583015441895, 0.10564422607421875, 0.10473322868347168, 0.10501313209533691, 0.10487556457519531, 0.10489678382873535, 0.10891413688659668, 0.1051642894744873, 0.10499286651611328, 0.1047067642211914, 0.10484170913696289, 0.1049046516418457, 0.10479426383972168, 0.10456347465515137, 0.10455656051635742, 0.10482168197631836, 0.10492825508117676, 0.10476922988891602, 0.10493206977844238, 0.10491347312927246, 0.10521888732910156, 0.1056056022644043, 0.10472536087036133, 0.10604095458984375, 0.10477113723754883, 0.10469222068786621, 0.10493636131286621, 0.10644888877868652, 0.10640096664428711, 0.10566520690917969, 0.10577702522277832, 0.10475564002990723, 0.10492610931396484, 0.10494637489318848, 0.10479283332824707, 0.105010986328125, 0.10658597946166992, 0.10476183891296387, 0.1049799919128418, 0.10531067848205566, 0.10486364364624023, 0.1052398681640625, 0.10511445999145508, 0.10496354103088379, 0.10515189170837402, 0.10495781898498535, 0.10543060302734375, 0.10521173477172852, 0.10517549514770508, 0.10530424118041992, 0.10527539253234863, 0.10501337051391602, 0.10497903823852539, 0.10538840293884277, 0.10543084144592285, 0.10501313209533691, 0.10532474517822266, 0.10530781745910645, 0.1050868034362793, 0.10494446754455566, 0.10512185096740723, 0.1052241325378418, 0.10508012771606445, 0.10524439811706543, 0.10480690002441406, 0.11873626708984375, 0.10510468482971191, 0.10508227348327637, 0.10559606552124023, 0.1049342155456543, 0.10473227500915527, 0.1061544418334961, 0.10595393180847168, 0.10506677627563477, 0.1052851676940918, 0.10499167442321777, 0.10490202903747559, 0.10508942604064941, 0.10483813285827637, 0.10519719123840332, 0.10507392883300781, 0.10504388809204102, 0.10538196563720703, 0.10523772239685059, 0.10621094703674316, 0.1051034927368164, 0.10540604591369629, 0.10518932342529297, 0.1052706241607666, 0.10512399673461914, 0.10521745681762695, 0.10524988174438477, 0.1048884391784668, 0.10540366172790527, 0.10502219200134277, 0.10520362854003906, 0.10561728477478027, 0.10517048835754395, 0.10499262809753418, 0.10475897789001465, 0.1050119400024414, 0.1050260066986084, 0.10477137565612793, 0.10673785209655762, 0.10560393333435059, 0.10505294799804688, 0.10468673706054688, 0.10549116134643555, 0.10510683059692383, 0.10477232933044434, 0.10498714447021484, 0.10486412048339844, 0.10508084297180176, 0.10476970672607422, 0.10493326187133789, 0.10539007186889648, 0.10491275787353516, 0.1051795482635498, 0.10626983642578125, 0.10503959655761719, 0.10510444641113281, 0.10515546798706055, 0.10522913932800293, 0.1055300235748291, 0.10499739646911621, 0.1049659252166748, 0.10531425476074219, 0.10465526580810547, 0.10523343086242676, 0.10488319396972656, 0.1053621768951416, 0.10543060302734375, 0.1050715446472168, 0.10470938682556152, 0.10532045364379883, 0.10489678382873535, 0.1048574447631836, 0.10516619682312012, 0.10501241683959961, 0.10489773750305176, 0.10487580299377441, 0.10581803321838379, 0.10499882698059082, 0.10484671592712402, 0.10497164726257324, 0.10484004020690918, 0.10468578338623047, 0.10520482063293457, 0.10526084899902344, 0.10512709617614746, 0.10494494438171387, 0.10550522804260254, 0.10539007186889648, 0.1049654483795166, 0.10672426223754883, 0.10498762130737305, 0.10531020164489746, 0.10526919364929199, 0.10489535331726074, 0.10485482215881348, 0.1053924560546875, 0.10526061058044434, 0.1112971305847168, 0.11601805686950684, 0.10498738288879395, 0.10491538047790527, 0.10645174980163574, 0.10553336143493652, 0.10508346557617188, 0.10478591918945312, 0.10642838478088379, 0.10519814491271973, 0.10529160499572754, 0.10488343238830566, 0.10500407218933105, 0.10473179817199707, 0.10495114326477051, 0.10506749153137207, 0.1062324047088623, 0.10509371757507324, 0.10455965995788574, 0.1057586669921875, 0.1050879955291748, 0.10503482818603516, 0.10489821434020996, 0.10502195358276367, 0.10511612892150879, 0.10474419593811035, 0.10487675666809082, 0.10517287254333496, 0.10489773750305176, 0.10520434379577637, 0.10482382774353027, 0.10437655448913574, 0.10479259490966797, 0.10474920272827148, 0.10470962524414062, 0.10465192794799805, 0.10547614097595215, 0.10512375831604004, 0.10496783256530762, 0.10494780540466309, 0.10571599006652832, 0.10522103309631348, 0.10508251190185547, 0.10464191436767578, 0.1050572395324707, 0.10500407218933105, 0.10503435134887695, 0.10470056533813477, 0.10661745071411133, 0.10535120964050293, 0.10526657104492188, 0.1064612865447998, 0.10501503944396973, 0.10491299629211426, 0.10513091087341309, 0.10540151596069336, 0.10509514808654785, 0.10499930381774902, 0.1052854061126709, 0.10506272315979004, 0.10541653633117676, 0.10514020919799805, 0.10509634017944336, 0.1051018238067627, 0.1053931713104248, 0.10514521598815918, 0.10494589805603027, 0.10522127151489258, 0.10489201545715332, 0.10532712936401367, 0.10523009300231934, 0.1047220230102539, 0.10474085807800293, 0.1050417423248291, 0.1051485538482666, 0.10513520240783691, 0.10507011413574219, 0.1052093505859375, 0.10518193244934082, 0.10495710372924805, 0.10516619682312012, 0.10521578788757324, 0.10544776916503906, 0.10527896881103516, 0.10524892807006836, 0.10564565658569336, 0.10474252700805664, 0.10482621192932129, 0.1048269271850586, 0.10476875305175781, 0.10484886169433594, 0.10506725311279297, 0.10504937171936035, 0.1057119369506836, 0.10507011413574219, 0.10502004623413086, 0.10589766502380371, 0.10513496398925781, 0.10495328903198242, 0.10599637031555176, 0.10532498359680176, 0.1054995059967041, 0.10511016845703125, 0.10538792610168457, 0.10544157028198242, 0.10531210899353027, 0.10500788688659668, 0.10497260093688965, 0.10512518882751465, 0.10525321960449219, 0.1052708625793457, 0.10507917404174805, 0.10550332069396973, 0.1048727035522461, 0.10521483421325684, 0.10525751113891602, 0.1051335334777832, 0.10479474067687988, 0.1055135726928711, 0.10485243797302246, 0.10523629188537598, 0.10494637489318848, 0.10542416572570801, 0.10487532615661621, 0.10535645484924316, 0.10519242286682129, 0.10487532615661621, 0.10489845275878906, 0.10484600067138672, 0.10471725463867188, 0.10519266128540039, 0.10504484176635742, 0.10475969314575195, 0.10496211051940918, 0.10503625869750977, 0.1052398681640625, 0.10489058494567871, 0.10513734817504883, 0.1047518253326416, 0.10511136054992676, 0.10548162460327148, 0.10502243041992188, 0.10481572151184082, 0.10719609260559082, 0.1064755916595459, 0.10516619682312012, 0.10697484016418457, 0.10530447959899902, 0.1063542366027832, 0.10546374320983887, 0.10783195495605469, 0.10793066024780273, 0.10638737678527832, 0.10814666748046875, 0.10544967651367188, 0.12177538871765137, 0.10536646842956543, 0.10537409782409668, 0.10569047927856445, 0.10546469688415527, 0.10513114929199219, 0.10503411293029785, 0.10559606552124023, 0.10502123832702637, 0.10509586334228516, 0.10528278350830078, 0.10485553741455078, 0.10551047325134277, 0.10496854782104492, 0.11015105247497559, 0.10519814491271973, 0.10511255264282227, 0.10512328147888184, 0.1048734188079834, 0.10504555702209473, 0.10523247718811035, 0.1055908203125, 0.10524582862854004, 0.10552406311035156, 0.10529899597167969, 0.1052100658416748, 0.10520577430725098, 0.10469794273376465, 0.10493135452270508, 0.10523414611816406, 0.10482048988342285, 0.1051931381225586, 0.10495114326477051, 0.1055452823638916, 0.10499286651611328, 0.10521674156188965, 0.10573887825012207, 0.10524368286132812, 0.10540628433227539, 0.1056818962097168, 0.10559701919555664, 0.10494136810302734, 0.10526037216186523, 0.1052858829498291, 0.10479521751403809, 0.10522294044494629, 0.10490250587463379, 0.10484457015991211, 0.104827880859375, 0.10515666007995605, 0.10505938529968262, 0.10515522956848145, 0.10563206672668457, 0.10480594635009766, 0.10528922080993652, 0.10469245910644531, 0.10506057739257812, 0.10538983345031738, 0.10532379150390625, 0.10495781898498535, 0.10499334335327148, 0.10538697242736816, 0.10645842552185059, 0.10475587844848633, 0.10536932945251465, 0.10498356819152832, 0.10499739646911621, 0.10486030578613281, 0.1051340103149414, 0.10539722442626953, 0.10510802268981934, 0.1048288345336914, 0.10466361045837402, 0.10471439361572266, 0.10477256774902344, 0.10536313056945801, 0.10454750061035156, 0.10500025749206543, 0.10475039482116699, 0.10531497001647949, 0.10589599609375, 0.10503053665161133, 0.10485386848449707, 0.10567164421081543, 0.10503935813903809, 0.10520553588867188, 0.1063694953918457, 0.10512518882751465, 0.10472345352172852, 0.10520315170288086, 0.10484170913696289, 0.10587239265441895, 0.10500192642211914, 0.1051332950592041, 0.10511422157287598, 0.10496401786804199, 0.10509324073791504, 0.1052100658416748, 0.10507345199584961, 0.10526609420776367, 0.10523509979248047, 0.10472297668457031, 0.10547399520874023, 0.1048116683959961, 0.10492348670959473, 0.1053617000579834, 0.10476064682006836, 0.10489368438720703, 0.10508918762207031, 0.10605931282043457, 0.10486745834350586, 0.10488533973693848, 0.10477852821350098, 0.10523700714111328, 0.10488438606262207, 0.1049652099609375, 0.10543560981750488, 0.10496211051940918, 0.10555028915405273, 0.10559916496276855, 0.10496735572814941, 0.10499930381774902, 0.10476326942443848, 0.10506105422973633, 0.10504817962646484, 0.10501694679260254, 0.10492300987243652, 0.10497856140136719, 0.10560750961303711, 0.1051490306854248, 0.10499787330627441, 0.10582375526428223, 0.10485243797302246, 0.1045846939086914, 0.10589218139648438, 0.1055300235748291, 0.10519933700561523, 0.10524201393127441, 0.10652852058410645, 0.10565567016601562, 0.10542988777160645, 0.10497236251831055, 0.10517096519470215, 0.10525965690612793, 0.10514354705810547, 0.10527706146240234, 0.1052696704864502, 0.10552453994750977, 0.10518336296081543, 0.10520458221435547, 0.10514616966247559, 0.10512995719909668, 0.10477209091186523, 0.10535168647766113, 0.10480594635009766, 0.10526084899902344, 0.1049187183380127, 0.10539078712463379, 0.10453271865844727, 0.10510873794555664, 0.10519218444824219, 0.10514354705810547, 0.10503125190734863, 0.1051478385925293, 0.10501766204833984, 0.10536074638366699, 0.10548543930053711, 0.10457611083984375, 0.10497450828552246, 0.10473489761352539, 0.10501766204833984, 0.10546469688415527, 0.10505104064941406, 0.10469627380371094, 0.10503530502319336, 0.10533642768859863, 0.10518240928649902, 0.1045539379119873, 0.10569095611572266, 0.10458922386169434, 0.10497665405273438, 0.10616421699523926, 0.10496735572814941, 0.10516953468322754, 0.10524487495422363, 0.10556912422180176, 0.10521960258483887, 0.10475778579711914, 0.10526514053344727, 0.10535168647766113, 0.10483884811401367, 0.10510849952697754, 0.10499715805053711, 0.10513448715209961, 0.10516524314880371, 0.10510063171386719, 0.10448336601257324, 0.10535120964050293, 0.10495305061340332, 0.10513424873352051, 0.10514140129089355, 0.10502219200134277, 0.10478663444519043, 0.10531759262084961, 0.10519146919250488, 0.10525965690612793, 0.10521483421325684, 0.10480904579162598, 0.10532855987548828, 0.10512471199035645, 0.1049649715423584, 0.10544276237487793, 0.10524892807006836, 0.10526227951049805, 0.10545849800109863, 0.10465168952941895, 0.10517024993896484, 0.10450577735900879, 0.10488605499267578, 0.10472440719604492, 0.1054079532623291, 0.10485625267028809, 0.10523748397827148, 0.10569357872009277, 0.10512113571166992, 0.10522770881652832, 0.10567688941955566, 0.10498785972595215, 0.10494470596313477, 0.10606789588928223, 0.10523414611816406, 0.10520148277282715, 0.10497355461120605, 0.1051030158996582, 0.10464358329772949, 0.1053001880645752, 0.10468339920043945, 0.10518622398376465, 0.10493326187133789, 0.10514187812805176, 0.10501766204833984, 0.10530996322631836, 0.10507702827453613, 0.10489058494567871, 0.10506033897399902, 0.10506677627563477, 0.10474729537963867, 0.10498762130737305, 0.10522842407226562, 0.10479474067687988, 0.10503411293029785, 0.10515189170837402, 0.10536599159240723, 0.10469985008239746, 0.10554265975952148, 0.10518741607666016, 0.10515284538269043, 0.10489511489868164, 0.10526490211486816, 0.10477876663208008, 0.10552835464477539, 0.10517263412475586, 0.10490822792053223, 0.10518121719360352, 0.10491633415222168, 0.10506701469421387, 0.10460495948791504, 0.10520577430725098, 0.10451936721801758, 0.10521745681762695, 0.10509181022644043, 0.10527944564819336, 0.10482954978942871, 0.1056971549987793, 0.10550713539123535, 0.1053779125213623, 0.10543417930603027, 0.10524177551269531, 0.10524582862854004, 0.10518836975097656, 0.10545849800109863, 0.10730147361755371, 0.10622668266296387, 0.10542654991149902, 0.10548949241638184, 0.10506224632263184, 0.1051945686340332, 0.10476040840148926, 0.10479402542114258, 0.105865478515625, 0.1051340103149414, 0.1048438549041748, 0.10560822486877441, 0.10473155975341797, 0.10515618324279785, 0.10533285140991211, 0.10489630699157715, 0.10474014282226562, 0.1051478385925293, 0.10499215126037598, 0.10575532913208008, 0.10515904426574707, 0.1050264835357666, 0.10544443130493164, 0.1048126220703125, 0.10514283180236816, 0.10515594482421875, 0.10549592971801758, 0.10531878471374512, 0.10611653327941895, 0.10602545738220215, 0.10523629188537598, 0.10465312004089355, 0.10509443283081055, 0.10508942604064941, 0.10530447959899902, 0.10492682456970215, 0.10504603385925293, 0.12193727493286133, 0.10647892951965332, 0.1055147647857666, 0.1062781810760498, 0.10532903671264648, 0.10539579391479492, 0.10666131973266602, 0.10582160949707031, 0.10518360137939453, 0.10566067695617676, 0.10558247566223145, 0.10545182228088379, 0.10529017448425293, 0.10505557060241699, 0.1054844856262207, 0.10528564453125, 0.1061394214630127, 0.10550284385681152, 0.1056663990020752, 0.10646438598632812, 0.10542178153991699, 0.10557436943054199, 0.10536026954650879, 0.10529327392578125, 0.10525345802307129, 0.10570359230041504, 0.10506129264831543, 0.10555386543273926, 0.1055762767791748, 0.10547661781311035, 0.10503172874450684, 0.10571074485778809, 0.1051337718963623, 0.1057441234588623, 0.10487890243530273, 0.10543107986450195, 0.10530662536621094, 0.1051185131072998, 0.10636234283447266, 0.10696816444396973, 0.10541439056396484, 0.10538864135742188, 0.10512113571166992, 0.10546994209289551, 0.10492181777954102, 0.1050560474395752, 0.10548639297485352, 0.10560321807861328, 0.10595178604125977, 0.10478568077087402, 0.1064152717590332, 0.10616374015808105, 0.10523629188537598, 0.1059422492980957, 0.10517716407775879, 0.1051321029663086, 0.10511445999145508, 0.1053311824798584, 0.10502839088439941, 0.10505032539367676, 0.10503888130187988, 0.10533881187438965, 0.1048743724822998, 0.10488200187683105, 0.10493993759155273, 0.10506415367126465, 0.10526442527770996, 0.10539650917053223, 0.10516023635864258, 0.1056058406829834, 0.10507559776306152, 0.10512590408325195, 0.10534238815307617, 0.10512971878051758, 0.10494613647460938, 0.1049802303314209, 0.10584282875061035, 0.10537528991699219, 0.10515999794006348, 0.10505914688110352, 0.1050715446472168, 0.10531258583068848, 0.1054222583770752, 0.1053624153137207, 0.10527229309082031, 0.10555267333984375, 0.10567712783813477, 0.1053304672241211, 0.1054372787475586, 0.10488390922546387, 0.10518884658813477, 0.10500001907348633, 0.10514259338378906, 0.10514092445373535, 0.1052560806274414, 0.10567522048950195, 0.1052851676940918, 0.10533928871154785, 0.10654759407043457, 0.10550236701965332, 0.1058502197265625, 0.10587263107299805, 0.10548210144042969, 0.10526490211486816, 0.10547208786010742, 0.1083688735961914, 0.10561299324035645, 0.10549187660217285, 0.10493040084838867, 0.10558366775512695, 0.10520243644714355, 0.1051778793334961, 0.10522794723510742, 0.10546016693115234, 0.10522985458374023, 0.10777735710144043, 0.10552287101745605, 0.10477852821350098, 0.10494780540466309, 0.10550570487976074, 0.10528779029846191, 0.10513043403625488, 0.10530614852905273, 0.10531878471374512, 0.10591888427734375, 0.10499191284179688, 0.10535812377929688, 0.10508465766906738, 0.10497713088989258, 0.10500478744506836, 0.10496640205383301, 0.10513854026794434, 0.1053304672241211, 0.10547876358032227, 0.10494518280029297, 0.10534143447875977, 0.10514593124389648, 0.10630369186401367, 0.10536336898803711, 0.10539078712463379, 0.10534071922302246, 0.10531830787658691, 0.10542631149291992, 0.1053764820098877, 0.10521936416625977, 0.10652709007263184, 0.10524535179138184, 0.1049501895904541, 0.10653877258300781, 0.10523033142089844, 0.10529255867004395, 0.1056680679321289, 0.10534977912902832, 0.1056661605834961, 0.10546588897705078, 0.1058187484741211, 0.1057577133178711, 0.10513043403625488, 0.10537934303283691, 0.10516738891601562, 0.10506868362426758, 0.10575056076049805, 0.10528707504272461, 0.10531044006347656, 0.10530304908752441, 0.10514664649963379, 0.10544037818908691, 0.1056523323059082, 0.10512137413024902, 0.10531115531921387, 0.10535407066345215, 0.10526752471923828, 0.10552549362182617, 0.10532617568969727, 0.10479164123535156, 0.10521149635314941, 0.10502982139587402, 0.10544633865356445, 0.10603928565979004, 0.10524749755859375, 0.10592126846313477, 0.1070098876953125, 0.10519790649414062, 0.10544204711914062, 0.10491371154785156, 0.10543155670166016, 0.10529899597167969, 0.10532593727111816, 0.10498547554016113, 0.10566091537475586, 0.1057884693145752, 0.10549497604370117, 0.10557079315185547, 0.10579967498779297, 0.10512351989746094, 0.10610365867614746, 0.10555696487426758, 0.10571861267089844, 0.10545158386230469, 0.10551929473876953, 0.10542798042297363, 0.10541319847106934, 0.10582637786865234, 0.10526084899902344, 0.1052250862121582, 0.1053462028503418, 0.10552477836608887, 0.1053915023803711, 0.10597777366638184, 0.10547947883605957, 0.10532665252685547, 0.1054682731628418, 0.10471725463867188, 0.10534858703613281, 0.10551905632019043, 0.10561919212341309, 0.1053013801574707, 0.10551834106445312, 0.10533428192138672, 0.10567545890808105, 0.1051332950592041, 0.10540199279785156, 0.10519146919250488, 0.10560822486877441, 0.1053469181060791, 0.10532665252685547, 0.1052398681640625, 0.10578036308288574, 0.10513091087341309, 0.1053617000579834, 0.10544443130493164, 0.10503149032592773, 0.10526585578918457, 0.10511040687561035, 0.10535621643066406, 0.10506677627563477, 0.10540485382080078, 0.1055765151977539, 0.10546088218688965, 0.1052708625793457, 0.10623574256896973, 0.10522770881652832, 0.10559678077697754, 0.10670042037963867, 0.1052858829498291, 0.10528731346130371, 0.10570621490478516, 0.10691285133361816, 0.10625672340393066, 0.10629510879516602, 0.10505890846252441, 0.10570669174194336, 0.10522341728210449, 0.10559225082397461, 0.10503649711608887, 0.10542035102844238, 0.10548019409179688, 0.10544466972351074, 0.10510039329528809, 0.10533475875854492, 0.10490679740905762, 0.1052403450012207, 0.10594534873962402, 0.10544109344482422, 0.10518670082092285, 0.10525989532470703, 0.1052248477935791, 0.105316162109375, 0.10539913177490234, 0.10502505302429199, 0.10535287857055664, 0.10519552230834961, 0.10559439659118652, 0.1054086685180664, 0.10513806343078613, 0.10599446296691895, 0.10559463500976562, 0.10507941246032715, 0.10491466522216797, 0.10495114326477051, 0.10514211654663086, 0.10527443885803223, 0.10509705543518066, 0.10525703430175781, 0.10579299926757812, 0.10521984100341797, 0.10531353950500488, 0.10613560676574707, 0.10515832901000977, 0.10518789291381836, 0.10643959045410156, 0.10594344139099121, 0.10554766654968262, 0.10521054267883301, 0.10561084747314453, 0.10513639450073242, 0.10533380508422852, 0.10538935661315918, 0.10487055778503418, 0.10543680191040039, 0.1053152084350586, 0.10549783706665039, 0.10508203506469727, 0.10606980323791504, 0.10509085655212402, 0.10493183135986328, 0.10544180870056152, 0.10512280464172363, 0.10518026351928711, 0.10555863380432129, 0.10521531105041504, 0.1053626537322998, 0.10520720481872559, 0.105377197265625, 0.10519742965698242, 0.10536956787109375, 0.10522341728210449, 0.10508990287780762, 0.1051797866821289, 0.10524940490722656, 0.1050252914428711, 0.10526132583618164, 0.10544443130493164, 0.10488462448120117, 0.1050410270690918, 0.10497498512268066, 0.10498547554016113, 0.10529589653015137, 0.10504937171936035, 0.10543489456176758, 0.10479235649108887, 0.10547208786010742, 0.10670280456542969, 0.1056208610534668, 0.10566592216491699, 0.10587072372436523, 0.1052863597869873, 0.10576629638671875, 0.10670757293701172, 0.10558938980102539, 0.10512161254882812, 0.10559868812561035, 0.10549068450927734, 0.10561418533325195, 0.10567259788513184, 0.10555100440979004, 0.10569000244140625, 0.10567522048950195, 0.10555148124694824, 0.10526609420776367, 0.10504937171936035, 0.10575675964355469, 0.10543346405029297, 0.10524702072143555, 0.10556244850158691, 0.1051337718963623, 0.10567855834960938, 0.10569143295288086, 0.10566592216491699, 0.10562324523925781, 0.10623717308044434, 0.10552668571472168, 0.10540127754211426, 0.10574102401733398, 0.1048734188079834, 0.10590076446533203, 0.10550332069396973, 0.10579752922058105, 0.10596585273742676, 0.10546660423278809, 0.10589790344238281, 0.10606503486633301, 0.10554933547973633, 0.10531377792358398, 0.10523223876953125, 0.10572504997253418, 0.10531234741210938, 0.10557174682617188, 0.1054234504699707, 0.1060938835144043, 0.10555148124694824, 0.10540533065795898, 0.10653042793273926, 0.10520458221435547, 0.10535335540771484, 0.10663032531738281, 0.1057741641998291, 0.10583138465881348, 0.10523200035095215, 0.1075139045715332, 0.10689473152160645, 0.1060481071472168, 0.10561823844909668, 0.10535788536071777, 0.10549211502075195, 0.10518956184387207, 0.10521984100341797, 0.1053614616394043, 0.10573768615722656, 0.10539102554321289, 0.10514259338378906, 0.1052854061126709, 0.10540509223937988, 0.10523271560668945, 0.10559248924255371, 0.10546755790710449, 0.10514450073242188, 0.1054689884185791, 0.10600686073303223, 0.10560846328735352, 0.10534119606018066, 0.10550332069396973, 0.10523056983947754, 0.10529279708862305, 0.10544681549072266, 0.10545611381530762, 0.10544347763061523, 0.10588431358337402, 0.10530424118041992, 0.10542702674865723, 0.10550951957702637, 0.10511493682861328, 0.10609722137451172, 0.10529565811157227, 0.10513710975646973, 0.10544753074645996, 0.10566329956054688, 0.10541868209838867, 0.10546183586120605, 0.10547971725463867, 0.10616779327392578, 0.10548806190490723, 0.10655474662780762, 0.10552263259887695, 0.10510826110839844, 0.10532760620117188, 0.1057584285736084, 0.10547661781311035, 0.10574817657470703, 0.10598468780517578, 0.1056833267211914, 0.10608196258544922, 0.10608935356140137, 0.10642576217651367, 0.10579466819763184, 0.10622262954711914, 0.10627150535583496, 0.10644221305847168, 0.10630393028259277, 0.10649943351745605, 0.10624241828918457, 0.10655856132507324, 0.1062159538269043, 0.10579562187194824, 0.10639667510986328, 0.10629653930664062, 0.10624957084655762, 0.10661125183105469, 0.1059577465057373, 0.10643577575683594, 0.12660765647888184, 0.13346052169799805, 0.13383698463439941, 0.13403534889221191, 0.13365697860717773, 0.13335275650024414, 0.13340973854064941, 0.13347983360290527, 0.133314847946167, 0.1333329677581787, 0.13339805603027344, 0.13439369201660156, 0.1338186264038086, 0.13419222831726074, 0.13339662551879883, 0.13408517837524414, 0.13405680656433105, 0.13381409645080566, 0.1339261531829834, 0.13364052772521973, 0.13357305526733398, 0.13352012634277344, 0.13351702690124512, 0.13332653045654297, 0.13330674171447754, 0.1334233283996582, 0.133864164352417, 0.1335301399230957, 0.13336634635925293, 0.13330984115600586, 0.13370537757873535, 0.13359427452087402, 0.13336920738220215, 0.13369488716125488, 0.13337087631225586, 0.13388419151306152, 0.13341665267944336, 0.13366436958312988, 0.13352417945861816, 0.1335160732269287, 0.13334369659423828, 0.1337599754333496, 0.13384056091308594, 0.13337922096252441, 0.13337159156799316, 0.13323521614074707, 0.1333928108215332, 0.1338045597076416, 0.13377022743225098, 0.13339591026306152, 0.13324904441833496, 0.13431167602539062, 0.13359856605529785, 0.13467168807983398, 0.1333141326904297, 0.13433504104614258, 0.13571381568908691, 0.13538122177124023, 0.13356447219848633, 526.3601763248444]
t_12_gzip = [0.11713075637817383, 0.129044771194458, 0.10468864440917969, 0.10460686683654785, 0.10453629493713379, 0.10503315925598145, 0.15204930305480957, 0.11807894706726074, 0.10482954978942871, 0.1046304702758789, 0.10483098030090332, 0.1043093204498291, 0.1047813892364502, 0.10446643829345703, 0.10491752624511719, 0.10482192039489746, 0.1045527458190918, 0.10432767868041992, 0.10444235801696777, 0.10467219352722168, 0.10483336448669434, 0.10453605651855469, 0.10530591011047363, 0.10423564910888672, 0.10494661331176758, 0.1051490306854248, 0.10451960563659668, 0.10421633720397949, 0.10458111763000488, 0.10462284088134766, 0.10449910163879395, 0.10447406768798828, 0.10466217994689941, 0.11950516700744629, 0.10477566719055176, 0.10459542274475098, 0.10501885414123535, 0.10488557815551758, 0.10471177101135254, 0.10576868057250977, 0.10477519035339355, 0.10513448715209961, 0.10510969161987305, 0.10472679138183594, 0.10470032691955566, 0.10445713996887207, 0.10409164428710938, 0.1045074462890625, 0.10447335243225098, 0.10422992706298828, 0.1041407585144043, 0.10406064987182617, 0.10472750663757324, 0.10390996932983398, 0.10450434684753418, 0.10443687438964844, 0.10422015190124512, 0.10444760322570801, 0.10509848594665527, 0.10412454605102539, 0.10465168952941895, 0.10394740104675293, 0.10533785820007324, 0.10506415367126465, 0.10558485984802246, 0.10452461242675781, 0.1044154167175293, 0.10480999946594238, 0.10463833808898926, 0.10475993156433105, 0.10463547706604004, 0.10473394393920898, 0.10489463806152344, 0.10468196868896484, 0.10483765602111816, 0.10464191436767578, 0.10476136207580566, 0.1049642562866211, 0.10505056381225586, 0.10502290725708008, 0.10463571548461914, 0.10516858100891113, 0.10505270957946777, 0.10465621948242188, 0.1054391860961914, 0.10498309135437012, 0.10473752021789551, 0.10596895217895508, 0.10531497001647949, 0.1050257682800293, 0.10516715049743652, 0.10458540916442871, 0.10542106628417969, 0.10496187210083008, 0.10503292083740234, 0.10477900505065918, 0.10466194152832031, 0.10512018203735352, 0.10490608215332031, 0.10461688041687012, 0.10509276390075684, 0.10485696792602539, 0.1053004264831543, 0.1049966812133789, 0.1053311824798584, 0.10469388961791992, 0.10509824752807617, 0.1049354076385498, 0.10489845275878906, 0.10515284538269043, 0.10501670837402344, 0.10470962524414062, 0.10499072074890137, 0.10467672348022461, 0.10492062568664551, 0.10482311248779297, 0.10517501831054688, 0.10519599914550781, 0.10474181175231934, 0.1053164005279541, 0.10549020767211914, 0.10474991798400879, 0.10499906539916992, 0.10520100593566895, 0.10500097274780273, 0.10506629943847656, 0.10496091842651367, 0.10455799102783203, 0.10501384735107422, 0.10503792762756348, 0.10488367080688477, 0.10507488250732422, 0.10527896881103516, 0.10473394393920898, 0.10575413703918457, 0.10497236251831055, 0.10506677627563477, 0.10477375984191895, 0.10512733459472656, 0.10761094093322754, 0.10519647598266602, 0.10491704940795898, 0.104888916015625, 0.10462450981140137, 0.10497570037841797, 0.10433197021484375, 0.10483479499816895, 0.10507845878601074, 0.10497331619262695, 0.10511279106140137, 0.10549283027648926, 0.10457801818847656, 0.10503125190734863, 0.10503506660461426, 0.10509085655212402, 0.10474395751953125, 0.10519003868103027, 0.10530710220336914, 0.10504436492919922, 0.10524535179138184, 0.10500049591064453, 0.10483908653259277, 0.10494780540466309, 0.10483360290527344, 0.10486030578613281, 0.1050412654876709, 0.10533523559570312, 0.10519671440124512, 0.10501646995544434, 0.10477399826049805, 0.10495352745056152, 0.1053311824798584, 0.1050255298614502, 0.10457396507263184, 0.10477781295776367, 0.10440206527709961, 0.10500884056091309, 0.10487031936645508, 0.10461807250976562, 0.10511946678161621, 0.10485219955444336, 0.1042931079864502, 0.10613656044006348, 0.10467958450317383, 0.10495591163635254, 0.10463356971740723, 0.10530257225036621, 0.10496926307678223, 0.10532188415527344, 0.10601472854614258, 0.10557794570922852, 0.10497212409973145, 0.10504865646362305, 0.10495638847351074, 0.10481047630310059, 0.10498976707458496, 0.1046898365020752, 0.105072021484375, 0.10486435890197754, 0.10538053512573242, 0.10522580146789551, 0.10556936264038086, 0.10502052307128906, 0.10500264167785645, 0.10462570190429688, 0.10497808456420898, 0.10450387001037598, 0.10500979423522949, 0.10471558570861816, 0.10508489608764648, 0.10491418838500977, 0.10489797592163086, 0.10524868965148926, 0.10502958297729492, 0.10529971122741699, 0.10526180267333984, 0.1045229434967041, 0.1048426628112793, 0.10494422912597656, 0.10510516166687012, 0.10472798347473145, 0.10500311851501465, 0.10513949394226074, 0.10480880737304688, 0.10598182678222656, 0.10519671440124512, 0.10480403900146484, 0.10564470291137695, 0.10490155220031738, 0.10492801666259766, 0.1056051254272461, 0.10535144805908203, 0.10483765602111816, 0.10492968559265137, 0.1049654483795166, 0.10506129264831543, 0.10477447509765625, 0.10487771034240723, 0.10469651222229004, 0.10512256622314453, 0.10475707054138184, 0.10498499870300293, 0.10472655296325684, 0.11044836044311523, 0.10480356216430664, 0.10532855987548828, 0.10466814041137695, 0.10515642166137695, 0.10456275939941406, 0.10485029220581055, 0.10486412048339844, 0.10470080375671387, 0.10497665405273438, 0.10473418235778809, 0.10530519485473633, 0.10470294952392578, 0.1044614315032959, 0.10485363006591797, 0.10462832450866699, 0.10479426383972168, 0.10485410690307617, 0.10443520545959473, 0.10474085807800293, 0.10470247268676758, 0.10483145713806152, 0.10446882247924805, 0.10472774505615234, 0.10482096672058105, 0.1049501895904541, 0.10441350936889648, 0.10479116439819336, 0.10489511489868164, 0.10485577583312988, 0.10491538047790527, 0.10545682907104492, 0.10480475425720215, 0.10462689399719238, 0.1061239242553711, 0.1048727035522461, 0.10513615608215332, 0.10457611083984375, 0.10539937019348145, 0.1078953742980957, 0.10539865493774414, 0.10454726219177246, 0.10499191284179688, 0.10485982894897461, 0.10496115684509277, 0.10441136360168457, 0.10503602027893066, 0.1047515869140625, 0.10485053062438965, 0.10485029220581055, 0.10518288612365723, 0.10476398468017578, 0.10477733612060547, 0.10535764694213867, 0.10469269752502441, 0.1048285961151123, 0.10470724105834961, 0.1051337718963623, 0.10442876815795898, 0.10505104064941406, 0.10483336448669434, 0.10488224029541016, 0.10443568229675293, 0.10466170310974121, 0.10456395149230957, 0.10474705696105957, 0.10483574867248535, 0.10494661331176758, 0.10513162612915039, 0.10510468482971191, 0.10465121269226074, 0.10541367530822754, 0.10509538650512695, 0.10478925704956055, 0.10490036010742188, 0.10497474670410156, 0.10533928871154785, 0.1046457290649414, 0.10534906387329102, 0.10589838027954102, 0.10526418685913086, 0.10492968559265137, 0.10661530494689941, 0.10519790649414062, 0.10515379905700684, 0.10521602630615234, 0.105255126953125, 0.1046748161315918, 0.10517239570617676, 0.10494256019592285, 0.10519051551818848, 0.10506892204284668, 0.10503315925598145, 0.10477495193481445, 0.10518217086791992, 0.10590028762817383, 0.10513472557067871, 0.10530257225036621, 0.10498261451721191, 0.1049795150756836, 0.1050269603729248, 0.10545039176940918, 0.10479068756103516, 0.10492849349975586, 0.10519123077392578, 0.10498237609863281, 0.10502052307128906, 0.10523390769958496, 0.10462236404418945, 0.10520219802856445, 0.10534191131591797, 0.10488343238830566, 0.10527300834655762, 0.10462594032287598, 0.10534167289733887, 0.10467052459716797, 0.10504770278930664, 0.10445284843444824, 0.10489797592163086, 0.10459566116333008, 0.10476827621459961, 0.10438776016235352, 0.10474777221679688, 0.10467195510864258, 0.10521936416625977, 0.10446667671203613, 0.10563445091247559, 0.10502839088439941, 0.10532855987548828, 0.1056361198425293, 0.10528779029846191, 0.10489797592163086, 0.105010986328125, 0.10495996475219727, 0.10528182983398438, 0.10521602630615234, 0.10499906539916992, 0.10489082336425781, 0.10513973236083984, 0.10512375831604004, 0.10476326942443848, 0.10518193244934082, 0.1052093505859375, 0.10504436492919922, 0.10482096672058105, 0.1049346923828125, 0.1049950122833252, 0.10512185096740723, 0.10497832298278809, 0.10498666763305664, 0.10485029220581055, 0.10473752021789551, 0.10521626472473145, 0.10462617874145508, 0.10495972633361816, 0.10474729537963867, 0.1052389144897461, 0.10481882095336914, 0.10504579544067383, 0.10511898994445801, 0.10531091690063477, 0.1044609546661377, 0.10543394088745117, 0.10525274276733398, 0.10477375984191895, 0.10468554496765137, 0.10503935813903809, 0.10469341278076172, 0.10489320755004883, 0.10463070869445801, 0.10505819320678711, 0.10557293891906738, 0.1049051284790039, 0.10476493835449219, 0.10576605796813965, 0.1047208309173584, 0.10494565963745117, 0.10640501976013184, 0.10479283332824707, 0.10524296760559082, 0.10437536239624023, 0.10676431655883789, 0.10562443733215332, 0.10509800910949707, 0.10511112213134766, 0.10485172271728516, 0.10484719276428223, 0.10505366325378418, 0.10471129417419434, 0.10489773750305176, 0.10500764846801758, 0.10493326187133789, 0.10462594032287598, 0.1048433780670166, 0.10501980781555176, 0.10482597351074219, 0.10547661781311035, 0.10499191284179688, 0.1053152084350586, 0.10501432418823242, 0.10561823844909668, 0.10509467124938965, 0.10487008094787598, 0.10498261451721191, 0.1053159236907959, 0.10512900352478027, 0.10546517372131348, 0.10484957695007324, 0.10493803024291992, 0.10587811470031738, 0.10507702827453613, 0.10495829582214355, 0.10466575622558594, 0.10469436645507812, 0.1053922176361084, 0.10488533973693848, 0.10494256019592285, 0.10506343841552734, 0.1047828197479248, 0.10554718971252441, 0.10505914688110352, 0.10524821281433105, 0.1057281494140625, 0.10504674911499023, 0.10503220558166504, 0.10621476173400879, 0.10521650314331055, 0.10502862930297852, 0.10525870323181152, 0.10521793365478516, 0.10511088371276855, 0.10547256469726562, 0.10535573959350586, 0.10531973838806152, 0.10510087013244629, 0.10484504699707031, 0.10513091087341309, 0.1048581600189209, 0.10546755790710449, 0.10511279106140137, 0.10521769523620605, 0.10551118850708008, 0.10507822036743164, 0.10519647598266602, 0.10515213012695312, 0.10508179664611816, 0.10527300834655762, 0.10524272918701172, 0.10529923439025879, 0.10530424118041992, 0.10558533668518066, 0.10514712333679199, 0.10548067092895508, 0.1050100326538086, 0.10516476631164551, 0.10556745529174805, 0.10505104064941406, 0.10590887069702148, 0.10565018653869629, 0.10480546951293945, 0.10486435890197754, 0.10533857345581055, 0.10485529899597168, 0.10538434982299805, 0.10515570640563965, 0.10522699356079102, 0.10554981231689453, 0.10490036010742188, 0.10500764846801758, 0.10603761672973633, 0.10481476783752441, 0.1053476333618164, 0.10635495185852051, 0.1051025390625, 0.10550665855407715, 0.10492086410522461, 0.10565376281738281, 0.1054372787475586, 0.10502195358276367, 0.10550284385681152, 0.1053781509399414, 0.10513043403625488, 0.10538506507873535, 0.10512280464172363, 0.10558414459228516, 0.1054534912109375, 0.10555171966552734, 0.10511898994445801, 0.10538077354431152, 0.10518240928649902, 0.10499978065490723, 0.10581064224243164, 0.10502505302429199, 0.1052393913269043, 0.10520195960998535, 0.10572290420532227, 0.10541296005249023, 0.105224609375, 0.10518407821655273, 0.10567927360534668, 0.10531115531921387, 0.10549092292785645, 0.10498046875, 0.1052863597869873, 0.10524773597717285, 0.10516929626464844, 0.10547780990600586, 0.10519838333129883, 0.10493612289428711, 0.10549497604370117, 0.10524749755859375, 0.10521602630615234, 0.10535502433776855, 0.10485053062438965, 0.10541772842407227, 0.1050577163696289, 0.10494804382324219, 0.10601997375488281, 0.10544323921203613, 0.1053016185760498, 0.10600805282592773, 0.10527491569519043, 0.1054530143737793, 0.10579323768615723, 0.10761189460754395, 0.10601568222045898, 0.10573935508728027, 0.10522317886352539, 0.10541582107543945, 0.1053311824798584, 0.10519862174987793, 0.10540127754211426, 0.10514545440673828, 0.10554695129394531, 0.10524797439575195, 0.10552382469177246, 0.1055138111114502, 0.10507774353027344, 0.10528755187988281, 0.10529851913452148, 0.10497331619262695, 0.10524916648864746, 0.10511422157287598, 0.10524177551269531, 0.10515999794006348, 0.10543203353881836, 0.10510063171386719, 0.10516023635864258, 0.10526752471923828, 0.1055135726928711, 0.10529470443725586, 0.1050262451171875, 0.10571455955505371, 0.10552549362182617, 0.10535407066345215, 0.10836911201477051, 0.11946368217468262, 0.22455859184265137, 0.14850187301635742, 0.13304471969604492, 0.1688992977142334, 0.10716891288757324, 0.1264944076538086, 0.1647193431854248, 0.10777473449707031, 0.16530656814575195, 0.10521173477172852, 0.10575008392333984, 0.10508871078491211, 0.1051182746887207, 0.10683989524841309, 0.1052398681640625, 0.10527658462524414, 0.10771560668945312, 0.10555911064147949, 0.1051323413848877, 0.1060340404510498, 0.10514569282531738, 0.10553860664367676, 0.10727286338806152, 0.10681629180908203, 0.10530924797058105, 0.10567212104797363, 0.12376785278320312, 0.13211965560913086, 0.10535073280334473, 0.10565733909606934, 0.12113833427429199, 0.12503981590270996, 0.10878992080688477, 0.1567525863647461, 0.10870623588562012, 0.15446710586547852, 0.14051461219787598, 0.10665583610534668, 0.10806775093078613, 0.10553979873657227, 0.10895013809204102, 0.10522270202636719, 0.11805558204650879, 0.10506582260131836, 0.10599398612976074, 0.10628056526184082, 0.10532021522521973, 0.10515952110290527, 0.1063530445098877, 0.10532116889953613, 0.10561871528625488, 0.10668516159057617, 0.10545492172241211, 0.1052999496459961, 0.10599040985107422, 0.10560083389282227, 0.11107015609741211, 0.1056833267211914, 0.10495352745056152, 0.10544586181640625, 0.10533761978149414, 0.10609579086303711, 0.10509276390075684, 0.10526371002197266, 0.10534191131591797, 0.10544514656066895, 0.10680651664733887, 0.10600161552429199, 0.1051027774810791, 0.1051778793334961, 0.1051795482635498, 0.10522198677062988, 0.10495901107788086, 0.10584092140197754, 0.10541415214538574, 0.10499930381774902, 0.1058187484741211, 0.10731911659240723, 0.10510969161987305, 0.10492539405822754, 0.10490989685058594, 0.10546493530273438, 0.10520601272583008, 0.10548520088195801, 0.10553717613220215, 0.1052541732788086, 0.10490036010742188, 0.10487961769104004, 0.10518574714660645, 0.10522699356079102, 0.10538744926452637, 0.10544204711914062, 0.10545969009399414, 0.10587453842163086, 0.10532379150390625, 0.1334857940673828, 0.13689422607421875, 0.10574007034301758, 0.10708880424499512, 0.1685175895690918, 0.1057744026184082, 0.1051328182220459, 0.18599414825439453, 0.10574769973754883, 0.1689138412475586, 0.10869407653808594, 0.10539793968200684, 0.17792701721191406, 0.10616374015808105, 0.10540437698364258, 0.16914987564086914, 0.10525751113891602, 0.10564756393432617, 0.10746502876281738, 0.10816693305969238, 0.10550570487976074, 0.10547256469726562, 0.10540270805358887, 0.10596466064453125, 0.10527253150939941, 0.10561299324035645, 0.10522866249084473, 0.10568070411682129, 0.10570096969604492, 0.10588788986206055, 0.1053311824798584, 0.10571503639221191, 0.10535502433776855, 0.10556769371032715, 0.10531735420227051, 0.10544943809509277, 0.10614395141601562, 0.10560059547424316, 0.10537052154541016, 0.10540270805358887, 0.10543441772460938, 0.10542774200439453, 0.1051180362701416, 0.10526609420776367, 0.10665535926818848, 0.10572361946105957, 0.10544276237487793, 0.10683107376098633, 0.10536623001098633, 0.10550260543823242, 0.1051628589630127, 0.10563921928405762, 0.1058204174041748, 0.10556817054748535, 0.10575199127197266, 0.10538458824157715, 0.10521769523620605, 0.10522723197937012, 0.10463976860046387, 0.1057126522064209, 0.10512590408325195, 0.1055755615234375, 0.1052865982055664, 0.10557746887207031, 0.1049659252166748, 0.10516095161437988, 0.1052093505859375, 0.10497021675109863, 0.10477328300476074, 0.10512447357177734, 0.1049492359161377, 0.10499739646911621, 0.1052088737487793, 0.1048746109008789, 0.10514020919799805, 0.10486960411071777, 0.10469460487365723, 0.10590529441833496, 0.10501480102539062, 0.10582756996154785, 0.10793471336364746, 0.10512495040893555, 0.10516810417175293, 0.10503363609313965, 0.1049509048461914, 0.10516929626464844, 0.10487055778503418, 0.1053769588470459, 0.10590291023254395, 0.10547780990600586, 0.10494542121887207, 0.10544061660766602, 0.1061697006225586, 0.10521912574768066, 0.10618996620178223, 0.10607504844665527, 0.10521912574768066, 0.1055450439453125, 0.10493922233581543, 0.1055300235748291, 0.1049039363861084, 0.10544085502624512, 0.10471963882446289, 0.10569143295288086, 0.10516023635864258, 0.10497784614562988, 0.10501432418823242, 0.10593080520629883, 0.10555553436279297, 0.10549020767211914, 0.10503649711608887, 0.10505247116088867, 0.10490012168884277, 0.10516476631164551, 0.1049497127532959, 0.10477089881896973, 0.10465025901794434, 0.10503745079040527, 0.10517191886901855, 0.10477948188781738, 0.10483360290527344, 0.10479474067687988, 0.1051785945892334, 0.1052093505859375, 0.10541129112243652, 0.1046760082244873, 0.1052098274230957, 0.10498189926147461, 0.10515141487121582, 0.10507726669311523, 0.1065676212310791, 0.10677909851074219, 0.10623931884765625, 0.10505080223083496, 0.1067204475402832, 0.10604476928710938, 0.10572171211242676, 0.10511422157287598, 0.10507965087890625, 0.1062173843383789, 0.10497403144836426, 0.10499835014343262, 0.10680484771728516, 0.10492134094238281, 0.10528707504272461, 0.10555005073547363, 0.10866427421569824, 0.10544323921203613, 0.10561275482177734, 0.10474300384521484, 0.10512280464172363, 0.10480332374572754, 0.10532212257385254, 0.10463809967041016, 0.1054229736328125, 0.10507369041442871, 0.10525965690612793, 0.10498881340026855, 0.10517215728759766, 0.10473179817199707, 0.10493707656860352, 0.1048884391784668, 0.10503721237182617, 0.10473251342773438, 0.10527515411376953, 0.10499715805053711, 0.10481023788452148, 0.10493707656860352, 0.1048588752746582, 0.10488557815551758, 0.10496234893798828, 0.10524129867553711, 0.10535883903503418, 0.10500311851501465, 0.10585403442382812, 0.10555315017700195, 0.1051950454711914, 0.10498571395874023, 0.10495209693908691, 0.10514068603515625, 0.10517072677612305, 0.10484743118286133, 0.1052558422088623, 0.10526728630065918, 0.1055138111114502, 0.10487103462219238, 0.10538411140441895, 0.10565352439880371, 0.10594296455383301, 0.1063394546508789, 0.10531926155090332, 0.1051332950592041, 0.1051175594329834, 0.10500979423522949, 0.10578393936157227, 0.10477566719055176, 0.10501289367675781, 0.10484457015991211, 0.10503053665161133, 0.10498189926147461, 0.10527682304382324, 0.10463643074035645, 0.1060178279876709, 0.10506248474121094, 0.10494017601013184, 0.10461306571960449, 0.10512852668762207, 0.10484170913696289, 0.10541152954101562, 0.1049656867980957, 0.10479259490966797, 0.10471606254577637, 0.10530686378479004, 0.10542130470275879, 0.10549592971801758, 0.10455965995788574, 0.10522174835205078, 0.10503339767456055, 0.10512065887451172, 0.10512614250183105, 0.10567831993103027, 0.10494256019592285, 0.1054379940032959, 0.10553622245788574, 0.10499191284179688, 0.10491490364074707, 0.10483074188232422, 0.10477137565612793, 0.10519528388977051, 0.1051023006439209, 0.10515832901000977, 0.10538578033447266, 0.10540628433227539, 0.10485219955444336, 0.10602784156799316, 0.10473132133483887, 0.10533380508422852, 0.10628032684326172, 0.10455131530761719, 0.10477662086486816, 0.10515069961547852, 0.10498309135437012, 0.10569190979003906, 0.10550975799560547, 0.10514092445373535, 0.10542869567871094, 0.10497379302978516, 0.10488414764404297, 0.10520792007446289, 0.10581660270690918, 0.1056971549987793, 0.10535836219787598, 0.10528063774108887, 0.10554122924804688, 0.1049661636352539, 0.10504531860351562, 0.10539460182189941, 0.10527586936950684, 0.10529923439025879, 0.10558080673217773, 0.10509133338928223, 0.10537266731262207, 0.10527276992797852, 0.10523247718811035, 0.10561299324035645, 0.10544061660766602, 0.10502910614013672, 0.1057429313659668, 0.10539722442626953, 0.10585713386535645, 0.10578012466430664, 0.1053931713104248, 0.10510993003845215, 0.10514950752258301, 0.10532736778259277, 0.10501646995544434, 0.10557818412780762, 0.10555577278137207, 0.10546398162841797, 0.10557913780212402, 0.10550904273986816, 0.1049799919128418, 0.10645246505737305, 0.10526895523071289, 0.10587477684020996, 0.10529112815856934, 0.10549616813659668, 0.10523724555969238, 0.10551166534423828, 0.10807323455810547, 0.10554385185241699, 0.10548877716064453, 0.10529971122741699, 0.10518574714660645, 0.10523247718811035, 0.10481119155883789, 0.10506176948547363, 0.10515904426574707, 0.10519576072692871, 0.10545516014099121, 0.1052098274230957, 0.10497045516967773, 0.10492920875549316, 0.1054389476776123, 0.10494828224182129, 0.10482621192932129, 0.10489630699157715, 0.10517501831054688, 0.10556793212890625, 0.10526108741760254, 0.10529136657714844, 0.10511541366577148, 0.10518074035644531, 0.10525012016296387, 0.1051340103149414, 0.10502052307128906, 0.10557937622070312, 0.10542011260986328, 0.1046149730682373, 0.10537290573120117, 0.10485386848449707, 0.10518693923950195, 0.10541343688964844, 0.10487771034240723, 0.1053006649017334, 0.10512661933898926, 0.1054997444152832, 0.1051337718963623, 0.10511255264282227, 0.10579586029052734, 0.10487246513366699, 0.10474514961242676, 0.10660505294799805, 0.1052403450012207, 0.10547184944152832, 0.1051788330078125, 0.10494756698608398, 0.1054840087890625, 0.10484075546264648, 0.10562729835510254, 0.10500526428222656, 0.10540628433227539, 0.10530686378479004, 0.1052544116973877, 0.10554218292236328, 0.10552859306335449, 0.10516905784606934, 0.10520100593566895, 0.10537958145141602, 0.10541653633117676, 0.10515022277832031, 0.10540270805358887, 0.10534477233886719, 0.10497093200683594, 0.10538172721862793, 0.10540008544921875, 0.10506224632263184, 0.10521316528320312, 0.10511302947998047, 0.10535526275634766, 0.10513997077941895, 0.10547852516174316, 0.1054842472076416, 0.10477256774902344, 0.10594439506530762, 0.10553407669067383, 0.10497760772705078, 0.10515928268432617, 0.10490226745605469, 0.10518836975097656, 0.10510587692260742, 0.10513496398925781, 0.10518813133239746, 0.10584735870361328, 0.1056821346282959, 0.10528445243835449, 0.10529494285583496, 0.10580825805664062, 0.10497236251831055, 0.10589432716369629, 0.10504627227783203, 0.10524487495422363, 0.1050715446472168, 0.10529923439025879, 0.10537838935852051, 0.10530519485473633, 0.10489654541015625, 0.10520029067993164, 0.10507988929748535, 0.1052086353302002, 0.10517048835754395, 0.1049203872680664, 0.10516500473022461, 0.10528206825256348, 0.10524678230285645, 0.10533881187438965, 0.10525298118591309, 0.10527300834655762, 0.10517501831054688, 0.10507345199584961, 0.1052544116973877, 0.10467648506164551, 0.10524392127990723, 0.10503506660461426, 0.10537934303283691, 0.10488581657409668, 0.1050567626953125, 0.10517692565917969, 0.10486602783203125, 0.1053156852722168, 0.10487174987792969, 0.10485696792602539, 0.10538864135742188, 0.10495185852050781, 0.10480046272277832, 0.10506200790405273, 0.10460638999938965, 0.104949951171875, 0.10503196716308594, 0.10493350028991699, 0.1047525405883789, 0.10531449317932129, 0.10509419441223145, 0.10501241683959961, 0.10570693016052246, 0.10519790649414062, 0.10492181777954102, 0.10673761367797852, 0.10533976554870605, 0.10524868965148926, 0.10644984245300293, 0.10729646682739258, 0.11031270027160645, 0.1056661605834961, 0.10522913932800293, 0.10520291328430176, 0.10517716407775879, 0.10527849197387695, 0.1052553653717041, 0.10506820678710938, 0.10527706146240234, 0.10508370399475098, 0.10543131828308105, 0.10538482666015625, 0.10558867454528809, 0.1050710678100586, 0.10517668724060059, 0.10497784614562988, 0.10532450675964355, 0.10528421401977539, 0.10518336296081543, 0.10501742362976074, 0.10499310493469238, 0.10485577583312988, 0.10483050346374512, 0.10504841804504395, 0.10517454147338867, 0.10551142692565918, 0.10501933097839355, 0.1060490608215332, 0.10507941246032715, 0.10535836219787598, 0.1049337387084961, 0.10497450828552246, 0.10527467727661133, 0.1051790714263916, 0.10525894165039062, 0.10509181022644043, 0.10648274421691895, 0.11004853248596191, 0.10527682304382324, 0.10548877716064453, 0.10609984397888184, 0.10518741607666016, 0.10663294792175293, 0.1057741641998291, 0.10566186904907227, 0.10511207580566406, 0.10555791854858398, 0.10535740852355957, 0.10549569129943848, 0.10536789894104004, 0.1053321361541748, 0.10530996322631836, 0.1052255630493164, 0.10525989532470703, 0.10509419441223145, 0.1054236888885498, 0.10521268844604492, 0.10548830032348633, 0.10535168647766113, 0.10509204864501953, 0.10574769973754883, 0.10543966293334961, 0.10471296310424805, 0.10503435134887695, 0.10472893714904785, 0.10530257225036621, 0.10506725311279297, 0.10581040382385254, 0.1051626205444336, 0.10524892807006836, 0.1055612564086914, 0.10495972633361816, 0.10519266128540039, 0.10468816757202148, 0.10533475875854492, 0.10503649711608887, 0.10541701316833496, 0.10499310493469238, 0.10531067848205566, 0.1050424575805664, 0.10509920120239258, 0.10495853424072266, 0.1050863265991211, 0.10462617874145508, 0.10563206672668457, 0.10488080978393555, 0.10512471199035645, 0.10555624961853027, 0.10535717010498047, 0.10491943359375, 0.10652518272399902, 0.10499119758605957, 0.10509204864501953, 0.10488104820251465, 0.10508060455322266, 0.10549211502075195, 0.10524153709411621, 0.10486245155334473, 0.1048433780670166, 0.10493874549865723, 0.10486364364624023, 0.1047675609588623, 0.1050405502319336, 0.10524201393127441, 0.10473752021789551, 0.10499858856201172, 0.10527873039245605, 0.10483551025390625, 0.10493111610412598, 0.10551714897155762, 0.10516643524169922, 0.10457849502563477, 0.10550713539123535, 0.10500097274780273, 0.10466837882995605, 0.10498237609863281, 0.10498857498168945, 0.10521245002746582, 0.10512304306030273, 0.10493087768554688, 0.10534524917602539, 0.10500431060791016, 0.1058037281036377, 0.10473823547363281, 0.10498738288879395, 0.10498762130737305, 0.10525798797607422, 0.10492968559265137, 0.10499930381774902, 0.10478353500366211, 0.10503506660461426, 0.10526490211486816, 0.10526156425476074, 0.10525822639465332, 0.1050117015838623, 0.1058197021484375, 0.10497164726257324, 0.10583829879760742, 0.10526728630065918, 0.10520124435424805, 0.10519576072692871, 0.10530948638916016, 0.10685133934020996, 0.10515284538269043, 0.10523319244384766, 0.10491704940795898, 0.1050870418548584, 0.10483026504516602, 0.10474038124084473, 0.10510587692260742, 0.10517024993896484, 0.10478353500366211, 0.10579729080200195, 0.10503053665161133, 0.10514688491821289, 0.10496377944946289, 0.10508584976196289, 0.10483908653259277, 0.10510540008544922, 0.1051177978515625, 0.10549020767211914, 0.10549688339233398, 0.10520672798156738, 0.10482001304626465, 0.10499167442321777, 0.10458731651306152, 0.10505247116088867, 0.10487151145935059, 0.10526394844055176, 0.10513162612915039, 0.10542750358581543, 0.10471796989440918, 0.10525035858154297, 0.10479116439819336, 0.10575532913208008, 0.10498666763305664, 0.10489773750305176, 0.10500001907348633, 0.10510826110839844, 0.10490798950195312, 0.1051175594329834, 0.10486531257629395, 0.10555028915405273, 0.10500311851501465, 0.10510778427124023, 0.10595035552978516, 0.1048593521118164, 0.10522294044494629, 0.10476374626159668, 0.10542964935302734, 0.10559678077697754, 0.1047966480255127, 0.10601806640625, 0.1052548885345459, 0.10499835014343262, 0.10529065132141113, 0.10527467727661133, 0.10461139678955078, 0.10529279708862305, 0.10512447357177734, 0.10530471801757812, 0.10486459732055664, 0.10556840896606445, 0.10495734214782715, 0.10540294647216797, 0.10483813285827637, 0.10505938529968262, 0.10473346710205078, 0.10525083541870117, 0.10483741760253906, 0.10515594482421875, 0.10497856140136719, 0.10529971122741699, 0.10475635528564453, 0.10500454902648926, 0.10585546493530273, 0.10507583618164062, 0.10561203956604004, 0.10559439659118652, 0.10484981536865234, 0.10501575469970703, 0.10472249984741211, 0.1049041748046875, 0.10487484931945801, 0.10498929023742676, 0.10481691360473633, 0.10571098327636719, 0.10528397560119629, 0.10512137413024902, 0.10478043556213379, 0.10572671890258789, 0.10485696792602539, 0.10591650009155273, 0.10524892807006836, 0.10558938980102539, 0.10489511489868164, 0.10523486137390137, 0.10514307022094727, 0.10520648956298828, 0.10480451583862305, 0.10520267486572266, 0.10469889640808105, 0.10530924797058105, 0.10446763038635254, 0.10545110702514648, 0.10478711128234863, 0.1057441234588623, 0.1048121452331543, 0.10511589050292969, 0.10477972030639648, 0.1051642894744873, 0.10515356063842773, 0.10527610778808594, 0.10467863082885742, 0.10498666763305664, 0.10491752624511719, 0.10508131980895996, 0.10470747947692871, 0.10507488250732422, 0.10474538803100586, 0.10517001152038574, 0.10489726066589355, 0.10502386093139648, 0.10465860366821289, 0.10497689247131348, 0.1050405502319336, 0.10476160049438477, 0.10506749153137207, 0.10488533973693848, 0.10472369194030762, 0.1047666072845459, 0.10486054420471191, 0.10503983497619629, 0.10455799102783203, 0.10516977310180664, 0.10486412048339844, 0.10487961769104004, 0.10533380508422852, 0.10490584373474121, 0.10465478897094727, 0.10617876052856445, 0.10476088523864746, 0.10533475875854492, 0.10476803779602051, 0.10664558410644531, 0.10640525817871094, 0.1056814193725586, 0.10484719276428223, 0.10535550117492676, 0.10442924499511719, 0.10538458824157715, 0.10449743270874023, 0.1052396297454834, 0.10509991645812988, 0.10521078109741211, 0.10507774353027344, 0.10538244247436523, 0.10487771034240723, 0.10474586486816406, 0.10517096519470215, 0.10509276390075684, 0.10474228858947754, 0.10633707046508789, 0.10515594482421875, 0.10497379302978516, 0.1051328182220459, 0.10528707504272461, 0.10517668724060059, 0.10500574111938477, 0.1048727035522461, 0.10543346405029297, 0.10501933097839355, 0.1056528091430664, 0.1053321361541748, 0.1051023006439209, 0.10520124435424805, 0.10499882698059082, 0.10498666763305664, 0.10519170761108398, 0.10482120513916016, 0.10562634468078613, 0.10501670837402344, 0.10597515106201172, 0.10527205467224121, 0.10541319847106934, 0.1056356430053711, 0.10581779479980469, 0.10500121116638184, 0.10697770118713379, 0.10527443885803223, 0.10512995719909668, 0.10495662689208984, 0.1055898666381836, 0.10472989082336426, 0.10515522956848145, 0.10494804382324219, 0.10490942001342773, 0.10498833656311035, 0.10489082336425781, 0.10489201545715332, 0.10509967803955078, 0.10564470291137695, 0.10524415969848633, 0.10511898994445801, 0.10512113571166992, 0.10522770881652832, 0.10500907897949219, 0.10538315773010254, 0.10484194755554199, 0.10504770278930664, 0.10515952110290527, 0.10509419441223145, 0.105377197265625, 0.10492467880249023, 0.10492873191833496, 0.10494875907897949, 0.10503983497619629, 0.10471844673156738, 0.10511088371276855, 0.1047065258026123, 0.10547375679016113, 0.1048438549041748, 0.10521078109741211, 0.10480904579162598, 0.10503339767456055, 0.10479116439819336, 0.10486292839050293, 0.10458827018737793, 0.10497093200683594, 0.10501575469970703, 0.10507321357727051, 0.10486888885498047, 0.10585498809814453, 0.10498356819152832, 0.1051328182220459, 0.10628962516784668, 0.10497903823852539, 0.1047368049621582, 0.10485363006591797, 0.10520648956298828, 0.10517716407775879, 0.10508394241333008, 0.10517120361328125, 0.10512900352478027, 0.1049809455871582, 0.10505104064941406, 0.10498929023742676, 0.1051483154296875, 0.10515356063842773, 0.10485219955444336, 0.10497641563415527, 0.10524487495422363, 0.10508155822753906, 0.10486888885498047, 0.10534858703613281, 0.10484147071838379, 0.1050100326538086, 0.10454559326171875, 0.10557103157043457, 0.10462594032287598, 0.1052846908569336, 0.1048591136932373, 0.10515117645263672, 0.1049964427947998, 0.10524654388427734, 0.10504770278930664, 0.10498571395874023, 0.10528779029846191, 0.10554885864257812, 0.1047365665435791, 0.10520267486572266, 0.10492420196533203, 0.10515069961547852, 0.1045689582824707, 0.10495567321777344, 0.10485005378723145, 0.10464215278625488, 0.10545730590820312, 0.10480093955993652, 0.10497188568115234, 0.10573053359985352, 0.10490822792053223, 0.10450577735900879, 0.10605335235595703, 0.10529279708862305, 0.10510611534118652, 0.10492467880249023, 0.10729336738586426, 0.10560059547424316, 0.10534095764160156, 0.10487985610961914, 0.10505270957946777, 0.10497331619262695, 0.1049952507019043, 0.10499072074890137, 0.10455870628356934, 0.10489892959594727, 0.10490226745605469, 0.10485482215881348, 0.10472536087036133, 0.10480928421020508, 0.10489678382873535, 0.10522818565368652, 0.10468673706054688, 0.10478019714355469, 0.10518169403076172, 0.10564470291137695, 0.1049802303314209, 0.10553812980651855, 0.10457658767700195, 0.10519027709960938, 0.10509395599365234, 0.10545635223388672, 0.10478019714355469, 0.10510659217834473, 0.10547542572021484, 0.10531330108642578, 0.10478878021240234, 0.10480570793151855, 0.10490155220031738, 0.10513854026794434, 0.10518598556518555, 0.10513973236083984, 0.10506296157836914, 0.10489130020141602, 0.10486435890197754, 0.10487723350524902, 0.1054832935333252, 0.10565066337585449, 0.10500335693359375, 0.10631203651428223, 0.10503792762756348, 0.10507607460021973, 0.10490584373474121, 0.10534930229187012, 0.10535240173339844, 0.10511565208435059, 0.10545992851257324, 0.1053919792175293, 0.10468411445617676, 0.10530257225036621, 0.10479307174682617, 0.10519886016845703, 0.10486912727355957, 0.10519552230834961, 0.10484075546264648, 0.10511016845703125, 0.1049647331237793, 0.10517358779907227, 0.10504555702209473, 0.1053934097290039, 0.10424470901489258, 0.10526442527770996, 0.10489916801452637, 0.10494852066040039, 0.10492944717407227, 0.10496139526367188, 0.10512590408325195, 0.10488510131835938, 0.1049036979675293, 0.10531497001647949, 0.10500717163085938, 0.10545945167541504, 0.10560035705566406, 0.10499095916748047, 0.1053934097290039, 0.10463833808898926, 0.10511422157287598, 0.10504007339477539, 0.10513687133789062, 0.10502982139587402, 0.10467290878295898, 0.10574722290039062, 0.10480952262878418, 0.10480046272277832, 0.10560750961303711, 0.10532140731811523, 0.10503268241882324, 0.10605597496032715, 0.10523390769958496, 0.10570502281188965, 0.10467743873596191, 0.10529732704162598, 0.10465097427368164, 0.10521507263183594, 0.10478377342224121, 0.1047816276550293, 0.10496377944946289, 0.1049659252166748, 0.10477805137634277, 0.10502362251281738, 0.10477113723754883, 0.10549211502075195, 0.1045832633972168, 0.10489416122436523, 0.10463142395019531, 0.10521173477172852, 0.10515379905700684, 0.10495328903198242, 0.10501599311828613, 0.10508012771606445, 0.10491490364074707, 0.10496807098388672, 0.10495734214782715, 0.10497188568115234, 0.10512375831604004, 0.1046760082244873, 0.10482668876647949, 0.1044321060180664, 0.10487794876098633, 0.10467958450317383, 0.10482382774353027, 0.10502481460571289, 0.10489964485168457, 0.10498833656311035, 0.1048135757446289, 0.10499000549316406, 0.10469484329223633, 0.10487675666809082, 0.10459399223327637, 0.10512280464172363, 0.10457134246826172, 0.10520672798156738, 0.10576868057250977, 0.10549283027648926, 0.10464906692504883, 0.10587835311889648, 0.10487055778503418, 0.10483598709106445, 0.10579895973205566, 0.108306884765625, 0.10546612739562988, 0.10531377792358398, 0.10486030578613281, 0.10517525672912598, 0.10493278503417969, 0.1047976016998291, 0.10507631301879883, 0.10710978507995605, 0.10574674606323242, 0.10520315170288086, 0.10520696640014648, 0.10556221008300781, 0.10535693168640137, 0.10529065132141113, 0.10541844367980957, 0.10495376586914062, 0.10498046875, 0.10471081733703613, 0.10539817810058594, 0.10526800155639648, 0.10508060455322266, 0.10540509223937988, 0.1051182746887207, 0.10502481460571289, 0.1051950454711914, 0.10566186904907227, 0.10527753829956055, 0.10557675361633301, 0.10531330108642578, 0.10540366172790527, 0.10486555099487305, 0.10507869720458984, 0.10469841957092285, 0.10534834861755371, 0.1052103042602539, 0.10498762130737305, 0.10957646369934082, 0.10536408424377441, 0.10519027709960938, 0.10595941543579102, 0.10509085655212402, 0.10521268844604492, 0.10663914680480957, 0.10530543327331543, 0.10610842704772949, 0.10526132583618164, 0.10513758659362793, 0.10518336296081543, 0.10517716407775879, 0.1052103042602539, 0.10505437850952148, 0.10529971122741699, 0.10520434379577637, 0.10501790046691895, 0.10528969764709473, 0.10563230514526367, 0.10525989532470703, 0.10515379905700684, 0.105133056640625, 0.10504269599914551, 0.10528159141540527, 0.10576772689819336, 0.10521864891052246, 0.10530662536621094, 0.10468697547912598, 0.10539388656616211, 0.10486626625061035, 0.10529518127441406, 0.10503005981445312, 0.10531401634216309, 0.10484170913696289, 0.10524940490722656, 0.10459446907043457, 0.10460066795349121, 0.1048429012298584, 0.10544085502624512, 0.10494875907897949, 0.10506653785705566, 0.10507822036743164, 0.10492229461669922, 0.10499334335327148, 0.10526776313781738, 0.1047515869140625, 0.10512638092041016, 0.10491585731506348, 0.10505294799804688, 0.10531377792358398, 0.10567283630371094, 0.10514044761657715, 0.10536384582519531, 0.10545468330383301, 0.10495519638061523, 0.10515379905700684, 0.10506367683410645, 0.1055605411529541, 0.10554265975952148, 0.1053776741027832, 0.1047675609588623, 0.10512113571166992, 0.10511660575866699, 0.10483837127685547, 0.10526704788208008, 0.10472965240478516, 0.10522317886352539, 0.10471320152282715, 0.10526418685913086, 0.10519981384277344, 0.10484194755554199, 0.10484457015991211, 0.10544753074645996, 0.10485267639160156, 0.10521292686462402, 0.10508942604064941, 0.10538220405578613, 0.10480165481567383, 0.10529208183288574, 0.10512924194335938, 0.1050407886505127, 0.10475540161132812, 0.10550260543823242, 0.10531044006347656, 0.10504722595214844, 0.105377197265625, 0.10580277442932129, 0.10515069961547852, 0.10489201545715332, 0.10528397560119629, 0.10462141036987305, 0.10513138771057129, 0.10535597801208496, 0.10514950752258301, 0.10560345649719238, 0.10525202751159668, 0.12848544120788574, 0.10920882225036621, 0.1057581901550293, 0.10497164726257324, 0.10614371299743652, 0.1052541732788086, 0.10540008544921875, 0.10468578338623047, 0.10806679725646973, 0.10559940338134766, 0.10573339462280273, 0.10477542877197266, 0.1054086685180664, 0.10505533218383789, 0.10507893562316895, 0.10684537887573242, 0.10561895370483398, 0.10523295402526855, 0.10544824600219727, 0.10543537139892578, 0.1051182746887207, 0.10478949546813965, 0.10523629188537598, 0.10525059700012207, 0.10491228103637695, 0.10473442077636719, 0.10513496398925781, 0.10554718971252441, 0.10538315773010254, 0.10500597953796387, 0.10477519035339355, 0.10489439964294434, 0.1049954891204834, 0.10513782501220703, 0.10483860969543457, 0.10491251945495605, 0.10579514503479004, 0.10465836524963379, 0.10567045211791992, 0.10520172119140625, 0.10592007637023926, 0.10523605346679688, 0.10495853424072266, 0.10504412651062012, 0.1048734188079834, 0.10495710372924805, 0.10535883903503418, 0.10483407974243164, 0.1058809757232666, 0.1048121452331543, 0.10529017448425293, 0.1057736873626709, 0.10525178909301758, 0.10474777221679688, 0.10515046119689941, 0.10488080978393555, 0.10526800155639648, 0.10505485534667969, 0.105682373046875, 0.10523867607116699, 0.10515427589416504, 0.10500621795654297, 0.10488057136535645, 0.10497164726257324, 0.10513830184936523, 0.10511493682861328, 0.10523295402526855, 0.10524868965148926, 0.10524106025695801, 0.10501861572265625, 0.10511136054992676, 0.10533785820007324, 0.10507512092590332, 0.10488033294677734, 0.10511398315429688, 0.10480523109436035, 0.10542750358581543, 0.10491490364074707, 0.10498881340026855, 0.10499119758605957, 0.10500788688659668, 0.10578083992004395, 0.10477256774902344, 0.10508203506469727, 0.10799407958984375, 0.10494804382324219, 0.10561847686767578, 0.10491275787353516, 0.10518288612365723, 0.10434293746948242, 0.10528826713562012, 0.10505533218383789, 0.10510683059692383, 0.10584783554077148, 0.1051933765411377, 0.10476422309875488, 0.10582828521728516, 0.10471034049987793, 0.10502433776855469, 0.10551190376281738, 0.10553312301635742, 0.10472464561462402, 0.10521960258483887, 0.10543274879455566, 0.105010986328125, 0.10477185249328613, 0.10517549514770508, 0.10510039329528809, 0.10535073280334473, 0.10479998588562012, 0.10492730140686035, 0.10497426986694336, 0.10550355911254883, 0.10473084449768066, 0.10464954376220703, 0.10486602783203125, 0.10475850105285645, 0.10465431213378906, 0.10506081581115723, 0.1049954891204834, 0.10477924346923828, 0.10590958595275879, 0.1054537296295166, 0.10524845123291016, 0.10532379150390625, 0.10491180419921875, 0.10511398315429688, 0.10501742362976074, 0.1055002212524414, 0.10523366928100586, 0.10504984855651855, 0.10499310493469238, 0.10448980331420898, 0.10483503341674805, 0.10510015487670898, 0.10447812080383301, 0.10497903823852539, 0.10494160652160645, 0.1048886775970459, 0.1044776439666748, 0.10530352592468262, 0.10489892959594727, 0.10484814643859863, 0.10468649864196777, 0.10532307624816895, 0.10528135299682617, 0.10588502883911133, 0.10557365417480469, 0.10512733459472656, 0.10515880584716797, 0.10560131072998047, 0.10696268081665039, 0.10506701469421387, 0.10549592971801758, 0.10470438003540039, 0.10477113723754883, 0.10488224029541016, 0.10534930229187012, 0.10486197471618652, 0.10487174987792969, 0.10529136657714844, 0.10478425025939941, 0.10507011413574219, 0.1057279109954834, 0.10496950149536133, 0.1052711009979248, 0.10515403747558594, 0.10489654541015625, 0.1048274040222168, 0.10478949546813965, 0.10533452033996582, 0.10491037368774414, 0.10458898544311523, 0.10496115684509277, 0.10505223274230957, 0.10491013526916504, 0.10530710220336914, 0.10573625564575195, 0.10537481307983398, 0.10481929779052734, 0.10596585273742676, 0.1050729751586914, 0.10474872589111328, 0.10502266883850098, 0.10491418838500977, 0.10523152351379395, 0.10482215881347656, 0.10479331016540527, 0.10505533218383789, 0.10522651672363281, 0.10508489608764648, 0.10607218742370605, 0.10473775863647461, 0.1050417423248291, 0.1060938835144043, 0.10482478141784668, 0.1055748462677002, 0.1052088737487793, 0.10549187660217285, 0.10486316680908203, 0.10511231422424316, 0.10537028312683105, 0.10503888130187988, 0.10483360290527344, 0.10497403144836426, 0.1050422191619873, 0.10469603538513184, 0.10516762733459473, 0.10501265525817871, 0.10496854782104492, 0.10509085655212402, 0.10459423065185547, 0.10507369041442871, 0.1052713394165039, 0.10513544082641602, 0.10494351387023926, 0.10510897636413574, 0.10512661933898926, 0.10511994361877441, 0.10480690002441406, 0.10498714447021484, 0.10522913932800293, 0.1054389476776123, 0.10505223274230957, 0.10528969764709473, 0.10512995719909668, 0.10514426231384277, 0.10481405258178711, 0.10479426383972168, 0.10488581657409668, 0.10486888885498047, 0.10472822189331055, 0.10498261451721191, 0.10473871231079102, 0.10457420349121094, 0.10489606857299805, 0.10516238212585449, 0.10479855537414551, 0.10493111610412598, 0.10604262351989746, 0.10500931739807129, 0.1054692268371582, 0.10613894462585449, 0.10506010055541992, 0.10498476028442383, 0.10505414009094238, 0.10517048835754395, 0.10573887825012207, 0.10543227195739746, 0.10482215881347656, 0.10525250434875488, 0.10531139373779297, 0.1052098274230957, 0.10521078109741211, 0.1055455207824707, 0.10540938377380371, 0.10497117042541504, 0.10550832748413086, 0.10521459579467773, 0.10496830940246582, 0.10470223426818848, 0.10544419288635254, 0.10498380661010742, 0.10496354103088379, 0.1058967113494873, 0.10530900955200195, 0.10545516014099121, 0.10520052909851074, 0.10467028617858887, 0.1049506664276123, 0.10507011413574219, 0.10524201393127441, 0.10481524467468262, 0.10499191284179688, 0.10554194450378418, 0.10483074188232422, 0.10503268241882324, 0.10508036613464355, 0.10498332977294922, 0.10471296310424805, 0.10475540161132812, 0.1050560474395752, 0.10453486442565918, 0.10529255867004395, 0.10482931137084961, 0.10485386848449707, 0.10552644729614258, 0.10497641563415527, 0.10490894317626953, 0.10601806640625, 0.10499429702758789, 0.10514259338378906, 0.10472273826599121, 0.10581660270690918, 0.1063232421875, 0.10547375679016113, 0.10504388809204102, 0.10459232330322266, 0.10484457015991211, 0.1048741340637207, 0.10481739044189453, 0.1046292781829834, 0.10474109649658203, 0.10512256622314453, 0.10505962371826172, 0.10473966598510742, 0.10517692565917969, 0.10451245307922363, 0.10497164726257324, 0.10464358329772949, 0.1050713062286377, 0.10463428497314453, 0.10569405555725098, 0.10480618476867676, 0.10500836372375488, 0.10475516319274902, 0.1047358512878418, 0.10488677024841309, 0.1047203540802002, 0.10453438758850098, 0.10484910011291504, 0.10551118850708008, 0.10478901863098145, 0.10517334938049316, 0.10493826866149902, 0.1047365665435791, 0.10519719123840332, 0.10480618476867676, 0.10487508773803711, 0.10494685173034668, 0.10476255416870117, 0.10531449317932129, 0.10499739646911621, 0.10495305061340332, 0.10572266578674316, 0.10469889640808105, 0.1049492359161377, 0.10594439506530762, 0.10484623908996582, 0.1047666072845459, 0.10498404502868652, 0.10472583770751953, 0.10512948036193848, 0.1049039363861084, 0.10512208938598633, 0.10480427742004395, 0.10486674308776855, 0.1047818660736084, 0.10514330863952637, 0.10483145713806152, 0.10540509223937988, 0.1046595573425293, 0.10492944717407227, 0.1051023006439209, 0.10463476181030273, 0.1047356128692627, 0.10466194152832031, 0.10480999946594238, 0.10456705093383789, 0.10439467430114746, 0.10451054573059082, 0.10464954376220703, 0.10493969917297363, 0.10440516471862793, 0.10441327095031738, 0.10489797592163086, 0.10495853424072266, 0.10529780387878418, 0.1048583984375, 0.1052408218383789, 0.10535192489624023, 0.10520005226135254, 0.10474133491516113, 0.10520100593566895, 0.10494017601013184, 0.10465717315673828, 0.10485982894897461, 0.10494565963745117, 0.1054530143737793, 0.10487675666809082, 0.1047661304473877, 0.10469698905944824, 0.10555672645568848, 0.10480594635009766, 0.10607028007507324, 0.10526180267333984, 0.10526299476623535, 0.1047821044921875, 0.10496664047241211, 0.1050560474395752, 0.10489273071289062, 0.10474944114685059, 0.10481095314025879, 0.10469603538513184, 0.10512280464172363, 0.10455012321472168, 0.10465478897094727, 0.10468006134033203, 0.10497689247131348, 0.10480952262878418, 0.10493040084838867, 0.10496711730957031, 0.10451030731201172, 0.10498166084289551, 0.10477828979492188, 0.10480427742004395, 0.10467028617858887, 0.10481405258178711, 0.1049799919128418, 0.10494828224182129, 0.10478377342224121, 0.10462832450866699, 0.1044161319732666, 0.1049349308013916, 0.10460329055786133, 0.10494589805603027, 0.10492372512817383, 0.10474419593811035, 0.10481023788452148, 0.10441875457763672, 0.10466480255126953, 0.10448360443115234, 0.10492753982543945, 0.10471248626708984, 0.10473132133483887, 0.10448288917541504, 0.10443758964538574, 0.10472989082336426, 0.10480761528015137, 0.10522317886352539, 0.10510587692260742, 0.10468554496765137, 0.10563874244689941, 0.104736328125, 0.10498714447021484, 0.10602021217346191, 0.10582399368286133, 0.10623526573181152, 0.10570764541625977, 0.10503792762756348, 0.1048123836517334, 0.10488057136535645, 0.10508537292480469, 0.10519123077392578, 0.10453295707702637, 0.10507702827453613, 0.1053612232208252, 0.10486268997192383, 0.10515880584716797, 0.10498881340026855, 0.10486030578613281, 0.1053926944732666, 0.10489797592163086, 0.10497641563415527, 0.10474038124084473, 0.1047515869140625, 0.10514259338378906, 0.10494446754455566, 0.10510063171386719, 0.1044464111328125, 0.10495209693908691, 0.10508537292480469, 0.10590648651123047, 0.10477757453918457, 0.10515141487121582, 0.10522699356079102, 0.10519170761108398, 0.10471487045288086, 0.10525965690612793, 0.1048591136932373, 0.10471725463867188, 0.10462617874145508, 0.10479569435119629, 0.10438728332519531, 0.1148228645324707, 0.10502004623413086, 0.10548734664916992, 0.10593795776367188, 0.10500311851501465, 0.10503172874450684, 0.10565066337585449, 0.10554027557373047, 0.10473942756652832, 0.1050422191619873, 0.10535311698913574, 0.10532784461975098, 0.1052854061126709, 0.10481739044189453, 0.10465550422668457, 0.10510540008544922, 0.10455203056335449, 0.10547804832458496, 0.10454034805297852, 0.1054697036743164, 0.10492873191833496, 0.10524296760559082, 0.10502886772155762, 0.10507321357727051, 0.10536837577819824, 0.10557985305786133, 0.1053922176361084, 0.10534310340881348, 0.10482048988342285, 0.10514354705810547, 0.10497379302978516, 0.10520434379577637, 0.10467863082885742, 0.10484552383422852, 0.10506510734558105, 0.10488700866699219, 0.10491394996643066, 0.10495471954345703, 0.10530304908752441, 0.10531067848205566, 0.10483741760253906, 0.10499262809753418, 0.10466933250427246, 0.10498809814453125, 0.10488224029541016, 0.10486555099487305, 0.10463261604309082, 0.10546112060546875, 0.10527992248535156, 0.10507011413574219, 0.10513615608215332, 0.10532259941101074, 0.1051950454711914, 0.1059880256652832, 0.10505938529968262, 0.1049187183380127, 0.10459089279174805, 0.10518145561218262, 0.10508012771606445, 0.10549068450927734, 0.10467123985290527, 0.10518312454223633, 0.10511207580566406, 0.1052396297454834, 0.1046900749206543, 0.10519123077392578, 0.10514068603515625, 0.10499882698059082, 0.10463285446166992, 0.10547161102294922, 0.10485172271728516, 0.10509014129638672, 0.10537219047546387, 0.1049196720123291, 0.10519742965698242, 0.10483217239379883, 0.10555481910705566, 0.10515213012695312, 0.1049957275390625, 0.10527682304382324, 0.10515666007995605, 0.1047813892364502, 0.10496234893798828, 0.10557818412780762, 0.10473299026489258, 0.10576939582824707, 0.10514974594116211, 0.10562968254089355, 0.10486841201782227, 0.10542082786560059, 0.10483002662658691, 0.10499978065490723, 0.10499215126037598, 0.10511636734008789, 0.10468411445617676, 0.10553193092346191, 0.10486769676208496, 0.10539507865905762, 0.10532307624816895, 0.10503458976745605, 0.10489916801452637, 0.10628032684326172, 0.10513424873352051, 0.10508394241333008, 0.10462737083435059, 0.10680007934570312, 0.10529637336730957, 0.10579085350036621, 0.10478401184082031, 0.10523223876953125, 0.10489344596862793, 0.10490131378173828, 0.10483098030090332, 0.10489535331726074, 0.10532999038696289, 0.10518836975097656, 0.10470247268676758, 0.10499382019042969, 0.10467338562011719, 0.10503172874450684, 0.1051173210144043, 0.10497522354125977, 0.10485029220581055, 0.10495591163635254, 0.10531234741210938, 0.1051783561706543, 0.10479426383972168, 0.10520434379577637, 0.10458660125732422, 0.10508346557617188, 0.1047368049621582, 0.10518503189086914, 0.10463953018188477, 0.10556197166442871, 0.10473179817199707, 0.10505342483520508, 0.1046595573425293, 0.10584402084350586, 0.10441160202026367, 0.1054372787475586, 0.10471343994140625, 0.10492515563964844, 0.10478591918945312, 0.10498666763305664, 0.10476946830749512, 0.1046295166015625, 0.10581517219543457, 0.1049799919128418, 0.10520148277282715, 0.10585570335388184, 0.10492563247680664, 0.10471701622009277, 0.105560302734375, 0.1052396297454834, 0.10516858100891113, 0.10524702072143555, 0.1060793399810791, 0.10529923439025879, 0.10509514808654785, 0.10545682907104492, 0.10444188117980957, 0.10471630096435547, 0.1051173210144043, 0.10496783256530762, 0.10519218444824219, 0.10534191131591797, 0.10513591766357422, 0.10500907897949219, 0.10443830490112305, 0.10527777671813965, 0.1045987606048584, 0.10501551628112793, 0.10493135452270508, 0.10507512092590332, 0.1049191951751709, 0.10528087615966797, 0.10470438003540039, 0.10534834861755371, 0.10533618927001953, 0.10522913932800293, 0.10519576072692871, 0.10547423362731934, 0.10460329055786133, 0.10516762733459473, 0.10477709770202637, 0.10505104064941406, 0.10457539558410645, 0.10486602783203125, 0.10466599464416504, 0.10485029220581055, 0.10575485229492188, 0.10506939888000488, 0.1048281192779541, 0.10565042495727539, 0.10503888130187988, 0.10494685173034668, 0.10619711875915527, 0.10535407066345215, 0.10526609420776367, 0.1048130989074707, 0.10501599311828613, 0.1049509048461914, 0.10536646842956543, 0.10531854629516602, 0.10516953468322754, 0.10460305213928223, 0.10478949546813965, 0.10540509223937988, 0.10484933853149414, 0.10587501525878906, 0.10499334335327148, 0.10492181777954102, 0.10503482818603516, 0.10491681098937988, 0.10476374626159668, 0.10522890090942383, 0.10458898544311523, 0.1049799919128418, 0.10476088523864746, 0.10496711730957031, 0.10491228103637695, 0.10538482666015625, 0.10501217842102051, 0.10518646240234375, 0.10475492477416992, 0.10488343238830566, 0.10500788688659668, 0.10503363609313965, 0.1047062873840332, 0.10527348518371582, 0.10525918006896973, 0.10448479652404785, 0.10494184494018555, 0.10461735725402832, 0.10508179664611816, 0.10482001304626465, 0.10469913482666016, 0.10463547706604004, 0.10485410690307617, 0.1046590805053711, 0.10464620590209961, 0.10585212707519531, 0.1049962043762207, 0.10470914840698242, 0.10583949089050293, 0.10487985610961914, 0.10527825355529785, 0.10528182983398438, 0.10728931427001953, 0.10582113265991211, 0.10515952110290527, 0.10516881942749023, 0.10504579544067383, 0.1048128604888916, 0.1043694019317627, 0.10500979423522949, 0.10483670234680176, 0.10559201240539551, 0.10502314567565918, 0.10522937774658203, 0.10525107383728027, 0.10493922233581543, 0.10466480255126953, 0.10532879829406738, 0.10478091239929199, 0.1048426628112793, 0.1048424243927002, 0.104644775390625, 0.1045832633972168, 0.10486888885498047, 0.10460186004638672, 0.10480594635009766, 0.10497927665710449, 0.10507774353027344, 0.10524868965148926, 0.10535383224487305, 0.10527658462524414, 0.11737227439880371, 0.10581183433532715, 0.1048288345336914, 0.1052865982055664, 0.10489058494567871, 0.10528373718261719, 0.10484576225280762, 0.10515642166137695, 0.10520267486572266, 0.10551857948303223, 0.10541892051696777, 0.10575270652770996, 0.10541319847106934, 0.10465550422668457, 0.10616469383239746, 0.10527276992797852, 0.10538291931152344, 0.10523724555969238, 0.10504770278930664, 0.10485982894897461, 0.10511565208435059, 0.10523319244384766, 0.10505890846252441, 0.1049802303314209, 0.10538077354431152, 0.1048879623413086, 0.10513877868652344, 0.10530614852905273, 0.10500049591064453, 0.10502409934997559, 0.1046915054321289, 0.10505223274230957, 0.1049952507019043, 0.10528922080993652, 0.10488247871398926, 0.10477209091186523, 0.10508966445922852, 0.10525345802307129, 0.10471248626708984, 0.10508060455322266, 0.10488319396972656, 0.10475611686706543, 0.10490131378173828, 0.1052098274230957, 0.1046602725982666, 0.10502862930297852, 0.10470747947692871, 0.10500311851501465, 0.1047513484954834, 0.10480403900146484, 0.10522818565368652, 0.10458087921142578, 0.10532569885253906, 0.1047050952911377, 0.10491466522216797, 0.1055150032043457, 0.10502481460571289, 0.10510897636413574, 0.1049644947052002, 0.1053316593170166, 0.10491371154785156, 0.10465478897094727, 0.1059262752532959, 0.10468363761901855, 0.10506820678710938, 0.10470938682556152, 0.10507059097290039, 0.10508608818054199, 0.10493016242980957, 0.10497307777404785, 0.1049344539642334, 0.10479140281677246, 0.10477828979492188, 0.1048269271850586, 0.10463762283325195, 0.10525751113891602, 0.10456466674804688, 0.1054847240447998, 0.10517549514770508, 0.10528397560119629, 0.10475659370422363, 0.10534858703613281, 0.10488224029541016, 0.10516214370727539, 0.10492682456970215, 0.10552597045898438, 0.1049189567565918, 0.10523700714111328, 0.10506486892700195, 0.1050114631652832, 0.10519218444824219, 0.10488128662109375, 0.10544157028198242, 0.10488343238830566, 0.10563397407531738, 0.10504484176635742, 0.10518026351928711, 0.10505175590515137, 0.1050863265991211, 0.10466146469116211, 0.1053016185760498, 0.1050715446472168, 0.10512423515319824, 0.10520601272583008, 0.10523509979248047, 0.10504913330078125, 0.10529351234436035, 0.10587024688720703, 0.10500645637512207, 0.10617494583129883, 0.10518836975097656, 0.10483956336975098, 0.10485553741455078, 0.10587596893310547, 0.10597419738769531, 0.10542488098144531, 0.10458135604858398, 0.10511231422424316, 0.10409379005432129, 0.10478925704956055, 0.10480332374572754, 0.10493636131286621, 0.1048421859741211, 0.10500383377075195, 0.10454797744750977, 0.10534405708312988, 0.10522675514221191, 0.10551953315734863, 0.10549616813659668, 0.1045525074005127, 0.10507416725158691, 0.10447001457214355, 0.10552191734313965, 0.10517334938049316, 0.10513997077941895, 0.10489082336425781, 0.10502219200134277, 0.10467267036437988, 0.10502743721008301, 0.10503053665161133, 0.10498571395874023, 0.10489535331726074, 0.10532212257385254, 0.10485243797302246, 0.1051485538482666, 0.10466742515563965, 0.10503792762756348, 0.10501790046691895, 0.10459661483764648, 0.1049504280090332, 0.10477757453918457, 0.10500073432922363, 0.10503578186035156, 0.1052861213684082, 0.10566210746765137, 0.1049644947052002, 0.10460090637207031, 0.10630416870117188, 0.10476350784301758, 0.10506272315979004, 0.10471272468566895, 0.10491347312927246, 0.10548591613769531, 0.10484480857849121, 0.10521245002746582, 0.10514020919799805, 0.10488176345825195, 0.10493302345275879, 0.10456538200378418, 0.10453557968139648, 0.10508918762207031, 0.10466480255126953, 0.10530996322631836, 0.10507726669311523, 0.10504817962646484, 0.1050114631652832, 0.10510873794555664, 0.10465312004089355, 0.10484504699707031, 0.10450124740600586, 0.1050267219543457, 0.10450482368469238, 0.10499215126037598, 0.10460114479064941, 0.1051018238067627, 0.10469555854797363, 0.10495281219482422, 0.1053168773651123, 0.10494804382324219, 0.10506534576416016, 0.10568094253540039, 0.10498332977294922, 0.10485696792602539, 0.10504341125488281, 0.1047515869140625, 0.10534262657165527, 0.10466361045837402, 0.10524988174438477, 0.10536623001098633, 0.10514116287231445, 0.10525131225585938, 0.10500097274780273, 0.10606980323791504, 0.10509347915649414, 0.10605525970458984, 0.10583853721618652, 0.1053006649017334, 0.10508179664611816, 0.10513758659362793, 0.10513973236083984, 0.10538125038146973, 0.10526299476623535, 0.10546183586120605, 0.10479021072387695, 0.10542893409729004, 0.10490775108337402, 0.10518932342529297, 0.10500264167785645, 0.10544753074645996, 0.10502266883850098, 0.1053626537322998, 0.10514330863952637, 0.10519862174987793, 0.10516667366027832, 0.10514402389526367, 0.10495996475219727, 0.1049947738647461, 0.10499906539916992, 0.10552048683166504, 0.10503959655761719, 0.10537886619567871, 0.1051783561706543, 0.10509848594665527, 0.1049811840057373, 0.10488748550415039, 0.10531902313232422, 0.1046288013458252, 0.10488033294677734, 0.10492801666259766, 0.10515260696411133, 0.10509252548217773, 0.10489177703857422, 0.10493731498718262, 0.10496234893798828, 0.1050870418548584, 0.10503578186035156, 0.10534143447875977, 0.10513114929199219, 0.10502862930297852, 0.10554361343383789, 0.10556602478027344, 0.10480380058288574, 0.10655856132507324, 0.10523557662963867, 0.10571670532226562, 0.10562324523925781, 0.10689973831176758, 0.1055765151977539, 0.10546755790710449, 0.10535192489624023, 0.10583901405334473, 0.10523033142089844, 0.1054840087890625, 0.10477304458618164, 0.10490274429321289, 0.1051645278930664, 0.10500764846801758, 0.10526275634765625, 0.10552763938903809, 0.10528874397277832, 0.1046438217163086, 0.10554790496826172, 0.10499429702758789, 0.10533666610717773, 0.1052238941192627, 0.10507845878601074, 0.10504841804504395, 0.10548281669616699, 0.10534119606018066, 0.10532307624816895, 0.10499167442321777, 0.10533857345581055, 0.10523319244384766, 0.10521292686462402, 0.10589432716369629, 0.10633015632629395, 0.10525822639465332, 0.1053779125213623, 0.10516214370727539, 0.10503292083740234, 0.10573363304138184, 0.10520148277282715, 0.10530948638916016, 0.11090469360351562, 0.10903024673461914, 0.10521388053894043, 0.10530591011047363, 0.10620522499084473, 0.10518097877502441, 0.10619711875915527, 0.10546755790710449, 0.10558962821960449, 0.10529422760009766, 0.10502910614013672, 0.10526704788208008, 0.10505509376525879, 0.10523676872253418, 0.10536646842956543, 0.10518527030944824, 0.10522317886352539, 0.10550141334533691, 0.10524415969848633, 0.10568833351135254, 0.10561752319335938, 0.10526609420776367, 0.10545039176940918, 0.10498452186584473, 0.10574531555175781, 0.10538053512573242, 0.10495305061340332, 0.10528302192687988, 0.10507464408874512, 0.10538983345031738, 0.10502934455871582, 0.10526180267333984, 0.1054539680480957, 0.10538911819458008, 0.10489892959594727, 0.10530900955200195, 0.10510516166687012, 0.10487174987792969, 0.10494422912597656, 0.10513663291931152, 0.10471510887145996, 0.10528683662414551, 0.10507607460021973, 0.10505819320678711, 0.1050572395324707, 0.10480618476867676, 0.10527801513671875, 0.1050567626953125, 0.10509014129638672, 0.10482168197631836, 0.10477089881896973, 0.10579395294189453, 0.1050267219543457, 0.10494709014892578, 0.10661649703979492, 0.10508465766906738, 0.10522150993347168, 0.10478091239929199, 0.10519242286682129, 0.10537028312683105, 0.1052088737487793, 0.10494518280029297, 0.10516977310180664, 0.10455870628356934, 0.10504031181335449, 0.10463595390319824, 0.10442829132080078, 0.10527658462524414, 0.1049797534942627, 0.10505223274230957, 0.10515260696411133, 0.10508060455322266, 0.1047217845916748, 0.10556340217590332, 0.10493707656860352, 0.10499763488769531, 0.1054544448852539, 0.10544037818908691, 0.10500717163085938, 0.10518074035644531, 0.1049795150756836, 0.10558938980102539, 0.10472464561462402, 0.10533809661865234, 0.10550165176391602, 0.10504388809204102, 0.10570597648620605, 0.10575699806213379, 0.10482001304626465, 0.10529708862304688, 0.10505080223083496, 0.10522770881652832, 0.10493326187133789, 0.10510921478271484, 0.1052253246307373, 0.1055765151977539, 0.10544610023498535, 0.10547447204589844, 0.10568070411682129, 0.10617375373840332, 0.10515093803405762, 0.10570669174194336, 0.10555052757263184, 0.10578632354736328, 0.10516023635864258, 0.10528254508972168, 0.10695552825927734, 0.10607123374938965, 0.10547423362731934, 0.10587620735168457, 0.1050870418548584, 0.10533452033996582, 0.10532903671264648, 0.1053931713104248, 0.10482192039489746, 0.10574626922607422, 0.10505366325378418, 0.10559630393981934, 0.10517001152038574, 0.10536456108093262, 0.10514450073242188, 0.10553121566772461, 0.1051938533782959, 0.1056058406829834, 0.10519576072692871, 0.10583257675170898, 0.10497879981994629, 0.10540175437927246, 0.10525631904602051, 0.1055307388305664, 0.10506892204284668, 0.10545611381530762, 0.10526561737060547, 0.1059255599975586, 0.10536003112792969, 0.10499334335327148, 0.10512757301330566, 0.10598087310791016, 0.1052103042602539, 0.10558676719665527, 0.10500216484069824, 0.10537862777709961, 0.10478353500366211, 0.10532736778259277, 0.10554265975952148, 0.10480904579162598, 0.10629892349243164, 0.10531902313232422, 0.10493183135986328, 0.10676312446594238, 0.10486245155334473, 0.10565328598022461, 0.10519099235534668, 0.10556268692016602, 0.10509872436523438, 0.1061253547668457, 0.10509777069091797, 0.1059122085571289, 0.10513043403625488, 0.10546708106994629, 0.10514688491821289, 0.10542082786560059, 0.10523724555969238, 0.10570836067199707, 0.1053323745727539, 0.10565495491027832, 0.10544919967651367, 0.10540986061096191, 0.10525798797607422, 0.1051781177520752, 0.10476541519165039, 0.10480856895446777, 0.10550951957702637, 0.10554885864257812, 0.10528230667114258, 0.10540652275085449, 0.1050567626953125, 0.10568356513977051, 0.10514092445373535, 0.10611081123352051, 0.10526013374328613, 0.10774946212768555, 0.10598516464233398, 0.10551238059997559, 0.10525345802307129, 0.10548615455627441, 0.10498428344726562, 0.10565924644470215, 0.10547375679016113, 0.10549783706665039, 0.10591888427734375, 0.10561084747314453, 0.1048893928527832, 0.10623359680175781, 0.10570096969604492, 0.10538530349731445, 0.10596489906311035, 0.10588979721069336, 0.10496330261230469, 0.10568547248840332, 0.10540437698364258, 0.10564446449279785, 0.10518765449523926, 0.1050562858581543, 0.10498380661010742, 0.10492873191833496, 0.10503482818603516, 0.10521888732910156, 0.10487103462219238, 0.1060945987701416, 0.10552692413330078, 0.10533285140991211, 0.10600566864013672, 0.10487747192382812, 0.10527706146240234, 0.10519695281982422, 0.10518240928649902, 0.1052548885345459, 0.10524892807006836, 0.10525894165039062, 0.10532021522521973, 0.10500526428222656, 0.10501742362976074, 0.10514450073242188, 0.10516357421875, 0.10531044006347656, 0.10491394996643066, 0.1052098274230957, 0.10493230819702148, 0.10557866096496582, 0.10517525672912598, 0.10539984703063965, 0.10495877265930176, 0.10518884658813477, 0.10502338409423828, 0.10487532615661621, 0.10511064529418945, 0.1053476333618164, 0.10541510581970215, 0.10542702674865723, 0.10495471954345703, 0.10679030418395996, 0.10534167289733887, 0.1049807071685791, 0.1064608097076416, 0.10465407371520996, 0.10543680191040039, 0.10561680793762207, 0.10765385627746582, 0.10591435432434082, 0.10603618621826172, 0.10518980026245117, 0.10517382621765137, 0.10513496398925781, 0.1049497127532959, 0.10486006736755371, 0.1051337718963623, 0.10524106025695801, 0.10503363609313965, 0.1053769588470459, 0.10570168495178223, 0.10535478591918945, 0.10545063018798828, 0.10538601875305176, 0.10547661781311035, 0.10522866249084473, 0.1051936149597168, 0.10569977760314941, 0.10519552230834961, 0.10540151596069336, 0.10482501983642578, 0.10491251945495605, 0.1050565242767334, 0.1055150032043457, 0.10553193092346191, 0.10567831993103027, 0.10514187812805176, 0.10596203804016113, 0.10512518882751465, 0.1052389144897461, 0.10547971725463867, 0.10484576225280762, 0.10505104064941406, 0.10482239723205566, 0.1053171157836914, 0.10561227798461914, 0.10557389259338379, 0.10523867607116699, 0.10626506805419922, 0.10558533668518066, 0.10547709465026855, 0.10662436485290527, 0.10584473609924316, 0.10526251792907715, 0.10521101951599121, 0.10492205619812012, 0.1051023006439209, 0.10483407974243164, 0.1051933765411377, 0.10513806343078613, 0.1050879955291748, 0.10544967651367188, 0.10540580749511719, 0.10529518127441406, 0.10561990737915039, 0.10530877113342285, 0.10491704940795898, 0.10518002510070801, 0.1049497127532959, 0.10536623001098633, 0.10540175437927246, 0.10509514808654785, 0.10526847839355469, 0.10522174835205078, 0.10505938529968262, 0.10548543930053711, 0.10503840446472168, 0.10553908348083496, 0.10510826110839844, 0.10520482063293457, 0.10521888732910156, 0.10498452186584473, 0.10515570640563965, 0.1055448055267334, 0.10539937019348145, 0.10529398918151855, 0.10521101951599121, 0.10495662689208984, 0.10537004470825195, 0.10542130470275879, 0.10529541969299316, 0.1053323745727539, 0.10582756996154785, 0.1054537296295166, 0.10515713691711426, 0.10522294044494629, 0.10665440559387207, 0.10559654235839844, 0.10567760467529297, 0.10646319389343262, 0.10522699356079102, 0.10524868965148926, 0.10538768768310547, 0.10544037818908691, 0.10527396202087402, 0.10510969161987305, 0.1047661304473877, 0.10564589500427246, 0.10526919364929199, 0.10541558265686035, 0.10507345199584961, 0.10565185546875, 0.10509538650512695, 0.10512089729309082, 0.10482549667358398, 0.10590600967407227, 0.10481572151184082, 0.10554695129394531, 0.10564708709716797, 0.10565614700317383, 0.1052558422088623, 0.10607290267944336, 0.1051633358001709, 0.10526561737060547, 0.10533881187438965, 0.10515713691711426, 0.10490989685058594, 0.10513019561767578, 0.10557746887207031, 0.10555338859558105, 0.1050558090209961, 0.10554742813110352, 0.10548758506774902, 0.10556602478027344, 0.105072021484375, 0.10532736778259277, 0.1047055721282959, 0.10532283782958984, 0.10529041290283203, 0.1054694652557373, 0.1058192253112793, 0.1053309440612793, 0.10533785820007324, 0.10667896270751953, 0.10498523712158203, 0.10538005828857422, 0.10633730888366699, 0.10635113716125488, 0.1051323413848877, 0.10550570487976074, 0.10693526268005371, 0.10606527328491211, 0.10589599609375, 0.1054537296295166, 0.1054236888885498, 0.10561418533325195, 0.10522246360778809, 0.10556316375732422, 0.10494709014892578, 0.1053781509399414, 0.10547471046447754, 0.10569477081298828, 0.10507917404174805, 0.10554265975952148, 0.10496330261230469, 0.10562276840209961, 0.10517477989196777, 0.10548114776611328, 0.10522198677062988, 0.10594010353088379, 0.10519242286682129, 0.10560083389282227, 0.10536909103393555, 0.10565972328186035, 0.10520768165588379, 0.1056509017944336, 0.1051640510559082, 0.10532760620117188, 0.10552477836608887, 0.10566329956054688, 0.1049039363861084, 0.10541129112243652, 0.10532593727111816, 0.10604667663574219, 0.10524344444274902, 0.10533976554870605, 0.10507440567016602, 0.10545134544372559, 0.10500597953796387, 0.10532021522521973, 0.10518479347229004, 0.10668516159057617, 0.10515880584716797, 0.10595488548278809, 0.10559654235839844, 0.10570359230041504, 0.10511064529418945, 0.1057734489440918, 0.10530328750610352, 0.1053764820098877, 0.10587930679321289, 0.1053464412689209, 0.10500192642211914, 0.1054532527923584, 0.10532045364379883, 0.10532760620117188, 0.10494589805603027, 0.10578727722167969, 0.10538339614868164, 0.10529899597167969, 0.10551929473876953, 0.1053464412689209, 0.10554718971252441, 0.10524845123291016, 0.10482501983642578, 0.10491251945495605, 0.10505414009094238, 0.10553932189941406, 0.10552215576171875, 0.10527515411376953, 0.10514998435974121, 0.10607171058654785, 0.10562324523925781, 0.10627985000610352, 0.10597491264343262, 0.10584163665771484, 0.10558938980102539, 0.10571837425231934, 0.10511255264282227, 0.10553812980651855, 0.10561084747314453, 0.10530853271484375, 0.10556912422180176, 0.1056969165802002, 0.10481643676757812, 0.10619831085205078, 0.10538029670715332, 0.10548043251037598, 0.10632443428039551, 0.10573005676269531, 0.10513091087341309, 0.1064450740814209, 0.10578632354736328, 0.10536623001098633, 0.10509705543518066, 0.10514593124389648, 0.10511326789855957, 0.1055300235748291, 0.10491466522216797, 0.10544848442077637, 0.10514569282531738, 0.1052558422088623, 0.10515642166137695, 0.10528111457824707, 0.10513663291931152, 0.10523867607116699, 0.1054227352142334, 0.105377197265625, 0.1052699089050293, 0.10513782501220703, 0.10531044006347656, 0.10517430305480957, 0.10531783103942871, 0.1054837703704834, 0.10517263412475586, 0.10519790649414062, 0.10526371002197266, 0.10529303550720215, 0.10508394241333008, 0.10571074485778809, 0.10503602027893066, 0.10523176193237305, 0.10473418235778809, 0.10555291175842285, 0.10539007186889648, 0.10543704032897949, 0.10504007339477539, 0.10538029670715332, 0.10516524314880371, 0.10534071922302246, 0.10499882698059082, 0.10518074035644531, 0.10521841049194336, 0.10513091087341309, 0.10517573356628418, 0.10511088371276855, 0.10631847381591797, 0.10548090934753418, 0.10607147216796875, 0.10591864585876465, 0.10543942451477051, 0.10597491264343262, 0.10575318336486816, 0.1086275577545166, 0.10577130317687988, 0.10539603233337402, 0.10530471801757812, 0.1051492691040039, 0.10543608665466309, 0.10546684265136719, 0.1052100658416748, 0.1053473949432373, 0.10548830032348633, 0.10553526878356934, 0.10556602478027344, 0.10947465896606445, 0.10506987571716309, 0.10573554039001465, 0.10513067245483398, 0.10567712783813477, 0.10477209091186523, 0.10528993606567383, 0.10535049438476562, 0.10538363456726074, 0.10529851913452148, 0.10506820678710938, 0.10513687133789062, 0.10559725761413574, 0.10582208633422852, 0.10559201240539551, 0.10534477233886719, 0.10533976554870605, 0.10524773597717285, 0.10560989379882812, 0.10511064529418945, 0.10521388053894043, 0.10516667366027832, 0.1054525375366211, 0.10531020164489746, 0.10558724403381348, 0.11907720565795898, 0.10612678527832031, 0.10515356063842773, 0.10637140274047852, 0.10544896125793457, 0.10560011863708496, 0.1069483757019043, 0.10597634315490723, 0.10511636734008789, 0.10565757751464844, 0.1050572395324707, 0.10543251037597656, 0.10552716255187988, 0.10539436340332031, 0.10573363304138184, 0.10553956031799316, 0.1052408218383789, 0.1055307388305664, 0.10483074188232422, 0.10498857498168945, 0.10496234893798828, 0.10480022430419922, 0.10459399223327637, 0.10499119758605957, 0.10449957847595215, 0.10468530654907227, 0.10439467430114746, 0.10443472862243652, 0.1041097640991211, 0.10497927665710449, 0.10449099540710449, 0.11027026176452637, 0.10440564155578613, 0.10472345352172852, 0.1053628921508789, 0.10466742515563965, 0.10441327095031738, 0.10455942153930664, 0.10436797142028809, 0.1054530143737793, 0.10429954528808594, 0.1044459342956543, 0.10413932800292969, 0.10436844825744629, 0.10497045516967773, 0.10420703887939453, 0.10412955284118652, 0.10452103614807129, 0.10442280769348145, 0.10483551025390625, 0.10488104820251465, 0.10491657257080078, 0.10421442985534668, 0.10591936111450195, 0.1042027473449707, 0.10453081130981445, 0.10398626327514648, 0.10471868515014648, 0.10490036010742188, 0.10515666007995605, 0.10422301292419434, 0.10457468032836914, 0.10412812232971191, 0.10425686836242676, 0.1043848991394043, 0.10445308685302734, 0.10398530960083008, 0.10499000549316406, 0.10435700416564941, 0.10490679740905762, 0.10473012924194336, 0.1046898365020752, 0.10465073585510254, 0.1046762466430664, 0.10445237159729004, 0.10430669784545898, 0.1051628589630127, 0.10486221313476562, 0.10469603538513184, 0.10579657554626465, 0.10489702224731445, 0.10428023338317871, 0.10450959205627441, 0.10523509979248047, 0.10460662841796875, 0.10536074638366699, 0.10497283935546875, 0.10537147521972656, 0.1042790412902832, 0.10464692115783691, 0.10426664352416992, 0.10453295707702637, 0.10438370704650879, 0.10465836524963379, 0.10420894622802734, 0.10506081581115723, 0.10451889038085938, 0.10470390319824219, 0.10545825958251953, 0.10477852821350098, 0.1040182113647461, 0.10599112510681152, 0.10461091995239258, 0.10506939888000488, 0.10432863235473633, 0.10672807693481445, 0.10445213317871094, 0.10494637489318848, 0.10418581962585449, 0.10485243797302246, 0.1044161319732666, 0.10457134246826172, 0.1045379638671875, 0.10460352897644043, 0.1047370433807373, 0.10457396507263184, 0.10451626777648926, 0.10502505302429199, 0.10453200340270996, 0.10480618476867676, 0.10502171516418457, 0.10471940040588379, 0.10432863235473633, 0.10465192794799805, 0.10474658012390137, 0.10481834411621094, 0.10458588600158691, 0.10459232330322266, 0.10432839393615723, 0.10452795028686523, 0.10422873497009277, 0.10441422462463379, 0.10387253761291504, 0.10536050796508789, 0.10473084449768066, 0.10468578338623047, 0.10443234443664551, 0.10561251640319824, 0.10496330261230469, 0.10437369346618652, 0.1044609546661377, 0.10474348068237305, 0.10427618026733398, 0.10486125946044922, 0.10475897789001465, 0.10498332977294922, 0.10572266578674316, 0.10511946678161621, 0.10426831245422363, 0.10634183883666992, 0.10425043106079102, 0.10447835922241211, 0.10458564758300781, 0.10491132736206055, 0.10515165328979492, 0.10499334335327148, 0.10465383529663086, 0.10468339920043945, 0.10479974746704102, 0.10472869873046875, 0.1043095588684082, 0.10452675819396973, 0.10452795028686523, 0.10495328903198242, 0.10448312759399414, 0.1046438217163086, 0.10430765151977539, 0.10463476181030273, 0.10471296310424805, 0.10419726371765137, 0.10444164276123047, 0.10405206680297852, 0.10449552536010742, 0.10466361045837402, 0.1044015884399414, 0.10515809059143066, 0.10443830490112305, 0.10479474067687988, 0.1052401065826416, 0.1059885025024414, 0.10485363006591797, 0.10575342178344727, 0.10585546493530273, 0.10578131675720215, 0.10526919364929199, 0.10567975044250488, 0.10492157936096191, 0.1054539680480957, 0.10467886924743652, 0.10588788986206055, 0.10541629791259766, 0.10582733154296875, 0.10515880584716797, 0.10531926155090332, 0.10621762275695801, 0.10541129112243652, 0.1064913272857666, 0.10619330406188965, 0.10546255111694336, 0.10537362098693848, 0.10535311698913574, 0.10566926002502441, 0.10545182228088379, 0.10553932189941406, 0.10540390014648438, 0.10549497604370117, 0.10512185096740723, 0.10543417930603027, 0.1050729751586914, 0.1057126522064209, 0.10597515106201172, 0.10552811622619629, 0.10519719123840332, 0.10565018653869629, 0.10512709617614746, 0.10593914985656738, 0.10515880584716797, 0.10537981986999512, 0.1051180362701416, 0.10559844970703125, 0.10530495643615723, 0.10568761825561523, 0.10510087013244629, 0.10554385185241699, 0.10488295555114746, 0.10570216178894043, 0.10489726066589355, 0.10552573204040527, 0.10499954223632812, 0.10575246810913086, 0.10532903671264648, 0.10549044609069824, 0.10509443283081055, 0.10528802871704102, 0.10531044006347656, 0.10516548156738281, 0.10520410537719727, 0.1065669059753418, 0.10494828224182129, 0.10547685623168945, 0.10545635223388672, 0.10638689994812012, 0.10643386840820312, 0.10568022727966309, 0.10701441764831543, 0.10605549812316895, 0.1054997444152832, 0.1059873104095459, 0.10718059539794922, 0.1064608097076416, 0.10517001152038574, 0.10544919967651367, 0.10491251945495605, 0.10558843612670898, 0.10503315925598145, 0.10539054870605469, 0.10485672950744629, 0.10559511184692383, 0.10486173629760742, 0.10560369491577148, 0.10530900955200195, 0.10543441772460938, 0.10497093200683594, 0.10579729080200195, 0.10508894920349121, 0.10526466369628906, 0.10533666610717773, 0.10535001754760742, 0.10486221313476562, 0.10541272163391113, 0.10541677474975586, 0.10551023483276367, 0.10602188110351562, 0.10552453994750977, 0.1060187816619873, 0.10534930229187012, 0.10498785972595215, 0.10629963874816895, 0.10506916046142578, 0.10503029823303223, 0.10524415969848633, 0.10541963577270508, 0.1051025390625, 0.10503888130187988, 0.10554242134094238, 0.10533356666564941, 0.10518050193786621, 0.10554766654968262, 0.1058199405670166, 0.10619521141052246, 0.10500645637512207, 0.10640668869018555, 0.10552191734313965, 0.10590744018554688, 0.10538792610168457, 0.10528397560119629, 0.10526323318481445, 0.1054384708404541, 0.10513830184936523, 0.10547208786010742, 0.1049957275390625, 0.10545992851257324, 0.10473966598510742, 0.10527920722961426, 0.10583925247192383, 0.1054830551147461, 0.10510873794555664, 0.10520601272583008, 0.10512518882751465, 0.10532093048095703, 0.10580801963806152, 0.10533690452575684, 0.10550689697265625, 0.10567665100097656, 0.10546588897705078, 0.10536956787109375, 0.10546708106994629, 0.10563945770263672, 0.10537195205688477, 0.10526204109191895, 0.10555720329284668, 0.10566544532775879, 0.10555863380432129, 0.10572957992553711, 0.1057438850402832, 0.10553765296936035, 0.10532069206237793, 0.10585236549377441, 0.10524177551269531, 0.10569262504577637, 0.1056215763092041, 0.10543012619018555, 0.10606026649475098, 0.10582447052001953, 0.10571122169494629, 0.10509037971496582, 0.10592103004455566, 0.10577845573425293, 0.10520458221435547, 0.10707879066467285, 0.1050260066986084, 0.10553860664367676, 0.10550737380981445, 0.10557937622070312, 0.10556674003601074, 0.10521173477172852, 0.10521841049194336, 0.10541129112243652, 0.10558652877807617, 0.10523629188537598, 0.10529065132141113, 0.10558366775512695, 0.10540509223937988, 0.1053776741027832, 0.10535192489624023, 0.10586977005004883, 0.10583066940307617, 0.10552978515625, 0.10566401481628418, 0.10550951957702637, 0.10535097122192383, 0.10595846176147461, 0.10549020767211914, 0.10508060455322266, 0.10521125793457031, 0.10535502433776855, 0.10527992248535156, 0.10549521446228027, 0.10574531555175781, 0.10609579086303711, 0.10565662384033203, 0.10625648498535156, 0.10528349876403809, 0.10548973083496094, 0.10516667366027832, 0.10550832748413086, 0.10528755187988281, 0.10566878318786621, 0.10518074035644531, 0.10599756240844727, 0.10588788986206055, 0.10545134544372559, 0.10531353950500488, 0.10681033134460449, 0.10557126998901367, 0.10513901710510254, 0.10673785209655762, 0.1059424877166748, 0.10526633262634277, 0.1056056022644043, 0.10782146453857422, 0.10558772087097168, 0.10584282875061035, 0.10532617568969727, 0.10583996772766113, 0.10539031028747559, 0.10535311698913574, 0.10569143295288086, 0.1055305004119873, 0.10535502433776855, 0.10547924041748047, 0.10547232627868652, 0.10522818565368652, 0.10516500473022461, 0.10522842407226562, 0.10601019859313965, 0.10537123680114746, 0.10549044609069824, 0.10523772239685059, 0.1058347225189209, 0.10511207580566406, 0.10526680946350098, 0.10592150688171387, 0.14437055587768555, 0.2660660743713379, 0.10641169548034668, 0.10865592956542969, 0.10808563232421875, 0.1083383560180664, 0.10717558860778809, 0.10564160346984863, 0.10560226440429688, 0.10564422607421875, 0.10535407066345215, 0.1058502197265625, 0.10609889030456543, 0.11142778396606445, 0.10528206825256348, 0.10674500465393066, 0.10572290420532227, 0.10683894157409668, 0.10765337944030762, 0.10647320747375488, 0.10766410827636719, 0.10565423965454102, 0.10713696479797363, 0.10618948936462402, 0.10552501678466797, 0.12463974952697754, 0.10589194297790527, 0.10719919204711914, 0.10599827766418457, 0.1055746078491211, 0.1052238941192627, 0.10598278045654297, 0.10549306869506836, 0.10568857192993164, 0.10618734359741211, 0.10569190979003906, 0.1064603328704834, 0.10602712631225586, 0.10524439811706543, 0.11043834686279297, 0.1053621768951416, 0.10577726364135742, 0.10531902313232422, 0.10559940338134766, 0.10542535781860352, 0.10556578636169434, 0.10544753074645996, 0.10564327239990234, 0.1055755615234375, 0.10550951957702637, 0.1056663990020752, 0.10564613342285156, 0.10538315773010254, 0.1054069995880127, 0.10553717613220215, 0.10539460182189941, 0.10537409782409668, 0.10520744323730469, 0.10552978515625, 0.10614538192749023, 0.10602116584777832, 0.1051335334777832, 0.10591816902160645, 0.10579085350036621, 0.10556817054748535, 0.10635757446289062, 0.10584473609924316, 0.10575008392333984, 0.10521149635314941, 0.1054222583770752, 0.10533690452575684, 0.1058053970336914, 0.10519742965698242, 0.10532307624816895, 0.10562539100646973, 0.10572147369384766, 0.10508346557617188, 0.10550904273986816, 0.10551953315734863, 0.1057736873626709, 0.10525870323181152, 0.10550832748413086, 0.10503363609313965, 0.10545754432678223, 0.10616064071655273, 0.10539627075195312, 0.10594296455383301, 0.10541486740112305, 0.10517239570617676, 0.10586023330688477, 0.10523223876953125, 0.10508561134338379, 0.10565757751464844, 0.10514497756958008, 0.10558605194091797, 0.10530853271484375, 0.10561442375183105, 0.10501432418823242, 0.10553622245788574, 0.10522222518920898, 0.1051027774810791, 0.10534048080444336, 0.10549426078796387, 0.10524344444274902, 0.10545539855957031, 0.10552358627319336, 0.1052696704864502, 0.10540628433227539, 0.10557746887207031, 0.1056373119354248, 0.10623717308044434, 0.10584902763366699, 0.1052401065826416, 0.10676288604736328, 0.10613584518432617, 0.10529136657714844, 0.10624027252197266, 0.10782980918884277, 0.10582375526428223, 0.10574603080749512, 0.10568499565124512, 0.10557055473327637, 0.10571789741516113, 0.10514378547668457, 0.10565686225891113, 0.10541915893554688, 0.10550332069396973, 0.10564661026000977, 0.10580587387084961, 0.10578131675720215, 0.10552000999450684, 0.10556936264038086, 0.10591435432434082, 0.10535955429077148, 0.10558581352233887, 0.10529518127441406, 0.10601544380187988, 0.10512495040893555, 0.10535860061645508, 0.1055760383605957, 0.10542607307434082, 0.10587430000305176, 0.10578536987304688, 0.10585260391235352, 0.10573816299438477, 0.10565352439880371, 0.10583710670471191, 0.10559558868408203, 0.10560083389282227, 0.10555005073547363, 0.10515832901000977, 0.1054227352142334, 0.1054222583770752, 0.10501480102539062, 0.1118323802947998, 0.10577988624572754, 0.10567069053649902, 0.10662722587585449, 0.10573673248291016, 0.10529136657714844, 0.10669279098510742, 0.10607385635375977, 0.10558414459228516, 0.10531330108642578, 0.10560417175292969, 0.10526657104492188, 0.1054697036743164, 0.10506129264831543, 0.10602092742919922, 0.10562849044799805, 0.10545849800109863, 0.10549473762512207, 0.10561919212341309, 0.10594391822814941, 0.1059880256652832, 0.10536909103393555, 0.10537505149841309, 0.1053776741027832, 0.10520434379577637, 0.10582256317138672, 0.10560059547424316, 0.10600519180297852, 0.10538840293884277, 0.1056671142578125, 0.10549545288085938, 0.10556364059448242, 0.105804443359375, 0.10575485229492188, 0.10552978515625, 0.10560107231140137, 0.10552787780761719, 0.10549473762512207, 0.10533261299133301, 0.10551333427429199, 0.1053612232208252, 0.10507035255432129, 0.10560965538024902, 0.10494637489318848, 0.10507702827453613, 0.10510778427124023, 0.10520601272583008, 0.10524344444274902, 0.10542178153991699, 0.1055450439453125, 0.10549521446228027, 0.10624361038208008, 0.10602617263793945, 0.10625767707824707, 0.10552525520324707, 0.10544967651367188, 0.10554265975952148, 0.10561227798461914, 0.1060185432434082, 0.10540533065795898, 0.10558223724365234, 0.10540580749511719, 0.10552000999450684, 0.10537433624267578, 0.10522985458374023, 0.10526633262634277, 0.10564661026000977, 0.10559606552124023, 0.10511422157287598, 0.1055152416229248, 0.10583829879760742, 0.10477638244628906, 0.10535526275634766, 0.10520815849304199, 0.10532546043395996, 0.10560107231140137, 0.10534143447875977, 0.10572481155395508, 0.10519790649414062, 0.10523223876953125, 0.10597467422485352, 0.10576081275939941, 0.1057438850402832, 0.10786128044128418, 0.10540199279785156, 0.10525822639465332, 0.1062173843383789, 0.10545063018798828, 0.10520386695861816, 0.10515737533569336, 0.10523271560668945, 0.1052696704864502, 0.1056356430053711, 0.10529947280883789, 0.1054997444152832, 0.10540390014648438, 0.10566401481628418, 0.10524630546569824, 0.10675358772277832, 0.10518288612365723, 0.10600805282592773, 0.10670661926269531, 0.10562562942504883, 0.10529804229736328, 0.10555553436279297, 0.10779666900634766, 0.10540366172790527, 0.1054835319519043, 0.10534501075744629, 0.10527682304382324, 0.10559272766113281, 0.10537910461425781, 0.10559225082397461, 0.10495996475219727, 0.10551953315734863, 0.10560464859008789, 0.10577511787414551, 0.10542130470275879, 0.1052405834197998, 0.10520219802856445, 0.10586428642272949, 0.10500192642211914, 0.1056981086730957, 0.10541105270385742, 0.10561895370483398, 0.1052098274230957, 0.10540366172790527, 0.10537576675415039, 0.10562372207641602, 0.10524511337280273, 0.10544657707214355, 0.10550594329833984, 0.10575556755065918, 0.10528397560119629, 0.10556864738464355, 0.10559964179992676, 0.10530328750610352, 0.10627579689025879, 0.10559439659118652, 0.10554838180541992, 0.10531353950500488, 0.10532593727111816, 0.10561585426330566, 0.10589265823364258, 0.10540366172790527, 0.10625505447387695, 0.10575652122497559, 0.10578227043151855, 0.10638713836669922, 0.10523056983947754, 0.10536575317382812, 0.10506010055541992, 0.10574030876159668, 0.10590171813964844, 0.10517621040344238, 0.1057891845703125, 0.10537576675415039, 0.1053304672241211, 0.1055290699005127, 0.10524725914001465, 0.10555291175842285, 0.1055753231048584, 0.10555839538574219, 0.10508584976196289, 0.10604286193847656, 0.10583281517028809, 0.10550928115844727, 0.10602211952209473, 0.10540413856506348, 0.10549092292785645, 0.1052851676940918, 0.1051628589630127, 0.10578346252441406, 0.10559558868408203, 0.1054985523223877, 0.10556268692016602, 0.10526752471923828, 0.10557818412780762, 0.10604572296142578, 0.10530924797058105, 0.10665202140808105, 0.1060328483581543, 0.10558629035949707, 0.10526347160339355, 0.10527920722961426, 0.10555386543273926, 0.10547614097595215, 0.10538697242736816, 0.10577034950256348, 0.10613369941711426, 0.10577130317687988, 0.10547661781311035, 0.1055757999420166, 0.10591006278991699, 0.10555601119995117, 0.10628080368041992, 0.10601615905761719, 0.10554718971252441, 0.10515213012695312, 0.10544919967651367, 0.1055610179901123, 0.10549712181091309, 0.10547447204589844, 0.10547995567321777, 0.10566329956054688, 0.1054844856262207, 0.10536718368530273, 0.10566210746765137, 0.10616803169250488, 0.10591483116149902, 0.10547614097595215, 0.10556292533874512, 0.10551905632019043, 0.10575342178344727, 0.10612607002258301, 0.10550689697265625, 0.1056222915649414, 0.10547137260437012, 0.10587668418884277, 0.10549259185791016, 0.1057898998260498, 0.10548710823059082, 0.10573315620422363, 0.105377197265625, 0.10553145408630371, 0.1053469181060791, 0.10529541969299316, 0.10563969612121582, 0.10580062866210938, 0.10559701919555664, 0.10552072525024414, 0.10549807548522949, 0.10519242286682129, 0.10535836219787598, 0.10534429550170898, 0.10552215576171875, 0.10571646690368652, 0.10529065132141113, 0.10567784309387207, 0.10547614097595215, 0.10628914833068848, 0.1054081916809082, 0.10575985908508301, 0.10666179656982422, 0.10615754127502441, 0.10521602630615234, 0.10594630241394043, 0.10793471336364746, 0.10619401931762695, 0.10549759864807129, 0.10587644577026367, 0.10563230514526367, 0.10568475723266602, 0.10512900352478027, 0.10558605194091797, 0.10564661026000977, 0.10533523559570312, 0.10564351081848145, 0.10531854629516602, 0.10582399368286133, 0.10548210144042969, 0.10580778121948242, 0.10602378845214844, 0.10548686981201172, 0.10645461082458496, 0.1060175895690918, 0.10581111907958984, 0.1055455207824707, 0.10551810264587402, 0.10558795928955078, 0.10557055473327637, 0.10538887977600098, 0.10596299171447754, 0.10624480247497559, 0.10602235794067383, 0.10587620735168457, 0.10593652725219727, 0.1054680347442627, 0.10587525367736816, 0.10537338256835938, 0.10553431510925293, 0.10545587539672852, 0.10550785064697266, 0.1049962043762207, 0.10612344741821289, 0.10608816146850586, 0.10531949996948242, 0.10667181015014648, 0.10565066337585449, 0.10561132431030273, 0.10693120956420898, 0.10613846778869629, 0.10587739944458008, 0.10553884506225586, 0.10552716255187988, 0.10571718215942383, 0.10522866249084473, 0.10554337501525879, 0.10550403594970703, 0.10549569129943848, 0.10565972328186035, 0.10551571846008301, 0.10555148124694824, 0.10595941543579102, 0.10595059394836426, 0.10550785064697266, 0.10592293739318848, 0.10571408271789551, 0.10529327392578125, 0.10593962669372559, 0.10548639297485352, 0.10529184341430664, 0.1058967113494873, 0.1053156852722168, 0.10603499412536621, 0.10549783706665039, 0.1058201789855957, 0.10519790649414062, 0.10564875602722168, 0.10528206825256348, 0.1057741641998291, 0.10515356063842773, 0.10526800155639648, 0.10550880432128906, 0.10510802268981934, 0.1049957275390625, 0.1055302619934082, 0.10529088973999023, 0.10537409782409668, 0.10527729988098145, 0.10565805435180664, 0.10620284080505371, 0.10569286346435547, 0.10606980323791504, 0.10556507110595703, 0.10727119445800781, 0.1056673526763916, 0.10637974739074707, 0.10570669174194336, 0.10508298873901367, 0.10557222366333008, 0.10514163970947266, 0.10612940788269043, 0.10552620887756348, 0.1056828498840332, 0.10537219047546387, 0.1055295467376709, 0.10541009902954102, 0.10604667663574219, 0.10535216331481934, 0.10606145858764648, 0.10543942451477051, 0.1060950756072998, 0.10496282577514648, 0.1062614917755127, 0.10547494888305664, 0.10591816902160645, 0.10538768768310547, 0.10539841651916504, 0.10566425323486328, 0.10562443733215332, 0.10541296005249023, 0.10547161102294922, 0.10541009902954102, 0.1059417724609375, 0.10514044761657715, 0.10580062866210938, 0.10573482513427734, 0.10556721687316895, 0.10556530952453613, 0.10597825050354004, 0.10529112815856934, 0.10573339462280273, 0.10538935661315918, 0.10550379753112793, 0.10533976554870605, 0.10590362548828125, 0.10529446601867676, 0.10579609870910645, 0.10525655746459961, 0.10582494735717773, 0.10523343086242676, 0.10662293434143066, 0.10523724555969238, 0.10547780990600586, 0.10639262199401855, 0.10609698295593262, 0.10519814491271973, 0.1057126522064209, 0.10851216316223145, 0.10608482360839844, 0.10539865493774414, 0.10574603080749512, 0.10541725158691406, 0.1058502197265625, 0.10553741455078125, 0.10546040534973145, 0.10551071166992188, 0.10548567771911621, 0.10567688941955566, 0.10557913780212402, 0.10538411140441895, 0.10550355911254883, 0.10562276840209961, 0.10589480400085449, 0.10538601875305176, 0.10551881790161133, 0.10576152801513672, 0.10633563995361328, 0.10549330711364746, 0.10549402236938477, 0.10538458824157715, 0.10563802719116211, 0.10558962821960449, 0.10568380355834961, 0.10528302192687988, 0.10650849342346191, 0.10554790496826172, 0.10567569732666016, 0.10543584823608398, 0.10579323768615723, 0.10534381866455078, 0.10582590103149414, 0.1054372787475586, 0.10589957237243652, 0.10550737380981445, 0.10584568977355957, 0.10533261299133301, 0.10565805435180664, 0.10623526573181152, 0.10559368133544922, 0.10538887977600098, 0.10730457305908203, 0.10537528991699219, 0.10584735870361328, 0.10539650917053223, 0.10547161102294922, 0.10635185241699219, 0.10586714744567871, 0.10594868659973145, 0.10591006278991699, 0.10588502883911133, 0.10585784912109375, 0.10584831237792969, 0.1056978702545166, 0.10547661781311035, 0.1058650016784668, 0.10553908348083496, 0.1057734489440918, 0.10548853874206543, 0.10564994812011719, 0.10587143898010254, 0.10582852363586426, 0.10560727119445801, 0.10524272918701172, 0.10582470893859863, 0.10572171211242676, 0.10519671440124512, 0.10597825050354004, 0.10557126998901367, 0.10606074333190918, 0.10614752769470215, 0.10580968856811523, 0.10562324523925781, 0.10563802719116211, 0.10572385787963867, 0.10542154312133789, 0.10562920570373535, 0.10557246208190918, 0.10657620429992676, 0.10527968406677246, 0.10531163215637207, 0.10593247413635254, 0.10748982429504395, 0.10643267631530762, 0.10570764541625977, 0.1063692569732666, 0.10696911811828613, 0.10561990737915039, 0.10679173469543457, 0.10623574256896973, 0.10580301284790039, 0.10588359832763672, 0.10553908348083496, 0.10607433319091797, 0.1058051586151123, 0.1062004566192627, 0.10623621940612793, 0.10589981079101562, 0.10521817207336426, 0.10586094856262207, 0.10550236701965332, 0.10579228401184082, 0.10573554039001465, 0.1058504581451416, 0.1050410270690918, 0.10546374320983887, 0.1056051254272461, 0.10611152648925781, 0.10573291778564453, 0.1058359146118164, 0.10563969612121582, 0.10562896728515625, 0.10591864585876465, 0.10556292533874512, 0.10564351081848145, 0.10612106323242188, 0.10559606552124023, 0.10569643974304199, 0.10551071166992188, 0.10533380508422852, 0.10558962821960449, 0.10604691505432129, 0.10626935958862305, 0.10560154914855957, 0.10579490661621094, 0.1055457592010498, 0.10546278953552246, 0.1058053970336914, 0.10521769523620605, 0.10605883598327637, 0.10521697998046875, 0.10606980323791504, 0.10516047477722168, 0.10688424110412598, 0.10593175888061523, 0.1062917709350586, 0.10685014724731445, 0.10555791854858398, 0.10588860511779785, 0.106658935546875, 0.1087958812713623, 0.10607361793518066, 0.10537385940551758, 0.1056356430053711, 0.10540533065795898, 0.10615873336791992, 0.10529446601867676, 0.10538530349731445, 0.1066884994506836, 0.10595703125, 0.10578060150146484, 0.1055901050567627, 0.10579848289489746, 0.10563540458679199, 0.10596942901611328, 0.10602736473083496, 0.10571002960205078, 0.10586094856262207, 0.10525870323181152, 0.10600113868713379, 0.10553574562072754, 0.10598587989807129, 0.10523819923400879, 0.10566186904907227, 0.10620284080505371, 0.10622644424438477, 0.10574626922607422, 0.10590744018554688, 0.10605359077453613, 0.10616755485534668, 0.10515856742858887, 0.10584521293640137, 0.10581660270690918, 0.10573029518127441, 0.10497474670410156, 0.10602903366088867, 0.10629129409790039, 0.1142888069152832, 0.10587263107299805, 0.10602116584777832, 0.10636472702026367, 0.10586905479431152, 0.10537242889404297, 0.10692572593688965, 0.10629820823669434, 0.10530328750610352, 0.10569286346435547, 0.10588407516479492, 0.1051175594329834, 0.10545969009399414, 0.10593247413635254, 0.10536026954650879, 0.10583972930908203, 0.10584616661071777, 0.10632872581481934, 0.10630083084106445, 0.10581564903259277, 0.10593342781066895, 0.1052699089050293, 0.10561704635620117, 0.10599875450134277, 0.10588216781616211, 0.10588836669921875, 0.10541462898254395, 0.10569405555725098, 0.10594820976257324, 0.10554766654968262, 0.1061098575592041, 0.10551142692565918, 0.10578632354736328, 0.10556983947753906, 0.10615968704223633, 0.10529637336730957, 0.10558247566223145, 0.10545611381530762, 0.10524129867553711, 0.1055612564086914, 0.10526680946350098, 0.10541629791259766, 0.1056222915649414, 0.10564208030700684, 0.10546183586120605, 0.10561823844909668, 0.10522961616516113, 0.10582137107849121, 0.10557961463928223, 0.1056663990020752, 0.10607314109802246, 0.10538005828857422, 0.10535764694213867, 0.10677957534790039, 0.10552477836608887, 0.10543608665466309, 0.10525107383728027, 0.10535573959350586, 0.10611772537231445, 0.1054689884185791, 0.10545754432678223, 0.10531497001647949, 0.10581684112548828, 0.10539817810058594, 0.10546422004699707, 0.10514140129089355, 0.10551214218139648, 0.10545063018798828, 0.10567951202392578, 0.10573577880859375, 0.10589075088500977, 0.10503220558166504, 0.10594916343688965, 0.10529136657714844, 0.10520410537719727, 0.10635709762573242, 0.10511040687561035, 0.10568380355834961, 0.10563230514526367, 0.10598254203796387, 0.10541582107543945, 0.10568523406982422, 0.10596227645874023, 0.10561275482177734, 0.1059410572052002, 0.10587334632873535, 0.10588550567626953, 0.1061246395111084, 0.10519814491271973, 0.1057577133178711, 0.10541868209838867, 0.10534191131591797, 0.10571956634521484, 0.10630130767822266, 0.10594892501831055, 0.10535883903503418, 0.11737585067749023, 0.10683465003967285, 0.1058800220489502, 0.10552382469177246, 0.10666513442993164, 0.10603857040405273, 0.10650801658630371, 0.10569095611572266, 0.10732483863830566, 0.10619783401489258, 0.10652446746826172, 0.10524988174438477, 0.10565876960754395, 0.10530471801757812, 0.10554671287536621, 0.10921573638916016, 0.10561466217041016, 0.10558414459228516, 0.10633730888366699, 0.10603904724121094, 0.10639142990112305, 0.10698294639587402, 0.10529685020446777, 0.10600876808166504, 0.10527396202087402, 0.10562968254089355, 0.10542511940002441, 0.10547685623168945, 0.10561013221740723, 0.10556864738464355, 0.10540890693664551, 0.10538792610168457, 0.10600733757019043, 0.1049644947052002, 0.10670709609985352, 0.10504364967346191, 0.1059572696685791, 0.10569572448730469, 0.10549473762512207, 0.10555887222290039, 0.10618114471435547, 0.10837984085083008, 0.10605168342590332, 0.10576462745666504, 0.10612678527832031, 0.10618948936462402, 0.10587501525878906, 0.10634970664978027, 0.10700535774230957, 0.10657334327697754, 0.10622501373291016, 0.10704684257507324, 0.10567641258239746, 0.10557198524475098, 0.10598492622375488, 0.1056814193725586, 0.10619163513183594, 0.10585212707519531, 0.10688376426696777, 0.10562372207641602, 0.10642576217651367, 0.10579228401184082, 0.105682373046875, 0.10524678230285645, 0.10596728324890137, 0.10514450073242188, 0.10625267028808594, 0.10527753829956055, 0.10605669021606445, 0.10526752471923828, 0.10611534118652344, 0.10586261749267578, 0.10631847381591797, 0.10549449920654297, 0.10527610778808594, 0.10554671287536621, 0.10629153251647949, 0.10560083389282227, 0.10611796379089355, 0.10543584823608398, 0.10615658760070801, 0.10638689994812012, 0.1058661937713623, 0.1061246395111084, 0.10614562034606934, 0.1054677963256836, 0.10588264465332031, 0.10529398918151855, 0.1061103343963623, 0.10572123527526855, 0.10563039779663086, 0.10617423057556152, 0.10675430297851562, 0.1056971549987793, 0.1059262752532959, 0.10592937469482422, 0.1068425178527832, 0.10547614097595215, 0.1056969165802002, 0.10723662376403809, 0.1063385009765625, 0.10530328750610352, 0.1060326099395752, 0.1060032844543457, 0.10618853569030762, 0.10531067848205566, 0.1057898998260498, 0.10520625114440918, 0.10645127296447754, 0.10572671890258789, 0.10610103607177734, 0.10560917854309082, 0.10552978515625, 0.1057291030883789, 0.10639667510986328, 0.10576868057250977, 0.10617876052856445, 0.10578584671020508, 0.10628032684326172, 0.1054544448852539, 0.10570240020751953, 0.1055140495300293, 0.10605669021606445, 0.10527467727661133, 0.10552597045898438, 0.10568571090698242, 0.10580658912658691, 0.10552406311035156, 0.10595417022705078, 0.10578703880310059, 0.10561108589172363, 0.1058650016784668, 0.10615134239196777, 0.10542631149291992, 0.10588359832763672, 0.10566496849060059, 0.10594606399536133, 0.10519289970397949, 0.10612893104553223, 0.10515570640563965, 0.10565924644470215, 0.10590481758117676, 0.10588550567626953, 0.10605311393737793, 0.10591268539428711, 0.10524487495422363, 0.10729694366455078, 0.10532832145690918, 0.10602641105651855, 0.10575246810913086, 0.10744953155517578, 0.10646700859069824, 0.10602617263793945, 0.10539984703063965, 0.10582923889160156, 0.10610532760620117, 0.10609316825866699, 0.10573840141296387, 0.10583257675170898, 0.10548996925354004, 0.10582113265991211, 0.10596060752868652, 0.10612630844116211, 0.10640144348144531, 0.1052398681640625, 0.1060938835144043, 0.10625362396240234, 0.10562872886657715, 0.1057131290435791, 0.10578155517578125, 0.10588288307189941, 0.1055305004119873, 0.10624217987060547, 0.10585379600524902, 0.1059865951538086, 0.10608911514282227, 0.10631632804870605, 0.10640239715576172, 0.10628461837768555, 0.1060936450958252, 0.10568594932556152, 0.10555887222290039, 0.10578632354736328, 0.10559630393981934, 0.1057441234588623, 0.10557675361633301, 0.10575723648071289, 0.10610008239746094, 0.10646271705627441, 0.10620379447937012, 0.10609316825866699, 0.10559582710266113, 0.10537362098693848, 0.10767555236816406, 0.10644650459289551, 0.10596919059753418, 0.10586929321289062, 0.10605216026306152, 0.10585641860961914, 0.1051175594329834, 0.10529899597167969, 0.10660481452941895, 0.10564541816711426, 0.10602593421936035, 0.10547065734863281, 0.10586667060852051, 0.10584902763366699, 0.1055912971496582, 0.10561847686767578, 0.10566425323486328, 0.10614633560180664, 0.10583043098449707, 0.10655999183654785, 0.10527276992797852, 0.10570073127746582, 0.10590410232543945, 0.10581088066101074, 0.10586190223693848, 0.10572624206542969, 0.10553789138793945, 0.10549235343933105, 0.10567760467529297, 0.10552525520324707, 0.10568714141845703, 0.1054985523223877, 0.10567736625671387, 0.10629558563232422, 0.10588693618774414, 0.10585355758666992, 0.10557222366333008, 0.10556769371032715, 0.10580015182495117, 0.1052544116973877, 0.10565471649169922, 0.10602045059204102, 0.10570383071899414, 0.10576033592224121, 0.10557174682617188, 0.10642051696777344, 0.10607314109802246, 0.10691642761230469, 0.10580801963806152, 0.10514593124389648, 0.1052696704864502, 0.10574221611022949, 0.10585737228393555, 0.10550260543823242, 0.10559558868408203, 0.10558414459228516, 0.10544586181640625, 0.10608315467834473, 0.10555839538574219, 0.10564136505126953, 0.10574626922607422, 0.10553312301635742, 0.10800957679748535, 0.1058962345123291, 0.10604071617126465, 0.10570597648620605, 0.10587143898010254, 0.10569453239440918, 0.10628342628479004, 0.10632085800170898, 0.1060335636138916, 0.10547709465026855, 0.10583257675170898, 0.10565447807312012, 0.10599732398986816, 0.10572671890258789, 0.10589075088500977, 0.10604071617126465, 0.10551285743713379, 0.10610818862915039, 0.10628247261047363, 0.10606122016906738, 0.10509538650512695, 0.10589027404785156, 0.10557818412780762, 0.10590863227844238, 0.10541248321533203, 0.10545897483825684, 0.10665035247802734, 0.10558867454528809, 0.10563850402832031, 0.10570263862609863, 0.10607314109802246, 0.10673832893371582, 0.10614609718322754, 0.10657572746276855, 0.10666966438293457, 0.10546255111694336, 0.10592103004455566, 0.10686445236206055, 0.10606646537780762, 0.10526776313781738, 0.10570693016052246, 0.10538029670715332, 0.10586094856262207, 0.10515546798706055, 0.10579919815063477, 0.10553622245788574, 0.10602521896362305, 0.10600519180297852, 0.10538172721862793, 0.10545754432678223, 0.10667228698730469, 0.10612249374389648, 0.10555624961853027, 0.10552024841308594, 0.10539102554321289, 0.10606575012207031, 0.10558795928955078, 0.10518407821655273, 0.10535812377929688, 0.10546135902404785, 0.10532712936401367, 0.10595583915710449, 0.1053776741027832, 0.10599327087402344, 0.10562825202941895, 0.10587143898010254, 0.10553407669067383, 0.10582280158996582, 0.10565471649169922, 528.1062672138214]  
t_14_gzip = [0.12322354316711426, 0.12769222259521484, 0.10349273681640625, 0.10338091850280762, 0.10405158996582031, 0.10395455360412598, 0.15526843070983887, 0.10413408279418945, 0.10394883155822754, 0.10376405715942383, 0.10472726821899414, 0.10329389572143555, 0.10348343849182129, 0.10303592681884766, 0.1041412353515625, 0.10363197326660156, 0.10407662391662598, 0.10334014892578125, 0.1038517951965332, 0.10351300239562988, 0.10377383232116699, 0.10381388664245605, 0.10349416732788086, 0.10400891304016113, 0.10342931747436523, 0.1031181812286377, 0.10351729393005371, 0.10341453552246094, 0.10385680198669434, 0.10411572456359863, 0.10372209548950195, 0.10346007347106934, 0.10360312461853027, 0.1038670539855957, 0.10349273681640625, 0.1038045883178711, 0.10327291488647461, 0.10381174087524414, 0.10350251197814941, 0.10399365425109863, 0.10339641571044922, 0.10332489013671875, 0.10399651527404785, 0.10359859466552734, 0.10396218299865723, 0.10329818725585938, 0.10324406623840332, 0.10367631912231445, 0.1032111644744873, 0.10358428955078125, 0.10330367088317871, 0.10401678085327148, 0.10342979431152344, 0.1036062240600586, 0.10374236106872559, 0.10381436347961426, 0.1041262149810791, 0.10324263572692871, 0.10494470596313477, 0.10369372367858887, 0.10344290733337402, 0.10323309898376465, 0.1035768985748291, 0.1040046215057373, 0.10375809669494629, 0.10370302200317383, 0.10349249839782715, 0.1031489372253418, 0.10369634628295898, 0.1031191349029541, 0.10340261459350586, 0.10332584381103516, 0.10352039337158203, 0.10366225242614746, 0.10373306274414062, 0.10338330268859863, 0.10360145568847656, 0.10385847091674805, 0.10345816612243652, 0.10388302803039551, 0.10322737693786621, 0.10322880744934082, 0.10367035865783691, 0.10314345359802246, 0.10368728637695312, 0.1034085750579834, 0.10349202156066895, 0.10347318649291992, 0.10347795486450195, 0.10337185859680176, 0.10276031494140625, 0.10355591773986816, 0.10322880744934082, 0.10369205474853516, 0.10340642929077148, 0.10364985466003418, 0.10347461700439453, 0.10362553596496582, 0.10422563552856445, 0.10396814346313477, 0.10474991798400879, 0.10416364669799805, 0.10374045372009277, 0.10430359840393066, 0.10464644432067871, 0.1040196418762207, 0.10536956787109375, 0.10381531715393066, 0.10362386703491211, 0.10439682006835938, 0.10509848594665527, 0.10462403297424316, 0.10447001457214355, 0.10366392135620117, 0.10402750968933105, 0.10370659828186035, 0.10393238067626953, 0.10294532775878906, 0.10329914093017578, 0.10376334190368652, 0.10347151756286621, 0.10372018814086914, 0.10365438461303711, 0.1038827896118164, 0.10377025604248047, 0.10339236259460449, 0.10369062423706055, 0.10368776321411133, 0.10395240783691406, 0.10321688652038574, 0.10355019569396973, 0.10374617576599121, 0.10331082344055176, 0.10369229316711426, 0.10368585586547852, 0.10360097885131836, 0.1036524772644043, 0.10368919372558594, 0.10387539863586426, 0.10358095169067383, 0.10403585433959961, 0.10912919044494629, 0.10361242294311523, 0.1038055419921875, 0.10386157035827637, 0.10379314422607422, 0.10330033302307129, 0.10373210906982422, 0.1136164665222168, 0.10457563400268555, 0.10408210754394531, 0.10459208488464355, 0.10429191589355469, 0.10396385192871094, 0.10506510734558105, 0.10446023941040039, 0.10453343391418457, 0.10384702682495117, 0.10410022735595703, 0.10437893867492676, 0.10409092903137207, 0.10384941101074219, 0.10441946983337402, 0.10400032997131348, 0.10500240325927734, 0.10401797294616699, 0.10433197021484375, 0.10423660278320312, 0.10428857803344727, 0.10406684875488281, 0.1040656566619873, 0.1037745475769043, 0.10405230522155762, 0.10424685478210449, 0.10397815704345703, 0.10388708114624023, 0.10389018058776855, 0.10387754440307617, 0.1043996810913086, 0.10405254364013672, 0.1041252613067627, 0.10389995574951172, 0.10413575172424316, 0.10448455810546875, 0.10440850257873535, 0.10393595695495605, 0.10365986824035645, 0.10381388664245605, 0.10396504402160645, 0.10423612594604492, 0.10374903678894043, 0.10430765151977539, 0.10346412658691406, 0.1039884090423584, 0.10393309593200684, 0.10355091094970703, 0.10470271110534668, 0.1038520336151123, 0.10431170463562012, 0.10459160804748535, 0.1043548583984375, 0.10402464866638184, 0.10433101654052734, 0.1042168140411377, 0.10389113426208496, 0.1038045883178711, 0.1033632755279541, 0.1040487289428711, 0.10379147529602051, 0.10372376441955566, 0.10389161109924316, 0.10385894775390625, 0.10416984558105469, 0.10356688499450684, 0.1038808822631836, 0.1039276123046875, 0.10359525680541992, 0.10371041297912598, 0.10364508628845215, 0.10409808158874512, 0.1040201187133789, 0.1037452220916748, 0.10414385795593262, 0.10365152359008789, 0.10430359840393066, 0.10446286201477051, 0.1042318344116211, 0.10365962982177734, 0.10378265380859375, 0.10391855239868164, 0.10370802879333496, 0.10401558876037598, 0.1038658618927002, 0.1041250228881836, 0.1039590835571289, 0.10332274436950684, 0.10516166687011719, 0.10360240936279297, 0.10358262062072754, 0.10353517532348633, 0.10360336303710938, 0.1037893295288086, 0.10331845283508301, 0.10345745086669922, 0.10387778282165527, 0.10377287864685059, 0.10397577285766602, 0.10371899604797363, 0.10487198829650879, 0.10362553596496582, 0.10445976257324219, 0.10451102256774902, 0.10399627685546875, 0.10383462905883789, 0.1054995059967041, 0.10508155822753906, 0.10433435440063477, 0.1034996509552002, 0.1038663387298584, 0.10400652885437012, 0.1038515567779541, 0.10387253761291504, 0.10314226150512695, 0.10468864440917969, 0.10522961616516113, 0.10512733459472656, 0.10392498970031738, 0.10355877876281738, 0.10403823852539062, 0.10337185859680176, 0.10452628135681152, 0.10361170768737793, 0.10403323173522949, 0.10378623008728027, 0.10423660278320312, 0.10413813591003418, 0.1038212776184082, 0.10366296768188477, 0.1038825511932373, 0.10399723052978516, 0.10386514663696289, 0.10422635078430176, 0.10451865196228027, 0.10397171974182129, 0.10384988784790039, 0.10355615615844727, 0.10383248329162598, 0.10801196098327637, 0.1051337718963623, 0.10421156883239746, 0.10541987419128418, 0.10408520698547363, 0.10373067855834961, 0.1039133071899414, 0.104339599609375, 0.10378098487854004, 0.10473179817199707, 0.10384654998779297, 0.10457658767700195, 0.10411882400512695, 0.10373187065124512, 0.10384535789489746, 0.10376381874084473, 0.1043701171875, 0.10350179672241211, 0.1042628288269043, 0.10413408279418945, 0.10377907752990723, 0.10417342185974121, 0.10390734672546387, 0.10420012474060059, 0.1037907600402832, 0.10392880439758301, 0.10393309593200684, 0.10365056991577148, 0.10405468940734863, 0.1039423942565918, 0.10361838340759277, 0.10422897338867188, 0.1034090518951416, 0.1038517951965332, 0.10361957550048828, 0.10405802726745605, 0.1042318344116211, 0.10417771339416504, 0.10368037223815918, 0.10393571853637695, 0.10373401641845703, 0.10412478446960449, 0.10439062118530273, 0.10658597946166992, 0.10426497459411621, 0.10428619384765625, 0.10402441024780273, 0.1041572093963623, 0.10398530960083008, 0.1041417121887207, 0.10404205322265625, 0.10389852523803711, 0.10426974296569824, 0.1048591136932373, 0.1040036678314209, 0.10402679443359375, 0.10407733917236328, 0.10501265525817871, 0.10444474220275879, 0.10640382766723633, 0.10675764083862305, 0.10768270492553711, 0.10687589645385742, 0.10740923881530762, 0.10721707344055176, 0.10712170600891113, 0.10694098472595215, 0.10869503021240234, 0.10390257835388184, 0.10465645790100098, 0.10411596298217773, 0.10411930084228516, 0.11113309860229492, 0.1043093204498291, 0.104095458984375, 0.10376143455505371, 0.1041264533996582, 0.10450434684753418, 0.10444140434265137, 0.10423827171325684, 0.1042630672454834, 0.10455584526062012, 0.10399556159973145, 0.10438728332519531, 0.10439586639404297, 0.10451459884643555, 0.10477662086486816, 0.10418176651000977, 0.10412144660949707, 0.1039116382598877, 0.10426092147827148, 0.10437393188476562, 0.10408520698547363, 0.10451579093933105, 0.10385298728942871, 0.10377764701843262, 0.10403871536254883, 0.10406327247619629, 0.10454344749450684, 0.10394644737243652, 0.1041557788848877, 0.10405230522155762, 0.10387229919433594, 0.1041264533996582, 0.10467720031738281, 0.10403609275817871, 0.10488224029541016, 0.10544753074645996, 0.10403275489807129, 0.10420656204223633, 0.10412931442260742, 0.10547852516174316, 0.10533452033996582, 0.10436749458312988, 0.10426688194274902, 0.10414242744445801, 0.1040806770324707, 0.10464978218078613, 0.10388636589050293, 0.10401511192321777, 0.10422420501708984, 0.10428214073181152, 0.10410785675048828, 0.10449838638305664, 0.10406827926635742, 0.10420680046081543, 0.10482525825500488, 0.10383486747741699, 0.10395622253417969, 0.10403633117675781, 0.10398721694946289, 0.10419297218322754, 0.10387611389160156, 0.10424304008483887, 0.1039268970489502, 0.10384798049926758, 0.10408878326416016, 0.104522705078125, 0.1041114330291748, 0.10449790954589844, 0.1045231819152832, 0.10453605651855469, 0.10409879684448242, 0.10431647300720215, 0.1040949821472168, 0.1039421558380127, 0.1041557788848877, 0.10420370101928711, 0.10390305519104004, 0.1047813892364502, 0.10409665107727051, 0.10417842864990234, 0.10513424873352051, 0.10405540466308594, 0.10409736633300781, 0.1050868034362793, 0.10479140281677246, 0.10389900207519531, 0.1041097640991211, 0.10382246971130371, 0.10438990592956543, 0.10433602333068848, 0.10415220260620117, 0.10410261154174805, 0.1040041446685791, 0.1041862964630127, 0.1041116714477539, 0.10402941703796387, 0.10488486289978027, 0.10392403602600098, 0.10401773452758789, 0.10396957397460938, 0.10373187065124512, 0.10370707511901855, 0.10405707359313965, 0.10431170463562012, 0.10405778884887695, 0.10367798805236816, 0.10352516174316406, 0.10406327247619629, 0.10328388214111328, 0.10378789901733398, 0.10416102409362793, 0.10376429557800293, 0.10443425178527832, 0.10413002967834473, 0.10391092300415039, 0.10417652130126953, 0.10377812385559082, 0.10457825660705566, 0.10427689552307129, 0.10428547859191895, 0.10379791259765625, 0.10374903678894043, 0.10372447967529297, 0.10363101959228516, 0.10450482368469238, 0.10366487503051758, 0.10441136360168457, 0.10365939140319824, 0.10468411445617676, 0.10373115539550781, 0.1038367748260498, 0.10484099388122559, 0.10414671897888184, 0.10410952568054199, 0.10350942611694336, 0.10397648811340332, 0.10415482521057129, 0.10359716415405273, 0.10435175895690918, 0.10381817817687988, 0.10389375686645508, 0.10419821739196777, 0.10415792465209961, 0.10384941101074219, 0.10431051254272461, 0.10411667823791504, 0.10409712791442871, 0.10429692268371582, 0.10408234596252441, 0.10399675369262695, 0.10420703887939453, 0.10382318496704102, 0.10405111312866211, 0.10410499572753906, 0.1040034294128418, 0.10410737991333008, 0.10375404357910156, 0.10371875762939453, 0.10352015495300293, 0.10415101051330566, 0.1038508415222168, 0.1047823429107666, 0.10407590866088867, 0.10388684272766113, 0.10419774055480957, 0.10383248329162598, 0.10410928726196289, 0.10423731803894043, 0.1041250228881836, 0.10378098487854004, 0.10373353958129883, 0.10375642776489258, 0.10380029678344727, 0.1045236587524414, 0.10404539108276367, 0.10454344749450684, 0.10449337959289551, 0.10439705848693848, 0.10358119010925293, 0.10480165481567383, 0.10482907295227051, 0.10418534278869629, 0.10380911827087402, 0.10649323463439941, 0.10466980934143066, 0.10426950454711914, 0.1036074161529541, 0.10402178764343262, 0.10371518135070801, 0.10409212112426758, 0.10406875610351562, 0.10380268096923828, 0.10357451438903809, 0.10384082794189453, 0.10471034049987793, 0.10362458229064941, 0.10388755798339844, 0.10377931594848633, 0.10424137115478516, 0.10415077209472656, 0.10394287109375, 0.10434770584106445, 0.10393524169921875, 0.10439348220825195, 0.10371971130371094, 0.10391902923583984, 0.10374116897583008, 0.10400533676147461, 0.10408759117126465, 0.10393023490905762, 0.10432195663452148, 0.10433268547058105, 0.10372805595397949, 0.10409808158874512, 0.10375237464904785, 0.10412478446960449, 0.10381698608398438, 0.10432291030883789, 0.10360002517700195, 0.10388636589050293, 0.1037447452545166, 0.10402798652648926, 0.10381555557250977, 0.10447430610656738, 0.1042938232421875, 0.10443568229675293, 0.1043093204498291, 0.1048576831817627, 0.1035153865814209, 0.10401082038879395, 0.10367417335510254, 0.10431194305419922, 0.10408616065979004, 0.1037893295288086, 0.10414958000183105, 0.10394001007080078, 0.10432052612304688, 0.10390686988830566, 0.10394454002380371, 0.10380744934082031, 0.10398268699645996, 0.10397219657897949, 0.10406613349914551, 0.10432720184326172, 0.10449814796447754, 0.1041266918182373, 0.10413455963134766, 0.10380101203918457, 0.10409903526306152, 0.1036064624786377, 0.10430049896240234, 0.10435676574707031, 0.10397553443908691, 0.10371184349060059, 0.10433101654052734, 0.10458135604858398, 0.10433650016784668, 0.10469532012939453, 0.1041419506072998, 0.10480904579162598, 0.10433173179626465, 0.10398292541503906, 0.10396742820739746, 0.10400843620300293, 0.1042168140411377, 0.10427999496459961, 0.10362362861633301, 0.1039128303527832, 0.10389018058776855, 0.1048276424407959, 0.10400772094726562, 0.10462594032287598, 0.10393404960632324, 0.10503578186035156, 0.1038062572479248, 0.10529875755310059, 0.1046760082244873, 0.10474777221679688, 0.10418987274169922, 0.10390114784240723, 0.10425829887390137, 0.10366487503051758, 0.10394287109375, 0.1038203239440918, 0.10405278205871582, 0.1039130687713623, 0.10409021377563477, 0.1040654182434082, 0.10431241989135742, 0.1040794849395752, 0.10418033599853516, 0.10387969017028809, 0.10399675369262695, 0.10372042655944824, 0.10505414009094238, 0.10415267944335938, 0.10405850410461426, 0.10359048843383789, 0.10412478446960449, 0.10433459281921387, 0.10407400131225586, 0.10371184349060059, 0.10381889343261719, 0.10395145416259766, 0.10369372367858887, 0.10376238822937012, 0.10398674011230469, 0.10368919372558594, 0.10382223129272461, 0.10412359237670898, 0.10379552841186523, 0.10375261306762695, 0.10352706909179688, 0.10490584373474121, 0.10383367538452148, 0.10416746139526367, 0.10340070724487305, 0.10401010513305664, 0.10353541374206543, 0.10453939437866211, 0.10417914390563965, 0.10457849502563477, 0.1046442985534668, 0.10544586181640625, 0.10408544540405273, 0.10408639907836914, 0.10492849349975586, 0.10580921173095703, 0.1050729751586914, 0.104644775390625, 0.10391664505004883, 0.10408949851989746, 0.10412001609802246, 0.10419225692749023, 0.10442781448364258, 0.10363125801086426, 0.10404348373413086, 0.1040799617767334, 0.10401654243469238, 0.1041707992553711, 0.10441231727600098, 0.10424590110778809, 0.10445189476013184, 0.10422420501708984, 0.10418367385864258, 0.10397529602050781, 0.10432314872741699, 0.10373139381408691, 0.10435700416564941, 0.10372591018676758, 0.1040339469909668, 0.1036672592163086, 0.10394859313964844, 0.10425186157226562, 0.10385584831237793, 0.10408139228820801, 0.1042327880859375, 0.10446310043334961, 0.1040189266204834, 0.10413050651550293, 0.10416388511657715, 0.10388994216918945, 0.10387516021728516, 0.10425806045532227, 0.10403609275817871, 0.10862565040588379, 0.10410094261169434, 0.10428953170776367, 0.10498881340026855, 0.10482382774353027, 0.10435056686401367, 0.10561656951904297, 0.10440564155578613, 0.10463237762451172, 0.10451531410217285, 0.10398530960083008, 0.10436296463012695, 0.10400962829589844, 0.10422897338867188, 0.10404324531555176, 0.10421299934387207, 0.10395598411560059, 0.10399198532104492, 0.10411262512207031, 0.10472702980041504, 0.10444331169128418, 0.10413742065429688, 0.10372352600097656, 0.10421204566955566, 0.10397982597351074, 0.10485672950744629, 0.10399079322814941, 0.10440468788146973, 0.10421395301818848, 0.10412859916687012, 0.10397028923034668, 0.10436487197875977, 0.10425734519958496, 0.10434675216674805, 0.10474109649658203, 0.10418701171875, 0.1042470932006836, 0.10391402244567871, 0.10401701927185059, 0.10395979881286621, 0.10433745384216309, 0.10404300689697266, 0.10361313819885254, 0.10363245010375977, 0.10404682159423828, 0.1036686897277832, 0.1041419506072998, 0.1038057804107666, 0.10393190383911133, 0.10366368293762207, 0.10435175895690918, 0.10440731048583984, 0.10474467277526855, 0.10377240180969238, 0.10519218444824219, 0.10398340225219727, 0.10408830642700195, 0.10376238822937012, 0.10376739501953125, 0.1043844223022461, 0.10403823852539062, 0.10395503044128418, 0.1039893627166748, 0.10350847244262695, 0.10410428047180176, 0.1039133071899414, 0.10375046730041504, 0.10426998138427734, 0.10347485542297363, 0.10407733917236328, 0.10368657112121582, 0.10446524620056152, 0.10411262512207031, 0.10431385040283203, 0.10394549369812012, 0.10425400733947754, 0.10436868667602539, 0.10393190383911133, 0.10387778282165527, 0.10390591621398926, 0.10377073287963867, 0.10421419143676758, 0.10375857353210449, 0.10446977615356445, 0.1045541763305664, 0.10427474975585938, 0.10443496704101562, 0.1037437915802002, 0.10460662841796875, 0.10401463508605957, 0.10377001762390137, 0.10396599769592285, 0.1036684513092041, 0.10351824760437012, 0.10392546653747559, 0.10379791259765625, 0.10447478294372559, 0.10374832153320312, 0.10390114784240723, 0.1042180061340332, 0.10485649108886719, 0.10364699363708496, 0.10535073280334473, 0.10432934761047363, 0.10436105728149414, 0.10382938385009766, 0.10530805587768555, 0.10414457321166992, 0.10407900810241699, 0.10394883155822754, 0.10392570495605469, 0.10412383079528809, 0.10391044616699219, 0.10395193099975586, 0.10344624519348145, 0.10367631912231445, 0.10377883911132812, 0.10414314270019531, 0.1036531925201416, 0.10392117500305176, 0.1036374568939209, 0.10408186912536621, 0.10350227355957031, 0.10410690307617188, 0.10392308235168457, 0.10427618026733398, 0.10338568687438965, 0.10376119613647461, 0.10377860069274902, 0.10388875007629395, 0.10419297218322754, 0.10449886322021484, 0.1040651798248291, 0.10431456565856934, 0.10432815551757812, 0.1040182113647461, 0.10414934158325195, 0.10384488105773926, 0.10432720184326172, 0.1041257381439209, 0.104522705078125, 0.10361933708190918, 0.10437870025634766, 0.10354995727539062, 0.10466170310974121, 0.10355210304260254, 0.10437798500061035, 0.10370874404907227, 0.10474014282226562, 0.1045384407043457, 0.10536503791809082, 0.1041102409362793, 0.10409903526306152, 0.10398006439208984, 0.1038045883178711, 0.10412096977233887, 0.1039280891418457, 0.10445642471313477, 0.1044461727142334, 0.10398244857788086, 0.10444426536560059, 0.1038210391998291, 0.10422348976135254, 0.10414910316467285, 0.1038980484008789, 0.10417008399963379, 0.1038970947265625, 0.10472249984741211, 0.10400652885437012, 0.10422849655151367, 0.10424542427062988, 0.10398030281066895, 0.10435914993286133, 0.10407352447509766, 0.10390138626098633, 0.10424971580505371, 0.10378813743591309, 0.10407805442810059, 0.10387086868286133, 0.1041872501373291, 0.10466146469116211, 0.10440516471862793, 0.10427308082580566, 0.1042029857635498, 0.1043238639831543, 0.1039578914642334, 0.10366559028625488, 0.10396838188171387, 0.10425400733947754, 0.10467171669006348, 0.10366249084472656, 0.10349559783935547, 0.1046152114868164, 0.10361266136169434, 0.1042172908782959, 0.10438323020935059, 0.10440397262573242, 0.10385918617248535, 0.10497641563415527, 0.10392427444458008, 0.1044614315032959, 0.10375285148620605, 0.1040961742401123, 0.10374975204467773, 0.10416436195373535, 0.10389852523803711, 0.1039130687713623, 0.10378742218017578, 0.10381340980529785, 0.10423970222473145, 0.10396480560302734, 0.1040945053100586, 0.10428142547607422, 0.10388731956481934, 0.10378742218017578, 0.10422086715698242, 0.1040811538696289, 0.10413384437561035, 0.1038200855255127, 0.1037743091583252, 0.10373568534851074, 0.10438156127929688, 0.10376119613647461, 0.10405898094177246, 0.10350561141967773, 0.10418939590454102, 0.10362696647644043, 0.10388445854187012, 0.10357260704040527, 0.10408616065979004, 0.10368132591247559, 0.10357332229614258, 0.10394716262817383, 0.103790283203125, 0.10367774963378906, 0.10423040390014648, 0.10354375839233398, 0.10397195816040039, 0.1034245491027832, 0.1036977767944336, 0.10399174690246582, 0.1034235954284668, 0.10421085357666016, 0.10392975807189941, 0.10524463653564453, 0.10367369651794434, 0.10537862777709961, 0.10378527641296387, 0.10392022132873535, 0.10427236557006836, 0.10490226745605469, 0.10602569580078125, 0.10442471504211426, 0.10384273529052734, 0.10420012474060059, 0.10418200492858887, 0.10477828979492188, 0.10386037826538086, 0.10398697853088379, 0.10394525527954102, 0.10389399528503418, 0.10392975807189941, 0.10397052764892578, 0.10458636283874512, 0.10389304161071777, 0.10463619232177734, 0.10408282279968262, 0.10382509231567383, 0.10376858711242676, 0.10391974449157715, 0.1037447452545166, 0.10428571701049805, 0.10426211357116699, 0.10387063026428223, 0.10383939743041992, 0.10394167900085449, 0.10386896133422852, 0.10472631454467773, 0.10380935668945312, 0.10436820983886719, 0.10482931137084961, 0.10415983200073242, 0.1037745475769043, 0.10418367385864258, 0.10406088829040527, 0.10428953170776367, 0.10407137870788574, 0.10398364067077637, 0.10471701622009277, 0.10393834114074707, 0.10400819778442383, 0.1040198802947998, 0.10503458976745605, 0.10419464111328125, 0.10545969009399414, 0.10425138473510742, 0.10452055931091309, 0.10366320610046387, 0.10393476486206055, 0.10352540016174316, 0.10434389114379883, 0.10330796241760254, 0.1041419506072998, 0.10366606712341309, 0.10398268699645996, 0.10383868217468262, 0.10374760627746582, 0.10388898849487305, 0.10378670692443848, 0.10408544540405273, 0.10392928123474121, 0.10367441177368164, 0.10398483276367188, 0.10393905639648438, 0.10422682762145996, 0.10396718978881836, 0.10383081436157227, 0.1039273738861084, 0.10341334342956543, 0.10465264320373535, 0.10399270057678223, 0.10385704040527344, 0.10335779190063477, 0.10410451889038086, 0.10386371612548828, 0.1036980152130127, 0.1039731502532959, 0.10379385948181152, 0.10388898849487305, 0.10408973693847656, 0.10357499122619629, 0.10386967658996582, 0.10355281829833984, 0.10378241539001465, 0.1036381721496582, 0.10401701927185059, 0.10426998138427734, 0.1039726734161377, 0.10408878326416016, 0.1034090518951416, 0.10513830184936523, 0.10414767265319824, 0.10462093353271484, 0.10408926010131836, 0.10411190986633301, 0.10378265380859375, 0.10399532318115234, 0.10437440872192383, 0.10405635833740234, 0.10411882400512695, 0.10423445701599121, 0.1034698486328125, 0.10416841506958008, 0.10378813743591309, 0.10386800765991211, 0.10403251647949219, 0.10402059555053711, 0.10367035865783691, 0.10355639457702637, 0.10421419143676758, 0.10410690307617188, 0.10386037826538086, 0.10453486442565918, 0.10387802124023438, 0.1040184497833252, 0.10424351692199707, 0.10388898849487305, 0.10378789901733398, 0.10385489463806152, 0.1046140193939209, 0.10381412506103516, 0.10393142700195312, 0.10418534278869629, 0.10430169105529785, 0.1039276123046875, 0.10437893867492676, 0.1040949821472168, 0.10422635078430176, 0.10367751121520996, 0.10395431518554688, 0.10368061065673828, 0.10412192344665527, 0.10442996025085449, 0.10400557518005371, 0.10422277450561523, 0.10424947738647461, 0.104034423828125, 0.10397958755493164, 0.10469961166381836, 0.10397577285766602, 0.1043848991394043, 0.10525274276733398, 0.1042325496673584, 0.1040496826171875, 0.10400676727294922, 0.10733819007873535, 0.10498881340026855, 0.10356521606445312, 0.10415196418762207, 0.10373258590698242, 0.1041412353515625, 0.10381031036376953, 0.1040792465209961, 0.10372042655944824, 0.1040034294128418, 0.10431098937988281, 0.10396838188171387, 0.10379195213317871, 0.1040642261505127, 0.1036980152130127, 0.11151313781738281, 0.11970353126525879, 0.10619926452636719, 0.10433650016784668, 0.10481524467468262, 0.10435843467712402, 0.10364174842834473, 0.10429835319519043, 0.1041412353515625, 0.10419154167175293, 0.1042633056640625, 0.10434913635253906, 0.10392284393310547, 0.10389065742492676, 0.10429859161376953, 0.10410666465759277, 0.1041264533996582, 0.10422992706298828, 0.10473060607910156, 0.10423946380615234, 0.10393428802490234, 0.10441946983337402, 0.1042013168334961, 0.10431766510009766, 0.10438776016235352, 0.10505390167236328, 0.1042015552520752, 0.10406064987182617, 0.10525703430175781, 0.1043391227722168, 0.10481500625610352, 0.10401225090026855, 0.1038510799407959, 0.10473203659057617, 0.10411262512207031, 0.10442590713500977, 0.10542106628417969, 0.10406017303466797, 0.10447430610656738, 0.10413432121276855, 0.10416173934936523, 0.1040198802947998, 0.10403203964233398, 0.10412049293518066, 0.10430502891540527, 0.10371613502502441, 0.10415315628051758, 0.10436701774597168, 0.10401225090026855, 0.10399794578552246, 0.10424137115478516, 0.1042337417602539, 0.10372281074523926, 0.10463070869445801, 0.10404157638549805, 0.10407257080078125, 0.10423707962036133, 0.10439896583557129, 0.10473084449768066, 0.10414552688598633, 0.10477471351623535, 0.10436463356018066, 0.10386085510253906, 0.10414791107177734, 0.1039724349975586, 0.10432291030883789, 0.10390210151672363, 0.10398244857788086, 0.1040349006652832, 0.10409331321716309, 0.10505938529968262, 0.10382556915283203, 0.10442471504211426, 0.10438251495361328, 0.10416579246520996, 0.10434150695800781, 0.10527348518371582, 0.10508966445922852, 0.1046907901763916, 0.10417962074279785, 0.10434579849243164, 0.10411572456359863, 0.10395288467407227, 0.10401749610900879, 0.10460901260375977, 0.10382676124572754, 0.10451579093933105, 0.10371732711791992, 0.10375809669494629, 0.10361003875732422, 0.10445094108581543, 0.10431790351867676, 0.10393762588500977, 0.10404658317565918, 0.10409688949584961, 0.10426807403564453, 0.10384416580200195, 0.10373902320861816, 0.10469508171081543, 0.1043248176574707, 0.10442137718200684, 0.10402846336364746, 0.10359048843383789, 0.1040496826171875, 0.10374832153320312, 0.10433268547058105, 0.1040797233581543, 0.10452771186828613, 0.10381126403808594, 0.10408711433410645, 0.10443496704101562, 0.10427546501159668, 0.10425281524658203, 0.10402250289916992, 0.1041727066040039, 0.10412001609802246, 0.10409188270568848, 0.1038978099822998, 0.10424423217773438, 0.1043701171875, 0.104217529296875, 0.10442209243774414, 0.1043236255645752, 0.10423755645751953, 0.10590338706970215, 0.10403323173522949, 0.10413599014282227, 0.10384845733642578, 0.107208251953125, 0.10454106330871582, 0.10448694229125977, 0.1041264533996582, 0.10449695587158203, 0.10408449172973633, 0.10445022583007812, 0.10406970977783203, 0.10405397415161133, 0.10407376289367676, 0.10494589805603027, 0.10555601119995117, 0.10562539100646973, 0.1052711009979248, 0.10563135147094727, 0.10429120063781738, 0.10430216789245605, 0.10391712188720703, 0.10404324531555176, 0.10474562644958496, 0.10416340827941895, 0.10440897941589355, 0.1038217544555664, 0.1041412353515625, 0.10399889945983887, 0.10454988479614258, 0.10501289367675781, 0.10446381568908691, 0.10473775863647461, 0.10439562797546387, 0.10440969467163086, 0.10372614860534668, 0.10437774658203125, 0.10413241386413574, 0.10411190986633301, 0.10407876968383789, 0.10369277000427246, 0.10435700416564941, 0.11494159698486328, 0.10416984558105469, 0.10467767715454102, 0.10494637489318848, 0.1046152114868164, 0.10406780242919922, 0.10561275482177734, 0.10476016998291016, 0.10445666313171387, 0.10413527488708496, 0.10453033447265625, 0.10442733764648438, 0.10602593421936035, 0.10548901557922363, 0.10454225540161133, 0.10391736030578613, 0.10448908805847168, 0.10416793823242188, 0.10492539405822754, 0.1039729118347168, 0.10406756401062012, 0.1041107177734375, 0.10439753532409668, 0.10431265830993652, 0.10387086868286133, 0.10443544387817383, 0.10389280319213867, 0.10422635078430176, 0.10405230522155762, 0.1039588451385498, 0.1041111946105957, 0.10455131530761719, 0.10419392585754395, 0.10403227806091309, 0.10387396812438965, 0.10448575019836426, 0.10404038429260254, 0.10410904884338379, 0.10537433624267578, 0.10548901557922363, 0.10521602630615234, 0.10496902465820312, 0.10401654243469238, 0.10387349128723145, 0.10431170463562012, 0.10378909111022949, 0.10393738746643066, 0.10386109352111816, 0.10365152359008789, 0.1041266918182373, 0.10403203964233398, 0.10573673248291016, 0.10434269905090332, 0.10397791862487793, 0.10503125190734863, 0.10383152961730957, 0.10409998893737793, 0.1038203239440918, 0.10516762733459473, 0.10473489761352539, 0.10433721542358398, 0.1036977767944336, 0.10398197174072266, 0.10406064987182617, 0.10375118255615234, 0.10423731803894043, 0.10531759262084961, 0.10404372215270996, 0.10508441925048828, 0.10405993461608887, 0.10394549369812012, 0.10376930236816406, 0.10512685775756836, 0.10439109802246094, 0.10448193550109863, 0.10389947891235352, 0.10390138626098633, 0.10380148887634277, 0.10383296012878418, 0.10428357124328613, 0.10430049896240234, 0.10413932800292969, 0.10381460189819336, 0.10435795783996582, 0.10431647300720215, 0.10393285751342773, 0.10468697547912598, 0.1039881706237793, 0.10433721542358398, 0.10404515266418457, 0.10368752479553223, 0.1039285659790039, 0.10366034507751465, 0.10399580001831055, 0.10423898696899414, 0.10473108291625977, 0.1040341854095459, 0.10410356521606445, 0.10380363464355469, 0.10453510284423828, 0.10442233085632324, 0.10353660583496094, 0.10574460029602051, 0.1042776107788086, 0.10391569137573242, 0.10405492782592773, 0.1052236557006836, 0.10446572303771973, 0.10397839546203613, 0.10409307479858398, 0.10360026359558105, 0.10377979278564453, 0.10415792465209961, 0.10384011268615723, 0.1042642593383789, 0.10373616218566895, 0.10393381118774414, 0.10426831245422363, 0.10547876358032227, 0.1038205623626709, 0.10516047477722168, 0.10554194450378418, 0.10402607917785645, 0.10438275337219238, 0.10401225090026855, 0.10435032844543457, 0.10385680198669434, 0.10384559631347656, 0.1040186882019043, 0.10423088073730469, 0.10398101806640625, 0.10436439514160156, 0.10404157638549805, 0.10395097732543945, 0.10437607765197754, 0.10413193702697754, 0.10432720184326172, 0.10396146774291992, 0.10504722595214844, 0.10445165634155273, 0.10395169258117676, 0.10389161109924316, 0.10405111312866211, 0.10444927215576172, 0.10389065742492676, 0.10448741912841797, 0.10388493537902832, 0.10438060760498047, 0.10408806800842285, 0.1042482852935791, 0.10559248924255371, 0.10450434684753418, 0.10395479202270508, 0.10391807556152344, 0.10389399528503418, 0.10413217544555664, 0.10420107841491699, 0.10396647453308105, 0.10428333282470703, 0.10398650169372559, 0.10398745536804199, 0.10413646697998047, 0.10403156280517578, 0.1041409969329834, 0.10387516021728516, 0.10389137268066406, 0.10397744178771973, 0.10385894775390625, 0.10397696495056152, 0.10416102409362793, 0.1045219898223877, 0.10519194602966309, 0.10420632362365723, 0.1039729118347168, 0.1042337417602539, 0.10392260551452637, 0.10374593734741211, 0.10394716262817383, 0.1036524772644043, 0.10467100143432617, 0.10409712791442871, 0.10391092300415039, 0.10650086402893066, 0.10440325736999512, 0.10479521751403809, 0.10429072380065918, 0.1035606861114502, 0.10354351997375488, 0.10379910469055176, 0.10422730445861816, 0.10375809669494629, 0.10478448867797852, 0.10385322570800781, 0.10452032089233398, 0.10387516021728516, 0.10424923896789551, 0.10415315628051758, 0.10384297370910645, 0.10579442977905273, 0.10524439811706543, 0.10428023338317871, 0.10366463661193848, 0.10380840301513672, 0.10436248779296875, 0.10399007797241211, 0.10418868064880371, 0.10409665107727051, 0.10384988784790039, 0.10381841659545898, 0.10388684272766113, 0.10413360595703125, 0.10424661636352539, 0.10438060760498047, 0.10384845733642578, 0.10387682914733887, 0.10378122329711914, 0.10388493537902832, 0.10435199737548828, 0.10403060913085938, 0.10381746292114258, 0.10429835319519043, 0.10428786277770996, 0.10394144058227539, 0.10359716415405273, 0.10404634475708008, 0.10381388664245605, 0.10419297218322754, 0.10407900810241699, 0.10413980484008789, 0.10390138626098633, 0.10394144058227539, 0.10376405715942383, 0.10439085960388184, 0.10434985160827637, 0.10393190383911133, 0.10461211204528809, 0.10378241539001465, 0.10356545448303223, 0.10380411148071289, 0.10391068458557129, 0.10393714904785156, 0.10397982597351074, 0.10431671142578125, 0.1045219898223877, 0.1047055721282959, 0.1038210391998291, 0.10522007942199707, 0.10404133796691895, 0.10422492027282715, 0.10503578186035156, 0.10687375068664551, 0.10480499267578125, 0.10421395301818848, 0.10402536392211914, 0.10424232482910156, 0.10394716262817383, 0.10437369346618652, 0.10360383987426758, 0.10404109954833984, 0.10381412506103516, 0.1040036678314209, 0.10442066192626953, 0.1040346622467041, 0.10393285751342773, 0.10374927520751953, 0.1041562557220459, 0.1038827896118164, 0.10380196571350098, 0.10433006286621094, 0.10437965393066406, 0.10438275337219238, 0.10377812385559082, 0.10394954681396484, 0.10378575325012207, 0.10398077964782715, 0.10467219352722168, 0.10445737838745117, 0.10443258285522461, 0.10456347465515137, 0.10466885566711426, 0.10413527488708496, 0.10403895378112793, 0.10417056083679199, 0.10378456115722656, 0.10410046577453613, 0.10368180274963379, 0.10389018058776855, 0.10965704917907715, 0.10395503044128418, 0.10428452491760254, 0.10373282432556152, 0.10451316833496094, 0.10382366180419922, 0.10404253005981445, 0.10572195053100586, 0.10451722145080566, 0.10361003875732422, 0.10409283638000488, 0.10422277450561523, 0.10390496253967285, 0.1040961742401123, 0.10437440872192383, 0.10396695137023926, 0.10425734519958496, 0.10404348373413086, 0.10412144660949707, 0.1042177677154541, 0.10391402244567871, 0.10419726371765137, 0.10405850410461426, 0.10429120063781738, 0.10422801971435547, 0.10402226448059082, 0.10468196868896484, 0.1039576530456543, 0.10405111312866211, 0.10399842262268066, 0.10400605201721191, 0.10448741912841797, 0.10382628440856934, 0.10412192344665527, 0.10365605354309082, 0.10423994064331055, 0.10383009910583496, 0.10441827774047852, 0.10374975204467773, 0.10410118103027344, 0.10386991500854492, 0.10408902168273926, 0.10410523414611816, 0.10353255271911621, 0.10413432121276855, 0.10407018661499023, 0.10429739952087402, 0.10399723052978516, 0.10460853576660156, 0.10359597206115723, 0.1039729118347168, 0.10391092300415039, 0.10468864440917969, 0.1041409969329834, 0.10367059707641602, 0.10537314414978027, 0.10412144660949707, 0.1037290096282959, 0.10436248779296875, 0.10370945930480957, 0.1049504280090332, 0.10371780395507812, 0.10455107688903809, 0.10387492179870605, 0.10368895530700684, 0.10384321212768555, 0.10368108749389648, 0.10401678085327148, 0.10381770133972168, 0.10386776924133301, 0.10383296012878418, 0.10367298126220703, 0.10382747650146484, 0.10376763343811035, 0.10445308685302734, 0.10399484634399414, 0.10407257080078125, 0.10401058197021484, 0.10371041297912598, 0.10427212715148926, 0.10372495651245117, 0.10405206680297852, 0.10396862030029297, 0.10397624969482422, 0.10405969619750977, 0.10416555404663086, 0.10380291938781738, 0.1041412353515625, 0.10421085357666016, 0.10441255569458008, 0.10383391380310059, 0.10399484634399414, 0.10393357276916504, 0.10398626327514648, 0.1041574478149414, 0.10426092147827148, 0.10453319549560547, 0.10392260551452637, 0.1038355827331543, 0.10398030281066895, 0.10447859764099121, 0.10515809059143066, 0.1045067310333252, 0.10547304153442383, 0.10430192947387695, 0.1037745475769043, 0.1041254997253418, 0.10658073425292969, 0.10440349578857422, 0.10399317741394043, 0.10385751724243164, 0.10388517379760742, 0.10408711433410645, 0.10355186462402344, 0.10414338111877441, 0.10361123085021973, 0.10386013984680176, 0.10391354560852051, 0.1040642261505127, 0.10397672653198242, 0.10391044616699219, 0.10418224334716797, 0.10434579849243164, 0.10428214073181152, 0.10383749008178711, 0.10411834716796875, 0.10457491874694824, 0.10417890548706055, 0.1041727066040039, 0.10431480407714844, 0.10424137115478516, 0.1038818359375, 0.10390472412109375, 0.1039128303527832, 0.10330986976623535, 0.10447120666503906, 0.10378718376159668, 0.10434913635253906, 0.10514211654663086, 0.10430455207824707, 0.1042168140411377, 0.10376310348510742, 0.1040189266204834, 0.1045079231262207, 0.10614228248596191, 0.1043088436126709, 0.10544013977050781, 0.10430312156677246, 0.10438418388366699, 0.10413718223571777, 0.10441446304321289, 0.10568356513977051, 0.10474586486816406, 0.10385799407958984, 0.10406994819641113, 0.10382962226867676, 0.10481810569763184, 0.1043863296508789, 0.10439610481262207, 0.10400772094726562, 0.10426068305969238, 0.10385251045227051, 0.10417461395263672, 0.10410213470458984, 0.10393738746643066, 0.10406994819641113, 0.10400772094726562, 0.10407543182373047, 0.10384416580200195, 0.10399079322814941, 0.10451006889343262, 0.10383081436157227, 0.1043550968170166, 0.10387134552001953, 0.10397982597351074, 0.10387539863586426, 0.10363531112670898, 0.1040496826171875, 0.10365009307861328, 0.10426187515258789, 0.10397720336914062, 0.10464000701904297, 0.1042635440826416, 0.10418343544006348, 0.10422682762145996, 0.10423731803894043, 0.1049191951751709, 0.10377287864685059, 0.10381412506103516, 0.10437679290771484, 0.10393071174621582, 0.10418224334716797, 0.10515618324279785, 0.10445308685302734, 0.10405302047729492, 0.10383915901184082, 0.10401344299316406, 0.10372710227966309, 0.10414719581604004, 0.10590791702270508, 0.10476422309875488, 0.10380315780639648, 0.10428619384765625, 0.10395097732543945, 0.10415148735046387, 0.10383796691894531, 0.10416603088378906, 0.10390949249267578, 0.10417580604553223, 0.10375809669494629, 0.10428500175476074, 0.10401606559753418, 0.10424232482910156, 0.10402107238769531, 0.10410571098327637, 0.10397720336914062, 0.10379576683044434, 0.10377192497253418, 0.10464310646057129, 0.1041107177734375, 0.10364842414855957, 0.10419797897338867, 0.10377073287963867, 0.10419535636901855, 0.1041250228881836, 0.10396385192871094, 0.10428905487060547, 0.10416650772094727, 0.10399174690246582, 0.10399770736694336, 0.10380053520202637, 0.10397768020629883, 0.1036078929901123, 0.10433268547058105, 0.10386252403259277, 0.10411524772644043, 0.10402703285217285, 0.10428357124328613, 0.10361289978027344, 0.10399889945983887, 0.10393214225769043, 0.10373091697692871, 0.10400724411010742, 0.10408782958984375, 0.10414409637451172, 0.10441708564758301, 0.10425162315368652, 0.10536742210388184, 0.10377144813537598, 0.10427522659301758, 0.10436415672302246, 0.10524272918701172, 0.10516071319580078, 0.10408759117126465, 0.1044766902923584, 0.10425424575805664, 0.10441994667053223, 0.10390567779541016, 0.10395526885986328, 0.10364437103271484, 0.10398554801940918, 0.10395669937133789, 0.10405755043029785, 0.10414552688598633, 0.10403943061828613, 0.10418272018432617, 0.10419273376464844, 0.10387825965881348, 0.10415005683898926, 0.10384774208068848, 0.11162567138671875, 0.10453152656555176, 0.10509300231933594, 0.10483860969543457, 0.10426712036132812, 0.10469937324523926, 0.10499906539916992, 0.10474967956542969, 0.10407829284667969, 0.10405325889587402, 0.10413455963134766, 0.10405874252319336, 0.10437893867492676, 0.10423946380615234, 0.10410952568054199, 0.10398554801940918, 0.10391426086425781, 0.10392975807189941, 0.11918020248413086, 0.10459780693054199, 0.10448575019836426, 0.10540056228637695, 0.10463690757751465, 0.10395359992980957, 0.10428905487060547, 0.10544490814208984, 0.10479474067687988, 0.1043386459350586, 0.1046133041381836, 0.10383772850036621, 0.1043252944946289, 0.10372138023376465, 0.10413122177124023, 0.10406947135925293, 0.10431027412414551, 0.1041569709777832, 0.1045842170715332, 0.1043388843536377, 0.10398674011230469, 0.10438680648803711, 0.104034423828125, 0.10419273376464844, 0.10420870780944824, 0.10357427597045898, 0.1043996810913086, 0.10404086112976074, 0.10364055633544922, 0.10426068305969238, 0.10385370254516602, 0.10437607765197754, 0.10364103317260742, 0.10397577285766602, 0.1034543514251709, 0.10395431518554688, 0.10389256477355957, 0.10397696495056152, 0.10394072532653809, 0.1039881706237793, 0.10374665260314941, 0.10402154922485352, 0.10379457473754883, 0.1042032241821289, 0.10390567779541016, 0.10450983047485352, 0.10385608673095703, 0.10383725166320801, 0.10412478446960449, 0.10381340980529785, 0.10444378852844238, 0.1040046215057373, 0.10472869873046875, 0.1039276123046875, 0.10424447059631348, 0.10505199432373047, 0.1040201187133789, 0.1039876937866211, 0.10422444343566895, 0.10437726974487305, 0.10423064231872559, 0.10381102561950684, 0.1040501594543457, 0.10385727882385254, 0.10457491874694824, 0.10429787635803223, 0.10431981086730957, 0.10396265983581543, 0.1041712760925293, 0.10399222373962402, 0.10388803482055664, 0.10416126251220703, 0.10386371612548828, 0.10419678688049316, 0.1047353744506836, 0.10400533676147461, 0.10366964340209961, 0.1045076847076416, 0.10419940948486328, 0.10433030128479004, 0.10533618927001953, 0.10416674613952637, 0.10342288017272949, 0.10438275337219238, 0.10420417785644531, 0.10455608367919922, 0.1039133071899414, 0.10439848899841309, 0.10458660125732422, 0.10451102256774902, 0.10377311706542969, 0.10401630401611328, 0.10399222373962402, 0.10429883003234863, 0.10410642623901367, 0.10377287864685059, 0.10408926010131836, 0.10390949249267578, 0.10407090187072754, 0.1039283275604248, 0.10499167442321777, 0.10403633117675781, 0.1038045883178711, 0.1057891845703125, 0.10473966598510742, 0.10387063026428223, 0.10418868064880371, 0.10647416114807129, 0.10518097877502441, 0.10386204719543457, 0.10397911071777344, 0.10377287864685059, 0.10445523262023926, 0.1049797534942627, 0.1049187183380127, 0.10408473014831543, 0.10406208038330078, 0.103790283203125, 0.10422205924987793, 0.10389399528503418, 0.10416483879089355, 0.10385322570800781, 0.10425853729248047, 0.10555696487426758, 0.10512065887451172, 0.10395598411560059, 0.10546326637268066, 0.10571765899658203, 0.10393285751342773, 0.10553145408630371, 0.10367822647094727, 0.105499267578125, 0.10423064231872559, 0.10440683364868164, 0.10440301895141602, 0.10387563705444336, 0.10411405563354492, 0.1040194034576416, 0.10393548011779785, 0.10506463050842285, 0.10399985313415527, 0.10430717468261719, 0.10372018814086914, 0.10437488555908203, 0.10411763191223145, 0.10369396209716797, 0.10420012474060059, 0.1036679744720459, 0.10444784164428711, 0.10390138626098633, 0.10442686080932617, 0.10454583168029785, 0.10401725769042969, 0.10416531562805176, 0.1037285327911377, 0.10442447662353516, 0.10426092147827148, 0.10472488403320312, 0.10413813591003418, 0.10378098487854004, 0.1039116382598877, 0.10382533073425293, 0.10431408882141113, 0.1051943302154541, 0.1041402816772461, 0.10505366325378418, 0.10375809669494629, 0.10408520698547363, 0.10384511947631836, 0.10409426689147949, 0.10433840751647949, 0.10448026657104492, 0.10342097282409668, 0.10411262512207031, 0.1037287712097168, 0.10426449775695801, 0.10401344299316406, 0.10392451286315918, 0.10376787185668945, 0.10518383979797363, 0.10544586181640625, 0.1048882007598877, 0.10445952415466309, 0.10483026504516602, 0.1052708625793457, 0.10421919822692871, 0.10410928726196289, 0.1040797233581543, 0.10382795333862305, 0.10417509078979492, 0.10381174087524414, 0.10408949851989746, 0.10480165481567383, 0.10388541221618652, 0.10413455963134766, 0.10388898849487305, 0.10403323173522949, 0.10396933555603027, 0.10457420349121094, 0.10500907897949219, 0.10441374778747559, 0.10430169105529785, 0.1040337085723877, 0.10384368896484375, 0.10442042350769043, 0.10516023635864258, 0.10429072380065918, 0.10401678085327148, 0.10381031036376953, 0.10399699211120605, 0.10402345657348633, 0.104644775390625, 0.10402727127075195, 0.10427618026733398, 0.10385799407958984, 0.10417366027832031, 0.10367584228515625, 0.10380363464355469, 0.10450983047485352, 0.10409808158874512, 0.10460925102233887, 0.10429072380065918, 0.10373663902282715, 0.10443592071533203, 0.10384535789489746, 0.1042470932006836, 0.1037437915802002, 0.10429906845092773, 0.10354256629943848, 0.10404157638549805, 0.10368061065673828, 0.10389900207519531, 0.10389971733093262, 0.10422754287719727, 0.1039278507232666, 0.1039266586303711, 0.1039266586303711, 0.10405182838439941, 0.1035618782043457, 0.10436415672302246, 0.10442066192626953, 0.10404062271118164, 0.1041877269744873, 0.10385751724243164, 0.10483407974243164, 0.10378718376159668, 0.10482335090637207, 0.10501933097839355, 0.10430717468261719, 0.1045527458190918, 0.10483002662658691, 0.10503292083740234, 0.10476398468017578, 0.1042320728302002, 0.10425591468811035, 0.10405659675598145, 0.10450363159179688, 0.10380768775939941, 0.10411834716796875, 0.1038053035736084, 0.10423707962036133, 0.10416746139526367, 0.10437870025634766, 0.1043853759765625, 0.10425472259521484, 0.10423636436462402, 0.10429263114929199, 0.10440397262573242, 0.105499267578125, 0.10454106330871582, 0.10556817054748535, 0.10778164863586426, 0.10796856880187988, 0.10550069808959961, 0.10410928726196289, 0.10449790954589844, 0.10485100746154785, 0.10409736633300781, 0.1056222915649414, 0.10622930526733398, 0.10548591613769531, 0.10581111907958984, 0.10382604598999023, 0.10437297821044922, 0.10372471809387207, 0.10397839546203613, 0.10382795333862305, 0.10375356674194336, 0.10432052612304688, 0.10370969772338867, 0.1042325496673584, 0.10396432876586914, 0.10488605499267578, 0.10404729843139648, 0.10545134544372559, 0.10434269905090332, 0.10475778579711914, 0.10444974899291992, 0.10415816307067871, 0.10385942459106445, 0.10348033905029297, 0.10284829139709473, 0.10335564613342285, 0.1028134822845459, 0.10393452644348145, 0.10273098945617676, 0.10338807106018066, 0.10383462905883789, 0.10326981544494629, 0.10367035865783691, 0.10308837890625, 0.10335659980773926, 0.1028449535369873, 0.1032567024230957, 0.10376143455505371, 0.10310220718383789, 0.10333609580993652, 0.10338735580444336, 0.10331416130065918, 0.10416984558105469, 0.10294103622436523, 0.10323357582092285, 0.10269880294799805, 0.10469365119934082, 0.10281562805175781, 0.10390710830688477, 0.10302424430847168, 0.10327935218811035, 0.1031484603881836, 0.10352468490600586, 0.10315561294555664, 0.10331320762634277, 0.10324573516845703, 0.10347604751586914, 0.10274910926818848, 0.10338115692138672, 0.10360836982727051, 0.10328269004821777, 0.10355424880981445, 0.10327029228210449, 0.10367989540100098, 0.10314512252807617, 0.10321235656738281, 0.10477924346923828, 0.10373806953430176, 0.1030738353729248, 0.10337018966674805, 0.10372614860534668, 0.1035153865814209, 0.10305213928222656, 0.10335183143615723, 0.10331869125366211, 0.1031339168548584, 0.10355639457702637, 0.10354757308959961, 0.10330390930175781, 0.1034235954284668, 0.10277676582336426, 0.10345649719238281, 0.1031339168548584, 0.10333967208862305, 0.10289788246154785, 0.10377883911132812, 0.10303115844726562, 0.10374593734741211, 0.10374283790588379, 0.10355401039123535, 0.10344791412353516, 0.10297083854675293, 0.10303354263305664, 0.10310912132263184, 0.10298418998718262, 0.103668212890625, 0.10314226150512695, 0.10333991050720215, 0.10324740409851074, 0.10340023040771484, 0.10370230674743652, 0.1031494140625, 0.10341882705688477, 0.10333418846130371, 0.10315370559692383, 0.1031031608581543, 0.10332441329956055, 0.10341167449951172, 0.10336470603942871, 0.10340666770935059, 0.10365700721740723, 0.103515625, 0.10323190689086914, 0.10307550430297852, 0.10467338562011719, 0.1032407283782959, 0.1036524772644043, 0.10299348831176758, 0.1055459976196289, 0.10372543334960938, 0.10339188575744629, 0.10319900512695312, 0.10304975509643555, 0.10356307029724121, 0.10420656204223633, 0.1031033992767334, 0.10337471961975098, 0.10318207740783691, 0.10335516929626465, 0.1031494140625, 0.10321831703186035, 0.1033623218536377, 0.10336542129516602, 0.10358405113220215, 0.10302114486694336, 0.10325741767883301, 0.10321640968322754, 0.10440373420715332, 0.1029963493347168, 0.10402536392211914, 0.10281491279602051, 0.10479140281677246, 0.10396671295166016, 0.10329461097717285, 0.10317659378051758, 0.10334539413452148, 0.1037755012512207, 0.10324239730834961, 0.10342144966125488, 0.10332298278808594, 0.10290646553039551, 0.10384821891784668, 0.10329937934875488, 0.10337615013122559, 0.10342288017272949, 0.10348105430603027, 0.10321283340454102, 0.10331511497497559, 0.10345172882080078, 0.10308551788330078, 0.10484743118286133, 0.1031181812286377, 0.10510635375976562, 0.10308337211608887, 0.10412025451660156, 0.10726451873779297, 0.10700631141662598, 0.10739326477050781, 0.10712575912475586, 0.1073451042175293, 0.10700201988220215, 0.10718417167663574, 0.10790562629699707, 0.10670304298400879, 0.10708427429199219, 0.10646748542785645, 0.10719466209411621, 0.10713934898376465, 0.10682368278503418, 0.10717129707336426, 0.10672831535339355, 0.10754251480102539, 0.10684370994567871, 0.1072998046875, 0.10671758651733398, 0.10793519020080566, 0.10842657089233398, 0.1077883243560791, 0.10758757591247559, 0.10681343078613281, 0.10703921318054199, 0.10716629028320312, 0.10782361030578613, 0.10737848281860352, 0.10690617561340332, 0.10668301582336426, 0.10831356048583984, 0.10772299766540527, 0.10646653175354004, 0.1064002513885498, 0.10683965682983398, 0.1069021224975586, 0.10850143432617188, 0.1068878173828125, 0.10677576065063477, 0.10798025131225586, 0.10724520683288574, 0.10705089569091797, 0.10672354698181152, 0.10818052291870117, 0.10705161094665527, 0.10686635971069336, 0.10737800598144531, 0.1067955493927002, 0.10751533508300781, 0.10724806785583496, 0.10727667808532715, 0.1068582534790039, 0.1068277359008789, 0.10730361938476562, 0.10707974433898926, 0.10737895965576172, 0.1074674129486084, 0.10681867599487305, 0.10679244995117188, 0.1071469783782959, 0.10722470283508301, 0.10703706741333008, 0.10737800598144531, 0.10675215721130371, 0.10699748992919922, 0.10678648948669434, 0.10684871673583984, 0.106964111328125, 0.10673904418945312, 0.10717391967773438, 0.10672116279602051, 0.1067190170288086, 0.10689282417297363, 0.10706853866577148, 0.10695171356201172, 0.10685324668884277, 0.10733342170715332, 0.10671734809875488, 0.10683274269104004, 0.10646724700927734, 0.10635685920715332, 0.10688018798828125, 0.10671758651733398, 0.1070866584777832, 0.10674929618835449, 0.10685205459594727, 0.10684847831726074, 0.10853123664855957, 0.10699629783630371, 0.10660934448242188, 0.1078653335571289, 0.10677504539489746, 0.10768342018127441, 0.10661077499389648, 0.10793137550354004, 0.10783219337463379, 0.10710978507995605, 0.10746645927429199, 0.10671710968017578, 0.1072075366973877, 0.10694456100463867, 0.10761547088623047, 0.10755777359008789, 0.1066887378692627, 0.10702633857727051, 0.10697054862976074, 0.10779500007629395, 0.10689568519592285, 0.10746645927429199, 0.10701322555541992, 0.10742425918579102, 0.10726714134216309, 0.10700440406799316, 0.10684990882873535, 0.10709118843078613, 0.1068425178527832, 0.10650777816772461, 0.10651922225952148, 0.1072227954864502, 0.10727858543395996, 0.1075127124786377, 0.10693836212158203, 0.10721826553344727, 0.10683608055114746, 0.10665297508239746, 0.10729146003723145, 0.10656952857971191, 0.10737848281860352, 0.1064915657043457, 0.10709595680236816, 0.1123647689819336, 0.10791563987731934, 0.10793519020080566, 0.10788440704345703, 0.10847616195678711, 0.10785436630249023, 0.10842180252075195, 0.10827159881591797, 0.1083829402923584, 0.10773587226867676, 0.10811710357666016, 0.10789632797241211, 0.10811877250671387, 0.10765552520751953, 0.10944843292236328, 0.10764861106872559, 0.10833954811096191, 0.10759210586547852, 0.10940432548522949, 0.1079871654510498, 0.10749936103820801, 0.1076195240020752, 0.10793590545654297, 0.10764861106872559, 0.10737109184265137, 0.1100006103515625, 0.10954976081848145, 0.10814428329467773, 0.10757040977478027, 0.10871529579162598, 0.10909581184387207, 0.10785722732543945, 0.10790896415710449, 0.10710692405700684, 0.10748004913330078, 0.10777878761291504, 0.10792350769042969, 0.10798263549804688, 0.10773825645446777, 0.10769867897033691, 0.10750174522399902, 0.10791850090026855, 0.10803365707397461, 0.1079864501953125, 0.10818815231323242, 0.10946512222290039, 0.1098017692565918, 0.1090245246887207, 0.10768651962280273, 0.1075904369354248, 0.10863995552062988, 0.10770916938781738, 0.10772180557250977, 0.10922455787658691, 0.10758209228515625, 0.1085057258605957, 0.10776448249816895, 0.10808515548706055, 0.10776543617248535, 0.1077418327331543, 0.10838007926940918, 0.10804200172424316, 0.10820817947387695, 0.10823845863342285, 0.10760331153869629, 0.10767531394958496, 0.1077880859375, 0.10822510719299316, 0.10788798332214355, 0.1078946590423584, 0.10785675048828125, 0.10798025131225586, 0.1078791618347168, 0.10796499252319336, 0.10883569717407227, 0.1077277660369873, 0.10769414901733398, 0.1076815128326416, 0.10793757438659668, 0.10803914070129395, 0.10816407203674316, 0.10846400260925293, 0.10803413391113281, 0.10845303535461426, 0.10829401016235352, 0.10764813423156738, 0.10785746574401855, 0.10753750801086426, 0.10772585868835449, 0.10758304595947266, 0.10805940628051758, 0.10770416259765625, 0.10823655128479004, 0.10772395133972168, 0.1083524227142334, 0.1144711971282959, 0.10877704620361328, 0.10813498497009277, 0.10742878913879395, 0.10807394981384277, 0.10976386070251465, 0.10790133476257324, 0.10785174369812012, 0.11054420471191406, 0.10863637924194336, 0.10807609558105469, 0.10757851600646973, 0.10741496086120605, 0.10765242576599121, 0.10939598083496094, 0.10948705673217773, 0.10782051086425781, 0.1074972152709961, 0.10866475105285645, 0.10765528678894043, 0.10825753211975098, 0.10774397850036621, 0.1076352596282959, 0.10826873779296875, 0.1075587272644043, 0.10775351524353027, 0.10817289352416992, 0.10804414749145508, 0.10818815231323242, 0.10791182518005371, 0.10761761665344238, 0.10839962959289551, 0.1076669692993164, 0.1080923080444336, 0.10882997512817383, 0.10819196701049805, 0.10781145095825195, 0.10763740539550781, 0.10783028602600098, 0.10786938667297363, 0.10825943946838379, 0.10802125930786133, 0.10808801651000977, 0.10795807838439941, 0.10756897926330566, 0.10802412033081055, 0.10886287689208984, 0.10802507400512695, 0.10809612274169922, 0.10802936553955078, 0.10850191116333008, 0.10786843299865723, 0.10827183723449707, 0.1089484691619873, 0.10782480239868164, 0.10774874687194824, 0.10862326622009277, 0.11005425453186035, 0.10898303985595703, 0.10788774490356445, 0.10734224319458008, 0.10812783241271973, 0.10803723335266113, 0.1080780029296875, 0.10777807235717773, 0.10753703117370605, 0.10847878456115723, 0.10700726509094238, 0.10820460319519043, 0.10818314552307129, 0.10832333564758301, 0.10765600204467773, 0.10764765739440918, 0.1082010269165039, 0.10795950889587402, 0.10847806930541992, 0.10830497741699219, 0.10820364952087402, 0.10831809043884277, 0.10869264602661133, 0.1082611083984375, 0.10944485664367676, 0.11020231246948242, 0.10827183723449707, 0.10829377174377441, 0.1080331802368164, 0.10826802253723145, 0.10817146301269531, 0.10786938667297363, 0.10801005363464355, 0.10944819450378418, 0.1082909107208252, 0.10755300521850586, 0.1075131893157959, 0.10847234725952148, 0.10746121406555176, 0.10883378982543945, 0.10811758041381836, 0.10823225975036621, 0.10811853408813477, 0.10808992385864258, 0.10837101936340332, 0.10780143737792969, 0.10844111442565918, 0.10758447647094727, 0.10821747779846191, 0.1081228256225586, 0.1076500415802002, 0.10829734802246094, 0.1084282398223877, 0.107818603515625, 0.10816740989685059, 0.10860347747802734, 0.10766339302062988, 0.1080179214477539, 0.1084299087524414, 0.10908746719360352, 0.10788297653198242, 0.1076347827911377, 0.10833215713500977, 0.10831761360168457, 0.10779285430908203, 0.10778260231018066, 0.10747408866882324, 0.10805225372314453, 0.10786986351013184, 0.10804128646850586, 0.10788440704345703, 0.10805988311767578, 0.10770988464355469, 0.10766172409057617, 0.10752344131469727, 0.10777401924133301, 0.10820960998535156, 0.10744047164916992, 0.10779929161071777, 0.10797333717346191, 0.10733866691589355, 0.10787796974182129, 0.10778617858886719, 0.1098928451538086, 0.10756278038024902, 0.10780119895935059, 0.10866236686706543, 0.10795116424560547, 0.10918784141540527, 0.1091911792755127, 0.10892152786254883, 0.10832524299621582, 0.10795354843139648, 0.10760927200317383, 0.1079859733581543, 0.10788869857788086, 0.10829830169677734, 0.10799551010131836, 0.10798048973083496, 0.10784220695495605, 0.10783004760742188, 0.10810184478759766, 0.10815978050231934, 0.10916829109191895, 0.10846829414367676, 0.10812497138977051, 0.1082756519317627, 0.10796165466308594, 0.10777902603149414, 0.1082010269165039, 0.10794782638549805, 0.10813403129577637, 0.10748839378356934, 0.1080942153930664, 0.10864925384521484, 0.10790109634399414, 0.10822200775146484, 0.10830926895141602, 0.10871052742004395, 0.10785102844238281, 0.10821056365966797, 0.10802435874938965, 0.10782384872436523, 0.10842275619506836, 0.10814094543457031, 0.1087343692779541, 0.1075742244720459, 0.10796952247619629, 0.10808396339416504, 0.10941100120544434, 0.10804462432861328, 0.10819077491760254, 0.10920858383178711, 0.10858273506164551, 0.10807466506958008, 0.1080780029296875, 0.10785293579101562, 0.10808968544006348, 0.10781240463256836, 0.107940673828125, 0.10759377479553223, 0.10886073112487793, 0.10815238952636719, 0.10777783393859863, 0.1095571517944336, 0.10830235481262207, 0.10842490196228027, 0.10768604278564453, 0.10786938667297363, 0.10756349563598633, 0.10851430892944336, 0.10825634002685547, 0.10827016830444336, 0.10839438438415527, 0.10838794708251953, 0.10802006721496582, 0.10950040817260742, 0.1093740463256836, 0.10886192321777344, 0.10806655883789062, 0.10834527015686035, 0.10797429084777832, 0.10836577415466309, 0.10759949684143066, 0.10794591903686523, 0.10785937309265137, 0.10915255546569824, 0.10769391059875488, 0.10924530029296875, 0.10834622383117676, 0.10934042930603027, 0.10973882675170898, 0.10948777198791504, 0.10744166374206543, 0.10790300369262695, 0.1085512638092041, 0.10808134078979492, 0.10809850692749023, 0.10941743850708008, 0.10839581489562988, 0.10852742195129395, 0.10825991630554199, 0.10814523696899414, 0.10860753059387207, 0.10806655883789062, 0.10819458961486816, 0.10903644561767578, 0.10826230049133301, 0.1089012622833252, 0.1079721450805664, 0.10857605934143066, 0.10812807083129883, 0.10824155807495117, 0.10809755325317383, 0.10878252983093262, 0.10821771621704102, 0.1082925796508789, 0.10820269584655762, 0.10856175422668457, 0.10886573791503906, 0.10864472389221191, 0.10857439041137695, 0.10848832130432129, 0.10799145698547363, 0.10814547538757324, 0.10834455490112305, 0.10860443115234375, 0.10933518409729004, 0.10852646827697754, 0.10852479934692383, 0.10906767845153809, 0.10808897018432617, 0.10841012001037598, 0.10826992988586426, 0.10906815528869629, 0.10794711112976074, 0.10839676856994629, 0.10878849029541016, 0.10855555534362793, 0.10785222053527832, 0.10854220390319824, 0.10903120040893555, 0.10834717750549316, 0.10825371742248535, 0.10975074768066406, 0.10837030410766602, 0.10820674896240234, 0.10809874534606934, 0.11056041717529297, 0.10898661613464355, 0.1086113452911377, 0.10808825492858887, 0.10814905166625977, 0.1080026626586914, 0.10822510719299316, 0.10847020149230957, 0.10810995101928711, 0.10827231407165527, 0.10818791389465332, 0.10828590393066406, 0.10785889625549316, 0.10802912712097168, 0.10837388038635254, 0.1084752082824707, 0.10877084732055664, 0.10818290710449219, 0.1089625358581543, 0.10833358764648438, 0.10815834999084473, 0.10897564888000488, 0.10802721977233887, 0.1081397533416748, 0.10854840278625488, 0.1090691089630127, 0.11002182960510254, 0.10894942283630371, 0.10880112648010254, 0.1091618537902832, 0.10941576957702637, 0.10927057266235352, 0.10996794700622559, 0.10858750343322754, 0.10907268524169922, 0.10954952239990234, 0.10965251922607422, 0.10968279838562012, 0.10963153839111328, 0.1107492446899414, 0.10982108116149902, 0.10975408554077148, 0.11041140556335449, 0.131119966506958, 0.17043209075927734, 0.17026758193969727, 0.1705484390258789, 0.17003989219665527, 0.16983652114868164, 0.17023801803588867, 0.17029213905334473, 0.16988015174865723, 0.17038202285766602, 0.16991829872131348, 0.1700422763824463, 0.1701219081878662, 0.1704113483428955, 0.16958236694335938, 0.17030000686645508, 0.16988658905029297, 0.17028021812438965, 0.17042899131774902, 0.1702122688293457, 0.17049026489257812, 0.16984319686889648, 0.17055583000183105, 0.16995668411254883, 0.17058253288269043, 0.17049527168273926, 0.1701335906982422, 0.1716938018798828, 0.17009282112121582, 0.1713263988494873, 0.17014122009277344, 0.17094159126281738, 0.170637845993042, 0.1698596477508545, 0.1702709197998047, 0.17025351524353027, 0.17025423049926758, 0.16963410377502441, 0.1704421043395996, 0.17031002044677734, 0.17028570175170898, 0.170515775680542, 0.17021512985229492, 0.1698472499847412, 0.17001008987426758, 0.1701037883758545, 0.17038655281066895, 0.16994619369506836, 0.17067813873291016, 0.17062163352966309, 0.17043733596801758, 0.17034125328063965, 0.17086338996887207, 0.16990137100219727, 0.17004990577697754, 0.17065811157226562, 0.1720726490020752, 0.17022299766540527, 0.17068147659301758, 0.17027997970581055, 0.1732785701751709, 0.1723499298095703, 0.17062973976135254, 0.17013287544250488, 0.1705002784729004, 0.16967439651489258, 0.17036724090576172, 0.17023515701293945, 0.16995787620544434, 0.1700429916381836, 0.17066407203674316, 0.170335054397583, 0.16988325119018555, 0.16988778114318848, 0.16992664337158203, 0.16980314254760742, 0.1708815097808838, 0.17065215110778809, 0.1707754135131836, 0.17040157318115234, 0.17087483406066895, 0.1704251766204834, 0.17031168937683105, 0.18225526809692383, 0.17085981369018555, 0.17087841033935547, 0.1712193489074707, 0.1705799102783203, 0.17162108421325684, 0.17013120651245117, 0.17041325569152832, 0.17000746726989746, 0.17061471939086914, 0.17021942138671875, 0.17000508308410645, 0.17123174667358398, 0.1698143482208252, 0.17051458358764648, 0.16986608505249023, 0.17065191268920898, 0.16997241973876953, 0.16991853713989258, 0.1701493263244629, 0.17007231712341309, 0.1705636978149414, 0.17090058326721191, 0.17120647430419922, 0.1696012020111084, 0.16973185539245605, 0.16910529136657715, 0.1692028045654297, 0.16905450820922852, 0.16993999481201172, 0.16990017890930176, 0.16905903816223145, 0.17079401016235352, 0.1698770523071289, 0.17040610313415527, 0.1700732707977295, 0.1684257984161377, 0.17026925086975098, 0.16882896423339844, 0.16895580291748047, 0.17317605018615723, 0.17413759231567383, 0.21390032768249512, 0.2042832374572754, 0.10796046257019043, 0.10815143585205078, 0.10990738868713379, 0.10913586616516113, 0.11069107055664062, 0.10975074768066406, 0.10817337036132812, 0.10924434661865234, 0.10948777198791504, 0.10911893844604492, 0.10772562026977539, 0.10841655731201172, 0.10830259323120117, 0.10792779922485352, 0.1089024543762207, 0.10854005813598633, 0.10851740837097168, 0.10767483711242676, 0.10801386833190918, 0.10803532600402832, 0.1082754135131836, 0.10801029205322266, 0.10790753364562988, 0.10853266716003418, 0.10821342468261719, 0.10814809799194336, 0.10873079299926758, 0.10933876037597656, 0.10821723937988281, 0.10880923271179199, 0.10868382453918457, 0.10861849784851074, 0.10765242576599121, 0.11057925224304199, 0.10867881774902344, 0.10944008827209473, 0.1081395149230957, 0.10819578170776367, 0.10811948776245117, 0.10808801651000977, 0.10859322547912598, 0.10839319229125977, 0.10836029052734375, 0.10811257362365723, 0.10845398902893066, 0.10763120651245117, 0.10788655281066895, 0.10802173614501953, 0.10820198059082031, 0.1080322265625, 0.10804963111877441, 0.10810708999633789, 0.10828948020935059, 0.10829591751098633, 0.10815000534057617, 0.10867857933044434, 0.10814809799194336, 0.10822701454162598, 0.10793662071228027, 0.10821652412414551, 0.10943794250488281, 0.10818672180175781, 0.10813212394714355, 0.10821318626403809, 0.10885310173034668, 0.10858559608459473, 0.10960102081298828, 0.10810208320617676, 0.10862994194030762, 0.10801482200622559, 0.10872030258178711, 0.10785818099975586, 0.10953402519226074, 0.108917236328125, 0.10819768905639648, 0.10842633247375488, 0.1086733341217041, 0.10872650146484375, 0.10825443267822266, 0.10857272148132324, 0.10840487480163574, 0.10825848579406738, 0.10907959938049316, 0.10830187797546387, 0.10837602615356445, 0.10865139961242676, 0.10844779014587402, 0.10809922218322754, 0.10842537879943848, 0.10779738426208496, 0.10812497138977051, 0.10875296592712402, 0.10820126533508301, 0.10762500762939453, 0.10834217071533203, 0.10800671577453613, 0.10835862159729004, 0.10849165916442871, 0.10811710357666016, 0.10823822021484375, 0.10757708549499512, 0.10830879211425781, 0.10799479484558105, 0.10840344429016113, 0.10919499397277832, 0.11040830612182617, 0.10842227935791016, 0.10907435417175293, 0.10799455642700195, 0.10824990272521973, 0.1087498664855957, 0.1080477237701416, 0.10901308059692383, 0.10797739028930664, 0.10950994491577148, 0.10903239250183105, 0.10914158821105957, 0.10836291313171387, 0.10826897621154785, 0.10895681381225586, 0.10828638076782227, 0.1080634593963623, 0.10923385620117188, 0.1084451675415039, 0.10831332206726074, 0.10871720314025879, 0.1084139347076416, 0.10845255851745605, 0.10799098014831543, 0.10803580284118652, 0.10823559761047363, 0.1085050106048584, 0.10832595825195312, 0.10861921310424805, 0.10883092880249023, 0.10773849487304688, 0.10840797424316406, 0.1079566478729248, 0.10882067680358887, 0.10791206359863281, 0.1085197925567627, 0.10801219940185547, 0.10847306251525879, 0.10847997665405273, 0.1079566478729248, 0.1083829402923584, 0.10809040069580078, 0.10868167877197266, 0.10807991027832031, 0.1085500717163086, 0.10785770416259766, 0.10834217071533203, 0.10832929611206055, 0.10808157920837402, 0.1081380844116211, 0.10827875137329102, 0.10805225372314453, 0.10804319381713867, 0.10817074775695801, 0.10819435119628906, 0.10848426818847656, 0.10829973220825195, 0.10828995704650879, 0.10823225975036621, 0.10936164855957031, 0.10868453979492188, 0.10869884490966797, 0.10827374458312988, 0.10925507545471191, 0.10819292068481445, 0.1096951961517334, 0.10822916030883789, 0.11039614677429199, 0.10856223106384277, 0.10777831077575684, 0.10843539237976074, 0.10903573036193848, 0.10798811912536621, 0.10809683799743652, 0.10825777053833008, 0.1081852912902832, 0.10802340507507324, 0.10831665992736816, 0.10789108276367188, 0.10820317268371582, 0.10792040824890137, 0.10859107971191406, 0.10857462882995605, 0.10823369026184082, 0.10804414749145508, 0.10810279846191406, 0.1078195571899414, 0.10808086395263672, 0.10837936401367188, 0.10809755325317383, 0.10766863822937012, 0.10921525955200195, 0.10865306854248047, 0.10802245140075684, 0.10826945304870605, 0.1088266372680664, 0.10816287994384766, 0.10786199569702148, 0.10808587074279785, 0.1082155704498291, 0.10789918899536133, 0.10792136192321777, 0.10831904411315918, 0.10830163955688477, 0.10829591751098633, 0.1083827018737793, 0.10861849784851074, 0.1081695556640625, 0.10809159278869629, 0.10954809188842773, 0.10861611366271973, 0.1080620288848877, 0.10840964317321777, 0.10872745513916016, 0.10773491859436035, 0.10832595825195312, 0.10820865631103516, 0.10865902900695801, 0.10828328132629395, 0.1080007553100586, 0.1082608699798584, 0.10947322845458984, 0.10847115516662598, 0.10813307762145996, 0.10860157012939453, 0.10801053047180176, 0.10811448097229004, 0.10837721824645996, 0.10802435874938965, 0.10912013053894043, 0.10890722274780273, 0.10834646224975586, 0.10810494422912598, 0.10805416107177734, 0.10778236389160156, 0.1082456111907959, 0.10807681083679199, 0.10874342918395996, 0.10811996459960938, 0.10812640190124512, 0.10834693908691406, 0.10791873931884766, 0.10850882530212402, 0.10823583602905273, 0.10825061798095703, 0.10856962203979492, 0.10853910446166992, 0.10814762115478516, 0.10842633247375488, 0.10842394828796387, 0.10807633399963379, 0.10811710357666016, 0.10862135887145996, 0.10912036895751953, 0.10803365707397461, 0.10873937606811523, 0.1080484390258789, 0.10835886001586914, 0.1077415943145752, 0.10800361633300781, 0.10888671875, 0.10840654373168945, 0.10794901847839355, 0.10839700698852539, 0.10812711715698242, 0.10811948776245117, 0.10838174819946289, 0.1082465648651123, 0.10809540748596191, 0.10819554328918457, 0.1082756519317627, 0.10847854614257812, 0.1081535816192627, 0.10815739631652832, 0.10871386528015137, 0.10836505889892578, 0.10922408103942871, 0.10789299011230469, 0.10813546180725098, 0.10791397094726562, 0.10815668106079102, 0.10821914672851562, 0.10796475410461426, 0.1085655689239502, 0.10878610610961914, 0.10845661163330078, 0.10823464393615723, 0.10874295234680176, 0.10803413391113281, 0.10794734954833984, 0.10789752006530762, 0.1078176498413086, 0.10760140419006348, 0.10822272300720215, 0.10798120498657227, 0.10832571983337402, 0.10820174217224121, 0.10936141014099121, 0.10787129402160645, 0.10856795310974121, 0.10808563232421875, 0.10877275466918945, 0.10890054702758789, 0.10878205299377441, 0.10764479637145996, 0.11012697219848633, 0.10828208923339844, 0.10823583602905273, 0.10835695266723633, 0.1083822250366211, 0.10821676254272461, 0.10827136039733887, 0.10810375213623047, 0.10771322250366211, 0.10778427124023438, 0.10853409767150879, 0.10811305046081543, 0.1095736026763916, 0.10802936553955078, 0.10772252082824707, 0.1085062026977539, 0.10823273658752441, 0.10860204696655273, 0.10818791389465332, 0.10841631889343262, 0.1080482006072998, 0.10792064666748047, 0.10802984237670898, 0.10785412788391113, 0.10879659652709961, 0.10807442665100098, 0.10810232162475586, 0.10882830619812012, 0.10846328735351562, 0.10807967185974121, 0.10853075981140137, 0.10825991630554199, 0.10860943794250488, 0.10786867141723633, 0.10799908638000488, 0.10802316665649414, 0.10856938362121582, 0.10770869255065918, 0.10806441307067871, 0.10806965827941895, 0.10883879661560059, 0.1082310676574707, 0.10878157615661621, 0.10835099220275879, 0.10785794258117676, 0.1083383560180664, 0.10797929763793945, 0.10847902297973633, 0.10861921310424805, 0.10852837562561035, 0.10793876647949219, 0.10792732238769531, 0.10833334922790527, 0.10816287994384766, 0.1083674430847168, 0.10822129249572754, 0.1080021858215332, 0.10766458511352539, 0.10801577568054199, 0.10823774337768555, 0.10834193229675293, 0.10888814926147461, 0.10805106163024902, 0.10804295539855957, 0.1082615852355957, 0.1079714298248291, 0.10906410217285156, 0.10866785049438477, 0.10827875137329102, 0.10838031768798828, 0.10854172706604004, 0.10950112342834473, 0.10810995101928711, 0.10834884643554688, 0.10856962203979492, 0.10836267471313477, 0.10851836204528809, 0.10813021659851074, 0.10825467109680176, 0.10775518417358398, 0.10842585563659668, 0.10867595672607422, 0.10866189002990723, 0.10802888870239258, 0.1083688735961914, 0.10812854766845703, 0.10869598388671875, 0.10851836204528809, 0.1080327033996582, 0.10942220687866211, 0.10844540596008301, 0.10843396186828613, 0.10813522338867188, 0.10834956169128418, 0.10833215713500977, 0.10806465148925781, 0.10776948928833008, 0.10770988464355469, 0.10796403884887695, 0.10799145698547363, 0.10807132720947266, 0.10809707641601562, 0.1078803539276123, 0.10758709907531738, 0.10789608955383301, 0.10807561874389648, 0.10789322853088379, 0.10860085487365723, 0.10828423500061035, 0.10858964920043945, 0.1088709831237793, 0.10807299613952637, 0.10804629325866699, 0.10815072059631348, 0.10780215263366699, 0.10824036598205566, 0.10816621780395508, 0.1080772876739502, 0.10767126083374023, 0.10794377326965332, 0.10802984237670898, 0.10797548294067383, 0.1080324649810791, 0.10849428176879883, 0.10841774940490723, 0.10803627967834473, 0.10816192626953125, 0.10801219940185547, 0.1080331802368164, 0.10816144943237305, 0.1084280014038086, 0.10963177680969238, 0.10927867889404297, 0.10791134834289551, 0.10783219337463379, 0.10891294479370117, 0.10824441909790039, 0.10861849784851074, 0.10773754119873047, 0.11006999015808105, 0.10867738723754883, 0.10857605934143066, 0.10805916786193848, 0.10768508911132812, 0.1081700325012207, 0.10829806327819824, 0.1080789566040039, 0.10797309875488281, 0.10811448097229004, 0.1076347827911377, 0.1082000732421875, 0.10844993591308594, 0.1079099178314209, 0.10827350616455078, 0.10807538032531738, 0.10787105560302734, 0.10826802253723145, 0.10804438591003418, 0.10770750045776367, 0.10816478729248047, 0.1077418327331543, 0.10783720016479492, 0.1080625057220459, 0.10868310928344727, 0.1085364818572998, 0.10842370986938477, 0.1082916259765625, 0.10847711563110352, 0.1084451675415039, 0.10834455490112305, 0.10831475257873535, 0.10841941833496094, 0.10797929763793945, 0.10822463035583496, 0.10885858535766602, 0.1128699779510498, 0.1081087589263916, 0.10791254043579102, 0.10843968391418457, 0.10826897621154785, 0.10782003402709961, 0.10919070243835449, 0.10864377021789551, 0.10782313346862793, 0.10802292823791504, 0.1079096794128418, 0.10840487480163574, 0.10792112350463867, 0.10829520225524902, 0.10815238952636719, 0.10844635963439941, 0.10825586318969727, 0.10862493515014648, 0.10957551002502441, 0.10876989364624023, 0.10804057121276855, 0.1080179214477539, 0.10789895057678223, 0.10825133323669434, 0.10853338241577148, 0.10830020904541016, 0.1086585521697998, 0.10792374610900879, 0.10804581642150879, 0.10826277732849121, 0.10821938514709473, 0.10873579978942871, 0.10843276977539062, 0.10858011245727539, 0.1080474853515625, 0.1132512092590332, 0.10833573341369629, 0.10806083679199219, 0.10847949981689453, 0.10854005813598633, 0.10832643508911133, 0.10812091827392578, 0.10819363594055176, 0.10829472541809082, 0.1083674430847168, 0.10857701301574707, 0.10817670822143555, 0.10830950736999512, 0.10812902450561523, 0.10779118537902832, 0.10926127433776855, 0.10774064064025879, 0.10928726196289062, 0.10803508758544922, 0.10833954811096191, 0.10815763473510742, 0.10843467712402344, 0.10915398597717285, 0.1080777645111084, 0.10824871063232422, 0.10808038711547852, 0.10852265357971191, 0.1083228588104248, 0.10794639587402344, 0.10808157920837402, 0.10844087600708008, 0.10878729820251465, 0.10817313194274902, 0.10895228385925293, 0.1083986759185791, 0.10817146301269531, 0.10888290405273438, 0.10790586471557617, 0.1095278263092041, 0.10821795463562012, 0.10879158973693848, 0.10818958282470703, 0.10815787315368652, 0.10825586318969727, 0.10863375663757324, 0.10901045799255371, 0.10857343673706055, 0.1087186336517334, 0.10818910598754883, 0.10866069793701172, 0.10838437080383301, 0.10787391662597656, 0.1083230972290039, 0.10812616348266602, 0.10808014869689941, 0.10807275772094727, 0.10797977447509766, 0.10852217674255371, 0.1076042652130127, 0.10793185234069824, 0.1083528995513916, 0.10973167419433594, 0.1083517074584961, 0.10862517356872559, 0.1087031364440918, 0.10769867897033691, 0.10810351371765137, 0.11094546318054199, 0.10929465293884277, 0.10813713073730469, 0.10849928855895996, 0.10802292823791504, 0.10805702209472656, 0.1082618236541748, 0.10811829566955566, 0.10821866989135742, 0.10816454887390137, 0.10787367820739746, 0.10825157165527344, 0.10788965225219727, 0.10824155807495117, 0.10808086395263672, 0.1085669994354248, 0.10794758796691895, 0.10786199569702148, 0.10845494270324707, 0.10807180404663086, 0.10799121856689453, 0.10799264907836914, 0.10785651206970215, 0.10770511627197266, 0.10843420028686523, 0.1076500415802002, 0.11327576637268066, 0.10824227333068848, 0.1078331470489502, 0.10855770111083984, 0.10776042938232422, 0.10936212539672852, 0.10866641998291016, 0.10847783088684082, 0.10804152488708496, 0.10807657241821289, 0.10824322700500488, 0.10815644264221191, 0.10829830169677734, 0.10834550857543945, 0.1091768741607666, 0.10834884643554688, 0.1085042953491211, 0.10820889472961426, 0.10771393775939941, 0.10798072814941406, 0.10789942741394043, 0.10854554176330566, 0.10852599143981934, 0.10858273506164551, 0.10819101333618164, 0.10776329040527344, 0.10790252685546875, 0.10814785957336426, 0.10799455642700195, 0.10834431648254395, 0.10788297653198242, 0.10856938362121582, 0.10842323303222656, 0.10834193229675293, 0.10840487480163574, 0.10845351219177246, 0.10823631286621094, 0.10770320892333984, 0.10814023017883301, 0.10787439346313477, 0.10767197608947754, 0.10779619216918945, 0.10794949531555176, 0.1080622673034668, 0.10761332511901855, 0.10853242874145508, 0.10802578926086426, 0.11071944236755371, 0.10822606086730957, 0.10788416862487793, 0.10805416107177734, 0.1086432933807373, 0.1078181266784668, 0.10772109031677246, 0.10787653923034668, 0.10788631439208984, 0.10875129699707031, 0.10828208923339844, 0.1081993579864502, 0.10788512229919434, 0.10914897918701172, 0.10766291618347168, 0.10826253890991211, 0.10901904106140137, 0.10860252380371094, 0.1077730655670166, 0.10834884643554688, 0.10837006568908691, 0.10814762115478516, 0.10809683799743652, 0.10809850692749023, 0.1080617904663086, 0.10814070701599121, 0.1087350845336914, 0.10981893539428711, 0.1099541187286377, 0.10931992530822754, 0.10798025131225586, 0.10930705070495605, 0.10957145690917969, 0.10799932479858398, 0.1094825267791748, 0.11003303527832031, 0.11027336120605469, 0.10880875587463379, 0.10803651809692383, 0.1075592041015625, 0.1080479621887207, 0.10788393020629883, 0.10778594017028809, 0.10776996612548828, 0.10798454284667969, 0.10805964469909668, 0.10811448097229004, 0.10811495780944824, 0.10777902603149414, 0.1078023910522461, 0.10868978500366211, 0.11185431480407715, 0.1083531379699707, 0.1086876392364502, 0.10828280448913574, 0.10863733291625977, 0.11092543601989746, 0.1093754768371582, 0.11174345016479492, 0.11049485206604004, 0.1122281551361084, 0.10964417457580566, 0.11470770835876465, 0.1123819351196289, 0.11028265953063965, 0.11065840721130371, 0.10916495323181152, 0.10843801498413086, 0.10845327377319336, 0.1090390682220459, 0.11033344268798828, 0.11001276969909668, 0.10959672927856445, 0.10892033576965332, 0.11019372940063477, 0.1086893081665039, 0.11517190933227539, 0.10867667198181152, 0.10795283317565918, 0.1081390380859375, 0.10905098915100098, 0.10839128494262695, 0.10851001739501953, 0.10792398452758789, 0.10819053649902344, 0.10799026489257812, 0.10793066024780273, 0.10842752456665039, 0.10784912109375, 0.10862112045288086, 0.10839724540710449, 0.11034321784973145, 0.10947036743164062, 0.10930132865905762, 0.10895967483520508, 0.11005163192749023, 0.11097335815429688, 0.11015748977661133, 0.10971736907958984, 0.11024928092956543, 0.10971808433532715, 0.10849213600158691, 0.10829520225524902, 0.11007118225097656, 0.10836458206176758, 0.10896515846252441, 0.10793614387512207, 0.10887813568115234, 0.10899519920349121, 0.1083226203918457, 0.10859322547912598, 0.10809016227722168, 0.10800600051879883, 0.1081686019897461, 0.10846519470214844, 0.10855650901794434, 0.10906124114990234, 0.10834050178527832, 0.10843706130981445, 0.10904455184936523, 0.10847187042236328, 0.10810208320617676, 0.10881710052490234, 0.1082303524017334, 0.10810375213623047, 0.10801482200622559, 0.10838723182678223, 0.10800766944885254, 0.10759210586547852, 0.1084439754486084, 0.10819554328918457, 0.10821747779846191, 0.1078181266784668, 0.1077280044555664, 0.10805320739746094, 0.10949873924255371, 0.11004972457885742, 0.11018633842468262, 0.10816788673400879, 0.10756373405456543, 0.10980749130249023, 0.10829567909240723, 0.10828495025634766, 0.10796141624450684, 0.1078042984008789, 0.10821390151977539, 0.10802698135375977, 0.10854530334472656, 0.10764336585998535, 0.10798263549804688, 0.10862851142883301, 0.10892796516418457, 0.10839152336120605, 0.10877132415771484, 0.10808873176574707, 0.10820245742797852, 0.10877633094787598, 0.10990405082702637, 0.10875630378723145, 0.1099846363067627, 0.1100165843963623, 0.11040973663330078, 0.11008405685424805, 0.11050105094909668, 0.1093134880065918, 0.10820269584655762, 0.11043643951416016, 0.11090803146362305, 0.11032319068908691, 0.11016678810119629, 0.10883331298828125, 0.10871767997741699, 0.1091160774230957, 0.10980629920959473, 0.10999155044555664, 0.10878133773803711, 0.10886931419372559, 0.10817456245422363, 0.10839986801147461, 0.10799765586853027, 0.10842299461364746, 0.10840868949890137, 0.10846138000488281, 0.10812759399414062, 0.10802268981933594, 0.10855531692504883, 0.10793328285217285, 0.10835480690002441, 0.10843849182128906, 0.10781550407409668, 0.10832524299621582, 0.1081695556640625, 0.10829854011535645, 0.10842514038085938, 0.1081075668334961, 0.10806417465209961, 0.1080787181854248, 0.10853719711303711, 0.10760164260864258, 0.10877299308776855, 0.10857367515563965, 0.10860848426818848, 0.10788154602050781, 0.11041855812072754, 0.1083681583404541, 0.10783052444458008, 0.108428955078125, 0.10764646530151367, 0.10846734046936035, 0.10778999328613281, 0.10799384117126465, 0.10773110389709473, 0.10813093185424805, 0.10843324661254883, 0.10848045349121094, 0.10857653617858887, 0.10833120346069336, 0.10780620574951172, 0.10819816589355469, 0.10793757438659668, 0.10831999778747559, 0.10873794555664062, 0.1080465316772461, 0.10828089714050293, 0.10797762870788574, 0.10829734802246094, 0.10788631439208984, 0.10824370384216309, 0.10826706886291504, 0.10813593864440918, 0.10870218276977539, 0.1080012321472168, 0.10792374610900879, 0.10780525207519531, 0.10837101936340332, 0.10838651657104492, 0.10842704772949219, 0.10776686668395996, 0.1081230640411377, 0.10815811157226562, 0.10813331604003906, 0.10842299461364746, 0.10791492462158203, 0.1089026927947998, 0.10811114311218262, 0.10870003700256348, 0.10865187644958496, 0.10832881927490234, 0.10796737670898438, 0.10793924331665039, 0.10852980613708496, 0.1085960865020752, 0.10837364196777344, 0.10780501365661621, 0.10792708396911621, 0.10799193382263184, 0.1084737777709961, 0.10795831680297852, 0.10812783241271973, 0.1077110767364502, 0.10826992988586426, 0.10764169692993164, 0.10830855369567871, 0.10745882987976074, 0.10806846618652344, 0.1077873706817627, 0.10784626007080078, 0.10752677917480469, 0.10738205909729004, 0.10810041427612305, 0.10731911659240723, 0.10823369026184082, 0.10784125328063965, 0.10789704322814941, 0.1082773208618164, 0.10758423805236816, 0.10859489440917969, 0.10826873779296875, 0.10806941986083984, 0.1078331470489502, 0.10776185989379883, 0.10762643814086914, 0.10771918296813965, 0.10768270492553711, 0.10781002044677734, 0.10876035690307617, 0.10812830924987793, 0.10723114013671875, 0.10764551162719727, 0.10844016075134277, 0.10794186592102051, 0.10795187950134277, 0.10888314247131348, 0.10824441909790039, 0.1078331470489502, 0.1078035831451416, 0.10771512985229492, 0.10824775695800781, 0.10774636268615723, 0.10828161239624023, 0.10770559310913086, 0.10806083679199219, 0.10819149017333984, 0.10759520530700684, 0.10767340660095215, 0.10715174674987793, 0.10784316062927246, 0.10779333114624023, 0.10814189910888672, 0.10770273208618164, 0.10786962509155273, 0.10783147811889648, 0.10799860954284668, 0.10762715339660645, 0.10772848129272461, 0.10808253288269043, 0.10775208473205566, 0.10815191268920898, 0.10755753517150879, 0.10752606391906738, 0.1077885627746582, 0.1074981689453125, 0.10740804672241211, 0.10785555839538574, 0.10720396041870117, 0.10847854614257812, 0.10759210586547852, 0.10796260833740234, 0.10747218132019043, 0.10758209228515625, 0.10785508155822754, 0.10742425918579102, 0.10802555084228516, 0.10727453231811523, 0.10926437377929688, 0.10781168937683105, 0.10829520225524902, 0.10724020004272461, 0.10857033729553223, 0.10746097564697266, 0.10821247100830078, 0.10944819450378418, 0.10892438888549805, 0.10856413841247559, 0.10796666145324707, 0.10812020301818848, 0.10801243782043457, 0.10829329490661621, 0.1073920726776123, 0.1076040267944336, 0.10765504837036133, 0.10772466659545898, 0.10773181915283203, 0.10800671577453613, 0.10806632041931152, 0.10801577568054199, 0.10786771774291992, 0.10811495780944824, 0.10784697532653809, 0.10736727714538574, 0.10820317268371582, 0.10796904563903809, 0.1079244613647461, 0.10749363899230957, 0.10795068740844727, 0.10859966278076172, 0.1082155704498291, 0.10994505882263184, 0.11095476150512695, 0.23453474044799805, 0.17149758338928223, 0.10620713233947754, 0.10328149795532227, 0.10569643974304199, 0.10605859756469727, 0.10641217231750488, 0.16697382926940918, 0.23610401153564453, 0.27936697006225586, 0.24216198921203613, 0.16547083854675293, 0.11199665069580078, 0.1121530532836914, 0.10517573356628418, 0.11220836639404297, 0.10944318771362305, 0.10951566696166992, 0.11659622192382812, 0.10757327079772949, 0.10513830184936523, 0.10461974143981934, 0.10516214370727539, 0.11325860023498535, 0.10949897766113281, 0.10458254814147949, 0.1043999195098877, 0.10548138618469238, 0.10466361045837402, 0.10645270347595215, 0.10484480857849121, 0.10474348068237305, 0.10516238212585449, 0.10508203506469727, 0.10428071022033691, 0.10468101501464844, 0.1061251163482666, 0.11547327041625977, 0.10514378547668457, 0.10439157485961914, 0.10500597953796387, 0.10660862922668457, 0.10806488990783691, 0.10486578941345215, 0.10636591911315918, 0.21412920951843262, 0.32115674018859863, 0.10795855522155762, 0.33017492294311523, 0.17692828178405762, 0.11903572082519531, 0.11978650093078613, 0.10475373268127441, 0.10456609725952148, 0.10443997383117676, 0.10561656951904297, 0.11652994155883789, 0.13917827606201172, 0.33553218841552734, 0.14587187767028809, 0.10491704940795898, 0.10486817359924316, 0.11725711822509766, 0.11882710456848145, 0.10431504249572754, 0.11542391777038574, 0.10676050186157227, 0.10600471496582031, 0.1084754467010498, 0.10455632209777832, 0.10436844825744629, 0.10480451583862305, 0.10471534729003906, 0.1126399040222168, 0.13448333740234375, 0.11459994316101074, 0.12869977951049805, 0.12494683265686035, 0.10898804664611816, 0.11122584342956543, 0.11594963073730469, 0.10777521133422852, 0.10999417304992676, 0.10505414009094238, 0.10468220710754395, 0.10518693923950195, 0.10505485534667969, 0.11873197555541992, 0.10429763793945312, 0.10462188720703125, 0.10480618476867676, 0.10622072219848633, 0.11010384559631348, 0.10460543632507324, 0.10461235046386719, 0.10438704490661621, 0.11695051193237305, 0.10761475563049316, 0.11099910736083984, 0.10410547256469727, 0.10476446151733398, 0.10862612724304199, 0.1049807071685791, 0.10448932647705078, 0.10441803932189941, 0.10500478744506836, 0.11279964447021484, 0.10533857345581055, 0.10401034355163574, 0.1136629581451416, 0.10752105712890625, 0.10929656028747559, 0.10519123077392578, 0.10492944717407227, 0.10442399978637695, 0.10576248168945312, 0.1142125129699707, 0.10527467727661133, 0.10468268394470215, 0.10551333427429199, 0.10584878921508789, 0.11511468887329102, 0.1058502197265625, 0.10576891899108887, 0.10537552833557129, 0.1063227653503418, 0.11482357978820801, 0.10672974586486816, 0.10586190223693848, 0.10503721237182617, 0.1053156852722168, 0.11150217056274414, 0.11019110679626465, 0.10598587989807129, 0.10556268692016602, 0.1058499813079834, 0.11654996871948242, 0.10573148727416992, 0.10594296455383301, 0.10599565505981445, 0.10594439506530762, 0.11723995208740234, 0.10550594329833984, 0.1161501407623291, 0.10569620132446289, 0.10613393783569336, 0.1181650161743164, 0.10606646537780762, 0.10527849197387695, 0.10686969757080078, 0.10602068901062012, 0.11056280136108398, 0.10613441467285156, 0.10507559776306152, 0.10991930961608887, 0.10581445693969727, 0.11370372772216797, 0.10567641258239746, 0.10531044006347656, 0.10573530197143555, 0.11320805549621582, 0.10681509971618652, 0.10583710670471191, 0.10496997833251953, 0.10522818565368652, 0.11464595794677734, 0.11060547828674316, 0.10579323768615723, 0.10590028762817383, 0.10585975646972656, 0.11130881309509277, 0.1057281494140625, 0.10630178451538086, 0.10548090934753418, 0.1059720516204834, 0.1099398136138916, 0.1148519515991211, 0.1059274673461914, 0.10669994354248047, 0.10651159286499023, 0.11220169067382812, 0.10585522651672363, 0.1058964729309082, 0.10510540008544922, 0.10599398612976074, 0.11168766021728516, 0.10606050491333008, 0.11492753028869629, 0.10565328598022461, 0.10600519180297852, 0.1124122142791748, 0.11336636543273926, 0.10710525512695312, 0.10508966445922852, 0.10545110702514648, 0.11320376396179199, 0.11059069633483887, 0.11414217948913574, 0.11233162879943848, 0.10619306564331055, 0.10917901992797852, 0.10683917999267578, 0.10466909408569336, 0.10629773139953613, 0.1162714958190918, 0.10901498794555664, 0.10546398162841797, 0.10565567016601562, 0.10622906684875488, 0.1139378547668457, 0.10934972763061523, 0.10536527633666992, 0.10541892051696777, 0.10571146011352539, 0.11303830146789551, 0.10561919212341309, 0.1056509017944336, 0.10562729835510254, 0.10546755790710449, 0.11071419715881348, 0.1054847240447998, 0.10870623588562012, 0.10516715049743652, 0.10644698143005371, 0.11467456817626953, 0.10536456108093262, 0.10579967498779297, 0.10595202445983887, 0.10667586326599121, 0.11209797859191895, 0.1088566780090332, 0.11387228965759277, 0.11568522453308105, 0.11421704292297363, 0.12315225601196289, 0.10778045654296875, 0.10593438148498535, 0.10883116722106934, 0.11386299133300781, 0.11726784706115723, 0.11004281044006348, 0.11275458335876465, 0.11165976524353027, 0.1134498119354248, 0.10922574996948242, 0.1059274673461914, 0.10632896423339844, 0.10581064224243164, 0.11098957061767578, 0.10598301887512207, 0.10536718368530273, 0.10478806495666504, 0.10629582405090332, 0.1122283935546875, 0.11162137985229492, 0.10549116134643555, 0.10490679740905762, 0.10542917251586914, 0.11096906661987305, 0.10624265670776367, 0.10631060600280762, 0.10550832748413086, 0.10649228096008301, 0.11012101173400879, 0.10559487342834473, 0.11556339263916016, 0.10628986358642578, 0.10653018951416016, 0.11206650733947754, 0.10534214973449707, 0.10583376884460449, 0.10628128051757812, 0.1050267219543457, 0.11027669906616211, 0.10579705238342285, 0.10480189323425293, 0.11766886711120605, 0.1078636646270752, 0.11806368827819824, 0.10557055473327637, 0.10529160499572754, 0.10586309432983398, 0.10570573806762695, 0.11559247970581055, 0.10486364364624023, 0.10486388206481934, 0.1055288314819336, 0.11664795875549316, 0.10952639579772949, 0.10570669174194336, 0.10415911674499512, 0.10468745231628418, 0.10915040969848633, 0.10525345802307129, 0.10501241683959961, 0.10391831398010254, 0.10512018203735352, 0.11284470558166504, 0.11124467849731445, 0.10601973533630371, 0.10564327239990234, 0.10655903816223145, 0.11013913154602051, 0.10577178001403809, 0.1055605411529541, 0.10528826713562012, 0.10671091079711914, 0.11406230926513672, 0.10724735260009766, 0.11176371574401855, 0.10571694374084473, 0.10585784912109375, 0.11074137687683105, 0.10599613189697266, 0.10512876510620117, 0.10502195358276367, 0.10574936866760254, 0.11542081832885742, 0.1058657169342041, 0.10530948638916016, 0.11031866073608398, 0.10684895515441895, 0.11208271980285645, 0.10658597946166992, 0.10559821128845215, 0.11212015151977539, 0.1146094799041748, 0.12275528907775879, 0.10595536231994629, 0.10593986511230469, 0.10596776008605957, 0.11116552352905273, 0.10973072052001953, 0.10539126396179199, 0.10503220558166504, 0.10619831085205078, 0.10584211349487305, 0.11427712440490723, 0.10581254959106445, 0.10483622550964355, 0.1052391529083252, 0.11286520957946777, 0.11680364608764648, 0.10860610008239746, 0.1061861515045166, 0.10570120811462402, 0.11426115036010742, 0.1056663990020752, 0.10602760314941406, 0.10507631301879883, 0.10595083236694336, 0.11080026626586914, 0.10606002807617188, 0.11011028289794922, 0.10609841346740723, 0.10592389106750488, 0.11429834365844727, 0.10568571090698242, 0.1050872802734375, 0.1053171157836914, 0.10515689849853516, 0.11042618751525879, 0.10589742660522461, 0.10575127601623535, 0.10614609718322754, 0.10975503921508789, 0.11326813697814941, 0.10555124282836914, 0.10557174682617188, 0.10565185546875, 0.10648608207702637, 0.11188316345214844, 0.10603523254394531, 0.10550451278686523, 0.10496735572814941, 0.10552644729614258, 0.11944437026977539, 0.10583758354187012, 0.10570502281188965, 0.10515499114990234, 0.10572004318237305, 0.1106266975402832, 0.10526108741760254, 0.10558676719665527, 0.10588240623474121, 0.10538458824157715, 0.10914778709411621, 0.11425113677978516, 0.10550332069396973, 0.10608482360839844, 0.10573554039001465, 0.11206412315368652, 0.10583925247192383, 0.10522818565368652, 0.10564374923706055, 0.10484933853149414, 0.11071944236755371, 0.10573196411132812, 0.11410689353942871, 0.10639262199401855, 0.10641002655029297, 0.11032605171203613, 0.1067957878112793, 0.10508584976196289, 0.10600137710571289, 0.10937952995300293, 0.1095271110534668, 0.1055152416229248, 0.10492944717407227, 0.11472439765930176, 0.1065833568572998, 0.10989999771118164, 0.10558342933654785, 0.10535264015197754, 0.10553741455078125, 0.10917520523071289, 0.10636210441589355, 0.10550379753112793, 0.10494756698608398, 0.10572004318237305, 0.1263561248779297, 0.10970759391784668, 0.10645675659179688, 0.10506772994995117, 0.10608148574829102, 0.11650967597961426, 0.1059575080871582, 0.1056983470916748, 0.10563921928405762, 0.10676312446594238, 0.11512947082519531, 0.1149895191192627, 0.10792875289916992, 0.10660600662231445, 0.10548043251037598, 0.11081099510192871, 0.1054680347442627, 0.10531044006347656, 0.10497188568115234, 0.10592031478881836, 0.1140899658203125, 0.10651540756225586, 0.11451554298400879, 0.10636639595031738, 0.10575389862060547, 0.1099085807800293, 0.1065223217010498, 0.10526037216186523, 0.10611438751220703, 0.10531330108642578, 0.11338591575622559, 0.10574769973754883, 0.10504698753356934, 0.10686612129211426, 0.11042141914367676, 0.1098787784576416, 0.10604023933410645, 0.10512828826904297, 0.10534811019897461, 0.11798477172851562, 0.10821747779846191, 0.1060798168182373, 0.10587739944458008, 0.10574650764465332, 0.11112117767333984, 0.10965895652770996, 0.10535979270935059, 0.10614371299743652, 0.10568046569824219, 0.11362433433532715, 0.10629916191101074, 0.10594391822814941, 0.1058204174041748, 0.10672521591186523, 0.11590051651000977, 0.10736417770385742, 0.1105194091796875, 0.10529780387878418, 0.10627484321594238, 0.11364197731018066, 0.10588741302490234, 0.10640668869018555, 0.10586047172546387, 0.10573554039001465, 0.11102604866027832, 0.10550761222839355, 0.10565185546875, 0.11015152931213379, 0.1060633659362793, 0.11381196975708008, 0.10606813430786133, 0.10543537139892578, 0.10478782653808594, 0.1058807373046875, 0.11056280136108398, 0.10577225685119629, 0.1056356430053711, 0.10588598251342773, 0.10974812507629395, 0.1137847900390625, 0.10556483268737793, 0.10574674606323242, 0.10735821723937988, 0.10540533065795898, 0.11207723617553711, 0.10569095611572266, 0.10553312301635742, 0.10587143898010254, 0.10467147827148438, 0.11225771903991699, 0.11327123641967773, 0.10538434982299805, 0.10508203506469727, 0.10525989532470703, 0.11065864562988281, 0.10502099990844727, 0.10430550575256348, 0.10570979118347168, 0.10591459274291992, 0.10905933380126953, 0.11806583404541016, 0.10778212547302246, 0.10524439811706543, 0.11151719093322754, 0.106353759765625, 0.10483264923095703, 0.10483193397521973, 0.1058495044708252, 0.10986781120300293, 0.10504937171936035, 0.10493040084838867, 0.10420036315917969, 0.11598038673400879, 0.11113309860229492, 0.10548257827758789, 0.10451650619506836, 0.10379147529602051, 0.10485959053039551, 0.10854125022888184, 0.10628151893615723, 0.10437273979187012, 0.10526919364929199, 0.10457491874694824, 0.12028884887695312, 0.10497140884399414, 0.10432553291320801, 0.10630512237548828, 0.10531044006347656, 0.10886740684509277, 0.11872696876525879, 0.10468006134033203, 0.105499267578125, 0.10729432106018066, 0.10909390449523926, 0.11728143692016602, 0.10536766052246094, 0.10490918159484863, 0.10451817512512207, 0.11687660217285156, 0.10627508163452148, 0.10626816749572754, 0.10411953926086426, 0.10606741905212402, 0.11551785469055176, 0.10448598861694336, 0.11487078666687012, 0.10546040534973145, 0.1054384708404541, 0.11202549934387207, 0.10486102104187012, 0.10442352294921875, 0.10501360893249512, 0.10470247268676758, 0.11033940315246582, 0.10508084297180176, 0.10435128211975098, 0.11432147026062012, 0.1062467098236084, 0.11080670356750488, 0.10493612289428711, 0.10496091842651367, 0.10456323623657227, 0.10503077507019043, 0.10988068580627441, 0.10509777069091797, 0.10421109199523926, 0.10628724098205566, 0.11899876594543457, 0.11322712898254395, 0.10493993759155273, 0.10422182083129883, 0.10461711883544922, 0.11132502555847168, 0.10642194747924805, 0.10496139526367188, 0.10449910163879395, 0.10462045669555664, 0.11448502540588379, 0.1066584587097168, 0.1103668212890625, 0.10528826713562012, 0.10512804985046387, 0.10951352119445801, 0.10523843765258789, 0.10414004325866699, 0.10478043556213379, 0.10521411895751953, 0.11277270317077637, 0.10585260391235352, 0.10492134094238281, 0.11172342300415039, 0.10543203353881836, 0.10928153991699219, 0.10504293441772461, 0.11438989639282227, 0.10483074188232422, 0.10588502883911133, 0.11019563674926758, 0.10475897789001465, 0.10470008850097656, 0.10500860214233398, 0.10987687110900879, 0.11373448371887207, 0.10563302040100098, 0.1047210693359375, 0.10604286193847656, 0.10490632057189941, 0.11000847816467285, 0.10478568077087402, 0.10478568077087402, 0.10473132133483887, 0.10501837730407715, 0.11792945861816406, 0.10500478744506836, 0.10518312454223633, 0.10468602180480957, 0.10544228553771973, 0.11100935935974121, 0.10443258285522461, 0.10459613800048828, 0.10429644584655762, 0.10469746589660645, 0.11030960083007812, 0.10524654388427734, 0.1138601303100586, 0.10504531860351562, 0.10614490509033203, 0.11160731315612793, 0.1054680347442627, 0.10491394996643066, 0.10463714599609375, 0.10635828971862793, 0.10979819297790527, 0.10495758056640625, 0.10476922988891602, 0.11537575721740723, 0.10640549659729004, 0.11060094833374023, 0.10482597351074219, 0.10471963882446289, 0.1043548583984375, 0.10488581657409668, 0.10882830619812012, 0.10501861572265625, 0.10480332374572754, 0.10495114326477051, 0.11501765251159668, 0.1121220588684082, 0.10494399070739746, 0.10407471656799316, 0.10482406616210938, 0.10419273376464844, 0.11105465888977051, 0.10489320755004883, 0.10448312759399414, 0.1058046817779541, 0.10453605651855469, 0.12258219718933105, 0.10516142845153809, 0.10542893409729004, 0.10488653182983398, 0.10609841346740723, 0.10950160026550293, 0.10576510429382324, 0.10500216484069824, 0.10693025588989258, 0.10515999794006348, 0.11223554611206055, 0.11134815216064453, 0.10526323318481445, 0.10467195510864258, 0.11338186264038086, 0.10668492317199707, 0.10529971122741699, 0.10504674911499023, 0.10645127296447754, 0.11817193031311035, 0.10591268539428711, 0.1053152084350586, 0.1168820858001709, 0.10570907592773438, 0.11852192878723145, 0.10548186302185059, 0.10537075996398926, 0.10542011260986328, 0.10519838333129883, 0.11802887916564941, 0.10567617416381836, 0.10472321510314941, 0.10474944114685059, 0.1111609935760498, 0.11125969886779785, 0.10478353500366211, 0.10445356369018555, 0.10439181327819824, 0.1052553653717041, 0.11375641822814941, 0.10519576072692871, 0.10532760620117188, 0.10475015640258789, 0.10557937622070312, 0.11229538917541504, 0.10977292060852051, 0.10516691207885742, 0.10647892951965332, 0.10459613800048828, 0.11543154716491699, 0.10517215728759766, 0.1055750846862793, 0.1044166088104248, 0.10445523262023926, 0.11118769645690918, 0.10511088371276855, 0.11044573783874512, 0.10494041442871094, 0.1051168441772461, 0.1142263412475586, 0.10465836524963379, 0.10452461242675781, 0.10452127456665039, 0.10473203659057617, 0.11056208610534668, 0.10501837730407715, 0.10419845581054688, 0.10894656181335449, 0.10474872589111328, 0.11402750015258789, 0.1048576831817627, 0.10483598709106445, 0.1048269271850586, 0.10445261001586914, 0.11115908622741699, 0.10524749755859375, 0.11183905601501465, 0.10828876495361328, 0.11436605453491211, 0.11082983016967773, 0.10528922080993652, 0.10456252098083496, 0.1046600341796875, 0.11155080795288086, 0.10652780532836914, 0.10444283485412598, 0.10484766960144043, 0.10446858406066895, 0.11458897590637207, 0.10712409019470215, 0.1106252670288086, 0.10451745986938477, 0.10543084144592285, 0.10944175720214844, 0.1058046817779541, 0.10440611839294434, 0.10434579849243164, 0.10478663444519043, 0.11612749099731445, 0.10663390159606934, 0.10929059982299805, 0.10794472694396973, 0.1048581600189209, 0.10914397239685059, 0.10452938079833984, 0.10481977462768555, 0.10444879531860352, 0.10518598556518555, 0.11385560035705566, 0.10500764846801758, 0.10589289665222168, 0.10541963577270508, 0.11274337768554688, 0.11025428771972656, 0.10558557510375977, 0.10468626022338867, 0.10473012924194336, 0.10524678230285645, 0.11313199996948242, 0.10491228103637695, 0.10490560531616211, 0.10450196266174316, 0.10531330108642578, 0.11689472198486328, 0.10543608665466309, 0.10434651374816895, 0.10458898544311523, 0.10535740852355957, 0.11404252052307129, 0.10489320755004883, 0.10465669631958008, 0.1045217514038086, 0.10670304298400879, 0.11115479469299316, 0.11013031005859375, 0.10527205467224121, 0.10478091239929199, 0.10603642463684082, 0.11340999603271484, 0.10507941246032715, 0.10534453392028809, 0.10457110404968262, 0.10532689094543457, 0.11175966262817383, 0.10501599311828613, 0.1093742847442627, 0.10623764991760254, 0.10492062568664551, 0.11417388916015625, 0.10445809364318848, 0.10432291030883789, 0.10417747497558594, 0.10568976402282715, 0.11063361167907715, 0.10501432418823242, 0.10464906692504883, 0.10468339920043945, 0.10955214500427246, 0.11922836303710938, 0.1052393913269043, 0.10478854179382324, 0.10509705543518066, 0.10413336753845215, 0.11797070503234863, 0.10486030578613281, 0.10443305969238281, 0.10545492172241211, 0.10611772537231445, 0.12940716743469238, 0.10432004928588867, 0.10550332069396973, 0.10529708862304688, 0.11070132255554199, 0.10499453544616699, 0.10441946983337402, 0.1039285659790039, 0.10506629943847656, 0.10988712310791016, 0.10516047477722168, 0.11464095115661621, 0.1058199405670166, 0.10746908187866211, 0.11322951316833496, 0.10515928268432617, 0.10459661483764648, 0.10401391983032227, 0.10482096672058105, 0.10874223709106445, 0.1046299934387207, 0.10490250587463379, 0.11265969276428223, 0.10702991485595703, 0.1116781234741211, 0.10449075698852539, 0.1043088436126709, 0.10476994514465332, 0.10491085052490234, 0.10932612419128418, 0.10572290420532227, 0.10453033447265625, 0.10469341278076172, 0.11322832107543945, 0.11065435409545898, 0.1050271987915039, 0.10526394844055176, 0.10479187965393066, 0.10512232780456543, 0.10885214805603027, 0.10511636734008789, 0.10459136962890625, 0.10457634925842285, 0.10517334938049316, 0.1140751838684082, 0.11081743240356445, 0.10479569435119629, 0.10470104217529297, 0.10547399520874023, 0.1098182201385498, 0.10442113876342773, 0.1048738956451416, 0.10434794425964355, 0.10535311698913574, 0.11330676078796387, 0.10489773750305176, 0.11018633842468262, 0.10458946228027344, 0.10567665100097656, 0.10954570770263672, 0.10491442680358887, 0.10453128814697266, 0.1040802001953125, 0.10664176940917969, 0.11298894882202148, 0.10523271560668945, 0.10476517677307129, 0.11057543754577637, 0.10532402992248535, 0.10972929000854492, 0.10522127151489258, 0.10442757606506348, 0.10492992401123047, 0.10480570793151855, 0.11311078071594238, 0.1051642894744873, 0.10426521301269531, 0.10544633865356445, 0.11081409454345703, 0.11014032363891602, 0.10443115234375, 0.10598945617675781, 0.10420680046081543, 0.10701990127563477, 0.11458468437194824, 0.10558247566223145, 0.10478687286376953, 0.10643339157104492, 0.10527992248535156, 0.11288619041442871, 0.10872030258178711, 0.10498952865600586, 0.10495734214782715, 0.10399603843688965, 0.11412763595581055, 0.10449528694152832, 0.10467028617858887, 0.1049344539642334, 0.10509753227233887, 0.11119532585144043, 0.10500621795654297, 0.10993027687072754, 0.10553836822509766, 0.10653281211853027, 0.11318564414978027, 0.10507512092590332, 0.1041569709777832, 0.1055905818939209, 0.10561800003051758, 0.10931110382080078, 0.1045989990234375, 0.10455489158630371, 0.10938000679016113, 0.10448670387268066, 0.11401200294494629, 0.10471630096435547, 0.10458850860595703, 0.10556459426879883, 0.11531591415405273, 0.1071465015411377, 0.1049034595489502, 0.10418367385864258, 0.10642862319946289, 0.12060689926147461, 0.10899615287780762, 0.1044151782989502, 0.10463905334472656, 0.10496068000793457, 540.9422471523285]
t_12_lzf = [0.15658855438232422, 0.12774372100830078, 0.1027219295501709, 0.10347819328308105, 0.11359953880310059, 0.12409830093383789, 0.15151715278625488, 0.10533857345581055, 0.10541486740112305, 0.1140592098236084, 0.10417509078979492, 0.10303783416748047, 0.10466456413269043, 0.10514044761657715, 0.11473226547241211, 0.10396575927734375, 0.10957741737365723, 0.10429930686950684, 0.10881900787353516, 0.1058049201965332, 0.10389208793640137, 0.10357427597045898, 0.10388803482055664, 0.10431265830993652, 0.11160540580749512, 0.10355567932128906, 0.10331177711486816, 0.11669039726257324, 0.1044013500213623, 0.10794758796691895, 0.10396385192871094, 0.10332584381103516, 0.10379171371459961, 0.10964202880859375, 0.10770916938781738, 0.10401391983032227, 0.10369181632995605, 0.1034250259399414, 0.10470271110534668, 0.1127922534942627, 0.10376143455505371, 0.1045687198638916, 0.10388875007629395, 0.1049661636352539, 0.11149287223815918, 0.10422444343566895, 0.1032564640045166, 0.10514712333679199, 0.10732483863830566, 0.10936999320983887, 0.11002588272094727, 0.10500836372375488, 0.10570669174194336, 0.11261630058288574, 0.10637593269348145, 0.10488486289978027, 0.10427165031433105, 0.10505247116088867, 0.11003398895263672, 0.10605382919311523, 0.10518002510070801, 0.11006832122802734, 0.10563516616821289, 0.11361455917358398, 0.10537314414978027, 0.10459399223327637, 0.10435986518859863, 0.10553956031799316, 0.1115109920501709, 0.10503792762756348, 0.10488486289978027, 0.1047663688659668, 0.11057281494140625, 0.11404204368591309, 0.10503697395324707, 0.10410070419311523, 0.10355734825134277, 0.10417675971984863, 0.10910177230834961, 0.10393333435058594, 0.10387492179870605, 0.10337328910827637, 0.10439777374267578, 0.10778260231018066, 0.11180996894836426, 0.10396623611450195, 0.10318899154663086, 0.10394501686096191, 0.10820174217224121, 0.10540270805358887, 0.10384392738342285, 0.10322713851928711, 0.10536885261535645, 0.10800337791442871, 0.10517668724060059, 0.11268401145935059, 0.10450410842895508, 0.10576629638671875, 0.10899853706359863, 0.1045844554901123, 0.10341954231262207, 0.10340571403503418, 0.10359549522399902, 0.108551025390625, 0.10421967506408691, 0.10380387306213379, 0.11326265335083008, 0.10440492630004883, 0.10974669456481934, 0.10389113426208496, 0.1044313907623291, 0.10415267944335938, 0.10378646850585938, 0.10907506942749023, 0.10420870780944824, 0.10384440422058105, 0.10315299034118652, 0.1037454605102539, 0.11613607406616211, 0.10668325424194336, 0.1038515567779541, 0.10314202308654785, 0.1034996509552002, 0.10753607749938965, 0.10437726974487305, 0.10387396812438965, 0.10362505912780762, 0.10413432121276855, 0.11181640625, 0.11151695251464844, 0.10363364219665527, 0.10375046730041504, 0.10499930381774902, 0.10810279846191406, 0.10503554344177246, 0.11629915237426758, 0.10394144058227539, 0.10437273979187012, 0.11558961868286133, 0.10377001762390137, 0.11501407623291016, 0.10570001602172852, 0.10615873336791992, 0.11772871017456055, 0.10367083549499512, 0.10428023338317871, 0.10393214225769043, 0.10400915145874023, 0.1102147102355957, 0.10381627082824707, 0.10373616218566895, 0.10811257362365723, 0.10639262199401855, 0.11390161514282227, 0.10445880889892578, 0.10410642623901367, 0.10374569892883301, 0.10400199890136719, 0.11068511009216309, 0.10408163070678711, 0.10430717468261719, 0.10359048843383789, 0.10471534729003906, 0.1170039176940918, 0.1044614315032959, 0.10350298881530762, 0.10400843620300293, 0.10444426536560059, 0.11054491996765137, 0.10368680953979492, 0.10349702835083008, 0.10335326194763184, 0.104949951171875, 0.11082792282104492, 0.11279559135437012, 0.10386919975280762, 0.10403871536254883, 0.10391664505004883, 0.10903382301330566, 0.10366106033325195, 0.1039133071899414, 0.10379219055175781, 0.10441112518310547, 0.10857200622558594, 0.10392022132873535, 0.11269569396972656, 0.1037740707397461, 0.10445356369018555, 0.11066007614135742, 0.10525083541870117, 0.10405158996582031, 0.10295844078063965, 0.10377621650695801, 0.10870981216430664, 0.10388636589050293, 0.10423493385314941, 0.1040501594543457, 0.11490678787231445, 0.11136460304260254, 0.10379552841186523, 0.11121702194213867, 0.10690665245056152, 0.1054692268371582, 0.1138148307800293, 0.10439109802246094, 0.10385966300964355, 0.1033334732055664, 0.10531282424926758, 0.11671638488769531, 0.10435128211975098, 0.104522705078125, 0.10336852073669434, 0.10451889038085938, 0.11325192451477051, 0.10348391532897949, 0.10419893264770508, 0.10395693778991699, 0.10483574867248535, 0.11052465438842773, 0.10963869094848633, 0.10437703132629395, 0.10435080528259277, 0.10458183288574219, 0.11245989799499512, 0.1041877269744873, 0.1039283275604248, 0.10325145721435547, 0.10504698753356934, 0.10899567604064941, 0.10421586036682129, 0.10844540596008301, 0.12844157218933105, 0.10442018508911133, 0.11245870590209961, 0.10371875762939453, 0.10431933403015137, 0.10489606857299805, 0.10381579399108887, 0.10994529724121094, 0.10405135154724121, 0.10325026512145996, 0.10622787475585938, 0.10636472702026367, 0.11246585845947266, 0.10492634773254395, 0.10457468032836914, 0.1047825813293457, 0.1048879623413086, 0.11075377464294434, 0.10479235649108887, 0.10452580451965332, 0.10426664352416992, 0.10424256324768066, 0.1174619197845459, 0.10460305213928223, 0.1045224666595459, 0.1042485237121582, 0.1051180362701416, 0.1106874942779541, 0.10514116287231445, 0.10518097877502441, 0.10487961769104004, 0.10432267189025879, 0.10983848571777344, 0.11378860473632812, 0.10422992706298828, 0.1055605411529541, 0.1037607192993164, 0.1114509105682373, 0.10428237915039062, 0.10424065589904785, 0.10462164878845215, 0.10337114334106445, 0.10984063148498535, 0.10368084907531738, 0.11378073692321777, 0.10554218292236328, 0.10595130920410156, 0.11705827713012695, 0.10423803329467773, 0.10407352447509766, 0.1041102409362793, 0.11389613151550293, 0.1058502197265625, 0.10416626930236816, 0.10383009910583496, 0.11250066757202148, 0.1091008186340332, 0.11072421073913574, 0.10402584075927734, 0.10416650772094727, 0.10390043258666992, 0.10956239700317383, 0.10439229011535645, 0.10452961921691895, 0.10380411148071289, 0.10448694229125977, 0.11293601989746094, 0.1104578971862793, 0.10516023635864258, 0.10515403747558594, 0.10481786727905273, 0.10803341865539551, 0.10497760772705078, 0.10418128967285156, 0.10402989387512207, 0.10494327545166016, 0.11220693588256836, 0.10532903671264648, 0.11039066314697266, 0.10474419593811035, 0.10498046875, 0.10866284370422363, 0.10539364814758301, 0.10487794876098633, 0.1043696403503418, 0.10560774803161621, 0.11286139488220215, 0.10477423667907715, 0.10382413864135742, 0.11083173751831055, 0.10564303398132324, 0.10907578468322754, 0.10512685775756836, 0.10484147071838379, 0.10443854331970215, 0.10551881790161133, 0.11423230171203613, 0.10482668876647949, 0.1066741943359375, 0.10482144355773926, 0.11177182197570801, 0.11116790771484375, 0.1054220199584961, 0.10491156578063965, 0.10443520545959473, 0.10530400276184082, 0.11344599723815918, 0.10497760772705078, 0.10492753982543945, 0.10435771942138672, 0.11292052268981934, 0.11095523834228516, 0.1129457950592041, 0.10540485382080078, 0.10438251495361328, 0.10450029373168945, 0.10955643653869629, 0.10522794723510742, 0.10443115234375, 0.10414505004882812, 0.10453605651855469, 0.1083526611328125, 0.10444211959838867, 0.11263823509216309, 0.10500550270080566, 0.1055002212524414, 0.11023616790771484, 0.10505127906799316, 0.10509562492370605, 0.10415863990783691, 0.10540246963500977, 0.10965180397033691, 0.10527396202087402, 0.10496783256530762, 0.11286163330078125, 0.10485172271728516, 0.10998749732971191, 0.10453629493713379, 0.10497236251831055, 0.10512948036193848, 0.10512852668762207, 0.10887694358825684, 0.10475778579711914, 0.10476255416870117, 0.10505175590515137, 0.11435198783874512, 0.11207914352416992, 0.10602521896362305, 0.10500025749206543, 0.10449647903442383, 0.10440349578857422, 0.10912680625915527, 0.10558390617370605, 0.10542869567871094, 0.10591745376586914, 0.1053609848022461, 0.11448812484741211, 0.11130237579345703, 0.10474848747253418, 0.10532355308532715, 0.10467028617858887, 0.1100931167602539, 0.10542654991149902, 0.10465884208679199, 0.10567188262939453, 0.10401582717895508, 0.11460280418395996, 0.10539746284484863, 0.11217546463012695, 0.10506725311279297, 0.10477375984191895, 0.1093754768371582, 0.10466790199279785, 0.10558319091796875, 0.10593891143798828, 0.10639357566833496, 0.11352658271789551, 0.10525321960449219, 0.10449528694152832, 0.11040139198303223, 0.10512542724609375, 0.10872268676757812, 0.10476088523864746, 0.10410857200622559, 0.10444808006286621, 0.11762785911560059, 0.10596966743469238, 0.10543632507324219, 0.10467052459716797, 0.10532522201538086, 0.11772608757019043, 0.11504340171813965, 0.10590624809265137, 0.10484552383422852, 0.10605907440185547, 0.11806845664978027, 0.10520124435424805, 0.10500574111938477, 0.10493326187133789, 0.10557341575622559, 0.11082077026367188, 0.1095118522644043, 0.10611248016357422, 0.1053466796875, 0.10494732856750488, 0.11219024658203125, 0.1049051284790039, 0.10479521751403809, 0.10493826866149902, 0.10640907287597656, 0.10960865020751953, 0.10495662689208984, 0.10895943641662598, 0.10944986343383789, 0.10645890235900879, 0.11363387107849121, 0.1114664077758789, 0.11333584785461426, 0.11236310005187988, 0.11164474487304688, 0.1399674415588379, 0.1068577766418457, 0.10547447204589844, 0.11194658279418945, 0.10977983474731445, 0.10594701766967773, 0.10588431358337402, 0.10453605651855469, 0.10496783256530762, 0.1134641170501709, 0.10494756698608398, 0.10537052154541016, 0.10463762283325195, 0.10503864288330078, 0.11504173278808594, 0.1069791316986084, 0.10484886169433594, 0.10356307029724121, 0.10628652572631836, 0.1133737564086914, 0.10347533226013184, 0.11090540885925293, 0.11142110824584961, 0.10539817810058594, 0.11167383193969727, 0.10976409912109375, 0.11992645263671875, 0.11177659034729004, 0.11229491233825684, 0.12412142753601074, 0.13331246376037598, 0.201202392578125, 0.26174044609069824, 0.13320016860961914, 0.10646748542785645, 0.10512924194335938, 0.11827445030212402, 0.2412266731262207, 0.23436641693115234, 0.21930241584777832, 0.2705669403076172, 0.11951828002929688, 0.10457491874694824, 0.10374641418457031, 0.10909461975097656, 0.11663317680358887, 0.10547041893005371, 0.10470390319824219, 0.10354399681091309, 0.10526204109191895, 0.10920524597167969, 0.1183319091796875, 0.11485099792480469, 0.11091327667236328, 0.11638140678405762, 0.12125110626220703, 0.1061854362487793, 0.1053166389465332, 0.10446858406066895, 0.1047067642211914, 0.11705422401428223, 0.10906815528869629, 0.11460638046264648, 0.18924951553344727, 0.2684516906738281, 0.10644316673278809, 0.16692376136779785, 0.2731454372406006, 0.11684489250183105, 0.10557293891906738, 0.11746025085449219, 0.10480046272277832, 0.10557103157043457, 0.10505151748657227, 0.10517430305480957, 0.11542177200317383, 0.1052396297454834, 0.1044919490814209, 0.10412716865539551, 0.10955548286437988, 0.11322808265686035, 0.10553956031799316, 0.10495638847351074, 0.1044001579284668, 0.1050574779510498, 0.11043071746826172, 0.10530304908752441, 0.10501933097839355, 0.10454988479614258, 0.10476851463317871, 0.10918831825256348, 0.1176309585571289, 0.10496664047241211, 0.11242198944091797, 0.28258538246154785, 0.10889601707458496, 0.2804384231567383, 0.1624748706817627, 0.11749887466430664, 0.1040041446685791, 0.21312618255615234, 0.2510237693786621, 0.23308944702148438, 0.24852776527404785, 0.17520546913146973, 0.3372666835784912, 0.10798215866088867, 0.11751770973205566, 0.12595725059509277, 0.1045541763305664, 0.10373711585998535, 0.10378527641296387, 0.11642622947692871, 0.11620569229125977, 0.10423469543457031, 0.11104369163513184, 0.11202359199523926, 0.11167693138122559, 0.11490535736083984, 0.11287832260131836, 0.10875391960144043, 0.1086735725402832, 0.3634352684020996, 0.10906505584716797, 0.11966729164123535, 0.12520575523376465, 0.11029219627380371, 0.11113119125366211, 0.11097502708435059, 0.11911129951477051, 0.10518622398376465, 0.1142268180847168, 0.10442590713500977, 0.10535025596618652, 0.11070537567138672, 0.10534381866455078, 0.10393929481506348, 0.10326337814331055, 0.1046149730682373, 0.10786724090576172, 0.1038510799407959, 0.10379219055175781, 0.10362434387207031, 0.11301565170288086, 0.1098330020904541, 0.10493731498718262, 0.1035759449005127, 0.10375189781188965, 0.10427474975585938, 0.10935115814208984, 0.10490798950195312, 0.10475039482116699, 0.11462211608886719, 0.10532689094543457, 0.11565756797790527, 0.10362029075622559, 0.10366392135620117, 0.10343050956726074, 0.10479211807250977, 0.11326241493225098, 0.1041572093963623, 0.10411429405212402, 0.10364460945129395, 0.10490202903747559, 0.1121513843536377, 0.11295270919799805, 0.10920214653015137, 0.1089773178100586, 0.10895466804504395, 0.11916589736938477, 0.1088402271270752, 0.10869646072387695, 0.10853123664855957, 0.10843706130981445, 0.1150972843170166, 0.10810470581054688, 0.11285829544067383, 0.11043858528137207, 0.11965608596801758, 0.10918426513671875, 0.10892081260681152, 0.10794973373413086, 0.10972309112548828, 0.11472630500793457, 0.10826444625854492, 0.1086127758026123, 0.10779047012329102, 0.1146237850189209, 0.12520551681518555, 0.11723542213439941, 0.10990571975708008, 0.10900402069091797, 0.1080780029296875, 0.11766886711120605, 0.11516952514648438, 0.11766886711120605, 0.11313390731811523, 0.1330397129058838, 0.1159975528717041, 0.11487913131713867, 0.10829520225524902, 0.1098177433013916, 0.11924076080322266, 0.10879087448120117, 0.10801386833190918, 0.10863447189331055, 0.1091458797454834, 0.11796283721923828, 0.12335777282714844, 0.10866856575012207, 0.1092672348022461, 0.11358428001403809, 0.10783267021179199, 0.11479640007019043, 0.10904121398925781, 0.10875439643859863, 0.11638522148132324, 0.1086273193359375, 0.10841965675354004, 0.113189697265625, 0.10943198204040527, 0.11295890808105469, 0.1079554557800293, 0.10778522491455078, 0.10830545425415039, 0.10755419731140137, 0.1172027587890625, 0.10861420631408691, 0.10801935195922852, 0.11667013168334961, 0.10961627960205078, 0.11164140701293945, 0.10880208015441895, 0.109710693359375, 0.11268377304077148, 0.12069034576416016, 0.10852193832397461, 0.1111302375793457, 0.1145164966583252, 0.11214351654052734, 0.12143373489379883, 0.10889720916748047, 0.1118927001953125, 0.11108708381652832, 0.10924839973449707, 0.11528968811035156, 0.10796356201171875, 0.10924720764160156, 0.1104893684387207, 0.11487102508544922, 0.10837864875793457, 0.11273622512817383, 0.10877299308776855, 0.10874700546264648, 0.1168985366821289, 0.1084740161895752, 0.10847687721252441, 0.11136460304260254, 0.11360692977905273, 0.11295795440673828, 0.10796666145324707, 0.10838603973388672, 0.11760258674621582, 0.11054444313049316, 0.11385488510131836, 0.10936474800109863, 0.10821294784545898, 0.10825848579406738, 0.11169958114624023, 0.10911011695861816, 0.10845708847045898, 0.10847139358520508, 0.1092844009399414, 0.1220238208770752, 0.10859894752502441, 0.10864973068237305, 0.10818862915039062, 0.10948896408081055, 0.11230349540710449, 0.10837268829345703, 0.10878586769104004, 0.10818648338317871, 0.10899138450622559, 0.11747169494628906, 0.11335229873657227, 0.10934758186340332, 0.10912728309631348, 0.10859394073486328, 0.11196374893188477, 0.10828113555908203, 0.10767030715942383, 0.10835146903991699, 0.11829090118408203, 0.10971927642822266, 0.11538863182067871, 0.10894203186035156, 0.10930991172790527, 0.1135096549987793, 0.10874819755554199, 0.10844707489013672, 0.10792303085327148, 0.10817670822143555, 0.11661815643310547, 0.10870838165283203, 0.10857725143432617, 0.11453652381896973, 0.10898971557617188, 0.11181759834289551, 0.10840415954589844, 0.10825943946838379, 0.10942935943603516, 0.10935521125793457, 0.11835098266601562, 0.10880231857299805, 0.1083676815032959, 0.1079719066619873, 0.11327958106994629, 0.111480712890625, 0.10806798934936523, 0.10822677612304688, 0.10863971710205078, 0.11656475067138672, 0.10870885848999023, 0.10917925834655762, 0.10836982727050781, 0.10964059829711914, 0.11505460739135742, 0.11288046836853027, 0.10809516906738281, 0.10854315757751465, 0.1080789566040039, 0.1205453872680664, 0.10841488838195801, 0.10840106010437012, 0.10944199562072754, 0.10878133773803711, 0.11890292167663574, 0.10813188552856445, 0.11974740028381348, 0.10991406440734863, 0.12246203422546387, 0.10891532897949219, 0.10842585563659668, 0.10845160484313965, 0.10897183418273926, 0.11355066299438477, 0.10832810401916504, 0.10815763473510742, 0.1117405891418457, 0.10897612571716309, 0.1158151626586914, 0.10823988914489746, 0.10826730728149414, 0.10829544067382812, 0.10830402374267578, 0.1154022216796875, 0.10854125022888184, 0.10887837409973145, 0.1088712215423584, 0.11441349983215332, 0.1174783706665039, 0.10944890975952148, 0.10870003700256348, 0.1082003116607666, 0.11334848403930664, 0.10850811004638672, 0.10793709754943848, 0.10813546180725098, 0.1084890365600586, 0.11255073547363281, 0.1150047779083252, 0.10951995849609375, 0.1082005500793457, 0.1090855598449707, 0.11356759071350098, 0.1080770492553711, 0.10892486572265625, 0.10784363746643066, 0.10837984085083008, 0.11324381828308105, 0.1088569164276123, 0.11807918548583984, 0.10903072357177734, 0.11390471458435059, 0.10892248153686523, 0.10897302627563477, 0.10769104957580566, 0.10897207260131836, 0.11248135566711426, 0.10894179344177246, 0.10815119743347168, 0.10732054710388184, 0.11727404594421387, 0.11445116996765137, 0.1091454029083252, 0.10824728012084961, 0.10810065269470215, 0.10896873474121094, 0.11356043815612793, 0.10811233520507812, 0.1081857681274414, 0.1105642318725586, 0.10849833488464355, 0.1252596378326416, 0.10769534111022949, 0.10753440856933594, 0.1075141429901123, 0.11314988136291504, 0.10911083221435547, 0.10872554779052734, 0.10760712623596191, 0.10920882225036621, 0.11810731887817383, 0.10823845863342285, 0.11494159698486328, 0.10808873176574707, 0.10868644714355469, 0.11292409896850586, 0.10855221748352051, 0.10804533958435059, 0.10770392417907715, 0.10891962051391602, 0.11749124526977539, 0.10808157920837402, 0.11235499382019043, 0.11165618896484375, 0.1079411506652832, 0.11359691619873047, 0.1079404354095459, 0.10798406600952148, 0.10817360877990723, 0.11560654640197754, 0.1107475757598877, 0.10792875289916992, 0.1075444221496582, 0.11260032653808594, 0.11200070381164551, 0.10917472839355469, 0.10883474349975586, 0.1083366870880127, 0.10888004302978516, 0.11578154563903809, 0.10801148414611816, 0.10846734046936035, 0.10791301727294922, 0.10861325263977051, 0.11812829971313477, 0.10914468765258789, 0.10913276672363281, 0.10780644416809082, 0.11002707481384277, 0.12060022354125977, 0.11242103576660156, 0.11376595497131348, 0.10944342613220215, 0.11342620849609375, 0.1085355281829834, 0.11133408546447754, 0.10916662216186523, 0.10881924629211426, 0.11574625968933105, 0.10801959037780762, 0.10861706733703613, 0.10804033279418945, 0.10852694511413574, 0.11331295967102051, 0.10782051086425781, 0.10837841033935547, 0.11219024658203125, 0.10868501663208008, 0.11753559112548828, 0.10861015319824219, 0.10808658599853516, 0.1085348129272461, 0.10869884490966797, 0.12703776359558105, 0.11166644096374512, 0.10771393775939941, 0.10825538635253906, 0.13406062126159668, 0.10867881774902344, 0.1109464168548584, 0.11261272430419922, 0.11179065704345703, 0.11868977546691895, 0.10841846466064453, 0.11211872100830078, 0.11541533470153809, 0.11208581924438477, 0.11971831321716309, 0.10983753204345703, 0.10828089714050293, 0.10897541046142578, 0.11433625221252441, 0.10942363739013672, 0.10881567001342773, 0.10818171501159668, 0.10881614685058594, 0.11203336715698242, 0.10876154899597168, 0.11656951904296875, 0.10896658897399902, 0.10935401916503906, 0.11469841003417969, 0.10921669006347656, 0.1080021858215332, 0.1135871410369873, 0.1141958236694336, 0.11969232559204102, 0.11683154106140137, 0.11356997489929199, 0.12357902526855469, 0.11885643005371094, 0.10892987251281738, 0.11272096633911133, 0.11295914649963379, 0.10846519470214844, 0.11390805244445801, 0.11026978492736816, 0.11026930809020996, 0.10887861251831055, 0.1162557601928711, 0.11412358283996582, 0.10909724235534668, 0.10848212242126465, 0.10816121101379395, 0.11282062530517578, 0.1091001033782959, 0.1095881462097168, 0.12017154693603516, 0.1157999038696289, 0.12694811820983887, 0.1213219165802002, 0.11669754981994629, 0.11366486549377441, 0.11827325820922852, 0.12375617027282715, 0.11115384101867676, 0.11765837669372559, 0.1150367259979248, 0.6094791889190674, 0.12791013717651367, 0.12456345558166504, 0.15335679054260254, 0.11804986000061035, 0.11135697364807129, 0.10894036293029785, 0.11444997787475586, 0.31100940704345703, 0.1331791877746582, 0.2777400016784668, 0.15850830078125, 0.10931777954101562, 0.27473878860473633, 0.19142365455627441, 0.14392995834350586, 0.26662492752075195, 0.10922384262084961, 0.10874104499816895, 0.10826730728149414, 0.10849332809448242, 0.1087644100189209, 0.11033463478088379, 0.10955214500427246, 0.10900306701660156, 0.10895180702209473, 0.10896539688110352, 0.1926407814025879, 0.334552526473999, 0.15218400955200195, 0.11944222450256348, 0.10878443717956543, 0.10908985137939453, 0.17301011085510254, 0.29779601097106934, 0.120697021484375, 0.10864448547363281, 0.10824108123779297, 0.10976171493530273, 0.10958147048950195, 0.10836625099182129, 0.1082756519317627, 0.10797715187072754, 0.10926485061645508, 0.1092371940612793, 0.1082620620727539, 0.10816311836242676, 0.10807228088378906, 0.10823178291320801, 0.10862421989440918, 0.1078338623046875, 0.10820245742797852, 0.10796642303466797, 0.10818004608154297, 0.10804605484008789, 0.10786676406860352, 0.10809493064880371, 0.1085824966430664, 0.10815787315368652, 0.10885787010192871, 0.10898232460021973, 0.10841894149780273, 0.10913372039794922, 0.10957527160644531, 0.10982465744018555, 0.1078791618347168, 0.10851144790649414, 0.10903716087341309, 0.10818934440612793, 0.1083836555480957, 0.10796260833740234, 0.10903429985046387, 0.1100621223449707, 0.10895967483520508, 0.1078341007232666, 0.10849595069885254, 0.10886096954345703, 0.10900449752807617, 0.10899972915649414, 0.108184814453125, 0.1079709529876709, 0.10814166069030762, 0.10895061492919922, 0.1089475154876709, 0.10808205604553223, 0.10782957077026367, 0.10834145545959473, 0.10872960090637207, 0.1094980239868164, 0.10777735710144043, 0.10786080360412598, 0.10778164863586426, 0.10956811904907227, 0.11032247543334961, 0.10854315757751465, 0.10892677307128906, 0.10892033576965332, 0.10908794403076172, 0.10929274559020996, 0.1084129810333252, 0.10788416862487793, 0.10904121398925781, 0.10846233367919922, 0.10905003547668457, 0.10930132865905762, 0.10774755477905273, 0.11529421806335449, 0.10870170593261719, 0.11226201057434082, 0.1097557544708252, 0.11689615249633789, 0.11511707305908203, 0.11253190040588379, 0.1137838363647461, 0.11795186996459961, 0.11332392692565918, 0.10997152328491211, 0.10897064208984375, 0.10978484153747559, 0.11401057243347168, 0.11128377914428711, 0.11422920227050781, 0.11530542373657227, 0.10826516151428223, 0.11191558837890625, 0.11127972602844238, 0.11054110527038574, 0.10881900787353516, 0.10834860801696777, 0.11135506629943848, 0.10907173156738281, 0.10881352424621582, 0.1081092357635498, 0.1082158088684082, 0.10895848274230957, 0.10842251777648926, 0.10843658447265625, 0.1096184253692627, 0.1083230972290039, 0.11010479927062988, 0.10857748985290527, 0.10884308815002441, 0.10770750045776367, 0.10827255249023438, 0.1085813045501709, 0.1085519790649414, 0.1091763973236084, 0.10837244987487793, 0.11021232604980469, 0.1152498722076416, 0.11452746391296387, 0.11241817474365234, 0.10968971252441406, 0.10824012756347656, 0.11458015441894531, 0.11507081985473633, 0.1134951114654541, 0.11119270324707031, 0.11061501502990723, 0.10951828956604004, 0.10992908477783203, 0.10917520523071289, 0.10927391052246094, 0.1085197925567627, 0.10902237892150879, 0.10837149620056152, 0.11714816093444824, 0.10960102081298828, 0.10771560668945312, 0.10883355140686035, 0.1088564395904541, 0.11018204689025879, 0.1093900203704834, 0.10933089256286621, 0.11467266082763672, 0.10892009735107422, 0.11036467552185059, 0.10831618309020996, 0.10946774482727051, 0.10839343070983887, 0.11294126510620117, 0.11103153228759766, 0.1082773208618164, 0.11034989356994629, 0.11657118797302246, 0.11710214614868164, 0.11771583557128906, 0.11444234848022461, 0.1109013557434082, 0.10865211486816406, 0.11488175392150879, 0.1176760196685791, 0.11765050888061523, 0.11127185821533203, 0.27820324897766113, 0.23694777488708496, 0.11403417587280273, 0.3615446090698242, 0.11475896835327148, 0.27032995223999023, 0.13683819770812988, 0.3115200996398926, 0.13890290260314941, 0.31390833854675293, 0.12293815612792969, 0.10909032821655273, 0.2565431594848633, 0.36156225204467773, 0.3143627643585205, 0.32112669944763184, 0.11296892166137695, 0.11128616333007812, 0.1142721176147461, 0.12896728515625, 0.12814640998840332, 0.10810351371765137, 0.10859012603759766, 0.10848522186279297, 0.10838723182678223, 0.10832929611206055, 0.10808014869689941, 0.1076512336730957, 0.10872554779052734, 0.10818076133728027, 0.10753941535949707, 0.10778284072875977, 0.10741114616394043, 0.12575125694274902, 0.10878753662109375, 0.1084282398223877, 0.10977315902709961, 0.10798430442810059, 0.10882735252380371, 0.10850071907043457, 0.1085965633392334, 0.10813140869140625, 0.10773015022277832, 0.10787463188171387, 0.11150908470153809, 0.11939764022827148, 0.13901281356811523, 0.10881328582763672, 0.10836577415466309, 0.11118245124816895, 0.11100244522094727, 0.10803103446960449, 0.10897970199584961, 0.11056065559387207, 0.11203408241271973, 0.11300802230834961, 0.11261272430419922, 0.11256837844848633, 0.11249399185180664, 0.11191463470458984, 0.10903263092041016, 0.11214733123779297, 0.11010861396789551, 0.10901927947998047, 0.10916781425476074, 0.10821413993835449, 0.10844564437866211, 0.1089484691619873, 0.10885214805603027, 0.11168718338012695, 0.10780596733093262, 0.10765957832336426, 0.10749530792236328, 0.10921335220336914, 0.11014819145202637, 0.10764908790588379, 0.10820960998535156, 0.10786581039428711, 0.10794949531555176, 0.10732913017272949, 0.10791635513305664, 0.10918903350830078, 0.10874176025390625, 0.10847187042236328, 0.10828757286071777, 0.1084907054901123, 0.10800361633300781, 0.1100308895111084, 0.11285209655761719, 0.11161541938781738, 0.11284756660461426, 0.11213088035583496, 0.11294031143188477, 0.11163020133972168, 0.1109311580657959, 0.1092841625213623, 0.11773490905761719, 0.11193060874938965, 0.11435627937316895, 0.11211991310119629, 0.10852265357971191, 0.11852478981018066, 0.10873126983642578, 0.108978271484375, 0.1105036735534668, 0.10915350914001465, 0.11401081085205078, 0.11257648468017578, 0.10933899879455566, 0.10898542404174805, 0.11023187637329102, 0.11656713485717773, 0.10868525505065918, 0.10831236839294434, 0.1087641716003418, 0.1147913932800293, 0.11080002784729004, 0.1143805980682373, 0.13985562324523926, 0.1097719669342041, 0.11356568336486816, 0.10870718955993652, 0.10831284523010254, 0.11026859283447266, 0.10837459564208984, 0.12085938453674316, 0.10904598236083984, 0.1077876091003418, 0.11943626403808594, 0.12214469909667969, 0.13123464584350586, 0.11187624931335449, 0.10910320281982422, 0.1091165542602539, 0.11490726470947266, 0.11069273948669434, 0.10873651504516602, 0.11016726493835449, 0.29583096504211426, 0.2546350955963135, 0.14483642578125, 0.1098790168762207, 0.36046481132507324, 0.1871345043182373, 0.12458014488220215, 0.1111137866973877, 0.10915637016296387, 0.11873102188110352, 0.1080784797668457, 0.1218407154083252, 0.13618707656860352, 0.11757802963256836, 0.10949873924255371, 0.10951519012451172, 0.11508750915527344, 0.10878515243530273, 0.10957527160644531, 0.10946178436279297, 0.10905957221984863, 0.1132056713104248, 0.11943173408508301, 0.11631369590759277, 0.10877180099487305, 0.11018848419189453, 0.11929750442504883, 0.11377739906311035, 0.10881614685058594, 0.12018322944641113, 0.11511898040771484, 0.10917186737060547, 0.11050033569335938, 0.11379718780517578, 0.10886454582214355, 0.11692595481872559, 0.11296677589416504, 0.10926127433776855, 0.11195731163024902, 0.11119318008422852, 0.11389875411987305, 0.10937261581420898, 0.10908317565917969, 0.11268377304077148, 0.12886977195739746, 0.11239242553710938, 0.11200141906738281, 0.11560440063476562, 0.1125037670135498, 0.11759567260742188, 0.10988950729370117, 0.10825467109680176, 0.1103525161743164, 0.14067459106445312, 0.1228938102722168, 0.1286330223083496, 0.10935473442077637, 0.11640501022338867, 0.11500859260559082, 0.10848164558410645, 0.11013340950012207, 0.10856080055236816, 0.11469173431396484, 0.11037921905517578, 0.11691617965698242, 0.1099860668182373, 0.11375713348388672, 0.10934305191040039, 0.10893988609313965, 0.10939645767211914, 0.11088728904724121, 0.11488556861877441, 0.11023783683776855, 0.1093590259552002, 0.10898089408874512, 0.11987590789794922, 0.11514878273010254, 0.10921478271484375, 0.10919761657714844, 0.10882353782653809, 0.10987472534179688, 0.11330628395080566, 0.10908627510070801, 0.10928511619567871, 0.10921430587768555, 0.1172633171081543, 0.11452937126159668, 0.10906696319580078, 0.10897994041442871, 0.10875391960144043, 0.11416363716125488, 0.10935163497924805, 0.10856080055236816, 0.10892438888549805, 0.10891985893249512, 0.11761045455932617, 0.10882329940795898, 0.11489033699035645, 0.10941863059997559, 0.10923314094543457, 0.11290216445922852, 0.10916256904602051, 0.1082613468170166, 0.10958242416381836, 0.10885190963745117, 0.1177828311920166, 0.10868549346923828, 0.10857915878295898, 0.11996793746948242, 0.11736130714416504, 0.1091303825378418, 0.10880565643310547, 0.10871505737304688, 0.10885739326477051, 0.11835765838623047, 0.1098482608795166, 0.10866498947143555, 0.10927438735961914, 0.11578130722045898, 0.11260724067687988, 0.10895609855651855, 0.10887384414672852, 0.1086878776550293, 0.10965800285339355, 0.11636066436767578, 0.10914945602416992, 0.10976719856262207, 0.10918617248535156, 0.11331057548522949, 0.11246514320373535, 0.10924530029296875, 0.10893058776855469, 0.1092529296875, 0.11799240112304688, 0.10918140411376953, 0.10872340202331543, 0.11207699775695801, 0.12051701545715332, 0.17720484733581543, 0.19650816917419434, 0.1842496395111084, 0.1398916244506836, 0.10828399658203125, 0.10857081413269043, 0.12709259986877441, 0.20282649993896484, 0.15785884857177734, 0.1353156566619873, 0.12745332717895508, 0.12958312034606934, 0.12695622444152832, 0.11480212211608887, 0.1089177131652832, 0.11123180389404297, 0.1092381477355957, 0.11004900932312012, 0.10944008827209473, 0.11253738403320312, 0.10954427719116211, 0.10956072807312012, 0.10928869247436523, 0.10867738723754883, 0.10977840423583984, 0.10878634452819824, 0.10885381698608398, 0.10876297950744629, 0.10914254188537598, 0.11171221733093262, 0.10995078086853027, 0.10881328582763672, 0.10887622833251953, 0.10912847518920898, 0.10988497734069824, 0.10924911499023438, 0.1094207763671875, 0.10890340805053711, 0.1099386215209961, 0.11024594306945801, 0.10905790328979492, 0.11026787757873535, 0.10933566093444824, 0.1098942756652832, 0.10953092575073242, 0.10874152183532715, 0.1089930534362793, 0.10959625244140625, 0.11026859283447266, 0.11043190956115723, 0.1091160774230957, 0.10891509056091309, 0.11028456687927246, 0.10985350608825684, 0.10972309112548828, 0.1089792251586914, 0.10921216011047363, 0.1100306510925293, 0.11047148704528809, 0.11420011520385742, 0.10908770561218262, 0.10881710052490234, 0.1088402271270752, 0.11163949966430664, 0.10996532440185547, 0.10919952392578125, 0.1092381477355957, 0.10867834091186523, 0.10988855361938477, 0.10910367965698242, 0.1086876392364502, 0.10915946960449219, 0.10909700393676758, 0.10977363586425781, 0.10976004600524902, 0.10957455635070801, 0.10904741287231445, 0.11009716987609863, 0.10938143730163574, 0.1090402603149414, 0.10888385772705078, 0.10882425308227539, 0.1098945140838623, 0.10870361328125, 0.1094961166381836, 0.11006522178649902, 0.11088681221008301, 0.11154675483703613, 0.10880112648010254, 0.10878872871398926, 0.10852813720703125, 0.10968255996704102, 0.1111598014831543, 0.10965418815612793, 0.10864639282226562, 0.10828113555908203, 0.11172342300415039, 0.11115503311157227, 0.10902690887451172, 0.1082460880279541, 0.10853767395019531, 0.10841822624206543, 0.11038637161254883, 0.10835099220275879, 0.10862874984741211, 0.10898876190185547, 0.10930800437927246, 0.11024808883666992, 0.10834813117980957, 0.109619140625, 0.10920429229736328, 0.11077427864074707, 0.10466837882995605, 0.10422658920288086, 0.10449910163879395, 0.10517597198486328, 0.10491013526916504, 0.10458946228027344, 0.10554695129394531, 0.10448527336120605, 0.10514521598815918, 0.1053168773651123, 0.10419940948486328, 0.10491275787353516, 0.10357141494750977, 0.10485053062438965, 0.10493683815002441, 0.10514140129089355, 0.10464119911193848, 0.10508918762207031, 0.10515427589416504, 0.10489845275878906, 0.10506367683410645, 0.10558056831359863, 0.1047067642211914, 0.10457158088684082, 0.10373139381408691, 0.10602402687072754, 0.10433650016784668, 0.10362601280212402, 0.10475564002990723, 0.10630130767822266, 0.10586357116699219, 0.10478448867797852, 0.1039588451385498, 0.10367989540100098, 0.10512137413024902, 0.10581636428833008, 0.10428524017333984, 0.1044149398803711, 0.10393667221069336, 0.10394883155822754, 0.10651397705078125, 0.10485291481018066, 0.10433578491210938, 0.10372400283813477, 0.10543680191040039, 0.10602426528930664, 0.10416865348815918, 0.10443329811096191, 0.10373044013977051, 0.10501623153686523, 0.10556960105895996, 0.10434246063232422, 0.10579133033752441, 0.10432744026184082, 0.10526084899902344, 0.10571789741516113, 0.10433292388916016, 0.10404658317565918, 0.10357832908630371, 0.10534405708312988, 0.10556983947753906, 0.10416221618652344, 0.1034250259399414, 0.10702204704284668, 0.10430145263671875, 0.1058962345123291, 0.10424661636352539, 0.10347676277160645, 0.1037912368774414, 0.10398411750793457, 0.10728096961975098, 0.10469198226928711, 0.10438728332519531, 0.10561466217041016, 0.10614871978759766, 0.10938215255737305, 0.10427737236022949, 0.10419225692749023, 0.10422062873840332, 0.104583740234375, 0.10583710670471191, 0.10370039939880371, 0.10402512550354004, 0.10312843322753906, 0.10471248626708984, 0.10593724250793457, 0.10659503936767578, 0.10500001907348633, 0.1044468879699707, 0.10527443885803223, 0.1066131591796875, 0.10410952568054199, 0.1051321029663086, 0.10388708114624023, 0.10529303550720215, 0.1050877571105957, 0.10525012016296387, 0.10445570945739746, 0.10717988014221191, 0.10516166687011719, 0.10585546493530273, 0.10491299629211426, 0.10421538352966309, 0.10350561141967773, 0.10500216484069824, 0.10613489151000977, 0.10482954978942871, 0.10412764549255371, 0.10429501533508301, 0.10765457153320312, 0.1048738956451416, 0.10755157470703125, 0.10455441474914551, 0.10374331474304199, 0.10474371910095215, 0.10627007484436035, 0.10518836975097656, 0.10444879531860352, 0.10419583320617676, 0.10546278953552246, 0.10635113716125488, 0.10622763633728027, 0.10475969314575195, 0.10403299331665039, 0.10431194305419922, 0.10402274131774902, 0.10627985000610352, 0.10452604293823242, 0.10402441024780273, 0.10389542579650879, 0.10400629043579102, 0.10582113265991211, 0.10746049880981445, 0.10649704933166504, 0.10418319702148438, 0.10404849052429199, 0.10674786567687988, 0.10419964790344238, 0.10390043258666992, 0.10413765907287598, 0.1057891845703125, 0.10722780227661133, 0.10439467430114746, 0.10565948486328125, 0.10523557662963867, 0.10407376289367676, 0.10556197166442871, 0.10437870025634766, 0.10439896583557129, 0.10422205924987793, 0.10464072227478027, 0.10557365417480469, 0.1047060489654541, 0.10378718376159668, 0.10396432876586914, 0.10646939277648926, 0.10603833198547363, 0.10422158241271973, 0.10399675369262695, 0.10377669334411621, 0.10410857200622559, 0.10579490661621094, 0.10444951057434082, 0.10433435440063477, 0.10566902160644531, 0.10515713691711426, 0.10661029815673828, 0.1046597957611084, 0.10470724105834961, 0.10470247268676758, 0.10535025596618652, 0.10660982131958008, 0.10393261909484863, 0.10388016700744629, 0.10322427749633789, 0.10454940795898438, 0.1054379940032959, 0.10444164276123047, 0.1065521240234375, 0.10446596145629883, 0.10538792610168457, 0.10690879821777344, 0.10443401336669922, 0.10404419898986816, 0.10391879081726074, 0.10431051254272461, 0.10596895217895508, 0.10481381416320801, 0.10416150093078613, 0.1059267520904541, 0.1047821044921875, 0.10581302642822266, 0.10495185852050781, 0.1041107177734375, 0.10396003723144531, 0.10400128364562988, 0.1054074764251709, 0.10485076904296875, 0.10466957092285156, 0.10411453247070312, 0.10582518577575684, 0.10553240776062012, 0.10570406913757324, 0.10460758209228516, 0.10467696189880371, 0.10454344749450684, 0.10522794723510742, 0.10413599014282227, 0.1037592887878418, 0.10347795486450195, 0.10378551483154297, 0.10428905487060547, 0.10729765892028809, 0.10461568832397461, 0.10424256324768066, 0.10471892356872559, 0.10346817970275879, 0.10775542259216309, 0.10454463958740234, 0.10376930236816406, 0.10539507865905762, 0.10396194458007812, 0.10669159889221191, 0.10519742965698242, 0.10499715805053711, 0.10515570640563965, 0.11507415771484375, 0.10679125785827637, 0.10993576049804688, 0.11574029922485352, 0.1164097785949707, 0.11922597885131836, 0.11286211013793945, 0.10860729217529297, 0.1065518856048584, 0.10413193702697754, 0.1060035228729248, 0.10413575172424316, 0.1475386619567871, 0.16733765602111816, 0.11595726013183594, 0.13131976127624512, 0.10857748985290527, 0.10694551467895508, 0.11061453819274902, 0.10992550849914551, 0.10520482063293457, 0.10384893417358398, 0.10333466529846191, 0.10503053665161133, 0.10365104675292969, 0.10314083099365234, 0.10369348526000977, 0.10349345207214355, 0.10337591171264648, 0.10371232032775879, 0.10374236106872559, 0.10339975357055664, 0.10291337966918945, 0.10392332077026367, 0.1161503791809082, 0.10624814033508301, 0.10505533218383789, 0.10373616218566895, 0.10358190536499023, 0.10771489143371582, 0.11062884330749512, 0.11257219314575195, 0.11592721939086914, 0.10572361946105957, 0.10718774795532227, 0.10326027870178223, 0.10303616523742676, 0.10915923118591309, 0.13511896133422852, 0.1414656639099121, 0.12870097160339355, 0.10485029220581055, 0.10321545600891113, 0.10418581962585449, 0.10393238067626953, 0.10332465171813965, 0.10436773300170898, 0.10405158996582031, 0.10500526428222656, 0.10390806198120117, 0.10400032997131348, 0.10285830497741699, 0.11471724510192871, 0.11392474174499512, 0.11689400672912598, 0.11105132102966309, 0.1148371696472168, 0.10880255699157715, 0.10354757308959961, 0.10376262664794922, 0.10365962982177734, 0.10314440727233887, 0.10442042350769043, 0.10314059257507324, 0.10266637802124023, 0.10381841659545898, 0.10478711128234863, 0.10342741012573242, 0.10333824157714844, 0.10361719131469727, 0.10309481620788574, 0.10300230979919434, 0.10416698455810547, 0.10921263694763184, 0.12073063850402832, 0.11776232719421387, 0.1160118579864502, 0.10574865341186523, 0.1066141128540039, 0.10490775108337402, 0.1045997142791748, 0.1048882007598877, 0.1045985221862793, 0.10769414901733398, 0.10458803176879883, 0.10688090324401855, 0.10432600975036621, 0.10550427436828613, 0.10544443130493164, 0.10556268692016602, 0.10570216178894043, 0.10372662544250488, 0.10433554649353027, 0.10634183883666992, 0.10521125793457031, 0.1054544448852539, 0.10598564147949219, 0.1058192253112793, 0.10779976844787598, 0.10593271255493164, 0.1058509349822998, 0.10455036163330078, 0.10743427276611328, 0.10681319236755371, 0.10490179061889648, 0.10439682006835938, 0.10391879081726074, 0.10749697685241699, 0.10618829727172852, 0.10648274421691895, 0.10443305969238281, 0.10582709312438965, 0.10415792465209961, 0.10461926460266113, 0.10498499870300293, 0.10417771339416504, 0.10446882247924805, 0.10379242897033691, 0.10371541976928711, 0.10703587532043457, 0.104583740234375, 0.10396814346313477, 0.10404634475708008, 0.1036992073059082, 0.10598421096801758, 0.12534141540527344, 0.10565304756164551, 0.10468912124633789, 0.1055138111114502, 0.10366582870483398, 0.11261534690856934, 0.10492205619812012, 0.10412144660949707, 0.10386061668395996, 0.10461568832397461, 0.11025738716125488, 0.10618853569030762, 0.10536050796508789, 0.1065673828125, 0.10460615158081055, 0.11304759979248047, 0.10742831230163574, 0.10672950744628906, 0.1082916259765625, 0.10616636276245117, 0.11142945289611816, 0.1073155403137207, 0.10613250732421875, 0.10926413536071777, 0.10534977912902832, 0.10514497756958008, 0.10699748992919922, 0.10853981971740723, 0.10805988311767578, 0.10488581657409668, 0.10798311233520508, 0.10937261581420898, 0.10503363609313965, 0.10449767112731934, 0.10750770568847656, 0.10500788688659668, 0.11430478096008301, 0.10784554481506348, 0.10958695411682129, 0.11028218269348145, 0.10993051528930664, 0.10682964324951172, 0.10869264602661133, 0.10528779029846191, 0.11319303512573242, 0.10441040992736816, 0.10511207580566406, 0.10772085189819336, 0.11097049713134766, 0.10823202133178711, 0.1072225570678711, 0.10438919067382812, 0.10842275619506836, 0.10825753211975098, 0.10924959182739258, 0.10729718208312988, 0.10437870025634766, 0.10552024841308594, 0.10375666618347168, 0.10880661010742188, 0.10710525512695312, 0.11089468002319336, 0.11229515075683594, 0.10508894920349121, 0.10465812683105469, 0.10399889945983887, 0.10599112510681152, 0.10687875747680664, 0.10485196113586426, 0.10425639152526855, 0.10361599922180176, 0.10496068000793457, 0.10808873176574707, 0.10463380813598633, 0.10889339447021484, 0.11275124549865723, 0.1082906723022461, 0.11179995536804199, 0.11294317245483398, 0.10549473762512207, 0.1037142276763916, 0.10479521751403809, 0.10633468627929688, 0.10605907440185547, 0.1050422191619873, 0.1045222282409668, 0.10408902168273926, 0.10561275482177734, 0.10469985008239746, 0.10452055931091309, 0.10433554649353027, 0.10684466361999512, 0.10549616813659668, 0.10442209243774414, 0.10581517219543457, 0.10645151138305664, 0.10480737686157227, 0.10933494567871094, 0.1072382926940918, 0.10974335670471191, 0.10455322265625, 0.10427522659301758, 0.11330628395080566, 0.11301589012145996, 0.10752534866333008, 0.10349535942077637, 0.1065061092376709, 0.10595178604125977, 0.10519742965698242, 0.10481405258178711, 0.10397195816040039, 0.10474061965942383, 0.10581445693969727, 0.10446906089782715, 0.10408186912536621, 0.10449647903442383, 0.10431456565856934, 0.10646939277648926, 0.10626697540283203, 0.10406851768493652, 0.10913276672363281, 0.10478568077087402, 0.10719656944274902, 0.10483407974243164, 0.1043252944946289, 0.10507917404174805, 0.10477709770202637, 0.10577702522277832, 0.10363507270812988, 0.10710883140563965, 0.10402870178222656, 0.10544681549072266, 0.10699748992919922, 0.1048438549041748, 0.10475516319274902, 0.10401082038879395, 0.10552811622619629, 0.10731148719787598, 0.10561108589172363, 0.10509109497070312, 0.10658812522888184, 0.10867691040039062, 0.1068260669708252, 0.10428690910339355, 0.1049354076385498, 0.10367417335510254, 0.10454607009887695, 0.1068568229675293, 0.10432076454162598, 0.10414004325866699, 0.1042635440826416, 0.10849404335021973, 0.10822677612304688, 0.10608768463134766, 0.10453605651855469, 0.10367584228515625, 0.10517668724060059, 0.10591292381286621, 0.1053321361541748, 0.10430240631103516, 0.10452556610107422, 0.10955643653869629, 0.1069643497467041, 0.10759782791137695, 0.10463333129882812, 0.10417032241821289, 0.10464596748352051, 0.10550427436828613, 0.10452008247375488, 0.10445451736450195, 0.10431838035583496, 0.10463452339172363, 0.10661959648132324, 0.1044459342956543, 0.10586857795715332, 0.10455942153930664, 0.10459113121032715, 0.10533428192138672, 0.10464668273925781, 0.10533261299133301, 0.10437989234924316, 0.10463118553161621, 0.1051340103149414, 0.10403156280517578, 0.10467839241027832, 0.10378360748291016, 0.11533904075622559, 0.10672259330749512, 0.10493254661560059, 0.10489535331726074, 0.10652709007263184, 0.10651397705078125, 0.10695123672485352, 0.10566115379333496, 0.10464262962341309, 0.10455656051635742, 0.10567378997802734, 0.10890722274780273, 0.10606884956359863, 0.10521149635314941, 0.10385012626647949, 0.10578417778015137, 0.10549569129943848, 0.10462069511413574, 0.10422945022583008, 0.10370421409606934, 0.10473799705505371, 0.1052696704864502, 0.10570120811462402, 0.10532355308532715, 0.10457420349121094, 0.10424494743347168, 0.10485553741455078, 0.1049659252166748, 0.10466814041137695, 0.10415148735046387, 0.10474419593811035, 0.1062474250793457, 0.11050653457641602, 0.10422277450561523, 0.10702276229858398, 0.1051945686340332, 0.10564136505126953, 0.10484647750854492, 0.10459184646606445, 0.10410118103027344, 0.10463070869445801, 0.10570883750915527, 0.10500001907348633, 0.10499739646911621, 0.10448241233825684, 0.10554075241088867, 0.10429000854492188, 0.10602021217346191, 0.10609316825866699, 0.1038815975189209, 0.10490632057189941, 0.10414266586303711, 0.10614800453186035, 0.1052558422088623, 0.10377264022827148, 0.10636472702026367, 0.10405564308166504, 0.10898518562316895, 0.10458230972290039, 0.10376334190368652, 0.10513782501220703, 0.10391354560852051, 0.10897397994995117, 0.10521602630615234, 0.10444331169128418, 0.10526919364929199, 0.10398244857788086, 0.10766339302062988, 0.10749316215515137, 0.10526061058044434, 0.1067194938659668, 0.10479593276977539, 0.10682392120361328, 0.10536336898803711, 0.1044161319732666, 0.10444211959838867, 0.10446929931640625, 0.10761523246765137, 0.10479116439819336, 0.10723400115966797, 0.10458111763000488, 0.10546302795410156, 0.10653901100158691, 0.10480070114135742, 0.10400390625, 0.10412406921386719, 0.10410737991333008, 0.10594964027404785, 0.10470962524414062, 0.10419535636901855, 0.10432815551757812, 0.10792922973632812, 0.10648560523986816, 0.10460352897644043, 0.10381364822387695, 0.10392069816589355, 0.10448384284973145, 0.10584092140197754, 0.10678791999816895, 0.10417437553405762, 0.10463738441467285, 0.10564374923706055, 0.10760140419006348, 0.10428047180175781, 0.10437297821044922, 0.10393261909484863, 0.10470795631408691, 0.10660362243652344, 0.1045687198638916, 0.10384297370910645, 0.10532331466674805, 0.10476493835449219, 0.1296985149383545, 0.10890579223632812, 0.12866711616516113, 0.11127638816833496, 0.10762643814086914, 0.10580229759216309, 0.10794687271118164, 0.10431051254272461, 0.10444760322570801, 0.10520267486572266, 0.10625743865966797, 0.1065521240234375, 0.10707855224609375, 0.10682964324951172, 0.10656261444091797, 0.10548853874206543, 0.10515093803405762, 0.10434222221374512, 0.10505962371826172, 0.10470938682556152, 0.10882568359375, 0.11351370811462402, 0.10660743713378906, 0.10649991035461426, 0.10692691802978516, 0.10631132125854492, 0.1047816276550293, 0.10454678535461426, 0.10515499114990234, 0.10509228706359863, 0.10671353340148926, 0.10448050498962402, 0.10408782958984375, 0.10504770278930664, 0.10479187965393066, 0.10739612579345703, 0.10424375534057617, 0.10799121856689453, 0.10504031181335449, 0.10492086410522461, 0.10692286491394043, 0.10499215126037598, 0.10442805290222168, 0.10493969917297363, 0.10556340217590332, 0.10793805122375488, 0.10600781440734863, 0.10496163368225098, 0.10517120361328125, 0.10498833656311035, 0.10968136787414551, 0.1048288345336914, 0.10720014572143555, 0.10533261299133301, 0.10519981384277344, 0.10587406158447266, 0.1059257984161377, 0.10428500175476074, 0.10650205612182617, 0.10429000854492188, 0.10555434226989746, 0.10587620735168457, 0.10387873649597168, 0.10573172569274902, 0.10432863235473633, 0.10623407363891602, 0.10434603691101074, 0.10422015190124512, 0.10382485389709473, 0.1064150333404541, 0.10675811767578125, 0.10471415519714355, 0.10429787635803223, 0.10427975654602051, 0.10461926460266113, 0.10601162910461426, 0.10449576377868652, 0.10595870018005371, 0.10468149185180664, 0.10466551780700684, 0.10787248611450195, 0.10820984840393066, 0.10390639305114746, 0.10472679138183594, 0.10390853881835938, 0.106536865234375, 0.10407853126525879, 0.10398173332214355, 0.10366344451904297, 0.1048574447631836, 0.10732197761535645, 0.10466480255126953, 0.10839986801147461, 0.10496330261230469, 0.10537934303283691, 0.10613441467285156, 0.10487484931945801, 0.10484981536865234, 0.10364127159118652, 0.10460543632507324, 0.10550618171691895, 0.10431861877441406, 0.10404133796691895, 0.10485553741455078, 0.10628485679626465, 0.10705065727233887, 0.10454702377319336, 0.1043696403503418, 0.10395050048828125, 0.10396361351013184, 0.10512042045593262, 0.10472679138183594, 0.10406899452209473, 0.10393571853637695, 0.10462403297424316, 0.10858583450317383, 0.1042938232421875, 0.10400700569152832, 0.10458493232727051, 0.10530638694763184, 0.10698699951171875, 0.10456037521362305, 0.10398125648498535, 0.10423040390014648, 0.10469746589660645, 0.10601925849914551, 0.1054844856262207, 0.10608196258544922, 0.10489964485168457, 0.10478091239929199, 0.10541963577270508, 0.1043550968170166, 0.10491228103637695, 0.10380887985229492, 0.10485386848449707, 0.10619163513183594, 0.10396862030029297, 0.10396790504455566, 0.10656499862670898, 0.10521173477172852, 0.10749936103820801, 0.10416126251220703, 0.10505342483520508, 0.10347485542297363, 0.10508179664611816, 0.10696554183959961, 0.10547590255737305, 0.10454082489013672, 0.1037144660949707, 0.10746049880981445, 0.10663676261901855, 0.10464096069335938, 0.10416841506958008, 0.10392379760742188, 0.10561227798461914, 0.10686254501342773, 0.10536408424377441, 0.10403180122375488, 0.10383987426757812, 0.10516500473022461, 0.10636377334594727, 0.10665106773376465, 0.10420465469360352, 0.10424256324768066, 0.10488033294677734, 0.10657358169555664, 0.10534524917602539, 0.1046438217163086, 0.10428547859191895, 0.10536718368530273, 0.1065521240234375, 0.10576558113098145, 0.10703516006469727, 0.10413146018981934, 0.1052086353302002, 0.10578346252441406, 0.10548520088195801, 0.10452389717102051, 0.10339736938476562, 0.10392951965332031, 0.10359048843383789, 0.1077578067779541, 0.10469293594360352, 0.10576081275939941, 0.10516810417175293, 0.10454201698303223, 0.10712933540344238, 0.10552024841308594, 0.10352897644042969, 0.1042184829711914, 0.10407829284667969, 0.10664749145507812, 0.1043851375579834, 0.10573148727416992, 0.10442137718200684, 0.10694694519042969, 0.10634136199951172, 0.10448288917541504, 0.10464096069335938, 0.10412168502807617, 0.10465288162231445, 0.10638546943664551, 0.10458207130432129, 0.10526752471923828, 0.1048433780670166, 0.10536932945251465, 0.10804080963134766, 0.10505270957946777, 0.10405778884887695, 0.10400986671447754, 0.1047663688659668, 0.10611462593078613, 0.10486149787902832, 0.10504841804504395, 0.1045536994934082, 0.10549592971801758, 0.10763430595397949, 0.10563850402832031, 0.1071622371673584, 0.10543131828308105, 0.10466384887695312, 0.10576248168945312, 0.10449647903442383, 0.1041879653930664, 0.10399127006530762, 0.10452651977539062, 0.10599231719970703, 0.10474443435668945, 0.10373616218566895, 0.10587024688720703, 0.10516738891601562, 0.10596776008605957, 0.10462641716003418, 0.10434579849243164, 0.10414910316467285, 0.10606694221496582, 0.10552501678466797, 0.10472774505615234, 0.10431623458862305, 0.10424375534057617, 0.10621523857116699, 0.10668039321899414, 0.10424256324768066, 0.10445213317871094, 0.10484147071838379, 0.10500335693359375, 0.1069495677947998, 0.10482168197631836, 0.1040503978729248, 0.10389947891235352, 0.1044468879699707, 0.10608911514282227, 0.10664796829223633, 0.10482573509216309, 0.1045985221862793, 0.10432076454162598, 0.10557723045349121, 0.10367989540100098, 0.10424566268920898, 0.10416483879089355, 0.10453462600708008, 0.10573697090148926, 0.10462284088134766, 0.10659980773925781, 0.10417628288269043, 0.10530543327331543, 0.10725831985473633, 0.10496354103088379, 0.10492777824401855, 0.10412311553955078, 0.10797238349914551, 0.10632061958312988, 0.10527253150939941, 0.10472464561462402, 0.10476374626159668, 0.10851025581359863, 0.10647034645080566, 0.10436463356018066, 0.10353660583496094, 0.1045534610748291, 0.10405993461608887, 0.10564041137695312, 0.10578703880310059, 0.10445785522460938, 0.10448646545410156, 0.10466885566711426, 0.10614895820617676, 0.10570120811462402, 0.10522866249084473, 0.10449838638305664, 0.10462641716003418, 0.10469412803649902, 0.10528826713562012, 0.10448813438415527, 0.10436844825744629, 0.1047663688659668, 0.10476851463317871, 0.10633397102355957, 0.10551786422729492, 0.10437536239624023, 0.10459208488464355, 0.10383963584899902, 0.10657024383544922, 0.10664486885070801, 0.10403919219970703, 0.10543656349182129, 0.1038064956665039, 0.1064612865447998, 0.10521554946899414, 0.10751175880432129, 0.10555624961853027, 0.10514044761657715, 0.10643792152404785, 0.1047677993774414, 0.1049354076385498, 0.10466742515563965, 0.10475325584411621, 0.10706305503845215, 0.10442209243774414, 0.10492563247680664, 0.10744452476501465, 0.10516238212585449, 0.10691165924072266, 0.10460829734802246, 0.10412979125976562, 0.10456323623657227, 0.10407114028930664, 0.10611677169799805, 0.10459184646606445, 0.10417938232421875, 0.10558342933654785, 0.10478687286376953, 0.10648918151855469, 0.10439276695251465, 0.10410833358764648, 0.10456418991088867, 0.10525393486022949, 0.1062612533569336, 0.10428142547607422, 0.10414290428161621, 0.10368585586547852, 0.10475277900695801, 0.10597681999206543, 0.10538434982299805, 0.10565710067749023, 0.10442948341369629, 0.10504746437072754, 0.1054227352142334, 0.10431790351867676, 0.10426592826843262, 0.10407209396362305, 0.1051337718963623, 0.10647153854370117, 0.10469794273376465, 0.10586214065551758, 0.10555887222290039, 0.1064443588256836, 0.10754680633544922, 0.10418105125427246, 0.10437321662902832, 0.1040339469909668, 0.1049795150756836, 0.10665106773376465, 0.10460948944091797, 0.10504007339477539, 0.10472941398620605, 0.10799837112426758, 0.1069631576538086, 0.10413932800292969, 0.10384130477905273, 0.10518240928649902, 0.10481095314025879, 0.1068720817565918, 0.10444831848144531, 0.10459041595458984, 0.10426878929138184, 0.1051936149597168, 0.10898447036743164, 0.10522055625915527, 0.1048433780670166, 0.10451269149780273, 0.10506415367126465, 0.10684728622436523, 0.1047813892364502, 0.10482215881347656, 0.10358428955078125, 0.10492467880249023, 0.10563898086547852, 0.10428237915039062, 0.10604071617126465, 0.10408544540405273, 0.10499310493469238, 0.10568380355834961, 0.10457015037536621, 0.10465717315673828, 0.10337686538696289, 0.10516166687011719, 0.1069180965423584, 0.1051785945892334, 0.10472512245178223, 0.10712838172912598, 0.10596275329589844, 0.10667991638183594, 0.10514307022094727, 0.10407209396362305, 0.1046602725982666, 0.10540270805358887, 0.10709238052368164, 0.10535216331481934, 0.10453963279724121, 0.1042625904083252, 0.10585212707519531, 0.1068565845489502, 0.10772180557250977, 0.10461163520812988, 0.10415220260620117, 0.10500216484069824, 0.10574674606323242, 0.10505437850952148, 0.10463714599609375, 0.10413980484008789, 0.10430669784545898, 0.10492157936096191, 0.10734987258911133, 0.10606741905212402, 0.10571646690368652, 0.10589337348937988, 0.1057896614074707, 0.10495352745056152, 0.10425949096679688, 0.10382533073425293, 0.1044459342956543, 0.10384631156921387, 0.1055905818939209, 0.10545587539672852, 0.10561633110046387, 0.10625100135803223, 0.10380935668945312, 0.10718584060668945, 0.10456037521362305, 0.10424375534057617, 0.1051483154296875, 0.10396289825439453, 0.10696697235107422, 0.10563325881958008, 0.10438132286071777, 0.10835957527160645, 0.1048734188079834, 0.10693216323852539, 0.10442543029785156, 0.10448741912841797, 0.10461783409118652, 0.10446929931640625, 0.10651612281799316, 0.10474824905395508, 0.10469555854797363, 0.10465717315673828, 0.10640764236450195, 0.10584759712219238, 0.1040792465209961, 0.1047217845916748, 0.1042029857635498, 0.10425376892089844, 0.10583353042602539, 0.10461711883544922, 0.1045064926147461, 0.1042337417602539, 0.1051332950592041, 0.1064295768737793, 0.10712242126464844, 0.10512661933898926, 0.10445570945739746, 0.10557961463928223, 0.10614681243896484, 0.10366439819335938, 0.10452628135681152, 0.10427522659301758, 0.10509610176086426, 0.10643386840820312, 0.10531806945800781, 0.10570907592773438, 0.10451078414916992, 0.10473179817199707, 0.10595703125, 0.1043844223022461, 0.10538601875305176, 0.10379886627197266, 0.10489416122436523, 0.10527753829956055, 0.10501742362976074, 0.1043996810913086, 0.10562562942504883, 0.10503625869750977, 0.10587096214294434, 0.10442781448364258, 0.104034423828125, 0.10443401336669922, 0.10466909408569336, 0.1070699691772461, 0.10439419746398926, 0.10451102256774902, 0.10421586036682129, 0.10460615158081055, 0.10732269287109375, 0.10572171211242676, 0.10488724708557129, 0.10452771186828613, 0.1044929027557373, 0.10622310638427734, 0.10445165634155273, 0.10461163520812988, 0.10465288162231445, 0.10535216331481934, 0.10788989067077637, 0.1068124771118164, 0.10498237609863281, 0.10406064987182617, 0.10598182678222656, 0.10581374168395996, 0.10550522804260254, 0.10637593269348145, 0.10497927665710449, 0.10528731346130371, 0.10748744010925293, 0.10580825805664062, 0.10570526123046875, 0.10775995254516602, 0.10695242881774902, 0.10762953758239746, 0.10511589050292969, 0.10509538650512695, 0.10489439964294434, 0.1049962043762207, 0.10653066635131836, 0.10598993301391602, 0.10509276390075684, 0.10517644882202148, 0.10659360885620117, 0.10622191429138184, 0.1060323715209961, 0.10525655746459961, 0.10526680946350098, 0.10564088821411133, 0.10628080368041992, 0.10529971122741699, 0.10621190071105957, 0.1055757999420166, 0.10602140426635742, 0.10774993896484375, 0.10686087608337402, 0.10540962219238281, 0.10525131225585938, 0.1060020923614502, 0.10642814636230469, 0.105621337890625, 0.10759949684143066, 0.10610079765319824, 0.10610413551330566, 0.10613799095153809, 0.10697054862976074, 0.10687589645385742, 0.10598540306091309, 0.10573720932006836, 0.10636162757873535, 0.10664558410644531, 0.10540509223937988, 0.10537147521972656, 0.10625076293945312, 0.10677242279052734, 0.10625219345092773, 0.10603165626525879, 0.10718154907226562, 0.10624837875366211, 0.10710954666137695, 0.10590863227844238, 0.10532951354980469, 0.10562872886657715, 0.1055760383605957, 0.10725593566894531, 0.10547685623168945, 0.10562729835510254, 0.10580801963806152, 0.10771417617797852, 0.10835504531860352, 0.1056661605834961, 0.1058344841003418, 0.10519576072692871, 0.10736227035522461, 0.10760664939880371, 0.10581374168395996, 0.10560059547424316, 0.10561704635620117, 0.1057286262512207, 0.10741400718688965, 0.10676121711730957, 0.10596036911010742, 0.10609030723571777, 0.10647392272949219, 0.10728573799133301, 0.10584807395935059, 0.10552501678466797, 0.10576272010803223, 0.10620617866516113, 0.10788798332214355, 0.10589098930358887, 0.10747194290161133, 0.10613059997558594, 0.10901904106140137, 0.1080162525177002, 0.10582375526428223, 0.1054532527923584, 0.1056070327758789, 0.1061248779296875, 0.10755705833435059, 0.10620808601379395, 0.105987548828125, 0.10669612884521484, 0.10701107978820801, 0.10834193229675293, 0.10575008392333984, 0.10581040382385254, 0.11005568504333496, 0.10731053352355957, 0.10715413093566895, 0.10611343383789062, 0.10567831993103027, 0.10572624206542969, 0.10723423957824707, 0.10779905319213867, 0.1056065559387207, 0.10587525367736816, 0.1052703857421875, 0.10637307167053223, 0.10666584968566895, 0.10533452033996582, 0.1052546501159668, 0.10537314414978027, 0.10647034645080566, 0.10823988914489746, 0.1070258617401123, 0.10617423057556152, 0.10524439811706543, 0.10499429702758789, 0.10652470588684082, 0.10526490211486816, 0.10518741607666016, 0.10472917556762695, 0.1058802604675293, 0.10775351524353027, 0.10587096214294434, 0.10729336738586426, 0.10581827163696289, 0.10674047470092773, 0.10700058937072754, 0.10526633262634277, 0.10564351081848145, 0.1051640510559082, 0.10602402687072754, 0.10703063011169434, 0.10592055320739746, 0.10564661026000977, 0.10738468170166016, 0.10599589347839355, 0.1061091423034668, 0.10552000999450684, 0.10546278953552246, 0.10530209541320801, 0.10593175888061523, 0.10687923431396484, 0.10554361343383789, 0.10548901557922363, 0.1054372787475586, 0.10684728622436523, 0.10747051239013672, 0.10573291778564453, 0.10617542266845703, 0.10529065132141113, 0.10549759864807129, 0.10692334175109863, 0.10557389259338379, 0.10627007484436035, 0.1055612564086914, 0.10639476776123047, 0.10781574249267578, 0.10783982276916504, 0.10560488700866699, 0.10536932945251465, 0.1061561107635498, 0.10690736770629883, 0.10606646537780762, 0.10587811470031738, 0.10539937019348145, 0.1066732406616211, 0.10808634757995605, 0.10719871520996094, 0.10771346092224121, 0.1146855354309082, 0.11440348625183105, 0.11070919036865234, 0.10585999488830566, 0.10578107833862305, 0.10627532005310059, 0.10527586936950684, 0.12162423133850098, 0.10628843307495117, 0.10566949844360352, 0.10704970359802246, 0.10596036911010742, 0.1076815128326416, 0.10612916946411133, 0.10560441017150879, 0.1060647964477539, 0.10629916191101074, 0.10863065719604492, 0.1147165298461914, 0.10643267631530762, 0.10632538795471191, 0.10670852661132812, 0.11168408393859863, 0.11622285842895508, 0.10972476005554199, 0.10637998580932617, 0.10783243179321289, 0.10559296607971191, 0.10625028610229492, 0.10550045967102051, 0.10667252540588379, 0.10790777206420898, 0.10799813270568848, 0.1067817211151123, 0.10588884353637695, 0.10639357566833496, 0.10796022415161133, 0.10615086555480957, 0.10604715347290039, 0.10566020011901855, 0.10689568519592285, 0.10834169387817383, 0.1063530445098877, 0.10606145858764648, 0.1065208911895752, 0.10634136199951172, 0.10725641250610352, 0.10635900497436523, 0.10554742813110352, 0.10456085205078125, 0.10621023178100586, 0.10636377334594727, 0.10616517066955566, 0.10559320449829102, 0.10622191429138184, 0.10867857933044434, 0.10828065872192383, 0.10548901557922363, 0.10664558410644531, 0.10622072219848633, 0.10591912269592285, 0.1072390079498291, 0.10619354248046875, 0.10620379447937012, 0.10580849647521973, 0.10620355606079102, 0.10711860656738281, 0.10601639747619629, 0.10549592971801758, 0.10564303398132324, 0.10628509521484375, 0.10815811157226562, 0.1051490306854248, 0.10611939430236816, 0.10589098930358887, 0.10684800148010254, 0.1076819896697998, 0.10592532157897949, 0.10726261138916016, 0.10569119453430176, 0.10737037658691406, 0.10672235488891602, 0.10659098625183105, 0.10592460632324219, 0.10551571846008301, 0.1058645248413086, 0.10764265060424805, 0.10610485076904297, 0.10779619216918945, 0.10828971862792969, 0.10952138900756836, 0.1077885627746582, 0.10576987266540527, 0.10562324523925781, 0.1056816577911377, 0.10700106620788574, 0.10777926445007324, 0.10573530197143555, 0.10596728324890137, 0.1054835319519043, 0.10859918594360352, 0.10863900184631348, 0.10567498207092285, 0.10652303695678711, 0.10588860511779785, 0.10726642608642578, 0.1068735122680664, 0.10557985305786133, 0.10541677474975586, 0.10534477233886719, 0.10611414909362793, 0.10827207565307617, 0.10604190826416016, 0.10556745529174805, 0.10552644729614258, 0.10584449768066406, 0.10763764381408691, 0.10593938827514648, 0.10577917098999023, 0.10529828071594238, 0.10662651062011719, 0.10654449462890625, 0.1065826416015625, 0.10664987564086914, 0.1065363883972168, 0.10564064979553223, 0.10670328140258789, 0.10563373565673828, 0.10501718521118164, 0.1056373119354248, 0.10584187507629395, 0.10755467414855957, 0.10547757148742676, 0.10542559623718262, 0.10738229751586914, 0.10566973686218262, 0.10717630386352539, 0.10560989379882812, 0.10550451278686523, 0.10576963424682617, 0.10563492774963379, 0.10752701759338379, 0.10541248321533203, 0.10632777214050293, 0.10509133338928223, 0.10723352432250977, 0.1068727970123291, 0.10506844520568848, 0.10537147521972656, 0.10519814491271973, 0.10637092590332031, 0.10634994506835938, 0.10636210441589355, 0.10566258430480957, 0.10466814041137695, 0.1059877872467041, 0.10751605033874512, 0.1067965030670166, 0.10528826713562012, 0.10569977760314941, 0.10576486587524414, 0.10672688484191895, 0.10652017593383789, 0.10537028312683105, 0.10511374473571777, 0.10602045059204102, 0.10712194442749023, 0.105621337890625, 0.10834622383117676, 0.10714101791381836, 0.10573434829711914, 0.10871624946594238, 0.10679459571838379, 0.10915112495422363, 0.11662936210632324, 0.11442732810974121, 0.11611270904541016, 0.10557270050048828, 0.1060643196105957, 0.11090898513793945, 0.10865259170532227, 0.10847163200378418, 0.11253046989440918, 0.10525965690612793, 0.10610771179199219, 0.10939216613769531, 0.10588216781616211, 0.1062312126159668, 0.10534548759460449, 0.10674762725830078, 0.11034440994262695, 0.11917328834533691, 0.1156606674194336, 0.10803675651550293, 0.1143643856048584, 0.1110532283782959, 0.11570405960083008, 0.11130881309509277, 0.11040735244750977, 0.1077427864074707, 0.11193299293518066, 0.11217331886291504, 0.1317124366760254, 0.1296401023864746, 0.11646485328674316, 0.10741090774536133, 0.11178731918334961, 0.10646343231201172, 0.10707759857177734, 0.10607528686523438, 0.10507845878601074, 0.10620665550231934, 0.10536766052246094, 0.10616898536682129, 0.10591387748718262, 0.10537147521972656, 0.10558843612670898, 0.10565638542175293, 0.10647916793823242, 0.10492086410522461, 0.10518598556518555, 0.10547065734863281, 0.10612082481384277, 0.10580706596374512, 0.10532641410827637, 0.10555052757263184, 0.10544157028198242, 0.10595369338989258, 0.10599994659423828, 0.10516047477722168, 0.1056358814239502, 0.10541558265686035, 0.1055765151977539, 0.10598397254943848, 0.10584092140197754, 0.10552239418029785, 0.10840225219726562, 0.10691189765930176, 0.10591459274291992, 0.10531949996948242, 0.10570836067199707, 0.10703277587890625, 0.10622715950012207, 0.10588312149047852, 0.10530400276184082, 0.10893464088439941, 0.10691690444946289, 0.11399626731872559, 0.1073615550994873, 0.1056053638458252, 0.1074836254119873, 0.11559033393859863, 0.11641716957092285, 0.11166834831237793, 0.11085700988769531, 0.10556483268737793, 0.11054635047912598, 0.10582947731018066, 0.10609769821166992, 0.10727381706237793, 0.10564327239990234, 0.10909914970397949, 0.10564947128295898, 0.10603475570678711, 0.10665583610534668, 0.10892987251281738, 0.10799407958984375, 0.10858917236328125, 0.10626721382141113, 0.10656094551086426, 0.1075601577758789, 0.1064612865447998, 0.10623717308044434, 0.10527777671813965, 0.10706162452697754, 0.10682559013366699, 0.10716962814331055, 0.10624313354492188, 0.10861492156982422, 0.10724163055419922, 0.10785603523254395, 0.10605049133300781, 0.10573434829711914, 0.10526108741760254, 0.10653185844421387, 0.10718417167663574, 0.10661959648132324, 0.10631799697875977, 0.1053159236907959, 0.10729169845581055, 0.10996174812316895, 0.10755491256713867, 0.10646247863769531, 0.10620808601379395, 0.10643982887268066, 0.10754823684692383, 0.10571908950805664, 0.1063990592956543, 0.10560107231140137, 0.1067514419555664, 0.10758066177368164, 0.10862565040588379, 0.1056210994720459, 0.10526299476623535, 0.1075434684753418, 0.11543416976928711, 0.1149446964263916, 0.11068367958068848, 0.10670971870422363, 0.11445260047912598, 0.11428022384643555, 0.11602258682250977, 0.11726212501525879, 0.11524438858032227, 0.11582207679748535, 0.11382818222045898, 0.1162271499633789, 0.2062833309173584, 0.20429491996765137, 0.17893004417419434, 0.10490584373474121, 0.10497617721557617, 0.1046149730682373, 0.10589218139648438, 0.10474181175231934, 0.10537433624267578, 0.10452532768249512, 0.10526156425476074, 0.1064300537109375, 0.10436654090881348, 0.10413217544555664, 0.10479497909545898, 0.10718393325805664, 0.1056058406829834, 0.10451054573059082, 0.10641288757324219, 0.10439944267272949, 0.10816621780395508, 0.10463094711303711, 0.10541462898254395, 0.10482954978942871, 0.10560011863708496, 0.10871005058288574, 0.1054837703704834, 0.10720539093017578, 0.10709786415100098, 0.10568547248840332, 0.10573172569274902, 0.10560369491577148, 0.10500431060791016, 0.1043391227722168, 0.10506486892700195, 0.10651659965515137, 0.10526227951049805, 0.10560059547424316, 0.10571932792663574, 0.10622763633728027, 0.10539436340332031, 0.10489654541015625, 0.10424995422363281, 0.10471773147583008, 0.1048421859741211, 0.10489511489868164, 0.10490942001342773, 0.10490775108337402, 0.10427713394165039, 0.1048891544342041, 0.10522842407226562, 0.10462427139282227, 0.10460948944091797, 0.10496091842651367, 0.10464310646057129, 0.10447120666503906, 0.10425901412963867, 0.10479950904846191, 0.10539126396179199, 0.10460877418518066, 0.10456228256225586, 0.10427069664001465, 0.10476851463317871, 0.1073765754699707, 0.10750651359558105, 0.10714530944824219, 0.15026021003723145, 0.19289374351501465, 0.17678022384643555, 0.2969684600830078, 0.19925403594970703, 0.11800670623779297, 0.28420305252075195, 0.2774777412414551, 0.28693175315856934, 0.10702037811279297, 0.1210942268371582, 0.10578274726867676, 0.10582089424133301, 0.10532021522521973, 0.10630488395690918, 0.1090688705444336, 0.11080694198608398, 0.27830982208251953, 0.19404077529907227, 0.2683708667755127, 0.2966799736022949, 0.2497413158416748, 0.3664853572845459, 0.32942652702331543, 0.2439572811126709, 0.10649323463439941, 0.11906003952026367, 0.10650348663330078, 0.10701131820678711, 0.10587382316589355, 0.10546422004699707, 0.10696792602539062, 0.10697579383850098, 0.10662150382995605, 0.2894606590270996, 0.2018289566040039, 0.3229832649230957, 0.2906606197357178, 0.3355729579925537, 0.35253000259399414, 0.14689993858337402, 0.10695409774780273, 0.10614132881164551, 0.1060495376586914, 0.10518002510070801, 0.10585403442382812, 0.1064598560333252, 0.10564446449279785, 0.10585236549377441, 0.10567045211791992, 0.1085202693939209, 0.10706210136413574, 0.11155533790588379, 0.10542058944702148, 0.10566568374633789, 0.10609841346740723, 0.10618185997009277, 0.10624504089355469, 0.10528707504272461, 0.11334466934204102, 0.29323601722717285, 0.16072297096252441, 0.31745171546936035, 0.2949225902557373, 0.22686982154846191, 0.25284409523010254, 0.1176457405090332, 0.10779595375061035, 0.10530972480773926, 0.1058492660522461, 0.10527586936950684, 0.10563921928405762, 0.10600423812866211, 0.10462737083435059, 0.10529279708862305, 0.10652732849121094, 0.10694766044616699, 0.10672664642333984, 0.10556912422180176, 0.10496211051940918, 0.10769796371459961, 0.2685532569885254, 0.2844400405883789, 0.30570411682128906, 0.27928996086120605, 0.2817366123199463, 0.11920762062072754, 0.10620617866516113, 0.10543489456176758, 0.10806536674499512, 0.11508488655090332, 0.1312551498413086, 0.11071252822875977, 0.107696533203125, 0.1135721206665039, 0.11371040344238281, 0.11356496810913086, 0.10615849494934082, 0.10653543472290039, 0.10549092292785645, 0.11216974258422852, 0.11195516586303711, 0.34067440032958984, 0.10954046249389648, 0.10686540603637695, 0.1168818473815918, 0.23962688446044922, 0.3455190658569336, 0.2047100067138672, 0.2664346694946289, 0.31732892990112305, 0.21180963516235352, 0.2149190902709961, 0.10588955879211426, 0.1085357666015625, 0.11599874496459961, 0.10566902160644531, 0.10576748847961426, 0.14348936080932617, 0.27857160568237305, 0.18950414657592773, 0.2525441646575928, 0.11923027038574219, 0.10540366172790527, 0.10567831993103027, 0.10609698295593262, 0.1062624454498291, 0.10542416572570801, 0.13774585723876953, 0.40041065216064453, 0.12155652046203613, 0.10623478889465332, 0.12039518356323242, 0.10667133331298828, 0.1056661605834961, 0.10515570640563965, 0.1428670883178711, 0.31755781173706055, 0.11541008949279785, 0.12902307510375977, 0.12011361122131348, 0.11602044105529785, 0.11380958557128906, 0.11178922653198242, 0.10807371139526367, 0.10889029502868652, 0.10611152648925781, 0.10833859443664551, 0.1069800853729248, 0.10677433013916016, 0.10727524757385254, 0.10583615303039551, 0.10543060302734375, 0.10653090476989746, 0.10555696487426758, 0.1062619686126709, 0.1064138412475586, 0.10636210441589355, 0.10613703727722168, 0.10549688339233398, 0.10685467720031738, 0.10571789741516113, 0.1061406135559082, 0.10572195053100586, 0.10581135749816895, 0.10564947128295898, 0.10630249977111816, 0.10687422752380371, 0.10639119148254395, 0.10587763786315918, 0.10531210899353027, 0.10616946220397949, 0.1062779426574707, 0.10586738586425781, 0.10564827919006348, 0.10560250282287598, 0.1056833267211914, 0.10486555099487305, 0.10553312301635742, 0.10643744468688965, 0.1059415340423584, 0.10565328598022461, 0.10527658462524414, 0.11029720306396484, 0.10849595069885254, 0.1058206558227539, 0.10519576072692871, 0.10541510581970215, 0.10613656044006348, 0.10602831840515137, 0.10758781433105469, 0.10605645179748535, 0.10554838180541992, 0.10497641563415527, 0.10560822486877441, 0.10577535629272461, 0.10529041290283203, 0.10538005828857422, 0.10556650161743164, 0.10521602630615234, 0.10628437995910645, 0.10643410682678223, 0.10584473609924316, 0.1058814525604248, 0.1055760383605957, 0.10552310943603516, 0.10621523857116699, 0.20949935913085938, 0.202317476272583, 0.10613584518432617, 0.10548281669616699, 0.106903076171875, 0.105987548828125, 0.10580658912658691, 0.10508942604064941, 0.1059272289276123, 0.10539865493774414, 0.10613250732421875, 0.10549139976501465, 0.10523247718811035, 0.10511398315429688, 0.10557413101196289, 0.10702633857727051, 0.10678935050964355, 0.10606813430786133, 0.1058206558227539, 0.10547971725463867, 0.10594582557678223, 0.10583090782165527, 0.10539031028747559, 0.10547661781311035, 0.10561800003051758, 0.10627532005310059, 0.10625171661376953, 0.10582137107849121, 0.10561609268188477, 0.10590314865112305, 0.10620379447937012, 0.10613322257995605, 0.10504531860351562, 0.10519671440124512, 0.10556793212890625, 0.10558962821960449, 0.10727572441101074, 0.10535335540771484, 0.10657119750976562, 0.10543179512023926, 0.1056818962097168, 0.10552573204040527, 0.10475873947143555, 0.10538029670715332, 0.10525345802307129, 0.10634922981262207, 0.11205792427062988, 0.10957813262939453, 0.1052701473236084, 0.10581755638122559, 0.10627102851867676, 0.10552716255187988, 0.10691165924072266, 0.10585737228393555, 0.10507845878601074, 0.1061093807220459, 0.10595893859863281, 0.10522317886352539, 0.10459733009338379, 0.1052556037902832, 0.1070866584777832, 0.10664129257202148, 0.10571527481079102, 0.1049644947052002, 0.10538578033447266, 0.10629105567932129, 0.10618710517883301, 0.10482668876647949, 0.10483431816101074, 0.10475754737854004, 0.10541391372680664, 0.10564351081848145, 0.10581064224243164, 0.1054847240447998, 0.1054379940032959, 0.10570812225341797, 0.10574221611022949, 0.10550618171691895, 0.10547828674316406, 0.10552239418029785, 0.10620951652526855, 0.10597634315490723, 0.10486865043640137, 0.1058039665222168, 0.10535883903503418, 0.10605454444885254, 0.1059577465057373, 0.10526037216186523, 0.10527706146240234, 0.10522651672363281, 0.10580968856811523, 0.10572600364685059, 0.10499215126037598, 0.10595822334289551, 0.10591959953308105, 0.10626626014709473, 0.10550379753112793, 0.10530471801757812, 0.10471081733703613, 0.10562467575073242, 0.1057894229888916, 0.1056206226348877, 0.1049957275390625, 0.10482215881347656, 0.1060633659362793, 0.10573840141296387, 0.10677695274353027, 0.10550332069396973, 0.10524749755859375, 0.10541486740112305, 0.10559773445129395, 0.10538125038146973, 0.10512495040893555, 0.1049964427947998, 0.1053621768951416, 0.10572123527526855, 0.10561752319335938, 0.1056222915649414, 0.10556745529174805, 0.10529327392578125, 0.10619783401489258, 0.10588788986206055, 0.10550332069396973, 0.10510087013244629, 0.10500097274780273, 0.10533976554870605, 0.10570788383483887, 0.10493206977844238, 0.10524487495422363, 0.10665416717529297, 0.10611939430236816, 0.10580563545227051, 0.10528397560119629, 0.10547280311584473, 0.10530424118041992, 0.10555386543273926, 0.10538172721862793, 0.10536313056945801, 0.10524392127990723, 0.10520339012145996, 0.10704636573791504, 0.10643982887268066, 0.1065523624420166, 0.1054987907409668, 0.10623693466186523, 0.10576939582824707, 0.10566830635070801, 0.1050252914428711, 0.10528945922851562, 0.10519218444824219, 0.10574078559875488, 0.10633087158203125, 0.1056058406829834, 0.10509419441223145, 0.10503458976745605, 0.10581421852111816, 0.10557174682617188, 0.10532498359680176, 0.1051626205444336, 0.10504841804504395, 0.10575580596923828, 0.1060645580291748, 0.10498857498168945, 0.10682988166809082, 0.10578227043151855, 0.10593390464782715, 0.10550594329833984, 0.1050713062286377, 0.10487699508666992, 0.10576653480529785, 0.10599422454833984, 0.10584211349487305, 0.10551166534423828, 0.10517621040344238, 0.10588908195495605, 0.1057898998260498, 0.10576295852661133, 0.10545802116394043, 0.10522150993347168, 0.10546565055847168, 0.10576987266540527, 0.1062626838684082, 0.1052851676940918, 0.10504913330078125, 0.10561227798461914, 0.10719561576843262, 0.10628080368041992, 0.10619497299194336, 0.10540008544921875, 0.10489320755004883, 0.10626053810119629, 0.10559558868408203, 0.1055750846862793, 0.10516595840454102, 0.1055901050567627, 0.1053621768951416, 0.10519075393676758, 0.10958743095397949, 0.28419017791748047, 0.12137937545776367, 0.1071319580078125, 0.10530948638916016, 0.1054842472076416, 0.10620427131652832, 0.10570669174194336, 0.10733580589294434, 0.10749077796936035, 0.10562562942504883, 0.1071767807006836, 0.2668027877807617, 0.22548341751098633, 0.11895251274108887, 0.10680580139160156, 0.10579752922058105, 0.1056971549987793, 0.10654735565185547, 0.1060636043548584, 0.10539770126342773, 0.1053614616394043, 0.10542559623718262, 0.1050717830657959, 0.10656857490539551, 0.10567617416381836, 0.10511469841003418, 0.10543417930603027, 0.10563945770263672, 0.10680818557739258, 0.10657119750976562, 0.10599684715270996, 0.10520815849304199, 0.1057133674621582, 0.10601425170898438, 0.1060628890991211, 0.10550236701965332, 0.10620450973510742, 0.10583114624023438, 0.10595202445983887, 0.1067352294921875, 0.1056833267211914, 0.10612916946411133, 0.10689377784729004, 0.10655713081359863, 0.10611391067504883, 0.1045994758605957, 0.10533404350280762, 0.10477590560913086, 0.10641765594482422, 0.10581636428833008, 0.10547423362731934, 0.10674667358398438, 0.10545063018798828, 0.10514664649963379, 0.10540509223937988, 0.1388857364654541, 0.24425554275512695, 0.24405503273010254, 0.2000105381011963, 0.11483383178710938, 0.10649394989013672, 0.10508370399475098, 0.1054072380065918, 0.10539078712463379, 0.10556650161743164, 0.10585975646972656, 0.1050722599029541, 0.10532402992248535, 0.10570025444030762, 0.10590744018554688, 0.10713601112365723, 0.10545468330383301, 0.10549759864807129, 0.10553860664367676, 0.10527586936950684, 0.11041522026062012, 0.1128227710723877, 0.11376523971557617, 0.1114034652709961, 0.11041927337646484, 0.1075136661529541, 0.10831046104431152, 0.10806441307067871, 0.11437225341796875, 0.11515378952026367, 0.1284620761871338, 0.10858035087585449, 0.10583233833312988, 0.10767793655395508, 0.10706162452697754, 0.1172177791595459, 0.17033743858337402, 0.12713313102722168, 0.24411439895629883, 0.23435115814208984, 0.19890952110290527, 0.10604000091552734, 0.10661911964416504, 0.10597681999206543, 0.1139535903930664, 0.13486099243164062, 0.11097574234008789, 0.12658357620239258, 0.11877322196960449, 0.10757851600646973, 0.10711550712585449, 0.10942268371582031, 0.11272144317626953, 0.11722016334533691, 0.17931175231933594, 0.22133755683898926, 0.1060340404510498, 0.12074828147888184, 0.15381574630737305, 0.13522911071777344, 0.23006582260131836, 0.2425675392150879, 0.20220732688903809, 0.1557319164276123, 0.256852388381958, 0.21835613250732422, 0.16028404235839844, 0.19032597541809082, 0.19629383087158203, 0.1541423797607422, 0.11670708656311035, 0.10694146156311035, 0.10514187812805176, 0.1064150333404541, 0.10484671592712402, 0.128129243850708, 0.20122575759887695, 0.14956402778625488, 0.18063855171203613, 0.16344404220581055, 0.13631248474121094, 0.16909170150756836, 0.15363025665283203, 0.11873912811279297, 0.15619730949401855, 0.15915942192077637, 0.12730860710144043, 0.12979722023010254, 0.11995577812194824, 0.28182458877563477, 0.3108842372894287, 0.4267101287841797, 0.3083639144897461, 0.3432803153991699, 0.5091331005096436, 0.3627817630767822, 0.1757962703704834, 0.32895398139953613, 0.44968438148498535, 0.2304387092590332, 0.514700174331665, 0.1565873622894287, 0.1772775650024414, 0.1097724437713623, 0.10558676719665527, 0.10454392433166504, 0.10476851463317871, 0.10624814033508301, 0.10663509368896484, 0.15745043754577637, 0.35079336166381836, 0.30883288383483887, 0.11120772361755371, 0.10543155670166016, 0.1056361198425293, 0.10521125793457031, 0.10528683662414551, 0.10659599304199219, 0.22562718391418457, 0.3145124912261963, 0.1083076000213623, 0.10577678680419922, 0.10583949089050293, 0.1056978702545166, 0.10495400428771973, 0.10740876197814941, 0.36400604248046875, 0.2287156581878662, 0.26160430908203125, 0.22826695442199707, 0.13949942588806152, 0.13283777236938477, 0.12619900703430176, 0.1202239990234375, 0.11411643028259277, 0.3093869686126709, 0.14963054656982422, 0.11952567100524902, 0.11924624443054199, 0.11711382865905762, 0.11411786079406738, 0.11120963096618652, 0.10838723182678223, 0.1064004898071289, 0.11116170883178711, 0.11047887802124023, 0.1067495346069336, 0.2587149143218994, 0.22948002815246582, 0.11446928977966309, 0.1148533821105957, 0.11757302284240723, 0.34267187118530273, 0.4297292232513428, 0.20714807510375977, 0.16130495071411133, 0.14331483840942383, 0.1313340663909912, 0.12207746505737305, 0.11544370651245117, 0.10958051681518555, 0.10642361640930176, 0.10493707656860352, 0.10644340515136719, 0.10471272468566895, 0.105072021484375, 0.10448622703552246, 0.10473918914794922, 0.10522341728210449, 0.1049807071685791, 0.10515069961547852, 0.10527181625366211, 0.10500836372375488, 0.10910534858703613, 0.10782217979431152, 0.10519838333129883, 0.10464286804199219, 0.10489654541015625, 0.10484552383422852, 0.10502839088439941, 0.10450434684753418, 0.10479998588562012, 0.10570311546325684, 0.10466694831848145, 0.10507750511169434, 0.10475921630859375, 0.10419034957885742, 0.10453557968139648, 0.11149454116821289, 0.10848331451416016, 0.10457777976989746, 0.10525655746459961, 0.10903692245483398, 0.20454692840576172, 0.2889370918273926, 0.21554064750671387, 0.18608832359313965, 0.1058664321899414, 0.1049795150756836, 0.1051931381225586, 0.10459518432617188, 0.1511859893798828, 0.3390848636627197, 0.13050198554992676, 0.10514640808105469, 0.10510063171386719, 0.10451126098632812, 0.10472393035888672, 0.10593366622924805, 0.10506939888000488, 0.10487508773803711, 0.10468935966491699, 0.10524916648864746, 0.10516858100891113, 0.10471200942993164, 0.10580062866210938, 0.10484170913696289, 0.1051785945892334, 0.10446667671203613, 0.10543179512023926, 0.1046898365020752, 0.10466790199279785, 0.10524511337280273, 0.10476160049438477, 0.10489439964294434, 0.10474395751953125, 0.10494804382324219, 0.10855984687805176, 0.10499978065490723, 0.10473322868347168, 0.10430026054382324, 0.10498547554016113, 0.10523748397827148, 0.10519671440124512, 0.10495114326477051, 0.10478544235229492, 0.10482311248779297, 0.10449409484863281, 0.10518074035644531, 0.10472488403320312, 0.1048746109008789, 0.1056818962097168, 0.10442590713500977, 0.10503220558166504, 0.10418868064880371, 0.1065835952758789, 0.10552453994750977, 0.10531854629516602, 0.10477113723754883, 0.10464048385620117, 0.1127171516418457, 0.10509824752807617, 0.10558819770812988, 0.1051943302154541, 0.10480546951293945, 0.10597968101501465, 0.10460186004638672, 0.10463762283325195, 0.10445308685302734, 0.10502910614013672, 0.10530781745910645, 0.10533642768859863, 0.10504698753356934, 0.11042022705078125, 0.10538363456726074, 0.10697412490844727, 0.10553836822509766, 0.10772919654846191, 0.1090233325958252, 0.2103285789489746, 0.2222728729248047, 0.10782599449157715, 0.10499906539916992, 0.11926937103271484, 0.10547304153442383, 0.10628366470336914, 0.12428712844848633, 0.10552096366882324, 0.10516619682312012, 0.10875606536865234, 0.10628294944763184, 0.10533761978149414, 0.10543274879455566, 0.11189055442810059, 0.27686595916748047, 0.37684059143066406, 0.19585680961608887, 0.10654544830322266, 0.1049952507019043, 0.20207834243774414, 0.25077152252197266, 0.1291060447692871, 0.12556719779968262, 0.2006232738494873, 0.2636258602142334, 0.143324613571167, 0.16637110710144043, 0.11748909950256348, 0.11395049095153809, 0.11756658554077148, 0.11413145065307617, 0.10863089561462402, 0.11085200309753418, 0.10803484916687012, 0.1329796314239502, 0.13180184364318848, 0.1069633960723877, 0.10692238807678223, 0.10715103149414062, 0.11658358573913574, 0.1069190502166748, 0.10821199417114258, 0.12034153938293457, 0.12485170364379883, 0.12262797355651855, 0.10841894149780273, 0.12131261825561523, 0.10710859298706055, 0.10730147361755371, 0.12187600135803223, 0.11910033226013184, 0.11904501914978027, 0.1196584701538086, 0.10788464546203613, 0.10694646835327148, 0.10882329940795898, 0.10654187202453613, 0.11119747161865234, 0.10552167892456055, 0.10593342781066895, 0.11374258995056152, 0.13317084312438965, 0.11973023414611816, 0.11538934707641602, 0.1063532829284668, 0.11184239387512207, 0.12421512603759766, 0.11789917945861816, 0.1113579273223877, 0.10867738723754883, 0.11134767532348633, 0.11520028114318848, 0.1130530834197998, 0.11282706260681152, 0.10714101791381836, 0.10637021064758301, 0.10517287254333496, 0.11583161354064941, 0.1074380874633789, 0.1145021915435791, 0.11252474784851074, 0.10872697830200195, 0.1185753345489502, 0.11358976364135742, 0.11126995086669922, 0.11584043502807617, 0.10764670372009277, 0.10933399200439453, 0.10763144493103027, 0.10682892799377441, 0.11334991455078125, 0.10556459426879883, 0.10575246810913086, 0.10532903671264648, 0.11298751831054688, 0.12405753135681152, 0.1401982307434082, 0.2792041301727295, 0.3795590400695801, 0.42894625663757324, 0.4284050464630127, 0.4082067012786865, 0.22768712043762207, 0.11570596694946289, 0.1291351318359375, 0.13632893562316895, 0.10874557495117188, 0.10742831230163574, 0.12361741065979004, 0.12388896942138672, 0.10597991943359375, 0.11687254905700684, 0.11786723136901855, 0.11157631874084473, 0.10773253440856934, 0.23932147026062012, 0.2341175079345703, 0.35744237899780273, 0.26160287857055664, 0.20340466499328613, 0.10645651817321777, 0.11829924583435059, 0.10725665092468262, 0.11180853843688965, 0.10938191413879395, 0.2872164249420166, 0.24684619903564453, 0.287294864654541, 0.31082725524902344, 0.171464204788208, 0.15759849548339844, 0.13815927505493164, 0.12803125381469727, 0.13000988960266113, 0.11387252807617188, 0.11663007736206055, 0.10670804977416992, 0.10655426979064941, 0.12613654136657715, 0.11825156211853027, 0.11615514755249023, 0.11249971389770508, 0.12396979331970215, 0.11489629745483398, 0.11494064331054688, 0.12508344650268555, 0.11241388320922852, 0.11121726036071777, 0.11147665977478027, 0.11118602752685547, 0.11482787132263184, 0.11476469039916992, 0.12684106826782227, 0.11285877227783203, 0.10595011711120605, 0.11374831199645996, 0.11145377159118652, 0.10628676414489746, 0.1113889217376709, 0.10964584350585938, 0.10686445236206055, 0.11392760276794434, 0.10613393783569336, 0.10542964935302734, 0.11461234092712402, 0.11674332618713379, 0.12335920333862305, 0.11490583419799805, 0.11230587959289551, 0.10721230506896973, 0.11767268180847168, 0.10759401321411133, 0.10630249977111816, 0.10781502723693848, 0.10724687576293945, 0.11147499084472656, 0.10725045204162598, 0.11080193519592285, 0.1060490608215332, 0.1061701774597168, 0.11463427543640137, 0.1059567928314209, 0.10890507698059082, 0.10758399963378906, 0.10652780532836914, 0.11334466934204102, 0.10613155364990234, 0.11193561553955078, 0.11377882957458496, 0.10664176940917969, 0.11788415908813477, 0.1064598560333252, 0.10737776756286621, 0.11202144622802734, 0.10601544380187988, 0.11418390274047852, 0.10644340515136719, 0.10577392578125, 0.10632061958312988, 0.11538386344909668, 0.12999868392944336, 0.1140749454498291, 0.1153714656829834, 0.11248493194580078, 0.11580252647399902, 0.10558128356933594, 0.10617256164550781, 0.10586214065551758, 0.10635876655578613, 0.1116323471069336, 0.11550045013427734, 0.1066741943359375, 0.10662579536437988, 0.10589194297790527, 0.1160275936126709, 0.11089754104614258, 0.11812472343444824, 0.11159777641296387, 0.10817646980285645, 0.11997842788696289, 0.12005305290222168, 0.10915064811706543, 0.12757396697998047, 0.12668585777282715, 0.11387324333190918, 0.1660752296447754, 0.1073157787322998, 0.11880803108215332, 0.10826897621154785, 0.11859512329101562, 0.11895895004272461, 0.10631179809570312, 0.11756396293640137, 0.10863828659057617, 0.10598564147949219, 0.10589742660522461, 0.10602283477783203, 0.1143491268157959, 0.10716748237609863, 0.11867880821228027, 0.12208104133605957, 0.14420723915100098, 0.19724559783935547, 0.11126255989074707, 0.11245179176330566, 0.12167119979858398, 0.16400599479675293, 0.13415861129760742, 0.13794827461242676, 0.1862647533416748, 0.10638904571533203, 0.10600924491882324, 0.11428427696228027, 0.10707759857177734, 0.1052093505859375, 0.1114802360534668, 0.10597753524780273, 0.11157488822937012, 0.10538387298583984, 0.10468673706054688, 0.10491108894348145, 0.10506558418273926, 0.10523653030395508, 0.10680365562438965, 0.11277055740356445, 0.10543155670166016, 0.10660338401794434, 0.1599283218383789, 0.105865478515625, 0.1161494255065918, 0.11570405960083008, 0.11362957954406738, 0.1083216667175293, 0.10582113265991211, 0.1051015853881836, 0.10693883895874023, 0.10660457611083984, 0.10561847686767578, 0.10582566261291504, 0.1051638126373291, 0.10535812377929688, 0.10508346557617188, 0.1049354076385498, 0.10524773597717285, 0.10537481307983398, 0.10601329803466797, 0.10517501831054688, 0.10486054420471191, 0.10526752471923828, 0.10493779182434082, 0.10496926307678223, 0.11274242401123047, 0.11526679992675781, 0.11504912376403809, 0.11449432373046875, 0.11435961723327637, 0.11417150497436523, 0.11522245407104492, 0.11367225646972656, 0.11549568176269531, 0.10759592056274414, 0.1050102710723877, 0.10455560684204102, 0.10524463653564453, 0.10572028160095215, 0.10512137413024902, 0.1047506332397461, 0.10568547248840332, 0.10705137252807617, 0.11654472351074219, 0.11325645446777344, 0.1104438304901123, 0.10554003715515137, 0.10495615005493164, 0.10612773895263672, 0.10517191886901855, 0.10486149787902832, 0.10471367835998535, 0.10530948638916016, 0.10455799102783203, 0.10493040084838867, 0.10498929023742676, 0.10494065284729004, 0.10467004776000977, 0.10450243949890137, 0.11507844924926758, 0.10527205467224121, 0.10517454147338867, 0.10506677627563477, 0.1056516170501709, 0.10473275184631348, 0.10492467880249023, 0.10501861572265625, 0.10512948036193848, 0.10557413101196289, 0.10469293594360352, 0.10498476028442383, 0.10550832748413086, 0.10520172119140625, 0.10516691207885742, 0.11449265480041504, 0.10791134834289551, 0.10615229606628418, 0.10512208938598633, 0.10492968559265137, 0.10626506805419922, 0.10868263244628906, 0.11461663246154785, 0.10831379890441895, 0.10488200187683105, 0.10510563850402832, 0.10762548446655273, 0.11031937599182129, 0.11409282684326172, 0.11328959465026855, 0.11737704277038574, 0.1125645637512207, 0.11243176460266113, 0.10806798934936523, 0.1064445972442627, 0.1082146167755127, 0.10662102699279785, 0.1089932918548584, 0.10545086860656738, 0.10560345649719238, 0.11118316650390625, 0.11055326461791992, 0.10757565498352051, 0.11013960838317871, 0.10661602020263672, 0.10955214500427246, 0.11045002937316895, 0.10643887519836426, 0.1063230037689209, 0.11257672309875488, 0.11528205871582031, 0.11633944511413574, 0.1159677505493164, 0.11062049865722656, 0.10475492477416992, 0.11568927764892578, 0.11597847938537598, 0.11326789855957031, 0.12218904495239258, 0.15343070030212402, 0.19534063339233398, 0.20242047309875488, 0.17863178253173828, 0.10555696487426758, 0.105072021484375, 0.10517477989196777, 0.17156100273132324, 0.11301326751708984, 0.10510063171386719, 0.10491037368774414, 0.10522103309631348, 0.10462021827697754, 0.1051473617553711, 0.10420656204223633, 0.10402178764343262, 0.10386466979980469, 0.1034841537475586, 0.10461735725402832, 0.10459518432617188, 0.10456371307373047, 0.1053609848022461, 0.18926000595092773, 0.18207144737243652, 0.1309499740600586, 0.13302874565124512, 0.1203920841217041, 0.12162017822265625, 0.10687875747680664, 0.10634326934814453, 0.1036679744720459, 0.10468864440917969, 0.10478901863098145, 0.10538578033447266, 0.10538291931152344, 0.10510015487670898, 0.10478401184082031, 0.10520792007446289, 0.10513472557067871, 0.10563826560974121, 0.10553288459777832, 0.10486674308776855, 0.10555458068847656, 0.10789299011230469, 0.10758447647094727, 0.10601377487182617, 0.1049354076385498, 0.10550045967102051, 0.10569643974304199, 0.10499000549316406, 0.10457515716552734, 0.10581469535827637, 0.1047353744506836, 0.10578680038452148, 0.10516190528869629, 0.10504984855651855, 0.10515165328979492, 0.10558223724365234, 0.10506367683410645, 0.10569071769714355, 0.10503554344177246, 0.10484170913696289, 0.1054069995880127, 0.10540032386779785, 0.10446643829345703, 0.10486769676208496, 0.10513496398925781, 0.10524106025695801, 0.10531353950500488, 0.10517668724060059, 0.10818052291870117, 0.1047673225402832, 0.1059257984161377, 0.10457658767700195, 0.10490822792053223, 0.10506010055541992, 0.10515213012695312, 0.10537600517272949, 0.10486745834350586, 0.10476469993591309, 0.10472679138183594, 0.10587835311889648, 0.10598969459533691, 0.10459446907043457, 0.10485649108886719, 0.10520648956298828, 0.1051938533782959, 0.10492801666259766, 0.10513186454772949, 0.1046292781829834, 0.10510754585266113, 0.10481381416320801, 0.10503029823303223, 0.10439443588256836, 0.10487866401672363, 0.10587239265441895, 0.10493969917297363, 0.10544180870056152, 0.10501861572265625, 0.10475564002990723, 0.10498785972595215, 0.10474848747253418, 0.10476088523864746, 0.10479474067687988, 0.10514974594116211, 0.10484576225280762, 0.10535550117492676, 0.10475754737854004, 0.1047525405883789, 0.10552000999450684, 0.10508036613464355, 0.10477495193481445, 0.10453915596008301, 0.10512256622314453, 0.10469508171081543, 0.1044931411743164, 0.10508251190185547, 0.10465574264526367, 0.10519623756408691, 0.10567307472229004, 0.10518574714660645, 0.1048741340637207, 0.10494422912597656, 0.1053624153137207, 0.1043710708618164, 0.10493779182434082, 0.1049187183380127, 0.10473275184631348, 0.10467052459716797, 0.10483098030090332, 0.105224609375, 0.10460495948791504, 0.10476207733154297, 0.10511374473571777, 0.10510563850402832, 0.10485410690307617, 0.10528445243835449, 0.10527539253234863, 0.10518479347229004, 0.1098489761352539, 0.1064138412475586, 0.10527563095092773, 0.10515594482421875, 0.10517668724060059, 0.10491538047790527, 0.10527372360229492, 0.10574603080749512, 0.10528063774108887, 0.10846972465515137, 0.10592293739318848, 0.10455918312072754, 0.10557913780212402, 0.10529136657714844, 0.10553979873657227, 0.11148929595947266, 0.10552239418029785, 0.1051473617553711, 0.10509657859802246, 0.10555505752563477, 0.1049492359161377, 0.10557031631469727, 0.10565376281738281, 0.1050412654876709, 0.10496187210083008, 0.10507965087890625, 0.10577893257141113, 0.10496187210083008, 0.10599207878112793, 0.10527825355529785, 0.10493159294128418, 0.10532498359680176, 0.10571956634521484, 0.10493707656860352, 0.10576486587524414, 0.10499238967895508, 0.10533952713012695, 0.10518717765808105, 0.10512924194335938, 0.10453462600708008, 0.10506796836853027, 0.10501694679260254, 0.1050412654876709, 0.10498809814453125, 0.10497832298278809, 0.10444784164428711, 0.10484123229980469, 0.10484862327575684, 0.10486865043640137, 0.10489845275878906, 0.1051795482635498, 0.10493755340576172, 0.10495305061340332, 0.10468673706054688, 0.10455536842346191, 0.10550546646118164, 0.10475945472717285, 0.10512042045593262, 0.10448288917541504, 0.10488629341125488, 0.1046750545501709, 0.10512900352478027, 0.10593295097351074, 0.10532474517822266, 0.10508155822753906, 0.10485219955444336, 0.105224609375, 0.10478591918945312, 0.10515999794006348, 0.10547828674316406, 0.10489058494567871, 0.1052711009979248, 0.10518550872802734, 0.10484027862548828, 0.10495162010192871, 0.10547161102294922, 0.10528349876403809, 0.10456418991088867, 0.10492467880249023, 0.10508108139038086, 0.10509085655212402, 0.10503530502319336, 0.10534238815307617, 0.10537910461425781, 0.1050562858581543, 0.10549545288085938, 0.10492348670959473, 0.10462093353271484, 0.10492515563964844, 0.10464668273925781, 0.10443735122680664, 0.10438990592956543, 0.10443568229675293, 0.10425209999084473, 0.10414671897888184, 0.1065511703491211, 0.1045074462890625, 0.10423779487609863, 0.10448217391967773, 0.10388422012329102, 0.1038970947265625, 0.1040487289428711, 0.10394835472106934, 0.10451865196228027, 0.10441255569458008, 0.10456180572509766, 0.10387444496154785, 0.10446667671203613, 0.10411953926086426, 0.10416960716247559, 0.1054835319519043, 0.10431289672851562, 0.10403943061828613, 0.10477805137634277, 0.1043541431427002, 0.10379910469055176, 0.10363888740539551, 0.10467290878295898, 0.10413026809692383, 0.10405778884887695, 0.1044156551361084, 0.10357999801635742, 0.10385537147521973, 0.10425519943237305, 0.10428357124328613, 0.10385775566101074, 0.10412836074829102, 0.10404849052429199, 0.10435914993286133, 0.103668212890625, 0.10411405563354492, 0.1043548583984375, 0.10399246215820312, 0.10446858406066895, 0.10427212715148926, 0.10346722602844238, 0.1038522720336914, 0.10388946533203125, 0.1042318344116211, 0.10389256477355957, 0.10394644737243652, 0.10434913635253906, 0.10389137268066406, 0.10384917259216309, 0.10385894775390625, 0.10397863388061523, 0.10420012474060059, 0.10412764549255371, 0.10411620140075684, 0.10388994216918945, 0.10370111465454102, 0.10364580154418945, 0.10544705390930176, 0.10481691360473633, 0.10430240631103516, 0.10454058647155762, 0.10408186912536621, 0.10430002212524414, 0.10460925102233887, 0.10445570945739746, 0.10380125045776367, 0.10431766510009766, 0.10416531562805176, 0.10458207130432129, 0.10413575172424316, 0.10439085960388184, 0.10413527488708496, 0.10399317741394043, 0.10395622253417969, 0.10432195663452148, 0.1038198471069336, 0.10440897941589355, 0.10369443893432617, 0.10391068458557129, 0.10410737991333008, 0.10448050498962402, 0.1039581298828125, 0.10356807708740234, 0.10398554801940918, 0.10415029525756836, 0.10412406921386719, 0.1041724681854248, 0.10410809516906738, 0.1039435863494873, 0.10402393341064453, 0.10414361953735352, 0.10390877723693848, 0.10448050498962402, 0.10428571701049805, 0.10446977615356445, 0.10434770584106445, 0.10563158988952637, 0.10464739799499512, 0.10392403602600098, 0.10489821434020996, 0.10420370101928711, 0.10401439666748047, 0.10440421104431152, 0.10487699508666992, 0.10393023490905762, 0.10392093658447266, 0.1043250560760498, 0.10373044013977051, 0.10458683967590332, 0.10452127456665039, 0.10417342185974121, 0.10477781295776367, 0.10489535331726074, 0.10486578941345215, 0.10464859008789062, 0.1045525074005127, 0.10405850410461426, 0.10381841659545898, 0.10491561889648438, 0.10358381271362305, 0.1043250560760498, 0.10385322570800781, 0.10498523712158203, 0.1043703556060791, 0.10466217994689941, 0.10430550575256348, 0.10356688499450684, 0.10430312156677246, 0.10433530807495117, 0.10496807098388672, 0.10415530204772949, 0.10406780242919922, 0.1036825180053711, 0.1042337417602539, 0.10467815399169922, 0.10445022583007812, 0.10389018058776855, 0.10470080375671387, 0.10458827018737793, 0.10387086868286133, 0.10396647453308105, 0.10431981086730957, 0.10384178161621094, 0.10426831245422363, 0.10428142547607422, 0.1036217212677002, 0.10376143455505371, 0.10494208335876465, 0.10485649108886719, 0.10437798500061035, 0.10431766510009766, 0.10450553894042969, 0.10386538505554199, 0.10434937477111816, 0.10411357879638672, 0.1040351390838623, 0.10418033599853516, 0.10463833808898926, 0.10462737083435059, 0.10496163368225098, 0.10404133796691895, 0.1049342155456543, 0.10379576683044434, 0.10441899299621582, 0.10439825057983398, 0.10423803329467773, 0.10479235649108887, 0.10442590713500977, 0.10406756401062012, 0.10425257682800293, 0.10415816307067871, 0.10419964790344238, 0.10472440719604492, 0.10437440872192383, 0.10381412506103516, 0.10400557518005371, 0.10448527336120605, 0.10423469543457031, 0.10434389114379883, 0.1044759750366211, 0.1040654182434082, 0.10431098937988281, 0.10408258438110352, 0.10437154769897461, 0.10437870025634766, 0.10438060760498047, 0.10459089279174805, 0.10424423217773438, 0.1048421859741211, 0.1045219898223877, 0.10411739349365234, 0.10469388961791992, 0.10397577285766602, 0.10414433479309082, 0.10405611991882324, 0.10503077507019043, 0.10413622856140137, 0.10404062271118164, 0.10452437400817871, 0.10879635810852051, 0.10446810722351074, 0.10608339309692383, 0.10483646392822266, 0.10375308990478516, 0.10450100898742676, 0.10474920272827148, 0.1046593189239502, 0.10595870018005371, 0.10508346557617188, 0.1041409969329834, 0.10427570343017578, 0.1041707992553711, 0.10406970977783203, 0.10411453247070312, 0.10497188568115234, 0.10492634773254395, 0.10431456565856934, 0.10464739799499512, 0.10485243797302246, 0.10474586486816406, 0.10473132133483887, 0.10422921180725098, 0.10362768173217773, 0.10468769073486328, 0.10417914390563965, 0.10469627380371094, 0.10409665107727051, 0.11046051979064941, 0.10541486740112305, 0.10537505149841309, 0.10512351989746094, 0.10496807098388672, 0.10518860816955566, 0.10570693016052246, 0.1048896312713623, 0.10532784461975098, 0.10509228706359863, 0.10497045516967773, 0.10483002662658691, 0.10511612892150879, 0.10507512092590332, 0.10486221313476562, 0.10490727424621582, 0.10556888580322266, 0.10532259941101074, 0.10448360443115234, 0.1051328182220459, 0.10509705543518066, 0.10550284385681152, 0.10521745681762695, 0.10475039482116699, 0.10465073585510254, 0.10516142845153809, 0.10509800910949707, 0.10606670379638672, 0.10533499717712402, 0.10521221160888672, 0.10498571395874023, 0.1051020622253418, 0.10520029067993164, 0.1049809455871582, 0.10538268089294434, 0.10569643974304199, 0.10549187660217285, 0.10542941093444824, 0.10507035255432129, 0.10482668876647949, 0.10583281517028809, 0.10488414764404297, 0.10529446601867676, 0.10508012771606445, 0.10514521598815918, 0.10465598106384277, 0.10515689849853516, 0.10548186302185059, 0.10548520088195801, 0.1061708927154541, 0.10820674896240234, 0.10771679878234863, 0.12375664710998535, 0.15623974800109863, 0.13608884811401367, 0.10939145088195801, 0.1060190200805664, 0.10524654388427734, 0.10572504997253418, 0.1051185131072998, 0.10485672950744629, 0.10534143447875977, 0.10569500923156738, 0.1048440933227539, 0.10529613494873047, 0.10489439964294434, 0.10545158386230469, 0.10497879981994629, 0.10547256469726562, 0.10521292686462402, 0.1051478385925293, 0.10524606704711914, 0.10509562492370605, 0.10512065887451172, 0.10574960708618164, 0.10581398010253906, 0.10549211502075195, 0.1054842472076416, 0.10519051551818848, 0.10517263412475586, 0.10478019714355469, 0.1054236888885498, 0.10539841651916504, 0.10507774353027344, 0.1050119400024414, 0.10513448715209961, 0.1048271656036377, 0.10587096214294434, 0.10474586486816406, 0.10482215881347656, 0.1050562858581543, 0.10504937171936035, 0.10525870323181152, 0.10495710372924805, 0.10495138168334961, 0.10459780693054199, 0.10547471046447754, 0.10515117645263672, 0.10504293441772461, 0.1048133373260498, 0.10531091690063477, 0.10468029975891113, 0.10481858253479004, 0.10499954223632812, 0.1109316349029541, 0.10486125946044922, 0.10559773445129395, 0.10467100143432617, 0.1050722599029541, 0.10603880882263184, 0.10559916496276855, 0.10510635375976562, 0.10502386093139648, 0.1051626205444336, 569.4801971912384]
t_14_lzf = [0.12035560607910156, 0.12053608894348145, 0.10399556159973145, 0.1038355827331543, 0.10418581962585449, 0.10410118103027344, 0.12750601768493652, 0.10717964172363281, 0.10434484481811523, 0.10378170013427734, 0.10425615310668945, 0.10424995422363281, 0.10458922386169434, 0.10414385795593262, 0.10431289672851562, 0.10437440872192383, 0.1038820743560791, 0.10389137268066406, 0.10399341583251953, 0.10418176651000977, 0.10476493835449219, 0.10363173484802246, 0.10403108596801758, 0.1035153865814209, 0.10449099540710449, 0.10354852676391602, 0.10375213623046875, 0.10363459587097168, 0.10323977470397949, 0.10415768623352051, 0.10370111465454102, 0.10349845886230469, 0.10354328155517578, 0.10402703285217285, 0.10409688949584961, 0.10377216339111328, 0.10411620140075684, 0.10432744026184082, 0.10428404808044434, 0.10400009155273438, 0.10360240936279297, 0.10370945930480957, 0.10313820838928223, 0.10415029525756836, 0.1036231517791748, 0.10429215431213379, 0.10471272468566895, 0.10527181625366211, 0.10408544540405273, 0.10410237312316895, 0.10409045219421387, 0.10371589660644531, 0.10439181327819824, 0.10423541069030762, 0.10410714149475098, 0.1055440902709961, 0.10399651527404785, 0.10415291786193848, 0.10402131080627441, 0.1038813591003418, 0.10386037826538086, 0.10411190986633301, 0.10447835922241211, 0.10376477241516113, 0.10447049140930176, 0.10367655754089355, 0.1070106029510498, 0.10428690910339355, 0.10496783256530762, 0.10418081283569336, 0.10423731803894043, 0.1045384407043457, 0.10412883758544922, 0.1045989990234375, 0.10435891151428223, 0.1044766902923584, 0.10446381568908691, 0.10465693473815918, 0.10454344749450684, 0.10414648056030273, 0.10450387001037598, 0.10412740707397461, 0.10464119911193848, 0.10448431968688965, 0.10389137268066406, 0.10455703735351562, 0.10393404960632324, 0.10444307327270508, 0.10469675064086914, 0.10419988632202148, 0.10426688194274902, 0.10461711883544922, 0.10448098182678223, 0.10378026962280273, 0.10435914993286133, 0.10403656959533691, 0.10422730445861816, 0.10423111915588379, 0.10392928123474121, 0.10415244102478027, 0.10422515869140625, 0.10413861274719238, 0.10407686233520508, 0.10469222068786621, 0.10429620742797852, 0.10392236709594727, 0.10413622856140137, 0.10417318344116211, 0.10428094863891602, 0.10389542579650879, 0.10454702377319336, 0.10426974296569824, 0.10398411750793457, 0.10431408882141113, 0.1040194034576416, 0.10381650924682617, 0.10512328147888184, 0.10417914390563965, 0.1042642593383789, 0.10411977767944336, 0.10472607612609863, 0.10349082946777344, 0.10453271865844727, 0.1071157455444336, 0.11028313636779785, 0.15662455558776855, 0.12153029441833496, 0.11249446868896484, 0.11511516571044922, 0.11400198936462402, 0.10903072357177734, 0.10635113716125488, 0.10788655281066895, 0.11166715621948242, 0.1118783950805664, 0.11336231231689453, 0.11262798309326172, 0.11406159400939941, 0.11320877075195312, 0.11130189895629883, 0.10811877250671387, 0.10460638999938965, 0.1042020320892334, 0.10680103302001953, 0.11418819427490234, 0.12674760818481445, 0.11152935028076172, 0.10446667671203613, 0.11009597778320312, 0.169189453125, 0.24266695976257324, 0.26318812370300293, 0.25161147117614746, 0.1464853286743164, 0.11309552192687988, 0.11694884300231934, 0.11012029647827148, 0.10503196716308594, 0.1068415641784668, 0.23893046379089355, 0.32551050186157227, 0.31708550453186035, 0.3995046615600586, 0.23694586753845215, 0.1130058765411377, 0.12403297424316406, 0.12435460090637207, 0.10516166687011719, 0.10740995407104492, 0.10562443733215332, 0.10439491271972656, 0.11945366859436035, 0.19361591339111328, 0.1666560173034668, 0.10628342628479004, 0.10478830337524414, 0.10521578788757324, 0.10592961311340332, 0.10446357727050781, 0.10540008544921875, 0.29495859146118164, 0.46968889236450195, 0.24761438369750977, 0.10598611831665039, 0.10467934608459473, 0.13917994499206543, 0.11632990837097168, 0.10781717300415039, 0.10772967338562012, 0.11510825157165527, 0.10780644416809082, 0.10601639747619629, 0.11028313636779785, 0.10581398010253906, 0.11011147499084473, 0.18184900283813477, 0.29082751274108887, 0.1122889518737793, 0.1118476390838623, 0.3145127296447754, 0.11682248115539551, 0.10762572288513184, 0.1042947769165039, 0.10459399223327637, 0.11302590370178223, 0.10801124572753906, 0.10432195663452148, 0.10746574401855469, 0.11487412452697754, 0.13707447052001953, 0.10957074165344238, 0.10889029502868652, 0.11272215843200684, 0.12735986709594727, 0.11505436897277832, 0.15462875366210938, 0.19011950492858887, 0.2220003604888916, 0.10535264015197754, 0.10558104515075684, 0.10447478294372559, 0.10688138008117676, 0.2128615379333496, 0.17218708992004395, 0.10447430610656738, 0.10860824584960938, 0.10667681694030762, 0.10780692100524902, 0.13050484657287598, 0.16907501220703125, 0.19478535652160645, 0.12118697166442871, 0.10705137252807617, 0.10493969917297363, 0.10492610931396484, 0.10729503631591797, 0.10689020156860352, 0.18494749069213867, 0.1072843074798584, 0.20858144760131836, 0.1073160171508789, 0.10466289520263672, 0.10494327545166016, 0.104705810546875, 0.10477685928344727, 0.10603117942810059, 0.10588932037353516, 0.12140059471130371, 0.131516695022583, 0.12118792533874512, 0.10445785522460938, 0.10479998588562012, 0.10410666465759277, 0.11139702796936035, 0.1082756519317627, 0.11788320541381836, 0.10552287101745605, 0.10550332069396973, 0.11069107055664062, 0.10465669631958008, 0.10621452331542969, 0.10472321510314941, 0.10500741004943848, 0.11347126960754395, 0.10468697547912598, 0.10411453247070312, 0.10437607765197754, 0.10423874855041504, 0.10384202003479004, 0.10455083847045898, 0.10471224784851074, 0.10416293144226074, 0.1048271656036377, 0.10349392890930176, 0.10468101501464844, 0.10392642021179199, 0.10523176193237305, 0.1039731502532959, 0.10401153564453125, 0.10495567321777344, 0.10382580757141113, 0.10474276542663574, 0.10741257667541504, 0.1079246997833252, 0.108795166015625, 0.11667346954345703, 0.1043250560760498, 0.10466361045837402, 0.10530447959899902, 0.11040163040161133, 0.10389995574951172, 0.10442996025085449, 0.10557436943054199, 0.10449028015136719, 0.10796761512756348, 0.1049661636352539, 0.10442066192626953, 0.10568451881408691, 0.10524559020996094, 0.1126089096069336, 0.1052556037902832, 0.10465502738952637, 0.10506319999694824, 0.10425877571105957, 0.10525083541870117, 0.11044144630432129, 0.11234474182128906, 0.10501909255981445, 0.10405683517456055, 0.10448765754699707, 0.10406899452209473, 0.10448765754699707, 0.10441231727600098, 0.10553598403930664, 0.10439014434814453, 0.10526323318481445, 0.1045682430267334, 0.10491776466369629, 0.10460901260375977, 0.10529708862304688, 0.10560989379882812, 0.10772395133972168, 0.10792922973632812, 0.10499024391174316, 0.10378551483154297, 0.10544562339782715, 0.10522651672363281, 0.11394524574279785, 0.10498857498168945, 0.10926365852355957, 0.10563921928405762, 0.10647368431091309, 0.10525274276733398, 0.10520100593566895, 0.10457849502563477, 0.10907626152038574, 0.10433578491210938, 0.10455799102783203, 0.10495948791503906, 0.10561680793762207, 0.11305046081542969, 0.10956692695617676, 0.10460281372070312, 0.10410213470458984, 0.1050105094909668, 0.10413122177124023, 0.10454654693603516, 0.10535383224487305, 0.10465312004089355, 0.10641002655029297, 0.10396122932434082, 0.10512661933898926, 0.10403847694396973, 0.10463666915893555, 0.10491132736206055, 0.1045389175415039, 0.10440826416015625, 0.10461235046386719, 0.10634517669677734, 0.10467076301574707, 0.10509276390075684, 0.10481953620910645, 0.10445189476013184, 0.1048424243927002, 0.10447168350219727, 0.10533690452575684, 0.10428786277770996, 0.10455775260925293, 0.10467147827148438, 0.10438656806945801, 0.10459327697753906, 0.10405230522155762, 0.1043698787689209, 0.1040949821472168, 0.10490655899047852, 0.10450887680053711, 0.10511589050292969, 0.10512590408325195, 0.10466623306274414, 0.10476422309875488, 0.10432863235473633, 0.10423088073730469, 0.1045691967010498, 0.10442757606506348, 0.1044917106628418, 0.10374975204467773, 0.10448455810546875, 0.10994362831115723, 0.11152052879333496, 0.10483050346374512, 0.10455965995788574, 0.10468745231628418, 0.10428714752197266, 0.10486912727355957, 0.10460686683654785, 0.10574960708618164, 0.10444211959838867, 0.10528445243835449, 0.10469722747802734, 0.1045229434967041, 0.10443472862243652, 0.10458564758300781, 0.10503482818603516, 0.10478520393371582, 0.10495924949645996, 0.10509252548217773, 0.10419011116027832, 0.10442686080932617, 0.10497546195983887, 0.10434961318969727, 0.1046755313873291, 0.10433578491210938, 0.10449886322021484, 0.1042485237121582, 0.1047062873840332, 0.10388326644897461, 0.1046762466430664, 0.10471749305725098, 0.10491347312927246, 0.10418200492858887, 0.10402178764343262, 0.10447001457214355, 0.10466217994689941, 0.10460615158081055, 0.10496020317077637, 0.1046597957611084, 0.10454607009887695, 0.10451579093933105, 0.1045839786529541, 0.10423612594604492, 0.1043245792388916, 0.10452604293823242, 0.10448431968688965, 0.10462236404418945, 0.10400915145874023, 0.10452485084533691, 0.10415220260620117, 0.10461163520812988, 0.10424637794494629, 0.10421013832092285, 0.1044762134552002, 0.10422182083129883, 0.10472273826599121, 0.10516166687011719, 0.10448908805847168, 0.10454821586608887, 0.10414004325866699, 0.10498380661010742, 0.10407710075378418, 0.1046142578125, 0.10448670387268066, 0.10461306571960449, 0.10458755493164062, 0.10431385040283203, 0.10493302345275879, 0.10424661636352539, 0.10487961769104004, 0.1040186882019043, 0.10423660278320312, 0.1045677661895752, 0.10479187965393066, 0.10423874855041504, 0.10466623306274414, 0.10462117195129395, 0.10401368141174316, 0.10435843467712402, 0.10582828521728516, 0.10421562194824219, 0.10442686080932617, 0.10412383079528809, 0.10464167594909668, 0.10439610481262207, 0.10535311698913574, 0.10469627380371094, 0.10499072074890137, 0.1060647964477539, 0.10450434684753418, 0.10411381721496582, 0.10400128364562988, 0.10454678535461426, 0.10434937477111816, 0.1046442985534668, 0.10412764549255371, 0.10410785675048828, 0.10454416275024414, 0.1195979118347168, 0.104278564453125, 0.10447812080383301, 0.10448503494262695, 0.10479617118835449, 0.1049337387084961, 0.10587048530578613, 0.1042473316192627, 0.10463643074035645, 0.1192162036895752, 0.10470294952392578, 0.10472822189331055, 0.1044468879699707, 0.10492420196533203, 0.10430574417114258, 0.10510563850402832, 0.10420489311218262, 0.10480856895446777, 0.10606813430786133, 0.10499453544616699, 0.1046593189239502, 0.10448479652404785, 0.10448837280273438, 0.12067151069641113, 0.19180870056152344, 0.10571670532226562, 0.1051187515258789, 0.10565590858459473, 0.10441327095031738, 0.1047966480255127, 0.10484433174133301, 0.10471558570861816, 0.10557174682617188, 0.1045541763305664, 0.10439276695251465, 0.1045377254486084, 0.10423922538757324, 0.10449051856994629, 0.10405707359313965, 0.1057746410369873, 0.10498785972595215, 0.10464906692504883, 0.10512614250183105, 0.10442185401916504, 0.10428023338317871, 0.10471129417419434, 0.10485506057739258, 0.10482287406921387, 0.10445547103881836, 0.10413908958435059, 0.10458707809448242, 0.10505223274230957, 0.10537004470825195, 0.10431623458862305, 0.10426974296569824, 0.10439920425415039, 0.10428071022033691, 0.10480070114135742, 0.1044168472290039, 0.10528779029846191, 0.10479259490966797, 0.10419273376464844, 0.10478949546813965, 0.10428524017333984, 0.10474944114685059, 0.10574221611022949, 0.1045370101928711, 0.10425591468811035, 0.10424637794494629, 0.1046299934387207, 0.10402393341064453, 0.1049046516418457, 0.10431957244873047, 0.10402369499206543, 0.10434675216674805, 0.10435628890991211, 0.10399341583251953, 0.1039426326751709, 0.10441303253173828, 0.10446977615356445, 0.10457873344421387, 0.10471749305725098, 0.10405778884887695, 0.10425829887390137, 0.10454607009887695, 0.10418534278869629, 0.10390710830688477, 0.1043851375579834, 0.10442090034484863, 0.10443282127380371, 0.10397887229919434, 0.10424947738647461, 0.10530948638916016, 0.1043844223022461, 0.10428309440612793, 0.10415768623352051, 0.10398054122924805, 0.1049044132232666, 0.1040191650390625, 0.10459280014038086, 0.10499715805053711, 0.10407209396362305, 0.10423707962036133, 0.10484552383422852, 0.10421371459960938, 0.10367918014526367, 0.10433626174926758, 0.10527849197387695, 0.10452127456665039, 0.10410022735595703, 0.10408329963684082, 0.10503125190734863, 0.10437369346618652, 0.10468769073486328, 0.10496068000793457, 0.10443782806396484, 0.10439872741699219, 0.10389113426208496, 0.10476446151733398, 0.10406136512756348, 0.10432004928588867, 0.10444521903991699, 0.10445833206176758, 0.10456538200378418, 0.10375690460205078, 0.10485005378723145, 0.10388708114624023, 0.1047670841217041, 0.10417842864990234, 0.10419416427612305, 0.10407447814941406, 0.10406041145324707, 0.10648322105407715, 0.10467004776000977, 0.10386276245117188, 0.10465836524963379, 0.10420846939086914, 0.10500693321228027, 0.10359358787536621, 0.10457158088684082, 0.10397505760192871, 0.10416769981384277, 0.10426974296569824, 0.11004805564880371, 0.10429072380065918, 0.10378837585449219, 0.10457134246826172, 0.10396146774291992, 0.10500741004943848, 0.1041257381439209, 0.10456037521362305, 0.10426855087280273, 0.10440349578857422, 0.10428261756896973, 0.10396885871887207, 0.10537266731262207, 0.1046602725982666, 0.1041562557220459, 0.10396480560302734, 0.10403728485107422, 0.10454678535461426, 0.10435748100280762, 0.10446023941040039, 0.10418081283569336, 0.10425615310668945, 0.10429978370666504, 0.10403823852539062, 0.10447311401367188, 0.10382914543151855, 0.10429644584655762, 0.10390353202819824, 0.10479068756103516, 0.10395288467407227, 0.10380291938781738, 0.10458517074584961, 0.1043243408203125, 0.10451412200927734, 0.10487937927246094, 0.10438919067382812, 0.10402488708496094, 0.10439562797546387, 0.10455608367919922, 0.1038968563079834, 0.1043400764465332, 0.10433411598205566, 0.10408782958984375, 0.10420513153076172, 0.10366559028625488, 0.10489559173583984, 0.10450506210327148, 0.10464286804199219, 0.1045525074005127, 0.10400152206420898, 0.10422301292419434, 0.10427021980285645, 0.10485219955444336, 0.10463523864746094, 0.10423135757446289, 0.1042931079864502, 0.10428643226623535, 0.10444307327270508, 0.10380029678344727, 0.10459637641906738, 0.10504484176635742, 0.10417842864990234, 0.10392498970031738, 0.10419082641601562, 0.10436129570007324, 0.10402154922485352, 0.1046445369720459, 0.10415935516357422, 0.10399746894836426, 0.10427689552307129, 0.10429215431213379, 0.10415768623352051, 0.10407495498657227, 0.10413289070129395, 0.10416841506958008, 0.10454249382019043, 0.10449099540710449, 0.1037440299987793, 0.10429596900939941, 0.10415315628051758, 0.1044473648071289, 0.10421538352966309, 0.10431957244873047, 0.10406279563903809, 0.1043086051940918, 0.10422325134277344, 0.10378098487854004, 0.10390901565551758, 0.10412859916687012, 0.10432624816894531, 0.10460519790649414, 0.10397076606750488, 0.1042013168334961, 0.10437631607055664, 0.10441756248474121, 0.10371661186218262, 0.10435032844543457, 0.10413837432861328, 0.10378599166870117, 0.10446667671203613, 0.10457253456115723, 0.10447549819946289, 0.10385847091674805, 0.10427093505859375, 0.1041727066040039, 0.10480165481567383, 0.1042027473449707, 0.10389947891235352, 0.10547733306884766, 0.10424995422363281, 0.1042637825012207, 0.1040658950805664, 0.10423398017883301, 0.10423159599304199, 0.10429525375366211, 0.10427999496459961, 0.1037747859954834, 0.10434317588806152, 0.10429215431213379, 0.10419106483459473, 0.1044921875, 0.10426545143127441, 0.1041562557220459, 0.10411763191223145, 0.1048269271850586, 0.10419988632202148, 0.10442590713500977, 0.10447835922241211, 0.10404276847839355, 0.10442256927490234, 0.10470080375671387, 0.10461926460266113, 0.10497331619262695, 0.10501980781555176, 0.10481595993041992, 0.10374736785888672, 0.10438823699951172, 0.10396981239318848, 0.1041724681854248, 0.10397744178771973, 0.10400533676147461, 0.10457253456115723, 0.1039130687713623, 0.10418128967285156, 0.10493588447570801, 0.10425329208374023, 0.10431194305419922, 0.10439538955688477, 0.10522103309631348, 0.10480785369873047, 0.10416364669799805, 0.10422444343566895, 0.10469865798950195, 0.10412311553955078, 0.10413837432861328, 0.1048126220703125, 0.10436582565307617, 0.10457062721252441, 0.10407423973083496, 0.10418939590454102, 0.10401296615600586, 0.10376858711242676, 0.10445380210876465, 0.10402035713195801, 0.10420989990234375, 0.10418486595153809, 0.10464143753051758, 0.10420989990234375, 0.10406684875488281, 0.10468506813049316, 0.1036827564239502, 0.10492777824401855, 0.10425329208374023, 0.10446047782897949, 0.10477352142333984, 0.10440182685852051, 0.10489153861999512, 0.10383319854736328, 0.1044301986694336, 0.10450983047485352, 0.10439586639404297, 0.104888916015625, 0.1042327880859375, 0.1042640209197998, 0.10478448867797852, 0.10483813285827637, 0.1039738655090332, 0.10466885566711426, 0.10457277297973633, 0.10423612594604492, 0.10442018508911133, 0.10406303405761719, 0.10437202453613281, 0.10371875762939453, 0.10411500930786133, 0.10464811325073242, 0.10513949394226074, 0.10494208335876465, 0.10378575325012207, 0.10446834564208984, 0.1042778491973877, 0.10436153411865234, 0.10451436042785645, 0.10482168197631836, 0.10467052459716797, 0.10428953170776367, 0.1047370433807373, 0.10415267944335938, 0.1043550968170166, 0.10430264472961426, 0.10449600219726562, 0.10456156730651855, 0.10389328002929688, 0.1044609546661377, 0.1041405200958252, 0.10425806045532227, 0.10438704490661621, 0.10390043258666992, 0.1043241024017334, 0.10435628890991211, 0.10529232025146484, 0.10357046127319336, 0.10407853126525879, 0.10475587844848633, 0.10424923896789551, 0.10454344749450684, 0.10506081581115723, 0.10477972030639648, 0.1046299934387207, 0.11406350135803223, 0.10488557815551758, 0.1052694320678711, 0.10479569435119629, 0.10432696342468262, 0.10492396354675293, 0.10443353652954102, 0.10540461540222168, 0.10472631454467773, 0.10456395149230957, 0.10453963279724121, 0.10431098937988281, 0.10477900505065918, 0.104461669921875, 0.10482382774353027, 0.10600996017456055, 0.10509967803955078, 0.10486578941345215, 0.10489320755004883, 0.10529017448425293, 0.10419583320617676, 0.10435891151428223, 0.10583996772766113, 0.10446953773498535, 0.10472464561462402, 0.10401558876037598, 0.10468745231628418, 0.10415792465209961, 0.10526633262634277, 0.10471582412719727, 0.10452914237976074, 0.10474896430969238, 0.10431718826293945, 0.10471653938293457, 0.1037740707397461, 0.1044769287109375, 0.10437798500061035, 0.10430073738098145, 0.10495591163635254, 0.10412287712097168, 0.10416579246520996, 0.1042640209197998, 0.1045222282409668, 0.10467839241027832, 0.10462737083435059, 0.10453414916992188, 0.10389161109924316, 0.10589194297790527, 0.10460519790649414, 0.10909342765808105, 0.10497403144836426, 0.1048431396484375, 0.10550713539123535, 0.10481977462768555, 0.10547351837158203, 0.1050271987915039, 0.10514974594116211, 0.10494732856750488, 0.1049501895904541, 0.1050729751586914, 0.10549449920654297, 0.10586857795715332, 0.10500550270080566, 0.1058497428894043, 0.10612130165100098, 0.10536885261535645, 0.10619235038757324, 0.10549187660217285, 0.10608601570129395, 0.10557365417480469, 0.10729384422302246, 0.12636065483093262, 0.16482067108154297, 0.16431641578674316, 0.16411495208740234, 0.16423869132995605, 0.16378545761108398, 0.1641683578491211, 0.16433429718017578, 0.16421151161193848, 0.1639552116394043, 0.16415023803710938, 0.1645970344543457, 0.16382837295532227, 0.1652078628540039, 0.16454696655273438, 0.16458773612976074, 0.1646411418914795, 0.16370344161987305, 0.1645059585571289, 0.1640021800994873, 0.17517662048339844, 0.1647050380706787, 0.1641373634338379, 0.16417860984802246, 0.16469383239746094, 0.1656482219696045, 0.1643996238708496, 0.16473913192749023, 0.16417574882507324, 0.16415834426879883, 0.1657395362854004, 0.16417598724365234, 0.16491436958312988, 0.16446518898010254, 0.16472101211547852, 0.16416001319885254, 0.16434979438781738, 0.16388940811157227, 0.16398167610168457, 0.1641697883605957, 0.16385865211486816, 0.163801908493042, 0.16455578804016113, 0.163954496383667, 0.16405081748962402, 0.16346359252929688, 0.16419053077697754, 0.16367769241333008, 0.16412115097045898, 0.16417193412780762, 0.16416072845458984, 0.16619539260864258, 0.16366982460021973, 0.1643688678741455, 0.16382336616516113, 0.16431736946105957, 0.1656479835510254, 0.1638932228088379, 0.16399717330932617, 0.16374731063842773, 0.16548371315002441, 0.16414642333984375, 0.1642463207244873, 0.16405510902404785, 0.16461682319641113, 0.1640162467956543, 0.16412687301635742, 0.16461920738220215, 0.16395950317382812, 0.1654520034790039, 0.16417551040649414, 0.16399312019348145, 0.16435027122497559, 0.16411638259887695, 0.16489505767822266, 0.1640627384185791, 0.1662139892578125, 0.16474509239196777, 0.16433405876159668, 0.16613078117370605, 0.16379714012145996, 0.16428041458129883, 0.16364717483520508, 0.16376781463623047, 0.16380524635314941, 0.16454648971557617, 0.1654052734375, 0.16405701637268066, 0.1647632122039795, 0.16412854194641113, 0.16506004333496094, 0.16500139236450195, 0.16478824615478516, 0.1637561321258545, 0.16654348373413086, 0.1644282341003418, 0.16384363174438477, 0.16477394104003906, 0.16413640975952148, 0.1641526222229004, 0.1646583080291748, 0.16374874114990234, 0.1641521453857422, 0.16411423683166504, 0.16456079483032227, 0.1646432876586914, 0.16426420211791992, 0.1642606258392334, 0.16385746002197266, 0.16419339179992676, 0.16373991966247559, 0.16388487815856934, 0.16347718238830566, 0.16361021995544434, 0.1638784408569336, 0.16314315795898438, 0.1645359992980957, 0.16336607933044434, 0.16291570663452148, 0.16347026824951172, 0.1637129783630371, 0.16428112983703613, 0.1629788875579834, 0.16697192192077637, 0.1681528091430664, 0.20632457733154297, 0.19800925254821777, 0.10429668426513672, 0.10420441627502441, 0.10436034202575684, 0.10467743873596191, 0.10399174690246582, 0.10445427894592285, 0.10440230369567871, 0.10401773452758789, 0.10436081886291504, 0.1047215461730957, 0.10459327697753906, 0.10454082489013672, 0.10458087921142578, 0.10526394844055176, 0.10445857048034668, 0.10494422912597656, 0.10417938232421875, 0.10428595542907715, 0.10418486595153809, 0.10487055778503418, 0.10428237915039062, 0.10391998291015625, 0.1042776107788086, 0.10474514961242676, 0.10459184646606445, 0.10419440269470215, 0.10419988632202148, 0.10417032241821289, 0.10407280921936035, 0.10441184043884277, 0.10461759567260742, 0.10540103912353516, 0.10486268997192383, 0.10423564910888672, 0.10484433174133301, 0.10393285751342773, 0.10400938987731934, 0.10366082191467285, 0.10528826713562012, 0.10446667671203613, 0.10448145866394043, 0.10400915145874023, 0.10491156578063965, 0.10397863388061523, 0.1043097972869873, 0.10459685325622559, 0.10367512702941895, 0.10451912879943848, 0.10419511795043945, 0.10421156883239746, 0.10430169105529785, 0.10413002967834473, 0.10453629493713379, 0.10437965393066406, 0.10418081283569336, 0.10446405410766602, 0.10411977767944336, 0.10444283485412598, 0.1041722297668457, 0.10456681251525879, 0.10366368293762207, 0.10418558120727539, 0.10400843620300293, 0.10568356513977051, 0.1047205924987793, 0.10405921936035156, 0.10424470901489258, 0.10438394546508789, 0.10435867309570312, 0.1045682430267334, 0.10398626327514648, 0.10447406768798828, 0.10383725166320801, 0.10451269149780273, 0.10378146171569824, 0.10399794578552246, 0.10384225845336914, 0.10427045822143555, 0.10432052612304688, 0.10494160652160645, 0.10491180419921875, 0.10397505760192871, 0.10459446907043457, 0.10438036918640137, 0.104248046875, 0.10432219505310059, 0.10485649108886719, 0.10500550270080566, 0.1042022705078125, 0.10438394546508789, 0.10449099540710449, 0.10426735877990723, 0.10518336296081543, 0.10433602333068848, 0.10474467277526855, 0.1042020320892334, 0.10511970520019531, 0.10466861724853516, 0.10454511642456055, 0.1050424575805664, 0.10441875457763672, 0.10439682006835938, 0.1045074462890625, 0.1045830249786377, 0.10419845581054688, 0.10500597953796387, 0.1052086353302002, 0.10463500022888184, 0.10479164123535156, 0.10462212562561035, 0.10442757606506348, 0.10489749908447266, 0.10428857803344727, 0.10427594184875488, 0.10409855842590332, 0.1049950122833252, 0.10379528999328613, 0.1048121452331543, 0.10431075096130371, 0.10485363006591797, 0.1044464111328125, 0.10450482368469238, 0.10506558418273926, 0.10439062118530273, 0.10486459732055664, 0.10500645637512207, 0.10541749000549316, 0.10920190811157227, 0.10446572303771973, 0.10438847541809082, 0.10436749458312988, 0.10419774055480957, 0.10411238670349121, 0.10491704940795898, 0.10508036613464355, 0.10413408279418945, 0.10471153259277344, 0.10398411750793457, 0.1048283576965332, 0.1042490005493164, 0.10488343238830566, 0.10400962829589844, 0.10455083847045898, 0.10441732406616211, 0.10415124893188477, 0.10421037673950195, 0.10454297065734863, 0.10464811325073242, 0.10445427894592285, 0.10464906692504883, 0.1043548583984375, 0.10399699211120605, 0.1047370433807373, 0.10618710517883301, 0.10488390922546387, 0.10475707054138184, 0.1046452522277832, 0.10469555854797363, 0.1045832633972168, 0.10514664649963379, 0.1044454574584961, 0.10439538955688477, 0.10432982444763184, 0.10439157485961914, 0.1055746078491211, 0.10422968864440918, 0.10892510414123535, 0.1044168472290039, 0.10431742668151855, 0.1043400764465332, 0.10413312911987305, 0.10458087921142578, 0.10451006889343262, 0.1044306755065918, 0.1052713394165039, 0.10437893867492676, 0.1042487621307373, 0.10405397415161133, 0.10512542724609375, 0.1042780876159668, 0.10450172424316406, 0.10515880584716797, 0.10478591918945312, 0.10429739952087402, 0.10447168350219727, 0.10438275337219238, 0.10413742065429688, 0.1048893928527832, 0.10441875457763672, 0.10434746742248535, 0.10430788993835449, 0.10395002365112305, 0.10476851463317871, 0.10411763191223145, 0.10435271263122559, 0.10459470748901367, 0.10420393943786621, 0.10469865798950195, 0.10424613952636719, 0.10457348823547363, 0.10440516471862793, 0.10433840751647949, 0.10444283485412598, 0.10496807098388672, 0.10457754135131836, 0.10454797744750977, 0.10602831840515137, 0.10491180419921875, 0.10460519790649414, 0.1045687198638916, 0.10418272018432617, 0.10457611083984375, 0.10420656204223633, 0.10449719429016113, 0.10443925857543945, 0.10427355766296387, 0.10453391075134277, 0.1046299934387207, 0.10441946983337402, 0.10425019264221191, 0.10453248023986816, 0.10424399375915527, 0.1056511402130127, 0.10472607612609863, 0.10434985160827637, 0.1047048568725586, 0.1041867733001709, 0.10420036315917969, 0.10431289672851562, 0.10552287101745605, 0.10481834411621094, 0.10419058799743652, 0.1048893928527832, 0.10416793823242188, 0.10448789596557617, 0.10456204414367676, 0.10434079170227051, 0.1045389175415039, 0.10434627532958984, 0.10442876815795898, 0.10387206077575684, 0.10442972183227539, 0.10416364669799805, 0.10446643829345703, 0.10414981842041016, 0.10502886772155762, 0.10438275337219238, 0.10413670539855957, 0.1043097972869873, 0.1045217514038086, 0.10455894470214844, 0.10486435890197754, 0.10484194755554199, 0.10421514511108398, 0.10413455963134766, 0.10448241233825684, 0.10379624366760254, 0.10433197021484375, 0.10445928573608398, 0.10476207733154297, 0.10449028015136719, 0.10425949096679688, 0.10449552536010742, 0.10369157791137695, 0.10462284088134766, 0.10415077209472656, 0.10411715507507324, 0.10445880889892578, 0.1042938232421875, 0.10469675064086914, 0.10502505302429199, 0.10433483123779297, 0.1045525074005127, 0.10422849655151367, 0.10466980934143066, 0.10434937477111816, 0.10449671745300293, 0.10515069961547852, 0.10470223426818848, 0.10437464714050293, 0.10442304611206055, 0.10488128662109375, 0.10400795936584473, 0.10471224784851074, 0.10421943664550781, 0.10425639152526855, 0.1042330265045166, 0.10475850105285645, 0.10420680046081543, 0.10416316986083984, 0.10452747344970703, 0.10446548461914062, 0.10443949699401855, 0.10515737533569336, 0.10431051254272461, 0.10451436042785645, 0.10444188117980957, 0.10498189926147461, 0.10430312156677246, 0.10468459129333496, 0.10455965995788574, 0.10480976104736328, 0.11302304267883301, 0.1049814224243164, 0.10558819770812988, 0.10447883605957031, 0.1049656867980957, 0.10472989082336426, 0.10474038124084473, 0.10487246513366699, 0.10533332824707031, 0.1045844554901123, 0.10503053665161133, 0.1044769287109375, 0.10456585884094238, 0.10445213317871094, 0.10465335845947266, 0.10485410690307617, 0.10546517372131348, 0.10428261756896973, 0.1049032211303711, 0.10510778427124023, 0.1048738956451416, 0.10436558723449707, 0.10426139831542969, 0.10564112663269043, 0.10422730445861816, 0.10495615005493164, 0.10457468032836914, 0.10468411445617676, 0.10450124740600586, 0.10474538803100586, 0.10464096069335938, 0.10445713996887207, 0.10451984405517578, 0.10469198226928711, 0.10476016998291016, 0.10476326942443848, 0.10407304763793945, 0.10485410690307617, 0.10442161560058594, 0.10481119155883789, 0.10437488555908203, 0.10457515716552734, 0.10454845428466797, 0.10422754287719727, 0.10441374778747559, 0.10436487197875977, 0.10441279411315918, 0.10548162460327148, 0.10515975952148438, 0.10459733009338379, 0.10492515563964844, 0.10476446151733398, 0.10450887680053711, 0.10429143905639648, 0.10462713241577148, 0.10432028770446777, 0.10469365119934082, 0.10370230674743652, 0.10470056533813477, 0.10392332077026367, 0.10459661483764648, 0.1043546199798584, 0.10477423667907715, 0.10516071319580078, 0.10409188270568848, 0.10543107986450195, 0.10446858406066895, 0.10438346862792969, 0.10463190078735352, 0.10439395904541016, 0.10491085052490234, 0.1055290699005127, 0.10470890998840332, 0.10452914237976074, 0.10457372665405273, 0.10505461692810059, 0.10423135757446289, 0.10510826110839844, 0.10380339622497559, 0.10465240478515625, 0.10416913032531738, 0.10467386245727539, 0.10416936874389648, 0.10480499267578125, 0.10436630249023438, 0.10417866706848145, 0.10417032241821289, 0.1046907901763916, 0.10411739349365234, 0.10443997383117676, 0.1049039363861084, 0.10475444793701172, 0.10452055931091309, 0.10480761528015137, 0.10452818870544434, 0.1048882007598877, 0.10481119155883789, 0.10422825813293457, 0.10445213317871094, 0.1040809154510498, 0.10511231422424316, 0.10400772094726562, 0.10460424423217773, 0.10590791702270508, 0.10898923873901367, 0.1045386791229248, 0.10428881645202637, 0.10460829734802246, 0.10434865951538086, 0.10421061515808105, 0.10483336448669434, 0.10667300224304199, 0.10517549514770508, 0.10422468185424805, 0.10459375381469727, 0.1046137809753418, 0.10416579246520996, 0.10479974746704102, 0.1054379940032959, 0.10518741607666016, 0.1041259765625, 0.1049795150756836, 0.1045539379119873, 0.10447978973388672, 0.10514950752258301, 0.10492062568664551, 0.10429143905639648, 0.10444831848144531, 0.10466909408569336, 0.10467720031738281, 0.10465526580810547, 0.1045222282409668, 0.10442757606506348, 0.10419321060180664, 0.10476517677307129, 0.10471963882446289, 0.1042640209197998, 0.10427093505859375, 0.10508489608764648, 0.10396814346313477, 0.10452675819396973, 0.1037445068359375, 0.10489583015441895, 0.10414338111877441, 0.10426497459411621, 0.10411214828491211, 0.10449719429016113, 0.10440587997436523, 0.10463809967041016, 0.10442781448364258, 0.10411286354064941, 0.10456204414367676, 0.1044764518737793, 0.10407233238220215, 0.10436630249023438, 0.10384130477905273, 0.10472774505615234, 0.10433673858642578, 0.10430598258972168, 0.10502004623413086, 0.1045839786529541, 0.1046299934387207, 0.10412096977233887, 0.10492491722106934, 0.10385990142822266, 0.10454583168029785, 0.10541439056396484, 0.10457229614257812, 0.10432004928588867, 0.10436630249023438, 0.10471105575561523, 0.10420036315917969, 0.1048123836517334, 0.10438203811645508, 0.10402226448059082, 0.10440516471862793, 0.1040036678314209, 0.10499167442321777, 0.10408520698547363, 0.10427522659301758, 0.10513615608215332, 0.10447311401367188, 0.10464143753051758, 0.10377979278564453, 0.10426902770996094, 0.1043546199798584, 0.10459232330322266, 0.10462093353271484, 0.10465168952941895, 0.10444521903991699, 0.10407185554504395, 0.10802006721496582, 0.10444045066833496, 0.10420036315917969, 0.10425925254821777, 0.10429978370666504, 0.10446786880493164, 0.10372257232666016, 0.10477089881896973, 0.10426759719848633, 0.10470962524414062, 0.104583740234375, 0.10444092750549316, 0.10438847541809082, 0.10399937629699707, 0.10449361801147461, 0.10401487350463867, 0.10542964935302734, 0.10450196266174316, 0.10442590713500977, 0.1044771671295166, 0.10429167747497559, 0.10445523262023926, 0.10435771942138672, 0.10516953468322754, 0.10477542877197266, 0.10457992553710938, 0.10469388961791992, 0.10413312911987305, 0.1046442985534668, 0.10492515563964844, 0.10443782806396484, 0.10434818267822266, 0.10459256172180176, 0.1043550968170166, 0.10404348373413086, 0.1047677993774414, 0.1041109561920166, 0.10452032089233398, 0.10432696342468262, 0.10442781448364258, 0.10402297973632812, 0.10413217544555664, 0.10417962074279785, 0.1045694351196289, 0.1045222282409668, 0.10440325736999512, 0.1047065258026123, 0.10440754890441895, 0.10357952117919922, 0.10445952415466309, 0.10390329360961914, 0.10547208786010742, 0.10477948188781738, 0.10455846786499023, 0.10449409484863281, 0.10414958000183105, 0.10463762283325195, 0.1042013168334961, 0.10462379455566406, 0.10437154769897461, 0.10467314720153809, 0.10406279563903809, 0.10436439514160156, 0.10427665710449219, 0.10495734214782715, 0.1045234203338623, 0.10425376892089844, 0.10456466674804688, 0.1046895980834961, 0.1043093204498291, 0.10479736328125, 0.10520291328430176, 0.10459423065185547, 0.10462093353271484, 0.10467863082885742, 0.10479855537414551, 0.10428190231323242, 0.10645580291748047, 0.1041560173034668, 0.10431981086730957, 0.10462498664855957, 0.10448431968688965, 0.10456371307373047, 0.10454177856445312, 0.10428047180175781, 0.10398602485656738, 0.1044924259185791, 0.10461282730102539, 0.10423088073730469, 0.10493683815002441, 0.1044917106628418, 0.10482001304626465, 0.10449790954589844, 0.1050574779510498, 0.10494852066040039, 0.10445237159729004, 0.10525155067443848, 0.10403990745544434, 0.1043083667755127, 0.10421299934387207, 0.10426688194274902, 0.1047067642211914, 0.10419702529907227, 0.10436606407165527, 0.1051950454711914, 0.10462641716003418, 0.10420727729797363, 0.10454702377319336, 0.10453224182128906, 0.10409712791442871, 0.10474300384521484, 0.10370755195617676, 0.10553789138793945, 0.10406231880187988, 0.10441112518310547, 0.10466814041137695, 0.10463762283325195, 0.10413908958435059, 0.10414004325866699, 0.10552358627319336, 0.1039736270904541, 0.1044769287109375, 0.10502815246582031, 0.10506653785705566, 0.10456347465515137, 0.10481476783752441, 0.1046290397644043, 0.1041100025177002, 0.10426807403564453, 0.10486865043640137, 0.1046609878540039, 0.10475730895996094, 0.10474753379821777, 0.10510921478271484, 0.1048116683959961, 0.10470867156982422, 0.10486221313476562, 0.1047523021697998, 0.10448932647705078, 0.10441851615905762, 0.10433745384216309, 0.10418295860290527, 0.10420823097229004, 0.10731959342956543, 0.10495400428771973, 0.10468077659606934, 0.10484528541564941, 0.10561347007751465, 0.10433626174926758, 0.10405516624450684, 0.10434651374816895, 0.10447859764099121, 0.10447430610656738, 0.10419178009033203, 0.1044771671295166, 0.10403585433959961, 0.10452103614807129, 0.10411453247070312, 0.10472607612609863, 0.10509324073791504, 0.10441470146179199, 0.10490107536315918, 0.10456085205078125, 0.10440635681152344, 0.10436463356018066, 0.10415315628051758, 0.10541176795959473, 0.10453486442565918, 0.10469412803649902, 0.10418105125427246, 0.10414409637451172, 0.10470366477966309, 0.10425162315368652, 0.10503220558166504, 0.10421895980834961, 0.10457706451416016, 0.10408639907836914, 0.10447335243225098, 0.10435175895690918, 0.10434174537658691, 0.10446858406066895, 0.10419774055480957, 0.10430908203125, 0.10437273979187012, 0.10430097579956055, 0.10419917106628418, 0.10483026504516602, 0.10469651222229004, 0.1043856143951416, 0.1054391860961914, 0.10463428497314453, 0.10502815246582031, 0.1043241024017334, 0.10441279411315918, 0.1050727367401123, 0.10397887229919434, 0.10520482063293457, 0.10388326644897461, 0.10477948188781738, 0.1046440601348877, 0.1046147346496582, 0.10428643226623535, 0.10417652130126953, 0.10407257080078125, 0.1041572093963623, 0.104278564453125, 0.10470890998840332, 0.10530281066894531, 0.10494756698608398, 0.10441970825195312, 0.10452127456665039, 0.10437202453613281, 0.10461902618408203, 0.10427069664001465, 0.1053617000579834, 0.10465264320373535, 0.10414457321166992, 0.10463881492614746, 0.10448098182678223, 0.10434556007385254, 0.10454988479614258, 0.1044011116027832, 0.10420870780944824, 0.10412383079528809, 0.10445594787597656, 0.10414314270019531, 0.10426163673400879, 0.10448980331420898, 0.10437440872192383, 0.1043863296508789, 0.10443401336669922, 0.10466337203979492, 0.10401010513305664, 0.10443925857543945, 0.10452938079833984, 0.10408854484558105, 0.1046900749206543, 0.10410475730895996, 0.10448455810546875, 0.10391592979431152, 0.10423970222473145, 0.10418891906738281, 0.10431981086730957, 0.1044614315032959, 0.10480141639709473, 0.1043863296508789, 0.10411524772644043, 0.10486626625061035, 0.10435223579406738, 0.10419487953186035, 0.1045377254486084, 0.10421228408813477, 0.10458779335021973, 0.10384225845336914, 0.10488176345825195, 0.10502815246582031, 0.10454106330871582, 0.10456585884094238, 0.10414791107177734, 0.10513424873352051, 0.10417866706848145, 0.10478973388671875, 0.1056513786315918, 0.11196446418762207, 0.10436511039733887, 0.10454106330871582, 0.10462737083435059, 0.10433673858642578, 0.10499787330627441, 0.10477280616760254, 0.10422587394714355, 0.10437560081481934, 0.10394024848937988, 0.10471582412719727, 0.10403919219970703, 0.10451245307922363, 0.10444211959838867, 0.10416698455810547, 0.10462307929992676, 0.10424375534057617, 0.1044626235961914, 0.10465812683105469, 0.10439753532409668, 0.10460925102233887, 0.10495638847351074, 0.1051032543182373, 0.10443782806396484, 0.10723996162414551, 0.10456156730651855, 0.10446476936340332, 0.10464358329772949, 0.10433626174926758, 0.10487699508666992, 0.10413599014282227, 0.10456728935241699, 0.10454845428466797, 0.1044015884399414, 0.10430240631103516, 0.10436344146728516, 0.1044914722442627, 0.10376787185668945, 0.10457158088684082, 0.1041107177734375, 0.10523509979248047, 0.10488772392272949, 0.10400128364562988, 0.10505557060241699, 0.10436606407165527, 0.10462570190429688, 0.10434508323669434, 0.10502099990844727, 0.10458683967590332, 0.10436725616455078, 0.10455203056335449, 0.10401058197021484, 0.10463881492614746, 0.10445785522460938, 0.10456991195678711, 0.10391116142272949, 0.10452532768249512, 0.10450077056884766, 0.10427546501159668, 0.10451245307922363, 0.1042165756225586, 0.10488200187683105, 0.10478949546813965, 0.10458779335021973, 0.10443353652954102, 0.1043236255645752, 0.10469269752502441, 0.10442829132080078, 0.10491275787353516, 0.10473823547363281, 0.10459375381469727, 0.1046450138092041, 0.10419273376464844, 0.10474586486816406, 0.10389232635498047, 0.10436558723449707, 0.10484004020690918, 0.10481595993041992, 0.10442638397216797, 0.1040794849395752, 0.1045074462890625, 0.10416126251220703, 0.10401105880737305, 0.10410380363464355, 0.10400795936584473, 0.1042788028717041, 0.1042327880859375, 0.1045374870300293, 0.1057438850402832, 0.10443997383117676, 0.10432934761047363, 0.10413002967834473, 0.10449481010437012, 0.10426807403564453, 0.10475897789001465, 0.10443568229675293, 0.10474705696105957, 0.10425996780395508, 0.10469603538513184, 0.10436844825744629, 0.10443925857543945, 0.1052103042602539, 0.10434794425964355, 0.10496878623962402, 0.10457491874694824, 0.10461997985839844, 0.10463690757751465, 0.1045064926147461, 0.10487508773803711, 0.10421943664550781, 0.10455989837646484, 0.10481667518615723, 0.10461020469665527, 0.1042032241821289, 0.10434103012084961, 0.1045219898223877, 0.10438227653503418, 0.10530757904052734, 0.10495162010192871, 0.10427188873291016, 0.1117095947265625, 0.10485577583312988, 0.1057894229888916, 0.10454511642456055, 0.10476946830749512, 0.10489130020141602, 0.10485458374023438, 0.10483694076538086, 0.10480403900146484, 0.10532855987548828, 0.10409665107727051, 0.1047360897064209, 0.10439443588256836, 0.10458064079284668, 0.10480594635009766, 0.10496854782104492, 0.1052393913269043, 0.10431385040283203, 0.10457539558410645, 0.10506749153137207, 0.10437989234924316, 0.10451817512512207, 0.10431551933288574, 0.10538387298583984, 0.1043238639831543, 0.10451674461364746, 0.10422658920288086, 0.10478639602661133, 0.10492825508117676, 0.10519862174987793, 0.10449790954589844, 0.1040642261505127, 0.10488486289978027, 0.10467886924743652, 0.10446476936340332, 0.10444831848144531, 0.10432243347167969, 0.10428714752197266, 0.10474705696105957, 0.10448312759399414, 0.1040797233581543, 0.10445189476013184, 0.10418868064880371, 0.1046743392944336, 0.10450530052185059, 0.10438060760498047, 0.10468864440917969, 0.10719871520996094, 0.10480570793151855, 0.1048579216003418, 0.10414814949035645, 0.10469889640808105, 0.10439634323120117, 0.1045536994934082, 0.10433721542358398, 0.10435366630554199, 0.10431385040283203, 0.10412263870239258, 0.10419392585754395, 0.10398292541503906, 0.10424065589904785, 0.10413670539855957, 0.10448002815246582, 0.10471487045288086, 0.10427021980285645, 0.10468626022338867, 0.1048285961151123, 0.10471796989440918, 0.10436105728149414, 0.10409951210021973, 0.10482931137084961, 0.10417747497558594, 0.10469818115234375, 0.10410499572753906, 0.10422873497009277, 0.10413837432861328, 0.10474395751953125, 0.10445833206176758, 0.10425472259521484, 0.10435175895690918, 0.10417962074279785, 0.10440373420715332, 0.10432291030883789, 0.10431051254272461, 0.10953855514526367, 0.104095458984375, 0.1049642562866211, 0.1041874885559082, 0.10413956642150879, 0.1044454574584961, 0.10459637641906738, 0.10464954376220703, 0.10435914993286133, 0.10527968406677246, 0.10448169708251953, 0.10484695434570312, 0.1042928695678711, 0.1042935848236084, 0.1042635440826416, 0.10413765907287598, 0.10472702980041504, 0.10425066947937012, 0.10423660278320312, 0.11262345314025879, 0.10454702377319336, 0.10488438606262207, 0.10428094863891602, 0.10426497459411621, 0.1045374870300293, 0.10437631607055664, 0.10436892509460449, 0.10552787780761719, 0.10461258888244629, 0.10445976257324219, 0.10458040237426758, 0.1036677360534668, 0.10400128364562988, 0.10417008399963379, 0.10498547554016113, 0.10396695137023926, 0.10408377647399902, 0.10490107536315918, 0.10454535484313965, 0.10406231880187988, 0.10542416572570801, 0.10432672500610352, 0.10477328300476074, 0.10460138320922852, 0.10523819923400879, 0.10456037521362305, 0.10484051704406738, 0.10518288612365723, 0.10488629341125488, 0.10482192039489746, 0.10440254211425781, 0.10453629493713379, 0.10424590110778809, 0.10506010055541992, 0.10453081130981445, 0.1046912670135498, 0.10448288917541504, 0.1044466495513916, 0.10465002059936523, 0.1043710708618164, 0.10422134399414062, 0.10421085357666016, 0.10433292388916016, 0.10436344146728516, 0.10429024696350098, 0.10454177856445312, 0.10444283485412598, 0.10443925857543945, 0.10430359840393066, 0.10420370101928711, 0.10432887077331543, 0.10420584678649902, 0.10443305969238281, 0.1041877269744873, 0.1046144962310791, 0.1052401065826416, 0.10448098182678223, 0.10440921783447266, 0.10443711280822754, 0.1050407886505127, 0.10411381721496582, 0.10429167747497559, 0.10498189926147461, 0.10451650619506836, 0.10477972030639648, 0.10412406921386719, 0.10486745834350586, 0.1045982837677002, 0.10484862327575684, 0.10435795783996582, 0.10435223579406738, 0.10457277297973633, 0.10463690757751465, 0.10457038879394531, 0.10429120063781738, 0.1042625904083252, 0.10497045516967773, 0.10480260848999023, 0.10470318794250488, 0.1040804386138916, 0.10453629493713379, 0.10413956642150879, 0.1043553352355957, 0.10467076301574707, 0.10490942001342773, 0.10493826866149902, 0.10462617874145508, 0.10809040069580078, 0.10453510284423828, 0.10454607009887695, 0.10398411750793457, 0.10422039031982422, 0.10438752174377441, 0.10401296615600586, 0.1045074462890625, 0.10482406616210938, 0.10435652732849121, 0.10424304008483887, 0.10452461242675781, 0.10425853729248047, 0.10440707206726074, 0.10459280014038086, 0.10451483726501465, 0.1050424575805664, 0.10472965240478516, 0.10428357124328613, 0.10480690002441406, 0.10434174537658691, 0.10438799858093262, 0.1044008731842041, 0.10546278953552246, 0.10427284240722656, 0.10454082489013672, 0.10420680046081543, 0.10424160957336426, 0.10468792915344238, 0.10511016845703125, 0.10448479652404785, 0.10431861877441406, 0.10456490516662598, 0.10464692115783691, 0.10399627685546875, 0.10474348068237305, 0.10426950454711914, 0.10459232330322266, 0.10470867156982422, 0.10425853729248047, 0.10454201698303223, 0.10394072532653809, 0.10464715957641602, 0.10414385795593262, 0.10416150093078613, 0.1046450138092041, 0.10477542877197266, 0.10424542427062988, 0.10418534278869629, 0.10439610481262207, 0.10480952262878418, 0.10477113723754883, 0.10452938079833984, 0.10453462600708008, 0.1042168140411377, 0.10420632362365723, 0.10419321060180664, 0.10416364669799805, 0.10448503494262695, 0.10431838035583496, 0.10454082489013672, 0.10454940795898438, 0.10411787033081055, 0.10518646240234375, 0.10442137718200684, 0.1047203540802002, 0.10463237762451172, 0.10467934608459473, 0.10462236404418945, 0.1053006649017334, 0.10470414161682129, 0.10492515563964844, 0.10458755493164062, 0.10456109046936035, 0.1044149398803711, 0.10492968559265137, 0.10417509078979492, 0.10504794120788574, 0.10396552085876465, 0.10461044311523438, 0.10434174537658691, 0.10465455055236816, 0.10447406768798828, 0.10408282279968262, 0.10455751419067383, 0.10420846939086914, 0.10455036163330078, 0.10493993759155273, 0.10431337356567383, 0.10460376739501953, 0.10420846939086914, 0.10453963279724121, 0.10461664199829102, 0.10506844520568848, 0.10511112213134766, 0.10556578636169434, 0.10493922233581543, 0.10429739952087402, 0.10443735122680664, 0.10422396659851074, 0.1046144962310791, 0.10461902618408203, 0.1048426628112793, 0.10511136054992676, 0.10423445701599121, 0.10480976104736328, 0.10425996780395508, 0.10439276695251465, 0.10425019264221191, 0.10449600219726562, 0.10460662841796875, 0.10489487648010254, 0.10516595840454102, 0.10423088073730469, 0.10433149337768555, 0.10461306571960449, 0.10423803329467773, 0.10448479652404785, 0.1046905517578125, 0.10486292839050293, 0.10424113273620605, 0.10491228103637695, 0.10445213317871094, 0.10496950149536133, 0.10492086410522461, 0.10419893264770508, 0.10436487197875977, 0.1040947437286377, 0.1048116683959961, 0.10464239120483398, 0.10491013526916504, 0.10472893714904785, 0.10450530052185059, 0.1044471263885498, 0.10493063926696777, 0.10448145866394043, 0.10445237159729004, 0.10459208488464355, 0.10429096221923828, 0.10452866554260254, 0.10471057891845703, 0.10391592979431152, 0.10477375984191895, 0.10769867897033691, 0.10478854179382324, 0.10570907592773438, 0.10413146018981934, 0.10499835014343262, 0.1045832633972168, 0.10470700263977051, 0.10462689399719238, 0.10431289672851562, 0.10486316680908203, 0.10448622703552246, 0.10467648506164551, 0.10431838035583496, 0.10448908805847168, 0.10464310646057129, 0.10457205772399902, 0.10591697692871094, 0.10428309440612793, 0.10491371154785156, 0.10485029220581055, 0.10499954223632812, 0.1043848991394043, 0.10424232482910156, 0.10540342330932617, 0.10441088676452637, 0.10457611083984375, 0.10424041748046875, 0.10480070114135742, 0.10456538200378418, 0.10527205467224121, 0.1044461727142334, 0.1042332649230957, 0.10447287559509277, 0.10423731803894043, 0.10476803779602051, 0.10462713241577148, 0.10439658164978027, 0.10497140884399414, 0.1041252613067627, 0.10537958145141602, 0.10385632514953613, 0.10436558723449707, 0.10441327095031738, 0.10500097274780273, 0.10515880584716797, 0.10474061965942383, 0.10519838333129883, 0.10483193397521973, 0.10571837425231934, 0.10504269599914551, 0.1047053337097168, 0.10489201545715332, 0.10407352447509766, 0.10457086563110352, 0.10419464111328125, 0.10443305969238281, 0.1047050952911377, 0.10435247421264648, 0.10470271110534668, 0.10413193702697754, 0.10446572303771973, 0.10400915145874023, 0.10451912879943848, 0.10448360443115234, 0.10515213012695312, 0.1051182746887207, 0.10444903373718262, 0.10474491119384766, 0.10399794578552246, 0.10448741912841797, 0.1046299934387207, 0.10534286499023438, 0.10450243949890137, 0.10431170463562012, 0.10447835922241211, 0.10474634170532227, 0.10434722900390625, 0.10534405708312988, 0.10454368591308594, 0.10459280014038086, 0.10418057441711426, 0.10473155975341797, 0.1044316291809082, 0.10447072982788086, 0.10433721542358398, 0.10494160652160645, 0.10933089256286621, 0.10439872741699219, 0.10466861724853516, 0.1039741039276123, 0.10418105125427246, 0.10428690910339355, 0.10441088676452637, 0.10421204566955566, 0.10403585433959961, 0.10473275184631348, 0.10435032844543457, 0.10434174537658691, 0.10419178009033203, 0.10429620742797852, 0.10439419746398926, 0.10428762435913086, 0.10442399978637695, 0.10425019264221191, 0.10497188568115234, 0.10387659072875977, 0.1042795181274414, 0.10411405563354492, 0.10386991500854492, 0.10415768623352051, 0.10433197021484375, 0.10415792465209961, 0.10500073432922363, 0.10470819473266602, 0.10445094108581543, 0.10399746894836426, 0.10488080978393555, 0.10402059555053711, 0.10423564910888672, 0.10490083694458008, 0.11583685874938965, 0.1046912670135498, 0.10416722297668457, 0.10479617118835449, 0.10419034957885742, 0.10527515411376953, 0.10436511039733887, 0.10457491874694824, 0.10408639907836914, 0.10426807403564453, 0.10432147979736328, 0.10454702377319336, 0.10466170310974121, 0.10413026809692383, 0.10434675216674805, 0.10491704940795898, 0.10390830039978027, 0.10474538803100586, 0.10409665107727051, 0.10496973991394043, 0.10482621192932129, 0.10475635528564453, 0.10528373718261719, 0.10729098320007324, 0.10593676567077637, 0.1050875186920166, 0.10463953018188477, 0.10431337356567383, 0.10470008850097656, 0.10456347465515137, 0.1039283275604248, 0.10468816757202148, 0.10479402542114258, 0.10451340675354004, 0.10466670989990234, 0.10449838638305664, 0.10447216033935547, 0.1041860580444336, 0.10481739044189453, 0.10559725761413574, 0.10490202903747559, 0.10484766960144043, 0.10454130172729492, 0.10501837730407715, 0.10418224334716797, 0.10402965545654297, 0.10413384437561035, 0.10545206069946289, 0.1040639877319336, 0.10496258735656738, 0.10430526733398438, 0.10425281524658203, 0.10496997833251953, 0.1053004264831543, 0.10461235046386719, 0.10497832298278809, 0.10434818267822266, 0.10460472106933594, 0.10419845581054688, 0.10439014434814453, 0.10418510437011719, 0.10422277450561523, 0.10504317283630371, 0.10460638999938965, 0.10442399978637695, 0.10396623611450195, 0.10455799102783203, 0.10420918464660645, 0.10459280014038086, 0.10430335998535156, 0.10487651824951172, 0.10429000854492188, 0.10419368743896484, 0.10434675216674805, 0.10432600975036621, 0.10462474822998047, 0.10460138320922852, 0.10412096977233887, 0.10480856895446777, 0.1050260066986084, 0.10504555702209473, 0.10404253005981445, 0.10427308082580566, 0.10429215431213379, 0.10441231727600098, 0.10449361801147461, 0.10416793823242188, 0.10533452033996582, 0.10426592826843262, 0.10469651222229004, 0.10435318946838379, 0.1049203872680664, 0.10492563247680664, 0.1040346622467041, 0.10515689849853516, 0.10428047180175781, 0.10413551330566406, 0.10451006889343262, 0.10465717315673828, 0.10499835014343262, 0.10441350936889648, 0.10440874099731445, 0.10409951210021973, 0.10426044464111328, 0.10439085960388184, 0.10447168350219727, 0.10441136360168457, 0.1042778491973877, 0.10468125343322754, 0.10427975654602051, 0.10501408576965332, 0.1043698787689209, 0.10451436042785645, 0.10440969467163086, 0.10435914993286133, 0.10471510887145996, 0.10460400581359863, 0.10508513450622559, 0.10496234893798828, 0.10619735717773438, 0.10926198959350586, 0.10473322868347168, 0.10448002815246582, 0.1042778491973877, 0.10503387451171875, 0.10459160804748535, 0.10453605651855469, 0.10543179512023926, 0.10445809364318848, 0.10511207580566406, 0.10418820381164551, 0.10452389717102051, 0.10481691360473633, 0.10440874099731445, 0.10541033744812012, 0.10480237007141113, 0.10506558418273926, 0.10457777976989746, 0.10516715049743652, 0.10489559173583984, 0.10428690910339355, 0.10442972183227539, 0.1048896312713623, 0.10515022277832031, 0.10410714149475098, 0.10449695587158203, 0.10452437400817871, 0.10436844825744629, 0.10522818565368652, 0.10444927215576172, 0.1046590805053711, 0.10431766510009766, 0.10489177703857422, 0.10454249382019043, 0.10475039482116699, 0.10417747497558594, 0.10443234443664551, 0.10469913482666016, 0.10448431968688965, 0.10420536994934082, 0.1045379638671875, 0.1042475700378418, 0.10458040237426758, 0.1041715145111084, 0.10471677780151367, 0.1041266918182373, 0.10429143905639648, 0.10613727569580078, 0.10561108589172363, 0.10465002059936523, 0.10428261756896973, 0.10453128814697266, 0.10418844223022461, 0.10457944869995117, 0.10474228858947754, 0.10456275939941406, 0.10429716110229492, 0.10420989990234375, 0.10438847541809082, 0.10436058044433594, 0.10435009002685547, 0.10480952262878418, 0.10426115989685059, 0.10538196563720703, 0.10478472709655762, 0.10475301742553711, 0.10414934158325195, 0.10515069961547852, 0.10434484481811523, 0.10460352897644043, 0.1050252914428711, 0.1048431396484375, 0.10465598106384277, 0.10453248023986816, 0.1047525405883789, 0.10439252853393555, 0.10505557060241699, 0.1045386791229248, 0.10402941703796387, 0.10466599464416504, 0.10428476333618164, 0.10413050651550293, 0.10469675064086914, 0.1041874885559082, 0.10460162162780762, 0.10402202606201172, 0.10485148429870605, 0.10391926765441895, 0.10419774055480957, 0.10437369346618652, 0.10463953018188477, 0.10490536689758301, 0.1045064926147461, 0.10493826866149902, 0.10442996025085449, 0.10474872589111328, 0.10434412956237793, 0.10440516471862793, 0.10448813438415527, 0.10396242141723633, 0.10462760925292969, 0.10396933555603027, 0.10406851768493652, 0.1093287467956543, 0.10451579093933105, 0.1048593521118164, 0.10407400131225586, 0.10471439361572266, 0.1040351390838623, 0.10490107536315918, 0.1047365665435791, 0.10617566108703613, 0.10465574264526367, 0.10426115989685059, 0.10497117042541504, 0.1043698787689209, 0.10411858558654785, 0.10453629493713379, 0.10553979873657227, 0.10465288162231445, 0.10438084602355957, 0.10467147827148438, 0.10482954978942871, 0.10449075698852539, 0.10518431663513184, 0.10436296463012695, 0.10453128814697266, 0.10431885719299316, 0.10447144508361816, 0.10421466827392578, 0.10480165481567383, 0.1045079231262207, 0.10455179214477539, 0.10487508773803711, 0.1042940616607666, 0.10429811477661133, 0.10422205924987793, 0.10439205169677734, 0.10451793670654297, 0.10430717468261719, 0.10445070266723633, 0.10412478446960449, 0.10465598106384277, 0.10430049896240234, 0.10409879684448242, 0.10442900657653809, 0.1040496826171875, 0.10503005981445312, 0.10386300086975098, 0.1044766902923584, 0.10424590110778809, 0.1044621467590332, 0.10452747344970703, 0.10446763038635254, 0.10430765151977539, 0.10432934761047363, 0.10429596900939941, 0.10447478294372559, 0.10489559173583984, 0.10451865196228027, 0.1041719913482666, 0.10440945625305176, 0.10422921180725098, 0.1045083999633789, 0.10393309593200684, 0.10465073585510254, 0.1053934097290039, 0.10458779335021973, 0.10458946228027344, 0.1042947769165039, 0.10462069511413574, 0.10376691818237305, 0.10540914535522461, 0.10421419143676758, 0.10425591468811035, 0.10442662239074707, 0.10433220863342285, 0.1042325496673584, 0.10390424728393555, 0.104644775390625, 0.10447573661804199, 0.10424947738647461, 0.10472321510314941, 0.1038665771484375, 0.10483121871948242, 0.10422444343566895, 0.10445070266723633, 0.10497331619262695, 0.10486221313476562, 0.10501646995544434, 0.1074514389038086, 0.10516023635864258, 0.10445690155029297, 0.10452103614807129, 0.1043083667755127, 0.10460996627807617, 0.10435914993286133, 0.10431838035583496, 0.10476875305175781, 0.1046285629272461, 0.10447883605957031, 0.1043393611907959, 0.10401701927185059, 0.10448479652404785, 0.10376715660095215, 0.10453629493713379, 0.10554814338684082, 0.1049489974975586, 0.10475993156433105, 0.10491347312927246, 0.10454344749450684, 0.10414361953735352, 0.10445213317871094, 0.10437273979187012, 0.10470104217529297, 0.10410284996032715, 0.10446000099182129, 0.1045386791229248, 0.10413646697998047, 0.10490131378173828, 0.10422706604003906, 0.10423445701599121, 0.10454058647155762, 0.1042337417602539, 0.10456252098083496, 0.10416913032531738, 0.10486078262329102, 0.1043999195098877, 0.10445594787597656, 0.10480117797851562, 0.10452866554260254, 0.10427713394165039, 0.10422825813293457, 0.10480713844299316, 0.10433769226074219, 0.10446882247924805, 0.10449743270874023, 0.10447335243225098, 0.1043553352355957, 0.10966348648071289, 0.10445547103881836, 0.10562682151794434, 0.10464954376220703, 0.10469651222229004, 0.10454463958740234, 0.10488653182983398, 0.10408186912536621, 0.10470032691955566, 0.1042487621307373, 0.1046133041381836, 0.10399508476257324, 0.10461282730102539, 0.10445380210876465, 0.1042933464050293, 0.1055135726928711, 0.10410952568054199, 0.10492205619812012, 0.10443329811096191, 0.10464954376220703, 0.1048426628112793, 0.10430526733398438, 0.10538601875305176, 0.10503721237182617, 0.10451936721801758, 0.10480833053588867, 0.10458540916442871, 0.10477542877197266, 0.1042633056640625, 0.10503435134887695, 0.10417366027832031, 0.10446739196777344, 0.10440254211425781, 0.10515356063842773, 0.10505938529968262, 0.10432195663452148, 0.10490584373474121, 0.1042327880859375, 0.10471725463867188, 0.10463261604309082, 0.10480928421020508, 0.10471820831298828, 0.10468101501464844, 0.10553789138793945, 0.10534024238586426, 0.1050417423248291, 0.10480189323425293, 0.10479211807250977, 0.10469222068786621, 0.10443878173828125, 0.10512685775756836, 0.10470104217529297, 0.10511589050292969, 0.10456490516662598, 0.10477375984191895, 0.10572052001953125, 0.10459208488464355, 0.10506153106689453, 0.10448336601257324, 0.10458970069885254, 0.10458087921142578, 0.10486531257629395, 0.10496330261230469, 0.1060786247253418, 0.10528755187988281, 0.104583740234375, 0.1050560474395752, 0.10493016242980957, 0.10465264320373535, 0.10494542121887207, 0.10477352142333984, 0.10508418083190918, 0.1047055721282959, 0.10523414611816406, 0.10561013221740723, 0.10483098030090332, 0.10523819923400879, 0.10448932647705078, 0.10523033142089844, 0.10434246063232422, 0.10483527183532715, 0.10471558570861816, 0.1052851676940918, 0.10483837127685547, 0.10455894470214844, 0.10534429550170898, 0.10467767715454102, 0.1047060489654541, 0.10515618324279785, 0.10454893112182617, 0.10513639450073242, 0.10430502891540527, 0.1047670841217041, 0.10435056686401367, 0.10608887672424316, 0.10520529747009277, 0.10578107833862305, 0.10455155372619629, 0.10465764999389648, 0.10471320152282715, 0.10478425025939941, 0.10469555854797363, 0.10471963882446289, 0.10470032691955566, 0.10486292839050293, 0.10476493835449219, 0.10459184646606445, 0.10428094863891602, 0.10479855537414551, 0.10507798194885254, 0.10538673400878906, 0.10510540008544922, 0.10501670837402344, 0.10532855987548828, 0.1044163703918457, 0.10493946075439453, 0.10451555252075195, 0.10479283332824707, 0.1055746078491211, 0.10466694831848145, 0.10504698753356934, 0.10492205619812012, 0.1049041748046875, 0.10451006889343262, 0.10525226593017578, 0.1095433235168457, 0.10464978218078613, 0.10498285293579102, 0.10498571395874023, 0.10494303703308105, 0.10471034049987793, 0.10552191734313965, 0.10511136054992676, 0.10441970825195312, 0.10503149032592773, 0.10472321510314941, 0.1046609878540039, 0.10484838485717773, 0.10508894920349121, 0.10486292839050293, 0.10481786727905273, 0.10516238212585449, 0.10471510887145996, 0.10498785972595215, 0.10447382926940918, 0.10498237609863281, 0.1046295166015625, 0.10450172424316406, 0.10462045669555664, 0.10446333885192871, 0.1047205924987793, 0.10446381568908691, 0.10462784767150879, 0.1052093505859375, 0.10403656959533691, 0.10509037971496582, 0.10452938079833984, 0.10513544082641602, 0.10616087913513184, 0.10535120964050293, 0.10531044006347656, 0.10478520393371582, 0.10486268997192383, 0.10427355766296387, 0.10482120513916016, 0.10536026954650879, 0.10484194755554199, 0.10422992706298828, 0.10454607009887695, 0.10510802268981934, 0.10466337203979492, 0.10514473915100098, 0.10465097427368164, 0.10454773902893066, 0.10483574867248535, 0.10423779487609863, 0.10501480102539062, 0.10438156127929688, 0.10528683662414551, 0.1048438549041748, 0.10524368286132812, 0.10488414764404297, 0.10435843467712402, 0.10452556610107422, 0.10426187515258789, 0.1047518253326416, 0.10446882247924805, 0.10470175743103027, 0.10457301139831543, 0.10424566268920898, 0.10470294952392578, 0.10444521903991699, 0.10437512397766113, 0.10469555854797363, 0.10457134246826172, 0.10507917404174805, 0.10408377647399902, 0.10483956336975098, 0.10524225234985352, 0.10493969917297363, 0.10505461692810059, 0.10476446151733398, 0.1046907901763916, 0.1042943000793457, 0.10487818717956543, 0.10460853576660156, 0.10579633712768555, 0.10452079772949219, 0.1049048900604248, 0.10483407974243164, 0.10507512092590332, 0.10466885566711426, 0.10449433326721191, 0.10559773445129395, 0.10502266883850098, 0.10483288764953613, 0.10533380508422852, 0.10456585884094238, 0.10496020317077637, 0.10475444793701172, 0.10466742515563965, 0.10464286804199219, 0.1047980785369873, 0.10450148582458496, 0.10470104217529297, 0.10522317886352539, 0.10497570037841797, 0.10483479499816895, 0.10461688041687012, 0.10482382774353027, 0.10478568077087402, 0.10457181930541992, 0.10463786125183105, 0.10476446151733398, 0.10480976104736328, 0.10528206825256348, 0.10508871078491211, 0.10623908042907715, 0.10539650917053223, 0.10521435737609863, 0.10458564758300781, 0.10494112968444824, 0.10471391677856445, 0.1048285961151123, 0.10459327697753906, 0.10480141639709473, 0.10471677780151367, 0.10428118705749512, 0.10471010208129883, 0.10468173027038574, 0.10477399826049805, 0.10471749305725098, 0.10491657257080078, 0.10590744018554688, 0.10539865493774414, 0.10480737686157227, 0.10509085655212402, 0.10552048683166504, 0.10498452186584473, 0.10412025451660156, 0.10523772239685059, 0.10533905029296875, 0.10494828224182129, 0.10473299026489258, 0.1047353744506836, 0.1045374870300293, 0.10451459884643555, 0.10526919364929199, 0.10485720634460449, 0.10461664199829102, 0.10483217239379883, 0.10496687889099121, 0.10463404655456543, 0.10505533218383789, 0.10506725311279297, 0.10460710525512695, 0.10534858703613281, 0.10471487045288086, 0.1044166088104248, 0.10506224632263184, 0.10456562042236328, 0.10529136657714844, 0.10457634925842285, 0.10507774353027344, 0.10503602027893066, 0.10483741760253906, 0.10467219352722168, 0.10436034202575684, 0.10486841201782227, 0.10467815399169922, 0.10510969161987305, 0.10448265075683594, 0.10457539558410645, 0.10499191284179688, 0.1043543815612793, 0.1051628589630127, 0.10451960563659668, 0.10455083847045898, 0.10478901863098145, 0.10428524017333984, 0.10519576072692871, 0.10538125038146973, 0.10481905937194824, 0.10460996627807617, 0.10472345352172852, 0.10472822189331055, 0.10504341125488281, 0.10466599464416504, 0.10535073280334473, 0.10511279106140137, 0.10441184043884277, 0.10466265678405762, 0.1048886775970459, 0.10460877418518066, 0.1050713062286377, 0.10437631607055664, 0.10471153259277344, 0.10460829734802246, 0.10472559928894043, 0.10498952865600586, 0.10485434532165527, 0.10483646392822266, 0.10423636436462402, 0.1054999828338623, 0.10509657859802246, 0.10540318489074707, 0.10492491722106934, 0.10463476181030273, 0.10500073432922363, 0.10473418235778809, 0.10514640808105469, 0.10507488250732422, 0.10477948188781738, 0.11435437202453613, 0.10587215423583984, 0.10559344291687012, 0.10469603538513184, 0.1046137809753418, 0.1048884391784668, 0.10445380210876465, 0.10520219802856445, 0.10583186149597168, 0.1049814224243164, 0.1052699089050293, 0.104827880859375, 0.10478973388671875, 0.10509824752807617, 0.10510015487670898, 0.10550570487976074, 0.10647153854370117, 0.10507869720458984, 0.10526680946350098, 0.10527729988098145, 0.10528182983398438, 0.10450315475463867, 0.10541439056396484, 0.10562539100646973, 0.10515689849853516, 0.10516095161437988, 0.10493779182434082, 0.10523748397827148, 0.10495328903198242, 0.10552358627319336, 0.10488772392272949, 0.10467958450317383, 0.10517477989196777, 0.10461282730102539, 0.10482501983642578, 0.10503411293029785, 0.10492920875549316, 0.10480260848999023, 0.10515093803405762, 0.10476827621459961, 0.10458540916442871, 0.10469412803649902, 0.10450863838195801, 0.10480809211730957, 0.10474205017089844, 0.10462808609008789, 0.10451793670654297, 0.10539793968200684, 0.10591340065002441, 0.10523533821105957, 0.10485982894897461, 0.10486030578613281, 0.10437154769897461, 0.10532069206237793, 0.10440611839294434, 0.1049644947052002, 0.1047365665435791, 0.10517001152038574, 0.10462808609008789, 0.10446739196777344, 0.10471034049987793, 0.10432219505310059, 0.10512876510620117, 0.10536408424377441, 0.10545682907104492, 0.10483455657958984, 0.10485458374023438, 0.10524535179138184, 0.10486817359924316, 0.10473012924194336, 0.1046450138092041, 0.1058347225189209, 0.10500431060791016, 0.10434865951538086, 0.10627532005310059, 0.10465121269226074, 0.10500454902648926, 0.10540628433227539, 0.10463857650756836, 0.10502290725708008, 0.10467529296875, 0.1049954891204834, 0.10507631301879883, 0.10506296157836914, 0.10460948944091797, 0.10516691207885742, 0.10476136207580566, 0.10468602180480957, 0.10469818115234375, 0.10451078414916992, 0.10474205017089844, 0.10499787330627441, 0.10479021072387695, 0.1054222583770752, 0.10449862480163574, 0.10523724555969238, 0.10449957847595215, 0.10462117195129395, 0.10482978820800781, 0.10444235801696777, 0.1051340103149414, 0.10433340072631836, 0.10481595993041992, 0.11087632179260254, 0.10511326789855957, 0.10508513450622559, 0.10444092750549316, 0.10450911521911621, 0.10430526733398438, 0.10486006736755371, 0.10473489761352539, 0.10635185241699219, 0.10514593124389648, 0.10506343841552734, 0.1053006649017334, 0.10460042953491211, 0.10516190528869629, 0.10486078262329102, 0.10615897178649902, 0.10540580749511719, 0.1051783561706543, 0.10465097427368164, 0.10553741455078125, 0.10518074035644531, 0.10539603233337402, 0.10497760772705078, 0.10466456413269043, 0.10510683059692383, 0.10498428344726562, 0.10452532768249512, 0.10560750961303711, 0.10493254661560059, 0.10489964485168457, 0.10520291328430176, 0.10489416122436523, 0.10474371910095215, 0.10431885719299316, 0.10511231422424316, 0.10491466522216797, 0.10488390922546387, 0.10472488403320312, 0.10487079620361328, 0.10465002059936523, 0.10466313362121582, 0.10476040840148926, 0.1047060489654541, 0.10450625419616699, 0.10445380210876465, 0.10508370399475098, 0.1045231819152832, 0.10431480407714844, 0.10496973991394043, 0.10431098937988281, 0.10494184494018555, 0.10474324226379395, 0.10464930534362793, 0.10459303855895996, 0.10490751266479492, 0.10486531257629395, 0.1057882308959961, 0.10488224029541016, 0.10483551025390625, 0.105072021484375, 0.10484743118286133, 0.10418939590454102, 0.10494709014892578, 0.10509252548217773, 0.10497665405273438, 0.10480809211730957, 0.10494661331176758, 0.10518884658813477, 0.10447096824645996, 0.10521173477172852, 0.10431838035583496, 0.10488438606262207, 0.10483264923095703, 0.10460090637207031, 0.10503029823303223, 0.10474061965942383, 0.10529375076293945, 0.10467886924743652, 0.10512900352478027, 0.10464167594909668, 0.10458827018737793, 0.10460281372070312, 0.10453653335571289, 0.10514354705810547, 0.10472702980041504, 0.10511565208435059, 0.10525655746459961, 0.10611939430236816, 0.10563158988952637, 0.10470223426818848, 0.10454750061035156, 0.10443615913391113, 0.10491490364074707, 0.1046288013458252, 0.10475969314575195, 0.10475015640258789, 0.1044154167175293, 0.10658502578735352, 0.13328337669372559, 0.1046900749206543, 0.10497522354125977, 0.10464310646057129, 0.10533571243286133, 0.10580182075500488, 0.1052703857421875, 0.10444355010986328, 0.10492801666259766, 0.10486578941345215, 0.10481929779052734, 0.1048746109008789, 0.10506749153137207, 0.10480189323425293, 0.10474538803100586, 0.10696649551391602, 0.10459399223327637, 0.10457587242126465, 0.10555100440979004, 0.10471200942993164, 0.10442638397216797, 0.1044008731842041, 0.10466909408569336, 0.1057896614074707, 0.10486841201782227, 0.1048119068145752, 0.10467910766601562, 0.1047670841217041, 0.10555577278137207, 0.10431265830993652, 0.1044919490814209, 0.10485363006591797, 0.10501980781555176, 0.10466313362121582, 0.10520696640014648, 0.10485053062438965, 0.10486197471618652, 0.10487699508666992, 0.10463118553161621, 0.1050405502319336, 0.10530257225036621, 0.10544395446777344, 0.1050117015838623, 0.10500550270080566, 0.10517573356628418, 0.10462284088134766, 0.10460686683654785, 0.10507941246032715, 0.10476899147033691, 0.10450029373168945, 0.10454702377319336, 0.10477924346923828, 0.10485672950744629, 0.10612678527832031, 0.10461878776550293, 0.10471582412719727, 0.10453295707702637, 0.1055445671081543, 0.1051030158996582, 0.10499000549316406, 0.10514998435974121, 0.10474824905395508, 0.10481429100036621, 0.10431647300720215, 0.10530829429626465, 0.10476183891296387, 0.10531282424926758, 0.10482954978942871, 0.10473847389221191, 0.10474777221679688, 0.10453104972839355, 0.10519003868103027, 0.10534882545471191, 0.10493135452270508, 0.10479950904846191, 0.10495710372924805, 0.10507416725158691, 0.1043250560760498, 0.10457515716552734, 0.1049954891204834, 0.10478973388671875, 0.10520148277282715, 0.10454583168029785, 0.10753822326660156, 0.10437583923339844, 0.10529804229736328, 0.10479569435119629, 0.10457754135131836, 0.10473084449768066, 0.10443258285522461, 0.10531067848205566, 0.10474634170532227, 0.10569286346435547, 0.10464000701904297, 0.10486674308776855, 0.10481095314025879, 0.10456085205078125, 0.10492801666259766, 0.1046915054321289, 0.10484576225280762, 0.10593819618225098, 0.10521554946899414, 0.10505366325378418, 0.10475492477416992, 0.10500216484069824, 0.10433006286621094, 0.10497426986694336, 0.10531401634216309, 0.10520553588867188, 0.10484814643859863, 0.10471677780151367, 0.10506272315979004, 0.10493183135986328, 0.10533833503723145, 0.10491180419921875, 0.10470962524414062, 0.10491085052490234, 0.1045374870300293, 0.10509371757507324, 0.11819243431091309, 0.13250327110290527, 0.10661888122558594, 0.10515284538269043, 0.1047511100769043, 0.10432863235473633, 0.10484075546264648, 0.10448670387268066, 0.1048281192779541, 0.1046907901763916, 0.10505175590515137, 0.10493946075439453, 0.10515451431274414, 0.1065359115600586, 0.10490608215332031, 0.10503888130187988, 0.10472488403320312, 0.10516619682312012, 0.1049959659576416, 0.10474562644958496, 0.10489010810852051, 0.10447430610656738, 0.10485696792602539, 0.10463166236877441, 0.10443735122680664, 0.10452055931091309, 0.10430121421813965, 0.10507893562316895, 0.10468769073486328, 0.10580587387084961, 0.1050252914428711, 0.10470342636108398, 0.10485148429870605, 0.10446429252624512, 0.10499429702758789, 0.10450005531311035, 0.10530614852905273, 0.10501289367675781, 0.10501313209533691, 0.10444808006286621, 0.10896611213684082, 0.10587215423583984, 0.10515904426574707, 0.10477900505065918, 0.10458159446716309, 0.10499358177185059, 0.10470819473266602, 0.10470390319824219, 0.10522031784057617, 0.10463476181030273, 0.10517239570617676, 0.10508608818054199, 0.10489439964294434, 0.10464024543762207, 0.10433173179626465, 0.10536837577819824, 0.10471534729003906, 0.10529899597167969, 0.10585999488830566, 0.10572171211242676, 0.10537171363830566, 0.10488438606262207, 0.10485649108886719, 0.10474729537963867, 0.10475540161132812, 0.10476994514465332, 0.10473871231079102, 0.10500121116638184, 0.10500836372375488, 0.10482168197631836, 0.10479307174682617, 0.10456299781799316, 0.10474061965942383, 0.10462450981140137, 0.10477209091186523, 0.10469269752502441, 0.1065514087677002, 0.10483551025390625, 0.10534954071044922, 0.10518765449523926, 0.10506343841552734, 0.10490536689758301, 0.10459685325622559, 0.10543227195739746, 0.10449409484863281, 0.10526418685913086, 0.10499215126037598, 0.10555243492126465, 0.10479235649108887, 0.10484671592712402, 0.10501646995544434, 0.10442137718200684, 0.10489797592163086, 0.10463261604309082, 0.10506057739257812, 0.10535073280334473, 0.10448575019836426, 0.10529851913452148, 0.10505294799804688, 0.1052393913269043, 0.10451722145080566, 0.10483360290527344, 0.10472679138183594, 0.1044163703918457, 0.10495114326477051, 0.10476255416870117, 0.10485243797302246, 0.10456705093383789, 0.10489988327026367, 0.10487842559814453, 0.10474133491516113, 0.1048882007598877, 0.10486626625061035, 0.10504555702209473, 0.10472559928894043, 0.10469555854797363, 0.10565042495727539, 0.10459017753601074, 0.10522150993347168, 0.10439252853393555, 0.10503244400024414, 0.10454845428466797, 0.10482335090637207, 0.10473370552062988, 0.10540008544921875, 0.10488438606262207, 0.10453510284423828, 0.105072021484375, 0.1052546501159668, 0.10487174987792969, 0.10467696189880371, 0.1051628589630127, 0.1052553653717041, 0.10469675064086914, 0.10468363761901855, 0.10529637336730957, 0.10519862174987793, 0.1053934097290039, 0.10416507720947266, 0.10485577583312988, 0.10456037521362305, 0.10493755340576172, 0.10480833053588867, 0.10553359985351562, 0.10491061210632324, 0.1049952507019043, 0.10489225387573242, 0.10498309135437012, 0.10482597351074219, 0.1047663688659668, 0.1047666072845459, 0.10487985610961914, 0.10502290725708008, 0.10503172874450684, 0.10527300834655762, 0.10591721534729004, 0.10584306716918945, 0.10509109497070312, 0.10487127304077148, 0.10439157485961914, 0.10504722595214844, 0.10459446907043457, 0.10498523712158203, 0.10499024391174316, 0.10464024543762207, 0.10486388206481934, 0.10502314567565918, 0.10500001907348633, 0.10469174385070801, 0.10477256774902344, 0.1048581600189209, 0.10544610023498535, 0.10575985908508301, 0.10486817359924316, 0.1047670841217041, 0.10575580596923828, 0.10495233535766602, 0.10514545440673828, 0.10460829734802246, 0.10625004768371582, 0.10453295707702637, 0.10540413856506348, 0.10448741912841797, 0.10490036010742188, 0.10509061813354492, 0.10561895370483398, 0.10472226142883301, 0.10466480255126953, 0.10485625267028809, 0.10496878623962402, 0.10525727272033691, 0.10492873191833496, 0.10491394996643066, 0.10477256774902344, 0.1049659252166748, 0.10509276390075684, 0.10439825057983398, 0.10529804229736328, 0.10487127304077148, 0.10498809814453125, 0.10505080223083496, 0.10454463958740234, 0.10553216934204102, 0.1044003963470459, 0.10499787330627441, 0.1044473648071289, 0.10480308532714844, 0.10532212257385254, 0.10468316078186035, 0.10476946830749512, 0.10465359687805176, 0.10502195358276367, 0.10480213165283203, 0.10490560531616211, 0.10479307174682617, 0.10448408126831055, 0.10478472709655762, 0.10458230972290039, 0.10504722595214844, 0.10600399971008301, 0.10498261451721191, 0.10504674911499023, 0.10474061965942383, 0.10516238212585449, 0.10525059700012207, 0.10478520393371582, 0.10541605949401855, 0.10511541366577148, 0.1047666072845459, 0.10453987121582031, 0.1050105094909668, 0.10459566116333008, 0.10508346557617188, 0.10463428497314453, 0.10484695434570312, 0.10537886619567871, 0.10483956336975098, 0.10505938529968262, 0.10551071166992188, 0.10477685928344727, 0.10504460334777832, 0.10498213768005371, 0.10494208335876465, 0.10423469543457031, 0.10507845878601074, 0.10484623908996582, 0.10511159896850586, 0.1054532527923584, 0.10543227195739746, 0.10548186302185059, 0.10482907295227051, 0.1108248233795166, 0.10543465614318848, 0.10640740394592285, 0.10506272315979004, 0.10498785972595215, 0.10535883903503418, 0.10515189170837402, 0.10519576072692871, 0.10570001602172852, 0.1055755615234375, 0.1054067611694336, 0.10471963882446289, 0.10526323318481445, 0.1046609878540039, 0.10520052909851074, 0.10512351989746094, 0.10692834854125977, 0.10550928115844727, 0.10543966293334961, 0.10612654685974121, 0.10483789443969727, 0.10497546195983887, 0.10484910011291504, 0.10598444938659668, 0.10508918762207031, 0.10534453392028809, 0.10489034652709961, 0.10495567321777344, 0.10522890090942383, 0.10591483116149902, 0.10495662689208984, 0.10540986061096191, 0.10470390319824219, 0.10497689247131348, 0.10473227500915527, 0.10543370246887207, 0.10472679138183594, 0.10493588447570801, 0.10530543327331543, 0.10493350028991699, 0.104949951171875, 0.10473799705505371, 0.10507774353027344, 0.10456442832946777, 0.10505390167236328, 0.10438251495361328, 0.10485625267028809, 0.10819149017333984, 0.10551977157592773, 0.10517692565917969, 0.10500168800354004, 0.10479021072387695, 0.10529112815856934, 0.10512328147888184, 0.10524606704711914, 0.10454297065734863, 0.10525989532470703, 0.10462450981140137, 0.1052255630493164, 0.1045539379119873, 0.1048884391784668, 0.10482120513916016, 0.10458183288574219, 0.10600447654724121, 0.10506415367126465, 0.10543346405029297, 0.10523080825805664, 0.10550665855407715, 0.10490036010742188, 0.1048426628112793, 0.10590028762817383, 0.10508394241333008, 0.10527682304382324, 0.10509514808654785, 0.10479927062988281, 0.10509085655212402, 0.10531949996948242, 0.10503435134887695, 0.10432934761047363, 0.10511302947998047, 0.10469317436218262, 0.10506582260131836, 0.10547351837158203, 0.1049649715423584, 0.10479211807250977, 0.10462450981140137, 0.10526323318481445, 0.1047050952911377, 0.10477399826049805, 0.10506796836853027, 0.10463309288024902, 0.10522627830505371, 0.1051175594329834, 0.10549116134643555, 0.1050100326538086, 0.10525393486022949, 0.10494589805603027, 0.10451149940490723, 0.1050267219543457, 0.10446834564208984, 0.10532712936401367, 0.10476922988891602, 0.10465216636657715, 0.11782050132751465, 0.10483503341674805, 0.10523796081542969, 0.10447430610656738, 0.10489439964294434, 0.10453963279724121, 0.10493063926696777, 0.10544157028198242, 0.10590934753417969, 0.10483455657958984, 0.10480809211730957, 0.10506367683410645, 0.10511541366577148, 0.10494804382324219, 0.10504913330078125, 0.10566568374633789, 0.10539722442626953, 0.10477471351623535, 0.10448646545410156, 0.10542631149291992, 0.1046907901763916, 0.10587787628173828, 0.10459399223327637, 0.10505390167236328, 0.10423803329467773, 0.10509896278381348, 0.10479354858398438, 0.10544180870056152, 0.10522103309631348, 0.10453033447265625, 0.10529613494873047, 0.10463094711303711, 0.10445332527160645, 0.10520219802856445, 0.1046910285949707, 0.10519075393676758, 0.10443878173828125, 0.10493946075439453, 0.10468196868896484, 0.1044151782989502, 0.10477590560913086, 0.10460472106933594, 0.10491800308227539, 0.10463571548461914, 0.1046438217163086, 0.1044151782989502, 0.10493779182434082, 0.10497093200683594, 0.10500168800354004, 0.10462212562561035, 0.10451078414916992, 0.1044764518737793, 0.10474514961242676, 0.10466694831848145, 0.10478448867797852, 0.10519599914550781, 0.10526704788208008, 0.10447120666503906, 0.10486578941345215, 0.10498547554016113, 0.10456418991088867, 0.10475897789001465, 0.10432291030883789, 0.10567998886108398, 0.10457777976989746, 0.10488080978393555, 0.10460543632507324, 0.10493636131286621, 0.1046915054321289, 0.10554146766662598, 0.10472393035888672, 0.10457634925842285, 0.10496902465820312, 0.10512328147888184, 0.1049046516418457, 0.10518097877502441, 0.10533809661865234, 0.10519242286682129, 0.10436534881591797, 0.10497236251831055, 0.10423803329467773, 0.1049659252166748, 0.10513925552368164, 0.10521864891052246, 0.10502481460571289, 0.10469269752502441, 0.10514688491821289, 0.10583949089050293, 0.10544157028198242, 0.10480213165283203, 0.10462355613708496, 0.10441017150878906, 0.10455632209777832, 0.10461640357971191, 0.10461258888244629, 0.10509371757507324, 0.10476922988891602, 0.10489606857299805, 0.10488390922546387, 0.10437130928039551, 0.10489797592163086, 0.10475397109985352, 0.10494685173034668, 0.10621404647827148, 0.1047811508178711, 0.10498404502868652, 0.1046903133392334, 0.10500645637512207, 0.10441708564758301, 0.10479164123535156, 0.10529804229736328, 0.10506677627563477, 0.10452556610107422, 0.10474324226379395, 0.1047675609588623, 0.10472869873046875, 0.10544466972351074, 0.10493779182434082, 0.10455822944641113, 0.10495209693908691, 0.10438179969787598, 0.10512304306030273, 0.1048576831817627, 0.10464835166931152, 0.10471534729003906, 0.10494709014892578, 0.10493636131286621, 0.10443472862243652, 0.1049802303314209, 0.10426139831542969, 0.10501766204833984, 0.10479497909545898, 0.10491228103637695, 0.10515761375427246, 0.10463905334472656, 0.1047353744506836, 0.10468697547912598, 0.10458898544311523, 0.10559630393981934, 0.10472989082336426, 0.10489177703857422, 0.1041421890258789, 0.10519671440124512, 0.10500121116638184, 0.10453271865844727, 0.10489344596862793, 0.10481119155883789, 0.10497570037841797, 0.10456323623657227, 0.10465812683105469, 0.10490608215332031, 0.10615944862365723, 0.10499811172485352, 0.104736328125, 0.10475778579711914, 0.10467410087585449, 0.10515570640563965, 0.10470199584960938, 0.10560035705566406, 0.10490965843200684, 0.10450601577758789, 0.10516619682312012, 0.10461664199829102, 0.10516619682312012, 0.10480213165283203, 0.10450339317321777, 0.10514497756958008, 0.10459256172180176, 0.1048576831817627, 0.10468769073486328, 0.10556721687316895, 0.10433030128479004, 0.10488104820251465, 0.10464286804199219, 0.10473084449768066, 0.10436606407165527, 0.10461235046386719, 0.10506224632263184, 0.10464668273925781, 0.10529541969299316, 0.1052696704864502, 0.10539555549621582, 0.10501599311828613, 0.10526895523071289, 0.10489320755004883, 0.10430192947387695, 0.10516047477722168, 0.10487604141235352, 0.1047976016998291, 0.10492730140686035, 0.10509800910949707, 0.10535097122192383, 0.10438323020935059, 0.10516786575317383, 0.10436749458312988, 0.10493779182434082, 0.10469627380371094, 0.10468840599060059, 0.10598540306091309, 0.10576128959655762, 0.1050717830657959, 0.1056063175201416, 0.1051626205444336, 0.1048882007598877, 0.10451459884643555, 0.10513567924499512, 0.10553407669067383, 0.10516953468322754, 0.10507988929748535, 0.10488724708557129, 0.10536003112792969, 0.10453534126281738, 0.10505032539367676, 0.10460281372070312, 0.1048593521118164, 0.1044321060180664, 0.10505080223083496, 0.11132359504699707, 0.10622310638427734, 0.10504817962646484, 0.10476875305175781, 0.10498404502868652, 0.10448741912841797, 0.10479426383972168, 0.10509490966796875, 0.10432076454162598, 0.1051335334777832, 0.10461997985839844, 0.10457444190979004, 0.10491156578063965, 0.10660147666931152, 0.10579562187194824, 0.10438752174377441, 0.10501885414123535, 0.10450387001037598, 0.10509252548217773, 0.10515904426574707, 0.10501718521118164, 0.10486054420471191, 0.10460376739501953, 0.10482239723205566, 0.10454368591308594, 0.10481500625610352, 0.10485410690307617, 0.10441255569458008, 0.10492563247680664, 0.10606622695922852, 0.10560894012451172, 0.10472989082336426, 0.10498714447021484, 0.10519623756408691, 0.1046133041381836, 0.10497331619262695, 0.10597515106201172, 0.1050262451171875, 0.10428023338317871, 0.1048583984375, 0.10479211807250977, 0.109832763671875, 0.10579776763916016, 0.1046605110168457, 0.10493755340576172, 0.104736328125, 0.10512590408325195, 0.10502910614013672, 0.1052091121673584, 0.10514283180236816, 0.10477447509765625, 0.10466313362121582, 0.10474896430969238, 0.10459017753601074, 0.10472822189331055, 0.1049509048461914, 0.10514187812805176, 0.10462427139282227, 0.10543203353881836, 0.10515809059143066, 0.10511445999145508, 0.10533905029296875, 0.10523533821105957, 0.10471153259277344, 0.10454511642456055, 0.10492348670959473, 0.10455942153930664, 0.1048896312713623, 0.10456538200378418, 0.10516214370727539, 0.10457324981689453, 0.10488533973693848, 0.10491347312927246, 0.10453319549560547, 0.10468721389770508, 0.10482192039489746, 0.10503554344177246, 0.10663223266601562, 0.10464239120483398, 0.10527706146240234, 0.10473442077636719, 0.1051790714263916, 0.10451412200927734, 0.10503077507019043, 0.10605430603027344, 0.10458683967590332, 0.1048886775970459, 0.10501313209533691, 0.10464024543762207, 0.1047830581665039, 0.10525321960449219, 0.10512328147888184, 0.10467219352722168, 0.10510563850402832, 0.10475325584411621, 0.10496926307678223, 0.10551762580871582, 0.10469508171081543, 0.10486745834350586, 0.1049041748046875, 0.10506391525268555, 0.1043393611907959, 0.10487699508666992, 0.10437488555908203, 0.10484695434570312, 0.10511493682861328, 0.10453581809997559, 0.10502886772155762, 0.10471773147583008, 0.10548043251037598, 0.10516571998596191, 0.1046590805053711, 0.10486102104187012, 0.10512471199035645, 0.10519766807556152, 0.10464072227478027, 0.10543298721313477, 0.10493922233581543, 0.10472559928894043, 0.10519886016845703, 0.10436224937438965, 0.10518503189086914, 0.1041707992553711, 0.10493826866149902, 0.10591864585876465, 0.10476183891296387, 0.10494852066040039, 0.10459494590759277, 0.10533022880554199, 0.1048276424407959, 0.10487604141235352, 0.10562252998352051, 0.1051321029663086, 0.1049039363861084, 0.10465764999389648, 0.1049036979675293, 0.10460925102233887, 0.10497379302978516, 0.10481429100036621, 0.10460400581359863, 0.10505914688110352, 0.10440492630004883, 0.1054837703704834, 0.10473322868347168, 0.10521245002746582, 0.10546302795410156, 0.10481119155883789, 0.10488247871398926, 0.10476040840148926, 0.10510897636413574, 0.10484004020690918, 0.10458207130432129, 0.10513734817504883, 0.10503816604614258, 0.10527968406677246, 0.10447502136230469, 0.10670971870422363, 0.10506653785705566, 0.10501885414123535, 0.10503768920898438, 0.10469770431518555, 0.10510587692260742, 0.1045694351196289, 0.10489702224731445, 0.1053168773651123, 0.10495305061340332, 0.10506701469421387, 0.10478568077087402, 0.10444927215576172, 0.10451602935791016, 0.10486674308776855, 0.10519623756408691, 0.1059567928314209, 0.10489654541015625, 0.10470700263977051, 0.10507035255432129, 0.10462045669555664, 0.10486578941345215, 0.10470890998840332, 0.1058344841003418, 0.10495400428771973, 0.1047520637512207, 0.1049504280090332, 0.10443544387817383, 0.10503768920898438, 0.1049811840057373, 0.10497140884399414, 0.10499119758605957, 0.1045069694519043, 0.10477781295776367, 0.10472416877746582, 0.10551285743713379, 0.10501623153686523, 0.10500741004943848, 0.10528945922851562, 0.10497617721557617, 0.10483217239379883, 0.10486626625061035, 0.10482311248779297, 0.10509371757507324, 0.10463070869445801, 0.10495686531066895, 0.10504961013793945, 0.10493850708007812, 0.10468268394470215, 0.10487675666809082, 0.10456037521362305, 0.10527896881103516, 0.10471534729003906, 0.10470914840698242, 0.1048583984375, 0.10450911521911621, 0.1048576831817627, 0.1044766902923584, 0.10537314414978027, 0.10453557968139648, 0.10515737533569336, 0.10482072830200195, 0.10455656051635742, 0.10550665855407715, 0.10563516616821289, 0.10456418991088867, 0.10493659973144531, 0.10469579696655273, 0.10533499717712402, 0.10446786880493164, 0.10482525825500488, 0.11178469657897949, 0.12286520004272461, 0.11445832252502441, 0.11693859100341797, 0.11614418029785156, 0.11754274368286133, 0.2267298698425293, 0.1850433349609375, 0.10785794258117676, 0.10667991638183594, 0.10574030876159668, 0.10552525520324707, 0.1061859130859375, 0.10594630241394043, 0.10504579544067383, 0.10517525672912598, 0.10546326637268066, 0.10589194297790527, 0.10625529289245605, 0.10523724555969238, 0.10749697685241699, 0.10514235496520996, 0.11708378791809082, 0.10567641258239746, 0.16148114204406738, 0.31997251510620117, 0.1429133415222168, 0.1051030158996582, 0.10482978820800781, 0.10583353042602539, 0.10592961311340332, 0.10506176948547363, 0.10498547554016113, 0.1054840087890625, 0.1063086986541748, 0.10524320602416992, 0.10887718200683594, 0.10551071166992188, 0.10571575164794922, 0.10762596130371094, 0.10477495193481445, 0.1059865951538086, 0.10517024993896484, 0.10570359230041504, 0.10505867004394531, 0.10520005226135254, 0.10514163970947266, 0.10535025596618652, 0.10628557205200195, 0.11517596244812012, 0.10944962501525879, 0.10533022880554199, 0.10762715339660645, 0.11486530303955078, 0.11342048645019531, 0.11558938026428223, 0.11246585845947266, 0.11294341087341309, 0.11543917655944824, 0.15655779838562012, 0.19541335105895996, 0.20572304725646973, 0.13206195831298828, 0.10722780227661133, 0.14630985260009766, 0.21985816955566406, 0.11981010437011719, 0.1051335334777832, 0.10543560981750488, 0.10536861419677734, 0.10484576225280762, 0.10492873191833496, 0.10492396354675293, 0.10494422912597656, 0.10479569435119629, 0.1056680679321289, 0.10556340217590332, 0.10609078407287598, 0.10550642013549805, 0.10516047477722168, 0.10610175132751465, 0.1055300235748291, 0.10617685317993164, 0.10520434379577637, 0.10663104057312012, 0.1052389144897461, 0.10505008697509766, 0.10528349876403809, 0.11062836647033691, 0.1047208309173584, 0.10544180870056152, 0.10459566116333008, 0.10512804985046387, 0.10511970520019531, 0.10480785369873047, 0.1051492691040039, 0.10480642318725586, 0.10504651069641113, 0.10457396507263184, 0.10496950149536133, 0.10531377792358398, 0.10501265525817871, 0.10513448715209961, 0.10543537139892578, 0.10502219200134277, 0.10512995719909668, 0.1051182746887207, 0.10531949996948242, 0.10485315322875977, 0.10570693016052246, 0.10452818870544434, 0.10489821434020996, 0.1047217845916748, 0.10506963729858398, 0.10492110252380371, 0.10489487648010254, 0.11172318458557129, 0.10499024391174316, 0.10532236099243164, 0.10485529899597168, 0.10483670234680176, 0.1049036979675293, 0.10503530502319336, 0.10498690605163574, 0.10596632957458496, 0.10523629188537598, 0.1049191951751709, 0.10533785820007324, 0.10506510734558105, 0.10467243194580078, 0.10477042198181152, 0.10492753982543945, 0.10641789436340332, 0.10514998435974121, 0.10542011260986328, 0.10580730438232422, 0.10471463203430176, 0.10508084297180176, 0.10486984252929688, 0.10487627983093262, 0.10480713844299316, 0.10500502586364746, 0.10484862327575684, 0.10458946228027344, 0.10543394088745117, 0.10471606254577637, 0.1052548885345459, 0.10497117042541504, 0.10450458526611328, 0.10499191284179688, 0.10476946830749512, 0.10473370552062988, 0.10497021675109863, 0.10513091087341309, 0.1046140193939209, 0.10500979423522949, 0.10466790199279785, 0.10481119155883789, 0.1051783561706543, 0.10443377494812012, 0.10503792762756348, 0.10476398468017578, 0.10472702980041504, 0.10476040840148926, 0.10463595390319824, 0.10483145713806152, 0.10452795028686523, 0.10461282730102539, 0.10444998741149902, 0.10504579544067383, 0.10464024543762207, 0.10489678382873535, 0.105316162109375, 0.10451841354370117, 0.1049187183380127, 0.10452699661254883, 0.10526084899902344, 0.10483789443969727, 0.1049799919128418, 0.10570120811462402, 0.10447001457214355, 0.1050560474395752, 0.10475540161132812, 0.10520720481872559, 0.10504961013793945, 0.10509920120239258, 0.1048133373260498, 0.10458874702453613, 0.10477399826049805, 0.10436153411865234, 0.10497069358825684, 0.10492491722106934, 0.10505270957946777, 0.10497331619262695, 0.10456061363220215, 0.1053769588470459, 0.10449767112731934, 0.10503864288330078, 0.10507059097290039, 0.10512971878051758, 0.10506653785705566, 0.10507893562316895, 0.10545659065246582, 0.10501599311828613, 0.10656118392944336, 0.1052558422088623, 0.10450482368469238, 0.1048879623413086, 0.10500741004943848, 0.10487890243530273, 0.10464906692504883, 0.10485434532165527, 0.1048588752746582, 0.10487985610961914, 0.1051473617553711, 0.10448217391967773, 0.10467815399169922, 0.10480976104736328, 0.10480475425720215, 0.10525298118591309, 0.10553264617919922, 0.10498929023742676, 0.10463738441467285, 0.10555672645568848, 0.10441923141479492, 0.1047816276550293, 0.10486388206481934, 0.10577988624572754, 0.1047670841217041, 0.10477900505065918, 0.1049954891204834, 0.10469961166381836, 0.10488319396972656, 0.10513710975646973, 0.10461616516113281, 0.10486006736755371, 0.10455465316772461, 0.10489416122436523, 0.10494565963745117, 0.10494780540466309, 0.1050105094909668, 0.10480308532714844, 0.1052546501159668, 0.10455322265625, 0.10484528541564941, 0.1046292781829834, 0.10470366477966309, 0.10501456260681152, 0.10480022430419922, 0.10461163520812988, 0.10474443435668945, 0.10478925704956055, 0.1046762466430664, 0.1047983169555664, 0.10594701766967773, 0.10570311546325684, 0.10633015632629395, 0.10490226745605469, 0.1051027774810791, 0.10597038269042969, 0.10548043251037598, 0.10539388656616211, 0.10583901405334473, 0.10490179061889648, 0.10484814643859863, 0.10494232177734375, 0.10488462448120117, 0.10568046569824219, 0.10464286804199219, 0.10520553588867188, 0.10479998588562012, 0.10509181022644043, 0.10552644729614258, 0.10452842712402344, 0.10524582862854004, 0.10552096366882324, 0.10504865646362305, 0.1047666072845459, 0.10486292839050293, 0.10488104820251465, 0.10489845275878906, 0.1048440933227539, 0.10486745834350586, 0.10515451431274414, 0.10485076904296875, 0.1045997142791748, 0.10506582260131836, 0.1046609878540039, 0.10514545440673828, 0.10482478141784668, 0.10541892051696777, 0.10490131378173828, 0.10501480102539062, 0.10528826713562012, 0.10474395751953125, 0.1053314208984375, 0.1053626537322998, 0.1052694320678711, 0.10751700401306152, 0.1051187515258789, 0.10514569282531738, 0.10446786880493164, 0.10517406463623047, 0.10495591163635254, 0.10507965087890625, 0.10474944114685059, 0.1042945384979248, 0.10598611831665039, 0.10445666313171387, 0.10550427436828613, 0.1049342155456543, 0.10483598709106445, 0.10512208938598633, 0.10479545593261719, 0.10547924041748047, 0.10526585578918457, 0.10522127151489258, 0.10463094711303711, 0.10548996925354004, 0.10519576072692871, 0.10454750061035156, 0.10484886169433594, 0.10571622848510742, 0.10514998435974121, 0.10511970520019531, 0.10489821434020996, 0.10580253601074219, 0.10474538803100586, 0.10522723197937012, 0.10449838638305664, 0.10505986213684082, 0.10478830337524414, 0.10503053665161133, 0.1052243709564209, 0.10482335090637207, 0.10492348670959473, 0.1044919490814209, 0.10548257827758789, 0.10483598709106445, 0.10487890243530273, 0.10525989532470703, 0.10454058647155762, 0.10516881942749023, 0.10465264320373535, 0.10514426231384277, 0.10518074035644531, 0.10666227340698242, 0.10592937469482422, 0.10478878021240234, 0.10510826110839844, 0.10460019111633301, 0.10520148277282715, 0.1050410270690918, 0.10498499870300293, 0.10515880584716797, 0.10507631301879883, 0.10484051704406738, 0.10493779182434082, 0.10489177703857422, 0.10505223274230957, 0.10490608215332031, 0.10491561889648438, 0.10516929626464844, 0.10474133491516113, 0.10499048233032227, 0.10521626472473145, 0.10521125793457031, 0.10495495796203613, 0.10506701469421387, 0.10442376136779785, 0.10684990882873535, 0.10541129112243652, 0.10524415969848633, 0.10506629943847656, 0.10480570793151855, 0.1048431396484375, 0.10480761528015137, 0.10494709014892578, 0.10446405410766602, 0.10488462448120117, 0.10485672950744629, 0.10506677627563477, 0.10490608215332031, 0.10460925102233887, 0.10497593879699707, 0.1052849292755127, 0.10508179664611816, 0.10502982139587402, 0.10479307174682617, 0.10544514656066895, 0.1048281192779541, 0.10498404502868652, 0.10546636581420898, 0.10509347915649414, 0.1073610782623291, 0.1051938533782959, 0.1048591136932373, 0.10470962524414062, 0.10483479499816895, 0.10463905334472656, 0.10522747039794922, 0.1045389175415039, 0.10530972480773926, 0.10465764999389648, 0.10480380058288574, 0.1050257682800293, 0.10443735122680664, 0.1051936149597168, 0.10474419593811035, 0.105316162109375, 0.10562634468078613, 0.10517001152038574, 0.10500216484069824, 0.10497283935546875, 0.10539007186889648, 0.10471558570861816, 0.10449600219726562, 0.10623931884765625, 0.10477137565612793, 0.1050424575805664, 0.10436797142028809, 0.1054840087890625, 0.10475826263427734, 0.1051034927368164, 0.10470438003540039, 0.10481858253479004, 0.10472893714904785, 0.10472941398620605, 0.10483050346374512, 0.1051626205444336, 0.10474634170532227, 0.10488390922546387, 0.10486149787902832, 0.10538148880004883, 0.10413217544555664, 0.10497832298278809, 0.10465359687805176, 0.10466361045837402, 0.10505819320678711, 0.1048429012298584, 0.10496950149536133, 0.10445046424865723, 0.1046757698059082, 0.10486555099487305, 0.10467672348022461, 0.10462641716003418, 0.10441780090332031, 0.10492992401123047, 0.10429930686950684, 0.10518002510070801, 0.10485219955444336, 0.10454130172729492, 0.10504364967346191, 0.10427141189575195, 0.10507059097290039, 0.10437512397766113, 0.10462737083435059, 0.10485005378723145, 0.10577917098999023, 0.10494565963745117, 0.1042928695678711, 0.1051490306854248, 0.10469651222229004, 0.10470080375671387, 0.10432910919189453, 0.10603785514831543, 0.10470294952392578, 0.10479450225830078, 0.10479235649108887, 0.10480523109436035, 0.10468816757202148, 0.1057431697845459, 0.1046144962310791, 0.10490727424621582, 0.10438156127929688, 0.10499286651611328, 0.10434365272521973, 0.10522913932800293, 0.10498929023742676, 0.10502147674560547, 0.10475277900695801, 0.10476136207580566, 0.10532188415527344, 0.10474228858947754, 0.10518980026245117, 0.10498952865600586, 0.10512900352478027, 0.10545778274536133, 0.1050424575805664, 0.10610532760620117, 0.10605072975158691, 0.10507392883300781, 0.10489702224731445, 0.10455060005187988, 0.10475969314575195, 0.10462069511413574, 0.10456347465515137, 0.10472440719604492, 0.10492968559265137, 0.10477685928344727, 0.10487794876098633, 0.10477614402770996, 0.1046895980834961, 0.10508966445922852, 0.10476088523864746, 0.1055746078491211, 0.10509109497070312, 0.10495519638061523, 0.10517692565917969, 0.10481929779052734, 0.10504627227783203, 0.10468459129333496, 0.10513687133789062, 0.1064603328704834, 0.10475826263427734, 0.10509133338928223, 0.10443615913391113, 0.10516357421875, 0.10444855690002441, 0.10549235343933105, 0.10469627380371094, 0.10471820831298828, 0.10491681098937988, 0.10472631454467773, 0.10515522956848145, 0.10453557968139648, 0.10504984855651855, 0.1052241325378418, 0.10496258735656738, 0.10473251342773438, 0.10452008247375488, 0.10503792762756348, 0.10501885414123535, 0.10499358177185059, 0.10497307777404785, 0.10523390769958496, 0.10470986366271973, 0.10460638999938965, 0.1050863265991211, 0.10451483726501465, 0.1049954891204834, 0.10508394241333008, 0.10507082939147949, 0.10484743118286133, 0.10446882247924805, 0.10522222518920898, 0.10434794425964355, 0.10537958145141602, 0.10464620590209961, 0.10480546951293945, 0.10458922386169434, 0.10451889038085938, 0.10507369041442871, 0.10537481307983398, 0.10464358329772949, 0.10450983047485352, 0.10465884208679199, 0.10514616966247559, 0.10505056381225586, 0.10554981231689453, 0.10546684265136719, 0.10493302345275879, 0.10504388809204102, 0.10496211051940918, 0.10510659217834473, 0.10466670989990234, 0.10523700714111328, 0.10461640357971191, 0.10497546195983887, 0.1052851676940918, 0.10511541366577148, 0.10488295555114746, 0.10483241081237793, 0.10482501983642578, 0.10443758964538574, 0.10461235046386719, 0.10515785217285156, 0.1047821044921875, 0.10476303100585938, 0.10492587089538574, 0.1050114631652832, 0.10509252548217773, 0.10531902313232422, 0.10554122924804688, 0.10520720481872559, 0.1099846363067627, 0.10505795478820801, 0.10593867301940918, 0.10453987121582031, 0.1053776741027832, 0.10505080223083496, 0.10516357421875, 0.10562658309936523, 0.10506963729858398, 0.10542845726013184, 0.10511183738708496, 0.10497117042541504, 0.10519528388977051, 0.10486912727355957, 0.10544061660766602, 0.10507988929748535, 0.10588860511779785, 0.1049947738647461, 0.10559749603271484, 0.10628795623779297, 0.10546302795410156, 0.10492777824401855, 0.10473942756652832, 0.10582637786865234, 0.10465812683105469, 0.10528349876403809, 0.10465598106384277, 0.10480523109436035, 0.10503339767456055, 0.10535931587219238, 0.10483312606811523, 0.104644775390625, 0.10493779182434082, 0.10518455505371094, 0.10505509376525879, 0.10511398315429688, 0.1047208309173584, 0.10536813735961914, 0.10564613342285156, 0.10504341125488281, 0.10500383377075195, 0.10466790199279785, 0.10522198677062988, 0.10449409484863281, 0.1052548885345459, 0.10458111763000488, 0.10575675964355469, 0.1059720516204834, 0.10602807998657227, 0.10504317283630371, 0.10490655899047852, 0.10502910614013672, 0.10460662841796875, 0.10573291778564453, 0.10512304306030273, 0.1047506332397461, 0.10473871231079102, 0.10474014282226562, 0.1051473617553711, 0.10462617874145508, 0.10488653182983398, 0.10464358329772949, 0.10493350028991699, 0.1059105396270752, 0.10541653633117676, 0.10503745079040527, 0.10493803024291992, 0.10542058944702148, 0.10475349426269531, 0.10519099235534668, 0.10627126693725586, 0.1048593521118164, 0.10508131980895996, 0.10480380058288574, 0.10539436340332031, 0.10498905181884766, 0.10525631904602051, 0.10514211654663086, 0.10471916198730469, 0.10516619682312012, 0.10479426383972168, 0.10486984252929688, 0.10513186454772949, 0.10473227500915527, 0.10493636131286621, 0.10458993911743164, 0.10510969161987305, 0.10436081886291504, 0.10487961769104004, 0.1049659252166748, 0.10475873947143555, 0.10515952110290527, 0.10504865646362305, 0.10555529594421387, 0.10497903823852539, 0.10515785217285156, 0.10473108291625977, 0.10471892356872559, 0.10499095916748047, 0.10471773147583008, 0.10537099838256836, 0.10467338562011719, 0.10474252700805664, 0.11882758140563965, 0.10524296760559082, 0.10536909103393555, 0.10449528694152832, 0.10555386543273926, 0.10488009452819824, 0.105072021484375, 0.1051950454711914, 0.10585570335388184, 0.10539078712463379, 0.10519766807556152, 0.10564160346984863, 0.10502099990844727, 0.10469460487365723, 0.10491824150085449, 0.10674190521240234, 0.10542440414428711, 0.1051778793334961, 0.10536789894104004, 0.10497331619262695, 0.1051180362701416, 0.10666990280151367, 0.10504436492919922, 0.10522794723510742, 0.10473012924194336, 0.10530471801757812, 0.10472893714904785, 0.1052088737487793, 0.10478806495666504, 0.10504627227783203, 0.1054677963256836, 0.1048126220703125, 0.10480618476867676, 0.10471439361572266, 0.10495162010192871, 0.10471987724304199, 0.10537505149841309, 0.10477519035339355, 0.10480642318725586, 0.10474705696105957, 0.10500121116638184, 0.1047661304473877, 0.10497117042541504, 0.10472583770751953, 0.10503339767456055, 0.10470962524414062, 0.10499882698059082, 0.10454130172729492, 0.10486936569213867, 0.10502338409423828, 0.10502767562866211, 0.10517525672912598, 0.10469627380371094, 0.10471105575561523, 0.1047050952911377, 0.1060948371887207, 0.10438418388366699, 0.10512876510620117, 0.10511541366577148, 0.10494852066040039, 0.10491371154785156, 0.10491442680358887, 0.10557365417480469, 0.10559701919555664, 0.10506033897399902, 0.10492229461669922, 0.10482430458068848, 0.10480141639709473, 0.10496234893798828, 0.10503101348876953, 0.10470175743103027, 0.1051325798034668, 0.10478615760803223, 0.10483932495117188, 0.10527157783508301, 0.10472440719604492, 0.10590553283691406, 0.10512900352478027, 0.10510778427124023, 0.10478353500366211, 0.10458946228027344, 0.1055305004119873, 0.10465788841247559, 0.10530805587768555, 0.10601162910461426, 0.10539555549621582, 0.10504579544067383, 0.10639119148254395, 0.10562443733215332, 0.10478687286376953, 0.10509991645812988, 0.10499715805053711, 0.10512018203735352, 0.10523819923400879, 0.10471391677856445, 0.10528278350830078, 0.10481500625610352, 0.1051170825958252, 0.10456967353820801, 0.10496997833251953, 0.10483765602111816, 0.10466980934143066, 0.10546255111694336, 0.10552763938903809, 0.10499858856201172, 0.1048734188079834, 0.10550403594970703, 0.10488629341125488, 0.10443329811096191, 0.10541939735412598, 0.1055152416229248, 0.10522961616516113, 0.10515427589416504, 0.10502910614013672, 0.1049346923828125, 0.1046895980834961, 0.10552239418029785, 537.3106207847595]
    
ax[n].plot(t_12_none[:-1], label=f"chunk size 4096, None")
ax[n].plot(t_14_none[:-1], label=f"chunk size 16384, None")
ax[n].plot(t_12_gzip[:-1], label=f"chunk size 4096, gzip")
ax[n].plot(t_14_gzip[:-1], label=f"chunk size 16384, gzip")
ax[n].plot(t_12_lzf[:-1], label=f"chunk size 4096, lzf")
ax[n].plot(t_14_lzf[:-1], label=f"chunk size 16384, lzf")
ax[n].legend(loc='center left')
ax[n].set_title('No versioning')
            
plt.xlabel("Number of transactions")
plt.ylabel("Time (in seconds)")
plt.show()

## Summary

We can clearly see that files with smallest file size, corresponding to smaller chunk sizes, are also the ones with largest creation times. **This is consistent with the effects of using smaller chunk sizes in HDF5 files.**

This behaviour suggests that for `test_large_fraction_changes_sparse`, larger chunk sizes generate larger files, but the size of the files grows modestly as the number of transactions grow. So, **if we are dealing with a large number of transactions, larger chunk sizes generate files that are of reasonable size while having faster creation times** (and probably faster IO speeds as well).

[Back to top](#home)

<a id='test2'></a>

# Test 2: Mostly appends (sparse)

In [ ]:
testname = "test_mostly_appends_sparse"

For this case, we have tested the following number of transactions:

```python
num_transactions_2 = [50, 100, 200]
```

### Generating new tests

Change `num_transactions_2`, `exponents_2` and `compression_2` as desired (the previous warning applies: beware of very large file sizes and creation times for large numbers of versions):

In [ ]:
num_transactions_2 = [25, 50, 100, 200]
exponents_2 = [12, 14]
compression_2 = [None, "gzip", "lzf"]

Create files:

In [ ]:
testcase = performance_tests.test_mostly_appends_sparse(path=path,
                                                        num_transactions=num_transactions_2, 
                                                        exponents=exponents_2, 
                                                        compression=compression_2)
testcase_2, msg = testcase.create_files()
if msg:
    print(msg)
t_sizes_2 = [test['theoretical_sizes'] for test in testcase.tests[-len(num_transactions_2):]]

If you wish to save the results for later, execute the cell below:

In [ ]:
#testcase.save(testcase_2)

### Reading previously computed tests

To read an existing `.json` file, uncomment the lines below.

In [ ]:
#with open(f"{testname}.json", "r") as json_in:
#    testcase_2 = json.load(json_in)
#t_sizes_2 = None

## Analysis

Repeating the same analysis as in the previous test, let's show the size information in a graph:

In [ ]:
num_transactions_2 = [test['num_transactions'] for test in testcase_2]
chunk_sizes_2 = [test['chunk_size'] for test in testcase_2]
compression_2 = [test['compression'] for test in testcase_2]

filesizes_2 = np.array([test['size'] for test in testcase_2])
sizelabels_2 = np.array([test['size_label'] for test in testcase_2])

n = len(set(num_transactions_2))
ncs = len(set(chunk_sizes_2))
ncomp = len(set(compression_2))

fig_mostly_appends = plt.figure(figsize=(14,10))

if t_sizes_2:
    plt.plot(num_transactions_2[:n], t_sizes_2, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_2[:n], 
                 filesizes_2[start+j*n:start+(j+1)*n], 
                 '*--', ms=12, 
                 label=f"Chunk size {chunk_sizes_2[start+j*n]}, {compression_2[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
plt.yticks(filesizes_2[-n:], sizelabels_2[-n:])
plt.xticks(num_transactions_2[:n])
plt.grid(True)
plt.show()

Changing the view to a logarithmic scale, we have the following:

In [ ]:
fig_mostly_appends_log = plt.figure(figsize=(14,10))

if t_sizes_2:
    plt.loglog(num_transactions_2[:n], t_sizes_2, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.loglog(num_transactions_2[:n], 
                   filesizes_2[start+j*n:start+(j+1)*n], 
                   '*--', ms=12, 
                   label=f"Chunk size 2**{chunk_sizes_2[start+j*n]}, {compression_2[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
plt.grid()
plt.xticks(num_transactions_2[:n], num_transactions_2[:n])
plt.yticks(filesizes_2[-n:], sizelabels_2[-n:])
plt.minorticks_off()
plt.show()

### Comparing compression algorithms

For each chunk size that we chose to test, let's compare the file sizes corresponding to each compression algorithm that we used.

In [ ]:
fig_comp_2, ax_comp_2 = plt.subplots(ncs, figsize=(10,8), sharey=True)
fig_comp_2.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_comp_2[j].loglog(num_transactions_2[:n],
                            filesizes_2[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"{compression_2[start]}")
        ax_comp_2[j].legend()
        ax_comp_2[j].set_title(f"Chunk Size {chunk_sizes_2[start+j*n]}")
        ax_comp_2[j].set_xticks(num_transactions_2[:n])
        ax_comp_2[j].set_xticklabels(num_transactions_2[:n])
        ax_comp_2[j].set_yticks(filesizes_2[[0, 3, 6]])
        ax_comp_2[j].set_yticklabels(sizelabels_2[[0, 3, 6]])
        ax_comp_2[j].minorticks_off()
        ax_comp_2[j].grid()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

### Comparing chunk sizes

Now, for each choice of compression algorithm, we compare different chunk sizes.

In [ ]:
fig_chunks_2, ax_chunks_2 = plt.subplots(ncomp, figsize=(10,10), sharey=True)
fig_chunks_2.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_chunks_2[i].loglog(num_transactions_2[:n],
                            filesizes_2[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"Chunk size {chunk_sizes_2[start+j*n]}")
        ax_chunks_2[i].legend()
        ax_chunks_2[i].set_title(f"Compression: {compression_2[start]}")
        ax_chunks_2[i].set_xticks(num_transactions_2[:n])
        ax_chunks_2[i].set_xticklabels(num_transactions_2[:n])
        ax_chunks_2[i].set_yticks(filesizes_2[[0, 3, 6]])
        ax_chunks_2[i].set_yticklabels(sizelabels_2[[0, 3, 6]])
        ax_chunks_2[i].minorticks_off()
        ax_chunks_2[i].grid()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

## Creation times

The creation times for each file are as follows.

In [ ]:
t_write_2 = np.array([test['t_write'][-1] for test in testcase_2])

fig_mostly_appends_times = plt.figure(figsize=(10,8))
for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_2[:n], 
                 t_write_2[start+j*n:start+(j+1)*n], 
                 'o--', ms=8, 
                 label=f"Chunk size {chunk_sizes_2[start+j*n]}, {compression_2[start]}")
plt.plot(200, 2.2488739490509033, 'k*', ms=10, label=f"Chunk size 4096, None, No Versioning")
plt.plot(200, 1.8240299224853516, '*', ms=10, label=f"Chunk size 16384, None, No Versioning")
plt.plot(200, 15.994575023651123, '*', ms=10, label=f"Chunk size 4096, gzip, No Versioning")
plt.plot(200, 19.856306552886963, '*', ms=10, label=f"Chunk size 16384, gzip, No Versioning")
plt.plot(200, 4.9140465259552, '*', ms=10, label=f"Chunk size 4096, lzf, No Versioning")
plt.plot(200, 4.654615163803101, '*', ms=10, label=f"Chunk size 16384, lzf, No Versioning")

plt.xlabel("Transactions")
plt.title(f"{testname} - creation times in seconds")
plt.legend()
plt.xticks(num_transactions_2[:n])
plt.show()

Again, the graphs below show, for each fixed number of transactions, the time required to add new versions as the file is created.

In [ ]:
fig_times_2, ax_2 = plt.subplots(n+1, figsize=(14,18))
fig_times_2.suptitle(f"{testname}: time to write each new version")

for i in range(n):
    for test in testcase_2:
        if test['num_transactions'] == num_transactions_2[i]:
            t_write = np.array(test['t_write'][:-1])
            ax_2[i].plot(t_write, 
                         label=f"chunk size {test['chunk_size']}, {test['compression']}")
            ax_2[i].legend(loc='center left')
            
t_12_none = [0.005884408950805664, 0.003976106643676758, 0.005988121032714844, 0.004587888717651367, 0.004330635070800781, 0.004065513610839844, 0.004004001617431641, 0.004034996032714844, 0.0045588016510009766, 0.005627870559692383, 0.0045926570892333984, 0.004375457763671875, 0.005529165267944336, 0.006089210510253906, 0.0062618255615234375, 0.006366252899169922, 0.00632476806640625, 0.006596803665161133, 0.006554603576660156, 0.0064983367919921875, 0.0066492557525634766, 0.006440401077270508, 0.0065915584564208984, 0.006189584732055664, 0.004422187805175781, 0.00432276725769043, 0.00482940673828125, 0.005034208297729492, 0.0074765682220458984, 0.006167888641357422, 0.007599592208862305, 0.008271455764770508, 0.005837678909301758, 0.005537271499633789, 0.00494384765625, 0.004715442657470703, 0.004849672317504883, 0.004532814025878906, 0.004547119140625, 0.0045201778411865234, 0.0047032833099365234, 0.004605770111083984, 0.004602193832397461, 0.0045931339263916016, 0.004697322845458984, 0.00472569465637207, 0.0047838687896728516, 0.004709482192993164, 0.004771232604980469, 0.004942655563354492, 0.004815578460693359, 0.004791259765625, 0.005019187927246094, 0.00496363639831543, 0.0048983097076416016, 0.0049092769622802734, 0.00491642951965332, 0.00507807731628418, 0.005045175552368164, 0.005074262619018555, 0.0050580501556396484, 0.005182981491088867, 0.005170106887817383, 0.0051708221435546875, 0.0052144527435302734, 0.005410194396972656, 0.005347728729248047, 0.005262613296508789, 0.005324602127075195, 0.0055084228515625, 0.005369901657104492, 0.005387544631958008, 0.0056607723236083984, 0.0060100555419921875, 0.005609989166259766, 0.005544185638427734, 0.006234884262084961, 0.0058269500732421875, 0.005841970443725586, 0.00565648078918457, 0.005704641342163086, 0.00580596923828125, 0.0058286190032958984, 0.005776643753051758, 0.0058171749114990234, 0.0058596134185791016, 0.005978822708129883, 0.006025552749633789, 0.006261587142944336, 0.006025791168212891, 0.0061643123626708984, 0.006105184555053711, 0.006104230880737305, 0.0063059329986572266, 0.0063364505767822266, 0.006224632263183594, 0.006289482116699219, 0.006338834762573242, 0.006424427032470703, 0.006459236145019531, 0.00648045539855957, 0.006489753723144531, 0.006653308868408203, 0.006536245346069336, 0.006665468215942383, 0.0066699981689453125, 0.006782054901123047, 0.006799221038818359, 0.007000446319580078, 0.007000446319580078, 0.007276296615600586, 0.007252931594848633, 0.0072498321533203125, 0.007338523864746094, 0.007464170455932617, 0.007527351379394531, 0.008202075958251953, 0.01232290267944336, 0.01506185531616211, 0.022532939910888672, 0.01746535301208496, 0.03231477737426758, 0.013988256454467773, 0.00936126708984375, 0.009217977523803711, 0.009205102920532227, 0.009131908416748047, 0.00913691520690918, 0.009191751480102539, 0.009159088134765625, 0.009427785873413086, 0.015913963317871094, 0.014039993286132812, 0.013657569885253906, 0.013644933700561523, 0.014122962951660156, 0.014210224151611328, 0.014889955520629883, 0.014418363571166992, 0.01618170738220215, 0.014866113662719727, 0.015360832214355469, 0.015102624893188477, 0.017272233963012695, 0.017613649368286133, 0.017513751983642578, 0.017626285552978516, 0.018149852752685547, 0.01839613914489746, 0.018093109130859375, 0.018397808074951172, 0.01837468147277832, 0.01845264434814453, 0.018759727478027344, 0.018862009048461914, 0.01860213279724121, 0.019008159637451172, 0.019146203994750977, 0.01902604103088379, 0.01921248435974121, 0.0195314884185791, 0.020043373107910156, 0.020177125930786133, 0.02021956443786621, 0.020146846771240234, 0.02039957046508789, 0.02053546905517578, 0.0207064151763916, 0.02078700065612793, 0.02113485336303711, 0.02098560333251953, 0.021432876586914062, 0.028810977935791016, 0.020264387130737305, 0.020908117294311523, 0.02152705192565918, 0.021557092666625977, 0.02186417579650879, 0.02185678482055664, 0.021990060806274414, 0.02210712432861328, 0.02222132682800293, 0.022085189819335938, 0.022216320037841797, 0.022418498992919922, 0.02315497398376465, 0.02249908447265625, 0.02276754379272461, 0.022629976272583008, 0.022866249084472656, 0.02302265167236328, 0.022982358932495117, 0.022984743118286133, 0.023129940032958984, 0.03657698631286621, 0.02625417709350586, 0.02486443519592285, 0.024814844131469727, 0.0253145694732666, 0.02525949478149414, 2.2488739490509033]
t_14_none = [0.005052328109741211, 0.003961086273193359, 0.004105091094970703, 0.004578351974487305, 0.004106760025024414, 0.0047454833984375, 0.004170894622802734, 0.0042705535888671875, 0.004260063171386719, 0.004277706146240234, 0.005647420883178711, 0.00484466552734375, 0.0045948028564453125, 0.005964040756225586, 0.00666499137878418, 0.006224155426025391, 0.006379127502441406, 0.0061719417572021484, 0.0059719085693359375, 0.005423545837402344, 0.005226850509643555, 0.005211830139160156, 0.005403280258178711, 0.005373477935791016, 0.005444526672363281, 0.005928993225097656, 0.004666805267333984, 0.004297494888305664, 0.004209280014038086, 0.004698753356933594, 0.0069544315338134766, 0.004773616790771484, 0.0044782161712646484, 0.0043582916259765625, 0.0043070316314697266, 0.004233121871948242, 0.004205465316772461, 0.004192352294921875, 0.0042917728424072266, 0.0044023990631103516, 0.0043294429779052734, 0.004244804382324219, 0.00428318977355957, 0.004296064376831055, 0.00436854362487793, 0.004400730133056641, 0.004558563232421875, 0.004389762878417969, 0.004385471343994141, 0.004744768142700195, 0.0046155452728271484, 0.004561662673950195, 0.004569292068481445, 0.004677295684814453, 0.00465083122253418, 0.00467681884765625, 0.004868030548095703, 0.0045506954193115234, 0.004668235778808594, 0.004806995391845703, 0.004713535308837891, 0.004927873611450195, 0.0047609806060791016, 0.0049250125885009766, 0.004897117614746094, 0.00491642951965332, 0.005006551742553711, 0.0049784183502197266, 0.005796194076538086, 0.00899195671081543, 0.008698463439941406, 0.012190580368041992, 0.018808841705322266, 0.011987686157226562, 0.007767200469970703, 0.01806950569152832, 0.015845775604248047, 0.02032184600830078, 0.010164976119995117, 0.006060123443603516, 0.006224632263183594, 0.00603032112121582, 0.005785465240478516, 0.0060117244720458984, 0.008776187896728516, 0.007915496826171875, 0.006129264831542969, 0.005921840667724609, 0.006175994873046875, 0.0059583187103271484, 0.005906820297241211, 0.005964040756225586, 0.005945682525634766, 0.006054878234863281, 0.005884647369384766, 0.0058476924896240234, 0.0057125091552734375, 0.005767107009887695, 0.006089925765991211, 0.00585627555847168, 0.0057947635650634766, 0.005810737609863281, 0.005891561508178711, 0.006794452667236328, 0.006376743316650391, 0.00643157958984375, 0.006325721740722656, 0.006222248077392578, 0.006289482116699219, 0.006401777267456055, 0.006424665451049805, 0.00643467903137207, 0.006436586380004883, 0.006654262542724609, 0.006988525390625, 0.006700277328491211, 0.0067234039306640625, 0.006682634353637695, 0.006718635559082031, 0.0067501068115234375, 0.006882667541503906, 0.006972074508666992, 0.007058620452880859, 0.0070836544036865234, 0.007303953170776367, 0.0071680545806884766, 0.007168292999267578, 0.007223367691040039, 0.007256031036376953, 0.007435321807861328, 0.007827281951904297, 0.012841463088989258, 0.011528968811035156, 0.011508703231811523, 0.011728763580322266, 0.01166391372680664, 0.012291193008422852, 0.011936426162719727, 0.012824296951293945, 0.012001752853393555, 0.011877775192260742, 0.011694192886352539, 0.012098073959350586, 0.012232542037963867, 0.011795997619628906, 0.01205897331237793, 0.012416839599609375, 0.013058185577392578, 0.014040946960449219, 0.013202190399169922, 0.01302480697631836, 0.014013051986694336, 0.014054298400878906, 0.013286590576171875, 0.013337850570678711, 0.013450145721435547, 0.013357162475585938, 0.013727426528930664, 0.014292001724243164, 0.014775514602661133, 0.01353764533996582, 0.013596057891845703, 0.013858795166015625, 0.014008283615112305, 0.014006376266479492, 0.01393747329711914, 0.014052391052246094, 0.015052318572998047, 0.014204978942871094, 0.014417409896850586, 0.014922380447387695, 0.01570582389831543, 0.014623403549194336, 0.014387845993041992, 0.014782428741455078, 0.0158846378326416, 0.014698505401611328, 0.014922380447387695, 0.016074657440185547, 0.014903545379638672, 0.0166475772857666, 0.019113779067993164, 0.022749900817871094, 0.01803755760192871, 0.017289161682128906, 0.017675161361694336, 0.017162561416625977, 0.016100645065307617, 0.016254186630249023, 0.017635107040405273, 0.017425060272216797, 0.017838239669799805, 0.01761603355407715, 0.017590045928955078, 0.017999649047851562, 0.017980098724365234, 0.018257617950439453, 0.017951011657714844, 0.018105506896972656, 0.01816391944885254, 1.8240299224853516]
t_12_gzip = [0.005095481872558594, 0.003894805908203125, 0.004030942916870117, 0.004536867141723633, 0.004189491271972656, 0.004185199737548828, 0.0042073726654052734, 0.0042247772216796875, 0.004283905029296875, 0.004767417907714844, 0.00426030158996582, 0.0042781829833984375, 0.00525665283203125, 0.0058100223541259766, 0.005030632019042969, 0.004947662353515625, 0.0050351619720458984, 0.005104541778564453, 0.005492687225341797, 0.0055387020111083984, 0.005425453186035156, 0.005342721939086914, 0.0053555965423583984, 0.005310773849487305, 0.006762266159057617, 0.006893157958984375, 0.015494823455810547, 0.006598949432373047, 0.007329463958740234, 0.0047948360443115234, 0.004755735397338867, 0.004767894744873047, 0.004845380783081055, 0.004756927490234375, 0.004579305648803711, 0.00444340705871582, 0.004524707794189453, 0.004612445831298828, 0.0045871734619140625, 0.004637479782104492, 0.004613637924194336, 0.0045545101165771484, 0.004623889923095703, 0.004629373550415039, 0.0046803951263427734, 0.004717111587524414, 0.004695415496826172, 0.004746913909912109, 0.004715919494628906, 0.004770517349243164, 0.004809141159057617, 0.004845142364501953, 0.00484466552734375, 0.004880428314208984, 0.004973173141479492, 0.004990816116333008, 0.005022287368774414, 0.004978179931640625, 0.00510859489440918, 0.005014657974243164, 0.00598454475402832, 0.005213737487792969, 0.00516200065612793, 0.0052301883697509766, 0.00533294677734375, 0.0053958892822265625, 0.0053386688232421875, 0.0054395198822021484, 0.005326747894287109, 0.005412101745605469, 0.0053844451904296875, 0.005541086196899414, 0.005458354949951172, 0.0055501461029052734, 0.0056247711181640625, 0.005550384521484375, 0.0055658817291259766, 0.0062198638916015625, 0.005927562713623047, 0.005810260772705078, 0.005874156951904297, 0.006020307540893555, 0.005947589874267578, 0.005969047546386719, 0.005936861038208008, 0.00596165657043457, 0.00616002082824707, 0.0061609745025634766, 0.006283283233642578, 0.006281614303588867, 0.006401777267456055, 0.006461143493652344, 0.006384849548339844, 0.006453037261962891, 0.006762981414794922, 0.006915092468261719, 0.007109165191650391, 0.006928443908691406, 0.007448434829711914, 0.007445335388183594, 0.0073087215423583984, 0.007143974304199219, 0.0072934627532958984, 0.007426261901855469, 0.00745701789855957, 0.0073909759521484375, 0.008735895156860352, 0.009332895278930664, 0.012809276580810547, 0.02032160758972168, 0.021391630172729492, 0.027298450469970703, 0.01199650764465332, 0.008442878723144531, 0.00843954086303711, 0.008117198944091797, 0.008203506469726562, 0.008025169372558594, 0.008327484130859375, 0.008157014846801758, 0.008642911911010742, 0.008343219757080078, 0.008423089981079102, 0.008485078811645508, 0.008596181869506836, 0.00871896743774414, 0.009026765823364258, 0.008650779724121094, 0.008630990982055664, 0.0086517333984375, 0.011086225509643555, 0.12656259536743164, 0.1271660327911377, 0.12528491020202637, 0.1293807029724121, 0.1368236541748047, 0.14418792724609375, 0.13575196266174316, 0.13747477531433105, 0.15037035942077637, 0.15006160736083984, 0.1544656753540039, 0.16714048385620117, 0.17346644401550293, 0.1760091781616211, 0.1791975498199463, 0.17914700508117676, 0.20178723335266113, 0.18016743659973145, 0.18083596229553223, 0.1826610565185547, 0.1934375762939453, 0.18900179862976074, 0.18738722801208496, 0.18925786018371582, 0.20012736320495605, 0.24095726013183594, 0.214569091796875, 0.226362943649292, 0.22116661071777344, 0.22498488426208496, 0.22211647033691406, 0.2283470630645752, 0.22559595108032227, 0.22539901733398438, 0.23554658889770508, 0.2310030460357666, 0.23758697509765625, 0.23178839683532715, 0.23832917213439941, 0.23340654373168945, 0.24418210983276367, 0.23745155334472656, 0.2432096004486084, 0.23902010917663574, 0.24476361274719238, 0.23978018760681152, 0.24919366836547852, 0.24508094787597656, 0.24871444702148438, 0.24844765663146973, 0.2534976005554199, 0.25864338874816895, 0.2646303176879883, 0.2604398727416992, 0.2707068920135498, 0.26511120796203613, 0.2732737064361572, 0.26650476455688477, 0.27078890800476074, 0.26847362518310547, 0.27245664596557617, 0.27706170082092285, 0.2774772644042969, 0.27338385581970215, 0.27741336822509766, 0.2838902473449707, 0.27995920181274414, 0.28702497482299805, 0.28322482109069824, 15.994575023651123]
t_14_gzip = [0.0047376155853271484, 0.0036063194274902344, 0.0037910938262939453, 0.004200935363769531, 0.004140377044677734, 0.004002571105957031, 0.0039179325103759766, 0.00396418571472168, 0.00397181510925293, 0.004034996032714844, 0.004439592361450195, 0.006632566452026367, 0.006019115447998047, 0.0052149295806884766, 0.00484013557434082, 0.00499725341796875, 0.004942417144775391, 0.0050275325775146484, 0.004992246627807617, 0.005248069763183594, 0.005224466323852539, 0.00510096549987793, 0.0052585601806640625, 0.005400657653808594, 0.005262851715087891, 0.0054967403411865234, 0.0056803226470947266, 0.004557132720947266, 0.004336833953857422, 0.004654645919799805, 0.0046100616455078125, 0.007605552673339844, 0.00472712516784668, 0.004583835601806641, 0.004510402679443359, 0.004450321197509766, 0.004328489303588867, 0.0042836666107177734, 0.00435185432434082, 0.004282951354980469, 0.004321098327636719, 0.0044116973876953125, 0.0044252872467041016, 0.0043942928314208984, 0.004400968551635742, 0.004499673843383789, 0.004404306411743164, 0.004395961761474609, 0.004450559616088867, 0.005116939544677734, 0.004990816116333008, 0.005479335784912109, 0.004981279373168945, 0.005033969879150391, 0.00541234016418457, 0.005095958709716797, 0.005002498626708984, 0.005151510238647461, 0.0051288604736328125, 0.00521087646484375, 0.005349874496459961, 0.005300760269165039, 0.005600452423095703, 0.004883527755737305, 0.004900693893432617, 0.005007743835449219, 0.005014896392822266, 0.005166292190551758, 0.0049245357513427734, 0.005037784576416016, 0.0051348209381103516, 0.005059480667114258, 0.005150794982910156, 0.005053520202636719, 0.005118370056152344, 0.005196094512939453, 0.00525355339050293, 0.0051953792572021484, 0.005322456359863281, 0.005238056182861328, 0.005323171615600586, 0.005361080169677734, 0.00655364990234375, 0.005407571792602539, 0.005555152893066406, 0.005524396896362305, 0.005474090576171875, 0.007734060287475586, 0.0074465274810791016, 0.01037144660949707, 0.009595632553100586, 0.018886089324951172, 0.008638381958007812, 0.011563539505004883, 0.018644094467163086, 0.010294914245605469, 0.009784698486328125, 0.012210845947265625, 0.027596235275268555, 0.015457868576049805, 0.018582582473754883, 0.01833939552307129, 0.020292282104492188, 0.01984095573425293, 0.01606297492980957, 0.011415958404541016, 0.006963253021240234, 0.006935834884643555, 0.006911277770996094, 0.0069866180419921875, 0.006823301315307617, 0.0066378116607666016, 0.006692171096801758, 0.006727933883666992, 0.0069789886474609375, 0.006938457489013672, 0.0069735050201416016, 0.007102012634277344, 0.007082700729370117, 0.007022857666015625, 0.007073402404785156, 0.007088661193847656, 0.007100343704223633, 0.007154703140258789, 0.007163047790527344, 0.00721287727355957, 0.007245063781738281, 0.0072383880615234375, 0.0073394775390625, 0.007397174835205078, 0.020755529403686523, 0.20426106452941895, 0.20526957511901855, 0.20949792861938477, 0.19198942184448242, 0.21091532707214355, 0.18141627311706543, 0.19932842254638672, 0.2035808563232422, 0.21067023277282715, 0.19975543022155762, 0.20692706108093262, 0.1964104175567627, 0.1888892650604248, 0.20659303665161133, 0.18398690223693848, 0.20241761207580566, 0.24671721458435059, 0.25972628593444824, 0.2532925605773926, 0.25791478157043457, 0.2846977710723877, 0.2576727867126465, 0.2632119655609131, 0.2663078308105469, 0.2545967102050781, 0.2638406753540039, 0.26303958892822266, 0.2671809196472168, 0.27011752128601074, 0.24837994575500488, 0.26972246170043945, 0.2794961929321289, 0.2851989269256592, 0.28403139114379883, 0.28865742683410645, 0.28013157844543457, 0.2782893180847168, 0.2865943908691406, 0.29433512687683105, 0.29001426696777344, 0.2856447696685791, 0.29914259910583496, 0.29636216163635254, 0.29041409492492676, 0.29900097846984863, 0.2927114963531494, 0.29627299308776855, 0.29582786560058594, 0.3378477096557617, 0.3141136169433594, 0.3147580623626709, 0.3232545852661133, 0.3170812129974365, 0.31471848487854004, 0.330425500869751, 0.3123486042022705, 0.31636953353881836, 0.324908971786499, 0.3200244903564453, 0.3203449249267578, 0.3303241729736328, 0.32868289947509766, 0.32697224617004395, 0.33595705032348633, 0.3444373607635498, 0.3442978858947754, 0.3665783405303955, 0.3456597328186035, 0.34459829330444336, 19.856306552886963]
t_12_lzf = [0.005535125732421875, 0.007625102996826172, 0.00765681266784668, 0.00439143180847168, 0.0043337345123291016, 0.004388570785522461, 0.004198551177978516, 0.0042362213134765625, 0.004354715347290039, 0.0047190189361572266, 0.0043261051177978516, 0.004750967025756836, 0.005665302276611328, 0.0053904056549072266, 0.005186557769775391, 0.005151271820068359, 0.0053043365478515625, 0.005057334899902344, 0.005152463912963867, 0.005285501480102539, 0.005191802978515625, 0.005278825759887695, 0.005246877670288086, 0.005462646484375, 0.005316495895385742, 0.004328489303588867, 0.004342317581176758, 0.00480961799621582, 0.004855155944824219, 0.006819009780883789, 0.0045566558837890625, 0.0044596195220947266, 0.0043828487396240234, 0.004317522048950195, 0.0044443607330322266, 0.004746437072753906, 0.004462480545043945, 0.004391670227050781, 0.004584789276123047, 0.0045239925384521484, 0.004653215408325195, 0.0045626163482666016, 0.00452876091003418, 0.0045642852783203125, 0.004627227783203125, 0.00463414192199707, 0.004674196243286133, 0.004655361175537109, 0.004746437072753906, 0.004721164703369141, 0.004763126373291016, 0.004777193069458008, 0.0047528743743896484, 0.004859447479248047, 0.0048639774322509766, 0.004909515380859375, 0.004972934722900391, 0.00498652458190918, 0.00588226318359375, 0.005128145217895508, 0.005894660949707031, 0.005167722702026367, 0.00511479377746582, 0.0056781768798828125, 0.005223989486694336, 0.005221366882324219, 0.005172252655029297, 0.005336284637451172, 0.0052852630615234375, 0.005357980728149414, 0.0054018497467041016, 0.0053730010986328125, 0.0054056644439697266, 0.005545139312744141, 0.00550532341003418, 0.006057262420654297, 0.0057830810546875, 0.00578761100769043, 0.005637168884277344, 0.005846977233886719, 0.005744457244873047, 0.005758762359619141, 0.005839109420776367, 0.0058441162109375, 0.005799770355224609, 0.0058231353759765625, 0.005898952484130859, 0.005942821502685547, 0.006010770797729492, 0.0060231685638427734, 0.0060329437255859375, 0.006417274475097656, 0.006406545639038086, 0.008505582809448242, 0.008271217346191406, 0.012238740921020508, 0.01885223388671875, 0.026947975158691406, 0.02303600311279297, 0.015421152114868164, 0.013669729232788086, 0.007749319076538086, 0.0076906681060791016, 0.007783651351928711, 0.007632255554199219, 0.007538318634033203, 0.007741689682006836, 0.007686138153076172, 0.007663726806640625, 0.007776737213134766, 0.007673740386962891, 0.0074694156646728516, 0.007554769515991211, 0.007610321044921875, 0.007761955261230469, 0.007662057876586914, 0.0077075958251953125, 0.007736921310424805, 0.007860183715820312, 0.007796287536621094, 0.007814168930053711, 0.007883071899414062, 0.00799870491027832, 0.0083465576171875, 0.008135318756103516, 0.008167743682861328, 0.008164405822753906, 0.008202552795410156, 0.008211374282836914, 0.008356571197509766, 0.009755373001098633, 0.039974212646484375, 0.03957176208496094, 0.03977179527282715, 0.0395050048828125, 0.04209256172180176, 0.042593955993652344, 0.042534589767456055, 0.04457545280456543, 0.04471421241760254, 0.04500174522399902, 0.04520845413208008, 0.04601740837097168, 0.04576396942138672, 0.046620845794677734, 0.04665493965148926, 0.06644582748413086, 0.05149555206298828, 0.051876068115234375, 0.05161857604980469, 0.05282139778137207, 0.05281329154968262, 0.05217146873474121, 0.05393481254577637, 0.05392932891845703, 0.0655984878540039, 0.052403926849365234, 0.05093741416931152, 0.05294537544250488, 0.057062387466430664, 0.05684304237365723, 0.05749344825744629, 0.05827975273132324, 0.060321807861328125, 0.06757235527038574, 0.05990934371948242, 0.059415340423583984, 0.059624433517456055, 0.058879852294921875, 0.05830264091491699, 0.058019161224365234, 0.05861330032348633, 0.07948017120361328, 0.0642554759979248, 0.06414675712585449, 0.0650789737701416, 0.06584525108337402, 0.06552267074584961, 0.06549811363220215, 0.07641744613647461, 0.06563830375671387, 0.06438183784484863, 0.06415724754333496, 0.0649113655090332, 0.06488871574401855, 0.06555747985839844, 0.06528091430664062, 0.07821869850158691, 0.06746435165405273, 0.06700491905212402, 0.06647610664367676, 0.06731152534484863, 0.06880855560302734, 0.08602118492126465, 0.08349442481994629, 0.07236790657043457, 0.0691995620727539, 0.0682363510131836, 0.06987309455871582, 0.07167363166809082, 4.9140465259552]
t_14_lzf = [0.004782676696777344, 0.0037407875061035156, 0.0038797855377197266, 0.003957033157348633, 0.003922700881958008, 0.004034996032714844, 0.0038526058197021484, 0.003892183303833008, 0.003920316696166992, 0.0039463043212890625, 0.005930423736572266, 0.0063915252685546875, 0.006108999252319336, 0.005449533462524414, 0.005909919738769531, 0.00583648681640625, 0.012305021286010742, 0.005998849868774414, 0.00535893440246582, 0.0053865909576416016, 0.005368232727050781, 0.005312919616699219, 0.00545501708984375, 0.005320310592651367, 0.005427360534667969, 0.005029201507568359, 0.00475621223449707, 0.004232645034790039, 0.004263162612915039, 0.004263639450073242, 0.00535130500793457, 0.006378650665283203, 0.004518985748291016, 0.004377603530883789, 0.004342555999755859, 0.004439353942871094, 0.004340648651123047, 0.0043582916259765625, 0.004476785659790039, 0.004351615905761719, 0.004404783248901367, 0.004613399505615234, 0.0044574737548828125, 0.0044252872467041016, 0.004503726959228516, 0.004473686218261719, 0.004459381103515625, 0.004573822021484375, 0.00459599494934082, 0.004621982574462891, 0.004773616790771484, 0.004730939865112305, 0.004700183868408203, 0.004691362380981445, 0.004738330841064453, 0.0047533512115478516, 0.00471186637878418, 0.0048525333404541016, 0.004841327667236328, 0.004843473434448242, 0.004863739013671875, 0.00485992431640625, 0.004838705062866211, 0.004930019378662109, 0.004884004592895508, 0.005079030990600586, 0.005017518997192383, 0.0051305294036865234, 0.00509190559387207, 0.00508880615234375, 0.005095243453979492, 0.005149126052856445, 0.005463600158691406, 0.0052449703216552734, 0.005992889404296875, 0.005278825759887695, 0.005310535430908203, 0.005852460861206055, 0.0053234100341796875, 0.005361795425415039, 0.005355119705200195, 0.005616903305053711, 0.0055501461029052734, 0.00556182861328125, 0.005628347396850586, 0.0057065486907958984, 0.005687713623046875, 0.005754709243774414, 0.005791187286376953, 0.005948066711425781, 0.0058743953704833984, 0.005805253982543945, 0.005842685699462891, 0.005889177322387695, 0.0059356689453125, 0.005911350250244141, 0.006121635437011719, 0.0060536861419677734, 0.0065577030181884766, 0.006171464920043945, 0.006226062774658203, 0.006253480911254883, 0.006253480911254883, 0.006346940994262695, 0.0064885616302490234, 0.006556034088134766, 0.008071660995483398, 0.008087158203125, 0.010337114334106445, 0.026645421981811523, 0.011922836303710938, 0.01497793197631836, 0.020490407943725586, 0.016446828842163086, 0.007666826248168945, 0.007160663604736328, 0.007134437561035156, 0.0071294307708740234, 0.007025718688964844, 0.0070645809173583984, 0.00703740119934082, 0.0072803497314453125, 0.007257223129272461, 0.007135868072509766, 0.007150888442993164, 0.00715184211730957, 0.007250547409057617, 0.0073354244232177734, 0.007485866546630859, 0.007420539855957031, 0.009599447250366211, 0.04337596893310547, 0.04352831840515137, 0.04067039489746094, 0.04223752021789551, 0.041025400161743164, 0.045085906982421875, 0.04251909255981445, 0.041353702545166016, 0.04077339172363281, 0.04096174240112305, 0.041196346282958984, 0.04172706604003906, 0.04050493240356445, 0.04051399230957031, 0.04167461395263672, 0.0448145866394043, 0.05007195472717285, 0.06864309310913086, 0.05081009864807129, 0.05138278007507324, 0.05396914482116699, 0.05219888687133789, 0.053655385971069336, 0.05419182777404785, 0.053781747817993164, 0.05453228950500488, 0.06563401222229004, 0.05328869819641113, 0.05324959754943848, 0.05303049087524414, 0.05307412147521973, 0.05329465866088867, 0.056046247482299805, 0.055161476135253906, 0.05539989471435547, 0.05518984794616699, 0.06678628921508789, 0.05585813522338867, 0.05701494216918945, 0.0572657585144043, 0.055673837661743164, 0.05759239196777344, 0.057352304458618164, 0.05726289749145508, 0.07276320457458496, 0.060452938079833984, 0.060211181640625, 0.06050395965576172, 0.06225872039794922, 0.06239008903503418, 0.06470298767089844, 0.06689286231994629, 0.07612013816833496, 0.060675859451293945, 0.06095552444458008, 0.059810638427734375, 0.0623173713684082, 0.06257009506225586, 0.06198477745056152, 0.06241559982299805, 0.07486844062805176, 0.06358480453491211, 0.06258535385131836, 0.06255602836608887, 0.06471610069274902, 0.0641024112701416, 0.0657343864440918, 0.08265566825866699, 0.06866192817687988, 4.654615163803101]

ax_2[n].plot(t_12_none[:-1], label=f"chunk size 4096, None")
ax_2[n].plot(t_14_none[:-1], label=f"chunk size 16384, None")
ax_2[n].plot(t_12_gzip[:-1], label=f"chunk size 4096, gzip")
ax_2[n].plot(t_14_gzip[:-1], label=f"chunk size 16384, gzip")
ax_2[n].plot(t_12_lzf[:-1], label=f"chunk size 4096, lzf")
ax_2[n].plot(t_14_lzf[:-1], label=f"chunk size 16384, lzf")
ax_2[n].legend(loc='center left')
ax_2[n].set_title('No versioning')
    
plt.xlabel("Number of transactions")
plt.ylabel("Time (in seconds)")
plt.show()

The result is similar to the first test: smaller chunk sizes correspond to smaller file sizes, but larger creation times. However, in this case, we can see there is a drop in performance when adding new versions as our file grows. This can be seen as an effect of the increase in the data size for each new version (since we are *mostly appending* data with each new version).

[Back to top](#home)

<a id='test3'></a>

# Test 3: Small fraction changes (sparse)

In [ ]:
testname = "test_small_fraction_changes_sparse"

We have tested the following numbers of versions (or transactions):

```python
num_transactions_3 = [50, 100, 500, 1000, 5000, 10000]
```

### Generating new tests

Change `num_transactions_3`, `exponents_3` and `compression_3` as desired:

In [ ]:
num_transactions_3 = [50, 100, 500, 1000, 5000]
exponents_3 = [12, 14]
compression_3 = [None, "gzip", "lzf"]

Create files:

In [ ]:
testcase = performance_tests.test_small_fraction_changes_sparse(path=path,
                                                                num_transactions=num_transactions_3, 
                                                                exponents=exponents_3, 
                                                                compression=compression_3)

testcase_3, msg = testcase.create_files()
if msg:
    print(msg)
t_sizes_3 = [test['theoretical_sizes'] for test in testcase.tests[-len(num_transactions_3):]]

If you wish to save the results for later, execute the cell below:

In [ ]:
#testcase.save(testcase_3)

### Reading previously computed tests

To open an existing `.json` file, use

In [ ]:
#with open(f"{testname}.json", "r") as json_in:
#    testcase_3 = json.load(json_in)
#t_sizes_3 = None

## Analysis

Again, let's show the size information in a graph:

In [ ]:
num_transactions_3 = [test['num_transactions'] for test in testcase_3]
chunk_sizes_3 = [test['chunk_size'] for test in testcase_3]
compression_3 = [test['compression'] for test in testcase_3]

filesizes_3 = np.array([test['size'] for test in testcase_3])
sizelabels_3 = np.array([test['size_label'] for test in testcase_3])

n = len(set(num_transactions_3))
ncs = len(set(exponents_3))
ncomp = len(set(compression_3))

fig_small_fraction_changes = plt.figure(figsize=(14,10))

if t_sizes_3:
    plt.plot(num_transactions_3[:n], t_sizes_3, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_3[:n], 
                 filesizes_3[start+j*n:start+(j+1)*n], 
                 '*--', ms=12, 
                 label=f"Chunk size {chunk_sizes_3[start+j*n]}, {compression_3[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
plt.xticks(num_transactions_3[:n])
selected = [0, 4, 8, 9]
plt.yticks(filesizes_3[selected], sizelabels_3[selected])
plt.grid(True)
plt.show()

Changing the view to a logarithmic scale, we have the following:

In [ ]:
fig_small_fraction_changes_log = plt.figure(figsize=(14,10))

if t_sizes_3:
    plt.loglog(num_transactions_3[:n], t_sizes_3, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.loglog(num_transactions_3[:n], 
                   filesizes_3[start+j*n:start+(j+1)*n], 
                   '*--', ms=12, 
                   label=f"Chunk size {chunk_sizes_3[start+j*n]}, {compression_3[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
plt.grid()
plt.xticks(num_transactions_3[:n], num_transactions_3[:n])
plt.yticks(filesizes_3[selected], sizelabels_3[selected])
plt.minorticks_off()
plt.show()

### Comparing compression algorithms

For each chunk size that we chose to test, let's compare the file sizes corresponding to each compression algorithm that we used.

In [ ]:
fig_comp_3, ax_comp_3 = plt.subplots(ncs, figsize=(10,8), sharey=True)
fig_comp_3.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_comp_3[j].loglog(num_transactions_3[:n],
                            filesizes_3[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"{compression_3[start]}")
        ax_comp_3[j].legend()
        ax_comp_3[j].set_title(f"Chunk Size {chunk_sizes_3[start+j*n]}")
        ax_comp_3[j].set_xticks(num_transactions_3[:n])
        ax_comp_3[j].set_xticklabels(num_transactions_3[:n])
        ax_comp_3[j].set_yticks(filesizes_3[[0, 3, 9]])
        ax_comp_3[j].set_yticklabels(sizelabels_3[[0, 3, 9]])
        ax_comp_3[j].minorticks_off()
        ax_comp_3[j].grid()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

### Comparing chunk sizes

Now, for each choice of compression algorithm, we compare different chunk sizes.

In [ ]:
fig_chunks_3, ax_chunks_3 = plt.subplots(ncomp, figsize=(10,10), sharey=True)
fig_chunks_3.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_chunks_3[i].loglog(num_transactions_3[:n],
                            filesizes_3[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"Chunk size {chunk_sizes_3[start+j*n]}")
        ax_chunks_3[i].legend()
        ax_chunks_3[i].set_title(f"Compression: {compression_3[start]}")
        ax_chunks_3[i].set_xticks(num_transactions_3[:n])
        ax_chunks_3[i].set_xticklabels(num_transactions_3[:n])
        ax_chunks_3[i].set_yticks(filesizes_3[[0, 3, 9]])
        ax_chunks_3[i].set_yticklabels(sizelabels_3[[0, 3, 9]])
        ax_chunks_3[i].minorticks_off()
        ax_chunks_3[i].grid()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

## Creation times

If we look at the creation times for these files, we have something like this:

In [ ]:
t_write_3 = np.array([test['t_write'][-1] for test in testcase_3])

fig_small_fraction_changes_times = plt.figure(figsize=(10,8))
for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_3[:n], 
                 t_write_3[start+j*n:start+(j+1)*n], 
                 '*--', ms=12, 
                 label=f"Chunk size {chunk_sizes_3[start+j*n]}, {compression_3[start]}")

plt.plot(5000, 21.61993956565857, 'ko', ms=10, label=f"Chunk size 4096, None, No versioning")
plt.plot(5000, 20.37408137321472, 'o', ms=10, label=f"Chunk size 16384, None, No versioning")
plt.plot(5000, 21.330241918563843, 'o', ms=10, label=f"Chunk size 4096, gzip, No versioning")
plt.plot(5000, 20.60214376449585, 'o', ms=10, label=f"Chunk size 16384, gzip, No versioning")
plt.plot(5000, 21.529611349105835, 'o', ms=10, label=f"Chunk size 4096, lzf, No versioning")
plt.plot(5000, 20.559864282608032, 'o', ms=10, label=f"Chunk size 16384, lzf, No versioning")
        
plt.xlabel("Transactions")
plt.title(f"{testname} - creation times in seconds")
plt.legend()
plt.xticks(num_transactions_3[:n])
plt.show()

Again, the graphs below show, for each fixed number of transactions, the time required to add new versions as the file is created.

In [ ]:
fig_times_3, ax_3 = plt.subplots(n+1, figsize=(14,18))
fig_times_3.suptitle(f"{testname}: time to write each new version")

for i in range(n):
    for test in testcase_3:
        if test['num_transactions'] == num_transactions_3[i]:
            t_write = np.array(test['t_write'][:-1])
            ax_3[i].plot(t_write, 
                         label=f"chunk size {test['chunk_size']}, {test['compression']}")
            ax_3[i].legend(loc='center left')
    
t_12_none = [0.006307125091552734, 0.0042421817779541016, 0.00421905517578125, 0.00444793701171875, 0.004334688186645508, 0.004337787628173828, 0.004228830337524414, 0.00434422492980957, 0.0042726993560791016, 0.004331111907958984, 0.004724979400634766, 0.004131793975830078, 0.005048513412475586, 0.005356550216674805, 0.004920244216918945, 0.004849433898925781, 0.004731178283691406, 0.0048465728759765625, 0.005530595779418945, 0.00535130500793457, 0.005369424819946289, 0.005224466323852539, 0.005441904067993164, 0.00540614128112793, 0.0052530765533447266, 0.005590200424194336, 0.0046634674072265625, 0.003919839859008789, 0.0038950443267822266, 0.003908395767211914, 0.0039713382720947266, 0.004876852035522461, 0.0044002532958984375, 0.006487846374511719, 0.00472569465637207, 0.00448918342590332, 0.005078554153442383, 0.0045909881591796875, 0.004650115966796875, 0.004299640655517578, 0.004099607467651367, 0.0039215087890625, 0.0038492679595947266, 0.0037763118743896484, 0.003696918487548828, 0.0037200450897216797, 0.0037088394165039062, 0.0036537647247314453, 0.003809690475463867, 0.0036478042602539062, 0.0036673545837402344, 0.0036847591400146484, 0.003950357437133789, 0.0036852359771728516, 0.003636598587036133, 0.003722667694091797, 0.0036916732788085938, 0.0036613941192626953, 0.003675222396850586, 0.003661632537841797, 0.0036821365356445312, 0.0037157535552978516, 0.0036945343017578125, 0.003666400909423828, 0.0036773681640625, 0.003670930862426758, 0.003624439239501953, 0.0036666393280029297, 0.003674745559692383, 0.003660440444946289, 0.0036280155181884766, 0.003673076629638672, 0.0036826133728027344, 0.003676176071166992, 0.0036780834197998047, 0.0036177635192871094, 0.0036721229553222656, 0.003637075424194336, 0.0036668777465820312, 0.0036687850952148438, 0.003681659698486328, 0.0036797523498535156, 0.003633260726928711, 0.0036826133728027344, 0.0036635398864746094, 0.0037245750427246094, 0.0037012100219726562, 0.003676176071166992, 0.0036211013793945312, 0.0036809444427490234, 0.003694772720336914, 0.0036690235137939453, 0.0036733150482177734, 0.003618955612182617, 0.0036618709564208984, 0.003679037094116211, 0.0036537647247314453, 0.0037305355072021484, 0.003668546676635742, 0.0036306381225585938, 0.003711700439453125, 0.0038118362426757812, 0.0038306713104248047, 0.0037224292755126953, 0.003712892532348633, 0.0037810802459716797, 0.0037314891815185547, 0.0037620067596435547, 0.0036416053771972656, 0.00373077392578125, 0.003698110580444336, 0.0037279129028320312, 0.0036880970001220703, 0.0036895275115966797, 0.003706216812133789, 0.003718137741088867, 0.0037713050842285156, 0.003915548324584961, 0.006498813629150391, 0.004030466079711914, 0.004390239715576172, 0.008835315704345703, 0.00585174560546875, 0.004410266876220703, 0.004225730895996094, 0.004095554351806641, 0.0039293766021728516, 0.0038285255432128906, 0.0037832260131835938, 0.003706216812133789, 0.0037021636962890625, 0.003724813461303711, 0.0036666393280029297, 0.003756999969482422, 0.003661632537841797, 0.0037698745727539062, 0.003695964813232422, 0.0037605762481689453, 0.0036878585815429688, 0.0036623477935791016, 0.003706216812133789, 0.003692626953125, 0.0037636756896972656, 0.0036797523498535156, 0.0037174224853515625, 0.0036890506744384766, 0.0037322044372558594, 0.0037941932678222656, 0.003667593002319336, 0.0037012100219726562, 0.0036466121673583984, 0.0037713050842285156, 0.0039033889770507812, 0.0036554336547851562, 0.0037157535552978516, 0.003683805465698242, 0.003683328628540039, 0.003683805465698242, 0.003681659698486328, 0.0036933422088623047, 0.0037424564361572266, 0.0036590099334716797, 0.0038297176361083984, 0.003703594207763672, 0.003721952438354492, 0.0036840438842773438, 0.003713369369506836, 0.003687143325805664, 0.003765106201171875, 0.004918336868286133, 0.004147052764892578, 0.003947019577026367, 0.004010677337646484, 0.0046651363372802734, 0.005381345748901367, 0.010109663009643555, 0.005366086959838867, 0.007653713226318359, 0.008263111114501953, 0.015277624130249023, 0.0050585269927978516, 0.006296634674072266, 0.008015871047973633, 0.007001638412475586, 0.005499124526977539, 0.009000778198242188, 0.005785942077636719, 0.004172563552856445, 0.004046201705932617, 0.004035234451293945, 0.0041124820709228516, 0.004174232482910156, 0.004043102264404297, 0.003970146179199219, 0.003923654556274414, 0.003663301467895508, 0.003689289093017578, 0.0036973953247070312, 0.003709554672241211, 0.003821134567260742, 0.003688335418701172, 0.0037004947662353516, 0.003679513931274414, 0.003683328628540039, 0.003643035888671875, 0.0036699771881103516, 0.0036504268646240234, 0.0037641525268554688, 0.0036902427673339844, 0.003650188446044922, 0.0036764144897460938, 0.0036439895629882812, 0.0038518905639648438, 0.0036973953247070312, 0.0037202835083007812, 0.003630399703979492, 0.0037119388580322266, 0.003693103790283203, 0.003690004348754883, 0.003687143325805664, 0.003632783889770508, 0.003675222396850586, 0.003658771514892578, 0.003685474395751953, 0.0036916732788085938, 0.003689289093017578, 0.003678560256958008, 0.003680706024169922, 0.0037102699279785156, 0.0036821365356445312, 0.003710031509399414, 0.0036725997924804688, 0.003720998764038086, 0.0036606788635253906, 0.003690958023071289, 0.0037276744842529297, 0.0036690235137939453, 0.003690481185913086, 0.003638029098510742, 0.003838062286376953, 0.003799915313720703, 0.0037250518798828125, 0.003694295883178711, 0.003751993179321289, 0.00399470329284668, 0.0038068294525146484, 0.0036957263946533203, 0.0036482810974121094, 0.0036842823028564453, 0.003642559051513672, 0.0036814212799072266, 0.003681182861328125, 0.003783702850341797, 0.003693103790283203, 0.0036516189575195312, 0.0037031173706054688, 0.003734588623046875, 0.0037059783935546875, 0.003649473190307617, 0.0037152767181396484, 0.0036382675170898438, 0.0037262439727783203, 0.003699779510498047, 0.003642559051513672, 0.003702878952026367, 0.0036520957946777344, 0.00371551513671875, 0.003710031509399414, 0.0036940574645996094, 0.0036458969116210938, 0.003700733184814453, 0.003720521926879883, 0.003734111785888672, 0.003717660903930664, 0.0036423206329345703, 0.003823995590209961, 0.00368499755859375, 0.0037071704864501953, 0.003695964813232422, 0.003722667694091797, 0.003756284713745117, 0.003640890121459961, 0.0036766529083251953, 0.0036852359771728516, 0.003831148147583008, 0.0036535263061523438, 0.003698110580444336, 0.003665924072265625, 0.003782510757446289, 0.0036859512329101562, 0.0036509037017822266, 0.0037021636962890625, 0.0036470890045166016, 0.0037026405334472656, 0.0038161277770996094, 0.0037164688110351562, 0.003653287887573242, 0.0036995410919189453, 0.0037131309509277344, 0.003783702850341797, 0.0036907196044921875, 0.0036520957946777344, 0.0037055015563964844, 0.0036439895629882812, 0.003683805465698242, 0.0036950111389160156, 0.003704547882080078, 0.00371551513671875, 0.0036993026733398438, 0.0037026405334472656, 0.003667116165161133, 0.0037145614624023438, 0.0037076473236083984, 0.0037217140197753906, 0.0036809444427490234, 0.0037276744842529297, 0.0037055015563964844, 0.0037174224853515625, 0.0036911964416503906, 0.003694295883178711, 0.003795146942138672, 0.0037927627563476562, 0.0037627220153808594, 0.003801584243774414, 0.003729104995727539, 0.003754138946533203, 0.0037081241607666016, 0.003753185272216797, 0.003759145736694336, 0.003735780715942383, 0.0037043094635009766, 0.0037589073181152344, 0.0037581920623779297, 0.0037505626678466797, 0.003756284713745117, 0.0037250518798828125, 0.003774881362915039, 0.0037126541137695312, 0.0037415027618408203, 0.0037894248962402344, 0.0037355422973632812, 0.0037708282470703125, 0.0037255287170410156, 0.0037581920623779297, 0.003747224807739258, 0.0037431716918945312, 0.0036878585815429688, 0.00392460823059082, 0.0037703514099121094, 0.003747224807739258, 0.0037534236907958984, 0.003742694854736328, 0.004068136215209961, 0.003906965255737305, 0.003830432891845703, 0.003751993179321289, 0.003722667694091797, 0.0037746429443359375, 0.0037584304809570312, 0.003765583038330078, 0.0038268566131591797, 0.003774404525756836, 0.0037698745727539062, 0.003729581832885742, 0.003786325454711914, 0.003777027130126953, 0.003755807876586914, 0.003780364990234375, 0.0039675235748291016, 0.003815174102783203, 0.003703594207763672, 0.003762483596801758, 0.004312276840209961, 0.003765106201171875, 0.003767251968383789, 0.003783702850341797, 0.003795146942138672, 0.0037078857421875, 0.0037703514099121094, 0.003774881362915039, 0.0037679672241210938, 0.0037212371826171875, 0.0037631988525390625, 0.003770589828491211, 0.003744363784790039, 0.003751993179321289, 0.003710031509399414, 0.003770589828491211, 0.003744363784790039, 0.003754854202270508, 0.0037505626678466797, 0.0037987232208251953, 0.003760814666748047, 0.003721475601196289, 0.003773927688598633, 0.0037474632263183594, 0.003749847412109375, 0.0037078857421875, 0.0037848949432373047, 0.003797292709350586, 0.003863096237182617, 0.0037543773651123047, 0.0037233829498291016, 0.0037424564361572266, 0.0037124156951904297, 0.003753185272216797, 0.0039014816284179688, 0.0038101673126220703, 0.0037670135498046875, 0.0037310123443603516, 0.0037491321563720703, 0.0037584304809570312, 0.0038073062896728516, 0.003720998764038086, 0.0038590431213378906, 0.0037474632263183594, 0.0037517547607421875, 0.0037653446197509766, 0.0037233829498291016, 0.0037610530853271484, 0.003701448440551758, 0.0037689208984375, 0.003838777542114258, 0.0037145614624023438, 0.003763437271118164, 0.003716707229614258, 0.0037620067596435547, 0.003762483596801758, 0.0037741661071777344, 0.0037424564361572266, 0.003802061080932617, 0.0037577152252197266, 0.0037610530853271484, 0.0037500858306884766, 0.0037162303924560547, 0.003770589828491211, 0.003720998764038086, 0.003785371780395508, 0.0038518905639648438, 0.0037298202514648438, 0.00376129150390625, 0.003813028335571289, 0.003767251968383789, 0.0037360191345214844, 0.0037565231323242188, 0.003804445266723633, 0.0037403106689453125, 0.003767728805541992, 0.003725290298461914, 0.0037894248962402344, 0.0037682056427001953, 0.0037691593170166016, 0.003716707229614258, 0.003864765167236328, 0.003771543502807617, 0.003792285919189453, 0.00377655029296875, 0.0037169456481933594, 0.003965616226196289, 0.003856658935546875, 0.0038099288940429688, 0.0037484169006347656, 0.0037229061126708984, 0.003753662109375, 0.0037462711334228516, 0.0037620067596435547, 0.003758668899536133, 0.0037720203399658203, 0.0037660598754882812, 0.003745555877685547, 0.003772735595703125, 0.0037491321563720703, 0.0037870407104492188, 0.003751516342163086, 0.0037841796875, 0.003721952438354492, 0.00383758544921875, 0.0037806034088134766, 0.0037412643432617188, 0.003774881362915039, 0.0037190914154052734, 0.0037648677825927734, 0.0037658214569091797, 0.003779172897338867, 0.0037648677825927734, 0.003728628158569336, 0.0037963390350341797, 0.003769397735595703, 0.003757953643798828, 0.0037140846252441406, 0.0037527084350585938, 0.003775358200073242, 0.0037598609924316406, 0.0037682056427001953, 0.003726959228515625, 0.0037508010864257812, 0.003721475601196289, 0.0037937164306640625, 0.0037865638732910156, 0.0037796497344970703, 0.003779172897338867, 0.0037305355072021484, 0.0037529468536376953, 0.0037648677825927734, 0.003771066665649414, 0.0037305355072021484, 0.0038247108459472656, 0.0038030147552490234, 0.003776073455810547, 0.0037508010864257812, 0.0037145614624023438, 0.0037407875061035156, 0.0038111209869384766, 0.0037643909454345703, 0.0038442611694335938, 0.003749847412109375, 0.003789663314819336, 0.0037186145782470703, 0.0037696361541748047, 0.0037643909454345703, 0.0037670135498046875, 0.0037293434143066406, 0.0038080215454101562, 0.003756284713745117, 0.0037717819213867188, 0.0037598609924316406, 0.0037381649017333984, 0.003796815872192383, 0.0037255287170410156, 0.00382232666015625, 0.0038383007049560547, 0.00373077392578125, 0.0037615299224853516, 0.003743886947631836, 0.003767251968383789, 0.0038139820098876953, 0.0037696361541748047, 0.0037817955017089844, 0.003747701644897461, 0.0038003921508789062, 0.0037610530853271484, 0.003789186477661133, 0.003726482391357422, 0.0037660598754882812, 0.0037240982055664062, 0.0038633346557617188, 0.0037903785705566406, 0.0037431716918945312, 0.003780364990234375, 0.0037488937377929688, 0.003793478012084961, 0.003790140151977539, 0.003801584243774414, 0.003782510757446289, 0.0037355422973632812, 0.0037543773651123047, 0.0037741661071777344, 0.003774404525756836, 0.0037424564361572266, 0.003832578659057617, 0.0037937164306640625, 0.0037615299224853516, 0.0037844181060791016, 0.003747701644897461, 0.0037789344787597656, 0.0038411617279052734, 0.0037670135498046875, 0.0038144588470458984, 0.00373077392578125, 0.0037708282470703125, 0.003734588623046875, 0.003766298294067383, 0.004260540008544922, 0.0038039684295654297, 0.0037953853607177734, 0.0037794113159179688, 0.004070281982421875, 0.003836393356323242, 0.0037369728088378906, 0.0038073062896728516, 0.003831148147583008, 0.0037491321563720703, 0.0037806034088134766, 0.0037415027618408203, 0.0037915706634521484, 0.0037767887115478516, 0.0037696361541748047, 0.003729104995727539, 0.0038030147552490234, 0.003770112991333008, 0.003756999969482422, 0.0037992000579833984, 0.003743410110473633, 0.003805398941040039, 0.0037479400634765625, 0.0037717819213867188, 0.0038475990295410156, 0.0037336349487304688, 0.0037980079650878906, 0.0037605762481689453, 0.0037682056427001953, 0.0037784576416015625, 0.003785371780395508, 0.003822803497314453, 0.0037496089935302734, 0.003801584243774414, 0.003769397735595703, 0.003818511962890625, 0.003725767135620117, 0.0037970542907714844, 0.003780364990234375, 0.0038175582885742188, 0.003926277160644531, 0.0037512779235839844, 0.0037927627563476562, 0.0037355422973632812, 0.0037949085235595703, 0.003888368606567383, 0.0038754940032958984, 0.0038192272186279297, 0.0037522315979003906, 0.003772735595703125, 0.003969907760620117, 0.004075288772583008, 0.0038220882415771484, 0.003764629364013672, 0.0037827491760253906, 0.003792285919189453, 0.0037882328033447266, 0.0037512779235839844, 0.0037903785705566406, 0.003814697265625, 0.0037751197814941406, 0.003991842269897461, 0.0037746429443359375, 0.003763437271118164, 0.003841400146484375, 0.0037665367126464844, 0.003858327865600586, 0.0037446022033691406, 0.0037708282470703125, 0.0037648677825927734, 0.003793954849243164, 0.0038051605224609375, 0.003812074661254883, 0.0037877559661865234, 0.0037384033203125, 0.003802776336669922, 0.0037856101989746094, 0.0037925243377685547, 0.0037355422973632812, 0.0037920475006103516, 0.003765106201171875, 0.0038459300994873047, 0.0037834644317626953, 0.0037467479705810547, 0.0037713050842285156, 0.003731966018676758, 0.0037696361541748047, 0.003796815872192383, 0.0038352012634277344, 0.0037975311279296875, 0.003774881362915039, 0.0037839412689208984, 0.003832101821899414, 0.003883838653564453, 0.0037462711334228516, 0.0038254261016845703, 0.0037889480590820312, 0.0038061141967773438, 0.0037751197814941406, 0.0038442611694335938, 0.003796100616455078, 0.0037474632263183594, 0.0038449764251708984, 0.003767728805541992, 0.0037393569946289062, 0.0037996768951416016, 0.003754138946533203, 0.0037996768951416016, 0.0037949085235595703, 0.0037834644317626953, 0.0038423538208007812, 0.0037641525268554688, 0.0037932395935058594, 0.003780841827392578, 0.0037992000579833984, 0.003763437271118164, 0.0037925243377685547, 0.0038118362426757812, 0.0037813186645507812, 0.0038001537322998047, 0.0038161277770996094, 0.0037949085235595703, 0.003767251968383789, 0.0037755966186523438, 0.0038416385650634766, 0.0037484169006347656, 0.003812551498413086, 0.003762960433959961, 0.003818035125732422, 0.0038003921508789062, 0.003855466842651367, 0.0038292407989501953, 0.0037593841552734375, 0.003771543502807617, 0.0038301944732666016, 0.003803253173828125, 0.003762483596801758, 0.003775358200073242, 0.0037696361541748047, 0.003836393356323242, 0.0038056373596191406, 0.003766775131225586, 0.0038225650787353516, 0.003732442855834961, 0.0037994384765625, 0.0038912296295166016, 0.003835439682006836, 0.0038208961486816406, 0.0037527084350585938, 0.003821134567260742, 0.003786325454711914, 0.004105329513549805, 0.0038335323333740234, 0.003772735595703125, 0.003794431686401367, 0.0037889480590820312, 0.0038542747497558594, 0.0038149356842041016, 0.0038568973541259766, 0.003844738006591797, 0.003852367401123047, 0.003889322280883789, 0.003806591033935547, 0.0038454532623291016, 0.003815174102783203, 0.0039272308349609375, 0.0038645267486572266, 0.0038356781005859375, 0.003865480422973633, 0.003814697265625, 0.0038423538208007812, 0.004077911376953125, 0.003820180892944336, 0.0038399696350097656, 0.003814697265625, 0.003836393356323242, 0.003832578659057617, 0.003850221633911133, 0.0038635730743408203, 0.0038208961486816406, 0.0038628578186035156, 0.003820180892944336, 0.003915309906005859, 0.003802061080932617, 0.0038475990295410156, 0.003832578659057617, 0.003823518753051758, 0.003880739212036133, 0.0038232803344726562, 0.003856658935546875, 0.0038263797760009766, 0.0039052963256835938, 0.0039119720458984375, 0.003818511962890625, 0.003837108612060547, 0.003818035125732422, 0.0038330554962158203, 0.004001140594482422, 0.003807544708251953, 0.003862619400024414, 0.0037996768951416016, 0.003833293914794922, 0.0038299560546875, 0.003980875015258789, 0.0038826465606689453, 0.003814220428466797, 0.003839254379272461, 0.003854036331176758, 0.003823995590209961, 0.0038166046142578125, 0.0038738250732421875, 0.003846406936645508, 0.00386810302734375, 0.003866434097290039, 0.003795146942138672, 0.0038466453552246094, 0.0038127899169921875, 0.003941059112548828, 0.0038390159606933594, 0.003799915313720703, 0.003865957260131836, 0.003810405731201172, 0.003854990005493164, 0.0039010047912597656, 0.0038077831268310547, 0.0038683414459228516, 0.0038154125213623047, 0.0038328170776367188, 0.0038280487060546875, 0.003881216049194336, 0.003863811492919922, 0.0038061141967773438, 0.0038301944732666016, 0.003862619400024414, 0.003858327865600586, 0.0038106441497802734, 0.0038797855377197266, 0.0038475990295410156, 0.003844022750854492, 0.0038857460021972656, 0.004054069519042969, 0.0038535594940185547, 0.0038666725158691406, 0.0038623809814453125, 0.003834247589111328, 0.003812551498413086, 0.0038597583770751953, 0.0038313865661621094, 0.003837108612060547, 0.003996372222900391, 0.0038192272186279297, 0.0038537979125976562, 0.0038466453552246094, 0.0038542747497558594, 0.003854990005493164, 0.003980159759521484, 0.0038661956787109375, 0.0038156509399414062, 0.0038421154022216797, 0.003853321075439453, 0.003865957260131836, 0.0038619041442871094, 0.003864288330078125, 0.003874540328979492, 0.003834962844848633, 0.003870725631713867, 0.003825664520263672, 0.0038862228393554688, 0.003873586654663086, 0.003864765167236328, 0.003851652145385742, 0.0038003921508789062, 0.003856182098388672, 0.003815889358520508, 0.003926753997802734, 0.0038590431213378906, 0.0038573741912841797, 0.0038657188415527344, 0.0038106441497802734, 0.0038771629333496094, 0.003935337066650391, 0.0038437843322753906, 0.0038650035858154297, 0.003840208053588867, 0.0038673877716064453, 0.0038623809814453125, 0.0038542747497558594, 0.0038585662841796875, 0.003826141357421875, 0.0038564205169677734, 0.0038650035858154297, 0.003859281539916992, 0.0038170814514160156, 0.0038759708404541016, 0.003884553909301758, 0.0038492679595947266, 0.003869295120239258, 0.003823518753051758, 0.0038673877716064453, 0.003865480422973633, 0.003877878189086914, 0.0039522647857666016, 0.0038385391235351562, 0.0038788318634033203, 0.0038340091705322266, 0.0039348602294921875, 0.0038967132568359375, 0.0039052963256835938, 0.0038444995880126953, 0.0038170814514160156, 0.003864765167236328, 0.0038597583770751953, 0.003941059112548828, 0.0038542747497558594, 0.003862142562866211, 0.003812551498413086, 0.0038597583770751953, 0.003844022750854492, 0.003908634185791016, 0.003855466842651367, 0.0038156509399414062, 0.003905773162841797, 0.00403141975402832, 0.0038666725158691406, 0.0038983821868896484, 0.003871917724609375, 0.003878355026245117, 0.0038449764251708984, 0.0038471221923828125, 0.0038683414459228516, 0.003880023956298828, 0.003859996795654297, 0.003843069076538086, 0.0038576126098632812, 0.003856658935546875, 0.0038650035858154297, 0.0038945674896240234, 0.003832101821899414, 0.0038857460021972656, 0.003880739212036133, 0.0038645267486572266, 0.0038106441497802734, 0.0038623809814453125, 0.003863811492919922, 0.0038604736328125, 0.003846883773803711, 0.003838062286376953, 0.003875732421875, 0.003822803497314453, 0.003922700881958008, 0.003872394561767578, 0.003824472427368164, 0.0038442611694335938, 0.004019260406494141, 0.003876924514770508, 0.003892660140991211, 0.003874540328979492, 0.0039403438568115234, 0.003873586654663086, 0.00391840934753418, 0.003881692886352539, 0.003869771957397461, 0.0038604736328125, 0.0038683414459228516, 0.003824949264526367, 0.0039043426513671875, 0.0038454532623291016, 0.003922462463378906, 0.0038604736328125, 0.0038194656372070312, 0.0038614273071289062, 0.0038106441497802734, 0.0038595199584960938, 0.0038766860961914062, 0.0038836002349853516, 0.003858327865600586, 0.0038254261016845703, 0.003885030746459961, 0.0038623809814453125, 0.0038721561431884766, 0.003870248794555664, 0.003835916519165039, 0.0038580894470214844, 0.0038619041442871094, 0.0038557052612304688, 0.003830432891845703, 0.00386810302734375, 0.003859281539916992, 0.003856658935546875, 0.003854990005493164, 0.0038704872131347656, 0.0038394927978515625, 0.0038251876831054688, 0.0039005279541015625, 0.0038611888885498047, 0.003834247589111328, 0.003876209259033203, 0.003815889358520508, 0.003885507583618164, 0.003929615020751953, 0.003833770751953125, 0.003863096237182617, 0.0038154125213623047, 0.003871917724609375, 0.003915309906005859, 0.0038857460021972656, 0.0038945674896240234, 0.003841876983642578, 0.0038671493530273438, 0.0038733482360839844, 0.004042387008666992, 0.004024028778076172, 0.0038504600524902344, 0.0038568973541259766, 0.0038733482360839844, 0.0038595199584960938, 0.003840208053588867, 0.0038912296295166016, 0.0038878917694091797, 0.003898143768310547, 0.0038878917694091797, 0.003827333450317383, 0.003869771957397461, 0.0038712024688720703, 0.0038728713989257812, 0.0038759708404541016, 0.003826618194580078, 0.0038673877716064453, 0.0038459300994873047, 0.0038459300994873047, 0.004361391067504883, 0.003853321075439453, 0.003887653350830078, 0.0038192272186279297, 0.0038781166076660156, 0.003960847854614258, 0.003832578659057617, 0.0038712024688720703, 0.0038328170776367188, 0.0038602352142333984, 0.0038666725158691406, 0.003885030746459961, 0.003875255584716797, 0.0038390159606933594, 0.0038585662841796875, 0.003877401351928711, 0.003854990005493164, 0.0038743019104003906, 0.0038361549377441406, 0.003871440887451172, 0.0038623809814453125, 0.0038537979125976562, 0.003816366195678711, 0.003873109817504883, 0.0038547515869140625, 0.0038449764251708984, 0.0039446353912353516, 0.0038275718688964844, 0.0038404464721679688, 0.0038328170776367188, 0.003903627395629883, 0.0038368701934814453, 0.0038144588470458984, 0.0039882659912109375, 0.003806591033935547, 0.0038442611694335938, 0.003902912139892578, 0.003831148147583008, 0.0038535594940185547, 0.0038423538208007812, 0.003869771957397461, 0.0038824081420898438, 0.0038771629333496094, 0.0038983821868896484, 0.0038328170776367188, 0.00386810302734375, 0.0038797855377197266, 0.0038793087005615234, 0.0038199424743652344, 0.003854990005493164, 0.0038509368896484375, 0.0038580894470214844, 0.0038530826568603516, 0.003814697265625, 0.0038628578186035156, 0.003833293914794922, 0.0039098262786865234, 0.003880739212036133, 0.003831624984741211, 0.0038840770721435547, 0.0038404464721679688, 0.003875255584716797, 0.003909111022949219, 0.003822803497314453, 0.0038886070251464844, 0.0038335323333740234, 0.0038602352142333984, 0.00386810302734375, 0.0038690567016601562, 0.0038423538208007812, 0.0038306713104248047, 0.0038940906524658203, 0.003864765167236328, 0.0038628578186035156, 0.003881216049194336, 0.003821134567260742, 0.0038535594940185547, 0.003850698471069336, 0.003920793533325195, 0.0038373470306396484, 0.0038788318634033203, 0.003862619400024414, 0.003877401351928711, 0.003854036331176758, 0.003934383392333984, 0.0038673877716064453, 0.0038247108459472656, 0.003907680511474609, 0.0039234161376953125, 0.003971099853515625, 0.003943204879760742, 0.0038318634033203125, 0.0038764476776123047, 0.003936052322387695, 0.0037965774536132812, 0.0038564205169677734, 0.0038383007049560547, 0.0038788318634033203, 0.003890514373779297, 0.003873586654663086, 0.003873109817504883, 0.0038423538208007812, 0.0038764476776123047, 0.003851652145385742, 0.0038602352142333984, 0.0038945674896240234, 0.0038290023803710938, 0.0038785934448242188, 0.0038771629333496094, 0.003872394561767578, 0.003848552703857422, 0.003881216049194336, 0.0038690567016601562, 0.0038607120513916016, 0.0038595199584960938, 0.0038216114044189453, 0.0038912296295166016, 0.003841876983642578, 0.00391840934753418, 0.003879070281982422, 0.0038499832153320312, 0.0038559436798095703, 0.0038480758666992188, 0.003884553909301758, 0.003946065902709961, 0.003837108612060547, 0.0038864612579345703, 0.0038335323333740234, 0.003873586654663086, 0.003992319107055664, 0.0039038658142089844, 0.0038743019104003906, 0.0038645267486572266, 0.0038738250732421875, 0.003926277160644531, 0.0038671493530273438, 0.00400853157043457, 0.003940105438232422, 0.003929853439331055, 0.003946781158447266, 0.00393366813659668, 0.003924369812011719, 0.003893136978149414, 0.003928661346435547, 0.0039331912994384766, 0.003895998001098633, 0.003934144973754883, 0.00399327278137207, 0.003896951675415039, 0.003939151763916016, 0.004175901412963867, 0.003947734832763672, 0.0040171146392822266, 0.003958225250244141, 0.0039479732513427734, 0.003909111022949219, 0.004122018814086914, 0.00399470329284668, 0.003888845443725586, 0.003930091857910156, 0.0039055347442626953, 0.003916501998901367, 0.003946781158447266, 0.003962516784667969, 0.003932952880859375, 0.003896474838256836, 0.003949642181396484, 0.0039272308349609375, 0.003950595855712891, 0.0039234161376953125, 0.003909111022949219, 0.0039501190185546875, 0.003943443298339844, 0.003921985626220703, 0.003940105438232422, 0.003915309906005859, 0.003936290740966797, 0.003934383392333984, 0.003937959671020508, 0.003952980041503906, 0.0039942264556884766, 0.00394129753112793, 0.003937482833862305, 0.003923177719116211, 0.00392460823059082, 0.003934621810913086, 0.003924846649169922, 0.004022359848022461, 0.003935337066650391, 0.0038785934448242188, 0.003924131393432617, 0.0039327144622802734, 0.003942012786865234, 0.003930807113647461, 0.0038976669311523438, 0.003947257995605469, 0.00395655632019043, 0.003967761993408203, 0.003956317901611328, 0.003912925720214844, 0.003940582275390625, 0.0039501190185546875, 0.003941059112548828, 0.003924369812011719, 0.0038962364196777344, 0.003926753997802734, 0.003908634185791016, 0.003979682922363281, 0.003924846649169922, 0.003912448883056641, 0.0039348602294921875, 0.003888368606567383, 0.0039386749267578125, 0.00397181510925293, 0.0038747787475585938, 0.003925323486328125, 0.0038955211639404297, 0.00399470329284668, 0.003976583480834961, 0.003899097442626953, 0.003935813903808594, 0.003888368606567383, 0.003950834274291992, 0.00551152229309082, 0.004138946533203125, 0.0040051937103271484, 0.00397944450378418, 0.004904508590698242, 0.004047393798828125, 0.004098415374755859, 0.00415492057800293, 0.004216194152832031, 0.003962278366088867, 0.00394439697265625, 0.0039920806884765625, 0.0041294097900390625, 0.003975391387939453, 0.004037380218505859, 0.003954410552978516, 0.004094123840332031, 0.0039215087890625, 0.003945350646972656, 0.0039234161376953125, 0.003987312316894531, 0.003979206085205078, 0.00391697883605957, 0.003953456878662109, 0.0039691925048828125, 0.003958463668823242, 0.003960132598876953, 0.003984689712524414, 0.003988504409790039, 0.003967761993408203, 0.0039632320404052734, 0.003931999206542969, 0.004078388214111328, 0.004017353057861328, 0.003921985626220703, 0.003966808319091797, 0.003954410552978516, 0.0040209293365478516, 0.0039463043212890625, 0.003927707672119141, 0.00396418571472168, 0.003992795944213867, 0.003947257995605469, 0.004004001617431641, 0.003944873809814453, 0.003975391387939453, 0.003949880599975586, 0.003972053527832031, 0.003945589065551758, 0.004042863845825195, 0.00395512580871582, 0.003968000411987305, 0.003946542739868164, 0.003969907760620117, 0.003951549530029297, 0.003942012786865234, 0.003937959671020508, 0.0040776729583740234, 0.0040090084075927734, 0.003916501998901367, 0.003943204879760742, 0.003968715667724609, 0.003935575485229492, 0.003957986831665039, 0.0040051937103271484, 0.003936052322387695, 0.003946781158447266, 0.003942012786865234, 0.003931283950805664, 0.003962993621826172, 0.0039272308349609375, 0.003928661346435547, 0.003971576690673828, 0.003927946090698242, 0.003965616226196289, 0.003954887390136719, 0.003964662551879883, 0.003953218460083008, 0.00391840934753418, 0.003945827484130859, 0.00394439697265625, 0.003977537155151367, 0.004266977310180664, 0.003957986831665039, 0.003977298736572266, 0.003964662551879883, 0.003957033157348633, 0.003951072692871094, 0.0039288997650146484, 0.003990888595581055, 0.003948211669921875, 0.003934621810913086, 0.003957509994506836, 0.003933906555175781, 0.0039937496185302734, 0.003984689712524414, 0.003955364227294922, 0.00395965576171875, 0.003916025161743164, 0.003956317901611328, 0.0039637088775634766, 0.003984928131103516, 0.003972053527832031, 0.0039174556732177734, 0.0039517879486083984, 0.0038988590240478516, 0.004003286361694336, 0.003958702087402344, 0.003995418548583984, 0.003980159759521484, 0.0039179325103759766, 0.0040166378021240234, 0.003964424133300781, 0.003915548324584961, 0.0039598941802978516, 0.003991842269897461, 0.004003047943115234, 0.003954410552978516, 0.003932476043701172, 0.003983974456787109, 0.003985881805419922, 0.004027605056762695, 0.00395512580871582, 0.003911495208740234, 0.003947257995605469, 0.0039060115814208984, 0.003972530364990234, 0.004140377044677734, 0.003948688507080078, 0.003966093063354492, 0.0039327144622802734, 0.003954172134399414, 0.004021167755126953, 0.00392603874206543, 0.004015207290649414, 0.003961086273193359, 0.003961086273193359, 0.00401616096496582, 0.0039288997650146484, 0.003975391387939453, 0.003945589065551758, 0.003962516784667969, 0.0040035247802734375, 0.003938913345336914, 0.003978729248046875, 0.003923654556274414, 0.00393986701965332, 0.003960609436035156, 0.003983020782470703, 0.003969907760620117, 0.003910064697265625, 0.003962039947509766, 0.003976583480834961, 0.003990888595581055, 0.003987312316894531, 0.0039119720458984375, 0.003965616226196289, 0.003980875015258789, 0.004073143005371094, 0.003992795944213867, 0.00394129753112793, 0.003986358642578125, 0.003962039947509766, 0.003983497619628906, 0.003975391387939453, 0.004018545150756836, 0.003973245620727539, 0.00397491455078125, 0.003988027572631836, 0.003976345062255859, 0.00390934944152832, 0.003959178924560547, 0.003948688507080078, 0.003968000411987305, 0.003968954086303711, 0.0039255619049072266, 0.0039520263671875, 0.003957033157348633, 0.003989219665527344, 0.004009246826171875, 0.003911018371582031, 0.0039594173431396484, 0.003973484039306641, 0.003981590270996094, 0.003996372222900391, 0.0039179325103759766, 0.003957986831665039, 0.00396275520324707, 0.0039577484130859375, 0.0041046142578125, 0.004061698913574219, 0.003951072692871094, 0.0039730072021484375, 0.00396728515625, 0.003926753997802734, 0.00397801399230957, 0.0039675235748291016, 0.0039730072021484375, 0.003973484039306641, 0.003932476043701172, 0.003969907760620117, 0.00396728515625, 0.003968477249145508, 0.003979206085205078, 0.003938913345336914, 0.003984928131103516, 0.003976583480834961, 0.003989219665527344, 0.003965616226196289, 0.004045724868774414, 0.003968477249145508, 0.003961324691772461, 0.003970146179199219, 0.003981590270996094, 0.003937244415283203, 0.0039789676666259766, 0.004086971282958984, 0.00397944450378418, 0.003975629806518555, 0.003943204879760742, 0.00397181510925293, 0.003989696502685547, 0.003980159759521484, 0.003972053527832031, 0.003934144973754883, 0.003953218460083008, 0.003998994827270508, 0.003983974456787109, 0.003980875015258789, 0.003929853439331055, 0.003975629806518555, 0.003990888595581055, 0.003965616226196289, 0.003972053527832031, 0.003938913345336914, 0.004029035568237305, 0.003972291946411133, 0.003966569900512695, 0.003987550735473633, 0.003923892974853516, 0.0039784908294677734, 0.003951549530029297, 0.004007816314697266, 0.003969430923461914, 0.003943204879760742, 0.004009723663330078, 0.003991127014160156, 0.004074811935424805, 0.003996133804321289, 0.003968477249145508, 0.0039441585540771484, 0.003943920135498047, 0.003962278366088867, 0.003956794738769531, 0.003977298736572266, 0.003968477249145508, 0.0039403438568115234, 0.004002809524536133, 0.003990650177001953, 0.004084348678588867, 0.003999233245849609, 0.003966331481933594, 0.003975868225097656, 0.00398564338684082, 0.004152774810791016, 0.003999471664428711, 0.0040705204010009766, 0.0039942264556884766, 0.003988742828369141, 0.003985881805419922, 0.003977537155151367, 0.0039522647857666016, 0.0039784908294677734, 0.003979682922363281, 0.003981113433837891, 0.003957986831665039, 0.00394129753112793, 0.003991127014160156, 0.003986835479736328, 0.003991127014160156, 0.003960132598876953, 0.003923892974853516, 0.003989696502685547, 0.004003047943115234, 0.003973245620727539, 0.003974199295043945, 0.003926753997802734, 0.003976345062255859, 0.004001140594482422, 0.003983020782470703, 0.003962993621826172, 0.003911733627319336, 0.00396418571472168, 0.0039825439453125, 0.003978252410888672, 0.003988742828369141, 0.0039980411529541016, 0.003986358642578125, 0.00400233268737793, 0.004007101058959961, 0.003984689712524414, 0.003946781158447266, 0.0039958953857421875, 0.003992557525634766, 0.003989458084106445, 0.003966093063354492, 0.00394439697265625, 0.004007816314697266, 0.003996133804321289, 0.004100799560546875, 0.004000663757324219, 0.003959178924560547, 0.004001617431640625, 0.0039904117584228516, 0.004078388214111328, 0.003992319107055664, 0.004044055938720703, 0.003977537155151367, 0.004001617431640625, 0.0039784908294677734, 0.0039751529693603516, 0.0039331912994384766, 0.0039615631103515625, 0.003995656967163086, 0.004008054733276367, 0.004010200500488281, 0.003980398178100586, 0.0040130615234375, 0.0040552616119384766, 0.004015684127807617, 0.004021167755126953, 0.00396728515625, 0.004033565521240234, 0.004045963287353516, 0.0040357112884521484, 0.003998279571533203, 0.004019975662231445, 0.004014015197753906, 0.004026174545288086, 0.004007577896118164, 0.004015207290649414, 0.003965854644775391, 0.0040225982666015625, 0.004029750823974609, 0.0040171146392822266, 0.004024505615234375, 0.003971099853515625, 0.0040264129638671875, 0.004019498825073242, 0.004003286361694336, 0.004014492034912109, 0.003976106643676758, 0.004007816314697266, 0.0040302276611328125, 0.004017353057861328, 0.004026651382446289, 0.003974199295043945, 0.00400543212890625, 0.0040476322174072266, 0.0040967464447021484, 0.0040323734283447266, 0.0039861202239990234, 0.004039287567138672, 0.004011869430541992, 0.004026889801025391, 0.004008293151855469, 0.0040509700775146484, 0.00405430793762207, 0.004038333892822266, 0.004019737243652344, 0.004026651382446289, 0.00398564338684082, 0.004054546356201172, 0.0040357112884521484, 0.004296541213989258, 0.004067659378051758, 0.004094600677490234, 0.004045724868774414, 0.0040302276611328125, 0.004046201705932617, 0.0040416717529296875, 0.004087686538696289, 0.0040361881256103516, 0.003977298736572266, 0.0040738582611083984, 0.00400853157043457, 0.0040874481201171875, 0.004024982452392578, 0.0040073394775390625, 0.004063606262207031, 0.004060268402099609, 0.0040435791015625, 0.004042387008666992, 0.0040035247802734375, 0.0040280818939208984, 0.004047393798828125, 0.004033565521240234, 0.004033803939819336, 0.003977537155151367, 0.004037141799926758, 0.004035472869873047, 0.004025936126708984, 0.004040718078613281, 0.003992319107055664, 0.004040241241455078, 0.004043102264404297, 0.00405430793762207, 0.00404047966003418, 0.003996849060058594, 0.004152059555053711, 0.004047870635986328, 0.0040285587310791016, 0.004054069519042969, 0.00417780876159668, 0.004114389419555664, 0.004069328308105469, 0.0041255950927734375, 0.004048347473144531, 0.004058361053466797, 0.004008769989013672, 0.004040718078613281, 0.004041433334350586, 0.004054546356201172, 0.004045009613037109, 0.00400090217590332, 0.00403594970703125, 0.004044055938720703, 0.004027605056762695, 0.004038333892822266, 0.004003047943115234, 0.0040090084075927734, 0.004080533981323242, 0.00457763671875, 0.004050016403198242, 0.004413127899169922, 0.004068613052368164, 0.004107952117919922, 0.004004955291748047, 0.004029989242553711, 0.0040340423583984375, 0.0040433406829833984, 0.004105091094970703, 0.004010677337646484, 0.004033088684082031, 0.004006147384643555, 0.004091978073120117, 0.00403285026550293, 0.00400543212890625, 0.004025936126708984, 0.0040018558502197266, 0.004099845886230469, 0.0040323734283447266, 0.003988742828369141, 0.004060506820678711, 0.004015684127807617, 0.00410151481628418, 0.00406956672668457, 0.004019260406494141, 0.00412297248840332, 0.004093647003173828, 0.004023551940917969, 0.004068136215209961, 0.003999233245849609, 0.004030704498291016, 0.004049062728881836, 0.004154682159423828, 0.004056692123413086, 0.003998756408691406, 0.004029512405395508, 0.004090547561645508, 0.004047393798828125, 0.00403594970703125, 0.00399470329284668, 0.004065036773681641, 0.004075050354003906, 0.004052639007568359, 0.004039287567138672, 0.0040073394775390625, 0.004038333892822266, 0.004033565521240234, 0.004022836685180664, 0.004046440124511719, 0.0040073394775390625, 0.00404667854309082, 0.004082441329956055, 0.004019260406494141, 0.0040454864501953125, 0.0040400028228759766, 0.0040264129638671875, 0.004056453704833984, 0.004028797149658203, 0.004052877426147461, 0.0040438175201416016, 0.003993988037109375, 0.004075050354003906, 0.00403594970703125, 0.004036426544189453, 0.004057407379150391, 0.004012346267700195, 0.00426030158996582, 0.004079103469848633, 0.004054546356201172, 0.00404667854309082, 0.0040056705474853516, 0.004056215286254883, 0.004053354263305664, 0.0040700435638427734, 0.004153013229370117, 0.004019498825073242, 0.004057407379150391, 0.0041141510009765625, 0.004016876220703125, 0.0041921138763427734, 0.004040241241455078, 0.004041910171508789, 0.004193782806396484, 0.004027128219604492, 0.004057645797729492, 0.004036903381347656, 0.004079103469848633, 0.004063129425048828, 0.0040149688720703125, 0.004058122634887695, 0.004052400588989258, 0.004061460494995117, 0.004050731658935547, 0.00402522087097168, 0.004062652587890625, 0.004051923751831055, 0.004084348678588867, 0.0040438175201416016, 0.004146575927734375, 0.004067897796630859, 0.004103899002075195, 0.0040111541748046875, 0.0040569305419921875, 0.004021167755126953, 0.004041194915771484, 0.004098415374755859, 0.004013538360595703, 0.0040509700775146484, 0.004004001617431641, 0.00408482551574707, 0.004052400588989258, 0.004011869430541992, 0.004027605056762695, 0.0040128231048583984, 0.0041179656982421875, 0.004042387008666992, 0.004003047943115234, 0.0040700435638427734, 0.0040149688720703125, 0.004086732864379883, 0.004027605056762695, 0.004018306732177734, 0.00403285026550293, 0.0040662288665771484, 0.004125356674194336, 0.004060983657836914, 0.0040247440338134766, 0.004060268402099609, 0.004076957702636719, 0.004059314727783203, 0.004052877426147461, 0.0041637420654296875, 0.004065275192260742, 0.004064798355102539, 0.0040929317474365234, 0.004050016403198242, 0.004011392593383789, 0.0040323734283447266, 0.004042148590087891, 0.0040476322174072266, 0.004050493240356445, 0.004034757614135742, 0.0040874481201171875, 0.004034280776977539, 0.004045009613037109, 0.0040378570556640625, 0.004004001617431641, 0.0040547847747802734, 0.004058361053466797, 0.004052877426147461, 0.004044532775878906, 0.004060506820678711, 0.004008054733276367, 0.004063129425048828, 0.004040956497192383, 0.004059791564941406, 0.004073619842529297, 0.004016876220703125, 0.004050016403198242, 0.004054546356201172, 0.0040705204010009766, 0.004071235656738281, 0.004014015197753906, 0.0040531158447265625, 0.00404667854309082, 0.004052400588989258, 0.004048585891723633, 0.004010677337646484, 0.00404047966003418, 0.004071235656738281, 0.004064798355102539, 0.004053592681884766, 0.004023075103759766, 0.004149913787841797, 0.004111051559448242, 0.0040111541748046875, 0.004053354263305664, 0.004027843475341797, 0.004076957702636719, 0.004140138626098633, 0.004102945327758789, 0.004072427749633789, 0.004019260406494141, 0.004107952117919922, 0.004071950912475586, 0.004612445831298828, 0.004084110260009766, 0.004057884216308594, 0.004330635070800781, 0.004191875457763672, 0.0040264129638671875, 0.0040967464447021484, 0.004087686538696289, 0.004060983657836914, 0.004074573516845703, 0.004030466079711914, 0.004075288772583008, 0.004060268402099609, 0.004055976867675781, 0.004105091094970703, 0.004073143005371094, 0.00407099723815918, 0.0040781497955322266, 0.0040662288665771484, 0.004068136215209961, 0.004082441329956055, 0.004042863845825195, 0.004071712493896484, 0.004081249237060547, 0.004103660583496094, 0.004059791564941406, 0.004046201705932617, 0.004062175750732422, 0.0040645599365234375, 0.004073381423950195, 0.004057168960571289, 0.00403904914855957, 0.004049539566040039, 0.004120349884033203, 0.004029512405395508, 0.004052162170410156, 0.004099607467651367, 0.004198312759399414, 0.004137277603149414, 0.0040400028228759766, 0.0040607452392578125, 0.004045963287353516, 0.00410771369934082, 0.004079341888427734, 0.004148960113525391, 0.004060029983520508, 0.004393815994262695, 0.004168272018432617, 0.004040956497192383, 0.0040781497955322266, 0.004090785980224609, 0.004095554351806641, 0.004122495651245117, 0.004035472869873047, 0.004087686538696289, 0.004130125045776367, 0.004030942916870117, 0.0040667057037353516, 0.004027605056762695, 0.0041124820709228516, 0.004138469696044922, 0.004059791564941406, 0.004114866256713867, 0.004026174545288086, 0.004128217697143555, 0.0040857791900634766, 0.004040718078613281, 0.0041043758392333984, 0.004082679748535156, 0.004075765609741211, 0.00407862663269043, 0.004042863845825195, 0.004080772399902344, 0.0040967464447021484, 0.00407719612121582, 0.004077911376953125, 0.0040361881256103516, 0.00408935546875, 0.00407099723815918, 0.004063606262207031, 0.004083156585693359, 0.004044294357299805, 0.004116535186767578, 0.004079580307006836, 0.004075288772583008, 0.00409245491027832, 0.004163026809692383, 0.004033088684082031, 0.004067659378051758, 0.0040776729583740234, 0.004077911376953125, 0.004084348678588867, 0.004088640213012695, 0.004153251647949219, 0.004070281982421875, 0.004093647003173828, 0.004078388214111328, 0.00403904914855957, 0.004087686538696289, 0.004102468490600586, 0.0040814876556396484, 0.004075765609741211, 0.004035472869873047, 0.00406336784362793, 0.004117012023925781, 0.004038572311401367, 0.004091978073120117, 0.0040400028228759766, 0.0041005611419677734, 0.004140138626098633, 0.004029750823974609, 0.004081249237060547, 0.0040187835693359375, 0.004116058349609375, 0.0041010379791259766, 0.004116535186767578, 0.0042569637298583984, 0.00417017936706543, 0.0041179656982421875, 0.0041217803955078125, 0.004085063934326172, 0.0041348934173583984, 0.0041713714599609375, 0.004117012023925781, 0.004111766815185547, 0.004085063934326172, 0.004124641418457031, 0.004103183746337891, 0.004115581512451172, 0.00413823127746582, 0.0041239261627197266, 0.004096269607543945, 0.0041351318359375, 0.004221439361572266, 0.006337881088256836, 0.006734371185302734, 0.004175901412963867, 0.004137754440307617, 0.004235744476318359, 0.004119157791137695, 0.00416111946105957, 0.004266262054443359, 0.004153728485107422, 0.004084348678588867, 0.004149436950683594, 0.004140377044677734, 0.0041348934173583984, 0.004111289978027344, 0.004092693328857422, 0.00414276123046875, 0.004105567932128906, 0.004140138626098633, 0.004128217697143555, 0.0041429996490478516, 0.0041048526763916016, 0.004120588302612305, 0.00416254997253418, 0.004193782806396484, 0.004150390625, 0.00410008430480957, 0.004110813140869141, 0.0041806697845458984, 0.0040950775146484375, 0.004121541976928711, 0.00408935546875, 0.004216909408569336, 0.004136562347412109, 0.004091024398803711, 0.004116058349609375, 0.0041391849517822266, 0.0041348934173583984, 0.004144906997680664, 0.004094600677490234, 0.004130840301513672, 0.0041234493255615234, 0.00411224365234375, 0.004144906997680664, 0.004098415374755859, 0.0041446685791015625, 0.004132986068725586, 0.004129171371459961, 0.004215240478515625, 0.004170417785644531, 0.0040988922119140625, 0.004137277603149414, 0.004125833511352539, 0.004139423370361328, 0.004122018814086914, 0.004095792770385742, 0.0042269229888916016, 0.004187822341918945, 0.004111289978027344, 0.0041081905364990234, 0.00408935546875, 0.004182100296020508, 0.004134654998779297, 0.004110574722290039, 0.004116058349609375, 0.004086971282958984, 0.0041828155517578125, 0.00413203239440918, 0.004083395004272461, 0.004125356674194336, 0.004149913787841797, 0.004143953323364258, 0.004133939743041992, 0.004102230072021484, 0.004192352294921875, 0.004137754440307617, 0.004131317138671875, 0.004120826721191406, 0.004148721694946289, 0.00409245491027832, 0.004122734069824219, 0.0041353702545166016, 0.004136800765991211, 0.004144430160522461, 0.004134654998779297, 0.004148006439208984, 0.004189729690551758, 0.004114866256713867, 0.004124164581298828, 0.004097461700439453, 0.004109859466552734, 0.004174232482910156, 0.004098176956176758, 0.004157304763793945, 0.004076480865478516, 0.0041828155517578125, 0.004117727279663086, 0.004250764846801758, 0.004150867462158203, 0.004183530807495117, 0.004150390625, 0.004145145416259766, 0.0041048526763916016, 0.004144430160522461, 0.004151344299316406, 0.004266977310180664, 0.004141330718994141, 0.004148244857788086, 0.0041391849517822266, 0.004141092300415039, 0.0041522979736328125, 0.004164457321166992, 0.004155635833740234, 0.004109859466552734, 0.004118680953979492, 0.004183292388916016, 0.004085063934326172, 0.004145622253417969, 0.004096269607543945, 0.004190683364868164, 0.004114627838134766, 0.004103899002075195, 0.004156827926635742, 0.004173755645751953, 0.004138469696044922, 0.004181623458862305, 0.004087209701538086, 0.0041506290435791016, 0.004142045974731445, 0.0041522979736328125, 0.004133939743041992, 0.004099130630493164, 0.004160642623901367, 0.004105329513549805, 0.0041217803955078125, 0.0041544437408447266, 0.004183053970336914, 0.004106760025024414, 0.0041544437408447266, 0.00413060188293457, 0.004133939743041992, 0.004147529602050781, 0.00410914421081543, 0.0041697025299072266, 0.004216670989990234, 0.004182577133178711, 0.004180908203125, 0.004099130630493164, 0.00422358512878418, 0.004152059555053711, 0.004102945327758789, 0.004151344299316406, 0.004172563552856445, 0.004244565963745117, 0.004126548767089844, 0.004114389419555664, 0.004160642623901367, 0.004112958908081055, 0.004141330718994141, 0.004125356674194336, 0.004137516021728516, 0.0041217803955078125, 0.0041577816009521484, 0.004147529602050781, 0.004166603088378906, 0.0041599273681640625, 0.004104137420654297, 0.004171609878540039, 0.004221200942993164, 0.004132270812988281, 0.004479169845581055, 0.004186153411865234, 0.004208087921142578, 0.004164934158325195, 0.005061626434326172, 0.006531238555908203, 0.005166769027709961, 0.00418400764465332, 0.0041294097900390625, 0.00417327880859375, 0.0041348934173583984, 0.004147529602050781, 0.004155397415161133, 0.004172563552856445, 0.00411224365234375, 0.0041599273681640625, 0.004144191741943359, 0.004140138626098633, 0.004146099090576172, 0.0041103363037109375, 0.0041387081146240234, 0.004203081130981445, 0.004180431365966797, 0.004177570343017578, 0.0041027069091796875, 0.0042264461517333984, 0.004147529602050781, 0.0041005611419677734, 0.004179716110229492, 0.0041697025299072266, 0.004225492477416992, 0.004194021224975586, 0.004108905792236328, 0.004203319549560547, 0.0041370391845703125, 0.004122734069824219, 0.0041522979736328125, 0.004166603088378906, 0.004164934158325195, 0.00413966178894043, 0.004094123840332031, 0.004148006439208984, 0.004152774810791016, 0.004177093505859375, 0.004156589508056641, 0.004166364669799805, 0.00412440299987793, 0.0042345523834228516, 0.0041790008544921875, 0.004172563552856445, 0.004146099090576172, 0.0041620731353759766, 0.00413203239440918, 0.00421452522277832, 0.004121303558349609, 0.0041353702545166016, 0.004124879837036133, 0.0042002201080322266, 0.0042302608489990234, 0.0041675567626953125, 0.004143476486206055, 0.0041828155517578125, 0.004121065139770508, 0.004151344299316406, 0.00415492057800293, 0.004212617874145508, 0.004183292388916016, 0.004133701324462891, 0.0041921138763427734, 0.004222869873046875, 0.004199981689453125, 0.004175424575805664, 0.004130363464355469, 0.0042400360107421875, 0.0041656494140625, 0.0042498111724853516, 0.004164695739746094, 0.004161834716796875, 0.004271268844604492, 0.004158973693847656, 0.004133462905883789, 0.004170656204223633, 0.004168272018432617, 0.004165172576904297, 0.0041768550872802734, 0.004151821136474609, 0.004111766815185547, 0.004144191741943359, 0.0041735172271728516, 0.004176139831542969, 0.004178762435913086, 0.004128456115722656, 0.004159688949584961, 0.004320621490478516, 0.004186153411865234, 0.004351377487182617, 0.00420379638671875, 0.0042264461517333984, 0.004184722900390625, 0.004121303558349609, 0.004161834716796875, 0.004225730895996094, 0.0041656494140625, 0.004155397415161133, 0.004127025604248047, 0.004174232482910156, 0.004131317138671875, 0.004138946533203125, 0.0041255950927734375, 0.004177570343017578, 0.004142284393310547, 0.00416254997253418, 0.004152536392211914, 0.004176139831542969, 0.0041294097900390625, 0.004110574722290039, 0.00415349006652832, 0.0042018890380859375, 0.00421452522277832, 0.0041654109954833984, 0.004105567932128906, 0.004207611083984375, 0.004366397857666016, 0.0041081905364990234, 0.004151344299316406, 0.004189729690551758, 0.0042531490325927734, 0.004173755645751953, 0.004130125045776367, 0.004155158996582031, 0.004143953323364258, 0.004168987274169922, 0.004173755645751953, 0.004187107086181641, 0.004279613494873047, 0.00418853759765625, 0.004228115081787109, 0.004097938537597656, 0.0041582584381103516, 0.004106283187866211, 0.0041353702545166016, 0.004201173782348633, 0.004108905792236328, 0.004169940948486328, 0.004129886627197266, 0.004200458526611328, 0.0041656494140625, 0.004121541976928711, 0.004188060760498047, 0.004172086715698242, 0.004341840744018555, 0.0041620731353759766, 0.00417327880859375, 0.004192352294921875, 0.004177093505859375, 0.004170894622802734, 0.004193305969238281, 0.00417327880859375, 0.004125118255615234, 0.004129886627197266, 0.0042150020599365234, 0.004117250442504883, 0.004143476486206055, 0.004125833511352539, 0.004187107086181641, 0.0042073726654052734, 0.004190206527709961, 0.0041658878326416016, 0.0041692256927490234, 0.0041522979736328125, 0.0041484832763671875, 0.004126548767089844, 0.0041615962982177734, 0.004176616668701172, 0.004300594329833984, 0.004221200942993164, 0.004216909408569336, 0.004133462905883789, 0.00446629524230957, 0.004173994064331055, 0.004189252853393555, 0.0041942596435546875, 0.004128694534301758, 0.004388570785522461, 0.004255056381225586, 0.0041866302490234375, 0.0041964054107666016, 0.004190921783447266, 0.004182577133178711, 0.0041408538818359375, 0.0041768550872802734, 0.004223823547363281, 0.004190921783447266, 0.0041620731353759766, 0.004126310348510742, 0.00423121452331543, 0.004145145416259766, 0.004286050796508789, 0.00417780876159668, 0.004198789596557617, 0.004141807556152344, 0.0041844844818115234, 0.0041272640228271484, 0.0042171478271484375, 0.004167318344116211, 0.004200458526611328, 0.004209995269775391, 0.004181623458862305, 0.00414729118347168, 0.004156589508056641, 0.004200458526611328, 0.004183292388916016, 0.0041654109954833984, 0.004158973693847656, 0.0042722225189208984, 0.0042150020599365234, 0.0041615962982177734, 0.004196882247924805, 0.004177093505859375, 0.0042362213134765625, 0.004175662994384766, 0.004160165786743164, 0.004386425018310547, 0.0042874813079833984, 0.004224538803100586, 0.00419163703918457, 0.004314422607421875, 0.0042188167572021484, 0.004201173782348633, 0.004258632659912109, 0.0042073726654052734, 0.004221916198730469, 0.004199981689453125, 0.0041844844818115234, 0.004240751266479492, 0.004224538803100586, 0.004266977310180664, 0.0042421817779541016, 0.004266500473022461, 0.004202127456665039, 0.0042493343353271484, 0.004277944564819336, 0.0041751861572265625, 0.004271030426025391, 0.004193305969238281, 0.004273891448974609, 0.004235744476318359, 0.004202842712402344, 0.004271984100341797, 0.004195213317871094, 0.004229307174682617, 0.004221439361572266, 0.0042378902435302734, 0.0042057037353515625, 0.004201650619506836, 0.0042192935943603516, 0.00422358512878418, 0.0042095184326171875, 0.0041887760162353516, 0.004212141036987305, 0.004259824752807617, 0.004267215728759766, 0.004231691360473633, 0.004225969314575195, 0.004220485687255859, 0.004232645034790039, 0.004188060760498047, 0.004278898239135742, 0.0042226314544677734, 0.004341602325439453, 0.004221200942993164, 0.0042552947998046875, 0.0042073726654052734, 0.004240989685058594, 0.004305124282836914, 0.004222393035888672, 0.0042116641998291016, 0.004191160202026367, 0.004307270050048828, 0.004218339920043945, 0.004194021224975586, 0.004252195358276367, 0.004262447357177734, 0.004243135452270508, 0.004235267639160156, 0.0042612552642822266, 0.0042231082916259766, 0.004229545593261719, 0.0042574405670166016, 0.004220247268676758, 0.004233121871948242, 0.004198551177978516, 0.0042324066162109375, 0.004251241683959961, 0.0041582584381103516, 0.004213809967041016, 0.004236459732055664, 0.0043065547943115234, 0.0042934417724609375, 0.00420832633972168, 0.004206418991088867, 0.0042133331298828125, 0.004202127456665039, 0.004233121871948242, 0.004286050796508789, 0.004183053970336914, 0.004210710525512695, 0.004192829132080078, 0.004387855529785156, 0.004230022430419922, 0.004181623458862305, 0.0042417049407958984, 0.004212617874145508, 0.004370927810668945, 0.004235506057739258, 0.004230022430419922, 0.00419163703918457, 0.004365444183349609, 0.004268169403076172, 0.004183530807495117, 0.0042247772216796875, 0.004197835922241211, 0.004264116287231445, 0.004230499267578125, 0.004216194152832031, 0.004241943359375, 0.0042476654052734375, 0.0042247772216796875, 0.0042421817779541016, 0.004175662994384766, 0.0042569637298583984, 0.004241943359375, 0.00477147102355957, 0.0045719146728515625, 0.00425410270690918, 0.004224538803100586, 0.0043141841888427734, 0.004252195358276367, 0.004248142242431641, 0.004239559173583984, 0.004251956939697266, 0.004255056381225586, 0.004209041595458984, 0.004238605499267578, 0.00417780876159668, 0.0042612552642822266, 0.004240512847900391, 0.004267692565917969, 0.0042307376861572266, 0.004194974899291992, 0.004261970520019531, 0.004308938980102539, 0.004218578338623047, 0.0042536258697509766, 0.0042574405670166016, 0.004343986511230469, 0.0042345523834228516, 0.004221916198730469, 0.004263162612915039, 0.004235267639160156, 0.004252910614013672, 0.004277706146240234, 0.004259586334228516, 0.004208087921142578, 0.004361867904663086, 0.00429224967956543, 0.00418400764465332, 0.0042035579681396484, 0.0042078495025634766, 0.004252195358276367, 0.004212617874145508, 0.004209041595458984, 0.004259586334228516, 0.004267454147338867, 0.0042378902435302734, 0.004237174987792969, 0.004335165023803711, 0.004219770431518555, 0.004236698150634766, 0.004253864288330078, 0.004274129867553711, 0.00424957275390625, 0.004239797592163086, 0.004283428192138672, 0.004248142242431641, 0.004182577133178711, 0.004240989685058594, 0.00425267219543457, 0.004258394241333008, 0.00426793098449707, 0.004216670989990234, 0.004244804382324219, 0.0042536258697509766, 0.00423431396484375, 0.004265785217285156, 0.004233598709106445, 0.004182100296020508, 0.004220485687255859, 0.004311561584472656, 0.004205942153930664, 0.00423884391784668, 0.0041961669921875, 0.0043909549713134766, 0.004265308380126953, 0.00419306755065918, 0.0042841434478759766, 0.004351139068603516, 0.004271507263183594, 0.004243135452270508, 0.004263877868652344, 0.004204750061035156, 0.00433039665222168, 0.004286527633666992, 0.00423431396484375, 0.004287242889404297, 0.004208087921142578, 0.004306316375732422, 0.004266262054443359, 0.0042209625244140625, 0.0043163299560546875, 0.004255056381225586, 0.004229545593261719, 0.004251241683959961, 0.0042591094970703125, 0.004206418991088867, 0.004249095916748047, 0.0042591094970703125, 0.004260063171386719, 0.004232645034790039, 0.004228353500366211, 0.00429987907409668, 0.004256486892700195, 0.0042192935943603516, 0.004250049591064453, 0.004250288009643555, 0.004237651824951172, 0.004224061965942383, 0.00420069694519043, 0.004248857498168945, 0.004231452941894531, 0.004254579544067383, 0.0042722225189208984, 0.0042667388916015625, 0.00423121452331543, 0.004259824752807617, 0.0043032169342041016, 0.00421452522277832, 0.004235267639160156, 0.0042572021484375, 0.004459857940673828, 0.004252910614013672, 0.004210233688354492, 0.004279613494873047, 0.004239797592163086, 0.004213571548461914, 0.004223346710205078, 0.00428318977355957, 0.0042247772216796875, 0.0043354034423828125, 0.004331111907958984, 0.00424647331237793, 0.004226207733154297, 0.004254817962646484, 0.0042724609375, 0.004250288009643555, 0.004251003265380859, 0.0042917728424072266, 0.004270792007446289, 0.00783848762512207, 0.0044558048248291016, 0.013324499130249023, 0.009720802307128906, 0.005074024200439453, 0.004620552062988281, 0.004515171051025391, 0.004370689392089844, 0.004354715347290039, 0.004343748092651367, 0.004361391067504883, 0.004347324371337891, 0.004329681396484375, 0.004408597946166992, 0.004312992095947266, 0.004322528839111328, 0.004352092742919922, 0.0042684078216552734, 0.0042591094970703125, 0.004300117492675781, 0.0043375492095947266, 0.004288673400878906, 0.004270076751708984, 0.004489421844482422, 0.004228115081787109, 0.004320383071899414, 0.004375934600830078, 0.004315614700317383, 0.004301309585571289, 0.00430750846862793, 0.004381656646728516, 0.004255533218383789, 0.004412174224853516, 0.004351139068603516, 0.004335165023803711, 0.004288196563720703, 0.00428462028503418, 0.004319667816162109, 0.004354715347290039, 0.0042934417724609375, 0.004329681396484375, 0.0042722225189208984, 0.005357503890991211, 0.004360675811767578, 0.004309892654418945, 0.004355430603027344, 0.0043680667877197266, 0.004283428192138672, 0.004303693771362305, 0.0044176578521728516, 0.004251956939697266, 0.004276275634765625, 0.004280567169189453, 0.004270792007446289, 0.00429224967956543, 0.004324197769165039, 0.004269599914550781, 0.004321098327636719, 0.004336357116699219, 0.004286289215087891, 0.004320859909057617, 0.0042722225189208984, 0.004350423812866211, 0.004339933395385742, 0.004255533218383789, 0.0043065547943115234, 0.004331350326538086, 0.004313945770263672, 0.004326820373535156, 0.004444122314453125, 0.004265308380126953, 0.004247903823852539, 0.004328727722167969, 0.004262447357177734, 0.0043256282806396484, 0.004458427429199219, 0.00436854362487793, 0.004295825958251953, 0.004369020462036133, 0.004302263259887695, 0.004323005676269531, 0.004259347915649414, 0.004282236099243164, 0.00426936149597168, 0.004214286804199219, 0.0042874813079833984, 0.004372596740722656, 0.004282236099243164, 0.004302501678466797, 0.004237174987792969, 0.004398822784423828, 0.004320621490478516, 0.004442691802978516, 0.004335880279541016, 0.0043065547943115234, 0.004311800003051758, 0.004302978515625, 0.004297733306884766, 0.004324197769165039, 0.0043032169342041016, 0.004408836364746094, 0.00428462028503418, 0.004328489303588867, 0.004349946975708008, 0.004281759262084961, 0.004343748092651367, 0.00431370735168457, 0.004296064376831055, 0.004297971725463867, 0.004298210144042969, 0.004351615905761719, 0.004311561584472656, 0.004271745681762695, 0.004368782043457031, 0.0043239593505859375, 0.004273414611816406, 0.004461526870727539, 0.004310131072998047, 0.004282236099243164, 0.004683971405029297, 0.004300117492675781, 0.004277229309082031, 0.00432133674621582, 0.004348278045654297, 0.004319190979003906, 0.0043027400970458984, 0.004311561584472656, 0.004275083541870117, 0.004293918609619141, 0.004261970520019531, 0.00433349609375, 0.0043621063232421875, 0.004316806793212891, 0.0043294429779052734, 0.00433659553527832, 0.004297018051147461, 0.004296064376831055, 0.00426936149597168, 0.0043354034423828125, 0.004293680191040039, 0.0043146610260009766, 0.00432586669921875, 0.004300355911254883, 0.004358530044555664, 0.004326820373535156, 0.004310131072998047, 0.004564046859741211, 0.004283428192138672, 0.004290342330932617, 0.00434422492980957, 0.004352569580078125, 0.004256486892700195, 0.004298686981201172, 0.00437474250793457, 0.0046536922454833984, 0.004326820373535156, 0.004325389862060547, 0.004315853118896484, 0.004328727722167969, 0.004251003265380859, 0.0042912960052490234, 0.0042972564697265625, 0.004452228546142578, 0.004406452178955078, 0.004369020462036133, 0.004300355911254883, 0.0044384002685546875, 0.004404306411743164, 0.004302501678466797, 0.004377603530883789, 0.004363298416137695, 0.00452876091003418, 0.004346370697021484, 0.004317283630371094, 0.004301309585571289, 0.004366636276245117, 0.004438877105712891, 0.004288911819458008, 0.004395008087158203, 0.0043795108795166016, 0.0043947696685791016, 0.004359245300292969, 0.004372119903564453, 0.004342794418334961, 0.004401445388793945, 0.0044209957122802734, 0.004380941390991211, 0.004409074783325195, 0.004342317581176758, 0.004431486129760742, 0.004348278045654297, 0.004305362701416016, 0.0044062137603759766, 0.004367828369140625, 0.004361867904663086, 0.004391908645629883, 0.00438690185546875, 0.004313945770263672, 0.004422187805175781, 0.004380464553833008, 0.004311084747314453, 0.0043582916259765625, 0.004312753677368164, 0.0043408870697021484, 0.004354238510131836, 0.004398822784423828, 0.004315376281738281, 0.004412412643432617, 0.004453420639038086, 0.004318714141845703, 0.004378557205200195, 0.0044138431549072266, 0.004732608795166016, 0.004415988922119141, 0.004387378692626953, 0.0043163299560546875, 0.004504680633544922, 0.004454135894775391, 0.004372358322143555, 0.0043582916259765625, 0.004426002502441406, 0.004330635070800781, 0.004350900650024414, 0.004367828369140625, 0.00439906120300293, 0.00469660758972168, 0.00450587272644043, 0.004401206970214844, 0.004358530044555664, 0.005064249038696289, 0.004331827163696289, 0.004422664642333984, 0.004447221755981445, 0.004324674606323242, 0.004347085952758789, 0.00439763069152832, 0.004444122314453125, 0.004368305206298828, 0.004346370697021484, 0.004317283630371094, 0.004349708557128906, 0.00447845458984375, 0.004398822784423828, 0.004404544830322266, 0.004313468933105469, 0.004491329193115234, 0.0043337345123291016, 0.004336833953857422, 0.0043985843658447266, 0.0043637752532958984, 0.004362583160400391, 0.004408836364746094, 0.004414081573486328, 0.004464387893676758, 0.00442957878112793, 0.0044329166412353516, 0.004301786422729492, 0.0044291019439697266, 0.004382133483886719, 0.004404544830322266, 0.00439906120300293, 0.0043294429779052734, 0.004463911056518555, 0.004436016082763672, 0.004430055618286133, 0.004349231719970703, 0.004502296447753906, 0.004413127899169922, 0.004386186599731445, 0.0043888092041015625, 0.0043985843658447266, 0.004344463348388672, 0.0043985843658447266, 0.0044553279876708984, 0.004304647445678711, 0.004379987716674805, 0.0044896602630615234, 0.004415035247802734, 0.004370212554931641, 0.004362583160400391, 0.0043544769287109375, 0.00437617301940918, 0.00441431999206543, 0.004319190979003906, 0.004374504089355469, 0.004389047622680664, 0.004348039627075195, 0.004430055618286133, 0.004319190979003906, 0.004343271255493164, 0.004361391067504883, 0.004395961761474609, 0.0043828487396240234, 0.004363059997558594, 0.004361629486083984, 0.004433870315551758, 0.004367828369140625, 0.0043294429779052734, 0.004490852355957031, 0.004422664642333984, 0.0043795108795166016, 0.004405498504638672, 0.004353761672973633, 0.004337787628173828, 0.004647731781005859, 0.004374027252197266, 0.004312276840209961, 0.004541873931884766, 0.004383563995361328, 0.004363536834716797, 0.0043370723724365234, 0.0043981075286865234, 0.004314899444580078, 0.004379987716674805, 0.004424095153808594, 0.004351377487182617, 0.004385948181152344, 0.004395484924316406, 0.00440526008605957, 0.004372835159301758, 0.004424095153808594, 0.0043964385986328125, 0.004384756088256836, 0.0044443607330322266, 0.004320383071899414, 0.004415273666381836, 0.004436492919921875, 0.0043849945068359375, 0.0043637752532958984, 0.004324197769165039, 0.00439763069152832, 0.004393577575683594, 0.0043811798095703125, 0.004479885101318359, 0.00450897216796875, 0.004364013671875, 0.004477500915527344, 0.004359006881713867, 0.004338502883911133, 0.004442453384399414, 0.004335641860961914, 0.0043487548828125, 0.004431486129760742, 0.0043790340423583984, 0.004420042037963867, 0.004438638687133789, 0.004410982131958008, 0.00435328483581543, 0.004435539245605469, 0.004378080368041992, 0.0044803619384765625, 0.00475001335144043, 0.004427433013916016, 0.004538059234619141, 0.004456996917724609, 0.004391193389892578, 0.0044269561767578125, 0.004434108734130859, 0.004353761672973633, 0.0043866634368896484, 0.004370927810668945, 0.004346132278442383, 0.004388093948364258, 0.004430055618286133, 0.004412651062011719, 0.004362344741821289, 0.004413127899169922, 0.0044171810150146484, 0.004429340362548828, 0.0044078826904296875, 0.0044515132904052734, 0.004357814788818359, 0.004389762878417969, 0.004512310028076172, 0.0043828487396240234, 0.004397153854370117, 0.004379987716674805, 0.00438690185546875, 0.0043680667877197266, 0.004495143890380859, 0.004343986511230469, 0.004388570785522461, 0.004450321197509766, 0.0043561458587646484, 0.0044002532958984375, 0.004430294036865234, 0.004415035247802734, 0.004366636276245117, 0.004384040832519531, 0.004457712173461914, 0.004427671432495117, 0.004422426223754883, 0.004462003707885742, 0.0045206546783447266, 0.004456758499145508, 0.004408121109008789, 0.004377126693725586, 0.004393815994262695, 0.00450444221496582, 0.004405498504638672, 0.005090951919555664, 0.0044727325439453125, 0.0043697357177734375, 0.004429340362548828, 0.005021333694458008, 0.004434347152709961, 0.004364967346191406, 0.004421234130859375, 0.004412412643432617, 0.0043752193450927734, 0.004322052001953125, 0.004433870315551758, 0.00447845458984375, 0.00438380241394043, 0.004375457763671875, 0.0044651031494140625, 0.004388332366943359, 0.00439453125, 0.004397153854370117, 0.004395008087158203, 0.004416465759277344, 0.0044994354248046875, 0.0043315887451171875, 0.004399776458740234, 0.004439592361450195, 0.004480838775634766, 0.004404306411743164, 0.004426002502441406, 0.004346609115600586, 0.004391670227050781, 0.004443168640136719, 0.004372596740722656, 0.0044062137603759766, 0.0044345855712890625, 0.004525184631347656, 0.004420757293701172, 0.0044345855712890625, 0.00433659553527832, 0.004403829574584961, 0.0044629573822021484, 0.004376411437988281, 0.004381656646728516, 0.004434823989868164, 0.004984855651855469, 0.00454401969909668, 0.004493236541748047, 0.004469394683837891, 0.004421234130859375, 0.0044422149658203125, 0.004431962966918945, 0.0044438838958740234, 0.004434347152709961, 0.004695892333984375, 0.004465341567993164, 0.004481077194213867, 0.004555463790893555, 0.00445103645324707, 0.004433393478393555, 0.00441288948059082, 0.0044214725494384766, 0.004415035247802734, 0.00435638427734375, 0.004385709762573242, 0.004538774490356445, 0.004393815994262695, 0.00443720817565918, 0.004435539245605469, 0.004422664642333984, 0.004414081573486328, 0.004464387893676758, 0.0044252872467041016, 0.004429340362548828, 0.004490852355957031, 0.0043904781341552734, 0.0044362545013427734, 0.004488229751586914, 0.00438237190246582, 0.0044193267822265625, 0.004427433013916016, 0.004505157470703125, 0.0044362545013427734, 0.0045583248138427734, 0.004429340362548828, 0.004435062408447266, 0.0044443607330322266, 0.004458427429199219, 0.004363298416137695, 0.004388570785522461, 0.00446009635925293, 0.00447845458984375, 0.004517078399658203, 0.004323244094848633, 0.004445075988769531, 0.004391908645629883, 0.004419565200805664, 0.0044291019439697266, 0.004426240921020508, 0.004348278045654297, 0.004416465759277344, 0.0044367313385009766, 0.004351615905761719, 0.004408597946166992, 0.004450082778930664, 0.0043909549713134766, 0.004431486129760742, 0.004422664642333984, 0.004408121109008789, 0.004426717758178711, 0.004495859146118164, 0.004357814788818359, 0.004414796829223633, 0.004401683807373047, 0.00438380241394043, 0.004412412643432617, 0.004525661468505859, 0.0043792724609375, 0.004427671432495117, 0.0044820308685302734, 0.00446009635925293, 0.004400491714477539, 0.0044558048248291016, 0.004397869110107422, 0.004423379898071289, 0.00440526008605957, 0.004509925842285156, 0.004439592361450195, 0.004447460174560547, 0.0043828487396240234, 0.0044422149658203125, 0.004578351974487305, 0.004512786865234375, 0.004423618316650391, 0.004449605941772461, 0.004494428634643555, 0.0044476985931396484, 0.004459381103515625, 0.004342555999755859, 0.0044040679931640625, 0.0044329166412353516, 0.0044345855712890625, 0.004414558410644531, 0.004403352737426758, 0.004431009292602539, 0.00442814826965332, 0.0044057369232177734, 0.004401445388793945, 0.004602909088134766, 0.004473686218261719, 0.00441288948059082, 0.0044100284576416016, 0.0044193267822265625, 0.0044209957122802734, 0.0044403076171875, 0.004465341567993164, 0.004418134689331055, 0.004461526870727539, 0.00441431999206543, 0.004418849945068359, 0.004401445388793945, 0.004381656646728516, 0.004440784454345703, 0.004452228546142578, 0.0044400691986083984, 0.0044252872467041016, 0.004409313201904297, 0.004377841949462891, 0.004488706588745117, 0.004428386688232422, 0.004369020462036133, 0.004498720169067383, 0.004425764083862305, 0.004422187805175781, 0.004454851150512695, 0.004506111145019531, 0.00445866584777832, 0.004528045654296875, 0.004456520080566406, 0.00438690185546875, 0.004467964172363281, 0.00461268424987793, 0.004584550857543945, 0.0043833255767822266, 0.0044558048248291016, 0.004465818405151367, 0.004452705383300781, 0.004467487335205078, 0.004476785659790039, 0.004405498504638672, 0.004494190216064453, 0.0045049190521240234, 0.00444340705871582, 0.004476070404052734, 0.004488706588745117, 0.0044744014739990234, 0.0044519901275634766, 0.004463911056518555, 0.0044705867767333984, 0.004497528076171875, 0.004503488540649414, 0.0043735504150390625, 0.004495382308959961, 0.004452228546142578, 0.004481315612792969, 0.004474639892578125, 0.0044710636138916016, 0.004441738128662109, 0.004708051681518555, 0.004586935043334961, 0.004445791244506836, 0.004529237747192383, 0.004503011703491211, 0.00448155403137207, 0.004541158676147461, 0.004465341567993164, 0.004545450210571289, 0.004592418670654297, 0.0044710636138916016, 0.0044498443603515625, 0.0045316219329833984, 0.004489421844482422, 0.004491329193115234, 0.004540205001831055, 0.0044803619384765625, 0.004512786865234375, 0.004524707794189453, 0.004587650299072266, 0.00448298454284668, 0.004487276077270508, 0.004480600357055664, 0.004563808441162109, 0.004479885101318359, 0.004490375518798828, 0.004530429840087891, 0.004501819610595703, 0.004517555236816406, 0.004477977752685547, 0.004401445388793945, 0.0044460296630859375, 0.004574775695800781, 0.004534244537353516, 0.004482746124267578, 0.004487037658691406, 0.004498004913330078, 0.004458189010620117, 0.004532814025878906, 0.004411458969116211, 0.004439353942871094, 0.004527568817138672, 0.004445552825927734, 0.004507541656494141, 0.004446268081665039, 0.004456996917724609, 0.0044460296630859375, 0.0044515132904052734, 0.004430294036865234, 0.004520416259765625, 0.004478931427001953, 0.004431486129760742, 0.00463414192199707, 0.004503488540649414, 0.0045163631439208984, 0.004575014114379883, 0.0045011043548583984, 0.0044744014739990234, 0.004521608352661133, 0.004471778869628906, 0.004438877105712891, 0.0045588016510009766, 0.004679679870605469, 0.004499673843383789, 0.004492044448852539, 0.0044977664947509766, 0.004507780075073242, 0.0044689178466796875, 0.004446983337402344, 0.004510164260864258, 0.0046956539154052734, 0.004457235336303711, 0.004550457000732422, 0.004492759704589844, 0.004487752914428711, 0.0045092105865478516, 0.004510641098022461, 0.004512310028076172, 0.0045130252838134766, 0.004462718963623047, 0.004508495330810547, 0.0045850276947021484, 0.004446744918823242, 0.004506111145019531, 0.004481077194213867, 0.004481077194213867, 0.004448413848876953, 0.00446629524230957, 0.004445314407348633, 0.004564762115478516, 0.004512310028076172, 0.0044689178466796875, 0.0044629573822021484, 0.004468679428100586, 0.0044519901275634766, 0.004524707794189453, 0.0044629573822021484, 0.0046138763427734375, 0.004511117935180664, 0.004477500915527344, 0.004432201385498047, 0.004541158676147461, 0.004514932632446289, 0.00461268424987793, 0.004431962966918945, 0.00449371337890625, 0.004654407501220703, 0.004476070404052734, 0.00454401969909668, 0.00446629524230957, 0.004437923431396484, 0.004477739334106445, 0.00456690788269043, 0.004450798034667969, 0.004510402679443359, 0.004537343978881836, 0.004460573196411133, 0.004479885101318359, 0.0044782161712646484, 0.004399299621582031, 0.004501819610595703, 0.0045812129974365234, 0.004452705383300781, 0.0045206546783447266, 0.004522800445556641, 0.004500150680541992, 0.004499673843383789, 0.004525661468505859, 0.004479646682739258, 0.0046083927154541016, 0.0044612884521484375, 0.004641532897949219, 0.004506111145019531, 0.004499912261962891, 0.0045244693756103516, 0.00453495979309082, 0.004508495330810547, 0.004460573196411133, 0.004555940628051758, 0.004501819610595703, 0.004486083984375, 0.004538536071777344, 0.004515647888183594, 0.0045702457427978516, 0.004458189010620117, 0.004473209381103516, 0.00463414192199707, 0.004534244537353516, 0.004424333572387695, 0.00456547737121582, 0.004511356353759766, 0.0047130584716796875, 0.00451207160949707, 0.004511356353759766, 0.004550457000732422, 0.004423379898071289, 0.0044825077056884766, 0.0045528411865234375, 0.004467010498046875, 0.004491090774536133, 0.004540920257568359, 0.004475831985473633, 0.004525899887084961, 0.004541635513305664, 0.0044708251953125, 0.004488706588745117, 0.004517793655395508, 0.004504680633544922, 0.004517078399658203, 0.004526376724243164, 0.004468441009521484, 0.0045626163482666016, 0.004477739334106445, 0.004477024078369141, 0.004520893096923828, 0.004522800445556641, 0.004517078399658203, 0.004530191421508789, 0.004471778869628906, 0.004487514495849609, 0.004565000534057617, 0.00449371337890625, 0.0044972896575927734, 0.004542350769042969, 0.00454401969909668, 0.004662752151489258, 0.004518270492553711, 0.0045125484466552734, 0.004594564437866211, 0.00449061393737793, 0.00447392463684082, 0.00451350212097168, 0.004511594772338867, 0.004456996917724609, 0.004563808441162109, 0.004503488540649414, 0.004492759704589844, 0.004547119140625, 0.004514217376708984, 0.004486799240112305, 0.004502296447753906, 0.004457235336303711, 0.004605770111083984, 0.004472494125366211, 0.00449371337890625, 0.004498720169067383, 0.004540920257568359, 0.0044956207275390625, 0.004478931427001953, 0.004509925842285156, 0.004508018493652344, 0.004522085189819336, 0.00450897216796875, 0.0045011043548583984, 0.004469156265258789, 0.0044558048248291016, 0.0045053958892822266, 0.004563331604003906, 0.0045359134674072266, 0.004549741744995117, 0.004502534866333008, 0.0045163631439208984, 0.004492998123168945, 0.004492521286010742, 0.004463911056518555, 0.004538297653198242, 0.004534006118774414, 0.004484415054321289, 0.004548072814941406, 0.00454401969909668, 0.004568815231323242, 0.004515886306762695, 0.004512786865234375, 0.0045146942138671875, 0.0044972896575927734, 0.0045070648193359375, 0.004506587982177734, 0.004408836364746094, 0.004494428634643555, 0.004697084426879883, 0.004557132720947266, 0.004547595977783203, 0.004525423049926758, 0.004484891891479492, 0.004546403884887695, 0.004489421844482422, 0.004487514495849609, 0.004578113555908203, 0.004513978958129883, 0.004473447799682617, 0.004492282867431641, 0.004488706588745117, 0.004473209381103516, 0.004553079605102539, 0.004525184631347656, 0.004473686218261719, 0.004553556442260742, 0.00447535514831543, 0.004485607147216797, 0.004513978958129883, 0.004498958587646484, 0.0045206546783447266, 0.0045354366302490234, 0.004457712173461914, 0.0044133663177490234, 0.00452733039855957, 0.004530429840087891, 0.004519939422607422, 0.0044422149658203125, 0.004514217376708984, 0.004554271697998047, 0.004445314407348633, 0.00449824333190918, 0.004564523696899414, 0.004616260528564453, 0.0045626163482666016, 0.004533529281616211, 0.004470348358154297, 0.0045201778411865234, 0.004546165466308594, 0.0044727325439453125, 0.004541873931884766, 0.004484415054321289, 0.00447845458984375, 0.0045680999755859375, 0.0044918060302734375, 0.004466056823730469, 0.004584550857543945, 0.0044939517974853516, 0.004457950592041016, 0.004535675048828125, 0.004514217376708984, 0.0049419403076171875, 0.004509687423706055, 0.004465818405151367, 0.004621267318725586, 0.0045435428619384766, 0.004533290863037109, 0.004553556442260742, 0.0045239925384521484, 0.004441738128662109, 0.004524707794189453, 0.004503726959228516, 0.004472970962524414, 0.004576921463012695, 0.0044994354248046875, 0.004569292068481445, 0.004477500915527344, 0.004476785659790039, 0.004575252532958984, 0.004464864730834961, 0.004467487335205078, 0.004585981369018555, 0.0044786930084228516, 0.004473209381103516, 0.004582881927490234, 0.004477977752685547, 0.004509687423706055, 0.004655122756958008, 0.004544734954833984, 0.0045261383056640625, 0.004468441009521484, 0.004486799240112305, 0.004542827606201172, 0.004532575607299805, 0.004483461380004883, 0.004555463790893555, 0.004488945007324219, 0.004578590393066406, 0.0045318603515625, 0.004479408264160156, 0.004472970962524414, 0.0045511722564697266, 0.004565238952636719, 0.004537820816040039, 0.004529714584350586, 0.0045185089111328125, 0.004538536071777344, 0.004476308822631836, 0.0045201778411865234, 0.004502534866333008, 0.004525423049926758, 0.004498720169067383, 0.004563331604003906, 0.004528045654296875, 0.004500150680541992, 0.004545450210571289, 0.004469871520996094, 0.004494428634643555, 0.004557609558105469, 0.004463672637939453, 0.004537105560302734, 0.0045130252838134766, 0.004559040069580078, 0.004556179046630859, 0.004506826400756836, 0.0045261383056640625, 0.004576683044433594, 0.004499197006225586, 0.004445791244506836, 0.00453639030456543, 0.0044934749603271484, 0.004568815231323242, 0.0044994354248046875, 0.004534006118774414, 0.004518985748291016, 0.0044863224029541016, 0.004474639892578125, 0.004653215408325195, 0.004582643508911133, 0.0045053958892822266, 0.004548788070678711, 0.004582643508911133, 0.004513263702392578, 0.004583120346069336, 0.004515171051025391, 0.004567146301269531, 0.004619598388671875, 0.0045320987701416016, 0.004665851593017578, 0.0045928955078125, 0.0045528411865234375, 0.004632472991943359, 0.004599571228027344, 0.0048220157623291016, 0.004534244537353516, 0.004595518112182617, 0.004596710205078125, 0.004599332809448242, 0.0046215057373046875, 0.00457453727722168, 0.004513263702392578, 0.004651308059692383, 0.004565715789794922, 0.004547834396362305, 0.00455927848815918, 0.0045244693756103516, 0.004620790481567383, 0.0045833587646484375, 0.0045735836029052734, 0.0046117305755615234, 0.0045909881591796875, 0.004544496536254883, 0.004633903503417969, 0.004539012908935547, 0.004624843597412109, 0.0046939849853515625, 0.004603147506713867, 0.0046002864837646484, 0.004650115966796875, 0.004629373550415039, 0.004605770111083984, 0.004679679870605469, 0.00469517707824707, 0.0046117305755615234, 0.0046329498291015625, 0.004786014556884766, 0.00452423095703125, 0.004621982574462891, 0.004561185836791992, 0.004528522491455078, 0.004608869552612305, 0.004608869552612305, 0.004596233367919922, 0.00466465950012207, 0.004549264907836914, 0.004586696624755859, 0.0046498775482177734, 0.004625558853149414, 0.004592180252075195, 0.004499912261962891, 0.00467991828918457, 0.004557132720947266, 0.004582643508911133, 0.004631757736206055, 0.004587888717651367, 0.004576206207275391, 0.004586696624755859, 0.004599571228027344, 0.0046007633209228516, 0.004575490951538086, 0.004595518112182617, 0.004578590393066406, 0.004606723785400391, 0.004590272903442383, 0.004615068435668945, 0.004549264907836914, 0.004698753356933594, 0.0046231746673583984, 0.0046596527099609375, 0.004614591598510742, 0.004587888717651367, 0.004583835601806641, 0.004569053649902344, 0.004557609558105469, 0.0045697689056396484, 0.004618644714355469, 0.0046234130859375, 0.004644155502319336, 0.0045320987701416016, 0.00477910041809082, 0.004571199417114258, 0.00465846061706543, 0.004584312438964844, 0.004627704620361328, 0.004572629928588867, 0.004638195037841797, 0.004549980163574219, 0.004648685455322266, 0.004570484161376953, 0.00452876091003418, 0.004666805267333984, 0.004522800445556641, 0.004590511322021484, 0.004642009735107422, 0.004644155502319336, 0.004568815231323242, 0.004549503326416016, 0.004554271697998047, 0.0046727657318115234, 0.0046002864837646484, 0.00462794303894043, 0.004580497741699219, 0.0045816898345947266, 0.0046541690826416016, 0.004563093185424805, 0.004624843597412109, 0.004617929458618164, 0.004645109176635742, 0.00466609001159668, 0.004579782485961914, 0.004563808441162109, 0.004672527313232422, 0.0046656131744384766, 0.004666566848754883, 0.004555702209472656, 0.004636526107788086, 0.004658222198486328, 0.0046062469482421875, 0.004609107971191406, 0.004584074020385742, 0.004552125930786133, 0.004660129547119141, 0.0046923160552978516, 0.004637241363525391, 0.004671335220336914, 0.004517793655395508, 0.0046617984771728516, 0.004536628723144531, 0.004498481750488281, 0.004617929458618164, 0.004567146301269531, 0.004562854766845703, 0.004606723785400391, 0.004563331604003906, 0.00461125373840332, 0.004608631134033203, 0.004575967788696289, 0.0046308040618896484, 0.004568576812744141, 0.0046939849853515625, 0.004611015319824219, 0.004622697830200195, 0.004642963409423828, 0.004578351974487305, 0.004655361175537109, 0.004670143127441406, 0.00459742546081543, 0.00472569465637207, 0.004599094390869141, 0.0046460628509521484, 0.0046694278717041016, 0.004595756530761719, 0.004660844802856445, 0.0046160221099853516, 0.004554271697998047, 0.004748106002807617, 0.0045659542083740234, 0.004576683044433594, 0.004586458206176758, 0.0045621395111083984, 0.0046520233154296875, 0.004599094390869141, 0.004624366760253906, 0.004662990570068359, 0.004620552062988281, 0.004796266555786133, 0.004626274108886719, 0.0046176910400390625, 0.004658699035644531, 0.004585981369018555, 0.004713296890258789, 0.00460505485534668, 0.004576444625854492, 0.0045926570892333984, 0.004605531692504883, 0.0046460628509521484, 0.004565238952636719, 0.004572868347167969, 0.00457453727722168, 0.004658937454223633, 0.004579305648803711, 0.004678487777709961, 0.004517316818237305, 0.004664897918701172, 0.004602193832397461, 0.0045392513275146484, 0.004632711410522461, 0.0045740604400634766, 0.004587411880493164, 0.004656076431274414, 0.004584789276123047, 0.004601240158081055, 0.004609823226928711, 0.0045623779296875, 0.004599809646606445, 0.004564523696899414, 0.0046041011810302734, 0.0047113895416259766, 0.004569053649902344, 0.004743814468383789, 0.00458526611328125, 0.0046539306640625, 0.004645347595214844, 0.004575014114379883, 0.004739522933959961, 0.0045928955078125, 0.004585981369018555, 0.0046901702880859375, 0.004580259323120117, 0.004842281341552734, 0.004651069641113281, 0.004593372344970703, 0.004579067230224609, 0.004563570022583008, 0.00464177131652832, 0.004625082015991211, 0.004559516906738281, 0.004686594009399414, 0.004568576812744141, 0.004691123962402344, 0.005611419677734375, 0.004765033721923828, 0.00458836555480957, 0.004615306854248047, 0.004656314849853516, 0.004590272903442383, 0.004641532897949219, 0.004644870758056641, 0.00459742546081543, 0.004674434661865234, 0.004627227783203125, 0.004564046859741211, 0.0046770572662353516, 0.004568815231323242, 0.0046041011810302734, 0.004616260528564453, 0.004558086395263672, 0.004638671875, 0.0045850276947021484, 0.004614591598510742, 0.004609107971191406, 0.004616975784301758, 0.004710674285888672, 0.004717588424682617, 0.00460362434387207, 0.0046689510345458984, 0.004632234573364258, 0.004574775695800781, 0.004568815231323242, 0.004614353179931641, 0.0046825408935546875, 0.004560708999633789, 0.004822731018066406, 0.0045833587646484375, 0.0046787261962890625, 0.004610300064086914, 0.004601001739501953, 0.004645586013793945, 0.00462794303894043, 0.004628419876098633, 0.004561424255371094, 0.004564046859741211, 0.004612922668457031, 0.0046694278717041016, 0.0045375823974609375, 0.004677534103393555, 0.004574775695800781, 0.004632711410522461, 0.004749298095703125, 0.00455927848815918, 0.004599332809448242, 0.004587650299072266, 0.004678487777709961, 0.004668474197387695, 0.0046236515045166016, 0.004644632339477539, 0.004664182662963867, 0.004615306854248047, 0.004672050476074219, 0.004603385925292969, 0.004625082015991211, 0.004642009735107422, 0.004593849182128906, 0.004660367965698242, 0.0046765804290771484, 0.004709005355834961, 0.004611015319824219, 0.004625558853149414, 0.00469660758972168, 0.00462031364440918, 0.0046100616455078125, 0.004588127136230469, 0.004586219787597656, 0.004775524139404297, 0.004586696624755859, 0.0046465396881103516, 0.00464940071105957, 0.004586696624755859, 0.004715919494628906, 0.0045986175537109375, 0.004670381546020508, 0.0046193599700927734, 0.004677772521972656, 0.004632234573364258, 0.004609107971191406, 0.004630088806152344, 0.004709720611572266, 0.004692792892456055, 0.004549264907836914, 0.004618406295776367, 0.004648447036743164, 0.00466156005859375, 0.004567861557006836, 0.00462794303894043, 0.004627227783203125, 0.004590749740600586, 0.004677772521972656, 0.004637002944946289, 0.004633426666259766, 0.004710674285888672, 0.0045816898345947266, 0.004624366760253906, 0.004582405090332031, 0.004618644714355469, 0.004713535308837891, 0.0045719146728515625, 0.0046482086181640625, 0.004724264144897461, 0.0047223567962646484, 0.004645109176635742, 0.004575014114379883, 0.004574775695800781, 0.00525355339050293, 0.0047435760498046875, 0.004705667495727539, 0.004605770111083984, 0.005017995834350586, 0.00463414192199707, 0.0047550201416015625, 0.004665851593017578, 0.004637718200683594, 0.0046656131744384766, 0.004611492156982422, 0.004665851593017578, 0.004695415496826172, 0.004580020904541016, 0.004624128341674805, 0.0047147274017333984, 0.005280256271362305, 0.004689693450927734, 0.00500035285949707, 0.004674673080444336, 0.0046215057373046875, 0.0047054290771484375, 0.004591464996337891, 0.004641532897949219, 0.004717588424682617, 0.004633426666259766, 0.0046770572662353516, 0.004584789276123047, 0.004663228988647461, 0.004611492156982422, 0.004586219787597656, 0.0046083927154541016, 0.004681587219238281, 0.004624128341674805, 0.004655122756958008, 0.004638671875, 0.004611015319824219, 0.004680633544921875, 0.004697561264038086, 0.004686594009399414, 0.00455021858215332, 0.0047075748443603516, 0.004637718200683594, 0.004595279693603516, 0.0046617984771728516, 0.004697084426879883, 0.004716157913208008, 0.0046329498291015625, 0.004639625549316406, 0.004720211029052734, 0.004605293273925781, 0.0046465396881103516, 0.004621267318725586, 0.004605531692504883, 0.004723787307739258, 0.004656076431274414, 0.004659175872802734, 0.004616975784301758, 0.004591941833496094, 0.0047113895416259766, 0.004626035690307617, 0.004637956619262695, 0.00462651252746582, 0.004606723785400391, 0.004658699035644531, 0.004637718200683594, 0.004567623138427734, 0.004700183868408203, 0.004597902297973633, 0.004641294479370117, 0.004614830017089844, 0.0046727657318115234, 0.004775285720825195, 0.004733562469482422, 0.004687309265136719, 0.00466465950012207, 0.004774808883666992, 0.004696846008300781, 0.004660606384277344, 0.004719257354736328, 0.004727840423583984, 0.004754066467285156, 0.004726409912109375, 0.004616498947143555, 0.004768848419189453, 0.004712581634521484, 0.0046694278717041016, 0.0048847198486328125, 0.004714488983154297, 0.004723787307739258, 0.004684925079345703, 0.004709005355834961, 0.004804849624633789, 0.004837512969970703, 0.00466609001159668, 0.004666805267333984, 0.0047113895416259766, 0.004716634750366211, 0.004728794097900391, 0.004679679870605469, 0.004732370376586914, 0.004729747772216797, 0.004937171936035156, 0.00470733642578125, 0.004674196243286133, 0.004805803298950195, 0.004732370376586914, 0.004779815673828125, 0.0047223567962646484, 0.0047686100006103516, 0.004782199859619141, 0.0046977996826171875, 0.0046727657318115234, 0.0046825408935546875, 0.0047283172607421875, 0.0046999454498291016, 0.004673480987548828, 0.004737138748168945, 0.004666805267333984, 0.004717588424682617, 0.004760026931762695, 0.004660844802856445, 0.004704713821411133, 0.00466156005859375, 0.0048274993896484375, 0.004700183868408203, 0.004755735397338867, 0.004691123962402344, 0.00462794303894043, 0.004778146743774414, 0.004712581634521484, 0.0046575069427490234, 0.00471186637878418, 0.004704952239990234, 0.004772186279296875, 0.004763603210449219, 0.0047454833984375, 0.0046536922454833984, 0.0046656131744384766, 0.00472569465637207, 0.0046520233154296875, 0.0046918392181396484, 0.004650115966796875, 0.004718303680419922, 0.004806041717529297, 0.0046384334564208984, 0.004708766937255859, 0.004675626754760742, 0.004698514938354492, 0.004694461822509766, 0.004718780517578125, 0.004731655120849609, 0.004687070846557617, 0.004701852798461914, 0.004751920700073242, 0.004705905914306641, 0.004777431488037109, 0.004709959030151367, 0.00469970703125, 0.004618406295776367, 0.00467371940612793, 0.004778146743774414, 0.004699230194091797, 0.004698276519775391, 0.004698514938354492, 0.00467228889465332, 0.004826784133911133, 0.004718780517578125, 0.004699230194091797, 0.004705190658569336, 0.00476837158203125, 0.0047130584716796875, 0.004651069641113281, 0.004754781723022461, 0.004675149917602539, 0.004791975021362305, 0.004691123962402344, 0.00534820556640625, 0.004708528518676758, 0.004682302474975586, 0.005226612091064453, 0.004735231399536133, 0.004817008972167969, 0.004641294479370117, 0.004692554473876953, 0.00466465950012207, 0.004745960235595703, 0.004654407501220703, 0.00469660758972168, 0.0047702789306640625, 0.004709482192993164, 0.0048296451568603516, 0.004754304885864258, 0.00471949577331543, 0.004850864410400391, 0.0047397613525390625, 0.004752397537231445, 0.0046803951263427734, 0.004782438278198242, 0.004769086837768555, 0.004735231399536133, 0.004724264144897461, 0.004712104797363281, 0.004763364791870117, 0.0046694278717041016, 0.004731178283691406, 0.00467371940612793, 0.004741668701171875, 0.00479435920715332, 0.004706144332885742, 0.004790306091308594, 0.004698276519775391, 0.0047245025634765625, 0.004827737808227539, 0.004736185073852539, 0.004750728607177734, 0.004788637161254883, 0.005391597747802734, 0.004756450653076172, 0.004868268966674805, 0.005152702331542969, 0.004962921142578125, 0.004917144775390625, 0.004820346832275391, 0.004705905914306641, 0.004797458648681641, 0.004674196243286133, 0.004736661911010742, 0.004790544509887695, 0.004656076431274414, 0.004702329635620117, 0.004695415496826172, 0.004696846008300781, 0.0047206878662109375, 0.004683017730712891, 0.0047760009765625, 0.004777431488037109, 0.004773139953613281, 0.00476384162902832, 0.004686832427978516, 0.004709959030151367, 0.004735231399536133, 0.00474858283996582, 0.004722118377685547, 0.004776716232299805, 0.004781007766723633, 0.004662513732910156, 0.00470733642578125, 0.004696846008300781, 0.004789113998413086, 0.004803657531738281, 0.004728078842163086, 0.004738330841064453, 0.00466156005859375, 0.004694700241088867, 0.0047435760498046875, 0.004767894744873047, 0.004742860794067383, 0.004748344421386719, 0.004737377166748047, 0.0046727657318115234, 0.0049283504486083984, 0.0047109127044677734, 0.004660129547119141, 0.00471186637878418, 0.004720926284790039, 0.004772186279296875, 0.004698038101196289, 0.004778861999511719, 0.004770994186401367, 0.004734992980957031, 0.004778385162353516, 0.004827737808227539, 0.004790067672729492, 0.0046672821044921875, 0.004817485809326172, 0.004738330841064453, 0.004744052886962891, 0.004697084426879883, 0.0047342777252197266, 0.0047223567962646484, 0.004649639129638672, 0.004769802093505859, 0.004892826080322266, 0.004729032516479492, 0.004750967025756836, 0.0046999454498291016, 0.0047605037689208984, 0.004739046096801758, 0.004800558090209961, 0.004754304885864258, 0.004647016525268555, 0.0048389434814453125, 0.004788637161254883, 0.004809856414794922, 0.004697084426879883, 0.00479435920715332, 0.004895448684692383, 0.004703044891357422, 0.0047800540924072266, 0.004710197448730469, 0.004879474639892578, 0.004743099212646484, 0.004912853240966797, 0.004739046096801758, 0.004744529724121094, 0.0047473907470703125, 0.004724979400634766, 0.004797697067260742, 0.004697084426879883, 0.004809379577636719, 0.004782676696777344, 0.004708766937255859, 0.004828214645385742, 0.004718780517578125, 0.0048067569732666016, 0.004681587219238281, 0.004860639572143555, 0.004775285720825195, 0.004723787307739258, 0.004778623580932617, 0.0047228336334228516, 0.004820823669433594, 0.004704713821411133, 0.004750967025756836, 0.004764556884765625, 0.004702568054199219, 0.004739284515380859, 0.004752635955810547, 0.004706382751464844, 0.00467681884765625, 0.004790067672729492, 0.004779338836669922, 0.004681587219238281, 0.004874706268310547, 0.004708290100097656, 0.004754543304443359, 0.004709959030151367, 0.0048847198486328125, 0.0047681331634521484, 0.0046634674072265625, 0.004723072052001953, 0.0048313140869140625, 0.005226612091064453, 0.004936933517456055, 0.004999399185180664, 0.004679441452026367, 0.004876613616943359, 0.004812002182006836, 0.0048601627349853516, 0.004716634750366211, 0.004771709442138672, 0.004777669906616211, 0.00471043586730957, 0.00473332405090332, 0.004744291305541992, 0.00491642951965332, 0.00482630729675293, 0.004752397537231445, 0.0047893524169921875, 0.004740476608276367, 0.004766941070556641, 0.004759073257446289, 0.004744529724121094, 0.004797935485839844, 0.0048067569732666016, 0.004821062088012695, 0.004744052886962891, 0.004748344421386719, 0.004733085632324219, 0.0047571659088134766, 0.004716157913208008, 0.004709482192993164, 0.004751682281494141, 0.004725217819213867, 0.004728555679321289, 0.004820585250854492, 0.0047762393951416016, 0.004717826843261719, 0.004733562469482422, 0.0047833919525146484, 0.00473475456237793, 0.0047533512115478516, 0.004767656326293945, 0.00477147102355957, 0.004727363586425781, 0.004854917526245117, 0.0048940181732177734, 0.004742622375488281, 0.004832267761230469, 0.004705667495727539, 0.004783153533935547, 0.004710197448730469, 0.0047419071197509766, 0.004836320877075195, 0.00471186637878418, 0.004777669906616211, 0.004774808883666992, 0.004808902740478516, 0.0047070980072021484, 0.0047168731689453125, 0.004734039306640625, 0.0047190189361572266, 0.004697561264038086, 0.004761695861816406, 0.0047876834869384766, 0.0047817230224609375, 0.004732608795166016, 0.004751443862915039, 0.004715919494628906, 0.00478672981262207, 0.0047607421875, 0.004812479019165039, 0.004714012145996094, 0.004746675491333008, 0.004804372787475586, 0.004680156707763672, 0.004779338836669922, 0.004814863204956055, 0.00478672981262207, 0.004671335220336914, 0.004806995391845703, 0.004896402359008789, 0.004876375198364258, 0.004891633987426758, 0.004731416702270508, 0.004884004592895508, 0.0048122406005859375, 0.004746198654174805, 0.004865884780883789, 0.004724264144897461, 0.004808187484741211, 0.004749774932861328, 0.004828691482543945, 0.004745960235595703, 0.0047760009765625, 0.0047283172607421875, 0.004739999771118164, 0.004837751388549805, 0.004652261734008789, 0.005031585693359375, 0.004696846008300781, 0.0048139095306396484, 0.0047109127044677734, 0.004773855209350586, 0.00474858283996582, 0.00479578971862793, 0.004719972610473633, 0.0047435760498046875, 0.0047228336334228516, 0.004749298095703125, 0.004739522933959961, 0.004755973815917969, 0.0046672821044921875, 0.0048410892486572266, 0.004731416702270508, 0.004785776138305664, 0.004701137542724609, 0.004732847213745117, 0.004780769348144531, 0.004790067672729492, 0.00478363037109375, 0.004779815673828125, 0.004752159118652344, 0.004716396331787109, 0.00475764274597168, 0.004808187484741211, 0.004743814468383789, 0.0047414302825927734, 0.0047607421875, 0.004834413528442383, 0.004900217056274414, 0.004731416702270508, 0.004796743392944336, 0.0047607421875, 0.004821300506591797, 0.004723310470581055, 0.00480961799621582, 0.004674673080444336, 0.00473332405090332, 0.004777193069458008, 0.0046536922454833984, 0.004784822463989258, 0.004846334457397461, 0.004866838455200195, 0.004687309265136719, 0.004752397537231445, 0.004773616790771484, 0.004783153533935547, 0.004790782928466797, 0.004785299301147461, 0.004824399948120117, 0.004799604415893555, 0.004801034927368164, 0.004752159118652344, 0.0047724246978759766, 0.00481867790222168, 0.004846096038818359, 0.0048198699951171875, 0.0047490596771240234, 0.004794120788574219, 0.004794120788574219, 0.00489354133605957, 0.00476527214050293, 0.004736661911010742, 0.004870176315307617, 0.0047435760498046875, 0.004946231842041016, 0.004841804504394531, 0.004899263381958008, 0.0046918392181396484, 0.004799365997314453, 0.0048143863677978516, 0.004885673522949219, 0.004753589630126953, 0.004799842834472656, 0.0047588348388671875, 0.004730224609375, 0.004819631576538086, 0.004776477813720703, 0.004768848419189453, 0.0048253536224365234, 0.005025148391723633, 0.0048143863677978516, 0.005015134811401367, 0.004887580871582031, 0.004777669906616211, 0.004791259765625, 0.0048449039459228516, 0.004876613616943359, 0.004755973815917969, 0.004736900329589844, 0.004811763763427734, 0.0048558712005615234, 0.004809141159057617, 0.00470280647277832, 0.0048351287841796875, 0.004739284515380859, 0.004831075668334961, 0.004767417907714844, 0.004820823669433594, 0.004822254180908203, 0.004801273345947266, 0.005001068115234375, 0.004871845245361328, 0.004801511764526367, 0.005043983459472656, 0.0048601627349853516, 0.004852771759033203, 0.004835605621337891, 0.004831552505493164, 0.004858255386352539, 0.00484013557434082, 0.004793405532836914, 0.004965782165527344, 0.004803657531738281, 0.0048828125, 0.004720211029052734, 0.004842519760131836, 0.004819154739379883, 0.004819631576538086, 0.0047762393951416016, 0.004818916320800781, 0.004903316497802734, 0.004733085632324219, 0.004785060882568359, 0.004744052886962891, 0.00483393669128418, 0.004743099212646484, 0.0048558712005615234, 0.004815578460693359, 0.005391359329223633, 0.004768848419189453, 0.004864692687988281, 0.004827976226806641, 0.004779815673828125, 0.004864215850830078, 0.004793405532836914, 0.004861116409301758, 0.00486445426940918, 0.0048635005950927734, 0.004769563674926758, 0.004728794097900391, 0.004865169525146484, 0.0048787593841552734, 0.004899501800537109, 0.004758596420288086, 0.004852294921875, 0.00487971305847168, 0.004841327667236328, 0.004756450653076172, 0.004811286926269531, 0.004873991012573242, 0.004755973815917969, 0.0047914981842041016, 0.005125522613525391, 0.0049877166748046875, 0.004752159118652344, 0.004819154739379883, 0.004819631576538086, 0.0052564144134521484, 0.004834651947021484, 0.004858732223510742, 0.004899501800537109, 0.0048580169677734375, 0.004767417907714844, 0.004800319671630859, 0.004895210266113281, 0.004788875579833984, 0.004872798919677734, 0.004780769348144531, 0.004817485809326172, 0.004758119583129883, 0.004782676696777344, 0.004838228225708008, 0.004864215850830078, 0.004750490188598633, 0.0047817230224609375, 0.0048236846923828125, 0.004843473434448242, 0.004796266555786133, 0.0048825740814208984, 0.00484919548034668, 0.004782438278198242, 0.004922151565551758, 0.004901409149169922, 0.0049343109130859375, 0.004875898361206055, 0.004755735397338867, 0.004899263381958008, 0.00480341911315918, 0.0049211978912353516, 0.004772186279296875, 0.004812479019165039, 0.004953861236572266, 0.005136251449584961, 0.0048182010650634766, 0.004926919937133789, 0.004772186279296875, 0.004867076873779297, 0.004954814910888672, 0.004765510559082031, 0.004811286926269531, 0.004767656326293945, 0.004952192306518555, 0.004815101623535156, 0.0048122406005859375, 0.004848003387451172, 0.004838705062866211, 0.004732847213745117, 0.00480961799621582, 0.004895210266113281, 0.0048065185546875, 0.004853010177612305, 0.004847288131713867, 0.004827737808227539, 0.004848957061767578, 0.00481104850769043, 0.004853725433349609, 0.0047724246978759766, 0.00487208366394043, 0.0047931671142578125, 0.004801511764526367, 0.004797935485839844, 0.004845857620239258, 0.004873991012573242, 0.004839420318603516, 0.0049707889556884766, 0.004958152770996094, 0.004794120788574219, 0.0047626495361328125, 0.004865407943725586, 0.004801034927368164, 0.005007266998291016, 0.004788637161254883, 0.004869937896728516, 0.004775524139404297, 0.0048296451568603516, 0.004926204681396484, 0.00484919548034668, 0.004823207855224609, 0.0048182010650634766, 0.004846811294555664, 0.00482487678527832, 0.004844188690185547, 0.0048716068267822266, 0.0048067569732666016, 0.004900693893432617, 0.004854440689086914, 0.00482940673828125, 0.004823207855224609, 0.004822492599487305, 0.00478053092956543, 0.004902839660644531, 0.004842042922973633, 0.004877567291259766, 0.004755258560180664, 0.004813432693481445, 0.004807949066162109, 0.004874467849731445, 0.0048067569732666016, 0.004886150360107422, 0.006065845489501953, 0.004944801330566406, 0.004920244216918945, 0.00547480583190918, 0.0050580501556396484, 0.0048830509185791016, 0.005217552185058594, 0.004942417144775391, 0.004820108413696289, 0.004933595657348633, 0.004883527755737305, 0.005043506622314453, 0.004828453063964844, 0.00500178337097168, 0.004767179489135742, 0.004845619201660156, 0.0049779415130615234, 0.004951000213623047, 0.004831790924072266, 0.00481104850769043, 0.004957914352416992, 0.004908084869384766, 0.004814863204956055, 0.004957914352416992, 0.004833221435546875, 0.0048427581787109375, 0.0052335262298583984, 0.005638599395751953, 0.0054476261138916016, 0.005881309509277344, 0.006731986999511719, 0.013641595840454102, 0.007578134536743164, 0.012163400650024414, 0.00588536262512207, 0.005339622497558594, 0.005386829376220703, 0.005286455154418945, 0.005190134048461914, 0.005051136016845703, 0.004821062088012695, 0.004880428314208984, 0.004817962646484375, 0.0048143863677978516, 0.004813432693481445, 0.004720211029052734, 0.00483250617980957, 0.004776477813720703, 0.0048139095306396484, 0.004745006561279297, 0.004851579666137695, 0.004792690277099609, 0.0047321319580078125, 0.005007743835449219, 0.0047473907470703125, 0.004850149154663086, 0.004719257354736328, 0.004805564880371094, 0.00476384162902832, 0.004839181900024414, 0.004705905914306641, 0.004755496978759766, 0.004792928695678711, 0.0047435760498046875, 0.004871368408203125, 0.004780769348144531, 0.0048198699951171875, 0.004803657531738281, 0.004749298095703125, 0.0048675537109375, 0.004761219024658203, 0.004836320877075195, 0.004769563674926758, 0.00484013557434082, 0.004792451858520508, 0.004764556884765625, 0.0047681331634521484, 0.004748106002807617, 0.004841327667236328, 0.004770040512084961, 0.004823207855224609, 0.004782199859619141, 0.00482630729675293, 0.004862070083618164, 0.004805088043212891, 0.004735231399536133, 0.004751682281494141, 0.004803657531738281, 0.004800558090209961, 0.004811525344848633, 0.004785299301147461, 0.004849910736083984, 0.004817485809326172, 0.004792213439941406, 0.004868507385253906, 0.004938602447509766, 0.004781007766723633, 0.0047149658203125, 0.004853010177612305, 0.0047876834869384766, 0.004822731018066406, 0.0047359466552734375, 0.004804372787475586, 0.004866838455200195, 0.0047533512115478516, 0.004854917526245117, 0.004776477813720703, 0.0048618316650390625, 0.0047261714935302734, 0.004809141159057617, 0.0048182010650634766, 0.004763603210449219, 0.004817008972167969, 0.004776716232299805, 0.004812717437744141, 0.0047986507415771484, 0.0048122406005859375, 0.004785060882568359, 0.004755496978759766, 0.004848957061767578, 0.0048258304595947266, 0.0047953128814697266, 0.0047359466552734375, 0.004820823669433594, 0.0047760009765625, 0.00494074821472168, 0.004737377166748047, 0.004826068878173828, 0.00482630729675293, 0.004775285720825195, 0.004826784133911133, 0.0047855377197265625, 0.004860877990722656, 0.0048143863677978516, 0.004840850830078125, 0.004796266555786133, 0.004889726638793945, 0.004850149154663086, 0.004776477813720703, 0.004808902740478516, 0.00475764274597168, 0.005638599395751953, 0.005528926849365234, 0.01120448112487793, 0.013004302978515625, 0.005725383758544922, 0.005422115325927734, 0.005325794219970703, 0.005185842514038086, 0.005023479461669922, 0.004835367202758789, 0.004971027374267578, 0.004841804504394531, 0.004902839660644531, 0.004904747009277344, 0.005063533782958984, 0.004870891571044922, 0.0049855709075927734, 0.0049245357513427734, 0.005038022994995117, 0.004882097244262695, 0.0049479007720947266, 0.004996538162231445, 0.004906892776489258, 0.004859209060668945, 0.0050411224365234375, 0.004828691482543945, 0.0049021244049072266, 0.005015373229980469, 0.0048122406005859375, 0.004931926727294922, 0.004938840866088867, 0.0049591064453125, 0.00487828254699707, 0.004888296127319336, 0.004919767379760742, 0.004979610443115234, 0.004807710647583008, 0.005350589752197266, 0.005166530609130859, 0.004965543746948242, 0.004882097244262695, 0.005044460296630859, 0.004926443099975586, 0.004943132400512695, 0.004895687103271484, 0.0049877166748046875, 0.004823923110961914, 0.004936695098876953, 0.004836082458496094, 0.005112648010253906, 0.0048618316650390625, 0.004942893981933594, 0.004941701889038086, 0.004919767379760742, 0.00489044189453125, 0.0049550533294677734, 0.004891872406005859, 0.004962444305419922, 0.0049130916595458984, 0.004911661148071289, 0.004956960678100586, 0.004991292953491211, 0.004955291748046875, 0.004919767379760742, 0.0049664974212646484, 0.004937887191772461, 0.004954814910888672, 0.005150318145751953, 0.005009889602661133, 0.0049152374267578125, 0.004983186721801758, 0.004937171936035156, 0.00487208366394043, 0.0050699710845947266, 0.004946231842041016, 0.004954338073730469, 0.0048830509185791016, 0.004899024963378906, 0.004973411560058594, 0.0049190521240234375, 0.005023479461669922, 0.005139589309692383, 0.004947662353515625, 0.004938840866088867, 0.004936933517456055, 0.004975795745849609, 0.004995107650756836, 0.004911661148071289, 0.004941225051879883, 0.004942655563354492, 0.0049860477447509766, 0.00492095947265625, 0.004929065704345703, 0.004916667938232422, 0.004895210266113281, 0.004925727844238281, 0.004822254180908203, 0.004993438720703125, 0.004888057708740234, 0.004939079284667969, 0.004817485809326172, 0.004974842071533203, 0.004960536956787109, 0.004950523376464844, 0.004884243011474609, 0.004925727844238281, 0.004877567291259766, 0.004927158355712891, 0.0049016475677490234, 0.005074977874755859, 0.0049283504486083984, 0.004972696304321289, 0.004935026168823242, 0.00513458251953125, 0.0049703121185302734, 0.004970550537109375, 0.004922389984130859, 0.005006313323974609, 0.004935026168823242, 0.004889011383056641, 0.00487208366394043, 0.004992008209228516, 0.004958629608154297, 0.004896879196166992, 0.004996299743652344, 0.004980802536010742, 0.005024909973144531, 0.004860401153564453, 0.004945039749145508, 0.005239248275756836, 0.005043506622314453, 0.004813194274902344, 0.0049915313720703125, 0.004941701889038086, 0.0049855709075927734, 0.005173206329345703, 0.0049533843994140625, 0.00491023063659668, 0.004998207092285156, 0.004940032958984375, 0.004980802536010742, 0.004923820495605469, 0.005104541778564453, 0.004872560501098633, 0.004915475845336914, 0.004922151565551758, 0.004971981048583984, 0.004858493804931641, 0.0049283504486083984, 0.0049703121185302734, 0.004973173141479492, 0.004899501800537109, 0.004909992218017578, 0.004925966262817383, 0.005105733871459961, 0.004944324493408203, 0.004951000213623047, 0.004943370819091797, 0.0049419403076171875, 0.004822492599487305, 0.00495457649230957, 0.004916191101074219, 0.005030632019042969, 0.0049016475677490234, 0.0050122737884521484, 0.00495457649230957, 0.004918098449707031, 0.004958152770996094, 0.0049343109130859375, 0.004935026168823242, 0.004985809326171875, 0.00498652458190918, 0.004912853240966797, 0.005007028579711914, 0.004919290542602539, 0.004982948303222656, 0.004941463470458984, 0.004967927932739258, 0.004961967468261719, 0.00493311882019043, 0.004980325698852539, 0.004991054534912109, 0.004952430725097656, 0.005076885223388672, 0.0049784183502197266, 0.00510859489440918, 0.007153987884521484, 0.009792804718017578, 0.010947227478027344, 0.005547523498535156, 0.005460977554321289, 0.005443096160888672, 0.0054018497467041016, 0.005110740661621094, 0.005051136016845703, 0.005041360855102539, 0.00494074821472168, 0.0049266815185546875, 0.0048732757568359375, 0.005500078201293945, 0.004927635192871094, 0.00489044189453125, 0.0049059391021728516, 0.0048520565032958984, 0.0049076080322265625, 0.004874706268310547, 0.004900217056274414, 0.004870414733886719, 0.0049245357513427734, 0.004845380783081055, 0.004877567291259766, 0.0049130916595458984, 0.004861116409301758, 0.004917621612548828, 0.004830360412597656, 0.0049664974212646484, 0.004887819290161133, 0.0048906803131103516, 0.004822969436645508, 0.004937171936035156, 0.004958629608154297, 0.004869222640991211, 0.004811525344848633, 0.004897356033325195, 21.61993956565857]
t_14_none = [0.004865884780883789, 0.0036411285400390625, 0.0038542747497558594, 0.003737211227416992, 0.004287242889404297, 0.003983974456787109, 0.0039174556732177734, 0.003820180892944336, 0.0038270950317382812, 0.003908872604370117, 0.005567073822021484, 0.003908872604370117, 0.0038237571716308594, 0.005055665969848633, 0.0056455135345458984, 0.0052683353424072266, 0.004984855651855469, 0.00494074821472168, 0.0052013397216796875, 0.005086421966552734, 0.005349636077880859, 0.004724740982055664, 0.005653858184814453, 0.005712032318115234, 0.005472421646118164, 0.005532741546630859, 0.005262851715087891, 0.004147529602050781, 0.0037643909454345703, 0.0036513805389404297, 0.004714012145996094, 0.003803253173828125, 0.0048065185546875, 0.00538182258605957, 0.0036478042602539062, 0.0036666393280029297, 0.0036170482635498047, 0.0036258697509765625, 0.003654003143310547, 0.003620624542236328, 0.0037131309509277344, 0.0036096572875976562, 0.0037162303924560547, 0.003632068634033203, 0.0036385059356689453, 0.0036208629608154297, 0.0036170482635498047, 0.0037965774536132812, 0.003630399703979492, 0.0036346912384033203, 0.003622293472290039, 0.0036275386810302734, 0.00366973876953125, 0.0036215782165527344, 0.0036554336547851562, 0.003631114959716797, 0.0037078857421875, 0.0036246776580810547, 0.003609895706176758, 0.0036275386810302734, 0.003635406494140625, 0.003629922866821289, 0.0035829544067382812, 0.003638744354248047, 0.003651142120361328, 0.0036208629608154297, 0.003686666488647461, 0.003875732421875, 0.0036318302154541016, 0.0036242008209228516, 0.003650665283203125, 0.0036458969116210938, 0.0036249160766601562, 0.003653287887573242, 0.0035986900329589844, 0.0036399364471435547, 0.0036382675170898438, 0.003625154495239258, 0.0035865306854248047, 0.0036454200744628906, 0.0036573410034179688, 0.0036056041717529297, 0.003633737564086914, 0.0036382675170898438, 0.003591299057006836, 0.003625154495239258, 0.003641843795776367, 0.0036191940307617188, 0.003643035888671875, 0.003583669662475586, 0.0036225318908691406, 0.003596782684326172, 0.003612995147705078, 0.003620624542236328, 0.003692150115966797, 0.003603219985961914, 0.0036330223083496094, 0.0036132335662841797, 0.0036351680755615234, 0.0036690235137939453, 0.003577709197998047, 0.0036323070526123047, 0.0035784244537353516, 0.003638029098510742, 0.003647327423095703, 0.0036182403564453125, 0.0035829544067382812, 0.0036287307739257812, 0.003653287887573242, 0.003631591796875, 0.003634214401245117, 0.003708362579345703, 0.003632783889770508, 0.003591299057006836, 0.003634214401245117, 0.003629446029663086, 0.0036461353302001953, 0.003590106964111328, 0.0036287307739257812, 0.0036563873291015625, 0.0036923885345458984, 0.0036237239837646484, 0.0036118030548095703, 0.0036520957946777344, 0.003612518310546875, 0.003631114959716797, 0.0036318302154541016, 0.0036423206329345703, 0.0036056041717529297, 0.0036652088165283203, 0.0036454200744628906, 0.003639698028564453, 0.003661632537841797, 0.003641843795776367, 0.0036666393280029297, 0.003603219985961914, 0.0036468505859375, 0.0036706924438476562, 0.003641366958618164, 0.0036025047302246094, 0.0036554336547851562, 0.00363922119140625, 0.003639698028564453, 0.0036437511444091797, 0.003599405288696289, 0.0036437511444091797, 0.0035851001739501953, 0.0036306381225585938, 0.003640890121459961, 0.003640413284301758, 0.0036809444427490234, 0.003665924072265625, 0.0036351680755615234, 0.003631114959716797, 0.0036580562591552734, 0.0035927295684814453, 0.0036361217498779297, 0.0035958290100097656, 0.0036292076110839844, 0.003643035888671875, 0.0036394596099853516, 0.003584623336791992, 0.003623485565185547, 0.0040607452392578125, 0.0037565231323242188, 0.003642559051513672, 0.0037724971771240234, 0.003647327423095703, 0.0040285587310791016, 0.003656148910522461, 0.003833770751953125, 0.0036449432373046875, 0.003639698028564453, 0.0036492347717285156, 0.003665924072265625, 0.003640890121459961, 0.00517725944519043, 0.005992889404296875, 0.0059871673583984375, 0.006059408187866211, 0.00688624382019043, 0.00736689567565918, 0.012582778930664062, 0.006854057312011719, 0.006018400192260742, 0.006400585174560547, 0.01830434799194336, 0.006022930145263672, 0.005172252655029297, 0.007170200347900391, 0.007926464080810547, 0.004803657531738281, 0.004059553146362305, 0.003991127014160156, 0.00405120849609375, 0.00401759147644043, 0.003996133804321289, 0.0038967132568359375, 0.003723621368408203, 0.003914356231689453, 0.0036497116088867188, 0.0036377906799316406, 0.0036864280700683594, 0.00403904914855957, 0.003657102584838867, 0.003660917282104492, 0.0036742687225341797, 0.003597736358642578, 0.0036330223083496094, 0.0037560462951660156, 0.003639698028564453, 0.0036509037017822266, 0.003654956817626953, 0.0036895275115966797, 0.0036134719848632812, 0.0036520957946777344, 0.003641366958618164, 0.003635406494140625, 0.0036177635192871094, 0.003664255142211914, 0.0035877227783203125, 0.0036344528198242188, 0.0036406517028808594, 0.0036439895629882812, 0.003652811050415039, 0.0036449432373046875, 0.003640413284301758, 0.003660440444946289, 0.0036389827728271484, 0.0036008358001708984, 0.0036306381225585938, 0.0035920143127441406, 0.0036361217498779297, 0.003669261932373047, 0.0036716461181640625, 0.0036575794219970703, 0.0035963058471679688, 0.0036361217498779297, 0.003676176071166992, 0.003664731979370117, 0.003602743148803711, 0.0037314891815185547, 0.003738880157470703, 0.0036613941192626953, 0.003729104995727539, 0.0036525726318359375, 0.003652811050415039, 0.0036056041717529297, 0.0036704540252685547, 0.0036787986755371094, 0.0036458969116210938, 0.003614187240600586, 0.003643035888671875, 0.003618478775024414, 0.003789186477661133, 0.0036776065826416016, 0.0036187171936035156, 0.003652334213256836, 0.0037398338317871094, 0.0036737918853759766, 0.0036597251892089844, 0.0036745071411132812, 0.003618478775024414, 0.003681182861328125, 0.0036911964416503906, 0.003670930862426758, 0.0036754608154296875, 0.0036127567291259766, 0.0036559104919433594, 0.0036363601684570312, 0.0036437511444091797, 0.003640413284301758, 0.0036630630493164062, 0.0036156177520751953, 0.0036585330963134766, 0.0036613941192626953, 0.0036356449127197266, 0.0036461353302001953, 0.003626108169555664, 0.0036568641662597656, 0.0036165714263916016, 0.003639698028564453, 0.0036432743072509766, 0.0036773681640625, 0.0036008358001708984, 0.003662586212158203, 0.003711700439453125, 0.003634929656982422, 0.0036602020263671875, 0.003613710403442383, 0.0036635398864746094, 0.003605365753173828, 0.003648519515991211, 0.0036668777465820312, 0.0036535263061523438, 0.0036115646362304688, 0.0038051605224609375, 0.003679513931274414, 0.0036537647247314453, 0.003662586212158203, 0.003615140914916992, 0.0036411285400390625, 0.0036182403564453125, 0.0036520957946777344, 0.003735780715942383, 0.0036339759826660156, 0.003680706024169922, 0.0036487579345703125, 0.003653287887573242, 0.0036344528198242188, 0.0036499500274658203, 0.003625154495239258, 0.0036725997924804688, 0.003790616989135742, 0.0036706924438476562, 0.0036268234252929688, 0.0036661624908447266, 0.00365447998046875, 0.003677845001220703, 0.003707408905029297, 0.003719329833984375, 0.0036623477935791016, 0.003622293472290039, 0.0038034915924072266, 0.003682851791381836, 0.003696441650390625, 0.0036482810974121094, 0.003620147705078125, 0.0036687850952148438, 0.00368499755859375, 0.0036475658416748047, 0.003614187240600586, 0.0036454200744628906, 0.003617525100708008, 0.0036635398864746094, 0.0036427974700927734, 0.0036771297454833984, 0.003668546676635742, 0.003629446029663086, 0.003667116165161133, 0.0036516189575195312, 0.003660917282104492, 0.003606557846069336, 0.0036721229553222656, 0.0036406517028808594, 0.0036613941192626953, 0.0037021636962890625, 0.003611326217651367, 0.0036661624908447266, 0.003629922866821289, 0.00368499755859375, 0.0037682056427001953, 0.00365447998046875, 0.003619670867919922, 0.0036661624908447266, 0.0036330223083496094, 0.003717660903930664, 0.003662586212158203, 0.003625631332397461, 0.003679513931274414, 0.003628253936767578, 0.0036444664001464844, 0.003652811050415039, 0.003685474395751953, 0.003633260726928711, 0.0036466121673583984, 0.0036268234252929688, 0.003721952438354492, 0.0038161277770996094, 0.003631591796875, 0.0036585330963134766, 0.0036242008209228516, 0.0036606788635253906, 0.003683328628540039, 0.0036852359771728516, 0.0036017894744873047, 0.003665447235107422, 0.0036683082580566406, 0.0037152767181396484, 0.0036911964416503906, 0.0036280155181884766, 0.0037026405334472656, 0.003622770309448242, 0.00368499755859375, 0.003698587417602539, 0.0037086009979248047, 0.003694295883178711, 0.0036361217498779297, 0.003683805465698242, 0.003674745559692383, 0.003670215606689453, 0.003736257553100586, 0.0036699771881103516, 0.003640890121459961, 0.0036613941192626953, 0.0036504268646240234, 0.003894805908203125, 0.003668546676635742, 0.003626108169555664, 0.0036814212799072266, 0.0036897659301757812, 0.003652811050415039, 0.003631591796875, 0.003670930862426758, 0.0036339759826660156, 0.0038177967071533203, 0.0036618709564208984, 0.003614187240600586, 0.003671407699584961, 0.0036351680755615234, 0.003652334213256836, 0.003683328628540039, 0.0036695003509521484, 0.0036759376525878906, 0.003680706024169922, 0.003683328628540039, 0.0036652088165283203, 0.0036661624908447266, 0.003635883331298828, 0.0036864280700683594, 0.003666400909423828, 0.003625154495239258, 0.003786802291870117, 0.0036759376525878906, 0.0036754608154296875, 0.003675222396850586, 0.0036182403564453125, 0.0036573410034179688, 0.0036597251892089844, 0.003704071044921875, 0.0036230087280273438, 0.003665447235107422, 0.003627300262451172, 0.0036814212799072266, 0.0036771297454833984, 0.0036821365356445312, 0.0036666393280029297, 0.0036287307739257812, 0.0036783218383789062, 0.003717184066772461, 0.0036580562591552734, 0.003620147705078125, 0.0036678314208984375, 0.0036263465881347656, 0.003670930862426758, 0.003719329833984375, 0.0037322044372558594, 0.0036728382110595703, 0.003626585006713867, 0.003684520721435547, 0.0036635398864746094, 0.003672361373901367, 0.003622770309448242, 0.0036695003509521484, 0.003628253936767578, 0.0037136077880859375, 0.0036661624908447266, 0.003625154495239258, 0.0036742687225341797, 0.0036356449127197266, 0.0038118362426757812, 0.0036690235137939453, 0.0036644935607910156, 0.0036439895629882812, 0.0036826133728027344, 0.0036818981170654297, 0.0036993026733398438, 0.003680706024169922, 0.003623485565185547, 0.0036878585815429688, 0.0036292076110839844, 0.0036928653717041016, 0.0036890506744384766, 0.00366973876953125, 0.003632068634033203, 0.003679037094116211, 0.0036547183990478516, 0.0038025379180908203, 0.0036649703979492188, 0.0036504268646240234, 0.003679513931274414, 0.0036530494689941406, 0.003825664520263672, 0.0036869049072265625, 0.0036842823028564453, 0.0036957263946533203, 0.0037009716033935547, 0.0036668777465820312, 0.003692150115966797, 0.003660440444946289, 0.0036187171936035156, 0.003694772720336914, 0.003634214401245117, 0.0037326812744140625, 0.003710508346557617, 0.0036356449127197266, 0.0036666393280029297, 0.003625631332397461, 0.003735065460205078, 0.0036907196044921875, 0.0036804676055908203, 0.003631114959716797, 0.0036802291870117188, 0.003693819046020508, 0.003688335418701172, 0.0036797523498535156, 0.0036478042602539062, 0.0036923885345458984, 0.003635406494140625, 0.0036973953247070312, 0.0036966800689697266, 0.003696918487548828, 0.003637552261352539, 0.003689289093017578, 0.0037834644317626953, 0.003682374954223633, 0.0036830902099609375, 0.00365447998046875, 0.003699064254760742, 0.0036542415618896484, 0.0036699771881103516, 0.0036859512329101562, 0.0036907196044921875, 0.0036847591400146484, 0.003639698028564453, 0.0036847591400146484, 0.0036804676055908203, 0.0037124156951904297, 0.0036580562591552734, 0.0036897659301757812, 0.0036497116088867188, 0.0037457942962646484, 0.0037240982055664062, 0.0036542415618896484, 0.003675699234008789, 0.003641366958618164, 0.0036704540252685547, 0.003677845001220703, 0.0037043094635009766, 0.0036466121673583984, 0.0036766529083251953, 0.0036377906799316406, 0.003723621368408203, 0.003696918487548828, 0.0036551952362060547, 0.0036802291870117188, 0.0036416053771972656, 0.003689289093017578, 0.0036873817443847656, 0.0036869049072265625, 0.0036466121673583984, 0.0036895275115966797, 0.00368499755859375, 0.0036885738372802734, 0.0037217140197753906, 0.003645658493041992, 0.0036983489990234375, 0.0036432743072509766, 0.0036852359771728516, 0.003755807876586914, 0.0037097930908203125, 0.0036902427673339844, 0.0036292076110839844, 0.0036919116973876953, 0.0037078857421875, 0.0036809444427490234, 0.0036382675170898438, 0.003680706024169922, 0.0036461353302001953, 0.0038003921508789062, 0.003738880157470703, 0.0036344528198242188, 0.003682851791381836, 0.0036623477935791016, 0.0036840438842773438, 0.0036957263946533203, 0.0038175582885742188, 0.0036373138427734375, 0.0036842823028564453, 0.0037059783935546875, 0.0036852359771728516, 0.003688335418701172, 0.0036439895629882812, 0.0037064552307128906, 0.003654956817626953, 0.003777027130126953, 0.0036852359771728516, 0.003716707229614258, 0.003659963607788086, 0.0037119388580322266, 0.0036792755126953125, 0.003717184066772461, 0.003671407699584961, 0.003656148910522461, 0.003693819046020508, 0.0037038326263427734, 0.0036973953247070312, 0.0036737918853759766, 0.0037088394165039062, 0.0036993026733398438, 0.0036509037017822266, 0.003670215606689453, 0.0036830902099609375, 0.0037190914154052734, 0.003649473190307617, 0.003682374954223633, 0.0036373138427734375, 0.0036809444427490234, 0.0037555694580078125, 0.003654956817626953, 0.0037009716033935547, 0.0036559104919433594, 0.003686189651489258, 0.00368499755859375, 0.0036869049072265625, 0.0036537647247314453, 0.00366973876953125, 0.003685474395751953, 0.0037097930908203125, 0.003692626953125, 0.003643035888671875, 0.0037081241607666016, 0.003634929656982422, 0.003797292709350586, 0.0037050247192382812, 0.0037086009979248047, 0.0036928653717041016, 0.0036411285400390625, 0.003688335418701172, 0.003699064254760742, 0.0036821365356445312, 0.003656148910522461, 0.0037016868591308594, 0.003652811050415039, 0.0037200450897216797, 0.003706693649291992, 0.003682851791381836, 0.003699779510498047, 0.003674745559692383, 0.0036873817443847656, 0.003786325454711914, 0.0037088394165039062, 0.0036635398864746094, 0.0037076473236083984, 0.0036695003509521484, 0.003759622573852539, 0.0036897659301757812, 0.0036542415618896484, 0.0037012100219726562, 0.0036575794219970703, 0.003708362579345703, 0.0036835670471191406, 0.0036869049072265625, 0.003719806671142578, 0.0037794113159179688, 0.003639698028564453, 0.003682374954223633, 0.003649473190307617, 0.003696441650390625, 0.0037147998809814453, 0.003693103790283203, 0.003648519515991211, 0.003679990768432617, 0.0036754608154296875, 0.0037734508514404297, 0.0036885738372802734, 0.0036585330963134766, 0.003707408905029297, 0.00365447998046875, 0.0037009716033935547, 0.0036847591400146484, 0.003678560256958008, 0.0036568641662597656, 0.0037147998809814453, 0.00372314453125, 0.0036869049072265625, 0.0036907196044921875, 0.003734111785888672, 0.0036916732788085938, 0.0036611557006835938, 0.0037279129028320312, 0.003725767135620117, 0.003706693649291992, 0.0037021636962890625, 0.0036492347717285156, 0.0036895275115966797, 0.0036971569061279297, 0.003691434860229492, 0.003664255142211914, 0.0037107467651367188, 0.003650665283203125, 0.0037713050842285156, 0.003702878952026367, 0.0036652088165283203, 0.0038022994995117188, 0.003643512725830078, 0.003699064254760742, 0.0036835670471191406, 0.0036966800689697266, 0.003653287887573242, 0.003693819046020508, 0.003680706024169922, 0.003686666488647461, 0.003715991973876953, 0.0036652088165283203, 0.0037131309509277344, 0.003648042678833008, 0.003706216812133789, 0.003687620162963867, 0.0036935806274414062, 0.0037000179290771484, 0.0036513805389404297, 0.0037000179290771484, 0.003700733184814453, 0.003696918487548828, 0.0036487579345703125, 0.0036993026733398438, 0.003651857376098633, 0.003695249557495117, 0.003798246383666992, 0.0036563873291015625, 0.003680706024169922, 0.0036559104919433594, 0.0037021636962890625, 0.0037078857421875, 0.003715991973876953, 0.0036351680755615234, 0.003692626953125, 0.003673076629638672, 0.0037641525268554688, 0.003710031509399414, 0.0037496089935302734, 0.0037088394165039062, 0.0036547183990478516, 0.0037043094635009766, 0.003710031509399414, 0.0037174224853515625, 0.0037343502044677734, 0.003702878952026367, 0.0037412643432617188, 0.003701925277709961, 0.0037126541137695312, 0.0036573410034179688, 0.0038645267486572266, 0.0036668777465820312, 0.003695964813232422, 0.003757953643798828, 0.003660917282104492, 0.003839254379272461, 0.0036695003509521484, 0.0037353038787841797, 0.003697633743286133, 0.0037071704864501953, 0.0038537979125976562, 0.003757953643798828, 0.0036902427673339844, 0.003688335418701172, 0.003767251968383789, 0.0036721229553222656, 0.0037088394165039062, 0.0037496089935302734, 0.003725290298461914, 0.003705263137817383, 0.003746509552001953, 0.0037376880645751953, 0.0037131309509277344, 0.003711700439453125, 0.003700733184814453, 0.0037207603454589844, 0.003756284713745117, 0.003670930862426758, 0.003710031509399414, 0.0037009716033935547, 0.0036847591400146484, 0.0037038326263427734, 0.003658771514892578, 0.0036916732788085938, 0.003659486770629883, 0.0037050247192382812, 0.00370025634765625, 0.0036859512329101562, 0.0037004947662353516, 0.003659486770629883, 0.003683328628540039, 0.0036945343017578125, 0.0037643909454345703, 0.0036470890045166016, 0.0036895275115966797, 0.0036542415618896484, 0.0036966800689697266, 0.0037403106689453125, 0.003660440444946289, 0.0036945343017578125, 0.003668546676635742, 0.0037131309509277344, 0.003711700439453125, 0.003690481185913086, 0.003661632537841797, 0.0037572383880615234, 0.0036988258361816406, 0.003715991973876953, 0.003690004348754883, 0.0038022994995117188, 0.0037250518798828125, 0.003677845001220703, 0.0037050247192382812, 0.003681659698486328, 0.0036993026733398438, 0.003731966018676758, 0.003662586212158203, 0.003714323043823242, 0.003695964813232422, 0.003699779510498047, 0.003675222396850586, 0.0037016868591308594, 0.003677845001220703, 0.003686666488647461, 0.003750324249267578, 0.0036919116973876953, 0.0037109851837158203, 0.0036690235137939453, 0.003698110580444336, 0.0037038326263427734, 0.003706216812133789, 0.0036783218383789062, 0.003707408905029297, 0.0036628246307373047, 0.0037546157836914062, 0.003711223602294922, 0.0036830902099609375, 0.0037407875061035156, 0.0036716461181640625, 0.0037069320678710938, 0.00370025634765625, 0.0037207603454589844, 0.0036840438842773438, 0.0036923885345458984, 0.0036923885345458984, 0.003704547882080078, 0.0037910938262939453, 0.0036783218383789062, 0.003697633743286133, 0.003662586212158203, 0.003707408905029297, 0.0037317276000976562, 0.0037031173706054688, 0.0038270950317382812, 0.003674030303955078, 0.003717660903930664, 0.003693819046020508, 0.0036988258361816406, 0.003660917282104492, 0.0036902427673339844, 0.0037119388580322266, 0.0037109851837158203, 0.0037093162536621094, 0.0037393569946289062, 0.0036973953247070312, 0.003668069839477539, 0.003716707229614258, 0.0037033557891845703, 0.0037071704864501953, 0.003729581832885742, 0.0036764144897460938, 0.0037288665771484375, 0.0038535594940185547, 0.0038881301879882812, 0.003831624984741211, 0.0037212371826171875, 0.0037186145782470703, 0.0037665367126464844, 0.003721952438354492, 0.003712177276611328, 0.003711223602294922, 0.0037207603454589844, 0.003720521926879883, 0.0037229061126708984, 0.003724813461303711, 0.0037169456481933594, 0.0037093162536621094, 0.0037093162536621094, 0.0036869049072265625, 0.0036745071411132812, 0.003720998764038086, 0.003673076629638672, 0.003711700439453125, 0.0037114620208740234, 0.003727436065673828, 0.0037260055541992188, 0.003690958023071289, 0.003705263137817383, 0.0037186145782470703, 0.003710508346557617, 0.0037698745727539062, 0.003726482391357422, 0.003681659698486328, 0.0037813186645507812, 0.0037012100219726562, 0.0036783218383789062, 0.003719329833984375, 0.003665447235107422, 0.0037183761596679688, 0.003727436065673828, 0.0037255287170410156, 0.0036928653717041016, 0.0037124156951904297, 0.0037238597869873047, 0.003711700439453125, 0.003719806671142578, 0.003684520721435547, 0.0037822723388671875, 0.0036787986755371094, 0.0037245750427246094, 0.003707408905029297, 0.0037488937377929688, 0.0037131309509277344, 0.0036628246307373047, 0.0036933422088623047, 0.003717660903930664, 0.0037260055541992188, 0.0036644935607910156, 0.0037126541137695312, 0.0036759376525878906, 0.003776073455810547, 0.0037064552307128906, 0.003679037094116211, 0.0036995410919189453, 0.0036706924438476562, 0.0037238597869873047, 0.003729581832885742, 0.003715991973876953, 0.0036826133728027344, 0.0037310123443603516, 0.0037238597869873047, 0.0037093162536621094, 0.0037288665771484375, 0.003673076629638672, 0.003713369369506836, 0.003679037094116211, 0.0037200450897216797, 0.0037717819213867188, 0.0036706924438476562, 0.003696918487548828, 0.003687620162963867, 0.00372314453125, 0.003713846206665039, 0.003715038299560547, 0.0037603378295898438, 0.0037283897399902344, 0.003729104995727539, 0.003717184066772461, 0.0037183761596679688, 0.0036809444427490234, 0.0037381649017333984, 0.0036957263946533203, 0.0037043094635009766, 0.003713846206665039, 0.0037391185760498047, 0.003736734390258789, 0.003681659698486328, 0.003729581832885742, 0.0037207603454589844, 0.0037086009979248047, 0.003688335418701172, 0.0038514137268066406, 0.003687620162963867, 0.0037653446197509766, 0.003728151321411133, 0.0036847591400146484, 0.0037200450897216797, 0.003665924072265625, 0.003740549087524414, 0.0037271976470947266, 0.003716707229614258, 0.003690481185913086, 0.0037298202514648438, 0.0037107467651367188, 0.0037016868591308594, 0.0037186145782470703, 0.0037016868591308594, 0.0037491321563720703, 0.003679990768432617, 0.003698110580444336, 0.003731250762939453, 0.0037360191345214844, 0.0037240982055664062, 0.003683328628540039, 0.0037424564361572266, 0.003734111785888672, 0.0037224292755126953, 0.003738880157470703, 0.0037088394165039062, 0.003733396530151367, 0.003729104995727539, 0.0037355422973632812, 0.003687143325805664, 0.0037031173706054688, 0.003683805465698242, 0.0037283897399902344, 0.003774404525756836, 0.0038127899169921875, 0.003743886947631836, 0.0036971569061279297, 0.0037202835083007812, 0.0037567615509033203, 0.003727436065673828, 0.0036759376525878906, 0.0037178993225097656, 0.0036842823028564453, 0.0037653446197509766, 0.003724813461303711, 0.0037283897399902344, 0.003738880157470703, 0.0037069320678710938, 0.00374603271484375, 0.003747701644897461, 0.0037446022033691406, 0.0036880970001220703, 0.0039081573486328125, 0.003751516342163086, 0.0037314891815185547, 0.003732442855834961, 0.0036933422088623047, 0.0037381649017333984, 0.0037076473236083984, 0.003743886947631836, 0.0037751197814941406, 0.0036954879760742188, 0.003771543502807617, 0.003705263137817383, 0.003733396530151367, 0.003733396530151367, 0.0037305355072021484, 0.0036973953247070312, 0.00374603271484375, 0.0037271976470947266, 0.003727436065673828, 0.003749370574951172, 0.0037047863006591797, 0.0038089752197265625, 0.0037136077880859375, 0.003738880157470703, 0.003789186477661133, 0.0037169456481933594, 0.0037317276000976562, 0.0036916732788085938, 0.003720998764038086, 0.003766298294067383, 0.0038368701934814453, 0.003718852996826172, 0.003774881362915039, 0.003731966018676758, 0.0037245750427246094, 0.0037729740142822266, 0.003780364990234375, 0.0037283897399902344, 0.0036802291870117188, 0.0037343502044677734, 0.0037622451782226562, 0.0037384033203125, 0.0037403106689453125, 0.003729581832885742, 0.0037271976470947266, 0.0037398338317871094, 0.0037300586700439453, 0.0037012100219726562, 0.0037314891815185547, 0.0037431716918945312, 0.003744363784790039, 0.003751993179321289, 0.003688335418701172, 0.0037200450897216797, 0.0037872791290283203, 0.0037326812744140625, 0.003730297088623047, 0.0037293434143066406, 0.0037317276000976562, 0.003699779510498047, 0.0037300586700439453, 0.003745555877685547, 0.0037364959716796875, 0.003690004348754883, 0.00371551513671875, 0.0037109851837158203, 0.003782033920288086, 0.0037245750427246094, 0.0037012100219726562, 0.0037178993225097656, 0.0037229061126708984, 0.003737926483154297, 0.003742694854736328, 0.00372314453125, 0.003740549087524414, 0.003718137741088867, 0.003735065460205078, 0.0038084983825683594, 0.003736734390258789, 0.003752470016479492, 0.0037453174591064453, 0.0037512779235839844, 0.0037384033203125, 0.0037391185760498047, 0.0037348270416259766, 0.003742694854736328, 0.003745555877685547, 0.0037238597869873047, 0.0037114620208740234, 0.0037984848022460938, 0.0037908554077148438, 0.0037186145782470703, 0.00372314453125, 0.0036923885345458984, 0.00373077392578125, 0.0037670135498046875, 0.0037431716918945312, 0.0038383007049560547, 0.003755331039428711, 0.003732919692993164, 0.0037317276000976562, 0.0037331581115722656, 0.003699779510498047, 0.003732442855834961, 0.0037009716033935547, 0.0037267208099365234, 0.0037364959716796875, 0.003865957260131836, 0.003728628158569336, 0.003709077835083008, 0.003747701644897461, 0.0037615299224853516, 0.003736257553100586, 0.003706216812133789, 0.0037343502044677734, 0.0037517547607421875, 0.003740549087524414, 0.0037391185760498047, 0.003691434860229492, 0.003736734390258789, 0.003743410110473633, 0.0038194656372070312, 0.0038344860076904297, 0.0038487911224365234, 0.003839731216430664, 0.0038123130798339844, 0.003854036331176758, 0.003834962844848633, 0.003825664520263672, 0.003832578659057617, 0.00380706787109375, 0.0038366317749023438, 0.0038433074951171875, 0.003824472427368164, 0.0037920475006103516, 0.003849506378173828, 0.003852367401123047, 0.003833293914794922, 0.0038280487060546875, 0.003780364990234375, 0.0038347244262695312, 0.0037994384765625, 0.0039052963256835938, 0.0038955211639404297, 0.0037946701049804688, 0.0038421154022216797, 0.0037894248962402344, 0.0038459300994873047, 0.003827333450317383, 0.003858327865600586, 0.0038526058197021484, 0.0038022994995117188, 0.0038390159606933594, 0.0038611888885498047, 0.0038352012634277344, 0.0038063526153564453, 0.0038504600524902344, 0.003836393356323242, 0.00383758544921875, 0.0038437843322753906, 0.0039005279541015625, 0.0038480758666992188, 0.003837108612060547, 0.0038428306579589844, 0.0038394927978515625, 0.0038003921508789062, 0.003850221633911133, 0.0038039684295654297, 0.0038313865661621094, 0.0038919448852539062, 0.0038487911224365234, 0.0038542747497558594, 0.0038144588470458984, 0.003829479217529297, 0.003849506378173828, 0.0038750171661376953, 0.0038514137268066406, 0.0038068294525146484, 0.003833770751953125, 0.0038297176361083984, 0.0038442611694335938, 0.0038824081420898438, 0.003846883773803711, 0.0038254261016845703, 0.003834962844848633, 0.0038471221923828125, 0.003815174102783203, 0.003835916519165039, 0.0037975311279296875, 0.003893136978149414, 0.0038361549377441406, 0.0038080215454101562, 0.0038428306579589844, 0.0038003921508789062, 0.003915548324584961, 0.00392460823059082, 0.0038115978240966797, 0.0038416385650634766, 0.003803730010986328, 0.0038280487060546875, 0.0038733482360839844, 0.003856658935546875, 0.0038454532623291016, 0.0038077831268310547, 0.003832578659057617, 0.0038557052612304688, 0.0038352012634277344, 0.0038025379180908203, 0.003855466842651367, 0.0038328170776367188, 0.003854990005493164, 0.0038437843322753906, 0.0038909912109375, 0.003840208053588867, 0.0038156509399414062, 0.0038917064666748047, 0.003850221633911133, 0.0038216114044189453, 0.003829479217529297, 0.003812551498413086, 0.003834962844848633, 0.0038945674896240234, 0.0038139820098876953, 0.0038421154022216797, 0.0038039684295654297, 0.0038518905639648438, 0.003848552703857422, 0.0038423538208007812, 0.0038449764251708984, 0.0038008689880371094, 0.0038716793060302734, 0.003851652145385742, 0.0038530826568603516, 0.0037992000579833984, 0.0038433074951171875, 0.0038843154907226562, 0.0038781166076660156, 0.003869771957397461, 0.00383758544921875, 0.003861665725708008, 0.003820657730102539, 0.0038993358612060547, 0.0038607120513916016, 0.003805398941040039, 0.0038328170776367188, 0.0038216114044189453, 0.0038728713989257812, 0.003980398178100586, 0.003821849822998047, 0.003845691680908203, 0.003816843032836914, 0.0038595199584960938, 0.004334211349487305, 0.003952741622924805, 0.0038645267486572266, 0.0038318634033203125, 0.0038728713989257812, 0.0038650035858154297, 0.003871440887451172, 0.0038864612579345703, 0.003834247589111328, 0.0038869380950927734, 0.0038733482360839844, 0.0038602352142333984, 0.00392913818359375, 0.003877878189086914, 0.003859996795654297, 0.003863811492919922, 0.003856182098388672, 0.003819704055786133, 0.0038776397705078125, 0.0043790340423583984, 0.003922462463378906, 0.0041692256927490234, 0.003862142562866211, 0.003889799118041992, 0.0038840770721435547, 0.0038509368896484375, 0.0038614273071289062, 0.0038166046142578125, 0.0038878917694091797, 0.0038306713104248047, 0.003922224044799805, 0.003922939300537109, 0.0038411617279052734, 0.0038826465606689453, 0.003833770751953125, 0.0038492679595947266, 0.0038688182830810547, 0.003863811492919922, 0.0038661956787109375, 0.0038557052612304688, 0.003866910934448242, 0.0038747787475585938, 0.0038628578186035156, 0.003872394561767578, 0.003834962844848633, 0.0038683414459228516, 0.0039446353912353516, 0.00386810302734375, 0.003830432891845703, 0.0039365291595458984, 0.003841400146484375, 0.003833770751953125, 0.003840923309326172, 0.0038335323333740234, 0.0038590431213378906, 0.0038750171661376953, 0.003887653350830078, 0.003858804702758789, 0.003837108612060547, 0.0038628578186035156, 0.003816843032836914, 0.0038590431213378906, 0.0038924217224121094, 0.00394749641418457, 0.003882169723510742, 0.0038292407989501953, 0.0038576126098632812, 0.003863811492919922, 0.0038852691650390625, 0.0038938522338867188, 0.0038285255432128906, 0.0038690567016601562, 0.0038442611694335938, 0.003924131393432617, 0.003884553909301758, 0.0038254261016845703, 0.0038690567016601562, 0.003856658935546875, 0.003882169723510742, 0.003835916519165039, 0.003894805908203125, 0.0038747787475585938, 0.0038604736328125, 0.003866910934448242, 0.003846406936645508, 0.003874540328979492, 0.003818511962890625, 0.003925800323486328, 0.003871917724609375, 0.0038704872131347656, 0.0038459300994873047, 0.003820657730102539, 0.003845691680908203, 0.003915309906005859, 0.0038340091705322266, 0.003874540328979492, 0.003821849822998047, 0.003857135772705078, 0.003976106643676758, 0.003879547119140625, 0.0038793087005615234, 0.003826618194580078, 0.003880023956298828, 0.0038819313049316406, 0.003861665725708008, 0.00398707389831543, 0.00383758544921875, 0.0038580894470214844, 0.0039010047912597656, 0.0038673877716064453, 0.003821134567260742, 0.003912925720214844, 0.0038826465606689453, 0.003870725631713867, 0.0038704872131347656, 0.003927946090698242, 0.003871917724609375, 0.0039060115814208984, 0.003879070281982422, 0.003899097442626953, 0.0038514137268066406, 0.0038809776306152344, 0.0038940906524658203, 0.003998994827270508, 0.003900766372680664, 0.004136085510253906, 0.004416704177856445, 0.004347324371337891, 0.0042150020599365234, 0.004199981689453125, 0.0041959285736083984, 0.005113363265991211, 0.004876136779785156, 0.006307363510131836, 0.006726264953613281, 0.004625558853149414, 0.004593610763549805, 0.004711627960205078, 0.004575490951538086, 0.004846811294555664, 0.004538059234619141, 0.00468897819519043, 0.00463104248046875, 0.0047054290771484375, 0.004719734191894531, 0.004594087600708008, 0.004804372787475586, 0.004342555999755859, 0.0044476985931396484, 0.004013538360595703, 0.004055976867675781, 0.004366397857666016, 0.0061643123626708984, 0.004873991012573242, 0.0040283203125, 0.003924369812011719, 0.0038776397705078125, 0.0038928985595703125, 0.003947257995605469, 0.0038917064666748047, 0.003956317901611328, 0.0039081573486328125, 0.0039021968841552734, 0.003946542739868164, 0.003879547119140625, 0.0038733482360839844, 0.0038690567016601562, 0.0038919448852539062, 0.0038945674896240234, 0.0039424896240234375, 0.0038797855377197266, 0.003865957260131836, 0.003832101821899414, 0.0038721561431884766, 0.003854513168334961, 0.003947257995605469, 0.0038704872131347656, 0.0038633346557617188, 0.003884553909301758, 0.0038442611694335938, 0.003890514373779297, 0.0039196014404296875, 0.0038497447967529297, 0.003875255584716797, 0.003846883773803711, 0.0038650035858154297, 0.0038738250732421875, 0.003869295120239258, 0.0039000511169433594, 0.0038576126098632812, 0.0038764476776123047, 0.0038766860961914062, 0.0038716793060302734, 0.0038504600524902344, 0.003979206085205078, 0.003913402557373047, 0.0038895606994628906, 0.003878355026245117, 0.0038504600524902344, 0.003865480422973633, 0.003912687301635742, 0.003894329071044922, 0.0038683414459228516, 0.0038421154022216797, 0.0038864612579345703, 0.0038306713104248047, 0.0039196014404296875, 0.004155635833740234, 0.0040895938873291016, 0.0039157867431640625, 0.003904581069946289, 0.004052162170410156, 0.004239559173583984, 0.003924131393432617, 0.003880023956298828, 0.0038967132568359375, 0.003907442092895508, 0.003879547119140625, 0.0038483142852783203, 0.003893136978149414, 0.003832578659057617, 0.003880739212036133, 0.0039539337158203125, 0.0038688182830810547, 0.003890514373779297, 0.0038480758666992188, 0.0039103031158447266, 0.0038933753967285156, 0.0039081573486328125, 0.00388336181640625, 0.003855466842651367, 0.0038771629333496094, 0.003894805908203125, 0.0038840770721435547, 0.0038962364196777344, 0.0038480758666992188, 0.003901958465576172, 0.0038700103759765625, 0.0039043426513671875, 0.0038471221923828125, 0.00391840934753418, 0.0038869380950927734, 0.00389862060546875, 0.003907680511474609, 0.0038576126098632812, 0.0039751529693603516, 0.003921031951904297, 0.003899097442626953, 0.003904104232788086, 0.003879070281982422, 0.003885984420776367, 0.0039033889770507812, 0.003945827484130859, 0.0038933753967285156, 0.0038518905639648438, 0.0038857460021972656, 0.0039038658142089844, 0.0045053958892822266, 0.003977298736572266, 0.003900289535522461, 0.003880739212036133, 0.004326820373535156, 0.003943443298339844, 0.00397038459777832, 0.0038993358612060547, 0.003892660140991211, 0.0038902759552001953, 0.003871440887451172, 0.003922462463378906, 0.003907442092895508, 0.003956794738769531, 0.0039048194885253906, 0.0038526058197021484, 0.0039288997650146484, 0.0038976669311523438, 0.003888845443725586, 0.003894329071044922, 0.003859996795654297, 0.003896951675415039, 0.00394749641418457, 0.003949642181396484, 0.003884553909301758, 0.0039055347442626953, 0.003896951675415039, 0.003856182098388672, 0.003929615020751953, 0.003881216049194336, 0.0038480758666992188, 0.0038585662841796875, 0.0038514137268066406, 0.003873586654663086, 0.0039031505584716797, 0.003840923309326172, 0.0038771629333496094, 0.0038454532623291016, 0.0038633346557617188, 0.0038919448852539062, 0.003992795944213867, 0.0038909912109375, 0.0038623809814453125, 0.0038840770721435547, 0.0038673877716064453, 0.0038728713989257812, 0.003928422927856445, 0.003869295120239258, 0.003882169723510742, 0.0038726329803466797, 0.003893136978149414, 0.003879070281982422, 0.003916025161743164, 0.003880739212036133, 0.003911495208740234, 0.0038988590240478516, 0.0038547515869140625, 0.00396728515625, 0.003913402557373047, 0.003897428512573242, 0.003902912139892578, 0.0038352012634277344, 0.0038995742797851562, 0.0038466453552246094, 0.003946542739868164, 0.0039055347442626953, 0.0038759708404541016, 0.0038988590240478516, 0.0038514137268066406, 0.0039033889770507812, 0.003920555114746094, 0.0038611888885498047, 0.0038912296295166016, 0.003854513168334961, 0.003873586654663086, 0.003877401351928711, 0.003899812698364258, 0.0038962364196777344, 0.0038595199584960938, 0.003866434097290039, 0.0038869380950927734, 0.0038962364196777344, 0.003910064697265625, 0.0038976669311523438, 0.003879547119140625, 0.0038831233978271484, 0.0038788318634033203, 0.003835916519165039, 0.0038754940032958984, 0.0038788318634033203, 0.0038862228393554688, 0.00397801399230957, 0.0038690567016601562, 0.003893136978149414, 0.003884553909301758, 0.0038945674896240234, 0.0038938522338867188, 0.003866434097290039, 0.004016399383544922, 0.0038580894470214844, 0.003926515579223633, 0.003899812698364258, 0.003840923309326172, 0.003899812698364258, 0.0038585662841796875, 0.0038886070251464844, 0.003941535949707031, 0.003857851028442383, 0.0038805007934570312, 0.003960132598876953, 0.003900766372680664, 0.0038826465606689453, 0.00391697883605957, 0.003930330276489258, 0.0038475990295410156, 0.003917217254638672, 0.004355192184448242, 0.003960371017456055, 0.00391077995300293, 0.0038962364196777344, 0.003896474838256836, 0.003928422927856445, 0.003904581069946289, 0.003911256790161133, 0.003867626190185547, 0.0039055347442626953, 0.003930807113647461, 0.0038950443267822266, 0.003865480422973633, 0.0038909912109375, 0.003892183303833008, 0.0039005279541015625, 0.00390934944152832, 0.0038671493530273438, 0.003890514373779297, 0.0039021968841552734, 0.003910541534423828, 0.00392603874206543, 0.0038406848907470703, 0.0038938522338867188, 0.0038650035858154297, 0.003950834274291992, 0.003988504409790039, 0.0038526058197021484, 0.0038819313049316406, 0.003864288330078125, 0.0038902759552001953, 0.0039577484130859375, 0.0038557052612304688, 0.0038852691650390625, 0.0038633346557617188, 0.0039157867431640625, 0.003910541534423828, 0.00391697883605957, 0.00390934944152832, 0.0038733482360839844, 0.00391387939453125, 0.0038928985595703125, 0.003920078277587891, 0.003888845443725586, 0.004016876220703125, 0.003916025161743164, 0.003899097442626953, 0.003913402557373047, 0.003909587860107422, 0.0038633346557617188, 0.003925323486328125, 0.003908634185791016, 0.0039081573486328125, 0.0038459300994873047, 0.004006385803222656, 0.0038650035858154297, 0.0039081573486328125, 0.003909111022949219, 0.0039005279541015625, 0.0038573741912841797, 0.003922462463378906, 0.0039141178131103516, 0.00392603874206543, 0.003892660140991211, 0.0038530826568603516, 0.0039038658142089844, 0.003914356231689453, 0.003902912139892578, 0.003911733627319336, 0.003872394561767578, 0.003935098648071289, 0.003917694091796875, 0.003930330276489258, 0.003889799118041992, 0.0038504600524902344, 0.0039081573486328125, 0.0038726329803466797, 0.004060029983520508, 0.0038983821868896484, 0.003857135772705078, 0.003910541534423828, 0.0038657188415527344, 0.003916025161743164, 0.003936052322387695, 0.0038564205169677734, 0.003907918930053711, 0.0038611888885498047, 0.003898143768310547, 0.003884553909301758, 0.003904104232788086, 0.003939151763916016, 0.003886699676513672, 0.003900289535522461, 0.0038928985595703125, 0.0038967132568359375, 0.004006385803222656, 0.0038938522338867188, 0.0039031505584716797, 0.00391077995300293, 0.003907203674316406, 0.0038874149322509766, 0.003936052322387695, 0.0039060115814208984, 0.0038983821868896484, 0.003910541534423828, 0.0038878917694091797, 0.0039408206939697266, 0.00392603874206543, 0.0038957595825195312, 0.003922462463378906, 0.00388336181640625, 0.003925800323486328, 0.003941535949707031, 0.0038976669311523438, 0.0038938522338867188, 0.003905057907104492, 0.003922700881958008, 0.0038874149322509766, 0.003939628601074219, 0.003926515579223633, 0.0038650035858154297, 0.003935813903808594, 0.003869771957397461, 0.0039234161376953125, 0.00396418571472168, 0.0039174556732177734, 0.003919124603271484, 0.0038836002349853516, 0.004065036773681641, 0.003999233245849609, 0.003870248794555664, 0.0039327144622802734, 0.003876209259033203, 0.0039348602294921875, 0.003977298736572266, 0.003954648971557617, 0.003916263580322266, 0.003877878189086914, 0.003914833068847656, 0.0038895606994628906, 0.003996610641479492, 0.003924369812011719, 0.0038840770721435547, 0.003894805908203125, 0.0038945674896240234, 0.003926992416381836, 0.003912687301635742, 0.003979682922363281, 0.003905057907104492, 0.003927707672119141, 0.0038979053497314453, 0.003874063491821289, 0.003939390182495117, 0.003957271575927734, 0.0039043426513671875, 0.004097938537597656, 0.0039784908294677734, 0.003932952880859375, 0.0038976669311523438, 0.003923654556274414, 0.0039055347442626953, 0.0038709640502929688, 0.00391840934753418, 0.003945112228393555, 0.003947257995605469, 0.0039331912994384766, 0.0039272308349609375, 0.003893613815307617, 0.003864765167236328, 0.003965854644775391, 0.003930568695068359, 0.003882884979248047, 0.0039052963256835938, 0.0038590431213378906, 0.003961324691772461, 0.003911733627319336, 0.003888368606567383, 0.003926753997802734, 0.0038886070251464844, 0.003981113433837891, 0.004008293151855469, 0.003894329071044922, 0.003922224044799805, 0.003920555114746094, 0.003945589065551758, 0.003964900970458984, 0.0038945674896240234, 0.003924369812011719, 0.0038979053497314453, 0.003929853439331055, 0.003908395767211914, 0.003929853439331055, 0.0039103031158447266, 0.0038843154907226562, 0.003942012786865234, 0.003935098648071289, 0.003946781158447266, 0.0039212703704833984, 0.004026174545288086, 0.003914833068847656, 0.0039217472076416016, 0.0039212703704833984, 0.0039386749267578125, 0.0038826465606689453, 0.003900766372680664, 0.003939151763916016, 0.003926515579223633, 0.003904104232788086, 0.003938198089599609, 0.0038988590240478516, 0.0039174556732177734, 0.003930807113647461, 0.003885984420776367, 0.003919124603271484, 0.003914356231689453, 0.00392603874206543, 0.003922462463378906, 0.003882169723510742, 0.003919839859008789, 0.003887653350830078, 0.003994464874267578, 0.003938913345336914, 0.0038797855377197266, 0.003986358642578125, 0.003949403762817383, 0.003972530364990234, 0.003923177719116211, 0.0038788318634033203, 0.003914833068847656, 0.003893136978149414, 0.003977775573730469, 0.004009246826171875, 0.003900289535522461, 0.0039479732513427734, 0.0038826465606689453, 0.003923654556274414, 0.003983497619628906, 0.003884553909301758, 0.004093170166015625, 0.0039048194885253906, 0.0039331912994384766, 0.004029989242553711, 0.003888845443725586, 0.003924131393432617, 0.003908872604370117, 0.003931522369384766, 0.003938198089599609, 0.003968477249145508, 0.0039370059967041016, 0.003991603851318359, 0.003938913345336914, 0.003948211669921875, 0.0039594173431396484, 0.0039501190185546875, 0.0038928985595703125, 0.0040471553802490234, 0.003962039947509766, 0.004221916198730469, 0.003957033157348633, 0.0038971900939941406, 0.003932952880859375, 0.0039479732513427734, 0.003927469253540039, 0.003977537155151367, 0.0039031505584716797, 0.003952503204345703, 0.003961086273193359, 0.0039119720458984375, 0.00394892692565918, 0.003912925720214844, 0.0039348602294921875, 0.003925800323486328, 0.003945112228393555, 0.0038673877716064453, 0.0039746761322021484, 0.003929615020751953, 0.0039403438568115234, 0.0039141178131103516, 0.003934621810913086, 0.003943443298339844, 0.003963470458984375, 0.004044294357299805, 0.0039215087890625, 0.0038940906524658203, 0.003920316696166992, 0.004093170166015625, 0.003939151763916016, 0.003925800323486328, 0.003903627395629883, 0.0039212703704833984, 0.003935813903808594, 0.0039331912994384766, 0.0039403438568115234, 0.003912210464477539, 0.003913164138793945, 0.0038864612579345703, 0.0039784908294677734, 0.003928661346435547, 0.003897428512573242, 0.004041910171508789, 0.00391387939453125, 0.004004716873168945, 0.003922224044799805, 0.0038764476776123047, 0.00393223762512207, 0.0038826465606689453, 0.003926992416381836, 0.003964900970458984, 0.0038840770721435547, 0.003942012786865234, 0.0038917064666748047, 0.00392913818359375, 0.003941535949707031, 0.003947257995605469, 0.003931999206542969, 0.004265785217285156, 0.003954410552978516, 0.003954648971557617, 0.004006624221801758, 0.003941059112548828, 0.0038967132568359375, 0.003934144973754883, 0.0039293766021728516, 0.003946542739868164, 0.0039403438568115234, 0.0038847923278808594, 0.003934621810913086, 0.003931999206542969, 0.003938436508178711, 0.0039348602294921875, 0.0038726329803466797, 0.003942966461181641, 0.0040242671966552734, 0.0039272308349609375, 0.003953456878662109, 0.003917217254638672, 0.00394129753112793, 0.003933429718017578, 0.003928184509277344, 0.003885507583618164, 0.0039517879486083984, 0.003945827484130859, 0.003924369812011719, 0.003920078277587891, 0.003875732421875, 0.0042591094970703125, 0.003968477249145508, 0.003930330276489258, 0.003933906555175781, 0.0038683414459228516, 0.00433039665222168, 0.004010677337646484, 0.0039215087890625, 0.003918886184692383, 0.003929853439331055, 0.003952503204345703, 0.0039408206939697266, 0.0038750171661376953, 0.003919839859008789, 0.0039310455322265625, 0.003939628601074219, 0.003945589065551758, 0.003892660140991211, 0.003923177719116211, 0.004000186920166016, 0.003942012786865234, 0.003954887390136719, 0.0038933753967285156, 0.003920555114746094, 0.00391697883605957, 0.003921985626220703, 0.003906965255737305, 0.003962039947509766, 0.003949642181396484, 0.00397038459777832, 0.003964424133300781, 0.003916263580322266, 0.003935337066650391, 0.0039424896240234375, 0.003937482833862305, 0.003947734832763672, 0.003925323486328125, 0.004021167755126953, 0.00397038459777832, 0.003953218460083008, 0.003963470458984375, 0.003916501998901367, 0.00394749641418457, 0.003991127014160156, 0.003926992416381836, 0.003933906555175781, 0.0039033889770507812, 0.003939628601074219, 0.003919363021850586, 0.00416254997253418, 0.00394439697265625, 0.00391697883605957, 0.003952980041503906, 0.003879070281982422, 0.003985404968261719, 0.003945112228393555, 0.0040171146392822266, 0.003944873809814453, 0.003889799118041992, 0.003973484039306641, 0.003945589065551758, 0.0039103031158447266, 0.0040073394775390625, 0.003900289535522461, 0.003935337066650391, 0.003997087478637695, 0.0039234161376953125, 0.003941059112548828, 0.0038874149322509766, 0.0039255619049072266, 0.004025697708129883, 0.003928661346435547, 0.00392460823059082, 0.0038928985595703125, 0.003939628601074219, 0.004014730453491211, 0.003968238830566406, 0.00395655632019043, 0.0039103031158447266, 0.003931999206542969, 0.00394129753112793, 0.003941774368286133, 0.0039370059967041016, 0.003899097442626953, 0.003944873809814453, 0.0039501190185546875, 0.003934144973754883, 0.003957033157348633, 0.003993511199951172, 0.003934144973754883, 0.003949642181396484, 0.00393366813659668, 0.003954887390136719, 0.003915309906005859, 0.003936767578125, 0.003961801528930664, 0.003954410552978516, 0.003914833068847656, 0.003954648971557617, 0.003934383392333984, 0.003947257995605469, 0.003962039947509766, 0.00394892692565918, 0.003957033157348633, 0.003939628601074219, 0.0039560794830322266, 0.00394892692565918, 0.0039865970611572266, 0.00400853157043457, 0.003931760787963867, 0.003973960876464844, 0.003966569900512695, 0.0039060115814208984, 0.00393223762512207, 0.00400090217590332, 0.00397181510925293, 0.00395512580871582, 0.00391840934753418, 0.003947734832763672, 0.003936767578125, 0.00394439697265625, 0.00395655632019043, 0.003916263580322266, 0.003939390182495117, 0.0039033889770507812, 0.003998756408691406, 0.003958940505981445, 0.004019737243652344, 0.003953218460083008, 0.003932476043701172, 0.004019975662231445, 0.004008769989013672, 0.003922939300537109, 0.003962516784667969, 0.003902435302734375, 0.004004955291748047, 0.003975629806518555, 0.003928661346435547, 0.003943443298339844, 0.004011631011962891, 0.004000663757324219, 0.0039556026458740234, 0.003918170928955078, 0.0039424896240234375, 0.003891468048095703, 0.0039637088775634766, 0.0041539669036865234, 0.003935098648071289, 0.003954410552978516, 0.003905057907104492, 0.003942251205444336, 0.004014492034912109, 0.00392913818359375, 0.003952980041503906, 0.003934144973754883, 0.003968954086303711, 0.004027128219604492, 0.004014253616333008, 0.003975629806518555, 0.003949880599975586, 0.003962039947509766, 0.004004001617431641, 0.003912687301635742, 0.003997802734375, 0.0039234161376953125, 0.0041713714599609375, 0.004040718078613281, 0.003939628601074219, 0.0039632320404052734, 0.003960132598876953, 0.0039501190185546875, 0.003969907760620117, 0.003956794738769531, 0.003950357437133789, 0.00397038459777832, 0.0039637088775634766, 0.003927946090698242, 0.0039691925048828125, 0.003970623016357422, 0.003947019577026367, 0.0039825439453125, 0.003916263580322266, 0.003955364227294922, 0.003946542739868164, 0.003964662551879883, 0.003968477249145508, 0.003911256790161133, 0.0039517879486083984, 0.003981828689575195, 0.004052162170410156, 0.0039865970611572266, 0.003909111022949219, 0.003965616226196289, 0.0039615631103515625, 0.003983259201049805, 0.003950595855712891, 0.003941774368286133, 0.00395512580871582, 0.0039594173431396484, 0.003957271575927734, 0.0039632320404052734, 0.003938436508178711, 0.003953695297241211, 0.0039234161376953125, 0.004007101058959961, 0.003954172134399414, 0.003924131393432617, 0.0039823055267333984, 0.0039937496185302734, 0.0040149688720703125, 0.003955364227294922, 0.003930091857910156, 0.0039598941802978516, 0.003936052322387695, 0.004000186920166016, 0.0039768218994140625, 0.003944873809814453, 0.003970623016357422, 0.003926992416381836, 0.003999233245849609, 0.00396418571472168, 0.003929853439331055, 0.0039784908294677734, 0.003918886184692383, 0.003952741622924805, 0.004029750823974609, 0.0039479732513427734, 0.003989696502685547, 0.003928661346435547, 0.003960609436035156, 0.004033327102661133, 0.003935575485229492, 0.003955841064453125, 0.003931760787963867, 0.00396275520324707, 0.004004716873168945, 0.003935098648071289, 0.003960371017456055, 0.0039250850677490234, 0.003962516784667969, 0.004170656204223633, 0.003930807113647461, 0.00395965576171875, 0.003919363021850586, 0.00395655632019043, 0.003976583480834961, 0.003984689712524414, 0.003952741622924805, 0.003922224044799805, 0.0039713382720947266, 0.003977775573730469, 0.003980875015258789, 0.003971576690673828, 0.003980159759521484, 0.003977537155151367, 0.0039746761322021484, 0.003977298736572266, 0.003980398178100586, 0.004018068313598633, 0.003979921340942383, 0.003995418548583984, 0.0039691925048828125, 0.003962993621826172, 0.003931999206542969, 0.003952503204345703, 0.0039997100830078125, 0.003965616226196289, 0.003970623016357422, 0.003910064697265625, 0.003959178924560547, 0.003978252410888672, 0.0039746761322021484, 0.003979206085205078, 0.00392603874206543, 0.003996849060058594, 0.003980159759521484, 0.0039708614349365234, 0.003969430923461914, 0.003945827484130859, 0.003983974456787109, 0.003998756408691406, 0.0040149688720703125, 0.003968238830566406, 0.003931283950805664, 0.00397944450378418, 0.00398707389831543, 0.003977298736572266, 0.003911256790161133, 0.0039942264556884766, 0.0040209293365478516, 0.0040814876556396484, 0.003985166549682617, 0.003994464874267578, 0.00397944450378418, 0.00397038459777832, 0.0040132999420166016, 0.004015684127807617, 0.003951311111450195, 0.003981113433837891, 0.003968000411987305, 0.0039670467376708984, 0.00401759147644043, 0.0039234161376953125, 0.0040280818939208984, 0.003966808319091797, 0.003980159759521484, 0.0039865970611572266, 0.00391697883605957, 0.00397944450378418, 0.0040740966796875, 0.004012107849121094, 0.004096508026123047, 0.004071712493896484, 0.004025936126708984, 0.003996610641479492, 0.003996849060058594, 0.003973960876464844, 0.003931522369384766, 0.003966093063354492, 0.003976583480834961, 0.003974199295043945, 0.003998994827270508, 0.0039424896240234375, 0.003987550735473633, 0.0039768218994140625, 0.003997087478637695, 0.003986835479736328, 0.003975629806518555, 0.004010200500488281, 0.00396728515625, 0.003985166549682617, 0.003966808319091797, 0.003937244415283203, 0.003966808319091797, 0.003999233245849609, 0.003976583480834961, 0.003973245620727539, 0.003952741622924805, 0.0039827823638916016, 0.003985404968261719, 0.004061698913574219, 0.003979682922363281, 0.003945589065551758, 0.003968954086303711, 0.004006147384643555, 0.003975391387939453, 0.0040090084075927734, 0.003933429718017578, 0.003988742828369141, 0.003988027572631836, 0.004007101058959961, 0.0039865970611572266, 0.003987550735473633, 0.0039708614349365234, 0.00399327278137207, 0.00400090217590332, 0.003993988037109375, 0.0039196014404296875, 0.003983497619628906, 0.0040683746337890625, 0.004006624221801758, 0.00399327278137207, 0.0039844512939453125, 0.0039882659912109375, 0.004080295562744141, 0.003999471664428711, 0.004012346267700195, 0.0039331912994384766, 0.003975391387939453, 0.0039784908294677734, 0.00398564338684082, 0.003973960876464844, 0.00395512580871582, 0.003960609436035156, 0.003979921340942383, 0.003987789154052734, 0.003998279571533203, 0.003951311111450195, 0.0039827823638916016, 0.0040018558502197266, 0.003983259201049805, 0.003989219665527344, 0.0039517879486083984, 0.004026651382446289, 0.003991842269897461, 0.0040302276611328125, 0.004014015197753906, 0.003943920135498047, 0.003984928131103516, 0.003937244415283203, 0.004111766815185547, 0.0040130615234375, 0.0039370059967041016, 0.003975868225097656, 0.003948211669921875, 0.004061698913574219, 0.00397181510925293, 0.003950595855712891, 0.004153728485107422, 0.0040111541748046875, 0.004014730453491211, 0.003988027572631836, 0.003966808319091797, 0.003972530364990234, 0.003995180130004883, 0.0039975643157958984, 0.004021644592285156, 0.003952980041503906, 0.0039958953857421875, 0.00410151481628418, 0.00399017333984375, 0.004036903381347656, 0.00395655632019043, 0.003994941711425781, 0.0040950775146484375, 0.004160881042480469, 0.004012107849121094, 0.003991603851318359, 0.0040090084075927734, 0.00400996208190918, 0.003999471664428711, 0.004010677337646484, 0.00394439697265625, 0.003978252410888672, 0.004003047943115234, 0.004019737243652344, 0.00397944450378418, 0.003947257995605469, 0.0039980411529541016, 0.0039937496185302734, 0.004006624221801758, 0.004018545150756836, 0.003941774368286133, 0.003988742828369141, 0.003991365432739258, 0.004018068313598633, 0.003995180130004883, 0.003960609436035156, 0.004007577896118164, 0.003999233245849609, 0.004100322723388672, 0.004046201705932617, 0.003966808319091797, 0.003993511199951172, 0.004011869430541992, 0.003996372222900391, 0.004018068313598633, 0.003985881805419922, 0.003981590270996094, 0.004004240036010742, 0.0040283203125, 0.0039899349212646484, 0.003984928131103516, 0.00399470329284668, 0.004050016403198242, 0.0039899349212646484, 0.003981828689575195, 0.0039463043212890625, 0.003998994827270508, 0.0041081905364990234, 0.00400853157043457, 0.003996849060058594, 0.0039670467376708984, 0.003995418548583984, 0.00399327278137207, 0.0040056705474853516, 0.004005908966064453, 0.0039825439453125, 0.003981113433837891, 0.004008293151855469, 0.004023551940917969, 0.0040209293365478516, 0.003957509994506836, 0.004168272018432617, 0.004077911376953125, 0.003991842269897461, 0.003987312316894531, 0.003971099853515625, 0.003981590270996094, 0.004014253616333008, 0.004011631011962891, 0.004174470901489258, 0.004028797149658203, 0.0040035247802734375, 0.003998994827270508, 0.003996610641479492, 0.004021883010864258, 0.003964424133300781, 0.0040130615234375, 0.004012107849121094, 0.004084587097167969, 0.004017829895019531, 0.0039730072021484375, 0.003981113433837891, 0.004000425338745117, 0.0040171146392822266, 0.004003763198852539, 0.003953218460083008, 0.00400233268737793, 0.00400233268737793, 0.003994941711425781, 0.0040051937103271484, 0.003950595855712891, 0.004011631011962891, 0.003997087478637695, 0.00399470329284668, 0.003983497619628906, 0.003943681716918945, 0.0040225982666015625, 0.004143953323364258, 0.004000425338745117, 0.004010677337646484, 0.003954887390136719, 0.004017829895019531, 0.004006147384643555, 0.004010438919067383, 0.0040073394775390625, 0.0039708614349365234, 0.00400233268737793, 0.003992319107055664, 0.0040130615234375, 0.00400853157043457, 0.0039784908294677734, 0.004101753234863281, 0.003993988037109375, 0.003996849060058594, 0.004105091094970703, 0.0041158199310302734, 0.004050254821777344, 0.0040323734283447266, 0.004049062728881836, 0.004055500030517578, 0.004029750823974609, 0.004086017608642578, 0.004203081130981445, 0.004245281219482422, 0.004271268844604492, 0.0042667388916015625, 0.004235744476318359, 0.0041217803955078125, 0.004204988479614258, 0.004035234451293945, 0.004073619842529297, 0.004012346267700195, 0.004053354263305664, 0.004675388336181641, 0.00405573844909668, 0.004294633865356445, 0.004051923751831055, 0.004002571105957031, 0.004029750823974609, 0.003960847854614258, 0.003995656967163086, 0.004021883010864258, 0.004029512405395508, 0.0040073394775390625, 0.003980159759521484, 0.004007577896118164, 0.0041179656982421875, 0.004025936126708984, 0.004020214080810547, 0.003990650177001953, 0.004013538360595703, 0.004423618316650391, 0.0041043758392333984, 0.004011869430541992, 0.0039861202239990234, 0.004038572311401367, 0.004014492034912109, 0.0040056705474853516, 0.004019260406494141, 0.003982067108154297, 0.004034519195556641, 0.004001617431640625, 0.004019260406494141, 0.004012346267700195, 0.004084587097167969, 0.004052400588989258, 0.0039975643157958984, 0.00402379035949707, 0.004016876220703125, 0.003969430923461914, 0.004023075103759766, 0.004006385803222656, 0.004000186920166016, 0.004021167755126953, 0.0039751529693603516, 0.004038810729980469, 0.0040225982666015625, 0.004120349884033203, 0.004034280776977539, 0.004054069519042969, 0.003976106643676758, 0.003984689712524414, 0.004018545150756836, 0.004001140594482422, 0.004081249237060547, 0.00395512580871582, 0.004004001617431641, 0.004012584686279297, 0.00403594970703125, 0.004027605056762695, 0.003980398178100586, 0.0040166378021240234, 0.0040128231048583984, 0.004007101058959961, 0.004020214080810547, 0.0039899349212646484, 0.004147529602050781, 0.0040378570556640625, 0.003975391387939453, 0.0040705204010009766, 0.004016876220703125, 0.003953218460083008, 0.0040166378021240234, 0.00397491455078125, 0.004111289978027344, 0.004019737243652344, 0.003963947296142578, 0.004025459289550781, 0.003992795944213867, 0.004063129425048828, 0.0040132999420166016, 0.00397181510925293, 0.0040187835693359375, 0.003981351852416992, 0.004063606262207031, 0.004018068313598633, 0.00397038459777832, 0.004036426544189453, 0.004037618637084961, 0.00402069091796875, 0.004058361053466797, 0.0039751529693603516, 0.003995656967163086, 0.004040956497192383, 0.004015207290649414, 0.0039975643157958984, 0.00397801399230957, 0.00409698486328125, 0.00404667854309082, 0.004029512405395508, 0.003987550735473633, 0.003966093063354492, 0.004015684127807617, 0.004033327102661133, 0.004161834716796875, 0.004034996032714844, 0.003962516784667969, 0.004015445709228516, 0.0040590763092041016, 0.003995656967163086, 0.004021406173706055, 0.0039844512939453125, 0.004027128219604492, 0.004060268402099609, 0.003987550735473633, 0.004029512405395508, 0.004130840301513672, 0.004076242446899414, 0.004022836685180664, 0.003994464874267578, 0.0040323734283447266, 0.004012584686279297, 0.004339933395385742, 0.004082441329956055, 0.004150390625, 0.0042002201080322266, 0.004201412200927734, 0.004189968109130859, 0.004230022430419922, 0.004233598709106445, 0.0042073726654052734, 0.00420069694519043, 0.004189491271972656, 0.004213809967041016, 0.0041942596435546875, 0.004202365875244141, 0.004240751266479492, 0.00420689582824707, 0.004479169845581055, 0.004047393798828125, 0.004031658172607422, 0.00409245491027832, 0.004040241241455078, 0.004036664962768555, 0.00404810905456543, 0.00411677360534668, 0.004064321517944336, 0.004026889801025391, 0.003997325897216797, 0.004031181335449219, 0.00410008430480957, 0.004004716873168945, 0.0040149688720703125, 0.003998994827270508, 0.0040471553802490234, 0.004058361053466797, 0.003996372222900391, 0.0040225982666015625, 0.003988981246948242, 0.004021883010864258, 0.004083156585693359, 0.0039861202239990234, 0.004048585891723633, 0.004189252853393555, 0.00403141975402832, 0.004256725311279297, 0.003999233245849609, 0.0040395259857177734, 0.004055976867675781, 0.004098176956176758, 0.004018068313598633, 0.004061698913574219, 0.004015684127807617, 0.004072904586791992, 0.0040607452392578125, 0.004027128219604492, 0.004030466079711914, 0.003985881805419922, 0.00404047966003418, 0.004055500030517578, 0.004041910171508789, 0.004030942916870117, 0.0040705204010009766, 0.00404047966003418, 0.00405120849609375, 0.00401759147644043, 0.004037141799926758, 0.004001617431640625, 0.004050493240356445, 0.004060983657836914, 0.004050016403198242, 0.004046201705932617, 0.00399470329284668, 0.004052639007568359, 0.004041910171508789, 0.00413823127746582, 0.0040340423583984375, 0.004044294357299805, 0.004011392593383789, 0.004034519195556641, 0.004040956497192383, 0.004044532775878906, 0.004036664962768555, 0.003983974456787109, 0.004032135009765625, 0.004042625427246094, 0.004018306732177734, 0.004018068313598633, 0.0047533512115478516, 0.004047393798828125, 0.004030466079711914, 0.004045724868774414, 0.004487752914428711, 0.004006385803222656, 0.0043277740478515625, 0.004059553146362305, 0.003989458084106445, 0.004047393798828125, 0.00397801399230957, 0.004090070724487305, 0.004031181335449219, 0.003993511199951172, 0.004033565521240234, 0.003990650177001953, 0.0040853023529052734, 0.004032135009765625, 0.003989458084106445, 0.0040433406829833984, 0.004024505615234375, 0.0041081905364990234, 0.0040662288665771484, 0.004001617431640625, 0.004052877426147461, 0.00403904914855957, 0.004058837890625, 0.004060268402099609, 0.004007577896118164, 0.004034519195556641, 0.004049062728881836, 0.004033803939819336, 0.004017353057861328, 0.003998517990112305, 0.004012107849121094, 0.004078865051269531, 0.0041942596435546875, 0.004133939743041992, 0.004057407379150391, 0.0041921138763427734, 0.004148006439208984, 0.00413966178894043, 0.004163503646850586, 0.004150867462158203, 0.004083156585693359, 0.0041353702545166016, 0.004134416580200195, 0.0045549869537353516, 0.004138946533203125, 0.004157304763793945, 0.00414276123046875, 0.004604816436767578, 0.004239559173583984, 0.004115104675292969, 0.004120588302612305, 0.004227638244628906, 0.004114389419555664, 0.00407099723815918, 0.0041005611419677734, 0.004129171371459961, 0.004111051559448242, 0.004083395004272461, 0.0041179656982421875, 0.004148006439208984, 0.004078388214111328, 0.00413203239440918, 0.004072666168212891, 0.004192829132080078, 0.004080295562744141, 0.004067420959472656, 0.0041351318359375, 0.004089832305908203, 0.004128456115722656, 0.0040895938873291016, 0.004075765609741211, 0.0041103363037109375, 0.00412440299987793, 0.004086732864379883, 0.0041141510009765625, 0.004061698913574219, 0.004134654998779297, 0.004105091094970703, 0.004108428955078125, 0.004106760025024414, 0.004193782806396484, 0.004086732864379883, 0.00410008430480957, 0.004094123840332031, 0.0041120052337646484, 0.004111766815185547, 0.0040628910064697266, 0.0041239261627197266, 0.004099130630493164, 0.0041120052337646484, 0.0040929317474365234, 0.004063606262207031, 0.004111289978027344, 0.004184722900390625, 0.004070281982421875, 0.004103422164916992, 0.004064798355102539, 0.004113197326660156, 0.004163026809692383, 0.004121065139770508, 0.0041484832763671875, 0.004082441329956055, 0.004179477691650391, 0.004109621047973633, 0.004060506820678711, 0.0041351318359375, 0.004221200942993164, 0.004098415374755859, 0.0041086673736572266, 0.0040798187255859375, 0.004116535186767578, 0.004136562347412109, 0.004113912582397461, 0.004106044769287109, 0.0040569305419921875, 0.004124879837036133, 0.004113197326660156, 0.004135608673095703, 0.0040967464447021484, 0.004130840301513672, 0.004084348678588867, 0.004147768020629883, 0.004117488861083984, 0.004172086715698242, 0.004107952117919922, 0.00408172607421875, 0.0041048526763916016, 0.0041692256927490234, 0.004073619842529297, 0.004192352294921875, 0.0040700435638427734, 0.004123210906982422, 0.0041849613189697266, 0.004069328308105469, 0.0040929317474365234, 0.0040781497955322266, 0.004334688186645508, 0.00415492057800293, 0.0040662288665771484, 0.004085540771484375, 0.004119157791137695, 0.0041124820709228516, 0.004124164581298828, 0.004064321517944336, 0.004112720489501953, 0.0041141510009765625, 0.004132986068725586, 0.004130840301513672, 0.0042266845703125, 0.004082202911376953, 0.004123687744140625, 0.0041179656982421875, 0.0042150020599365234, 0.004213809967041016, 0.004098653793334961, 0.0041391849517822266, 0.004131317138671875, 0.0041387081146240234, 0.004114389419555664, 0.004081249237060547, 0.004110097885131836, 0.004167318344116211, 0.004099130630493164, 0.004111289978027344, 0.004060506820678711, 0.004163265228271484, 0.004146099090576172, 0.004105329513549805, 0.004151105880737305, 0.00410914421081543, 0.0041599273681640625, 0.004111051559448242, 0.0041046142578125, 0.004114389419555664, 0.004120826721191406, 0.004099130630493164, 0.004134178161621094, 0.004149198532104492, 0.004160404205322266, 0.004128217697143555, 0.0041217803955078125, 0.004137516021728516, 0.0041387081146240234, 0.004090309143066406, 0.004122018814086914, 0.0041351318359375, 0.004141330718994141, 0.004134178161621094, 0.00408625602722168, 0.004118680953979492, 0.004152774810791016, 0.004107236862182617, 0.004118919372558594, 0.0040798187255859375, 0.004120349884033203, 0.004200935363769531, 0.004087924957275391, 0.004215240478515625, 0.004095792770385742, 0.004164934158325195, 0.004134654998779297, 0.004088163375854492, 0.004129171371459961, 0.004122018814086914, 0.004130363464355469, 0.004110813140869141, 0.0040798187255859375, 0.004183053970336914, 0.004144906997680664, 0.00412750244140625, 0.004152059555053711, 0.004138946533203125, 0.004091739654541016, 0.004113435745239258, 0.0041217803955078125, 0.004149675369262695, 0.004147052764892578, 0.004309654235839844, 0.004191160202026367, 0.00423121452331543, 0.0040967464447021484, 0.004166841506958008, 0.004112958908081055, 0.004178762435913086, 0.004142045974731445, 0.0040721893310546875, 0.00424504280090332, 0.0041351318359375, 0.0041615962982177734, 0.0041353702545166016, 0.004097938537597656, 0.004136085510253906, 0.004159688949584961, 0.0041234493255615234, 0.004137516021728516, 0.004100322723388672, 0.004153728485107422, 0.0041277408599853516, 0.004175424575805664, 0.004126071929931641, 0.004143714904785156, 0.0041272640228271484, 0.004145622253417969, 0.004147052764892578, 0.004136800765991211, 0.004237174987792969, 0.0041141510009765625, 0.004133939743041992, 0.004212141036987305, 0.004086732864379883, 0.004123210906982422, 0.004109382629394531, 0.004208564758300781, 0.004133939743041992, 0.004100799560546875, 0.004148721694946289, 0.004144430160522461, 0.0041561126708984375, 0.004186868667602539, 0.004102945327758789, 0.004155397415161133, 0.004157304763793945, 0.00414276123046875, 0.0041544437408447266, 0.004086017608642578, 0.004179239273071289, 0.004112720489501953, 0.004141330718994141, 0.004160165786743164, 0.004159450531005859, 0.0041315555572509766, 0.004131317138671875, 0.004150867462158203, 0.0041713714599609375, 0.0042722225189208984, 0.004115104675292969, 0.0041582584381103516, 0.004213809967041016, 0.004115104675292969, 0.004140615463256836, 0.004151582717895508, 0.0041959285736083984, 0.004185914993286133, 0.004113912582397461, 0.004184722900390625, 0.004188060760498047, 0.004172801971435547, 0.004168510437011719, 0.004118919372558594, 0.004163503646850586, 0.004129886627197266, 0.004174470901489258, 0.004147052764892578, 0.004240751266479492, 0.004117250442504883, 0.004144906997680664, 0.004140138626098633, 0.004228115081787109, 0.0041446685791015625, 0.004110574722290039, 0.0041201114654541016, 0.004179477691650391, 0.004105567932128906, 0.004140377044677734, 0.004099130630493164, 0.0041961669921875, 0.004164457321166992, 0.004099607467651367, 0.004179477691650391, 0.0041582584381103516, 0.004151821136474609, 0.004178047180175781, 0.004131793975830078, 0.004182577133178711, 0.004143476486206055, 0.004145622253417969, 0.004143953323364258, 0.004168987274169922, 0.0041162967681884766, 0.00411534309387207, 0.004159450531005859, 0.0041522979736328125, 0.004220485687255859, 0.004110813140869141, 0.004148244857788086, 0.004190921783447266, 0.004101753234863281, 0.004145145416259766, 0.004113197326660156, 0.004133462905883789, 0.004208087921142578, 0.0040929317474365234, 0.0041675567626953125, 0.0041599273681640625, 0.004216432571411133, 0.00414586067199707, 0.004102230072021484, 0.004149198532104492, 0.0041599273681640625, 0.004153013229370117, 0.004147768020629883, 0.004110813140869141, 0.004357814788818359, 0.0041005611419677734, 0.00413966178894043, 0.004153013229370117, 0.004176139831542969, 0.0041522979736328125, 0.004105806350708008, 0.004128694534301758, 0.00422215461730957, 0.004120349884033203, 0.004141569137573242, 0.004111528396606445, 0.004174470901489258, 0.004163026809692383, 0.004123687744140625, 0.0041577816009521484, 0.004150867462158203, 0.004225015640258789, 0.004143714904785156, 0.004189491271972656, 0.0041539669036865234, 0.0041162967681884766, 0.004175901412963867, 0.004159450531005859, 0.004208803176879883, 0.004118680953979492, 0.004158973693847656, 0.004098653793334961, 0.004309892654418945, 0.0041806697845458984, 0.0041179656982421875, 0.004141330718994141, 0.00415349006652832, 0.004160165786743164, 0.004173994064331055, 0.004113197326660156, 0.004333972930908203, 0.004169940948486328, 0.004140615463256836, 0.0041582584381103516, 0.004159688949584961, 0.0041201114654541016, 0.004149198532104492, 0.004149913787841797, 0.0041961669921875, 0.004143953323364258, 0.004128694534301758, 0.004146099090576172, 0.004287242889404297, 0.004127025604248047, 0.0041658878326416016, 0.004136800765991211, 0.004147529602050781, 0.004482269287109375, 0.004139900207519531, 0.00419306755065918, 0.004213809967041016, 0.004135608673095703, 0.004144191741943359, 0.004151344299316406, 0.004225015640258789, 0.00416111946105957, 0.00411534309387207, 0.004175901412963867, 0.0041882991790771484, 0.004178524017333984, 0.0041637420654296875, 0.004127979278564453, 0.0041544437408447266, 0.004149436950683594, 0.0041806697845458984, 0.004175424575805664, 0.0041599273681640625, 0.00410151481628418, 0.004174232482910156, 0.004161834716796875, 0.0042536258697509766, 0.004178285598754883, 0.0041353702545166016, 0.0041904449462890625, 0.004240989685058594, 0.004132270812988281, 0.0041751861572265625, 0.004116058349609375, 0.004203081130981445, 0.004181861877441406, 0.004120349884033203, 0.004147052764892578, 0.0041882991790771484, 0.004185199737548828, 0.004159212112426758, 0.004118919372558594, 0.004167318344116211, 0.004139900207519531, 0.004166126251220703, 0.00417017936706543, 0.004277229309082031, 0.004147052764892578, 0.004154205322265625, 0.0041730403900146484, 0.0041887760162353516, 0.004179477691650391, 0.004201412200927734, 0.004180431365966797, 0.004181861877441406, 0.004137992858886719, 0.004155635833740234, 0.004137516021728516, 0.004183769226074219, 0.004189729690551758, 0.0041735172271728516, 0.00417637825012207, 0.004169464111328125, 0.00410914421081543, 0.00417637825012207, 0.004152774810791016, 0.004191398620605469, 0.0041656494140625, 0.004124879837036133, 0.0041620731353759766, 0.004232168197631836, 0.004109859466552734, 0.0041620731353759766, 0.004134178161621094, 0.004315614700317383, 0.004183769226074219, 0.00411224365234375, 0.004147768020629883, 0.004167795181274414, 0.004151105880737305, 0.004179954528808594, 0.004110574722290039, 0.0041768550872802734, 0.004189729690551758, 0.004177570343017578, 0.004187107086181641, 0.004163265228271484, 0.004159688949584961, 0.004172801971435547, 0.0041620731353759766, 0.0041811466217041016, 0.004161834716796875, 0.004128456115722656, 0.004163503646850586, 0.004311800003051758, 0.004143953323364258, 0.004168033599853516, 0.004184722900390625, 0.0041675567626953125, 0.004191875457763672, 0.004119396209716797, 0.004190683364868164, 0.004157304763793945, 0.0041925907135009766, 0.004181861877441406, 0.00418543815612793, 0.004129648208618164, 0.0041768550872802734, 0.004159688949584961, 0.0041844844818115234, 0.004168987274169922, 0.004146575927734375, 0.00417327880859375, 0.004182577133178711, 0.00413823127746582, 0.0041790008544921875, 0.0041158199310302734, 0.004233360290527344, 0.004154205322265625, 0.004160404205322266, 0.004165172576904297, 0.004247188568115234, 0.004279375076293945, 0.0041468143463134766, 0.004115581512451172, 0.004198789596557617, 0.004170894622802734, 0.00489497184753418, 0.004273653030395508, 0.004182100296020508, 0.004202842712402344, 0.004168033599853516, 0.0042226314544677734, 0.00439763069152832, 0.004228830337524414, 0.0042345523834228516, 0.004217624664306641, 0.004174947738647461, 0.004193782806396484, 0.004169940948486328, 0.004198312759399414, 0.004296541213989258, 0.004221677780151367, 0.004259586334228516, 0.004181385040283203, 0.004208087921142578, 0.004268646240234375, 0.004313468933105469, 0.004183053970336914, 0.004187583923339844, 0.004180192947387695, 0.004216670989990234, 0.004191398620605469, 0.004198789596557617, 0.004178524017333984, 0.004208803176879883, 0.004172563552856445, 0.004193782806396484, 0.004220485687255859, 0.004198789596557617, 0.0041506290435791016, 0.004201650619506836, 0.004209756851196289, 0.0041468143463134766, 0.0041599273681640625, 0.004136085510253906, 0.004233121871948242, 0.004186391830444336, 0.004122018814086914, 0.004194736480712891, 0.00429081916809082, 0.004185676574707031, 0.004205465316772461, 0.004164695739746094, 0.004175901412963867, 0.004206180572509766, 0.004168033599853516, 0.004182100296020508, 0.004191160202026367, 0.004145622253417969, 0.004204511642456055, 0.004155635833740234, 0.0042247772216796875, 0.004147529602050781, 0.004142045974731445, 0.004206418991088867, 0.004225492477416992, 0.00416874885559082, 0.004180431365966797, 0.004226207733154297, 0.004256248474121094, 0.004215717315673828, 0.004156589508056641, 0.004143238067626953, 0.004200458526611328, 0.004184722900390625, 0.004186391830444336, 0.004114389419555664, 0.004176139831542969, 0.004177093505859375, 0.00418543815612793, 0.004210948944091797, 0.00420379638671875, 0.004143238067626953, 0.004168272018432617, 0.004288196563720703, 0.004151821136474609, 0.004158735275268555, 0.004124641418457031, 0.004243612289428711, 0.005023002624511719, 0.0044422149658203125, 0.004277944564819336, 0.004202365875244141, 0.004481792449951172, 0.0042531490325927734, 0.004215717315673828, 0.004268646240234375, 0.004426240921020508, 0.004262685775756836, 0.004210233688354492, 0.0042536258697509766, 0.004160165786743164, 0.004228353500366211, 0.004373073577880859, 0.004307270050048828, 0.0041980743408203125, 0.004201412200927734, 0.00414276123046875, 0.0042324066162109375, 0.004268169403076172, 0.0041522979736328125, 0.004179954528808594, 0.004222869873046875, 0.004271030426025391, 0.0042035579681396484, 0.004225254058837891, 0.004376649856567383, 0.004216909408569336, 0.004191398620605469, 0.004316568374633789, 0.004339694976806641, 0.00415802001953125, 0.0041730403900146484, 0.0042018890380859375, 0.004286289215087891, 0.0042078495025634766, 0.0041446685791015625, 0.004171133041381836, 0.0043222904205322266, 0.004181623458862305, 0.004209280014038086, 0.004220008850097656, 0.004263877868652344, 0.004190921783447266, 0.0043506622314453125, 0.004214763641357422, 0.004300832748413086, 0.004179954528808594, 0.004206657409667969, 0.004210472106933594, 0.004221200942993164, 0.004209041595458984, 0.004275083541870117, 0.004235029220581055, 0.004231452941894531, 0.004136085510253906, 0.004235506057739258, 0.00418543815612793, 0.004156827926635742, 0.004200935363769531, 0.004206657409667969, 0.004216670989990234, 0.004216432571411133, 0.004166841506958008, 0.004230976104736328, 0.004202365875244141, 0.004230022430419922, 0.004241466522216797, 0.004209041595458984, 0.004172801971435547, 0.004224061965942383, 0.004277944564819336, 0.0041654109954833984, 0.004269599914550781, 0.004168987274169922, 0.004258632659912109, 0.004192352294921875, 0.004155158996582031, 0.00422978401184082, 0.004206418991088867, 0.004218339920043945, 0.004195690155029297, 0.004158496856689453, 0.00423431396484375, 0.004194021224975586, 0.004190206527709961, 0.004187822341918945, 0.004207134246826172, 0.004229307174682617, 0.004184246063232422, 0.00420069694519043, 0.0042095184326171875, 0.004183530807495117, 0.0041811466217041016, 0.00426936149597168, 0.004212141036987305, 0.004144906997680664, 0.00420832633972168, 0.004208803176879883, 0.004207611083984375, 0.004189729690551758, 0.00424957275390625, 0.004246950149536133, 0.004225969314575195, 0.0042171478271484375, 0.004204511642456055, 0.004228115081787109, 0.0041620731353759766, 0.0042078495025634766, 0.004223346710205078, 0.004227638244628906, 0.00423121452331543, 0.004145622253417969, 0.0042955875396728516, 0.0042476654052734375, 0.00416874885559082, 0.004193305969238281, 0.004193544387817383, 0.00422358512878418, 0.004204511642456055, 0.00423431396484375, 0.0042302608489990234, 0.004225969314575195, 0.004210233688354492, 0.004205226898193359, 0.004214048385620117, 0.00417780876159668, 0.004206418991088867, 0.004281520843505859, 0.004175424575805664, 0.004210233688354492, 0.004158735275268555, 0.0042493343353271484, 0.004259586334228516, 0.004187822341918945, 0.004202365875244141, 0.004210710525512695, 0.004219770431518555, 0.00421452522277832, 0.004175662994384766, 0.004224061965942383, 0.004216432571411133, 0.004212379455566406, 0.004207134246826172, 0.004217624664306641, 0.004169464111328125, 0.00419926643371582, 0.004197597503662109, 0.004310131072998047, 0.004231691360473633, 0.0041790008544921875, 0.0042934417724609375, 0.004194498062133789, 0.0042226314544677734, 0.004248619079589844, 0.004203319549560547, 0.004206657409667969, 0.004179954528808594, 0.0041925907135009766, 0.004254341125488281, 0.004164695739746094, 0.004212379455566406, 0.004181861877441406, 0.004232883453369141, 0.004166841506958008, 0.004202127456665039, 0.004277944564819336, 0.004163503646850586, 0.004291057586669922, 0.004168987274169922, 0.004297733306884766, 0.004204988479614258, 0.0041539669036865234, 0.004185676574707031, 0.0042438507080078125, 0.004212379455566406, 0.00421142578125, 0.004181385040283203, 0.004255533218383789, 0.0042285919189453125, 0.004204511642456055, 0.004198312759399414, 0.004190206527709961, 0.004175901412963867, 0.004221916198730469, 0.004312038421630859, 0.0041887760162353516, 0.0041921138763427734, 0.004178762435913086, 0.0042726993560791016, 0.004220485687255859, 0.004194974899291992, 0.0042078495025634766, 0.004235029220581055, 0.004228353500366211, 0.004251718521118164, 0.004415035247802734, 0.004175424575805664, 0.004209995269775391, 0.0042040348052978516, 0.004215717315673828, 0.004185199737548828, 0.004199028015136719, 0.004214286804199219, 0.004261493682861328, 0.004177570343017578, 0.004199504852294922, 0.00416254997253418, 0.004260063171386719, 0.004224300384521484, 0.0041561126708984375, 0.004208564758300781, 0.004231929779052734, 0.004225492477416992, 0.00421142578125, 0.0042378902435302734, 0.004282236099243164, 0.004227161407470703, 0.0042150020599365234, 0.004235506057739258, 0.0042438507080078125, 0.004236698150634766, 0.004206657409667969, 0.0042819976806640625, 0.004176616668701172, 0.0041925907135009766, 0.004171133041381836, 0.00421905517578125, 0.0042095184326171875, 0.004168987274169922, 0.004251003265380859, 0.004197835922241211, 0.0042171478271484375, 0.004222869873046875, 0.00423121452331543, 0.004161357879638672, 0.004226207733154297, 0.0042078495025634766, 0.004275083541870117, 0.0042018890380859375, 0.004176616668701172, 0.004205942153930664, 0.0043087005615234375, 0.004279375076293945, 0.004230976104736328, 0.0042247772216796875, 0.004216670989990234, 0.004204988479614258, 0.00418400764465332, 0.004238128662109375, 0.004257678985595703, 0.0043659210205078125, 0.0042231082916259766, 0.004231929779052734, 0.0041904449462890625, 0.004203081130981445, 0.004240751266479492, 0.004194021224975586, 0.0042133331298828125, 0.004174709320068359, 0.004271268844604492, 0.004221916198730469, 0.004183769226074219, 0.004315853118896484, 0.004233360290527344, 0.00421905517578125, 0.004279136657714844, 0.004227638244628906, 0.004212379455566406, 0.004242897033691406, 0.0042192935943603516, 0.0042531490325927734, 0.004218339920043945, 0.0041964054107666016, 0.004212379455566406, 0.0042896270751953125, 0.004190921783447266, 0.004208803176879883, 0.004172086715698242, 0.004278898239135742, 0.004218578338623047, 0.0041773319244384766, 0.004239797592163086, 0.004228830337524414, 0.004229307174682617, 0.004190683364868164, 0.004221677780151367, 0.004179239273071289, 0.004221439361572266, 0.0042378902435302734, 0.004301786422729492, 0.004258394241333008, 0.004198551177978516, 0.004244804382324219, 0.004269838333129883, 0.004163980484008789, 0.0042150020599365234, 0.004182338714599609, 0.0042531490325927734, 0.004210948944091797, 0.0041790008544921875, 0.004247188568115234, 0.004209041595458984, 0.0042231082916259766, 0.004245758056640625, 0.004244327545166016, 0.004182100296020508, 0.004251003265380859, 0.004227399826049805, 0.00423121452331543, 0.004308938980102539, 0.004195690155029297, 0.004225492477416992, 0.004282236099243164, 0.004225969314575195, 0.004232883453369141, 0.004302024841308594, 0.004246950149536133, 0.004260063171386719, 0.004210710525512695, 0.004226207733154297, 0.004213571548461914, 0.004236936569213867, 0.004244565963745117, 0.004259824752807617, 0.0041942596435546875, 0.004222869873046875, 0.0042607784271240234, 0.0041751861572265625, 0.004240751266479492, 0.004184246063232422, 0.004265785217285156, 0.004203319549560547, 0.004193305969238281, 0.0042345523834228516, 0.004224538803100586, 0.004214286804199219, 0.004235744476318359, 0.004319906234741211, 0.004195451736450195, 0.004232168197631836, 0.004224538803100586, 0.004258871078491211, 0.004235506057739258, 0.00423121452331543, 0.004243135452270508, 0.004251956939697266, 0.0041828155517578125, 0.004236698150634766, 0.0041942596435546875, 0.004267692565917969, 0.004218339920043945, 0.004469394683837891, 0.004281759262084961, 0.004234790802001953, 0.0042340755462646484, 0.00425267219543457, 0.004278659820556641, 0.00437617301940918, 0.004214048385620117, 0.004260540008544922, 0.004220247268676758, 0.004215240478515625, 0.00428462028503418, 0.0042378902435302734, 0.004207134246826172, 0.0042421817779541016, 0.0042400360107421875, 0.0042531490325927734, 0.004212856292724609, 0.004183769226074219, 0.004256248474121094, 0.0042111873626708984, 0.0041751861572265625, 0.004220724105834961, 0.004233360290527344, 0.004229545593261719, 0.004226207733154297, 0.004177570343017578, 0.004225015640258789, 0.004248142242431641, 0.004244089126586914, 0.004235982894897461, 0.0042803287506103516, 0.004235506057739258, 0.004327058792114258, 0.0042858123779296875, 0.004189252853393555, 0.004233837127685547, 0.004190206527709961, 0.00469517707824707, 0.004314422607421875, 0.004207134246826172, 0.004325389862060547, 0.00424957275390625, 0.004202127456665039, 0.004236936569213867, 0.004264116287231445, 0.0042743682861328125, 0.004282236099243164, 0.004235982894897461, 0.004240989685058594, 0.0042362213134765625, 0.004233837127685547, 0.004231929779052734, 0.00432276725769043, 0.0041866302490234375, 0.004304170608520508, 0.004826068878173828, 0.0045697689056396484, 0.004308462142944336, 0.004230499267578125, 0.004239559173583984, 0.0042340755462646484, 0.004264116287231445, 0.004225015640258789, 0.004245281219482422, 0.004291057586669922, 0.004228830337524414, 0.0042684078216552734, 0.004196882247924805, 0.004355907440185547, 0.004233121871948242, 0.004225730895996094, 0.004283905029296875, 0.004232883453369141, 0.004250049591064453, 0.004240989685058594, 0.0042302608489990234, 0.004207134246826172, 0.004241466522216797, 0.0042972564697265625, 0.004308223724365234, 0.004233837127685547, 0.004190921783447266, 0.004320621490478516, 0.004244804382324219, 0.004191875457763672, 0.004268169403076172, 0.004271745681762695, 0.004265785217285156, 0.004245758056640625, 0.00423884391784668, 0.004328250885009766, 0.004279136657714844, 0.004248380661010742, 0.004268169403076172, 0.0042629241943359375, 0.004210472106933594, 0.004244804382324219, 0.004303455352783203, 0.004214286804199219, 0.00442194938659668, 0.004293680191040039, 0.0042743682861328125, 0.004242658615112305, 0.0042171478271484375, 0.004260063171386719, 0.004262447357177734, 0.00427699089050293, 0.004250526428222656, 0.0042476654052734375, 0.004224300384521484, 0.00426793098449707, 0.004337787628173828, 0.004218101501464844, 0.0042531490325927734, 0.0042188167572021484, 0.004288196563720703, 0.004243373870849609, 0.004236936569213867, 0.004475831985473633, 0.004319906234741211, 0.004319906234741211, 0.004281520843505859, 0.004256248474121094, 0.004206180572509766, 0.0042302608489990234, 0.004415035247802734, 0.004246950149536133, 0.004252910614013672, 0.004223823547363281, 0.004305124282836914, 0.00424647331237793, 0.004183769226074219, 0.00423884391784668, 0.004439830780029297, 0.004269123077392578, 0.0042378902435302734, 0.0042989253997802734, 0.004206657409667969, 0.004247903823852539, 0.004298210144042969, 0.004201412200927734, 0.004241466522216797, 0.004202365875244141, 0.004292488098144531, 0.004286289215087891, 0.0042629241943359375, 0.004346132278442383, 0.00431513786315918, 0.004358530044555664, 0.00426936149597168, 0.004271268844604492, 0.004222393035888672, 0.004246234893798828, 0.004257678985595703, 0.004255533218383789, 0.004264354705810547, 0.004234790802001953, 0.00431060791015625, 0.004264354705810547, 0.0042154788970947266, 0.004244804382324219, 0.004278898239135742, 0.004267692565917969, 0.00423431396484375, 0.00420832633972168, 0.004286289215087891, 0.004275083541870117, 0.004242897033691406, 0.00426173210144043, 0.0042154788970947266, 0.004204511642456055, 0.004247188568115234, 0.004374265670776367, 0.004235029220581055, 0.00428009033203125, 0.004284381866455078, 0.004259586334228516, 0.004235982894897461, 0.004192829132080078, 0.004293203353881836, 0.004262208938598633, 0.004278659820556641, 0.004262685775756836, 0.004261016845703125, 0.004185914993286133, 0.004237174987792969, 0.00429081916809082, 0.004204511642456055, 0.0042493343353271484, 0.004219532012939453, 0.004311323165893555, 0.0042629241943359375, 0.00423884391784668, 0.00436091423034668, 0.004244565963745117, 0.004258632659912109, 0.004278421401977539, 0.004244089126586914, 0.00418853759765625, 0.004240512847900391, 0.0042798519134521484, 0.004290342330932617, 0.00424504280090332, 0.004209756851196289, 0.0043201446533203125, 0.004250526428222656, 0.004217624664306641, 0.00426173210144043, 0.004281520843505859, 0.004248142242431641, 0.004250526428222656, 0.004273891448974609, 0.004240274429321289, 0.004260540008544922, 0.0042552947998046875, 0.004288196563720703, 0.004267692565917969, 0.004205465316772461, 0.004269838333129883, 0.004420280456542969, 0.004246234893798828, 0.004264354705810547, 0.004297733306884766, 0.0042781829833984375, 0.004255771636962891, 0.0042417049407958984, 0.004289150238037109, 0.004273891448974609, 0.004242897033691406, 0.004258871078491211, 0.004255056381225586, 0.004375457763671875, 0.0047833919525146484, 0.004260540008544922, 0.004211902618408203, 0.004719257354736328, 0.004315614700317383, 0.004275083541870117, 0.0042574405670166016, 0.004355192184448242, 0.004338264465332031, 0.004263639450073242, 0.004303932189941406, 0.00421452522277832, 0.004273891448974609, 0.0042324066162109375, 0.004316568374633789, 0.004244089126586914, 0.004214286804199219, 0.004251241683959961, 0.00424957275390625, 0.00431513786315918, 0.004250049591064453, 0.004271268844604492, 0.00422358512878418, 0.004256010055541992, 0.004265785217285156, 0.004294157028198242, 0.004248380661010742, 0.004252433776855469, 0.004294157028198242, 0.004247903823852539, 0.004235029220581055, 0.004241943359375, 0.004285097122192383, 0.004328250885009766, 0.0043714046478271484, 0.004292011260986328, 0.00420689582824707, 0.004271745681762695, 0.0042781829833984375, 0.0043697357177734375, 0.004255771636962891, 0.004231929779052734, 0.004297971725463867, 0.004305124282836914, 0.005064725875854492, 0.004365205764770508, 0.004292726516723633, 0.004246950149536133, 0.004766941070556641, 0.0044291019439697266, 0.004378318786621094, 0.004322052001953125, 0.004303693771362305, 0.0042307376861572266, 0.004355907440185547, 0.0043163299560546875, 0.004324913024902344, 0.004285573959350586, 0.0043065547943115234, 0.004235029220581055, 0.0042743682861328125, 0.004243612289428711, 0.004302501678466797, 0.0042574405670166016, 0.0042421817779541016, 0.004289150238037109, 0.0042438507080078125, 0.0042133331298828125, 0.004271745681762695, 0.004280805587768555, 0.004290103912353516, 0.004278659820556641, 0.004249095916748047, 0.004222393035888672, 0.004255056381225586, 0.004302263259887695, 0.0042650699615478516, 0.004237651824951172, 0.004229545593261719, 0.004282474517822266, 0.004454135894775391, 0.004242658615112305, 0.004258155822753906, 0.004278421401977539, 0.004254341125488281, 0.0043027400970458984, 0.004286050796508789, 0.004256725311279297, 0.004273176193237305, 0.004289150238037109, 0.004275321960449219, 0.004240751266479492, 0.004239082336425781, 0.004285573959350586, 0.0043642520904541016, 0.00423741340637207, 0.004312992095947266, 0.004300355911254883, 0.004271745681762695, 0.00429224967956543, 0.0042226314544677734, 0.004424095153808594, 0.00429081916809082, 0.004271745681762695, 0.004286050796508789, 0.004262685775756836, 0.00424957275390625, 0.004344463348388672, 0.004303932189941406, 0.004254579544067383, 0.004252910614013672, 0.004306793212890625, 0.004299163818359375, 0.004293918609619141, 0.00429987907409668, 0.004256248474121094, 0.004378795623779297, 0.004305124282836914, 0.004315376281738281, 0.004289388656616211, 0.004242897033691406, 0.004317760467529297, 0.004328489303588867, 0.004264354705810547, 0.004303455352783203, 0.0042879581451416016, 0.004278659820556641, 0.004359006881713867, 0.00432276725769043, 0.004233360290527344, 0.004263639450073242, 0.004270792007446289, 0.004271984100341797, 0.004266977310180664, 0.004374980926513672, 0.004363059997558594, 0.004273414611816406, 0.004228353500366211, 0.004301786422729492, 0.004289150238037109, 0.004279613494873047, 0.004286527633666992, 0.004296541213989258, 0.004251003265380859, 0.004271745681762695, 0.004288434982299805, 0.004278898239135742, 0.004326343536376953, 0.004286050796508789, 0.004394054412841797, 0.004281759262084961, 0.004247426986694336, 0.0042726993560791016, 0.004314422607421875, 0.004275798797607422, 0.004265546798706055, 0.004273176193237305, 0.004242420196533203, 0.00428318977355957, 0.0042879581451416016, 0.004279375076293945, 0.004286289215087891, 0.004233121871948242, 0.004363059997558594, 0.004281282424926758, 0.004214286804199219, 0.004288911819458008, 0.0042896270751953125, 0.004273653030395508, 0.004290103912353516, 0.004264116287231445, 0.004230976104736328, 0.0042765140533447266, 0.0043790340423583984, 0.004318952560424805, 0.004275798797607422, 0.0042684078216552734, 0.004331111907958984, 0.004309177398681641, 0.0042340755462646484, 0.004276752471923828, 0.00429224967956543, 0.00429081916809082, 0.00429987907409668, 0.004313230514526367, 0.0042688846588134766, 0.004294633865356445, 0.004269123077392578, 0.004305124282836914, 0.004300117492675781, 0.004252433776855469, 0.004334211349487305, 0.004286527633666992, 0.0042340755462646484, 0.004413127899169922, 0.004302024841308594, 0.004288196563720703, 0.0042917728424072266, 0.004294395446777344, 0.00429224967956543, 0.0042438507080078125, 0.0042934417724609375, 0.004327535629272461, 0.004247426986694336, 0.004294157028198242, 0.004291057586669922, 0.0043065547943115234, 0.004271507263183594, 0.004368305206298828, 0.004269123077392578, 0.0043065547943115234, 0.004271507263183594, 0.004306793212890625, 0.0042877197265625, 0.004259586334228516, 0.004359245300292969, 0.004311800003051758, 0.004232645034790039, 0.004270792007446289, 0.004324913024902344, 0.0043849945068359375, 0.00431370735168457, 0.004311561584472656, 0.004274129867553711, 0.004302501678466797, 0.004271268844604492, 0.004276752471923828, 0.004298686981201172, 0.004259586334228516, 0.004340410232543945, 0.0042896270751953125, 0.004263639450073242, 0.004349231719970703, 0.004324197769165039, 0.004292011260986328, 0.004318952560424805, 0.004310131072998047, 0.004265546798706055, 0.004300832748413086, 0.004268169403076172, 0.004305601119995117, 0.004400730133056641, 0.004261493682861328, 0.0043447017669677734, 0.004307270050048828, 0.004251956939697266, 0.004317760467529297, 0.004304409027099609, 0.0042879581451416016, 0.0043315887451171875, 0.004327535629272461, 0.0042531490325927734, 0.004272937774658203, 0.004383563995361328, 0.004279375076293945, 0.004294395446777344, 0.004240751266479492, 0.00436711311340332, 0.004305839538574219, 0.00427556037902832, 0.0043146610260009766, 0.0042912960052490234, 0.0042836666107177734, 0.004324436187744141, 0.004329204559326172, 0.004266977310180664, 0.004283905029296875, 0.004469633102416992, 0.004268646240234375, 0.004315853118896484, 0.004324913024902344, 0.004294157028198242, 0.004335641860961914, 0.004267692565917969, 0.004304409027099609, 0.004294395446777344, 0.004309415817260742, 0.004396677017211914, 0.004318714141845703, 0.004300594329833984, 0.004278659820556641, 0.004347085952758789, 0.004245281219482422, 0.0043146610260009766, 0.00432276725769043, 0.00428462028503418, 0.004283428192138672, 0.004460334777832031, 0.004276275634765625, 0.004354000091552734, 0.0043141841888427734, 0.004664182662963867, 0.0043489933013916016, 0.004251718521118164, 0.0043888092041015625, 0.004300832748413086, 0.004265308380126953, 0.004347324371337891, 0.004315853118896484, 0.00432133674621582, 0.004295825958251953, 0.004305124282836914, 0.0042572021484375, 0.004367351531982422, 0.00434422492980957, 0.004273414611816406, 0.004346370697021484, 0.004298686981201172, 0.004311561584472656, 0.004314899444580078, 0.004280805587768555, 0.004314899444580078, 0.004308938980102539, 0.004410743713378906, 0.004338502883911133, 0.004307985305786133, 0.00432586669921875, 0.0043680667877197266, 0.004316568374633789, 0.004252433776855469, 0.004319906234741211, 0.004315614700317383, 0.004317045211791992, 0.004339933395385742, 0.004332542419433594, 0.004279136657714844, 0.004325151443481445, 0.004332542419433594, 0.004326581954956055, 0.004301786422729492, 0.004283905029296875, 0.0043604373931884766, 0.004286289215087891, 0.00429534912109375, 0.00439143180847168, 0.004304409027099609, 0.004343509674072266, 0.004354953765869141, 0.0043332576751708984, 0.004279136657714844, 0.004302501678466797, 0.004388332366943359, 0.004281044006347656, 0.0043485164642333984, 0.0043489933013916016, 0.004327297210693359, 0.004294157028198242, 0.004302263259887695, 0.00435948371887207, 0.0043179988861083984, 0.004340171813964844, 0.0043201446533203125, 0.004327535629272461, 0.004312992095947266, 0.004343748092651367, 0.004405498504638672, 0.004305839538574219, 0.004332780838012695, 0.004328727722167969, 0.004481315612792969, 0.0043370723724365234, 0.004354953765869141, 0.004309892654418945, 0.00435185432434082, 0.004419565200805664, 0.004279613494873047, 0.004349708557128906, 0.004529476165771484, 0.0043337345123291016, 0.00434565544128418, 0.0042572021484375, 0.004336833953857422, 0.004311084747314453, 0.00432586669921875, 0.004330873489379883, 0.0043299198150634766, 0.004279613494873047, 0.004309415817260742, 0.004369020462036133, 0.0043087005615234375, 0.004445075988769531, 0.004463672637939453, 0.0044803619384765625, 0.004549980163574219, 0.004425525665283203, 0.00429844856262207, 0.004319667816162109, 0.004414558410644531, 0.004357814788818359, 0.004330873489379883, 0.004332542419433594, 0.004318952560424805, 0.0043261051177978516, 0.004288673400878906, 0.004307746887207031, 0.004333019256591797, 0.004303932189941406, 0.004339933395385742, 0.004352092742919922, 0.004273176193237305, 0.0043637752532958984, 0.004324674606323242, 0.004273176193237305, 0.004334211349487305, 0.0050411224365234375, 0.0050411224365234375, 0.004407405853271484, 0.0043182373046875, 0.004652500152587891, 0.0043566226959228516, 0.0043146610260009766, 0.004324436187744141, 0.004398345947265625, 0.004286289215087891, 0.0043735504150390625, 0.0043413639068603516, 0.004296302795410156, 0.00435638427734375, 0.004300355911254883, 0.004301309585571289, 0.004362344741821289, 0.00433039665222168, 0.0043621063232421875, 0.004326820373535156, 0.004312038421630859, 0.00447535514831543, 0.004342794418334961, 0.004281759262084961, 0.004334211349487305, 0.0043523311614990234, 0.0043489933013916016, 0.004551887512207031, 0.0043675899505615234, 0.004304170608520508, 0.004331111907958984, 0.004857301712036133, 0.00435638427734375, 0.004397153854370117, 0.00437164306640625, 0.004347562789916992, 0.0043408870697021484, 0.004363059997558594, 0.004304409027099609, 0.0043621063232421875, 0.004417896270751953, 0.004304647445678711, 0.0043947696685791016, 0.004389762878417969, 0.004369020462036133, 0.004410982131958008, 0.004423379898071289, 0.004328012466430664, 0.004350185394287109, 0.004358530044555664, 0.004347562789916992, 0.004342317581176758, 0.0043010711669921875, 0.004434108734130859, 0.0043146610260009766, 0.004353761672973633, 0.004383563995361328, 0.004340648651123047, 0.004395723342895508, 0.004759073257446289, 0.004383563995361328, 0.004324913024902344, 0.004401445388793945, 0.0046253204345703125, 0.004393100738525391, 0.004365205764770508, 0.00437164306640625, 0.004427433013916016, 0.0044002532958984375, 0.004325389862060547, 0.00441288948059082, 0.0043828487396240234, 0.0043506622314453125, 0.004347085952758789, 0.004322528839111328, 0.004433870315551758, 0.004384040832519531, 0.004348278045654297, 0.004448413848876953, 0.004366397857666016, 0.0043299198150634766, 0.00443267822265625, 0.004480600357055664, 0.004415273666381836, 0.004507303237915039, 0.00444483757019043, 0.004393100738525391, 0.004445314407348633, 0.004436016082763672, 0.0044558048248291016, 0.004480600357055664, 0.0044248104095458984, 0.0044939517974853516, 0.004479169845581055, 0.004433155059814453, 0.0044252872467041016, 0.004426240921020508, 0.0044765472412109375, 0.004425764083862305, 0.004454612731933594, 0.004422903060913086, 0.0045735836029052734, 0.0048542022705078125, 0.004435062408447266, 0.004405975341796875, 0.004477024078369141, 0.004418134689331055, 0.004445075988769531, 0.004460811614990234, 0.004435062408447266, 0.004376888275146484, 0.0044956207275390625, 0.00452113151550293, 0.004385471343994141, 0.0044896602630615234, 0.0044481754302978516, 0.004475116729736328, 0.004444122314453125, 0.0043926239013671875, 0.0044133663177490234, 0.004499197006225586, 0.004410505294799805, 0.0043621063232421875, 0.0044994354248046875, 0.0044481754302978516, 0.004448890686035156, 0.0044536590576171875, 0.004465341567993164, 0.004404306411743164, 0.004515171051025391, 0.004430055618286133, 0.004393100738525391, 0.004450082778930664, 0.004460811614990234, 0.004438877105712891, 0.004467010498046875, 0.004435300827026367, 0.0044782161712646484, 0.0044786930084228516, 0.00444483757019043, 0.00440216064453125, 0.004448652267456055, 0.0044133663177490234, 0.004418611526489258, 0.0044629573822021484, 0.004433631896972656, 0.004423379898071289, 0.004554271697998047, 0.004440784454345703, 0.004525423049926758, 0.004418134689331055, 0.004479885101318359, 0.0045015811920166016, 0.004388570785522461, 0.004433631896972656, 0.004438638687133789, 0.0044972896575927734, 0.004523754119873047, 0.0044307708740234375, 0.004392862319946289, 0.004418611526489258, 0.004499912261962891, 0.004387855529785156, 0.004442930221557617, 0.004492998123168945, 0.004469156265258789, 0.004430532455444336, 0.004425048828125, 0.004384279251098633, 0.00442814826965332, 0.0045375823974609375, 0.004390239715576172, 0.0044286251068115234, 0.004475831985473633, 0.004471302032470703, 0.0044345855712890625, 0.004438877105712891, 0.0044116973876953125, 0.00447392463684082, 0.0044858455657958984, 0.0044095516204833984, 0.0044078826904296875, 0.004530429840087891, 0.004477500915527344, 0.004502058029174805, 0.004460334777832031, 0.004392147064208984, 0.004470109939575195, 0.004499673843383789, 0.004621267318725586, 0.00445103645324707, 0.0044476985931396484, 0.00443577766418457, 0.004419565200805664, 0.004469633102416992, 0.004403114318847656, 0.004453182220458984, 0.004515886306762695, 0.0044116973876953125, 0.004456520080566406, 0.004485130310058594, 0.004439353942871094, 0.004531383514404297, 0.004476070404052734, 0.004487276077270508, 0.00504612922668457, 0.004506111145019531, 0.004456520080566406, 0.0045011043548583984, 0.004470109939575195, 0.004453182220458984, 0.004477977752685547, 0.004431247711181641, 0.004435062408447266, 0.004488229751586914, 0.004509925842285156, 0.004434823989868164, 0.004485130310058594, 0.00442957878112793, 0.0044820308685302734, 0.004503726959228516, 0.004479885101318359, 0.004442691802978516, 0.004496335983276367, 0.004475593566894531, 0.004404544830322266, 0.0045719146728515625, 0.004488706588745117, 0.0044400691986083984, 0.0044863224029541016, 0.0044438838958740234, 0.0044481754302978516, 0.0044384002685546875, 0.0044324398040771484, 0.0044286251068115234, 0.0045261383056640625, 0.0044710636138916016, 0.004428863525390625, 0.004454135894775391, 0.0044672489166259766, 0.004433870315551758, 0.004458189010620117, 0.0044443607330322266, 0.004411458969116211, 0.004562854766845703, 0.0044329166412353516, 0.004433393478393555, 0.004598855972290039, 0.004500150680541992, 0.0045299530029296875, 0.00446009635925293, 0.004479885101318359, 0.0044558048248291016, 0.004458189010620117, 0.004448890686035156, 0.004462003707885742, 0.004423618316650391, 0.00442814826965332, 0.004477977752685547, 0.004396677017211914, 0.004484891891479492, 0.004479169845581055, 0.004483461380004883, 0.0044612884521484375, 0.0044405460357666016, 0.0044171810150146484, 0.004461050033569336, 0.0045166015625, 0.004454374313354492, 0.004442930221557617, 0.004475116729736328, 0.0045392513275146484, 0.004454135894775391, 0.004494667053222656, 0.0044460296630859375, 0.0044803619384765625, 0.004506349563598633, 0.004396677017211914, 0.004450082778930664, 0.0044879913330078125, 0.004471778869628906, 0.004488706588745117, 0.004485368728637695, 0.00446319580078125, 0.004479885101318359, 0.004520893096923828, 0.00441288948059082, 0.004540681838989258, 0.004476070404052734, 0.0044879913330078125, 0.00451970100402832, 0.004500865936279297, 0.004584789276123047, 0.004512310028076172, 0.004481077194213867, 0.004415273666381836, 0.004486799240112305, 0.004431962966918945, 0.004484653472900391, 0.004492759704589844, 0.004477739334106445, 0.004437923431396484, 0.004486799240112305, 0.0044820308685302734, 0.004439592361450195, 0.004500389099121094, 0.0044934749603271484, 0.004475116729736328, 0.004483461380004883, 0.004449129104614258, 0.0043981075286865234, 0.004485368728637695, 0.004464149475097656, 0.004415273666381836, 0.0045053958892822266, 0.004544496536254883, 0.004578828811645508, 0.004462003707885742, 0.004432201385498047, 0.004448413848876953, 0.004525423049926758, 0.00452733039855957, 0.004589557647705078, 0.004415273666381836, 0.004476070404052734, 0.004534244537353516, 0.004461526870727539, 0.004472255706787109, 0.004492044448852539, 0.004506587982177734, 0.004468441009521484, 0.004549503326416016, 0.004483461380004883, 0.0044744014739990234, 0.004510402679443359, 0.004429817199707031, 0.0045623779296875, 0.004506111145019531, 0.0044786930084228516, 0.004473209381103516, 0.004584550857543945, 0.004456520080566406, 0.0045337677001953125, 0.0044841766357421875, 0.004439592361450195, 0.004469871520996094, 0.004437446594238281, 0.004488945007324219, 0.004500627517700195, 0.004466056823730469, 0.004454374313354492, 0.0045452117919921875, 0.004513740539550781, 0.004454135894775391, 0.004483222961425781, 0.004484415054321289, 0.004445791244506836, 0.004481077194213867, 0.004457712173461914, 0.004436016082763672, 0.004606485366821289, 0.004478931427001953, 0.004439592361450195, 0.004525661468505859, 0.004485130310058594, 0.0044634342193603516, 0.00449681282043457, 0.0045545101165771484, 0.004427671432495117, 0.004514932632446289, 0.0045185089111328125, 0.00445556640625, 0.004459381103515625, 0.004452943801879883, 0.004498481750488281, 0.004459381103515625, 0.004450321197509766, 0.00450587272644043, 0.0044744014739990234, 0.0044710636138916016, 0.004500865936279297, 0.0045435428619384766, 0.004456996917724609, 0.004521846771240234, 0.004416942596435547, 0.004478931427001953, 0.0044710636138916016, 0.004528045654296875, 0.004454374313354492, 0.0045261383056640625, 0.004425525665283203, 0.004491567611694336, 0.004532575607299805, 0.004485607147216797, 0.004475593566894531, 0.0045053958892822266, 0.004442691802978516, 0.0044553279876708984, 0.004523754119873047, 0.004428863525390625, 0.0044918060302734375, 0.004540205001831055, 0.004474639892578125, 0.004503011703491211, 0.0045070648193359375, 0.004559516906738281, 0.0045261383056640625, 0.004464864730834961, 0.004418373107910156, 0.004478931427001953, 0.004501819610595703, 0.004506587982177734, 0.004656553268432617, 0.0044994354248046875, 0.004427909851074219, 0.0045282840728759766, 0.004489421844482422, 0.004419803619384766, 0.004477500915527344, 0.0044705867767333984, 0.004476308822631836, 0.00447535514831543, 0.004494905471801758, 0.004496574401855469, 0.00452423095703125, 0.004494190216064453, 0.0046520233154296875, 0.004595756530761719, 0.004442453384399414, 0.0045964717864990234, 0.0045087337493896484, 0.0044634342193603516, 0.004503011703491211, 0.004476308822631836, 0.0045092105865478516, 0.004510641098022461, 0.0044879913330078125, 0.004431009292602539, 0.004538297653198242, 0.0045282840728759766, 0.004504680633544922, 0.004449605941772461, 0.004517793655395508, 0.004544734954833984, 0.00444340705871582, 0.0045452117919921875, 0.004528999328613281, 0.004506826400756836, 0.004492282867431641, 0.004584074020385742, 0.004511833190917969, 0.00447392463684082, 0.004581451416015625, 0.004463911056518555, 0.004528045654296875, 0.004557371139526367, 0.004487752914428711, 0.0045702457427978516, 0.0045032501220703125, 0.0044710636138916016, 0.004591464996337891, 0.004508256912231445, 0.0044782161712646484, 0.0045032501220703125, 0.0045318603515625, 0.004517078399658203, 0.004531145095825195, 0.004520893096923828, 0.0044765472412109375, 0.004557371139526367, 0.0045812129974365234, 0.004540681838989258, 0.004483461380004883, 0.00449371337890625, 0.004603385925292969, 0.004456758499145508, 0.004523754119873047, 0.004537343978881836, 0.004521608352661133, 0.004723310470581055, 0.0045948028564453125, 0.004553079605102539, 0.004591941833496094, 0.004487037658691406, 0.004520893096923828, 0.0045473575592041016, 0.00475621223449707, 0.004534482955932617, 0.004540920257568359, 0.0045468807220458984, 0.00452733039855957, 0.0045201778411865234, 0.00449061393737793, 0.004635810852050781, 0.0045168399810791016, 0.004462242126464844, 0.0045659542083740234, 0.004486083984375, 0.004536867141723633, 0.004517793655395508, 0.004517555236816406, 0.004461526870727539, 0.004579067230224609, 0.004524946212768555, 0.004499673843383789, 0.004471540451049805, 0.0045053958892822266, 0.004547595977783203, 0.0044596195220947266, 0.0045087337493896484, 0.0045623779296875, 0.004513740539550781, 0.004529714584350586, 0.00450897216796875, 0.004535675048828125, 0.004506349563598633, 0.004546165466308594, 0.004485368728637695, 0.004509449005126953, 0.00450587272644043, 0.0045053958892822266, 0.004527091979980469, 0.004521369934082031, 0.004494190216064453, 0.004563808441162109, 0.004540205001831055, 0.0044629573822021484, 0.004536151885986328, 0.004498481750488281, 0.004538536071777344, 0.004544496536254883, 0.004494190216064453, 0.004480123519897461, 0.004588603973388672, 0.004549503326416016, 0.004611968994140625, 0.004488229751586914, 0.0044972896575927734, 0.004660844802856445, 0.004505634307861328, 0.004523277282714844, 0.004540920257568359, 0.004505634307861328, 0.005064964294433594, 0.0045626163482666016, 0.00449061393737793, 0.0046367645263671875, 0.004543781280517578, 0.004462242126464844, 0.0045626163482666016, 0.004534006118774414, 0.0045185089111328125, 0.004460334777832031, 0.0045166015625, 0.004580259323120117, 0.004477500915527344, 0.004480838775634766, 0.0045320987701416016, 0.004611015319824219, 0.005108833312988281, 0.004586696624755859, 0.004548072814941406, 0.004582643508911133, 0.004500865936279297, 0.004485607147216797, 0.0046193599700927734, 0.004540443420410156, 0.0045015811920166016, 0.0045359134674072266, 0.00451350212097168, 0.0045185089111328125, 0.0045397281646728516, 0.004534006118774414, 0.004570722579956055, 0.004481315612792969, 0.004553318023681641, 0.004614591598510742, 0.004498958587646484, 0.0045490264892578125, 0.004523038864135742, 0.0045435428619384766, 0.004612922668457031, 0.0045468807220458984, 0.004449367523193359, 0.004589080810546875, 0.00452876091003418, 0.004526853561401367, 0.004536867141723633, 0.00457000732421875, 0.004559516906738281, 0.0045452117919921875, 0.004523038864135742, 0.004525899887084961, 0.0046083927154541016, 0.004578828811645508, 0.004529714584350586, 0.004456520080566406, 0.00450897216796875, 0.004553318023681641, 0.004488945007324219, 0.004543304443359375, 0.004536867141723633, 0.00459599494934082, 0.004528045654296875, 0.004566669464111328, 0.004549264907836914, 0.0045757293701171875, 0.004522562026977539, 0.004471540451049805, 0.004557609558105469, 0.004518747329711914, 0.00451207160949707, 0.004518985748291016, 0.004506111145019531, 0.004514932632446289, 0.004615306854248047, 0.004555940628051758, 0.004586219787597656, 0.004472494125366211, 0.004555463790893555, 0.004521369934082031, 0.004507303237915039, 0.004527568817138672, 0.004546165466308594, 0.0044896602630615234, 0.004567623138427734, 0.00461578369140625, 0.004512310028076172, 0.004557132720947266, 0.004549741744995117, 0.004509687423706055, 0.00457310676574707, 0.004546165466308594, 0.0046923160552978516, 0.004631757736206055, 0.004513740539550781, 0.004504680633544922, 0.00454258918762207, 0.004547119140625, 0.0045893192291259766, 0.004549741744995117, 0.0045299530029296875, 0.004564046859741211, 0.004570960998535156, 0.004601955413818359, 0.004559993743896484, 0.0045795440673828125, 0.004678249359130859, 0.0046765804290771484, 0.005640506744384766, 0.005497455596923828, 0.011849403381347656, 0.006026744842529297, 0.004992246627807617, 0.0048885345458984375, 0.004803180694580078, 0.004580497741699219, 0.004622459411621094, 0.004555702209472656, 0.004579305648803711, 0.004530906677246094, 0.0045278072357177734, 0.0046007633209228516, 0.00452876091003418, 0.004545688629150391, 0.004564762115478516, 20.37408137321472]    
t_12_gzip = [0.005019426345825195, 0.0036797523498535156, 0.0039370059967041016, 0.0040247440338134766, 0.0041255950927734375, 0.004110813140869141, 0.003873109817504883, 0.0039048194885253906, 0.0038890838623046875, 0.005067586898803711, 0.004423618316650391, 0.0038971900939941406, 0.005048274993896484, 0.0042266845703125, 0.004247426986694336, 0.004342317581176758, 0.004170894622802734, 0.004330635070800781, 0.004359722137451172, 0.004315376281738281, 0.004235267639160156, 0.004228115081787109, 0.004469156265258789, 0.0042726993560791016, 0.004433393478393555, 0.00419163703918457, 0.0039942264556884766, 0.003774881362915039, 0.003765106201171875, 0.0038843154907226562, 0.004065990447998047, 0.0061337947845458984, 0.0041294097900390625, 0.0037658214569091797, 0.003705739974975586, 0.0036857128143310547, 0.003710031509399414, 0.0036818981170654297, 0.003770112991333008, 0.0036394596099853516, 0.003699064254760742, 0.0036351680755615234, 0.003723621368408203, 0.0036950111389160156, 0.003656148910522461, 0.0036416053771972656, 0.003695964813232422, 0.003710031509399414, 0.0036907196044921875, 0.003778219223022461, 0.003693819046020508, 0.003704547882080078, 0.0036652088165283203, 0.0036983489990234375, 0.0036869049072265625, 0.003673553466796875, 0.0037665367126464844, 0.003634929656982422, 0.0037093162536621094, 0.0036547183990478516, 0.003701448440551758, 0.0036957263946533203, 0.0037004947662353516, 0.0036344528198242188, 0.004302263259887695, 0.0037021636962890625, 0.003680706024169922, 0.0036826133728027344, 0.0041065216064453125, 0.0036306381225585938, 0.0036814212799072266, 0.003636598587036133, 0.0037207603454589844, 0.0037190914154052734, 0.003629922866821289, 0.0037589073181152344, 0.003622293472290039, 0.0036771297454833984, 0.003696441650390625, 0.0036966800689697266, 0.0036318302154541016, 0.0036687850952148438, 0.0037326812744140625, 0.0036983489990234375, 0.0036764144897460938, 0.0036339759826660156, 0.0036745071411132812, 0.0036416053771972656, 0.0037047863006591797, 0.003676176071166992, 0.0037076473236083984, 0.0036466121673583984, 0.003690958023071289, 0.003687620162963867, 0.0037064552307128906, 0.0036940574645996094, 0.0036351680755615234, 0.003688812255859375, 0.003651857376098633, 0.00371551513671875, 0.0036995410919189453, 0.003706216812133789, 0.0037012100219726562, 0.0037658214569091797, 0.00370025634765625, 0.0036852359771728516, 0.0036940574645996094, 0.00365447998046875, 0.0036804676055908203, 0.003632783889770508, 0.0037729740142822266, 0.003697633743286133, 0.003645658493041992, 0.003690958023071289, 0.0036516189575195312, 0.0040323734283447266, 0.003896474838256836, 0.0036869049072265625, 0.003769397735595703, 0.0036416053771972656, 0.003681182861328125, 0.0037016868591308594, 0.003809690475463867, 0.003656148910522461, 0.0037064552307128906, 0.0036325454711914062, 0.003714323043823242, 0.003766298294067383, 0.003645181655883789, 0.0037844181060791016, 0.003713369369506836, 0.0036830902099609375, 0.0037107467651367188, 0.003690481185913086, 0.003643035888671875, 0.0036923885345458984, 0.003700733184814453, 0.0037097930908203125, 0.003703594207763672, 0.003643512725830078, 0.0036840438842773438, 0.003650188446044922, 0.0036957263946533203, 0.0036950111389160156, 0.003692150115966797, 0.003637075424194336, 0.003699064254760742, 0.003726959228515625, 0.0037102699279785156, 0.0036988258361816406, 0.003652811050415039, 0.0036804676055908203, 0.0036475658416748047, 0.003694295883178711, 0.003683805465698242, 0.0036916732788085938, 0.0036780834197998047, 0.0037279129028320312, 0.003704071044921875, 0.003685474395751953, 0.0036928653717041016, 0.003693103790283203, 0.0036869049072265625, 0.0036542415618896484, 0.0036933422088623047, 0.003736257553100586, 0.0037713050842285156, 0.0037217140197753906, 0.003662109375, 0.003717184066772461, 0.003749370574951172, 0.003710031509399414, 0.0037076473236083984, 0.0037603378295898438, 0.0037124156951904297, 0.0036630630493164062, 0.0036916732788085938, 0.0036695003509521484, 0.0037055015563964844, 0.005208253860473633, 0.0060613155364990234, 0.006060361862182617, 0.004389047622680664, 0.004312753677368164, 0.005007743835449219, 0.011871576309204102, 0.010283470153808594, 0.007518291473388672, 0.004340648651123047, 0.005371570587158203, 0.01241302490234375, 0.005328655242919922, 0.008139848709106445, 0.008144855499267578, 0.004428863525390625, 0.00404810905456543, 0.004082918167114258, 0.004111289978027344, 0.004086971282958984, 0.0040056705474853516, 0.0039060115814208984, 0.0037469863891601562, 0.003709554672241211, 0.0036618709564208984, 0.003693103790283203, 0.003724336624145508, 0.0036611557006835938, 0.0038793087005615234, 0.0036580562591552734, 0.0037078857421875, 0.003688812255859375, 0.0036983489990234375, 0.0037217140197753906, 0.003733396530151367, 0.0036954879760742188, 0.0036783218383789062, 0.0037078857421875, 0.003666400909423828, 0.003691434860229492, 0.0036602020263671875, 0.003721952438354492, 0.003690481185913086, 0.0036983489990234375, 0.003828287124633789, 0.0038034915924072266, 0.0036935806274414062, 0.0037055015563964844, 0.0036978721618652344, 0.0036623477935791016, 0.003705739974975586, 0.003671407699584961, 0.0037565231323242188, 0.003704071044921875, 0.003675222396850586, 0.0037059783935546875, 0.0036535263061523438, 0.0036919116973876953, 0.0036983489990234375, 0.0037031173706054688, 0.003660440444946289, 0.003685474395751953, 0.003720521926879883, 0.0036983489990234375, 0.0037107467651367188, 0.0036551952362060547, 0.003699779510498047, 0.003650188446044922, 0.0037276744842529297, 0.0037000179290771484, 0.00368499755859375, 0.0037946701049804688, 0.003745555877685547, 0.0036995410919189453, 0.003696441650390625, 0.003702402114868164, 0.0036399364471435547, 0.0037078857421875, 0.0036611557006835938, 0.003697633743286133, 0.003774881362915039, 0.0036346912384033203, 0.0037016868591308594, 0.0036520957946777344, 0.003704071044921875, 0.003772258758544922, 0.0036919116973876953, 0.003653287887573242, 0.0037033557891845703, 0.003710508346557617, 0.003683805465698242, 0.0036962032318115234, 0.0036628246307373047, 0.003710508346557617, 0.003653287887573242, 0.003683805465698242, 0.0036847591400146484, 0.0037038326263427734, 0.0036590099334716797, 0.003820657730102539, 0.003691434860229492, 0.003716707229614258, 0.0036966800689697266, 0.0036656856536865234, 0.0037107467651367188, 0.0036764144897460938, 0.0036814212799072266, 0.0037572383880615234, 0.0036890506744384766, 0.0036995410919189453, 0.003650188446044922, 0.003702878952026367, 0.0037271976470947266, 0.003719329833984375, 0.0036764144897460938, 0.0037026405334472656, 0.0036437511444091797, 0.0040035247802734375, 0.003710508346557617, 0.003674745559692383, 0.003700733184814453, 0.003676891326904297, 0.0037174224853515625, 0.003742218017578125, 0.0037131309509277344, 0.0037147998809814453, 0.0037627220153808594, 0.0037145614624023438, 0.003709077835083008, 0.003728151321411133, 0.0036585330963134766, 0.0037000179290771484, 0.0036690235137939453, 0.0037164688110351562, 0.003826618194580078, 0.0037145614624023438, 0.003728151321411133, 0.003676176071166992, 0.003728628158569336, 0.003709554672241211, 0.0037314891815185547, 0.003808259963989258, 0.003824949264526367, 0.0038084983825683594, 0.003776073455810547, 0.0037584304809570312, 0.003713369369506836, 0.003747224807739258, 0.003780364990234375, 0.003811359405517578, 0.0037794113159179688, 0.0037157535552978516, 0.003835916519165039, 0.0037322044372558594, 0.0037627220153808594, 0.003760099411010742, 0.003752470016479492, 0.0037679672241210938, 0.003718852996826172, 0.0037775039672851562, 0.003753662109375, 0.0037543773651123047, 0.0037038326263427734, 0.003768444061279297, 0.0037665367126464844, 0.0038115978240966797, 0.003756999969482422, 0.0037043094635009766, 0.003763914108276367, 0.0037093162536621094, 0.003754138946533203, 0.0037419795989990234, 0.0037584304809570312, 0.003770112991333008, 0.003732919692993164, 0.0037572383880615234, 0.003775358200073242, 0.0037567615509033203, 0.003717184066772461, 0.003846406936645508, 0.003817319869995117, 0.0038406848907470703, 0.003766298294067383, 0.003824949264526367, 0.0037546157836914062, 0.0037119388580322266, 0.003789663314819336, 0.0037505626678466797, 0.0037813186645507812, 0.0037717819213867188, 0.0037806034088134766, 0.0037658214569091797, 0.0037946701049804688, 0.0037479400634765625, 0.003731966018676758, 0.0037627220153808594, 0.003756284713745117, 0.003749847412109375, 0.0037539005279541016, 0.003729581832885742, 0.003764629364013672, 0.0037088394165039062, 0.003751039505004883, 0.0038063526153564453, 0.003812074661254883, 0.0037565231323242188, 0.0038132667541503906, 0.0037620067596435547, 0.0037860870361328125, 0.00376129150390625, 0.003704547882080078, 0.003789663314819336, 0.0037462711334228516, 0.0037729740142822266, 0.003759622573852539, 0.0037195682525634766, 0.0037920475006103516, 0.0037250518798828125, 0.003770112991333008, 0.0038423538208007812, 0.0037314891815185547, 0.003764629364013672, 0.003711223602294922, 0.0037996768951416016, 0.003760814666748047, 0.003786325454711914, 0.0037088394165039062, 0.0037724971771240234, 0.00379180908203125, 0.003759622573852539, 0.0037686824798583984, 0.003737926483154297, 0.003774404525756836, 0.003838062286376953, 0.0037789344787597656, 0.003818035125732422, 0.0037255287170410156, 0.0037660598754882812, 0.003747224807739258, 0.003806591033935547, 0.0037767887115478516, 0.003793954849243164, 0.003780364990234375, 0.003745555877685547, 0.0037784576416015625, 0.0037620067596435547, 0.003785848617553711, 0.0037386417388916016, 0.0037860870361328125, 0.003750324249267578, 0.0038390159606933594, 0.0037927627563476562, 0.003731966018676758, 0.0037949085235595703, 0.0037369728088378906, 0.003765106201171875, 0.0037755966186523438, 0.003778219223022461, 0.0037717819213867188, 0.003828287124633789, 0.0037682056427001953, 0.003809690475463867, 0.0037691593170166016, 0.003742218017578125, 0.0037806034088134766, 0.0037789344787597656, 0.0037810802459716797, 0.0038030147552490234, 0.0038437843322753906, 0.0037877559661865234, 0.003734111785888672, 0.003761768341064453, 0.003833293914794922, 0.0037403106689453125, 0.003771543502807617, 0.0037343502044677734, 0.003776073455810547, 0.0037631988525390625, 0.0037703514099121094, 0.003732919692993164, 0.0038139820098876953, 0.003781557083129883, 0.0038208961486816406, 0.0037937164306640625, 0.003741025924682617, 0.003766298294067383, 0.003821134567260742, 0.0038132667541503906, 0.003780364990234375, 0.003731250762939453, 0.0037763118743896484, 0.0037186145782470703, 0.0037631988525390625, 0.003783702850341797, 0.00377655029296875, 0.003928184509277344, 0.0037157535552978516, 0.003772258758544922, 0.0037605762481689453, 0.0037741661071777344, 0.0037240982055664062, 0.003769397735595703, 0.003767251968383789, 0.003787994384765625, 0.0037622451782226562, 0.003726482391357422, 0.0037827491760253906, 0.003728151321411133, 0.0037703514099121094, 0.003779888153076172, 0.0037729740142822266, 0.003772258758544922, 0.0038890838623046875, 0.0037903785705566406, 0.0037641525268554688, 0.003772258758544922, 0.0037271976470947266, 0.0037779808044433594, 0.0038323402404785156, 0.0037903785705566406, 0.0037703514099121094, 0.0037097930908203125, 0.003767728805541992, 0.003725767135620117, 0.0037806034088134766, 0.0038254261016845703, 0.0037305355072021484, 0.003772735595703125, 0.003744840621948242, 0.0037746429443359375, 0.003763437271118164, 0.003809690475463867, 0.003732442855834961, 0.0037992000579833984, 0.003782510757446289, 0.003761768341064453, 0.0037806034088134766, 0.003724813461303711, 0.003777027130126953, 0.0038995742797851562, 0.003823995590209961, 0.003770589828491211, 0.0037267208099365234, 0.0037751197814941406, 0.003737211227416992, 0.003773212432861328, 0.0037627220153808594, 0.0037860870361328125, 0.003816366195678711, 0.0037446022033691406, 0.0037724971771240234, 0.0037691593170166016, 0.0037856101989746094, 0.003740549087524414, 0.0037920475006103516, 0.003797292709350586, 0.0037877559661865234, 0.0037689208984375, 0.003725767135620117, 0.003801584243774414, 0.003758668899536133, 0.0037751197814941406, 0.00379180908203125, 0.0038001537322998047, 0.0038034915924072266, 0.003885984420776367, 0.0037796497344970703, 0.0038225650787353516, 0.0037975311279296875, 0.003745555877685547, 0.0037949085235595703, 0.003790140151977539, 0.0037925243377685547, 0.00380706787109375, 0.0037958621978759766, 0.003779888153076172, 0.0037527084350585938, 0.0038919448852539062, 0.0038025379180908203, 0.003748655319213867, 0.0037784576416015625, 0.003736734390258789, 0.0037899017333984375, 0.0037851333618164062, 0.003795623779296875, 0.0038046836853027344, 0.003737211227416992, 0.003782033920288086, 0.003801107406616211, 0.003791332244873047, 0.003740549087524414, 0.0038635730743408203, 0.0037958621978759766, 0.0037734508514404297, 0.0037746429443359375, 0.003751993179321289, 0.003782987594604492, 0.0037431716918945312, 0.003789186477661133, 0.0038518905639648438, 0.0037317276000976562, 0.0037794113159179688, 0.0037419795989990234, 0.0037937164306640625, 0.0037975311279296875, 0.003780841827392578, 0.0037975311279296875, 0.003821849822998047, 0.003784656524658203, 0.0037870407104492188, 0.003816366195678711, 0.0037200450897216797, 0.003773212432861328, 0.0037660598754882812, 0.003851175308227539, 0.0037927627563476562, 0.003736257553100586, 0.003780364990234375, 0.0038285255432128906, 0.0038275718688964844, 0.003790140151977539, 0.003782987594604492, 0.0037784576416015625, 0.0037491321563720703, 0.0038084983825683594, 0.0037822723388671875, 0.003778219223022461, 0.0037550926208496094, 0.0037946701049804688, 0.003816843032836914, 0.003787994384765625, 0.003801107406616211, 0.003740549087524414, 0.0038084983825683594, 0.0037529468536376953, 0.003820657730102539, 0.003840208053588867, 0.0037436485290527344, 0.003818035125732422, 0.0037488937377929688, 0.0038046836853027344, 0.0037841796875, 0.003777742385864258, 0.0037512779235839844, 0.0038785934448242188, 0.0037856101989746094, 0.003801107406616211, 0.0037908554077148438, 0.0037457942962646484, 0.0037949085235595703, 0.00373077392578125, 0.0039038658142089844, 0.0038063526153564453, 0.003781557083129883, 0.003810405731201172, 0.0037546157836914062, 0.0038428306579589844, 0.003786325454711914, 0.003863811492919922, 0.0037767887115478516, 0.0037391185760498047, 0.003783702850341797, 0.0037860870361328125, 0.003793001174926758, 0.0037415027618408203, 0.0038559436798095703, 0.003793954849243164, 0.003738880157470703, 0.0037844181060791016, 0.0038034915924072266, 0.0038297176361083984, 0.003887176513671875, 0.0037996768951416016, 0.003782987594604492, 0.003785848617553711, 0.0038111209869384766, 0.0037467479705810547, 0.0037953853607177734, 0.0037403106689453125, 0.003796815872192383, 0.003830432891845703, 0.003750324249267578, 0.003804445266723633, 0.0037577152252197266, 0.0037870407104492188, 0.0037877559661865234, 0.0037910938262939453, 0.003911256790161133, 0.003765583038330078, 0.0038008689880371094, 0.0038051605224609375, 0.0037910938262939453, 0.0037598609924316406, 0.003799915313720703, 0.003805875778198242, 0.0037827491760253906, 0.003797769546508789, 0.003832101821899414, 0.003791332244873047, 0.0037703514099121094, 0.0038013458251953125, 0.003856182098388672, 0.003746509552001953, 0.003815889358520508, 0.003751993179321289, 0.0037872791290283203, 0.0037670135498046875, 0.003805875778198242, 0.003755807876586914, 0.003812551498413086, 0.003772735595703125, 0.003793478012084961, 0.0037834644317626953, 0.0037565231323242188, 0.0037996768951416016, 0.0037555694580078125, 0.003825664520263672, 0.003779888153076172, 0.0037431716918945312, 0.003810405731201172, 0.0037496089935302734, 0.0037848949432373047, 0.0037970542907714844, 0.0038139820098876953, 0.003915548324584961, 0.0037620067596435547, 0.0037949085235595703, 0.0037834644317626953, 0.003796815872192383, 0.0037555694580078125, 0.0037927627563476562, 0.0037870407104492188, 0.003782033920288086, 0.003816843032836914, 0.003741025924682617, 0.0037870407104492188, 0.003747701644897461, 0.003791332244873047, 0.003847837448120117, 0.0037603378295898438, 0.0037763118743896484, 0.0037467479705810547, 0.003788471221923828, 0.003808736801147461, 0.0038022994995117188, 0.003806591033935547, 0.003790616989135742, 0.003799915313720703, 0.003808259963989258, 0.0038268566131591797, 0.0038940906524658203, 0.0038423538208007812, 0.0038607120513916016, 0.0038292407989501953, 0.0038373470306396484, 0.00382232666015625, 0.003839731216430664, 0.0037975311279296875, 0.003832578659057617, 0.004071474075317383, 0.0038039684295654297, 0.0038385391235351562, 0.0037937164306640625, 0.003852367401123047, 0.0038492679595947266, 0.003849029541015625, 0.0038301944732666016, 0.003803253173828125, 0.003851175308227539, 0.003846406936645508, 0.0038361549377441406, 0.0038378238677978516, 0.003852367401123047, 0.0038449764251708984, 0.003837108612060547, 0.0040111541748046875, 0.0039064884185791016, 0.003858804702758789, 0.0038690567016601562, 0.003844022750854492, 0.003833770751953125, 0.0038003921508789062, 0.003833770751953125, 0.0038039684295654297, 0.0038962364196777344, 0.0038423538208007812, 0.0038042068481445312, 0.003836393356323242, 0.003808736801147461, 0.003840208053588867, 0.0038404464721679688, 0.0038444995880126953, 0.0038461685180664062, 0.0038111209869384766, 0.003853321075439453, 0.0038416385650634766, 0.0038208961486816406, 0.0038018226623535156, 0.0038640499114990234, 0.0038661956787109375, 0.0038411617279052734, 0.003835439682006836, 0.003902435302734375, 0.003854990005493164, 0.003852367401123047, 0.0038547515869140625, 0.003847837448120117, 0.0038139820098876953, 0.003852367401123047, 0.0038213729858398438, 0.003843545913696289, 0.003882169723510742, 0.003808736801147461, 0.003859996795654297, 0.0038225650787353516, 0.0038650035858154297, 0.0038475990295410156, 0.003864288330078125, 0.0038671493530273438, 0.0038263797760009766, 0.0038499832153320312, 0.003838062286376953, 0.003843069076538086, 0.003805398941040039, 0.0038461685180664062, 0.0038411617279052734, 0.003854036331176758, 0.0038671493530273438, 0.0039157867431640625, 0.0038480758666992188, 0.0038480758666992188, 0.0038437843322753906, 0.0038459300994873047, 0.0038232803344726562, 0.003855466842651367, 0.0038139820098876953, 0.003894805908203125, 0.0038576126098632812, 0.0038137435913085938, 0.0038568973541259766, 0.0038175582885742188, 0.003843069076538086, 0.0038623809814453125, 0.0038580894470214844, 0.0038666725158691406, 0.003815174102783203, 0.0038404464721679688, 0.0038475990295410156, 0.003836393356323242, 0.003864288330078125, 0.0038204193115234375, 0.0038421154022216797, 0.0038373470306396484, 0.003851175308227539, 0.003953218460083008, 0.003854513168334961, 0.0038597583770751953, 0.0038661956787109375, 0.003850698471069336, 0.003816843032836914, 0.003833770751953125, 0.003816843032836914, 0.0038928985595703125, 0.003854513168334961, 0.0038263797760009766, 0.003847360610961914, 0.003817319869995117, 0.0038483142852783203, 0.003918647766113281, 0.0038671493530273438, 0.0038390159606933594, 0.003805398941040039, 0.0038383007049560547, 0.0038433074951171875, 0.0038504600524902344, 0.003860950469970703, 0.0038232803344726562, 0.0038428306579589844, 0.0038657188415527344, 0.0038750171661376953, 0.0039157867431640625, 0.0038504600524902344, 0.003849029541015625, 0.003859281539916992, 0.0038650035858154297, 0.0038187503814697266, 0.0038547515869140625, 0.003813028335571289, 0.003901243209838867, 0.0038611888885498047, 0.003821134567260742, 0.0038945674896240234, 0.0038497447967529297, 0.003874063491821289, 0.003902912139892578, 0.0038323402404785156, 0.0038635730743408203, 0.0038254261016845703, 0.0038695335388183594, 0.00386810302734375, 0.0038635730743408203, 0.003873586654663086, 0.0038127899169921875, 0.0038683414459228516, 0.003870248794555664, 0.0039293766021728516, 0.003902435302734375, 0.003878355026245117, 0.0038700103759765625, 0.0038809776306152344, 0.003859281539916992, 0.0038144588470458984, 0.003862142562866211, 0.0038831233978271484, 0.0038666725158691406, 0.003877401351928711, 0.003818035125732422, 0.003904581069946289, 0.003840923309326172, 0.003897428512573242, 0.003857135772705078, 0.0038275718688964844, 0.0038559436798095703, 0.003835916519165039, 0.0038673877716064453, 0.003859281539916992, 0.003875255584716797, 0.0038619041442871094, 0.003830432891845703, 0.0038976669311523438, 0.0038526058197021484, 0.0038673877716064453, 0.003993988037109375, 0.0038399696350097656, 0.003857135772705078, 0.0038819313049316406, 0.0038602352142333984, 0.0038313865661621094, 0.0038726329803466797, 0.0038864612579345703, 0.0038814544677734375, 0.0038738250732421875, 0.0038290023803710938, 0.0038988590240478516, 0.0038907527923583984, 0.003864765167236328, 0.0038564205169677734, 0.0038411617279052734, 0.003913164138793945, 0.0038499832153320312, 0.0039212703704833984, 0.003879547119140625, 0.0038437843322753906, 0.0038826465606689453, 0.0038356781005859375, 0.003874540328979492, 0.003873586654663086, 0.003935337066650391, 0.003879547119140625, 0.003849506378173828, 0.003865480422973633, 0.003861665725708008, 0.003881216049194336, 0.0038895606994628906, 0.003913402557373047, 0.0038597583770751953, 0.0038576126098632812, 0.0038673877716064453, 0.003849506378173828, 0.003940105438232422, 0.003875255584716797, 0.003854990005493164, 0.0038619041442871094, 0.0038406848907470703, 0.0038852691650390625, 0.0038738250732421875, 0.003874540328979492, 0.003873109817504883, 0.003868579864501953, 0.003872394561767578, 0.0038299560546875, 0.003921031951904297, 0.003862619400024414, 0.003845691680908203, 0.0039484500885009766, 0.003871440887451172, 0.0038721561431884766, 0.00391697883605957, 0.00384521484375, 0.0038721561431884766, 0.0038216114044189453, 0.003865480422973633, 0.0039017200469970703, 0.0038950443267822266, 0.003885030746459961, 0.0038361549377441406, 0.0038700103759765625, 0.00396418571472168, 0.0038690567016601562, 0.003882169723510742, 0.003873586654663086, 0.003864288330078125, 0.0038330554962158203, 0.0038805007934570312, 0.0038826465606689453, 0.0038650035858154297, 0.003938198089599609, 0.003844738006591797, 0.0038912296295166016, 0.0039768218994140625, 0.0038700103759765625, 0.0038352012634277344, 0.003885030746459961, 0.0038836002349853516, 0.0039000511169433594, 0.0038747787475585938, 0.0038390159606933594, 0.0038764476776123047, 0.004039287567138672, 0.0038945674896240234, 0.0038721561431884766, 0.0038454532623291016, 0.0038971900939941406, 0.0038535594940185547, 0.00392460823059082, 0.0038836002349853516, 0.003840208053588867, 0.003885984420776367, 0.0038633346557617188, 0.003895998001098633, 0.003924369812011719, 0.0038299560546875, 0.0038671493530273438, 0.0038576126098632812, 0.003876924514770508, 0.004262208938598633, 0.003860950469970703, 0.003889322280883789, 0.0038535594940185547, 0.003904581069946289, 0.003872394561767578, 0.003908395767211914, 0.003889322280883789, 0.0038573741912841797, 0.0038852691650390625, 0.0038890838623046875, 0.003880023956298828, 0.003834962844848633, 0.003900766372680664, 0.003895998001098633, 0.0038771629333496094, 0.0038764476776123047, 0.003854036331176758, 0.003889799118041992, 0.003906965255737305, 0.003874063491821289, 0.003880739212036133, 0.003847837448120117, 0.003882169723510742, 0.0038437843322753906, 0.004010915756225586, 0.0038886070251464844, 0.0038535594940185547, 0.0038788318634033203, 0.003843545913696289, 0.0038902759552001953, 0.003948211669921875, 0.0038623809814453125, 0.0038900375366210938, 0.0038597583770751953, 0.00388336181640625, 0.003885030746459961, 0.003916501998901367, 0.003897428512573242, 0.0038633346557617188, 0.0038843154907226562, 0.003876209259033203, 0.003895998001098633, 0.003916740417480469, 0.0038611888885498047, 0.0038840770721435547, 0.0038743019104003906, 0.003871917724609375, 0.0038480758666992188, 0.003907442092895508, 0.0038928985595703125, 0.0038967132568359375, 0.003966569900512695, 0.0038912296295166016, 0.003888368606567383, 0.0038971900939941406, 0.0038979053497314453, 0.0038962364196777344, 0.003850698471069336, 0.0038881301879882812, 0.003845691680908203, 0.003927707672119141, 0.0038900375366210938, 0.0038597583770751953, 0.003904104232788086, 0.0038671493530273438, 0.0038971900939941406, 0.003941774368286133, 0.0038547515869140625, 0.003899812698364258, 0.0038499832153320312, 0.003896951675415039, 0.003893613815307617, 0.0039005279541015625, 0.00409388542175293, 0.003871440887451172, 0.0038886070251464844, 0.003989458084106445, 0.003903627395629883, 0.003898143768310547, 0.0038471221923828125, 0.0038907527923583984, 0.0038967132568359375, 0.003883838653564453, 0.003920316696166992, 0.0038633346557617188, 0.0039014816284179688, 0.0038797855377197266, 0.0038890838623046875, 0.0038394927978515625, 0.003901243209838867, 0.0038938522338867188, 0.0038805007934570312, 0.0038919448852539062, 0.0038650035858154297, 0.0038895606994628906, 0.0039043426513671875, 0.003877878189086914, 0.003887176513671875, 0.003871917724609375, 0.0038950443267822266, 0.003853321075439453, 0.0039234161376953125, 0.003968238830566406, 0.0038814544677734375, 0.003873586654663086, 0.0038542747497558594, 0.0038864612579345703, 0.00393366813659668, 0.003855466842651367, 0.0038776397705078125, 0.003840923309326172, 0.0038983821868896484, 0.0038819313049316406, 0.0039033889770507812, 0.003879547119140625, 0.003840923309326172, 0.0039048194885253906, 0.00389862060546875, 0.003901243209838867, 0.003881692886352539, 0.003880023956298828, 0.003907442092895508, 0.0039048194885253906, 0.003928184509277344, 0.0038728713989257812, 0.003930330276489258, 0.003983974456787109, 0.00401759147644043, 0.003931760787963867, 0.0038881301879882812, 0.003920555114746094, 0.003931283950805664, 0.003934383392333984, 0.0042116641998291016, 0.003974199295043945, 0.003924369812011719, 0.003935575485229492, 0.0039632320404052734, 0.0039386749267578125, 0.0038917064666748047, 0.0039484500885009766, 0.003973484039306641, 0.003927946090698242, 0.003934383392333984, 0.003876209259033203, 0.003939151763916016, 0.0038683414459228516, 0.0040073394775390625, 0.00393223762512207, 0.0038726329803466797, 0.003940582275390625, 0.003885984420776367, 0.004457950592041016, 0.004085063934326172, 0.0042362213134765625, 0.003951072692871094, 0.003907918930053711, 0.003992795944213867, 0.0039539337158203125, 0.003899097442626953, 0.003942728042602539, 0.0038886070251464844, 0.003968000411987305, 0.00392460823059082, 0.003899097442626953, 0.003925800323486328, 0.003882169723510742, 0.003942728042602539, 0.003985166549682617, 0.003951072692871094, 0.003939628601074219, 0.003907442092895508, 0.00393366813659668, 0.003992795944213867, 0.0038933753967285156, 0.003923177719116211, 0.0038924217224121094, 0.003937244415283203, 0.0040225982666015625, 0.003938436508178711, 0.003931760787963867, 0.003938198089599609, 0.003948688507080078, 0.003939628601074219, 0.00393366813659668, 0.003922462463378906, 0.0038971900939941406, 0.00393986701965332, 0.003941535949707031, 0.0039215087890625, 0.003932476043701172, 0.003892660140991211, 0.003942012786865234, 0.0039441585540771484, 0.003934383392333984, 0.003947019577026367, 0.003896951675415039, 0.003949165344238281, 0.003940105438232422, 0.00395965576171875, 0.0038933753967285156, 0.003946065902709961, 0.0039522647857666016, 0.003953456878662109, 0.003992557525634766, 0.003890514373779297, 0.003943443298339844, 0.003931522369384766, 0.003937721252441406, 0.003922939300537109, 0.0038957595825195312, 0.003935337066650391, 0.003955841064453125, 0.003937244415283203, 0.0039255619049072266, 0.0038957595825195312, 0.0039157867431640625, 0.0039048194885253906, 0.00397944450378418, 0.0039310455322265625, 0.0038874149322509766, 0.003934144973754883, 0.0039043426513671875, 0.003987312316894531, 0.003964424133300781, 0.0038862228393554688, 0.00397491455078125, 0.003897428512573242, 0.004027366638183594, 0.003988504409790039, 0.0038852691650390625, 0.003917694091796875, 0.0038940906524658203, 0.003929615020751953, 0.003973245620727539, 0.003897428512573242, 0.003929853439331055, 0.003904104232788086, 0.004099369049072266, 0.0039272308349609375, 0.003933429718017578, 0.003933429718017578, 0.0038847923278808594, 0.003938913345336914, 0.003919124603271484, 0.003954410552978516, 0.003921031951904297, 0.003906965255737305, 0.003936767578125, 0.003937244415283203, 0.003908634185791016, 0.003947019577026367, 0.003928422927856445, 0.004094123840332031, 0.0039517879486083984, 0.003996372222900391, 0.003993988037109375, 0.00395965576171875, 0.0039010047912597656, 0.003977775573730469, 0.003910541534423828, 0.003996372222900391, 0.003941774368286133, 0.003908872604370117, 0.003941774368286133, 0.003907442092895508, 0.003958225250244141, 0.003989219665527344, 0.003922939300537109, 0.0039479732513427734, 0.003896951675415039, 0.00393986701965332, 0.004007816314697266, 0.003916025161743164, 0.003933429718017578, 0.0038912296295166016, 0.003926277160644531, 0.003950595855712891, 0.003958940505981445, 0.004023075103759766, 0.003914833068847656, 0.0041027069091796875, 0.0043447017669677734, 0.004461526870727539, 0.004342555999755859, 0.004252433776855469, 0.004266023635864258, 0.004258155822753906, 0.004230499267578125, 0.004820585250854492, 0.004266500473022461, 0.0046803951263427734, 0.006506443023681641, 0.007000923156738281, 0.006875038146972656, 0.006822824478149414, 0.00690770149230957, 0.006875753402709961, 0.0070037841796875, 0.006422758102416992, 0.0061457157135009766, 0.0062563419342041016, 0.006198883056640625, 0.006352663040161133, 0.005103349685668945, 0.0040853023529052734, 0.004189729690551758, 0.0050928592681884766, 0.004042863845825195, 0.006006002426147461, 0.004001617431640625, 0.003965616226196289, 0.003949165344238281, 0.004021406173706055, 0.00396728515625, 0.003943681716918945, 0.003960847854614258, 0.003954410552978516, 0.003974437713623047, 0.004078388214111328, 0.003960371017456055, 0.00393366813659668, 0.0039522647857666016, 0.003974437713623047, 0.004021644592285156, 0.003939390182495117, 0.003911256790161133, 0.003968000411987305, 0.003920793533325195, 0.004030942916870117, 0.00397038459777832, 0.003930091857910156, 0.0039637088775634766, 0.004436016082763672, 0.0040471553802490234, 0.0039637088775634766, 0.003952741622924805, 0.0042266845703125, 0.004003047943115234, 0.003975868225097656, 0.003946065902709961, 0.00391697883605957, 0.003940105438232422, 0.003933906555175781, 0.00400233268737793, 0.003942251205444336, 0.004106760025024414, 0.003972768783569336, 0.003912448883056641, 0.004009246826171875, 0.003949880599975586, 0.003914594650268555, 0.003946065902709961, 0.003974199295043945, 0.004008054733276367, 0.003961086273193359, 0.003965139389038086, 0.003936290740966797, 0.0039098262786865234, 0.004006385803222656, 0.003961324691772461, 0.003911018371582031, 0.00396418571472168, 0.003916501998901367, 0.003926515579223633, 0.0039980411529541016, 0.0039043426513671875, 0.003954410552978516, 0.00391840934753418, 0.003944873809814453, 0.004075050354003906, 0.003917217254638672, 0.0039463043212890625, 0.003935337066650391, 0.0039539337158203125, 0.003969669342041016, 0.003990650177001953, 0.003962039947509766, 0.003923177719116211, 0.003970623016357422, 0.00397181510925293, 0.004534721374511719, 0.004004716873168945, 0.003960132598876953, 0.003957033157348633, 0.003966808319091797, 0.003953218460083008, 0.003992795944213867, 0.003968954086303711, 0.003953695297241211, 0.003960132598876953, 0.003998279571533203, 0.00397944450378418, 0.0039520263671875, 0.003983736038208008, 0.003942251205444336, 0.0040662288665771484, 0.003934144973754883, 0.003964662551879883, 0.00395512580871582, 0.003960609436035156, 0.003989458084106445, 0.003935575485229492, 0.003957033157348633, 0.003964662551879883, 0.003966569900512695, 0.003977775573730469, 0.003973960876464844, 0.003992319107055664, 0.003962516784667969, 0.0039560794830322266, 0.004014492034912109, 0.003936052322387695, 0.0039861202239990234, 0.0039751529693603516, 0.0039594173431396484, 0.0039081573486328125, 0.004007101058959961, 0.003969907760620117, 0.003985404968261719, 0.0040435791015625, 0.0039348602294921875, 0.00397181510925293, 0.0039806365966796875, 0.003943204879760742, 0.0039463043212890625, 0.0039157867431640625, 0.003980159759521484, 0.003963947296142578, 0.003971576690673828, 0.003952503204345703, 0.003918886184692383, 0.0039598941802978516, 0.0039713382720947266, 0.004038572311401367, 0.003951549530029297, 0.003930091857910156, 0.0039653778076171875, 0.004012107849121094, 0.003957271575927734, 0.003972053527832031, 0.003937482833862305, 0.003971576690673828, 0.004026651382446289, 0.004003047943115234, 0.003960609436035156, 0.003999471664428711, 0.004096031188964844, 0.003988981246948242, 0.003969430923461914, 0.00398707389831543, 0.0039272308349609375, 0.0039560794830322266, 0.003983497619628906, 0.003950595855712891, 0.003967761993408203, 0.003912210464477539, 0.003954648971557617, 0.003940582275390625, 0.004011631011962891, 0.003979682922363281, 0.0039196014404296875, 0.003952741622924805, 0.003942728042602539, 0.004002809524536133, 0.0039784908294677734, 0.003927707672119141, 0.003965616226196289, 0.00397491455078125, 0.004009723663330078, 0.0040400028228759766, 0.003921985626220703, 0.003983974456787109, 0.003941535949707031, 0.003999948501586914, 0.003963470458984375, 0.003923177719116211, 0.003962039947509766, 0.0039250850677490234, 0.0041425228118896484, 0.003968000411987305, 0.003930091857910156, 0.003974437713623047, 0.003939151763916016, 0.003972291946411133, 0.004013538360595703, 0.003908395767211914, 0.003973484039306641, 0.003943443298339844, 0.003968000411987305, 0.0040051937103271484, 0.003921985626220703, 0.00396418571472168, 0.003935575485229492, 0.003962993621826172, 0.0040934085845947266, 0.0039196014404296875, 0.00410914421081543, 0.003937244415283203, 0.003979206085205078, 0.004017829895019531, 0.003929615020751953, 0.00396728515625, 0.003939151763916016, 0.003964662551879883, 0.003969669342041016, 0.003984928131103516, 0.003983259201049805, 0.003926753997802734, 0.003960609436035156, 0.003968477249145508, 0.003969907760620117, 0.003977298736572266, 0.003951549530029297, 0.003988504409790039, 0.00395965576171875, 0.00397181510925293, 0.003958225250244141, 0.0039234161376953125, 0.0039746761322021484, 0.003965616226196289, 0.004098415374755859, 0.003975868225097656, 0.003939390182495117, 0.003963947296142578, 0.003965854644775391, 0.003961086273193359, 0.00397181510925293, 0.003935575485229492, 0.003965854644775391, 0.00396418571472168, 0.003968715667724609, 0.004228353500366211, 0.004076719284057617, 0.004023551940917969, 0.0041162967681884766, 0.004101991653442383, 0.004046440124511719, 0.00405120849609375, 0.0040416717529296875, 0.004032611846923828, 0.004086971282958984, 0.00403141975402832, 0.0040416717529296875, 0.003991127014160156, 0.00402522087097168, 0.004033327102661133, 0.004132986068725586, 0.004063129425048828, 0.003996372222900391, 0.004026889801025391, 0.004033803939819336, 0.0040285587310791016, 0.004030942916870117, 0.003989219665527344, 0.004045248031616211, 0.004010677337646484, 0.004081249237060547, 0.004035472869873047, 0.003979921340942383, 0.004041910171508789, 0.004048347473144531, 0.004041910171508789, 0.0040471553802490234, 0.004050254821777344, 0.003997325897216797, 0.004032135009765625, 0.004033803939819336, 0.004034757614135742, 0.004071474075317383, 0.004113435745239258, 0.00405573844909668, 0.00404667854309082, 0.004062652587890625, 0.004034996032714844, 0.00399470329284668, 0.0040395259857177734, 0.004045724868774414, 0.00404047966003418, 0.004047870635986328, 0.003999471664428711, 0.004022359848022461, 0.0040187835693359375, 0.004042387008666992, 0.004083395004272461, 0.004006147384643555, 0.004019975662231445, 0.004045963287353516, 0.004069328308105469, 0.004025697708129883, 0.003986358642578125, 0.004052162170410156, 0.0040509700775146484, 0.004053354263305664, 0.004022359848022461, 0.00400090217590332, 0.004030466079711914, 0.004181623458862305, 0.004002094268798828, 0.004030704498291016, 0.003990650177001953, 0.004029035568237305, 0.004088640213012695, 0.004004716873168945, 0.004015207290649414, 0.003984928131103516, 0.0040204524993896484, 0.004095315933227539, 0.003999233245849609, 0.004042625427246094, 0.003990650177001953, 0.004021644592285156, 0.0040760040283203125, 0.003996849060058594, 0.004041433334350586, 0.003984212875366211, 0.004017353057861328, 0.004082441329956055, 0.004004716873168945, 0.004029512405395508, 0.00413823127746582, 0.0041086673736572266, 0.004044771194458008, 0.004029035568237305, 0.004026889801025391, 0.004011869430541992, 0.004119157791137695, 0.004041194915771484, 0.003998994827270508, 0.004038095474243164, 0.0039827823638916016, 0.0044765472412109375, 0.004059553146362305, 0.004036664962768555, 0.004029989242553711, 0.004036426544189453, 0.004019021987915039, 0.004036664962768555, 0.003994464874267578, 0.004023551940917969, 0.004043102264404297, 0.004030704498291016, 0.004027843475341797, 0.0040051937103271484, 0.0040929317474365234, 0.00402522087097168, 0.0043561458587646484, 0.004071950912475586, 0.004002809524536133, 0.004073619842529297, 0.004022121429443359, 0.004027128219604492, 0.004057407379150391, 0.00404047966003418, 0.004003286361694336, 0.0040395259857177734, 0.004045963287353516, 0.004040241241455078, 0.004045724868774414, 0.003998517990112305, 0.004030466079711914, 0.00410151481628418, 0.004054069519042969, 0.0040509700775146484, 0.004015445709228516, 0.004029512405395508, 0.004048824310302734, 0.004077911376953125, 0.0040547847747802734, 0.0040853023529052734, 0.004046916961669922, 0.0040476322174072266, 0.004088878631591797, 0.004044055938720703, 0.004006624221801758, 0.004032611846923828, 0.0040493011474609375, 0.004049777984619141, 0.0040302276611328125, 0.004007816314697266, 0.004044055938720703, 0.004148006439208984, 0.00400543212890625, 0.004045724868774414, 0.003988981246948242, 0.0040242671966552734, 0.004092693328857422, 0.004005908966064453, 0.004057645797729492, 0.003994464874267578, 0.004045724868774414, 0.004064321517944336, 0.004000425338745117, 0.004030704498291016, 0.00400233268737793, 0.0040318965911865234, 0.004174470901489258, 0.0040090084075927734, 0.004036426544189453, 0.0040073394775390625, 0.004090070724487305, 0.0040340423583984375, 0.003996372222900391, 0.004040956497192383, 0.00400090217590332, 0.0040814876556396484, 0.0040285587310791016, 0.004024505615234375, 0.0040607452392578125, 0.003998756408691406, 0.004079341888427734, 0.004045248031616211, 0.003997087478637695, 0.004046440124511719, 0.0039975643157958984, 0.004114866256713867, 0.0040874481201171875, 0.0039899349212646484, 0.004059553146362305, 0.004143714904785156, 0.004045724868774414, 0.0040590763092041016, 0.004008054733276367, 0.0040662288665771484, 0.004038810729980469, 0.004042625427246094, 0.004039764404296875, 0.004004955291748047, 0.0041713714599609375, 0.004058837890625, 0.004060268402099609, 0.004052639007568359, 0.00400853157043457, 0.004054069519042969, 0.0040280818939208984, 0.004033565521240234, 0.004047393798828125, 0.0040187835693359375, 0.004071235656738281, 0.004042148590087891, 0.0040378570556640625, 0.0040585994720458984, 0.004125833511352539, 0.004117727279663086, 0.0041866302490234375, 0.004162311553955078, 0.0040814876556396484, 0.004071950912475586, 0.004053354263305664, 0.004052162170410156, 0.004033803939819336, 0.004064083099365234, 0.004072904586791992, 0.0040721893310546875, 0.004046440124511719, 0.004015684127807617, 0.0040569305419921875, 0.004053592681884766, 0.004071235656738281, 0.004049062728881836, 0.0040738582611083984, 0.0040128231048583984, 0.004059314727783203, 0.004075288772583008, 0.004050731658935547, 0.00404810905456543, 0.004012584686279297, 0.004059791564941406, 0.0041501522064208984, 0.004073143005371094, 0.004038333892822266, 0.004008293151855469, 0.004303455352783203, 0.004060268402099609, 0.004052400588989258, 0.004081249237060547, 0.004015207290649414, 0.004044771194458008, 0.004090785980224609, 0.0040740966796875, 0.004034519195556641, 0.004158973693847656, 0.004094839096069336, 0.00412440299987793, 0.004059553146362305, 0.004068851470947266, 0.004065275192260742, 0.0041277408599853516, 0.0040781497955322266, 0.004065752029418945, 0.004026174545288086, 0.0040547847747802734, 0.004107475280761719, 0.004036664962768555, 0.0041425228118896484, 0.004180431365966797, 0.004102945327758789, 0.004078388214111328, 0.004046440124511719, 0.0040607452392578125, 0.004034280776977539, 0.004218339920043945, 0.004048824310302734, 0.004044294357299805, 0.004062175750732422, 0.00405120849609375, 0.004056453704833984, 0.004033088684082031, 0.0040585994720458984, 0.004042148590087891, 0.004039287567138672, 0.004062652587890625, 0.004011392593383789, 0.0040667057037353516, 0.004099845886230469, 0.004016399383544922, 0.0040514469146728516, 0.0041141510009765625, 0.0040509700775146484, 0.004133462905883789, 0.0040187835693359375, 0.004032611846923828, 0.00402522087097168, 0.0040547847747802734, 0.004084348678588867, 0.004013538360595703, 0.00407099723815918, 0.004014253616333008, 0.004111766815185547, 0.004052400588989258, 0.004305124282836914, 0.004088401794433594, 0.00407862663269043, 0.004058122634887695, 0.004050731658935547, 0.004209995269775391, 0.00406646728515625, 0.004065036773681641, 0.004070281982421875, 0.004076480865478516, 0.0040073394775390625, 0.004049539566040039, 0.0040569305419921875, 0.004046201705932617, 0.004185676574707031, 0.004010915756225586, 0.004078388214111328, 0.004090070724487305, 0.004060506820678711, 0.004049777984619141, 0.004057407379150391, 0.004023551940917969, 0.004066944122314453, 0.0041043758392333984, 0.004053831100463867, 0.004060983657836914, 0.004024505615234375, 0.004062652587890625, 0.004049777984619141, 0.004070281982421875, 0.004071712493896484, 0.0040166378021240234, 0.004060983657836914, 0.0040531158447265625, 0.0040585994720458984, 0.004099130630493164, 0.004101753234863281, 0.004073143005371094, 0.004048347473144531, 0.0040705204010009766, 0.004041194915771484, 0.004020214080810547, 0.0040743350982666016, 0.004142284393310547, 0.004009246826171875, 0.0040435791015625, 0.0040302276611328125, 0.004103422164916992, 0.0041272640228271484, 0.004026889801025391, 0.0040547847747802734, 0.004021406173706055, 0.004052162170410156, 0.004139423370361328, 0.004021167755126953, 0.00406193733215332, 0.0040280818939208984, 0.004124641418457031, 0.0040662288665771484, 0.004021644592285156, 0.004120588302612305, 0.004050731658935547, 0.0041124820709228516, 0.004155158996582031, 0.004029035568237305, 0.004072666168212891, 0.0040853023529052734, 0.004065036773681641, 0.0040667057037353516, 0.004029273986816406, 0.004079580307006836, 0.0040819644927978516, 0.004080295562744141, 0.004068136215209961, 0.004023551940917969, 0.00407099723815918, 0.004065036773681641, 0.004113912582397461, 0.004075765609741211, 0.004036426544189453, 0.0040743350982666016, 0.0040721893310546875, 0.0040972232818603516, 0.004091501235961914, 0.0040929317474365234, 0.004121303558349609, 0.00407862663269043, 0.004057407379150391, 0.004086017608642578, 0.004061222076416016, 0.004025459289550781, 0.004099130630493164, 0.004073619842529297, 0.004094839096069336, 0.004082441329956055, 0.0040242671966552734, 0.004067182540893555, 0.004076957702636719, 0.004086732864379883, 0.004076957702636719, 0.00402379035949707, 0.0040628910064697266, 0.004129171371459961, 0.004032611846923828, 0.004064798355102539, 0.004079341888427734, 0.0040738582611083984, 0.004130125045776367, 0.0040509700775146484, 0.004053831100463867, 0.004021167755126953, 0.004115581512451172, 0.0042285919189453125, 0.004109382629394531, 0.004128694534301758, 0.004121541976928711, 0.004091978073120117, 0.004103660583496094, 0.004104137420654297, 0.004118680953979492, 0.004152536392211914, 0.00411224365234375, 0.004102468490600586, 0.0041925907135009766, 0.0041773319244384766, 0.004098653793334961, 0.004099130630493164, 0.005034685134887695, 0.00416111946105957, 0.004123687744140625, 0.004251003265380859, 0.004584074020385742, 0.004160165786743164, 0.0041162967681884766, 0.004450321197509766, 0.004185914993286133, 0.004122257232666016, 0.004118204116821289, 0.00418853759765625, 0.004086971282958984, 0.004104137420654297, 0.004117250442504883, 0.004271507263183594, 0.004170417785644531, 0.004074573516845703, 0.004133701324462891, 0.004210710525512695, 0.0041158199310302734, 0.004106760025024414, 0.004071235656738281, 0.004229068756103516, 0.004101276397705078, 0.004068613052368164, 0.00411534309387207, 0.004121303558349609, 0.00416111946105957, 0.004119396209716797, 0.0040853023529052734, 0.004123687744140625, 0.004424571990966797, 0.004223346710205078, 0.004099607467651367, 0.0041921138763427734, 0.004191875457763672, 0.004106760025024414, 0.004099845886230469, 0.004115581512451172, 0.004186868667602539, 0.004125118255615234, 0.004077434539794922, 0.004106283187866211, 0.004159212112426758, 0.00412440299987793, 0.004121541976928711, 0.0040781497955322266, 0.0042150020599365234, 0.004130363464355469, 0.004136323928833008, 0.004129171371459961, 0.004134654998779297, 0.004071950912475586, 0.004101991653442383, 0.00421142578125, 0.004140377044677734, 0.004109382629394531, 0.004074573516845703, 0.004112958908081055, 0.004130125045776367, 0.004160165786743164, 0.004103899002075195, 0.0040514469146728516, 0.004113674163818359, 0.004181623458862305, 0.004086017608642578, 0.004122495651245117, 0.004075288772583008, 0.004155874252319336, 0.004146575927734375, 0.004082441329956055, 0.004095792770385742, 0.004077434539794922, 0.004198551177978516, 0.004128932952880859, 0.004090309143066406, 0.00411224365234375, 0.004126787185668945, 0.004113674163818359, 0.004118680953979492, 0.004098176956176758, 0.004260540008544922, 0.0041043758392333984, 0.0041048526763916016, 0.004126787185668945, 0.00414729118347168, 0.004096031188964844, 0.004118680953979492, 0.004116535186767578, 0.004230022430419922, 0.004128932952880859, 0.004075527191162109, 0.004117012023925781, 0.00417017936706543, 0.0041196346282958984, 0.0041255950927734375, 0.00407719612121582, 0.004105329513549805, 0.0041561126708984375, 0.004086971282958984, 0.004112720489501953, 0.004071235656738281, 0.00426483154296875, 0.004103422164916992, 0.004088878631591797, 0.004118204116821289, 0.004128217697143555, 0.004120588302612305, 0.004135847091674805, 0.0040814876556396484, 0.0041446685791015625, 0.0041027069091796875, 0.004114627838134766, 0.004121065139770508, 0.004090309143066406, 0.0041582584381103516, 0.004129171371459961, 0.004134178161621094, 0.004106998443603516, 0.0041315555572509766, 0.004094362258911133, 0.004129886627197266, 0.004109382629394531, 0.004140615463256836, 0.004105329513549805, 0.00411677360534668, 0.004132270812988281, 0.004173994064331055, 0.004086971282958984, 0.004254579544067383, 0.004124164581298828, 0.00417637825012207, 0.00426793098449707, 0.0044057369232177734, 0.004150867462158203, 0.0042035579681396484, 0.004086494445800781, 0.004116535186767578, 0.0041255950927734375, 0.00420689582824707, 0.004148721694946289, 0.00407862663269043, 0.004115581512451172, 0.004147768020629883, 0.004153251647949219, 0.004132986068725586, 0.0041043758392333984, 0.0041599273681640625, 0.004119157791137695, 0.004136800765991211, 0.004130363464355469, 0.00416874885559082, 0.00414729118347168, 0.004133701324462891, 0.004129171371459961, 0.004136323928833008, 0.004144906997680664, 0.00420379638671875, 0.004119873046875, 0.004128694534301758, 0.0041561126708984375, 0.004131317138671875, 0.0041429996490478516, 0.0041217803955078125, 0.0041959285736083984, 0.004094600677490234, 0.00412440299987793, 0.0041315555572509766, 0.0041468143463134766, 0.004142284393310547, 0.004082202911376953, 0.004130840301513672, 0.0041887760162353516, 0.004083871841430664, 0.004155158996582031, 0.004097700119018555, 0.004206657409667969, 0.0041408538818359375, 0.004178762435913086, 0.004128456115722656, 0.004101753234863281, 0.004168510437011719, 0.004135608673095703, 0.0041387081146240234, 0.004143238067626953, 0.00413966178894043, 0.004137754440307617, 0.004132986068725586, 0.004086732864379883, 0.004160642623901367, 0.0041446685791015625, 0.004123687744140625, 0.004186153411865234, 0.004148006439208984, 0.004101991653442383, 0.004144430160522461, 0.00413823127746582, 0.004149436950683594, 0.0041391849517822266, 0.004168033599853516, 0.004177093505859375, 0.004206418991088867, 0.0041446685791015625, 0.004146099090576172, 0.0041124820709228516, 0.004187107086181641, 0.004149913787841797, 0.004105806350708008, 0.0041468143463134766, 0.004167079925537109, 0.0041408538818359375, 0.00413966178894043, 0.0041081905364990234, 0.004144191741943359, 0.004128456115722656, 0.004151344299316406, 0.004157543182373047, 0.004110813140869141, 0.004163026809692383, 0.004148960113525391, 0.004170656204223633, 0.004145383834838867, 0.004130840301513672, 0.004086494445800781, 0.004186868667602539, 0.004141807556152344, 0.004243135452270508, 0.004180192947387695, 0.004095315933227539, 0.004146099090576172, 0.004208803176879883, 0.004124879837036133, 0.004156351089477539, 0.004092216491699219, 0.004219532012939453, 0.004147529602050781, 0.0040934085845947266, 0.0041561126708984375, 0.0041294097900390625, 0.004148721694946289, 0.00414729118347168, 0.004106998443603516, 0.004132509231567383, 0.0041561126708984375, 0.00416874885559082, 0.004162788391113281, 0.004190206527709961, 0.004185914993286133, 0.004145622253417969, 0.004171609878540039, 0.004184722900390625, 0.00414586067199707, 0.004106044769287109, 0.004137277603149414, 0.004194498062133789, 0.004101753234863281, 0.004304409027099609, 0.004111051559448242, 0.004197359085083008, 0.004141807556152344, 0.00411534309387207, 0.004128456115722656, 0.004146575927734375, 0.0041446685791015625, 0.004149675369262695, 0.004091978073120117, 0.0042040348052978516, 0.004145145416259766, 0.004161834716796875, 0.004149198532104492, 0.0042035579681396484, 0.004276752471923828, 0.004129648208618164, 0.004149198532104492, 0.004275321960449219, 0.004159212112426758, 0.00410008430480957, 0.0041387081146240234, 0.004196882247924805, 0.004107236862182617, 0.0041599273681640625, 0.004150390625, 0.004206180572509766, 0.004154682159423828, 0.004121303558349609, 0.004161834716796875, 0.004164695739746094, 0.004149198532104492, 0.0041425228118896484, 0.00410771369934082, 0.004140615463256836, 0.0041577816009521484, 0.00416254997253418, 0.004152774810791016, 0.004122495651245117, 0.004269123077392578, 0.0041468143463134766, 0.004157543182373047, 0.004149198532104492, 0.004370450973510742, 0.00413823127746582, 0.004151105880737305, 0.004197359085083008, 0.004101276397705078, 0.004133701324462891, 0.004133701324462891, 0.0041692256927490234, 0.004146099090576172, 0.004110097885131836, 0.0041658878326416016, 0.0041196346282958984, 0.004187583923339844, 0.00417017936706543, 0.004105091094970703, 0.004143953323364258, 0.004181623458862305, 0.004137992858886719, 0.004151821136474609, 0.004100322723388672, 0.004169940948486328, 0.004158735275268555, 0.004145145416259766, 0.004216670989990234, 0.004199028015136719, 0.004137992858886719, 0.004154205322265625, 0.00414586067199707, 0.004322052001953125, 0.004217624664306641, 0.0041315555572509766, 0.004179239273071289, 0.0042362213134765625, 0.004109382629394531, 0.004160404205322266, 0.004134654998779297, 0.0044596195220947266, 0.0041561126708984375, 0.004179954528808594, 0.004177093505859375, 0.004146575927734375, 0.004159212112426758, 0.004197597503662109, 0.004156589508056641, 0.00420379638671875, 0.004174470901489258, 0.004169464111328125, 0.0042018890380859375, 0.00414586067199707, 0.004112720489501953, 0.004149913787841797, 0.004208564758300781, 0.004118919372558594, 0.004158496856689453, 0.004109859466552734, 0.004209280014038086, 0.004163503646850586, 0.004118680953979492, 0.004141807556152344, 0.004185914993286133, 0.0041387081146240234, 0.004149675369262695, 0.004119873046875, 0.004148721694946289, 0.004157543182373047, 0.004167079925537109, 0.0041577816009521484, 0.004103422164916992, 0.004147052764892578, 0.004143714904785156, 0.004158973693847656, 0.004233121871948242, 0.004177570343017578, 0.0041027069091796875, 0.0041577816009521484, 0.004155874252319336, 0.004164695739746094, 0.004127979278564453, 0.004112720489501953, 0.004148244857788086, 0.0042629241943359375, 0.004129886627197266, 0.004168510437011719, 0.004120826721191406, 0.004221439361572266, 0.004166364669799805, 0.0041408538818359375, 0.0041446685791015625, 0.004132747650146484, 0.004189014434814453, 0.0041561126708984375, 0.004137992858886719, 0.004322528839111328, 0.004172086715698242, 0.004149675369262695, 0.004217863082885742, 0.0042574405670166016, 0.004181623458862305, 0.004572629928588867, 0.0044286251068115234, 0.0041735172271728516, 0.004218339920043945, 0.004183769226074219, 0.004278659820556641, 0.004208087921142578, 0.004177570343017578, 0.004232883453369141, 0.004197835922241211, 0.004190206527709961, 0.0042095184326171875, 0.0042307376861572266, 0.004194736480712891, 0.004214048385620117, 0.0042035579681396484, 0.004239320755004883, 0.00419306755065918, 0.004760265350341797, 0.004230499267578125, 0.004210710525512695, 0.004265546798706055, 0.004258155822753906, 0.004207611083984375, 0.004181861877441406, 0.004215717315673828, 0.004192829132080078, 0.004227876663208008, 0.004214763641357422, 0.004183530807495117, 0.004228115081787109, 0.0042073726654052734, 0.0041904449462890625, 0.0042302608489990234, 0.004321098327636719, 0.004221200942993164, 0.004206418991088867, 0.0041806697845458984, 0.004253864288330078, 0.004224061965942383, 0.0042133331298828125, 0.0043103694915771484, 0.004326343536376953, 0.0041811466217041016, 0.004247188568115234, 0.004266500473022461, 0.004160881042480469, 0.0042116641998291016, 0.004221677780151367, 0.004254341125488281, 0.004198312759399414, 0.004180192947387695, 0.004271030426025391, 0.004243612289428711, 0.004226207733154297, 0.00422358512878418, 0.004218578338623047, 0.004189252853393555, 0.0042498111724853516, 0.004227876663208008, 0.004247188568115234, 0.004230976104736328, 0.00418543815612793, 0.004221916198730469, 0.0042781829833984375, 0.004192352294921875, 0.004209756851196289, 0.004187822341918945, 0.0043506622314453125, 0.00421905517578125, 0.004195451736450195, 0.004240274429321289, 0.004258632659912109, 0.004241466522216797, 0.004209041595458984, 0.004230499267578125, 0.004176616668701172, 0.004250764846801758, 0.004196643829345703, 0.004266977310180664, 0.00419926643371582, 0.0041751861572265625, 0.004244089126586914, 0.004269123077392578, 0.004182577133178711, 0.004197835922241211, 0.004253864288330078, 0.004275321960449219, 0.0042362213134765625, 0.004281282424926758, 0.004233837127685547, 0.004229068756103516, 0.004247903823852539, 0.004214286804199219, 0.004220485687255859, 0.0041849613189697266, 0.0042264461517333984, 0.004292488098144531, 0.004351377487182617, 0.004216670989990234, 0.004189729690551758, 0.00428318977355957, 0.00424504280090332, 0.0041904449462890625, 0.0042209625244140625, 0.004223823547363281, 0.004218339920043945, 0.004231691360473633, 0.0042400360107421875, 0.004195690155029297, 0.004241466522216797, 0.00423884391784668, 0.00457000732421875, 0.004261016845703125, 0.004240512847900391, 0.004282236099243164, 0.0043141841888427734, 0.00420832633972168, 0.004263162612915039, 0.0042269229888916016, 0.0042459964752197266, 0.00423884391784668, 0.004194974899291992, 0.0042836666107177734, 0.004244089126586914, 0.004217624664306641, 0.004247903823852539, 0.004224300384521484, 0.004202365875244141, 0.0042192935943603516, 0.004293203353881836, 0.004183053970336914, 0.0042378902435302734, 0.004191398620605469, 0.004280805587768555, 0.004214286804199219, 0.00427699089050293, 0.004273414611816406, 0.004237651824951172, 0.004217863082885742, 0.004244804382324219, 0.004236936569213867, 0.004257917404174805, 0.004250288009643555, 0.004215240478515625, 0.004264116287231445, 0.004201173782348633, 0.004196643829345703, 0.0043370723724365234, 0.0042421817779541016, 0.00420832633972168, 0.0042171478271484375, 0.004241943359375, 0.004239797592163086, 0.004265308380126953, 0.004180908203125, 0.0042536258697509766, 0.0042572021484375, 0.004232168197631836, 0.004235982894897461, 0.004230976104736328, 0.0042569637298583984, 0.004232168197631836, 0.004385471343994141, 0.004203081130981445, 0.004225492477416992, 0.004189968109130859, 0.004266977310180664, 0.004236459732055664, 0.004194974899291992, 0.0042645931243896484, 0.0042383670806884766, 0.004227399826049805, 0.0042684078216552734, 0.004244804382324219, 0.004198789596557617, 0.004251003265380859, 0.0042285919189453125, 0.004259824752807617, 0.004229307174682617, 0.004183530807495117, 0.004228353500366211, 0.004307985305786133, 0.004279613494873047, 0.004250764846801758, 0.004227161407470703, 0.0042307376861572266, 0.004255533218383789, 0.004209280014038086, 0.004279613494873047, 0.004246950149536133, 0.0042612552642822266, 0.004268646240234375, 0.004263401031494141, 0.004210472106933594, 0.004238605499267578, 0.004777193069458008, 0.004229068756103516, 0.0042417049407958984, 0.004271507263183594, 0.00457453727722168, 0.004248619079589844, 0.004256486892700195, 0.004236459732055664, 0.004233360290527344, 0.004231452941894531, 0.004254579544067383, 0.00423741340637207, 0.004201412200927734, 0.004225969314575195, 0.004389524459838867, 0.00419306755065918, 0.004253864288330078, 0.00422978401184082, 0.004258155822753906, 0.0042459964752197266, 0.004178285598754883, 0.0042438507080078125, 0.004238128662109375, 0.0042378902435302734, 0.004240274429321289, 0.004231452941894531, 0.004208803176879883, 0.004225730895996094, 0.004233837127685547, 0.004233121871948242, 0.004231691360473633, 0.0042073726654052734, 0.0042877197265625, 0.004296064376831055, 0.004273653030395508, 0.00423121452331543, 0.004268169403076172, 0.004228353500366211, 0.004241466522216797, 0.00418400764465332, 0.004317283630371094, 0.0042552947998046875, 0.004233360290527344, 0.004239559173583984, 0.004235982894897461, 0.004568576812744141, 0.004342794418334961, 0.004395008087158203, 0.004231452941894531, 0.00423741340637207, 0.0042667388916015625, 0.004244089126586914, 0.004242658615112305, 0.004210948944091797, 0.0042514801025390625, 0.0042417049407958984, 0.004244089126586914, 0.004289388656616211, 0.004260540008544922, 0.00424504280090332, 0.004233360290527344, 0.004387617111206055, 0.0042400360107421875, 0.00423121452331543, 0.004253387451171875, 0.00422978401184082, 0.004240989685058594, 0.0041828155517578125, 0.00426483154296875, 0.004241228103637695, 0.00424504280090332, 0.004251003265380859, 0.0042722225189208984, 0.004198312759399414, 0.004281520843505859, 0.0043065547943115234, 0.004227638244628906, 0.004237651824951172, 0.004189729690551758, 0.004283428192138672, 0.004247188568115234, 0.004288196563720703, 0.0042417049407958984, 0.004237651824951172, 0.0042188167572021484, 0.004254817962646484, 0.004267692565917969, 0.004231691360473633, 0.004227876663208008, 0.004236459732055664, 0.00428009033203125, 0.004278898239135742, 0.0042247772216796875, 0.004232883453369141, 0.004298686981201172, 0.0042057037353515625, 0.004261493682861328, 0.004263877868652344, 0.004228115081787109, 0.004233360290527344, 0.004201173782348633, 0.0042951107025146484, 0.004263401031494141, 0.004239082336425781, 0.004258871078491211, 0.004239320755004883, 0.00423741340637207, 0.004264116287231445, 0.004373073577880859, 0.004219532012939453, 0.004249095916748047, 0.004219532012939453, 0.004314422607421875, 0.004267692565917969, 0.004212141036987305, 0.0043087005615234375, 0.00426936149597168, 0.0042591094970703125, 0.004238605499267578, 0.00424957275390625, 0.0042269229888916016, 0.004254341125488281, 0.0043222904205322266, 0.004219770431518555, 0.004250049591064453, 0.0042133331298828125, 0.004315614700317383, 0.004350185394287109, 0.004288911819458008, 0.004278659820556641, 0.004238605499267578, 0.004260063171386719, 0.0042531490325927734, 0.004239320755004883, 0.004198312759399414, 0.0042324066162109375, 0.004245758056640625, 0.004440784454345703, 0.004270315170288086, 0.0042078495025634766, 0.0042836666107177734, 0.004284381866455078, 0.004226207733154297, 0.004250526428222656, 0.0042667388916015625, 0.004252433776855469, 0.004267454147338867, 0.0043070316314697266, 0.004229307174682617, 0.0042345523834228516, 0.004324197769165039, 0.004400014877319336, 0.004261016845703125, 0.004221439361572266, 0.004298210144042969, 0.00439000129699707, 0.0042324066162109375, 0.0042536258697509766, 0.004253864288330078, 0.004246950149536133, 0.004244804382324219, 0.004220008850097656, 0.0042877197265625, 0.004257917404174805, 0.0042493343353271484, 0.004274129867553711, 0.0042722225189208984, 0.004232645034790039, 0.0042629241943359375, 0.00429081916809082, 0.004199981689453125, 0.004266500473022461, 0.004281044006347656, 0.004239797592163086, 0.0042514801025390625, 0.004306793212890625, 0.004274606704711914, 0.004233360290527344, 0.004252910614013672, 0.004255056381225586, 0.004275798797607422, 0.004233121871948242, 0.004267215728759766, 0.0042917728424072266, 0.0042269229888916016, 0.004242897033691406, 0.004221677780151367, 0.004315614700317383, 0.004251003265380859, 0.004212141036987305, 0.004263162612915039, 0.0042307376861572266, 0.00427699089050293, 0.004261016845703125, 0.0042836666107177734, 0.004202604293823242, 0.004256486892700195, 0.0042765140533447266, 0.004273891448974609, 0.0042285919189453125, 0.004214048385620117, 0.0043108463287353516, 0.004390239715576172, 0.004221916198730469, 0.0042514801025390625, 0.0042726993560791016, 0.0042765140533447266, 0.004263401031494141, 0.004271745681762695, 0.004287242889404297, 0.004266500473022461, 0.0043125152587890625, 0.004320621490478516, 0.004277229309082031, 0.004500865936279297, 0.004354238510131836, 0.004302501678466797, 0.004256010055541992, 0.0042879581451416016, 0.004309892654418945, 0.004293918609619141, 0.0043125152587890625, 0.00441288948059082, 0.004271984100341797, 0.004303455352783203, 0.004367351531982422, 0.004293680191040039, 0.004297733306884766, 0.004267692565917969, 0.004349231719970703, 0.004307746887207031, 0.004260063171386719, 0.00429844856262207, 0.0042781829833984375, 0.004344463348388672, 0.004282951354980469, 0.004312992095947266, 0.004268646240234375, 0.00431370735168457, 0.00435638427734375, 0.0042514801025390625, 0.004296541213989258, 0.004253864288330078, 0.004343748092651367, 0.004315614700317383, 0.004258632659912109, 0.004378795623779297, 0.004317045211791992, 0.004328250885009766, 0.004465579986572266, 0.004317045211791992, 0.00427556037902832, 0.004303932189941406, 0.00437164306640625, 0.004265546798706055, 0.0043146610260009766, 0.004332542419433594, 0.004369974136352539, 0.0042989253997802734, 0.004298686981201172, 0.004269599914550781, 0.004320621490478516, 0.004312276840209961, 0.0043337345123291016, 0.004287004470825195, 0.004266500473022461, 0.004359006881713867, 0.004286050796508789, 0.004355430603027344, 0.004325151443481445, 0.004308938980102539, 0.00431513786315918, 0.004330873489379883, 0.004319906234741211, 0.004325151443481445, 0.004299640655517578, 0.004366874694824219, 0.004280805587768555, 0.004288673400878906, 0.004294633865356445, 0.004344940185546875, 0.004330873489379883, 0.004266977310180664, 0.004305839538574219, 0.004300832748413086, 0.004340648651123047, 0.0043103694915771484, 0.0043146610260009766, 0.004252195358276367, 0.004328012466430664, 0.004344463348388672, 0.0043566226959228516, 0.004337310791015625, 0.004317283630371094, 0.004301309585571289, 0.004450082778930664, 0.0043299198150634766, 0.004342555999755859, 0.004309415817260742, 0.004324197769165039, 0.004351615905761719, 0.0043201446533203125, 0.004288434982299805, 0.00433659553527832, 0.004301309585571289, 0.0042724609375, 0.004318952560424805, 0.004332542419433594, 0.00432896614074707, 0.004291534423828125, 0.0043392181396484375, 0.004269123077392578, 0.004314899444580078, 0.004312992095947266, 0.004415750503540039, 0.004306793212890625, 0.0042803287506103516, 0.00438380241394043, 0.00430750846862793, 0.004271984100341797, 0.0043566226959228516, 0.004323005676269531, 0.004328250885009766, 0.004326820373535156, 0.0043179988861083984, 0.004267454147338867, 0.004298686981201172, 0.0043680667877197266, 0.004259824752807617, 0.004305839538574219, 0.004309177398681641, 0.0043218135833740234, 0.0043277740478515625, 0.004280567169189453, 0.00431513786315918, 0.004300594329833984, 0.004357337951660156, 0.004329681396484375, 0.004415035247802734, 0.004268646240234375, 0.004289388656616211, 0.004446744918823242, 0.0042912960052490234, 0.0043201446533203125, 0.004343509674072266, 0.004311561584472656, 0.004332542419433594, 0.004340648651123047, 0.0042705535888671875, 0.004300355911254883, 0.00439763069152832, 0.00439906120300293, 0.004328012466430664, 0.004277706146240234, 0.004315614700317383, 0.004323244094848633, 0.0043544769287109375, 0.0043218135833740234, 0.004313945770263672, 0.004263162612915039, 0.004443645477294922, 0.004381895065307617, 0.004304409027099609, 0.004300355911254883, 0.004326820373535156, 0.004343509674072266, 0.004331111907958984, 0.00434112548828125, 0.004315614700317383, 0.004506826400756836, 0.004392385482788086, 0.0043125152587890625, 0.004366159439086914, 0.0042819976806640625, 0.00440216064453125, 0.004312276840209961, 0.004285573959350586, 0.004370689392089844, 0.004326820373535156, 0.004319190979003906, 0.00435948371887207, 0.00433659553527832, 0.004301786422729492, 0.004418373107910156, 0.0043811798095703125, 0.004282951354980469, 0.004325151443481445, 0.0043447017669677734, 0.004461765289306641, 0.004320383071899414, 0.004334926605224609, 0.004285097122192383, 0.004323720932006836, 0.0043184757232666016, 0.004332780838012695, 0.004299640655517578, 0.004282474517822266, 0.004349470138549805, 0.004338502883911133, 0.004270076751708984, 0.004333019256591797, 0.004317045211791992, 0.00433659553527832, 0.004317045211791992, 0.004332542419433594, 0.004289388656616211, 0.004342555999755859, 0.0044841766357421875, 0.0042724609375, 0.004315614700317383, 0.004328727722167969, 0.004324197769165039, 0.004332304000854492, 0.004296064376831055, 0.00432133674621582, 0.004284858703613281, 0.004344463348388672, 0.004351139068603516, 0.004290580749511719, 0.004278898239135742, 0.0043070316314697266, 0.00438237190246582, 0.004293918609619141, 0.004305124282836914, 0.004312276840209961, 0.004336118698120117, 0.0043218135833740234, 0.004312038421630859, 0.004282712936401367, 0.004298686981201172, 0.004334926605224609, 0.004418849945068359, 0.004337310791015625, 0.004264354705810547, 0.004464626312255859, 0.004330873489379883, 0.004283428192138672, 0.004319667816162109, 0.0043027400970458984, 0.004316806793212891, 0.004349470138549805, 0.0043375492095947266, 0.0042879581451416016, 0.0042972564697265625, 0.004367351531982422, 0.004288434982299805, 0.004329681396484375, 0.0043239593505859375, 0.004321098327636719, 0.004317760467529297, 0.004283905029296875, 0.004342317581176758, 0.004300355911254883, 0.004453182220458984, 0.004363536834716797, 0.004316091537475586, 0.004284381866455078, 0.004385948181152344, 0.004314899444580078, 0.00427556037902832, 0.004344940185546875, 0.004349231719970703, 0.004311800003051758, 0.00432586669921875, 0.004339694976806641, 0.004323482513427734, 0.0043337345123291016, 0.00439143180847168, 0.004294157028198242, 0.004317522048950195, 0.0042803287506103516, 0.004393577575683594, 0.0043642520904541016, 0.004312038421630859, 0.004341840744018555, 0.004321575164794922, 0.004354000091552734, 0.00436091423034668, 0.004324913024902344, 0.004267692565917969, 0.004406452178955078, 0.0043888092041015625, 0.004307746887207031, 0.004357576370239258, 0.004328250885009766, 0.004314422607421875, 0.004379987716674805, 0.004336118698120117, 0.004280567169189453, 0.004327058792114258, 0.004329681396484375, 0.0043451786041259766, 0.004367828369140625, 0.004297971725463867, 0.004358053207397461, 0.00433802604675293, 0.0043184757232666016, 0.004360675811767578, 0.004316568374633789, 0.004411220550537109, 0.004370450973510742, 0.004349946975708008, 0.004292011260986328, 0.004318714141845703, 0.0043828487396240234, 0.0043582916259765625, 0.004356861114501953, 0.004339456558227539, 0.004329681396484375, 0.004348278045654297, 0.00430607795715332, 0.0043506622314453125, 0.004359245300292969, 0.00432896614074707, 0.004335641860961914, 0.0043468475341796875, 0.004311084747314453, 0.004395484924316406, 0.004335641860961914, 0.004302024841308594, 0.004342079162597656, 0.00436854362487793, 0.004390716552734375, 0.004319190979003906, 0.004336357116699219, 0.004303932189941406, 0.004422903060913086, 0.004405975341796875, 0.004299163818359375, 0.004328012466430664, 0.0043447017669677734, 0.004346370697021484, 0.004369020462036133, 0.004299640655517578, 0.004347324371337891, 0.004348278045654297, 0.004327535629272461, 0.004383087158203125, 0.004343986511230469, 0.0043561458587646484, 0.004392385482788086, 0.004335880279541016, 0.0042934417724609375, 0.0043485164642333984, 0.004339694976806641, 0.0044803619384765625, 0.004395246505737305, 0.0043485164642333984, 0.004335165023803711, 0.004334211349487305, 0.004379987716674805, 0.00429534912109375, 0.004364967346191406, 0.004340171813964844, 0.004317522048950195, 0.004351139068603516, 0.004513263702392578, 0.00429987907409668, 0.004349231719970703, 0.004324197769165039, 0.004361391067504883, 0.004351615905761719, 0.0042912960052490234, 0.004399538040161133, 0.004335880279541016, 0.004312038421630859, 0.00434112548828125, 0.004354000091552734, 0.00433659553527832, 0.004329204559326172, 0.0043523311614990234, 0.004307985305786133, 0.004410982131958008, 0.004439353942871094, 0.00429081916809082, 0.004342555999755859, 0.004394054412841797, 0.004348278045654297, 0.004349470138549805, 0.004372835159301758, 0.004312992095947266, 0.004370212554931641, 0.0043294429779052734, 0.004349946975708008, 0.004328489303588867, 0.004318714141845703, 0.004378080368041992, 0.004346132278442383, 0.004300355911254883, 0.0043566226959228516, 0.004376649856567383, 0.004473209381103516, 0.0043642520904541016, 0.004340648651123047, 0.004349946975708008, 0.0043582916259765625, 0.004357814788818359, 0.004313230514526367, 0.004347085952758789, 0.00440526008605957, 0.004465341567993164, 0.004354953765869141, 0.004366636276245117, 0.004340410232543945, 0.004337787628173828, 0.004362821578979492, 0.004314422607421875, 0.004354238510131836, 0.0043447017669677734, 0.0043795108795166016, 0.0043430328369140625, 0.004305362701416016, 0.004401445388793945, 0.004754543304443359, 0.004411220550537109, 0.004411935806274414, 0.004379987716674805, 0.004362583160400391, 0.004512786865234375, 0.004353523254394531, 0.0043599605560302734, 0.004589557647705078, 0.004474163055419922, 0.004407405853271484, 0.004474163055419922, 0.004385471343994141, 0.0043697357177734375, 0.004427909851074219, 0.0043926239013671875, 0.004381418228149414, 0.004400491714477539, 0.004392862319946289, 0.004389047622680664, 0.004465579986572266, 0.004389286041259766, 0.004355907440185547, 0.004502296447753906, 0.004402637481689453, 0.004399776458740234, 0.004438877105712891, 0.004401445388793945, 0.0043621063232421875, 0.004448890686035156, 0.0043811798095703125, 0.004347085952758789, 0.004403114318847656, 0.004446744918823242, 0.00438380241394043, 0.004391193389892578, 0.0043792724609375, 0.0043871402740478516, 0.0044214725494384766, 0.0044476985931396484, 0.004363059997558594, 0.004372835159301758, 0.004425764083862305, 0.00439143180847168, 0.00440669059753418, 0.004389762878417969, 0.0043485164642333984, 0.004385948181152344, 0.004460811614990234, 0.004361152648925781, 0.004382610321044922, 0.004586219787597656, 0.004412651062011719, 0.004392862319946289, 0.004410743713378906, 0.004360675811767578, 0.0044002532958984375, 0.0043947696685791016, 0.0043888092041015625, 0.004418611526489258, 0.004359722137451172, 0.004458904266357422, 0.004393100738525391, 0.004369258880615234, 0.004424333572387695, 0.0044403076171875, 0.004431962966918945, 0.00444340705871582, 0.004416227340698242, 0.004482746124267578, 0.0044324398040771484, 0.004438161849975586, 0.004367828369140625, 0.004426002502441406, 0.0043756961822509766, 0.004407167434692383, 0.004425048828125, 0.004422664642333984, 0.004369020462036133, 0.004450321197509766, 0.004399776458740234, 0.004421710968017578, 0.004431247711181641, 0.0044400691986083984, 0.004398345947265625, 0.0044100284576416016, 0.004383563995361328, 0.004395484924316406, 0.004484891891479492, 0.0044403076171875, 0.004373311996459961, 0.004401683807373047, 0.004425048828125, 0.004412651062011719, 0.004403591156005859, 0.004408359527587891, 0.004480123519897461, 0.004405498504638672, 0.004503965377807617, 0.0043604373931884766, 0.004408121109008789, 0.004411220550537109, 0.004411458969116211, 0.0044155120849609375, 0.004492044448852539, 0.004358768463134766, 0.004393339157104492, 0.0044558048248291016, 0.004379987716674805, 0.004426717758178711, 0.00442194938659668, 0.0044002532958984375, 0.004397153854370117, 0.004498958587646484, 0.0045013427734375, 0.004431009292602539, 0.0044708251953125, 0.004401683807373047, 0.004400491714477539, 0.004427433013916016, 0.0043942928314208984, 0.004424333572387695, 0.004443645477294922, 0.004445552825927734, 0.004428386688232422, 0.004530668258666992, 0.004384040832519531, 0.004416942596435547, 0.00443720817565918, 0.004411458969116211, 0.004388093948364258, 0.004403829574584961, 0.004385232925415039, 0.0044384002685546875, 0.004462242126464844, 0.0043833255767822266, 0.004414796829223633, 0.004441261291503906, 0.00440526008605957, 0.00441431999206543, 0.004463911056518555, 0.004472017288208008, 0.00442194938659668, 0.004456520080566406, 0.004392385482788086, 0.004462242126464844, 0.004414796829223633, 0.0044252872467041016, 0.004416227340698242, 0.0044269561767578125, 0.0043487548828125, 0.004426717758178711, 0.004420042037963867, 0.004469633102416992, 0.004478931427001953, 0.004359722137451172, 0.0045413970947265625, 0.004422426223754883, 0.004416465759277344, 0.004513740539550781, 0.004419088363647461, 0.004467487335205078, 0.0043871402740478516, 0.004420757293701172, 0.004387855529785156, 0.004467964172363281, 0.004423618316650391, 0.004378080368041992, 0.004425764083862305, 0.004420280456542969, 0.00441741943359375, 0.0045201778411865234, 0.004429817199707031, 0.004412412643432617, 0.004479408264160156, 0.004446268081665039, 0.0044176578521728516, 0.00444483757019043, 0.004415988922119141, 0.004424095153808594, 0.004449605941772461, 0.004408836364746094, 0.004388093948364258, 0.0044672489166259766, 0.0044214725494384766, 0.004560947418212891, 0.004472017288208008, 0.004558563232421875, 0.004431009292602539, 0.004458427429199219, 0.004404306411743164, 0.004374504089355469, 0.004454135894775391, 0.004465341567993164, 0.004380941390991211, 0.0044291019439697266, 0.004458189010620117, 0.004440784454345703, 0.004448413848876953, 0.004471778869628906, 0.004370927810668945, 0.00445246696472168, 0.004451751708984375, 0.004393577575683594, 0.004609107971191406, 0.004414796829223633, 0.004427194595336914, 0.0044367313385009766, 0.0044405460357666016, 0.004405975341796875, 0.00448918342590332, 0.0044307708740234375, 0.00436091423034668, 0.004613161087036133, 0.004416704177856445, 0.004421710968017578, 0.004456996917724609, 0.0044214725494384766, 0.004412174224853516, 0.004511117935180664, 0.004427909851074219, 0.0044095516204833984, 0.004460334777832031, 0.004422903060913086, 0.004479169845581055, 0.004442930221557617, 0.0044291019439697266, 0.004478931427001953, 0.004477262496948242, 0.00442051887512207, 0.004380226135253906, 0.004456758499145508, 0.004530668258666992, 0.004418373107910156, 0.004427194595336914, 0.004438161849975586, 0.004409313201904297, 0.0045013427734375, 0.004431009292602539, 0.00440216064453125, 0.004452228546142578, 0.004434108734130859, 0.004424333572387695, 0.004488229751586914, 0.004439592361450195, 0.004411935806274414, 0.00449061393737793, 0.004423618316650391, 0.004412174224853516, 0.0045680999755859375, 0.00442194938659668, 0.004445075988769531, 0.0044405460357666016, 0.0044324398040771484, 0.004400730133056641, 0.004482746124267578, 0.004413127899169922, 0.00439143180847168, 0.004464626312255859, 0.004461050033569336, 0.004454612731933594, 0.0044651031494140625, 0.0044329166412353516, 0.004411220550537109, 0.004508256912231445, 0.004444122314453125, 0.004406452178955078, 0.0044307708740234375, 0.004450321197509766, 0.004467487335205078, 0.004448413848876953, 0.0044269561767578125, 0.004403114318847656, 0.004525661468505859, 0.004449605941772461, 0.004376888275146484, 0.0044918060302734375, 0.004556894302368164, 0.00445556640625, 0.004445552825927734, 0.00445103645324707, 0.00450587272644043, 0.0044651031494140625, 0.004429817199707031, 0.004464864730834961, 0.004498004913330078, 0.004511117935180664, 0.004464387893676758, 0.004427194595336914, 0.0044329166412353516, 0.004476308822631836, 0.0044329166412353516, 0.004431486129760742, 0.0044708251953125, 0.004530906677246094, 0.0044209957122802734, 0.004430055618286133, 0.004397153854370117, 0.0044536590576171875, 0.0044825077056884766, 0.00439906120300293, 0.004434347152709961, 0.004460334777832031, 0.004463911056518555, 0.004449367523193359, 0.004456281661987305, 0.0044133663177490234, 0.004436969757080078, 0.004492282867431641, 0.00441431999206543, 0.004418373107910156, 0.004483938217163086, 0.0044443607330322266, 0.004433870315551758, 0.004463672637939453, 0.004389762878417969, 0.0044193267822265625, 0.004493236541748047, 0.0044252872467041016, 0.00443720817565918, 0.004467010498046875, 0.004441499710083008, 0.00455164909362793, 0.004461526870727539, 0.0044252872467041016, 0.004437446594238281, 0.004490852355957031, 0.00442194938659668, 0.004434823989868164, 0.004437446594238281, 0.004487514495849609, 0.004431486129760742, 0.004441738128662109, 0.004396915435791016, 0.004446268081665039, 0.004494190216064453, 0.004396200180053711, 0.00443720817565918, 0.004430294036865234, 0.004533290863037109, 0.004472017288208008, 0.004491567611694336, 0.004396200180053711, 0.004466056823730469, 0.0044956207275390625, 0.004883289337158203, 0.004464626312255859, 0.004489421844482422, 0.004698991775512695, 0.0047605037689208984, 0.004477739334106445, 0.004431247711181641, 0.00448918342590332, 0.004448890686035156, 0.004424333572387695, 0.004515171051025391, 0.004461526870727539, 0.004453182220458984, 0.004477739334106445, 0.0044558048248291016, 0.004399776458740234, 0.0049517154693603516, 0.004555940628051758, 0.004883527755737305, 0.004440784454345703, 0.004445552825927734, 0.004586219787597656, 0.004422426223754883, 0.004486083984375, 0.004466533660888672, 0.0044689178466796875, 0.004442930221557617, 0.004483938217163086, 0.004408359527587891, 0.004475593566894531, 0.00450444221496582, 0.0044097900390625, 0.004435539245605469, 0.004519462585449219, 0.004474639892578125, 0.00447535514831543, 0.004484891891479492, 0.00442051887512207, 0.004531383514404297, 0.004524707794189453, 0.004441499710083008, 0.004538297653198242, 0.0044498443603515625, 0.004456281661987305, 0.0044994354248046875, 0.004480600357055664, 0.004410266876220703, 0.0045053958892822266, 0.004462003707885742, 0.004407167434692383, 0.004493236541748047, 0.004464626312255859, 0.004462480545043945, 0.004460811614990234, 0.004461050033569336, 0.004411935806274414, 0.0044994354248046875, 0.004470348358154297, 0.0044231414794921875, 0.0044748783111572266, 0.004446506500244141, 0.004446506500244141, 0.004516124725341797, 0.004461526870727539, 0.004414558410644531, 0.004525423049926758, 0.004554033279418945, 0.004415988922119141, 0.004471778869628906, 0.004454135894775391, 0.0044574737548828125, 0.004445552825927734, 0.0044476985931396484, 0.00443577766418457, 0.004578590393066406, 0.00447535514831543, 0.004445075988769531, 0.004494428634643555, 0.004511594772338867, 0.004479408264160156, 0.004489421844482422, 0.0044918060302734375, 0.004508495330810547, 0.00470423698425293, 0.004510164260864258, 0.004487514495849609, 0.004456996917724609, 0.004496574401855469, 0.004546642303466797, 0.0044934749603271484, 0.004488229751586914, 0.004502534866333008, 0.0046291351318359375, 0.004530429840087891, 0.00449061393737793, 0.0044422149658203125, 0.0045397281646728516, 0.0045125484466552734, 0.004468441009521484, 0.0045146942138671875, 0.004521608352661133, 0.004484415054321289, 0.004523515701293945, 0.004509449005126953, 0.00458216667175293, 0.0046727657318115234, 0.004487276077270508, 0.0044558048248291016, 0.0045278072357177734, 0.0045053958892822266, 0.004611015319824219, 0.0045375823974609375, 0.004500865936279297, 0.0045773983001708984, 0.004513263702392578, 0.0044629573822021484, 0.004548072814941406, 0.004509449005126953, 0.004475116729736328, 0.0045130252838134766, 0.004490375518798828, 0.004513978958129883, 0.00454258918762207, 0.004503488540649414, 0.004483699798583984, 0.004551410675048828, 0.00449371337890625, 0.004598855972290039, 0.004515171051025391, 0.004477262496948242, 0.0045664310455322266, 0.004445075988769531, 0.0044858455657958984, 0.0045130252838134766, 0.004548788070678711, 0.004473686218261719, 0.00449681282043457, 0.0044536590576171875, 0.004490852355957031, 0.004563331604003906, 0.00446629524230957, 0.004494667053222656, 0.0044977664947509766, 0.0044710636138916016, 0.004492282867431641, 0.0045053958892822266, 0.00448298454284668, 0.004529714584350586, 0.0044901371002197266, 0.004481077194213867, 0.004521369934082031, 0.004496097564697266, 0.004563808441162109, 0.004560947418212891, 0.004510164260864258, 0.004587888717651367, 0.004557132720947266, 0.0045261383056640625, 0.0045092105865478516, 0.004495859146118164, 0.004519462585449219, 0.0045680999755859375, 0.004491567611694336, 0.004531145095825195, 0.004533290863037109, 0.004520416259765625, 0.004501819610595703, 0.00452423095703125, 0.004496574401855469, 0.004523754119873047, 0.00457453727722168, 0.004503011703491211, 0.004615306854248047, 0.004534721374511719, 0.004522562026977539, 0.004520893096923828, 0.004510641098022461, 0.004449367523193359, 0.004590034484863281, 0.004504203796386719, 0.004472255706787109, 0.004519939422607422, 0.004538536071777344, 0.004523515701293945, 0.0051233768463134766, 0.004532575607299805, 0.004599571228027344, 0.004767179489135742, 0.004606008529663086, 0.00455021858215332, 0.004479885101318359, 0.00458526611328125, 0.00451207160949707, 0.004468441009521484, 0.004541635513305664, 0.0044858455657958984, 0.004537820816040039, 0.004510402679443359, 0.0045163631439208984, 0.00464177131652832, 0.004643678665161133, 0.004523277282714844, 0.004515171051025391, 0.004503488540649414, 0.004525184631347656, 0.0045623779296875, 0.0045316219329833984, 0.004519224166870117, 0.004525184631347656, 0.0045146942138671875, 0.004504203796386719, 0.004562854766845703, 0.0044651031494140625, 0.004501819610595703, 0.00456547737121582, 0.004476785659790039, 0.004614114761352539, 0.004533290863037109, 0.004528522491455078, 0.00455021858215332, 0.004517078399658203, 0.00452733039855957, 0.004550457000732422, 0.00453495979309082, 0.00448918342590332, 0.0045816898345947266, 0.004858493804931641, 0.004629850387573242, 0.004569530487060547, 0.004468441009521484, 0.0045318603515625, 0.0045282840728759766, 0.004503488540649414, 0.004550933837890625, 0.004519224166870117, 0.004514455795288086, 0.0045299530029296875, 0.004546403884887695, 0.004589557647705078, 0.0044596195220947266, 0.004525423049926758, 0.004564762115478516, 0.004503011703491211, 0.0046384334564208984, 0.004553318023681641, 0.004517793655395508, 0.004503011703491211, 0.004528999328613281, 0.004485130310058594, 0.004570484161376953, 0.0045354366302490234, 0.0044612884521484375, 0.0045397281646728516, 0.0045473575592041016, 0.004551887512207031, 0.004624128341674805, 0.004498720169067383, 0.004469156265258789, 0.004594564437866211, 0.004535675048828125, 0.004621028900146484, 0.00449681282043457, 0.0044956207275390625, 0.004568815231323242, 0.004487514495849609, 0.004516124725341797, 0.004583597183227539, 0.004506111145019531, 0.004540681838989258, 0.00453639030456543, 0.004555463790893555, 0.0045816898345947266, 0.004532814025878906, 0.004460811614990234, 0.004552602767944336, 0.00451970100402832, 0.004495382308959961, 0.0045244693756103516, 0.0045337677001953125, 0.004489421844482422, 0.0045549869537353516, 0.004525899887084961, 0.004474163055419922, 0.00452876091003418, 0.004521846771240234, 0.004581451416015625, 0.004470348358154297, 0.004578828811645508, 0.0045626163482666016, 0.0045223236083984375, 0.004536151885986328, 0.004541158676147461, 0.004520893096923828, 0.004561662673950195, 0.0046656131744384766, 0.004515647888183594, 0.004571199417114258, 0.004548072814941406, 0.004491329193115234, 0.00458979606628418, 0.0045452117919921875, 0.004477739334106445, 0.0045354366302490234, 0.004533052444458008, 0.004644632339477539, 0.0045468807220458984, 0.004507303237915039, 0.004523754119873047, 0.00462031364440918, 0.00455474853515625, 0.0045435428619384766, 0.004483222961425781, 0.0045702457427978516, 0.004750728607177734, 0.004498958587646484, 0.004553556442260742, 0.00452876091003418, 0.0045318603515625, 0.004518032073974609, 0.0045223236083984375, 0.004477739334106445, 0.004563808441162109, 0.004530668258666992, 0.0044708251953125, 0.004587411880493164, 0.004514932632446289, 0.005033969879150391, 0.004496335983276367, 0.0045206546783447266, 0.004519462585449219, 0.004537105560302734, 0.004595756530761719, 0.004571437835693359, 0.0044858455657958984, 0.004554033279418945, 0.004591941833496094, 0.004485607147216797, 0.004559040069580078, 0.004517793655395508, 0.004536867141723633, 0.004511117935180664, 0.004525661468505859, 0.0044841766357421875, 0.0045740604400634766, 0.004519939422607422, 0.0045015811920166016, 0.004548549652099609, 0.00452423095703125, 0.0046999454498291016, 0.004506111145019531, 0.004511833190917969, 0.004561662673950195, 0.004520893096923828, 0.00452876091003418, 0.004539012908935547, 0.004481077194213867, 0.004525184631347656, 0.004587411880493164, 0.004505634307861328, 0.004517555236816406, 0.004578590393066406, 0.0045506954193115234, 0.004534721374511719, 0.004544258117675781, 0.004511594772338867, 0.004585981369018555, 0.004544734954833984, 0.0045244693756103516, 0.00454401969909668, 0.004553794860839844, 0.004522085189819336, 0.00455164909362793, 0.004575967788696289, 0.0045588016510009766, 0.004540205001831055, 0.004651784896850586, 0.004578590393066406, 0.004508256912231445, 0.0045430660247802734, 0.0045931339263916016, 0.004511356353759766, 0.004535675048828125, 0.004529476165771484, 0.004547595977783203, 0.00455927848815918, 0.0045397281646728516, 0.004525661468505859, 0.004567384719848633, 0.004546165466308594, 0.0045130252838134766, 0.0045623779296875, 0.004573822021484375, 0.00469970703125, 0.0045053958892822266, 0.004546642303466797, 0.004535198211669922, 0.004546642303466797, 0.004589080810546875, 0.004545927047729492, 0.004525423049926758, 0.004549503326416016, 0.0045931339263916016, 0.004520416259765625, 0.004561662673950195, 0.004588127136230469, 0.004548788070678711, 0.004555940628051758, 0.004517555236816406, 0.004497051239013672, 0.004611968994140625, 0.0045261383056640625, 0.004492998123168945, 0.004537343978881836, 0.0045757293701171875, 0.004603385925292969, 0.00452113151550293, 0.004546165466308594, 0.0045623779296875, 0.0045528411865234375, 0.004609823226928711, 0.004566669464111328, 0.00448918342590332, 0.0046117305755615234, 0.00457763671875, 0.004513978958129883, 0.0045795440673828125, 0.004578828811645508, 0.004562854766845703, 0.004557132720947266, 0.004570484161376953, 0.004537343978881836, 0.0046041011810302734, 0.004578828811645508, 0.004580974578857422, 0.004522085189819336, 0.004606008529663086, 0.004714012145996094, 0.00450444221496582, 0.004556417465209961, 0.004575490951538086, 0.004557132720947266, 0.004781246185302734, 0.004613161087036133, 0.004527091979980469, 0.0046253204345703125, 0.004534244537353516, 0.0045928955078125, 0.004587888717651367, 0.004534244537353516, 0.004607200622558594, 0.004506349563598633, 0.0045778751373291016, 0.004528045654296875, 0.0045413970947265625, 0.004575490951538086, 0.004546642303466797, 0.004509687423706055, 0.00456547737121582, 0.004617929458618164, 0.004754543304443359, 0.00460052490234375, 0.0045511722564697266, 0.004537343978881836, 0.004643917083740234, 0.004572868347167969, 0.004541873931884766, 0.004544734954833984, 0.004534006118774414, 0.004585742950439453, 0.004604339599609375, 0.004536628723144531, 0.0045833587646484375, 0.004494905471801758, 0.004576444625854492, 0.004615306854248047, 0.004570722579956055, 0.004556417465209961, 0.0045528411865234375, 0.004521369934082031, 0.0047054290771484375, 0.004546642303466797, 0.004617214202880859, 0.0046122074127197266, 0.0046117305755615234, 0.00465846061706543, 0.004569053649902344, 0.004601478576660156, 0.004582881927490234, 0.004624605178833008, 0.004593372344970703, 0.004591941833496094, 0.004559040069580078, 0.0046994686126708984, 0.004593372344970703, 0.004547834396362305, 0.004622936248779297, 0.004595756530761719, 0.00463414192199707, 0.004574775695800781, 0.0046308040618896484, 0.004619121551513672, 0.004603862762451172, 0.004645109176635742, 0.004639148712158203, 0.004580974578857422, 0.004623889923095703, 0.004596710205078125, 0.0046498775482177734, 0.0046503543853759766, 0.004607677459716797, 0.004640102386474609, 0.0045397281646728516, 0.004616498947143555, 0.004628181457519531, 0.004626035690307617, 0.004620790481567383, 0.004586935043334961, 0.0045549869537353516, 0.004664897918701172, 0.004622220993041992, 0.004590511322021484, 0.0046498775482177734, 0.004591464996337891, 0.00480961799621582, 0.004608631134033203, 0.004603385925292969, 0.0046541690826416016, 0.004575967788696289, 0.004642963409423828, 0.004651546478271484, 0.0045604705810546875, 0.004838705062866211, 0.0046234130859375, 0.00461125373840332, 0.004667520523071289, 0.004615306854248047, 0.004612445831298828, 0.004576206207275391, 0.00467991828918457, 0.004616498947143555, 0.004601001739501953, 0.004625797271728516, 0.0046122074127197266, 0.004641056060791016, 0.00478672981262207, 0.004838228225708008, 0.0047092437744140625, 0.004663705825805664, 0.004690408706665039, 0.00462794303894043, 0.004782915115356445, 0.004687070846557617, 0.004645347595214844, 0.0046961307525634766, 0.004574298858642578, 0.004622459411621094, 0.004663228988647461, 0.0045680999755859375, 0.004622459411621094, 0.004636526107788086, 0.004620790481567383, 0.004641532897949219, 0.0046613216400146484, 0.0045909881591796875, 0.004694938659667969, 0.00460362434387207, 0.00470280647277832, 0.004559993743896484, 0.004592418670654297, 0.004837989807128906, 0.004572391510009766, 0.004637479782104492, 0.0046138763427734375, 0.004648685455322266, 0.004641532897949219, 0.004608869552612305, 0.0046236515045166016, 0.0046122074127197266, 0.004643678665161133, 0.004647016525268555, 0.004574775695800781, 0.004693508148193359, 0.004613399505615234, 0.004580974578857422, 0.00463557243347168, 0.004647493362426758, 0.004651546478271484, 0.004666328430175781, 0.004606723785400391, 0.004647970199584961, 0.004605531692504883, 0.0046236515045166016, 0.004644870758056641, 0.004662752151489258, 0.004691362380981445, 0.004826068878173828, 0.00458526611328125, 0.004736185073852539, 0.00463557243347168, 0.0046939849853515625, 0.004611492156982422, 0.004616737365722656, 0.004629611968994141, 0.0046193599700927734, 0.0046389102935791016, 0.004605293273925781, 0.004563331604003906, 0.0046923160552978516, 0.004610538482666016, 0.004729747772216797, 0.004606723785400391, 0.004608154296875, 0.004663944244384766, 0.0045735836029052734, 0.0046236515045166016, 0.004642486572265625, 0.004641532897949219, 0.004655599594116211, 0.004622697830200195, 0.004603385925292969, 0.004678249359130859, 0.004935741424560547, 0.0046541690826416016, 0.004603147506713867, 0.004637956619262695, 0.004984378814697266, 0.004643917083740234, 0.00464177131652832, 0.004656076431274414, 0.004648447036743164, 0.004634857177734375, 0.004620790481567383, 0.004668712615966797, 0.0045528411865234375, 0.004633188247680664, 0.0046846866607666016, 0.00469970703125, 0.0046539306640625, 0.004666805267333984, 0.004627704620361328, 0.004635810852050781, 0.004631519317626953, 0.004638671875, 0.00468754768371582, 0.004637002944946289, 0.004611968994140625, 0.004580497741699219, 0.004664897918701172, 0.004625082015991211, 0.0045778751373291016, 0.004643678665161133, 0.0046389102935791016, 0.004774570465087891, 0.004618644714355469, 0.0046443939208984375, 0.004698276519775391, 0.004614591598510742, 0.004646778106689453, 0.004621982574462891, 0.004592180252075195, 0.004699230194091797, 0.0046198368072509766, 0.004707813262939453, 0.0046520233154296875, 0.004631757736206055, 0.0046536922454833984, 0.004597902297973633, 0.004643678665161133, 0.00468134880065918, 0.004625797271728516, 0.00464177131652832, 0.004631757736206055, 0.004618167877197266, 0.004775047302246094, 0.004659414291381836, 0.0046482086181640625, 0.004632711410522461, 0.00464177131652832, 0.004666805267333984, 0.004685640335083008, 0.004715442657470703, 0.004649162292480469, 0.004583835601806641, 0.004664421081542969, 0.00463104248046875, 0.0047266483306884766, 0.004613161087036133, 0.0046596527099609375, 0.004643917083740234, 0.004645824432373047, 0.004676342010498047, 0.004634380340576172, 0.004596233367919922, 0.0046710968017578125, 0.004626274108886719, 0.0047757625579833984, 0.0045812129974365234, 0.004633188247680664, 0.004704713821411133, 0.0046122074127197266, 0.004692554473876953, 0.004647493362426758, 0.00468134880065918, 0.004687070846557617, 0.0046329498291015625, 0.0046367645263671875, 0.004639387130737305, 0.004616737365722656, 0.004631996154785156, 0.0045833587646484375, 0.004670619964599609, 0.004694938659667969, 0.00463104248046875, 0.004683971405029297, 0.004674673080444336, 0.00461888313293457, 0.0046579837799072266, 0.004729747772216797, 0.0046885013580322266, 0.00463104248046875, 0.00466609001159668, 0.004638671875, 0.004643917083740234, 0.0047185420989990234, 0.004656553268432617, 0.004593610763549805, 0.004661083221435547, 0.004625797271728516, 0.004711627960205078, 0.004599094390869141, 0.004632472991943359, 0.004643917083740234, 0.004613399505615234, 0.004684925079345703, 0.004670619964599609, 0.004628419876098633, 0.004692792892456055, 0.00461125373840332, 0.0048351287841796875, 0.004627704620361328, 0.0046367645263671875, 0.004676103591918945, 0.004604339599609375, 0.0046939849853515625, 0.004663705825805664, 0.0046541690826416016, 0.004812717437744141, 0.004670619964599609, 0.0046541690826416016, 0.004660367965698242, 0.004667043685913086, 0.004671573638916016, 0.004589557647705078, 0.004736661911010742, 0.0046918392181396484, 0.0046122074127197266, 0.004705905914306641, 0.004639387130737305, 0.004779815673828125, 0.004662513732910156, 0.004925251007080078, 0.004695892333984375, 0.004630327224731445, 0.004709005355834961, 0.004665851593017578, 0.004784345626831055, 0.004652500152587891, 0.004655122756958008, 0.004703521728515625, 0.004621982574462891, 0.004741668701171875, 0.004674434661865234, 0.004612445831298828, 0.004680156707763672, 0.0046405792236328125, 0.004689693450927734, 0.004601478576660156, 0.00463557243347168, 0.004664182662963867, 0.004682779312133789, 0.0047838687896728516, 0.004667758941650391, 0.004616737365722656, 0.0047149658203125, 0.004667520523071289, 0.004671812057495117, 0.004609107971191406, 0.004657268524169922, 0.0047032833099365234, 0.00461578369140625, 0.0046672821044921875, 0.004664421081542969, 0.004633188247680664, 0.004707813262939453, 0.004700899124145508, 0.004699230194091797, 0.004642486572265625, 0.004668712615966797, 0.004687786102294922, 0.004631519317626953, 0.004711627960205078, 0.004705905914306641, 0.0046329498291015625, 0.004737377166748047, 0.004676342010498047, 0.004724740982055664, 0.004627704620361328, 0.00464630126953125, 0.004807949066162109, 0.004651784896850586, 0.00472259521484375, 0.00464320182800293, 0.0046308040618896484, 0.004679679870605469, 0.004647016525268555, 0.004677772521972656, 0.004624605178833008, 0.0046656131744384766, 0.0047359466552734375, 0.004614830017089844, 0.004641056060791016, 0.004649639129638672, 0.0046634674072265625, 0.0047855377197265625, 0.00468897819519043, 0.004702568054199219, 0.0046596527099609375, 0.004636287689208984, 0.004640102386474609, 0.004599571228027344, 0.004754066467285156, 0.00464630126953125, 0.0046062469482421875, 0.004677295684814453, 0.00465846061706543, 0.0047681331634521484, 0.004626274108886719, 0.004637241363525391, 0.004668712615966797, 0.004688739776611328, 0.004675149917602539, 0.004659891128540039, 0.004642009735107422, 0.004710674285888672, 0.004665851593017578, 0.004695415496826172, 0.0046350955963134766, 0.004690885543823242, 0.004663228988647461, 0.004597663879394531, 0.004705190658569336, 0.004785299301147461, 0.004739522933959961, 0.004626274108886719, 0.004668712615966797, 0.0046732425689697266, 0.0046651363372802734, 0.004690647125244141, 0.004670381546020508, 0.004619598388671875, 0.004708290100097656, 0.004682302474975586, 0.004681587219238281, 0.004612445831298828, 0.004680633544921875, 0.004792213439941406, 0.004739999771118164, 0.0046749114990234375, 0.004662752151489258, 0.004675149917602539, 0.004669904708862305, 0.00469207763671875, 0.004694938659667969, 0.004705190658569336, 0.00468134880065918, 0.004668474197387695, 0.004683971405029297, 0.004724025726318359, 0.0046689510345458984, 0.0046596527099609375, 0.00474238395690918, 0.0046885013580322266, 0.004716396331787109, 0.004631519317626953, 0.004728078842163086, 0.004665851593017578, 0.0046367645263671875, 0.004708290100097656, 0.004692554473876953, 0.004743337631225586, 0.00463414192199707, 0.004691362380981445, 0.004767417907714844, 0.004763126373291016, 0.00486445426940918, 0.004748821258544922, 0.004729032516479492, 0.004708528518676758, 0.004715681076049805, 0.004820585250854492, 0.004762172698974609, 0.0047130584716796875, 0.004693031311035156, 0.0047092437744140625, 0.004826068878173828, 0.0046634674072265625, 0.0047609806060791016, 0.004713773727416992, 0.00484013557434082, 0.004708528518676758, 0.004740476608276367, 0.004705190658569336, 0.004694223403930664, 0.004727840423583984, 0.00473475456237793, 0.004706382751464844, 0.004758596420288086, 0.004732608795166016, 0.004808902740478516, 0.004716634750366211, 0.004773616790771484, 0.0047397613525390625, 0.004668712615966797, 0.004779338836669922, 0.0047566890716552734, 0.004770040512084961, 0.00469207763671875, 0.004742145538330078, 0.004731416702270508, 0.004727363586425781, 0.004753828048706055, 0.004754781723022461, 0.004743099212646484, 0.0047490596771240234, 0.004754304885864258, 0.004711151123046875, 0.0048100948333740234, 0.004813194274902344, 0.004775285720825195, 0.004718780517578125, 0.004702329635620117, 0.004750728607177734, 0.004718780517578125, 0.004700422286987305, 0.004761695861816406, 0.0047643184661865234, 0.004776954650878906, 0.004703521728515625, 0.004714012145996094, 0.004756927490234375, 0.004749298095703125, 0.0049402713775634766, 0.004738330841064453, 0.004738330841064453, 0.0047397613525390625, 0.0047512054443359375, 0.0047149658203125, 0.004713535308837891, 0.0048065185546875, 0.00489497184753418, 0.0047647953033447266, 0.0047075748443603516, 0.0047953128814697266, 0.004725456237792969, 0.004684925079345703, 0.004771232604980469, 0.0047185420989990234, 0.00479888916015625, 0.004714250564575195, 0.0047304630279541016, 0.004728555679321289, 0.004906177520751953, 0.004797458648681641, 0.004745960235595703, 0.004744291305541992, 0.004754304885864258, 0.004736423492431641, 0.004736900329589844, 0.00481724739074707, 0.004830121994018555, 0.004738807678222656, 0.004740715026855469, 0.004722118377685547, 0.004754543304443359, 0.0047931671142578125, 0.004701375961303711, 0.00476527214050293, 0.004753828048706055, 0.0047969818115234375, 0.0046956539154052734, 0.004726886749267578, 0.0047452449798583984, 0.004709720611572266, 0.004868984222412109, 0.004748344421386719, 0.004752397537231445, 0.004917144775390625, 0.0047626495361328125, 0.004758596420288086, 0.004705190658569336, 0.004838466644287109, 0.004764556884765625, 0.004730701446533203, 0.004697322845458984, 0.0047338008880615234, 0.004771232604980469, 0.004734992980957031, 0.0047457218170166016, 0.004761934280395508, 0.004824161529541016, 0.004750728607177734, 0.004734992980957031, 0.004772186279296875, 0.0047757625579833984, 0.0047457218170166016, 0.004755258560180664, 0.004769563674926758, 0.00470423698425293, 0.0047457218170166016, 0.00476837158203125, 0.0048389434814453125, 0.005004167556762695, 0.00472712516784668, 0.004770040512084961, 0.0047490596771240234, 0.004793405532836914, 0.0047566890716552734, 0.004903316497802734, 0.004811525344848633, 0.0047605037689208984, 0.004798412322998047, 0.0047147274017333984, 0.004731416702270508, 0.004731893539428711, 0.0048334598541259766, 0.004812479019165039, 0.004772663116455078, 0.004744052886962891, 0.0047566890716552734, 0.004802703857421875, 0.004731893539428711, 0.004710674285888672, 0.004772186279296875, 0.004717826843261719, 0.004776716232299805, 0.00470733642578125, 0.0048046112060546875, 0.004757404327392578, 0.004738807678222656, 0.004767417907714844, 0.004742622375488281, 0.004813671112060547, 0.004712104797363281, 0.0047495365142822266, 0.0047283172607421875, 0.004758596420288086, 0.00474858283996582, 0.004754543304443359, 0.004764080047607422, 0.004754543304443359, 0.00475764274597168, 0.004745960235595703, 0.004801034927368164, 0.0048062801361083984, 0.004751443862915039, 0.004745960235595703, 0.0046961307525634766, 0.004759073257446289, 0.004820823669433594, 0.004711151123046875, 0.004754304885864258, 0.004752159118652344, 0.004794597625732422, 0.004708766937255859, 0.004747152328491211, 0.004750251770019531, 0.004833698272705078, 0.004883527755737305, 0.004742622375488281, 0.004823207855224609, 0.004776716232299805, 0.004771232604980469, 0.005194902420043945, 0.004793882369995117, 0.004742860794067383, 0.004762172698974609, 0.004813671112060547, 0.004700660705566406, 0.004781246185302734, 0.004753589630126953, 0.004808902740478516, 0.004739046096801758, 0.004705905914306641, 0.004763126373291016, 0.0047605037689208984, 0.0048253536224365234, 0.004742860794067383, 0.004731655120849609, 0.005370140075683594, 0.004823446273803711, 0.004724264144897461, 0.0047533512115478516, 0.004767417907714844, 0.004801511764526367, 0.004870176315307617, 0.004738807678222656, 0.0047760009765625, 0.004774570465087891, 0.004780769348144531, 0.004752397537231445, 0.0047380924224853516, 0.004798412322998047, 0.004777431488037109, 0.004773378372192383, 0.0047855377197265625, 0.004849910736083984, 0.004768848419189453, 0.004725933074951172, 0.0048656463623046875, 0.0047910213470458984, 0.004812955856323242, 0.004700183868408203, 0.004800319671630859, 0.004751920700073242, 0.004837751388549805, 0.004720449447631836, 0.004746437072753906, 0.0047760009765625, 0.004778861999511719, 0.004769802093505859, 0.004755258560180664, 0.004787445068359375, 0.004795074462890625, 0.004780769348144531, 0.00476837158203125, 0.004736900329589844, 0.004830121994018555, 0.004762887954711914, 0.004747629165649414, 0.004717111587524414, 0.004775524139404297, 0.004820585250854492, 0.004715681076049805, 0.004801034927368164, 0.004760265350341797, 0.004803657531738281, 0.004784345626831055, 0.0047457218170166016, 0.00481104850769043, 0.004760026931762695, 0.004830360412597656, 0.004836559295654297, 0.0047647953033447266, 0.004765987396240234, 0.004777431488037109, 0.004790306091308594, 0.0047588348388671875, 0.004813671112060547, 0.004772186279296875, 0.004788398742675781, 0.004807710647583008, 0.004821300506591797, 0.0047724246978759766, 0.004754066467285156, 0.0047914981842041016, 0.004778146743774414, 0.0047948360443115234, 0.004709482192993164, 0.00480961799621582, 0.004938840866088867, 0.004730939865112305, 0.0048449039459228516, 0.0047605037689208984, 0.004801273345947266, 0.004729270935058594, 0.004849433898925781, 0.004755258560180664, 0.004782915115356445, 0.004758596420288086, 0.004757881164550781, 0.004862308502197266, 0.004735469818115234, 0.0048160552978515625, 0.00478053092956543, 0.0048520565032958984, 0.004759550094604492, 0.004796743392944336, 0.0048046112060546875, 0.0048656463623046875, 0.004790067672729492, 0.004784822463989258, 0.004765987396240234, 0.004773378372192383, 0.004765748977661133, 0.004791259765625, 0.004708766937255859, 0.004820823669433594, 0.004768848419189453, 0.004800081253051758, 0.00474858283996582, 0.00482630729675293, 0.004778385162353516, 0.0048449039459228516, 0.004805564880371094, 0.004763126373291016, 0.004812955856323242, 0.004758596420288086, 0.0048084259033203125, 0.004797697067260742, 0.00482487678527832, 0.004758358001708984, 0.004723548889160156, 0.004805564880371094, 0.004757881164550781, 0.004831790924072266, 0.0047876834869384766, 0.004761695861816406, 0.0047664642333984375, 0.004762172698974609, 0.004781961441040039, 0.004738807678222656, 0.004802703857421875, 0.004772663116455078, 0.004771709442138672, 0.004790782928466797, 0.004776954650878906, 0.004800081253051758, 0.00474858283996582, 0.004817962646484375, 0.00479578971862793, 0.0049588680267333984, 0.004755258560180664, 0.00482630729675293, 0.0047948360443115234, 0.004779815673828125, 0.0047893524169921875, 0.0047762393951416016, 0.004784345626831055, 0.004740238189697266, 0.0047910213470458984, 0.004773139953613281, 0.004710197448730469, 0.004840850830078125, 0.004748344421386719, 0.004930734634399414, 0.004743337631225586, 0.004830837249755859, 0.0047533512115478516, 0.004801273345947266, 0.004751920700073242, 0.004765748977661133, 0.004851818084716797, 0.0047528743743896484, 0.00478363037109375, 0.004799842834472656, 0.005423307418823242, 0.004801511764526367, 0.0048024654388427734, 0.005031108856201172, 0.005025148391723633, 0.004757881164550781, 0.004798412322998047, 0.004784584045410156, 0.004812002182006836, 0.004812717437744141, 0.004793882369995117, 0.004803657531738281, 0.004782915115356445, 0.004789829254150391, 0.0047948360443115234, 0.0047571659088134766, 0.004983425140380859, 0.00484013557434082, 0.0047833919525146484, 0.004735469818115234, 0.004829883575439453, 0.004773378372192383, 0.0048084259033203125, 0.004746198654174805, 0.0047914981842041016, 0.004836320877075195, 0.004757881164550781, 0.00481414794921875, 0.004792451858520508, 0.004847288131713867, 0.004864692687988281, 0.004809856414794922, 0.00479888916015625, 0.004791975021362305, 0.004825592041015625, 0.0048062801361083984, 0.004841327667236328, 0.0047740936279296875, 0.0048024654388427734, 0.004770040512084961, 0.005270719528198242, 0.004854440689086914, 0.004868507385253906, 0.005164623260498047, 0.004990816116333008, 0.00482940673828125, 0.004885435104370117, 0.004850864410400391, 0.0048291683197021484, 0.004937887191772461, 0.004857778549194336, 0.004947185516357422, 0.004778623580932617, 0.004906654357910156, 0.004880189895629883, 0.004851102828979492, 0.004824161529541016, 0.004927873611450195, 0.004895925521850586, 0.004857540130615234, 0.004808902740478516, 0.004840850830078125, 0.004887104034423828, 0.004809141159057617, 0.00487971305847168, 0.004828691482543945, 0.004902839660644531, 0.004842042922973633, 0.004876613616943359, 0.00482177734375, 0.0048754215240478516, 0.004902362823486328, 0.004865169525146484, 0.004867076873779297, 0.004854679107666016, 0.004865407943725586, 0.0048329830169677734, 0.0048618316650390625, 0.004858493804931641, 0.004847526550292969, 0.0048329830169677734, 0.004862785339355469, 0.004838228225708008, 0.00510096549987793, 0.0048749446868896484, 0.004825592041015625, 0.004885196685791016, 0.004876613616943359, 0.004920244216918945, 0.004846334457397461, 0.00489497184753418, 0.004858493804931641, 0.0048885345458984375, 0.004859209060668945, 0.0048787593841552734, 0.0048487186431884766, 0.004850625991821289, 0.004828929901123047, 0.004850625991821289, 0.004961967468261719, 0.004796743392944336, 0.004904747009277344, 0.004852771759033203, 0.004865407943725586, 0.0047800540924072266, 0.0048906803131103516, 0.004841804504394531, 0.004864931106567383, 0.004790544509887695, 0.00486445426940918, 0.004863262176513672, 0.00480198860168457, 0.004966259002685547, 0.004909038543701172, 0.004882335662841797, 0.004794120788574219, 0.0048542022705078125, 0.004859447479248047, 0.004878520965576172, 0.004822969436645508, 0.004871845245361328, 0.0050506591796875, 0.0049173831939697266, 0.004824161529541016, 0.004851579666137695, 0.004843473434448242, 0.0048465728759765625, 0.004871845245361328, 0.004856109619140625, 0.004840373992919922, 0.004859209060668945, 0.0048639774322509766, 0.0050258636474609375, 0.004891395568847656, 0.00487828254699707, 0.0048828125, 0.004881381988525391, 0.004780769348144531, 0.0049021244049072266, 0.004877328872680664, 0.004992246627807617, 0.004817962646484375, 0.004897594451904297, 0.0048215389251708984, 0.004876613616943359, 0.004804849624633789, 0.004898548126220703, 0.00485539436340332, 0.0048749446868896484, 0.004823207855224609, 0.004841804504394531, 0.004911899566650391, 0.004799604415893555, 0.004979372024536133, 0.004843711853027344, 0.004920005798339844, 0.004804134368896484, 0.004823446273803711, 0.004877805709838867, 0.0049250125885009766, 0.004812955856323242, 0.0048389434814453125, 0.0048944950103759766, 0.004876852035522461, 0.004877805709838867, 0.004814624786376953, 0.004873752593994141, 0.00487971305847168, 0.004858493804931641, 0.0048313140869140625, 0.004850625991821289, 0.00484919548034668, 0.004877328872680664, 0.004885435104370117, 0.004831790924072266, 0.004925727844238281, 0.004807233810424805, 0.004865407943725586, 0.0048558712005615234, 0.0049285888671875, 0.004865169525146484, 0.00495600700378418, 0.004796743392944336, 0.004930257797241211, 0.004881620407104492, 0.004875898361206055, 0.00482487678527832, 0.005188465118408203, 0.004914999008178711, 0.004869937896728516, 0.004846096038818359, 0.004843235015869141, 0.004924774169921875, 0.004791736602783203, 0.004990577697753906, 0.00485682487487793, 0.004910469055175781, 0.004826784133911133, 0.004869222640991211, 0.0048596858978271484, 0.004912614822387695, 0.004804849624633789, 0.004858732223510742, 0.004887580871582031, 0.004885196685791016, 0.004887104034423828, 0.004926443099975586, 0.004842281341552734, 0.004885196685791016, 0.004864215850830078, 0.004854917526245117, 0.004868507385253906, 0.004865169525146484, 0.0048558712005615234, 0.004845857620239258, 0.004812479019165039, 0.004897356033325195, 0.004870176315307617, 0.004881143569946289, 0.00484466552734375, 0.0049440860748291016, 21.330241918563843]
t_14_gzip = [0.004904031753540039, 0.003627300262451172, 0.0038373470306396484, 0.0038824081420898438, 0.003992557525634766, 0.00394892692565918, 0.003850221633911133, 0.0038404464721679688, 0.0038454532623291016, 0.003849029541015625, 0.006167411804199219, 0.0052433013916015625, 0.004303932189941406, 0.005156993865966797, 0.005330562591552734, 0.004849433898925781, 0.005106687545776367, 0.004937171936035156, 0.005033016204833984, 0.005158901214599609, 0.0050814151763916016, 0.004916191101074219, 0.005063295364379883, 0.0051326751708984375, 0.004900217056274414, 0.0051767826080322266, 0.004841327667236328, 0.004131793975830078, 0.003742694854736328, 0.004481315612792969, 0.004096508026123047, 0.004202604293823242, 0.0057184696197509766, 0.0037071704864501953, 0.0036344528198242188, 0.0036487579345703125, 0.003631591796875, 0.0036716461181640625, 0.003648996353149414, 0.0037500858306884766, 0.0036401748657226562, 0.0036394596099853516, 0.0036635398864746094, 0.0036203861236572266, 0.003644704818725586, 0.003631591796875, 0.0036284923553466797, 0.003694295883178711, 0.003634214401245117, 0.0036509037017822266, 0.0036330223083496094, 0.003620624542236328, 0.0036363601684570312, 0.003626108169555664, 0.0036377906799316406, 0.0036177635192871094, 0.0035996437072753906, 0.0037353038787841797, 0.003634929656982422, 0.0036673545837402344, 0.003632068634033203, 0.003589630126953125, 0.0036242008209228516, 0.0036034584045410156, 0.0036132335662841797, 0.003614664077758789, 0.0036172866821289062, 0.0035898685455322266, 0.0036585330963134766, 0.0037043094635009766, 0.003621816635131836, 0.003628969192504883, 0.0035991668701171875, 0.0036458969116210938, 0.003602266311645508, 0.0036551952362060547, 0.003628253936767578, 0.003656148910522461, 0.0035924911499023438, 0.0036401748657226562, 0.0036089420318603516, 0.0036759376525878906, 0.0036497116088867188, 0.0036096572875976562, 0.003644704818725586, 0.0036041736602783203, 0.0036537647247314453, 0.003674030303955078, 0.0036509037017822266, 0.003618478775024414, 0.0036516189575195312, 0.0036296844482421875, 0.0036389827728271484, 0.0036721229553222656, 0.0037393569946289062, 0.003657817840576172, 0.003599405288696289, 0.0036339759826660156, 0.0036427974700927734, 0.003656148910522461, 0.003602266311645508, 0.003646373748779297, 0.003651857376098633, 0.0036516189575195312, 0.003645658493041992, 0.0036029815673828125, 0.003638744354248047, 0.00365447998046875, 0.0036475658416748047, 0.003657102584838867, 0.0036325454711914062, 0.0036084651947021484, 0.0037522315979003906, 0.0037021636962890625, 0.003664731979370117, 0.0036797523498535156, 0.003615140914916992, 0.003669261932373047, 0.0036115646362304688, 0.003656625747680664, 0.003628253936767578, 0.003638029098510742, 0.0036132335662841797, 0.003851652145385742, 0.0036389827728271484, 0.003638744354248047, 0.003780364990234375, 0.003615856170654297, 0.003642559051513672, 0.0036203861236572266, 0.003627300262451172, 0.0036535263061523438, 0.003652334213256836, 0.003668546676635742, 0.003606557846069336, 0.0036308765411376953, 0.0036389827728271484, 0.0036497116088867188, 0.0039920806884765625, 0.0036618709564208984, 0.003618001937866211, 0.0036444664001464844, 0.0036270618438720703, 0.0036716461181640625, 0.0036461353302001953, 0.0035898685455322266, 0.0036513805389404297, 0.003636598587036133, 0.0036330223083496094, 0.003705739974975586, 0.003648519515991211, 0.0036029815673828125, 0.003640413284301758, 0.003733396530151367, 0.003606557846069336, 0.003633737564086914, 0.003601551055908203, 0.0036492347717285156, 0.0036516189575195312, 0.0036482810974121094, 0.0036089420318603516, 0.003628969192504883, 0.003621339797973633, 0.003638029098510742, 0.0036988258361816406, 0.0036039352416992188, 0.0037043094635009766, 0.0036220550537109375, 0.003630399703979492, 0.0037736892700195312, 0.0036406517028808594, 0.0035991668701171875, 0.0036668777465820312, 0.0036110877990722656, 0.003662109375, 0.0036687850952148438, 0.003635883331298828, 0.0036628246307373047, 0.003631114959716797, 0.003603219985961914, 0.003647327423095703, 0.003596782684326172, 0.0036444664001464844, 0.003621816635131836, 0.00571131706237793, 0.00601649284362793, 0.005969047546386719, 0.0060482025146484375, 0.006997823715209961, 0.013018131256103516, 0.013612031936645508, 0.005885124206542969, 0.0055997371673583984, 0.0046193599700927734, 0.0074214935302734375, 0.015157938003540039, 0.012171030044555664, 0.004998445510864258, 0.004189491271972656, 0.004147768020629883, 0.004067182540893555, 0.004004001617431641, 0.004021644592285156, 0.00407719612121582, 0.003900766372680664, 0.003768444061279297, 0.0037360191345214844, 0.003641843795776367, 0.0036458969116210938, 0.0036911964416503906, 0.0036644935607910156, 0.003681659698486328, 0.0036287307739257812, 0.0036759376525878906, 0.00365447998046875, 0.003649473190307617, 0.003606081008911133, 0.0036554336547851562, 0.003610849380493164, 0.0037240982055664062, 0.0036492347717285156, 0.003679990768432617, 0.0036611557006835938, 0.0036127567291259766, 0.0036602020263671875, 0.0036427974700927734, 0.0036330223083496094, 0.0036072731018066406, 0.003651142120361328, 0.003620147705078125, 0.0036630630493164062, 0.003652811050415039, 0.003679037094116211, 0.003662109375, 0.003626585006713867, 0.003669261932373047, 0.0036573410034179688, 0.003710508346557617, 0.003625631332397461, 0.003658294677734375, 0.0036208629608154297, 0.003764629364013672, 0.003733396530151367, 0.0036122798919677734, 0.003682374954223633, 0.0036172866821289062, 0.003660917282104492, 0.003693103790283203, 0.0036623477935791016, 0.003622770309448242, 0.003666400909423828, 0.003629446029663086, 0.0037698745727539062, 0.0037763118743896484, 0.003629446029663086, 0.0036787986755371094, 0.003610849380493164, 0.003738880157470703, 0.0037584304809570312, 0.0036542415618896484, 0.0036156177520751953, 0.0036742687225341797, 0.0036804676055908203, 0.0036749839782714844, 0.0036580562591552734, 0.003677845001220703, 0.003643512725830078, 0.003773927688598633, 0.0036516189575195312, 0.003665924072265625, 0.0036652088165283203, 0.0038590431213378906, 0.0036470890045166016, 0.003669261932373047, 0.003673553466796875, 0.003652334213256836, 0.0036122798919677734, 0.0036504268646240234, 0.003621339797973633, 0.0036628246307373047, 0.0036504268646240234, 0.003659486770629883, 0.0036780834197998047, 0.0036101341247558594, 0.003663301467895508, 0.003690958023071289, 0.003828287124633789, 0.0036246776580810547, 0.0036613941192626953, 0.003618478775024414, 0.0036597251892089844, 0.0037169456481933594, 0.00362396240234375, 0.0036623477935791016, 0.003625631332397461, 0.003793478012084961, 0.0036690235137939453, 0.0036613941192626953, 0.0036172866821289062, 0.0036690235137939453, 0.0036232471466064453, 0.0037207603454589844, 0.0036628246307373047, 0.0036230087280273438, 0.003660440444946289, 0.0036208629608154297, 0.003656148910522461, 0.0036649703979492188, 0.003699779510498047, 0.0036497116088867188, 0.003659486770629883, 0.0036308765411376953, 0.003782510757446289, 0.0036737918853759766, 0.003736257553100586, 0.003668546676635742, 0.003670930862426758, 0.0036640167236328125, 0.003683805465698242, 0.0036878585815429688, 0.0036368370056152344, 0.003715038299560547, 0.0036592483520507812, 0.0036780834197998047, 0.003703594207763672, 0.0036242008209228516, 0.00370025634765625, 0.0036420822143554688, 0.0036721229553222656, 0.003701448440551758, 0.003667593002319336, 0.0037217140197753906, 0.0036406517028808594, 0.0036978721618652344, 0.0036745071411132812, 0.0036826133728027344, 0.0036363601684570312, 0.0036706924438476562, 0.0036499500274658203, 0.0036840438842773438, 0.003673553466796875, 0.0036683082580566406, 0.0036766529083251953, 0.0036361217498779297, 0.0036852359771728516, 0.0036745071411132812, 0.003673076629638672, 0.0036458969116210938, 0.003675699234008789, 0.003636598587036133, 0.003809213638305664, 0.003688335418701172, 0.0036406517028808594, 0.003676891326904297, 0.003641366958618164, 0.0036818981170654297, 0.003667593002319336, 0.0036787986755371094, 0.003629446029663086, 0.0044519901275634766, 0.0037088394165039062, 0.003622770309448242, 0.003749370574951172, 0.00403904914855957, 0.003735065460205078, 0.003645658493041992, 0.0038080215454101562, 0.003821849822998047, 0.0038378238677978516, 0.003699064254760742, 0.0036497116088867188, 0.003687620162963867, 0.0037162303924560547, 0.0036661624908447266, 0.0037097930908203125, 0.003701448440551758, 0.003741025924682617, 0.003641366958618164, 0.003859281539916992, 0.0037000179290771484, 0.003669261932373047, 0.0037000179290771484, 0.003679037094116211, 0.0036292076110839844, 0.003742218017578125, 0.0037529468536376953, 0.0036270618438720703, 0.003686189651489258, 0.003644704818725586, 0.003674030303955078, 0.003751516342163086, 0.0036764144897460938, 0.0036535263061523438, 0.003818035125732422, 0.0036945343017578125, 0.0036818981170654297, 0.0036678314208984375, 0.0036780834197998047, 0.0036382675170898438, 0.003744363784790039, 0.003620624542236328, 0.003682851791381836, 0.0037355422973632812, 0.0037026405334472656, 0.00380706787109375, 0.0036470890045166016, 0.00366973876953125, 0.003690004348754883, 0.003753185272216797, 0.0036287307739257812, 0.0036699771881103516, 0.003638029098510742, 0.00371551513671875, 0.0038585662841796875, 0.0036759376525878906, 0.0036721229553222656, 0.0036296844482421875, 0.0037031173706054688, 0.0036983489990234375, 0.0036504268646240234, 0.003805398941040039, 0.0036804676055908203, 0.0036468505859375, 0.003666400909423828, 0.0036919116973876953, 0.003729104995727539, 0.0036759376525878906, 0.0036804676055908203, 0.0036346912384033203, 0.0037136077880859375, 0.003692626953125, 0.003680706024169922, 0.0036792755126953125, 0.0036394596099853516, 0.003701925277709961, 0.0036411285400390625, 0.0036897659301757812, 0.003676891326904297, 0.0036728382110595703, 0.0036368370056152344, 0.0037202835083007812, 0.003697633743286133, 0.0036911964416503906, 0.0036826133728027344, 0.0036954879760742188, 0.003754138946533203, 0.0037806034088134766, 0.0038156509399414062, 0.0039904117584228516, 0.003907203674316406, 0.003936052322387695, 0.0039021968841552734, 0.003945589065551758, 0.004045248031616211, 0.003951311111450195, 0.004001617431640625, 0.003936767578125, 0.004132509231567383, 0.004062175750732422, 0.004065036773681641, 0.004052877426147461, 0.003942728042602539, 0.003995180130004883, 0.004029750823974609, 0.003943443298339844, 0.004130125045776367, 0.004063129425048828, 0.004118919372558594, 0.004186391830444336, 0.004062175750732422, 0.004107952117919922, 0.004064798355102539, 0.004248380661010742, 0.004128217697143555, 0.00409245491027832, 0.0041179656982421875, 0.0040929317474365234, 0.004105806350708008, 0.004106998443603516, 0.004069805145263672, 0.0041141510009765625, 0.004104137420654297, 0.004122257232666016, 0.004124164581298828, 0.004059553146362305, 0.004177570343017578, 0.004190921783447266, 0.004111289978027344, 0.004097938537597656, 0.004111051559448242, 0.004054546356201172, 0.004141569137573242, 0.004119396209716797, 0.004100799560546875, 0.00413203239440918, 0.0040776729583740234, 0.0040874481201171875, 0.0041501522064208984, 0.004073381423950195, 0.0041046142578125, 0.004059553146362305, 0.004114389419555664, 0.0041561126708984375, 0.004063844680786133, 0.004116058349609375, 0.00405430793762207, 0.004309654235839844, 0.004121065139770508, 0.0040740966796875, 0.0041043758392333984, 0.004110813140869141, 0.0041048526763916016, 0.004164695739746094, 0.0040836334228515625, 0.004118919372558594, 0.0041124820709228516, 0.004105567932128906, 0.004125356674194336, 0.004064083099365234, 0.004118442535400391, 0.004092216491699219, 0.004201650619506836, 0.004117250442504883, 0.0041179656982421875, 0.004076957702636719, 0.004118680953979492, 0.004095554351806641, 0.004122018814086914, 0.0041217803955078125, 0.004075050354003906, 0.004102230072021484, 0.004145622253417969, 0.00411677360534668, 0.0041544437408447266, 0.004095315933227539, 0.004102230072021484, 0.004114627838134766, 0.004134416580200195, 0.004094123840332031, 0.004108905792236328, 0.004141092300415039, 0.004102230072021484, 0.00407719612121582, 0.004118919372558594, 0.004170894622802734, 0.0040740966796875, 0.0041544437408447266, 0.004075288772583008, 0.004129886627197266, 0.004187107086181641, 0.004073143005371094, 0.00410008430480957, 0.0040700435638427734, 0.004155635833740234, 0.004128932952880859, 0.004215717315673828, 0.004113435745239258, 0.004113435745239258, 0.004113435745239258, 0.00412440299987793, 0.004082918167114258, 0.0041429996490478516, 0.00410771369934082, 0.004124164581298828, 0.0041425228118896484, 0.0040607452392578125, 0.004155397415161133, 0.004128932952880859, 0.004108428955078125, 0.004121541976928711, 0.004271507263183594, 0.004061222076416016, 0.003937244415283203, 0.0039081573486328125, 0.0038979053497314453, 0.003925323486328125, 0.003906726837158203, 0.0040781497955322266, 0.004050493240356445, 0.004159450531005859, 0.004189491271972656, 0.004141807556152344, 0.004228353500366211, 0.003866910934448242, 0.00389862060546875, 0.0038673877716064453, 0.003823518753051758, 0.003884553909301758, 0.0038864612579345703, 0.003877401351928711, 0.004188060760498047, 0.004297018051147461, 0.003705739974975586, 0.003702402114868164, 0.003704547882080078, 0.0036509037017822266, 0.0037276744842529297, 0.0037724971771240234, 0.0037050247192382812, 0.003709554672241211, 0.0036423206329345703, 0.0036864280700683594, 0.0036733150482177734, 0.003702878952026367, 0.003709077835083008, 0.0037689208984375, 0.003717184066772461, 0.0036516189575195312, 0.003701925277709961, 0.0038192272186279297, 0.0036988258361816406, 0.0037064552307128906, 0.003709077835083008, 0.003650188446044922, 0.0037004947662353516, 0.003720998764038086, 0.003701925277709961, 0.0037119388580322266, 0.0037055015563964844, 0.0036962032318115234, 0.0036661624908447266, 0.0036983489990234375, 0.003795623779296875, 0.0037047863006591797, 0.0037767887115478516, 0.003697633743286133, 0.0036928653717041016, 0.0036957263946533203, 0.003694295883178711, 0.003661632537841797, 0.003703594207763672, 0.003665924072265625, 0.0037310123443603516, 0.003892660140991211, 0.0036611557006835938, 0.0037114620208740234, 0.0036656856536865234, 0.003713846206665039, 0.0036978721618652344, 0.0037031173706054688, 0.003656625747680664, 0.0037031173706054688, 0.003701925277709961, 0.0036916732788085938, 0.003699064254760742, 0.003690481185913086, 0.0036916732788085938, 0.0036597251892089844, 0.003756284713745117, 0.0037047863006591797, 0.003708362579345703, 0.003696441650390625, 0.0036652088165283203, 0.003694295883178711, 0.0036916732788085938, 0.0037004947662353516, 0.003667593002319336, 0.003699064254760742, 0.0036444664001464844, 0.0037012100219726562, 0.0039017200469970703, 0.003656148910522461, 0.0037081241607666016, 0.003656625747680664, 0.0036966800689697266, 0.0036945343017578125, 0.003702402114868164, 0.003667116165161133, 0.0037055015563964844, 0.0037245750427246094, 0.003729581832885742, 0.003695249557495117, 0.0036759376525878906, 0.0037331581115722656, 0.003675222396850586, 0.0037026405334472656, 0.0037822723388671875, 0.003718852996826172, 0.003710031509399414, 0.003670215606689453, 0.0037157535552978516, 0.0037078857421875, 0.0037081241607666016, 0.003665447235107422, 0.0037114620208740234, 0.003653287887573242, 0.0037031173706054688, 0.003765583038330078, 0.003657102584838867, 0.0037157535552978516, 0.0036699771881103516, 0.003698110580444336, 0.0037169456481933594, 0.0037078857421875, 0.0036537647247314453, 0.0037038326263427734, 0.0037147998809814453, 0.0037441253662109375, 0.0037086009979248047, 0.003673553466796875, 0.0037119388580322266, 0.0037071704864501953, 0.0037305355072021484, 0.003705739974975586, 0.003732919692993164, 0.003731250762939453, 0.0036804676055908203, 0.003726959228515625, 0.0037164688110351562, 0.003717184066772461, 0.0036792755126953125, 0.0037169456481933594, 0.003670215606689453, 0.0037839412689208984, 0.0038383007049560547, 0.0036804676055908203, 0.0037202835083007812, 0.0036840438842773438, 0.0037140846252441406, 0.003720521926879883, 0.0037071704864501953, 0.0036661624908447266, 0.003738880157470703, 0.0037238597869873047, 0.003714323043823242, 0.003719806671142578, 0.003682374954223633, 0.003704547882080078, 0.0036716461181640625, 0.0037088394165039062, 0.003820657730102539, 0.0037164688110351562, 0.003731966018676758, 0.0036683082580566406, 0.003705263137817383, 0.0037276744842529297, 0.003709554672241211, 0.003676891326904297, 0.003710508346557617, 0.00370025634765625, 0.003770589828491211, 0.003721952438354492, 0.0036742687225341797, 0.0037136077880859375, 0.0036897659301757812, 0.0037086009979248047, 0.004012107849121094, 0.003816843032836914, 0.0037336349487304688, 0.0037086009979248047, 0.0037088394165039062, 0.0037250518798828125, 0.0037114620208740234, 0.0037233829498291016, 0.0037147998809814453, 0.003695964813232422, 0.0037271976470947266, 0.0037107467651367188, 0.0037202835083007812, 0.0037336349487304688, 0.0037124156951904297, 0.0036835670471191406, 0.0037310123443603516, 0.0036749839782714844, 0.0037224292755126953, 0.0037975311279296875, 0.0036745071411132812, 0.0038182735443115234, 0.0036873817443847656, 0.0037467479705810547, 0.003717184066772461, 0.0037221908569335938, 0.0036983489990234375, 0.0037288665771484375, 0.0037250518798828125, 0.0037114620208740234, 0.0037207603454589844, 0.003683328628540039, 0.0037393569946289062, 0.0036880970001220703, 0.00372314453125, 0.003718852996826172, 0.0037336349487304688, 0.0038063526153564453, 0.0036902427673339844, 0.0037331581115722656, 0.0037114620208740234, 0.0037126541137695312, 0.0036792755126953125, 0.003745555877685547, 0.0037980079650878906, 0.003881216049194336, 0.0037200450897216797, 0.0037186145782470703, 0.0037283897399902344, 0.003726482391357422, 0.0036764144897460938, 0.0037221908569335938, 0.0038034915924072266, 0.0037221908569335938, 0.003729581832885742, 0.003726482391357422, 0.0037114620208740234, 0.0036661624908447266, 0.003725767135620117, 0.0037145614624023438, 0.0037126541137695312, 0.0037386417388916016, 0.0036737918853759766, 0.0037348270416259766, 0.0037512779235839844, 0.0043811798095703125, 0.004091024398803711, 0.0037469863891601562, 0.003721952438354492, 0.0037183761596679688, 0.0037276744842529297, 0.0036764144897460938, 0.003727436065673828, 0.0037348270416259766, 0.003726959228515625, 0.0038275718688964844, 0.003706216812133789, 0.0037240982055664062, 0.0036971569061279297, 0.0037431716918945312, 0.0037140846252441406, 0.003723621368408203, 0.0036878585815429688, 0.0037240982055664062, 0.0037703514099121094, 0.003732919692993164, 0.003731966018676758, 0.0037636756896972656, 0.0037355422973632812, 0.0036821365356445312, 0.003717184066772461, 0.003842592239379883, 0.0037360191345214844, 0.00373077392578125, 0.0037326812744140625, 0.003724813461303711, 0.003743410110473633, 0.003741741180419922, 0.003674745559692383, 0.003741741180419922, 0.0037479400634765625, 0.0037288665771484375, 0.003727436065673828, 0.0037581920623779297, 0.003720998764038086, 0.003671884536743164, 0.0037412643432617188, 0.0037124156951904297, 0.003722667694091797, 0.003725290298461914, 0.0036699771881103516, 0.0037567615509033203, 0.0037250518798828125, 0.003733396530151367, 0.0036809444427490234, 0.0037322044372558594, 0.003690958023071289, 0.0037391185760498047, 0.0038352012634277344, 0.0057866573333740234, 0.0058062076568603516, 0.005614280700683594, 0.003802061080932617, 0.0037622451782226562, 0.003728628158569336, 0.003715038299560547, 0.0037674903869628906, 0.0038602352142333984, 0.0037806034088134766, 0.0037415027618408203, 0.0037360191345214844, 0.0037567615509033203, 0.003687143325805664, 0.0037429332733154297, 0.003682851791381836, 0.004000663757324219, 0.003739595413208008, 0.0036814212799072266, 0.0037496089935302734, 0.0037081241607666016, 0.0037224292755126953, 0.0037393569946289062, 0.0038607120513916016, 0.0037500858306884766, 0.0036864280700683594, 0.003809690475463867, 0.003754138946533203, 0.003737926483154297, 0.0036923885345458984, 0.0037317276000976562, 0.003679037094116211, 0.0038001537322998047, 0.0037183761596679688, 0.003697633743286133, 0.0038955211639404297, 0.0036890506744384766, 0.003721475601196289, 0.0037267208099365234, 0.0037317276000976562, 0.003746509552001953, 0.0036706924438476562, 0.003763914108276367, 0.003745555877685547, 0.003760814666748047, 0.003679513931274414, 0.0037348270416259766, 0.003688335418701172, 0.003777742385864258, 0.003805875778198242, 0.0037915706634521484, 0.0037419795989990234, 0.0037708282470703125, 0.003755331039428711, 0.003735780715942383, 0.0037386417388916016, 0.003747701644897461, 0.003702402114868164, 0.003738880157470703, 0.003731966018676758, 0.003725290298461914, 0.0037555694580078125, 0.0037975311279296875, 0.0036907196044921875, 0.003795623779296875, 0.003742218017578125, 0.003689289093017578, 0.003745555877685547, 0.003702878952026367, 0.0037374496459960938, 0.003739595413208008, 0.003736257553100586, 0.0036895275115966797, 0.003744840621948242, 0.003766775131225586, 0.0037398338317871094, 0.003747701644897461, 0.003795146942138672, 0.0037453174591064453, 0.0036995410919189453, 0.003737211227416992, 0.0037844181060791016, 0.0037224292755126953, 0.003744840621948242, 0.0037202835083007812, 0.003753662109375, 0.0037343502044677734, 0.0037581920623779297, 0.0036995410919189453, 0.0037696361541748047, 0.003734111785888672, 0.0037336349487304688, 0.003751516342163086, 0.0037050247192382812, 0.0037496089935302734, 0.0037021636962890625, 0.0037636756896972656, 0.0038080215454101562, 0.0037164688110351562, 0.003753185272216797, 0.0037102699279785156, 0.0037484169006347656, 0.003744840621948242, 0.0037603378295898438, 0.0037262439727783203, 0.0037469863891601562, 0.003743410110473633, 0.0037565231323242188, 0.0037386417388916016, 0.003704547882080078, 0.0037567615509033203, 0.0036878585815429688, 0.003773927688598633, 0.0037393569946289062, 0.0037565231323242188, 0.003873586654663086, 0.0037140846252441406, 0.0037643909454345703, 0.003748178482055664, 0.003758668899536133, 0.003698587417602539, 0.003751516342163086, 0.003748655319213867, 0.0037431716918945312, 0.003757476806640625, 0.003711700439453125, 0.003737926483154297, 0.003717660903930664, 0.0037317276000976562, 0.0037386417388916016, 0.0038356781005859375, 0.0037360191345214844, 0.0037026405334472656, 0.003752470016479492, 0.0037555694580078125, 0.0037288665771484375, 0.0036995410919189453, 0.0037584304809570312, 0.0037393569946289062, 0.0037436485290527344, 0.003744363784790039, 0.0036895275115966797, 0.0037505626678466797, 0.0036988258361816406, 0.003743886947631836, 0.0039517879486083984, 0.004143714904785156, 0.0037550926208496094, 0.0037162303924560547, 0.0037865638732910156, 0.0042722225189208984, 0.003773212432861328, 0.0037631988525390625, 0.0037069320678710938, 0.0037713050842285156, 0.003757953643798828, 0.0037720203399658203, 0.003705263137817383, 0.003731966018676758, 0.0037145614624023438, 0.003810882568359375, 0.003737211227416992, 0.0036950111389160156, 0.003754138946533203, 0.0037126541137695312, 0.0037415027618408203, 0.0037407875061035156, 0.003744840621948242, 0.003828763961791992, 0.0037164688110351562, 0.0037398338317871094, 0.003746509552001953, 0.003755331039428711, 0.003717184066772461, 0.0037689208984375, 0.0037832260131835938, 0.003803730010986328, 0.003737926483154297, 0.0037131309509277344, 0.003755331039428711, 0.0037384033203125, 0.0037314891815185547, 0.003752470016479492, 0.003892183303833008, 0.0037724971771240234, 0.0037076473236083984, 0.003736734390258789, 0.003782033920288086, 0.003746509552001953, 0.003698110580444336, 0.0037412643432617188, 0.003748178482055664, 0.0037665367126464844, 0.003744363784790039, 0.0037603378295898438, 0.0037424564361572266, 0.0038368701934814453, 0.003874540328979492, 0.0037403106689453125, 0.003785371780395508, 0.0037467479705810547, 0.003699064254760742, 0.0037488937377929688, 0.003755807876586914, 0.0037658214569091797, 0.0037212371826171875, 0.003739595413208008, 0.0038003921508789062, 0.0037462711334228516, 0.003770112991333008, 0.003712177276611328, 0.0038537979125976562, 0.0037152767181396484, 0.0037467479705810547, 0.0037794113159179688, 0.003773927688598633, 0.0037581920623779297, 0.0037183761596679688, 0.0037653446197509766, 0.003756999969482422, 0.003755331039428711, 0.0037207603454589844, 0.003880739212036133, 0.0037491321563720703, 0.003771543502807617, 0.003739595413208008, 0.0037288665771484375, 0.003763914108276367, 0.003718137741088867, 0.003808736801147461, 0.003813028335571289, 0.003693103790283203, 0.0037508010864257812, 0.0037603378295898438, 0.003761768341064453, 0.0037496089935302734, 0.0037615299224853516, 0.003910541534423828, 0.0038022994995117188, 0.0037190914154052734, 0.0037374496459960938, 0.0038099288940429688, 0.0037641525268554688, 0.0037467479705810547, 0.0037734508514404297, 0.0037145614624023438, 0.003812551498413086, 0.0037467479705810547, 0.0037016868591308594, 0.0037496089935302734, 0.0037221908569335938, 0.0037453174591064453, 0.003748655319213867, 0.0037686824798583984, 0.0037653446197509766, 0.003705739974975586, 0.0037522315979003906, 0.0037572383880615234, 0.00374603271484375, 0.003722667694091797, 0.003745555877685547, 0.0037255287170410156, 0.003800630569458008, 0.003763914108276367, 0.003753662109375, 0.003747701644897461, 0.003717184066772461, 0.0037584304809570312, 0.0037317276000976562, 0.003762960433959961, 0.003753662109375, 0.003718137741088867, 0.003770112991333008, 0.0037696361541748047, 0.003836393356323242, 0.003959178924560547, 0.003846883773803711, 0.0038628578186035156, 0.0038471221923828125, 0.003824949264526367, 0.003797769546508789, 0.0038328170776367188, 0.0038034915924072266, 0.0038671493530273438, 0.0038862228393554688, 0.0038046836853027344, 0.0038411617279052734, 0.0038101673126220703, 0.00383758544921875, 0.00383758544921875, 0.003965139389038086, 0.0038475990295410156, 0.0038754940032958984, 0.003868579864501953, 0.0038652420043945312, 0.0038328170776367188, 0.0038623809814453125, 0.0038132667541503906, 0.0038323402404785156, 0.003843069076538086, 0.0038232803344726562, 0.0039522647857666016, 0.0038526058197021484, 0.0038671493530273438, 0.003852367401123047, 0.0038492679595947266, 0.003823995590209961, 0.003855466842651367, 0.0038084983825683594, 0.003912210464477539, 0.003839254379272461, 0.003798246383666992, 0.003839731216430664, 0.0038099288940429688, 0.0039174556732177734, 0.003873586654663086, 0.003861665725708008, 0.0038614273071289062, 0.003802061080932617, 0.0038497447967529297, 0.0038580894470214844, 0.003839254379272461, 0.003864765167236328, 0.003815174102783203, 0.003873109817504883, 0.0038411617279052734, 0.003845691680908203, 0.0039005279541015625, 0.0038771629333496094, 0.003858327865600586, 0.0038301944732666016, 0.0038444995880126953, 0.003832578659057617, 0.0038399696350097656, 0.0038089752197265625, 0.0038940906524658203, 0.003832101821899414, 0.0038111209869384766, 0.003845691680908203, 0.0038056373596191406, 0.0038542747497558594, 0.003981351852416992, 0.0038442611694335938, 0.003857851028442383, 0.003834247589111328, 0.003842592239379883, 0.0038576126098632812, 0.003864288330078125, 0.0038602352142333984, 0.003835439682006836, 0.0038712024688720703, 0.0038793087005615234, 0.003864288330078125, 0.0038404464721679688, 0.00386810302734375, 0.0038580894470214844, 0.0038597583770751953, 0.003846883773803711, 0.003804922103881836, 0.0038611888885498047, 0.0038099288940429688, 0.003890514373779297, 0.0038661956787109375, 0.0038170814514160156, 0.0038607120513916016, 0.0038154125213623047, 0.0038475990295410156, 0.003911733627319336, 0.0038230419158935547, 0.0038514137268066406, 0.003817319869995117, 0.0038602352142333984, 0.003879547119140625, 0.0038793087005615234, 0.003868579864501953, 0.003818988800048828, 0.0038619041442871094, 0.0038652420043945312, 0.003902435302734375, 0.004086732864379883, 0.003842592239379883, 0.003852367401123047, 0.0038573741912841797, 0.0038461685180664062, 0.0038220882415771484, 0.0038743019104003906, 0.003869771957397461, 0.0038802623748779297, 0.0038530826568603516, 0.00382232666015625, 0.0038640499114990234, 0.0038177967071533203, 0.003912925720214844, 0.003937959671020508, 0.003846406936645508, 0.0038552284240722656, 0.003816366195678711, 0.003866910934448242, 0.00393223762512207, 0.0038306713104248047, 0.0038514137268066406, 0.0038247108459472656, 0.0038487911224365234, 0.003859996795654297, 0.0038793087005615234, 0.003856658935546875, 0.0038251876831054688, 0.0038695335388183594, 0.00388336181640625, 0.003857851028442383, 0.0038208961486816406, 0.003863096237182617, 0.0038712024688720703, 0.003854990005493164, 0.0038728713989257812, 0.003827333450317383, 0.0038576126098632812, 0.0038623809814453125, 0.003854036331176758, 0.0038900375366210938, 0.0038247108459472656, 0.0038576126098632812, 0.0038335323333740234, 0.003850698471069336, 0.003920793533325195, 0.0038390159606933594, 0.003853321075439453, 0.003846406936645508, 0.003844738006591797, 0.0038518905639648438, 0.0038704872131347656, 0.003949165344238281, 0.003839731216430664, 0.0038499832153320312, 0.0038652420043945312, 0.003869295120239258, 0.0038847923278808594, 0.003824949264526367, 0.005163431167602539, 0.00428009033203125, 0.004580497741699219, 0.004148244857788086, 0.004173994064331055, 0.004209995269775391, 0.005401611328125, 0.004570960998535156, 0.004170894622802734, 0.00469207763671875, 0.005828380584716797, 0.00554203987121582, 0.005362033843994141, 0.005418062210083008, 0.005606651306152344, 0.005410194396972656, 0.0056400299072265625, 0.0054395198822021484, 0.005439281463623047, 0.005475044250488281, 0.005525350570678711, 0.005477190017700195, 0.005671501159667969, 0.0041351318359375, 0.003916501998901367, 0.003961801528930664, 0.004395008087158203, 0.004296302795410156, 0.006746768951416016, 0.004077434539794922, 0.003936052322387695, 0.003955841064453125, 0.003896951675415039, 0.003892183303833008, 0.0038995742797851562, 0.003912210464477539, 0.004072904586791992, 0.003909587860107422, 0.0038955211639404297, 0.0038852691650390625, 0.003911495208740234, 0.0038793087005615234, 0.003908872604370117, 0.0038864612579345703, 0.0038933753967285156, 0.003862142562866211, 0.0038461685180664062, 0.00392603874206543, 0.0038917064666748047, 0.003897428512573242, 0.0039789676666259766, 0.0038461685180664062, 0.0038940906524658203, 0.0038487911224365234, 0.00393986701965332, 0.003879070281982422, 0.0038433074951171875, 0.003904581069946289, 0.003835439682006836, 0.0038869380950927734, 0.003926992416381836, 0.003844022750854492, 0.0038902759552001953, 0.0038454532623291016, 0.003882884979248047, 0.0038764476776123047, 0.0038919448852539062, 0.0038824081420898438, 0.0038416385650634766, 0.0038967132568359375, 0.0038771629333496094, 0.0038933753967285156, 0.003895998001098633, 0.0038509368896484375, 0.0038843154907226562, 0.003916501998901367, 0.003922700881958008, 0.00392460823059082, 0.0039033889770507812, 0.003871440887451172, 0.003911018371582031, 0.0038924217224121094, 0.0038459300994873047, 0.0038793087005615234, 0.0038900375366210938, 0.0038890838623046875, 0.0038917064666748047, 0.0038390159606933594, 0.003983497619628906, 0.003854036331176758, 0.003918886184692383, 0.0038797855377197266, 0.003834962844848633, 0.0038840770721435547, 0.003843069076538086, 0.003873109817504883, 0.00392913818359375, 0.0039005279541015625, 0.0038862228393554688, 0.0038492679595947266, 0.00386810302734375, 0.003888368606567383, 0.00393223762512207, 0.003883838653564453, 0.0038957595825195312, 0.003896951675415039, 0.0038826465606689453, 0.0038988590240478516, 0.0038852691650390625, 0.0038471221923828125, 0.003863811492919922, 0.003883838653564453, 0.0038933753967285156, 0.0039975643157958984, 0.003925323486328125, 0.0038864612579345703, 0.003897428512573242, 0.0038759708404541016, 0.003842592239379883, 0.0038900375366210938, 0.0038955211639404297, 0.00390625, 0.003909587860107422, 0.003877878189086914, 0.0038979053497314453, 0.003853321075439453, 0.0040667057037353516, 0.0039048194885253906, 0.003972291946411133, 0.003988027572631836, 0.003891468048095703, 0.0038547515869140625, 0.003939390182495117, 0.0038771629333496094, 0.003907203674316406, 0.003900289535522461, 0.0038645267486572266, 0.003935337066650391, 0.003875732421875, 0.003933429718017578, 0.0038766860961914062, 0.0038406848907470703, 0.0039017200469970703, 0.003935098648071289, 0.003857851028442383, 0.003904104232788086, 0.0038530826568603516, 0.003872394561767578, 0.003877401351928711, 0.003917694091796875, 0.003893613815307617, 0.003849029541015625, 0.003884553909301758, 0.003981590270996094, 0.003906965255737305, 0.003899812698364258, 0.003941535949707031, 0.0040853023529052734, 0.003907203674316406, 0.003888845443725586, 0.0039026737213134766, 0.003931522369384766, 0.003882169723510742, 0.00389862060546875, 0.0039081573486328125, 0.003865480422973633, 0.0038907527923583984, 0.003909111022949219, 0.0039043426513671875, 0.003904104232788086, 0.00386810302734375, 0.0038814544677734375, 0.0038399696350097656, 0.0039520263671875, 0.0039103031158447266, 0.0038623809814453125, 0.003892183303833008, 0.0038869380950927734, 0.0038862228393554688, 0.003923177719116211, 0.0038466453552246094, 0.0038917064666748047, 0.0038623809814453125, 0.003879070281982422, 0.0039014816284179688, 0.003911495208740234, 0.0038950443267822266, 0.003995180130004883, 0.00393986701965332, 0.003933906555175781, 0.00399327278137207, 0.003935098648071289, 0.0038573741912841797, 0.003904581069946289, 0.0038917064666748047, 0.003890514373779297, 0.0039010047912597656, 0.003850698471069336, 0.0038852691650390625, 0.003887176513671875, 0.0039021968841552734, 0.003863096237182617, 0.00390934944152832, 0.003902912139892578, 0.003977298736572266, 0.0038907527923583984, 0.003847360610961914, 0.003890514373779297, 0.0039103031158447266, 0.003916025161743164, 0.003881692886352539, 0.003860950469970703, 0.0038902759552001953, 0.0038280487060546875, 0.003954648971557617, 0.0039021968841552734, 0.0038585662841796875, 0.003894805908203125, 0.0038573741912841797, 0.004128217697143555, 0.0039560794830322266, 0.003866434097290039, 0.003880739212036133, 0.0040585994720458984, 0.003946542739868164, 0.0039055347442626953, 0.003905773162841797, 0.0039010047912597656, 0.003840208053588867, 0.003874063491821289, 0.0038573741912841797, 0.0039463043212890625, 0.0038976669311523438, 0.003836393356323242, 0.003899812698364258, 0.0038492679595947266, 0.003911495208740234, 0.003932476043701172, 0.003853321075439453, 0.003916263580322266, 0.0038411617279052734, 0.003913164138793945, 0.003973245620727539, 0.003919124603271484, 0.003918886184692383, 0.003885030746459961, 0.003904581069946289, 0.003884553909301758, 0.0038862228393554688, 0.003895282745361328, 0.003916740417480469, 0.003882884979248047, 0.003859996795654297, 0.003920316696166992, 0.003862619400024414, 0.004024505615234375, 0.003911018371582031, 0.0038907527923583984, 0.0038955211639404297, 0.003877401351928711, 0.003908872604370117, 0.003957509994506836, 0.0038688182830810547, 0.003899097442626953, 0.0038580894470214844, 0.003904104232788086, 0.003956317901611328, 0.003862619400024414, 0.003932952880859375, 0.0038640499114990234, 0.0038900375366210938, 0.003900289535522461, 0.003930091857910156, 0.0038983821868896484, 0.00386810302734375, 0.003912210464477539, 0.003930568695068359, 0.0039017200469970703, 0.0038950443267822266, 0.003866434097290039, 0.0039031505584716797, 0.0039386749267578125, 0.0039501190185546875, 0.0039103031158447266, 0.003949642181396484, 0.003916501998901367, 0.003916501998901367, 0.003913164138793945, 0.0038590431213378906, 0.003912210464477539, 0.003932476043701172, 0.003920316696166992, 0.0038938522338867188, 0.004004716873168945, 0.003918647766113281, 0.003931999206542969, 0.003908395767211914, 0.0039026737213134766, 0.0038640499114990234, 0.003916740417480469, 0.003865480422973633, 0.0039484500885009766, 0.0039043426513671875, 0.0038535594940185547, 0.003947019577026367, 0.003866434097290039, 0.0039017200469970703, 0.0040280818939208984, 0.003887176513671875, 0.003904581069946289, 0.0038514137268066406, 0.003911018371582031, 0.00391077995300293, 0.00393986701965332, 0.003930330276489258, 0.0038568973541259766, 0.003927946090698242, 0.0038933753967285156, 0.003959178924560547, 0.003926277160644531, 0.0038628578186035156, 0.003920316696166992, 0.0039033889770507812, 0.0038864612579345703, 0.003904104232788086, 0.0038557052612304688, 0.003907680511474609, 0.003897428512573242, 0.003902435302734375, 0.0038573741912841797, 0.0039031505584716797, 0.003893613815307617, 0.0038976669311523438, 0.003928422927856445, 0.0038819313049316406, 0.003912448883056641, 0.003918886184692383, 0.0038940906524658203, 0.0038938522338867188, 0.0038640499114990234, 0.0039081573486328125, 0.003892183303833008, 0.003943681716918945, 0.003889322280883789, 0.004012584686279297, 0.0039234161376953125, 0.004000425338745117, 0.003896474838256836, 0.003882169723510742, 0.0039055347442626953, 0.0038955211639404297, 0.0038576126098632812, 0.003911733627319336, 0.003876209259033203, 0.003939151763916016, 0.0039179325103759766, 0.0038728713989257812, 0.0039010047912597656, 0.0039520263671875, 0.003957509994506836, 0.003902435302734375, 0.003934383392333984, 0.00391077995300293, 0.0038535594940185547, 0.00391697883605957, 0.003942966461181641, 0.003864765167236328, 0.003907442092895508, 0.0038738250732421875, 0.004052162170410156, 0.003908634185791016, 0.003918647766113281, 0.003923654556274414, 0.0038521289825439453, 0.00389862060546875, 0.0039713382720947266, 0.003920316696166992, 0.003916501998901367, 0.003844022750854492, 0.003919839859008789, 0.003923177719116211, 0.0039064884185791016, 0.00390934944152832, 0.0038576126098632812, 0.004380226135253906, 0.003959178924560547, 0.003952503204345703, 0.003935098648071289, 0.00393986701965332, 0.003908872604370117, 0.003918647766113281, 0.003943920135498047, 0.003871917724609375, 0.003945827484130859, 0.003912687301635742, 0.003981351852416992, 0.003952741622924805, 0.0038738250732421875, 0.003952503204345703, 0.003915309906005859, 0.0038869380950927734, 0.00392460823059082, 0.0038726329803466797, 0.003918170928955078, 0.003983259201049805, 0.003860950469970703, 0.003930807113647461, 0.003887176513671875, 0.0039026737213134766, 0.0040171146392822266, 0.003931522369384766, 0.003936052322387695, 0.003885984420776367, 0.0039331912994384766, 0.003931283950805664, 0.003914594650268555, 0.003914594650268555, 0.00388336181640625, 0.0039196014404296875, 0.003973484039306641, 0.003911733627319336, 0.003942251205444336, 0.0038747787475585938, 0.003904104232788086, 0.003920316696166992, 0.0039136409759521484, 0.003874540328979492, 0.003922939300537109, 0.00392460823059082, 0.003907442092895508, 0.003907203674316406, 0.0038928985595703125, 0.0038940906524658203, 0.003914833068847656, 0.0039196014404296875, 0.003899097442626953, 0.003895998001098633, 0.003896474838256836, 0.003868579864501953, 0.00396728515625, 0.0038995742797851562, 0.0038912296295166016, 0.003923892974853516, 0.0038781166076660156, 0.0040094852447509766, 0.00392460823059082, 0.0038976669311523438, 0.0039675235748291016, 0.0038874149322509766, 0.003928184509277344, 0.003950357437133789, 0.0038857460021972656, 0.003912687301635742, 0.003878355026245117, 0.003924131393432617, 0.0039217472076416016, 0.003936052322387695, 0.003913164138793945, 0.0038650035858154297, 0.003930568695068359, 0.004016876220703125, 0.003945827484130859, 0.003943681716918945, 0.0038771629333496094, 0.0039408206939697266, 0.0039179325103759766, 0.00391697883605957, 0.00391840934753418, 0.003891468048095703, 0.0039348602294921875, 0.00391077995300293, 0.0039212703704833984, 0.0038869380950927734, 0.003930330276489258, 0.0039365291595458984, 0.003930330276489258, 0.003922939300537109, 0.003916025161743164, 0.003961324691772461, 0.003935098648071289, 0.003911495208740234, 0.0038909912109375, 0.003906965255737305, 0.003895282745361328, 0.003929615020751953, 0.0039234161376953125, 0.0038716793060302734, 0.003911733627319336, 0.0039250850677490234, 0.003927469253540039, 0.003949642181396484, 0.0039052963256835938, 0.003915071487426758, 0.003922462463378906, 0.0039196014404296875, 0.003947019577026367, 0.0038836002349853516, 0.003920078277587891, 0.00399017333984375, 0.003927707672119141, 0.0038907527923583984, 0.003946542739868164, 0.0039217472076416016, 0.003939628601074219, 0.003918886184692383, 0.003907203674316406, 0.003910064697265625, 0.003922462463378906, 0.003911733627319336, 0.003924846649169922, 0.00392460823059082, 0.0040132999420166016, 0.003928184509277344, 0.003926753997802734, 0.003927946090698242, 0.0038945674896240234, 0.003941774368286133, 0.0038940906524658203, 0.003991365432739258, 0.003938198089599609, 0.0038836002349853516, 0.0039327144622802734, 0.003910064697265625, 0.003984928131103516, 0.003943204879760742, 0.0039157867431640625, 0.003945350646972656, 0.0039520263671875, 0.0038754940032958984, 0.0039501190185546875, 0.003926992416381836, 0.003926753997802734, 0.003922224044799805, 0.0038878917694091797, 0.003963947296142578, 0.00393223762512207, 0.0039501190185546875, 0.003931999206542969, 0.003905773162841797, 0.0039637088775634766, 0.003945112228393555, 0.00394892692565918, 0.00394129753112793, 0.0038983821868896484, 0.003934144973754883, 0.003916263580322266, 0.0039768218994140625, 0.003924369812011719, 0.0038919448852539062, 0.004034996032714844, 0.003890514373779297, 0.003989696502685547, 0.003919124603271484, 0.003903627395629883, 0.003940582275390625, 0.003892660140991211, 0.0039446353912353516, 0.003985166549682617, 0.003927707672119141, 0.003962039947509766, 0.003897428512573242, 0.00392913818359375, 0.00407862663269043, 0.003893136978149414, 0.003945827484130859, 0.003928422927856445, 0.0039327144622802734, 0.0039825439453125, 0.003909111022949219, 0.0039446353912353516, 0.003899812698364258, 0.003937482833862305, 0.003923654556274414, 0.003945827484130859, 0.003948211669921875, 0.003879070281982422, 0.003938436508178711, 0.003958463668823242, 0.003939151763916016, 0.003965616226196289, 0.0038993358612060547, 0.003949880599975586, 0.003930330276489258, 0.003920316696166992, 0.003949642181396484, 0.003878355026245117, 0.003945589065551758, 0.0039212703704833984, 0.003929615020751953, 0.003912925720214844, 0.0039784908294677734, 0.003939151763916016, 0.003925323486328125, 0.003921031951904297, 0.0038957595825195312, 0.00397944450378418, 0.00393986701965332, 0.003931522369384766, 0.003931283950805664, 0.003954648971557617, 0.003962516784667969, 0.003950595855712891, 0.003966093063354492, 0.0039293766021728516, 0.0039119720458984375, 0.003930330276489258, 0.0039522647857666016, 0.00394439697265625, 0.003919363021850586, 0.0038895606994628906, 0.0039331912994384766, 0.003902435302734375, 0.004081249237060547, 0.0039670467376708984, 0.003908872604370117, 0.003946542739868164, 0.0039484500885009766, 0.003979206085205078, 0.003926753997802734, 0.003904581069946289, 0.003924369812011719, 0.003895282745361328, 0.003928661346435547, 0.004015207290649414, 0.004059791564941406, 0.003947734832763672, 0.0039157867431640625, 0.003962278366088867, 0.003993511199951172, 0.003908634185791016, 0.003922224044799805, 0.0039196014404296875, 0.003935813903808594, 0.003950595855712891, 0.0039331912994384766, 0.0039556026458740234, 0.0039033889770507812, 0.004085063934326172, 0.003951311111450195, 0.004014253616333008, 0.003953218460083008, 0.003913164138793945, 0.003974437713623047, 0.0039517879486083984, 0.003959178924560547, 0.003941535949707031, 0.003908872604370117, 0.003940582275390625, 0.0039157867431640625, 0.003942251205444336, 0.004069089889526367, 0.003919124603271484, 0.0039386749267578125, 0.00397944450378418, 0.003975868225097656, 0.003960847854614258, 0.0039026737213134766, 0.0039484500885009766, 0.003945827484130859, 0.003947734832763672, 0.00397491455078125, 0.003932476043701172, 0.003955841064453125, 0.004021883010864258, 0.00397181510925293, 0.003916740417480469, 0.003968715667724609, 0.0041124820709228516, 0.003957986831665039, 0.0039408206939697266, 0.003924131393432617, 0.003972291946411133, 0.003965854644775391, 0.0039675235748291016, 0.0039708614349365234, 0.003907680511474609, 0.0039827823638916016, 0.003946781158447266, 0.003935575485229492, 0.003960609436035156, 0.003924131393432617, 0.0039484500885009766, 0.003952503204345703, 0.003953695297241211, 0.003947019577026367, 0.003922700881958008, 0.003958940505981445, 0.003952503204345703, 0.003954172134399414, 0.003954648971557617, 0.0039017200469970703, 0.003965854644775391, 0.0039746761322021484, 0.003961801528930664, 0.003953218460083008, 0.0039119720458984375, 0.00394749641418457, 0.0041272640228271484, 0.00398707389831543, 0.003962278366088867, 0.003951072692871094, 0.004077434539794922, 0.003911256790161133, 0.003980159759521484, 0.003953456878662109, 0.0041925907135009766, 0.0039670467376708984, 0.003932476043701172, 0.003998517990112305, 0.003957033157348633, 0.00391697883605957, 0.003938198089599609, 0.003910064697265625, 0.004084110260009766, 0.00395655632019043, 0.003925323486328125, 0.003975629806518555, 0.003933429718017578, 0.004004478454589844, 0.003946542739868164, 0.003957271575927734, 0.003953695297241211, 0.0038938522338867188, 0.003932952880859375, 0.004019498825073242, 0.003912448883056641, 0.003977060317993164, 0.0039331912994384766, 0.0039560794830322266, 0.004019737243652344, 0.0039212703704833984, 0.003957271575927734, 0.003956317901611328, 0.003962516784667969, 0.0040378570556640625, 0.003926753997802734, 0.003963947296142578, 0.0039038658142089844, 0.003954172134399414, 0.00395512580871582, 0.003966093063354492, 0.0039637088775634766, 0.003926753997802734, 0.00397038459777832, 0.003980875015258789, 0.003972291946411133, 0.003989696502685547, 0.003922462463378906, 0.0039520263671875, 0.003936767578125, 0.003973484039306641, 0.004072904586791992, 0.003941535949707031, 0.00396418571472168, 0.003958702087402344, 0.003974437713623047, 0.003951311111450195, 0.003968954086303711, 0.003948211669921875, 0.003947734832763672, 0.0039560794830322266, 0.003966808319091797, 0.0039119720458984375, 0.003971576690673828, 0.004067659378051758, 0.003947734832763672, 0.003979682922363281, 0.0039386749267578125, 0.0039632320404052734, 0.003979206085205078, 0.003968715667724609, 0.004300355911254883, 0.00393223762512207, 0.003972053527832031, 0.003949880599975586, 0.003933429718017578, 0.003972768783569336, 0.003911018371582031, 0.003938913345336914, 0.003955364227294922, 0.0039463043212890625, 0.003925800323486328, 0.003966093063354492, 0.003954887390136719, 0.00395655632019043, 0.003962278366088867, 0.003922462463378906, 0.003980398178100586, 0.003957033157348633, 0.00397801399230957, 0.004527091979980469, 0.003996133804321289, 0.003931283950805664, 0.00396275520324707, 0.003990650177001953, 0.003959178924560547, 0.003928422927856445, 0.003965616226196289, 0.0040280818939208984, 0.003987550735473633, 0.003957033157348633, 0.003940582275390625, 0.0041124820709228516, 0.003949642181396484, 0.003985881805419922, 0.0039730072021484375, 0.003939151763916016, 0.003974199295043945, 0.003964900970458984, 0.003975868225097656, 0.003978252410888672, 0.003931283950805664, 0.004027843475341797, 0.0039479732513427734, 0.00406646728515625, 0.003973722457885742, 0.003937244415283203, 0.0040056705474853516, 0.003966808319091797, 0.003966331481933594, 0.003961086273193359, 0.003973960876464844, 0.003972291946411133, 0.003962278366088867, 0.0039768218994140625, 0.003975391387939453, 0.00394129753112793, 0.003968238830566406, 0.004018306732177734, 0.003972053527832031, 0.003976583480834961, 0.003942966461181641, 0.003960847854614258, 0.00397944450378418, 0.0039708614349365234, 0.0039560794830322266, 0.0039424896240234375, 0.003969430923461914, 0.00398564338684082, 0.003983736038208008, 0.0039942264556884766, 0.0039730072021484375, 0.003956794738769531, 0.003931760787963867, 0.004771709442138672, 0.003987789154052734, 0.003946542739868164, 0.004003047943115234, 0.003984928131103516, 0.003977537155151367, 0.0039713382720947266, 0.003935813903808594, 0.0040743350982666016, 0.003996133804321289, 0.004026889801025391, 0.003972530364990234, 0.0039386749267578125, 0.003979206085205078, 0.003985404968261719, 0.003962993621826172, 0.003977298736572266, 0.003966808319091797, 0.003977537155151367, 0.003968715667724609, 0.0040225982666015625, 0.00397491455078125, 0.003946781158447266, 0.0039594173431396484, 0.004045009613037109, 0.003968715667724609, 0.003954410552978516, 0.003925323486328125, 0.003968238830566406, 0.003961801528930664, 0.003990888595581055, 0.004077434539794922, 0.0039653778076171875, 0.003962039947509766, 0.0039637088775634766, 0.0039730072021484375, 0.003976106643676758, 0.0039577484130859375, 0.0039653778076171875, 0.0039272308349609375, 0.004060029983520508, 0.003974199295043945, 0.003945350646972656, 0.003955841064453125, 0.003992319107055664, 0.004087686538696289, 0.003985881805419922, 0.003940105438232422, 0.003972530364990234, 0.003934383392333984, 0.0040283203125, 0.003961801528930664, 0.0039424896240234375, 0.003979682922363281, 0.003941059112548828, 0.004036664962768555, 0.003991127014160156, 0.003946781158447266, 0.0040547847747802734, 0.003931999206542969, 0.004053592681884766, 0.0039958953857421875, 0.003926992416381836, 0.003977060317993164, 0.003930330276489258, 0.0040378570556640625, 0.003972291946411133, 0.003951549530029297, 0.003960847854614258, 0.003939390182495117, 0.004114389419555664, 0.0040013790130615234, 0.003952503204345703, 0.0039806365966796875, 0.00393366813659668, 0.003977060317993164, 0.0040166378021240234, 0.003930330276489258, 0.00400853157043457, 0.003968238830566406, 0.0039937496185302734, 0.004033803939819336, 0.003965854644775391, 0.0039637088775634766, 0.003940105438232422, 0.003984928131103516, 0.004005908966064453, 0.003942966461181641, 0.00399470329284668, 0.00393366813659668, 0.004000425338745117, 0.004019737243652344, 0.003951072692871094, 0.003960132598876953, 0.003947734832763672, 0.003963947296142578, 0.0040280818939208984, 0.003986835479736328, 0.0039825439453125, 0.003949880599975586, 0.003981351852416992, 0.004006385803222656, 0.0039365291595458984, 0.003994941711425781, 0.0039331912994384766, 0.003983259201049805, 0.0039844512939453125, 0.003996372222900391, 0.003989219665527344, 0.004019021987915039, 0.003977537155151367, 0.003972291946411133, 0.003994464874267578, 0.0039997100830078125, 0.003933429718017578, 0.00400090217590332, 0.003980875015258789, 0.003996133804321289, 0.003972768783569336, 0.0039484500885009766, 0.0039751529693603516, 0.004046916961669922, 0.004011392593383789, 0.003957033157348633, 0.003939628601074219, 0.003974199295043945, 0.003965616226196289, 0.004006147384643555, 0.003983497619628906, 0.003948211669921875, 0.003984212875366211, 0.003985166549682617, 0.0040130615234375, 0.003973245620727539, 0.003947019577026367, 0.0045316219329833984, 0.004003286361694336, 0.004000663757324219, 0.003985166549682617, 0.0042493343353271484, 0.00400233268737793, 0.004049539566040039, 0.003947019577026367, 0.0039730072021484375, 0.003945827484130859, 0.003999948501586914, 0.00401759147644043, 0.003957509994506836, 0.003975391387939453, 0.003947734832763672, 0.003983974456787109, 0.003977060317993164, 0.003984928131103516, 0.004004716873168945, 0.003949403762817383, 0.003968238830566406, 0.0039844512939453125, 0.00400090217590332, 0.00397491455078125, 0.004034280776977539, 0.003991603851318359, 0.0039997100830078125, 0.004009246826171875, 0.003991127014160156, 0.003949165344238281, 0.003999948501586914, 0.003996849060058594, 0.003992319107055664, 0.0039920806884765625, 0.0039370059967041016, 0.003997325897216797, 0.004080295562744141, 0.004006862640380859, 0.003995180130004883, 0.003958940505981445, 0.003997325897216797, 0.003990650177001953, 0.0040435791015625, 0.0040435791015625, 0.003928184509277344, 0.003996610641479492, 0.00398707389831543, 0.004006624221801758, 0.004540443420410156, 0.004138946533203125, 0.004002094268798828, 0.0040683746337890625, 0.004545450210571289, 0.004095315933227539, 0.004015445709228516, 0.00404667854309082, 0.003983259201049805, 0.003960847854614258, 0.004011392593383789, 0.003958225250244141, 0.0040435791015625, 0.004027366638183594, 0.004038333892822266, 0.004022359848022461, 0.003950357437133789, 0.004061460494995117, 0.003984689712524414, 0.003964424133300781, 0.0039806365966796875, 0.003954648971557617, 0.00406956672668457, 0.003999948501586914, 0.003971099853515625, 0.0040645599365234375, 0.003997087478637695, 0.004061460494995117, 0.003998279571533203, 0.003948688507080078, 0.00397801399230957, 0.003957509994506836, 0.0040438175201416016, 0.003986835479736328, 0.00395512580871582, 0.004038810729980469, 0.0039615631103515625, 0.00412750244140625, 0.00399327278137207, 0.003961801528930664, 0.004000663757324219, 0.003970623016357422, 0.00404810905456543, 0.004000425338745117, 0.003954887390136719, 0.0040090084075927734, 0.003950357437133789, 0.004061460494995117, 0.003979206085205078, 0.004065036773681641, 0.003998994827270508, 0.003937244415283203, 0.0040628910064697266, 0.004004001617431641, 0.0039653778076171875, 0.003998994827270508, 0.003939628601074219, 0.004063844680786133, 0.003981828689575195, 0.003948211669921875, 0.003984212875366211, 0.0039670467376708984, 0.0040514469146728516, 0.004014730453491211, 0.003965854644775391, 0.004007101058959961, 0.003965854644775391, 0.00410151481628418, 0.003999948501586914, 0.003955364227294922, 0.00398564338684082, 0.003953695297241211, 0.004035234451293945, 0.003996849060058594, 0.003949165344238281, 0.004015684127807617, 0.004045724868774414, 0.004050493240356445, 0.003991842269897461, 0.0039844512939453125, 0.003979682922363281, 0.003965616226196289, 0.004061698913574219, 0.003975391387939453, 0.00396418571472168, 0.004023313522338867, 0.00395512580871582, 0.004108905792236328, 0.004016876220703125, 0.003968000411987305, 0.003988981246948242, 0.003996372222900391, 0.0040471553802490234, 0.004000186920166016, 0.003979921340942383, 0.004027843475341797, 0.003968000411987305, 0.004079103469848633, 0.004000186920166016, 0.004113435745239258, 0.004006385803222656, 0.004025697708129883, 0.004006862640380859, 0.003991603851318359, 0.003951549530029297, 0.0039861202239990234, 0.004015922546386719, 0.003999233245849609, 0.004004240036010742, 0.003956794738769531, 0.003992319107055664, 0.00403285026550293, 0.00419163703918457, 0.004004716873168945, 0.003969430923461914, 0.0040204524993896484, 0.004020214080810547, 0.0040090084075927734, 0.0040130615234375, 0.003969669342041016, 0.004004955291748047, 0.00405120849609375, 0.004015207290649414, 0.004013776779174805, 0.003966093063354492, 0.0040662288665771484, 0.004027605056762695, 0.004010915756225586, 0.004015445709228516, 0.003987789154052734, 0.004006624221801758, 0.0040323734283447266, 0.00401616096496582, 0.003997325897216797, 0.0039730072021484375, 0.004000186920166016, 0.004022836685180664, 0.004102230072021484, 0.0040171146392822266, 0.004000425338745117, 0.0039937496185302734, 0.004034757614135742, 0.00400853157043457, 0.00402069091796875, 0.003991842269897461, 0.0040283203125, 0.004030704498291016, 0.004020214080810547, 0.0040132999420166016, 0.003988504409790039, 0.004036903381347656, 0.004016399383544922, 0.004010200500488281, 0.0040094852447509766, 0.003958702087402344, 0.004018306732177734, 0.0039958953857421875, 0.00399327278137207, 0.003996849060058594, 0.003973484039306641, 0.004027128219604492, 0.003991127014160156, 0.0040149688720703125, 0.004005908966064453, 0.0039823055267333984, 0.004004955291748047, 0.003983736038208008, 0.004052162170410156, 0.004013776779174805, 0.003978252410888672, 0.0040514469146728516, 0.003996610641479492, 0.004008054733276367, 0.004006385803222656, 0.00398707389831543, 0.00409388542175293, 0.00400090217590332, 0.004029750823974609, 0.00413966178894043, 0.004038572311401367, 0.0040187835693359375, 0.0040094852447509766, 0.004011869430541992, 0.003999233245849609, 0.0039789676666259766, 0.004016399383544922, 0.004029512405395508, 0.004114866256713867, 0.004025697708129883, 0.004038572311401367, 0.003980875015258789, 0.004048824310302734, 0.004017353057861328, 0.004011392593383789, 0.004048585891723633, 0.00396728515625, 0.004015445709228516, 0.004017353057861328, 0.004006147384643555, 0.004031658172607422, 0.003978729248046875, 0.004063606262207031, 0.004025697708129883, 0.004006385803222656, 0.004045724868774414, 0.003978729248046875, 0.003989458084106445, 0.004026651382446289, 0.004004001617431641, 0.004031181335449219, 0.003990888595581055, 0.004010438919067383, 0.0040090084075927734, 0.004013776779174805, 0.004036664962768555, 0.0039730072021484375, 0.004032611846923828, 0.004003047943115234, 0.004019737243652344, 0.0040283203125, 0.003967761993408203, 0.004013538360595703, 0.0040073394775390625, 0.004013776779174805, 0.00404667854309082, 0.004046916961669922, 0.004038095474243164, 0.004044055938720703, 0.004022121429443359, 0.00402069091796875, 0.004033088684082031, 0.00402379035949707, 0.0040111541748046875, 0.00403594970703125, 0.004038810729980469, 0.0039806365966796875, 0.004048347473144531, 0.0041484832763671875, 0.004036664962768555, 0.004042625427246094, 0.004009246826171875, 0.004036664962768555, 0.004057168960571289, 0.004035472869873047, 0.004065990447998047, 0.003983974456787109, 0.004030466079711914, 0.0040836334228515625, 0.0039784908294677734, 0.004082679748535156, 0.003989696502685547, 0.00400853157043457, 0.004075288772583008, 0.003988504409790039, 0.004004955291748047, 0.0039768218994140625, 0.004027128219604492, 0.0040853023529052734, 0.0039942264556884766, 0.004034280776977539, 0.0039904117584228516, 0.004039287567138672, 0.0041370391845703125, 0.0039958953857421875, 0.004052400588989258, 0.003985166549682617, 0.0040242671966552734, 0.004086732864379883, 0.003988742828369141, 0.0040318965911865234, 0.0039789676666259766, 0.004051685333251953, 0.004058361053466797, 0.003985166549682617, 0.0040471553802490234, 0.004075765609741211, 0.004078388214111328, 0.004030466079711914, 0.0039784908294677734, 0.00405120849609375, 0.0041124820709228516, 0.004111766815185547, 0.0040476322174072266, 0.004015684127807617, 0.0040204524993896484, 0.004030704498291016, 0.0041272640228271484, 0.004058837890625, 0.0040094852447509766, 0.004048585891723633, 0.004032135009765625, 0.003985881805419922, 0.004019737243652344, 0.004042148590087891, 0.004027366638183594, 0.004030942916870117, 0.003980398178100586, 0.0040285587310791016, 0.0040323734283447266, 0.004029035568237305, 0.004039287567138672, 0.003988981246948242, 0.004052877426147461, 0.004029512405395508, 0.004057168960571289, 0.004023313522338867, 0.003980159759521484, 0.0040225982666015625, 0.0040209293365478516, 0.004037141799926758, 0.004039764404296875, 0.003986358642578125, 0.004035234451293945, 0.004038333892822266, 0.00403594970703125, 0.004076242446899414, 0.003980159759521484, 0.004040241241455078, 0.004019260406494141, 0.004046916961669922, 0.00402069091796875, 0.003979682922363281, 0.004032611846923828, 0.00403141975402832, 0.00408625602722168, 0.004129648208618164, 0.003987550735473633, 0.004021167755126953, 0.004026174545288086, 0.004057407379150391, 0.004034519195556641, 0.004023551940917969, 0.004014015197753906, 0.004072904586791992, 0.004004001617431641, 0.004042863845825195, 0.004086017608642578, 0.00403594970703125, 0.00407862663269043, 0.003997087478637695, 0.004091501235961914, 0.003990888595581055, 0.0040204524993896484, 0.004075050354003906, 0.003999233245849609, 0.0040514469146728516, 0.003988742828369141, 0.0042798519134521484, 0.004033803939819336, 0.004000425338745117, 0.004018545150756836, 0.00399327278137207, 0.004065513610839844, 0.004030466079711914, 0.003996372222900391, 0.004035472869873047, 0.0039882659912109375, 0.004076957702636719, 0.00403285026550293, 0.003995656967163086, 0.004034757614135742, 0.003991603851318359, 0.0040740966796875, 0.004058361053466797, 0.004001140594482422, 0.004050731658935547, 0.004046916961669922, 0.004030466079711914, 0.004028797149658203, 0.0040132999420166016, 0.004071950912475586, 0.0040433406829833984, 0.0040361881256103516, 0.0040285587310791016, 0.004096269607543945, 0.004042863845825195, 0.004060983657836914, 0.004057645797729492, 0.004054546356201172, 0.004006385803222656, 0.004039287567138672, 0.004054546356201172, 0.004069328308105469, 0.004061460494995117, 0.00399327278137207, 0.004134416580200195, 0.004047393798828125, 0.004038572311401367, 0.004051923751831055, 0.004008769989013672, 0.0040740966796875, 0.0040628910064697266, 0.0040700435638427734, 0.004055976867675781, 0.0040705204010009766, 0.0040171146392822266, 0.004034757614135742, 0.004072666168212891, 0.0040416717529296875, 0.004068136215209961, 0.004016399383544922, 0.004034757614135742, 0.004045724868774414, 0.004059553146362305, 0.004045963287353516, 0.003988504409790039, 0.004042387008666992, 0.004057884216308594, 0.00404810905456543, 0.0040435791015625, 0.00399327278137207, 0.004044294357299805, 0.004040956497192383, 0.004034996032714844, 0.004059791564941406, 0.004047393798828125, 0.004038095474243164, 0.0040285587310791016, 0.004075765609741211, 0.004037618637084961, 0.004057407379150391, 0.004044771194458008, 0.004130125045776367, 0.004029750823974609, 0.004128217697143555, 0.0040166378021240234, 0.004042148590087891, 0.004073619842529297, 0.003983259201049805, 0.004057645797729492, 0.004034519195556641, 0.004040956497192383, 0.004108428955078125, 0.004052400588989258, 0.004046440124511719, 0.0041277408599853516, 0.004068613052368164, 0.004075527191162109, 0.004025459289550781, 0.00405120849609375, 0.004035472869873047, 0.004050731658935547, 0.004055976867675781, 0.003981590270996094, 0.004051923751831055, 0.004033803939819336, 0.0040874481201171875, 0.004261016845703125, 0.004242897033691406, 0.004256248474121094, 0.004151105880737305, 0.00408935546875, 0.004145383834838867, 0.004098415374755859, 0.004163503646850586, 0.004127025604248047, 0.004100799560546875, 0.004149675369262695, 0.0041501522064208984, 0.004136800765991211, 0.004290342330932617, 0.0041086673736572266, 0.004184722900390625, 0.004142284393310547, 0.004149198532104492, 0.0041615962982177734, 0.004155874252319336, 0.004102468490600586, 0.004134178161621094, 0.004141569137573242, 0.00413966178894043, 0.004162311553955078, 0.0041043758392333984, 0.0042934417724609375, 0.004201173782348633, 0.004141330718994141, 0.004153251647949219, 0.004121065139770508, 0.004183053970336914, 0.0042171478271484375, 0.004105806350708008, 0.004137516021728516, 0.00417327880859375, 0.004240512847900391, 0.00416254997253418, 0.004097461700439453, 0.004167795181274414, 0.0041484832763671875, 0.004149198532104492, 0.004161357879638672, 0.00426936149597168, 0.004170656204223633, 0.004167318344116211, 0.004152774810791016, 0.004153251647949219, 0.004368782043457031, 0.004190683364868164, 0.004141807556152344, 0.004179716110229492, 0.004151582717895508, 0.004113674163818359, 0.004182100296020508, 0.004204988479614258, 0.004166603088378906, 0.004141330718994141, 0.0041120052337646484, 0.00416254997253418, 0.0041544437408447266, 0.0042057037353515625, 0.004180908203125, 0.004167795181274414, 0.004120588302612305, 0.004157066345214844, 0.0041351318359375, 0.004190921783447266, 0.0042078495025634766, 0.004136085510253906, 0.004156827926635742, 0.004235267639160156, 0.004189729690551758, 0.004185199737548828, 0.004221200942993164, 0.004238605499267578, 0.004147529602050781, 0.0040988922119140625, 0.004170417785644531, 0.00417637825012207, 0.004163265228271484, 0.004137754440307617, 0.004120588302612305, 0.00420689582824707, 0.004223346710205078, 0.004149913787841797, 0.004144191741943359, 0.004185676574707031, 0.004122495651245117, 0.004133939743041992, 0.004161834716796875, 0.004167318344116211, 0.0041387081146240234, 0.004117488861083984, 0.004137754440307617, 0.0042078495025634766, 0.0041086673736572266, 0.004152059555053711, 0.00420832633972168, 0.004213809967041016, 0.0041506290435791016, 0.004092216491699219, 0.004162788391113281, 0.004155635833740234, 0.004181861877441406, 0.004181385040283203, 0.004115581512451172, 0.00415349006652832, 0.004181385040283203, 0.004181623458862305, 0.004153728485107422, 0.004189014434814453, 0.0041697025299072266, 0.004172563552856445, 0.004146575927734375, 0.004183769226074219, 0.004184246063232422, 0.004117727279663086, 0.004148244857788086, 0.004185914993286133, 0.004118680953979492, 0.004195690155029297, 0.004248857498168945, 0.004221439361572266, 0.004155635833740234, 0.004083156585693359, 0.0041544437408447266, 0.0041811466217041016, 0.0041599273681640625, 0.004159450531005859, 0.004119396209716797, 0.004191160202026367, 0.004224300384521484, 0.004166364669799805, 0.004167079925537109, 0.004167795181274414, 0.004171848297119141, 0.004166841506958008, 0.004167318344116211, 0.004218339920043945, 0.004162788391113281, 0.004123210906982422, 0.004183769226074219, 0.004358053207397461, 0.004120349884033203, 0.004189014434814453, 0.00410008430480957, 0.004204511642456055, 0.004153251647949219, 0.004125118255615234, 0.0041582584381103516, 0.0041768550872802734, 0.004163265228271484, 0.0041768550872802734, 0.00410151481628418, 0.0042607784271240234, 0.004266262054443359, 0.004163265228271484, 0.004217386245727539, 0.004170656204223633, 0.004122257232666016, 0.004141807556152344, 0.004154682159423828, 0.004174709320068359, 0.004158496856689453, 0.004115581512451172, 0.00414276123046875, 0.0041790008544921875, 0.004125833511352539, 0.0042002201080322266, 0.004244327545166016, 0.0041997432708740234, 0.004160642623901367, 0.004108428955078125, 0.004183769226074219, 0.004159688949584961, 0.004149913787841797, 0.004173755645751953, 0.0041201114654541016, 0.004151821136474609, 0.0042231082916259766, 0.004177570343017578, 0.004188060760498047, 0.004197835922241211, 0.004121065139770508, 0.004191398620605469, 0.004177570343017578, 0.004218101501464844, 0.004153728485107422, 0.00412750244140625, 0.0041370391845703125, 0.004227161407470703, 0.004145383834838867, 0.00416111946105957, 0.004152059555053711, 0.004235744476318359, 0.004178047180175781, 0.0041582584381103516, 0.004200458526611328, 0.004177093505859375, 0.004189014434814453, 0.004151344299316406, 0.004180908203125, 0.004202127456665039, 0.004168510437011719, 0.004164457321166992, 0.004187822341918945, 0.004177093505859375, 0.0041370391845703125, 0.004156351089477539, 0.0042324066162109375, 0.004147768020629883, 0.004158735275268555, 0.004133701324462891, 0.004228830337524414, 0.00419306755065918, 0.00416254997253418, 0.0042345523834228516, 0.004292964935302734, 0.004215717315673828, 0.004196643829345703, 0.004136562347412109, 0.0042073726654052734, 0.00414729118347168, 0.004159450531005859, 0.004176139831542969, 0.00420832633972168, 0.004120826721191406, 0.004241228103637695, 0.004155635833740234, 0.004185914993286133, 0.004209280014038086, 0.0041675567626953125, 0.004118680953979492, 0.004167795181274414, 0.004172086715698242, 0.004158496856689453, 0.004172801971435547, 0.004178285598754883, 0.00414729118347168, 0.0041811466217041016, 0.004219770431518555, 0.0043184757232666016, 0.004182100296020508, 0.004126310348510742, 0.004191398620605469, 0.00419306755065918, 0.004202604293823242, 0.004164457321166992, 0.004163503646850586, 0.004261493682861328, 0.004224538803100586, 0.004145622253417969, 0.004216432571411133, 0.004197359085083008, 0.004193544387817383, 0.004202365875244141, 0.004201173782348633, 0.004148960113525391, 0.0041697025299072266, 0.0042040348052978516, 0.004208803176879883, 0.004157066345214844, 0.004134416580200195, 0.004178285598754883, 0.004246234893798828, 0.0042188167572021484, 0.004166364669799805, 0.004122734069824219, 0.004292011260986328, 0.00424957275390625, 0.0041315555572509766, 0.0041828155517578125, 0.004238605499267578, 0.004163980484008789, 0.004227638244628906, 0.004229545593261719, 0.004174947738647461, 0.004207134246826172, 0.004198789596557617, 0.004189252853393555, 0.004186391830444336, 0.004153728485107422, 0.0041654109954833984, 0.004225254058837891, 0.0041656494140625, 0.004183769226074219, 0.004140615463256836, 0.0042781829833984375, 0.0041942596435546875, 0.004137516021728516, 0.004181623458862305, 0.004201173782348633, 0.004174709320068359, 0.004179239273071289, 0.004149436950683594, 0.004203319549560547, 0.004171848297119141, 0.004168033599853516, 0.004152774810791016, 0.004192352294921875, 0.004112720489501953, 0.004173755645751953, 0.0041806697845458984, 0.004228353500366211, 0.004181861877441406, 0.0041391849517822266, 0.004230499267578125, 0.004227638244628906, 0.004147052764892578, 0.004191398620605469, 0.004141807556152344, 0.004248142242431641, 0.004258632659912109, 0.004172086715698242, 0.004179954528808594, 0.00418543815612793, 0.004171848297119141, 0.004191160202026367, 0.004143238067626953, 0.004209280014038086, 0.004166126251220703, 0.004231691360473633, 0.00429844856262207, 0.004258871078491211, 0.0041692256927490234, 0.004209041595458984, 0.004251956939697266, 0.004159212112426758, 0.004178762435913086, 0.004216909408569336, 0.004245281219482422, 0.004219770431518555, 0.004135608673095703, 0.0041849613189697266, 0.004273414611816406, 0.004307746887207031, 0.00427699089050293, 0.004206657409667969, 0.004235506057739258, 0.0042035579681396484, 0.004206657409667969, 0.004239797592163086, 0.004182577133178711, 0.004251003265380859, 0.0041468143463134766, 0.004201650619506836, 0.00419306755065918, 0.004177093505859375, 0.004212856292724609, 0.004250049591064453, 0.004148721694946289, 0.004194974899291992, 0.004167318344116211, 0.004197359085083008, 0.004209280014038086, 0.004151105880737305, 0.004194021224975586, 0.004242897033691406, 0.0041577816009521484, 0.004186391830444336, 0.004352092742919922, 0.004207134246826172, 0.004179239273071289, 0.004141807556152344, 0.004225015640258789, 0.004200458526611328, 0.004170656204223633, 0.004179716110229492, 0.004177570343017578, 0.004160881042480469, 0.004221916198730469, 0.004172801971435547, 0.004221916198730469, 0.004171609878540039, 0.004232883453369141, 0.004162788391113281, 0.004189491271972656, 0.00419306755065918, 0.0041790008544921875, 0.004176139831542969, 0.004189014434814453, 0.0043315887451171875, 0.004187822341918945, 0.0041925907135009766, 0.0042514801025390625, 0.004330158233642578, 0.004198312759399414, 0.004273891448974609, 0.004183053970336914, 0.0041620731353759766, 0.004203319549560547, 0.004218578338623047, 0.004197359085083008, 0.004192352294921875, 0.0044438838958740234, 0.004263639450073242, 0.004239559173583984, 0.004204988479614258, 0.004204988479614258, 0.004221439361572266, 0.004158973693847656, 0.004218578338623047, 0.004254579544067383, 0.0041582584381103516, 0.004209280014038086, 0.004149913787841797, 0.004283905029296875, 0.004193544387817383, 0.004350185394287109, 0.004215240478515625, 0.004271984100341797, 0.0042188167572021484, 0.004190921783447266, 0.004175901412963867, 0.004204511642456055, 0.004289150238037109, 0.0042266845703125, 0.00431060791015625, 0.0042874813079833984, 0.00418400764465332, 0.0042209625244140625, 0.004266977310180664, 0.004152774810791016, 0.004205465316772461, 0.004216670989990234, 0.004243135452270508, 0.004209756851196289, 0.004180192947387695, 0.004204273223876953, 0.004191160202026367, 0.004185914993286133, 0.004191875457763672, 0.0042192935943603516, 0.004188060760498047, 0.004201173782348633, 0.004192352294921875, 0.0042018890380859375, 0.004195451736450195, 0.0042231082916259766, 0.0042226314544677734, 0.004254341125488281, 0.004192352294921875, 0.0041921138763427734, 0.004182577133178711, 0.004284858703613281, 0.004202365875244141, 0.004168987274169922, 0.0042269229888916016, 0.004232645034790039, 0.004205942153930664, 0.004180192947387695, 0.004242658615112305, 0.0041882991790771484, 0.0042073726654052734, 0.004202365875244141, 0.004311323165893555, 0.0042111873626708984, 0.0041751861572265625, 0.004210948944091797, 0.004263401031494141, 0.004155158996582031, 0.004203081130981445, 0.0041539669036865234, 0.004256725311279297, 0.004420757293701172, 0.004293680191040039, 0.004248857498168945, 0.0042417049407958984, 0.0042629241943359375, 0.004207611083984375, 0.00427556037902832, 0.00421142578125, 0.004194498062133789, 0.0041806697845458984, 0.004202127456665039, 0.0042095184326171875, 0.004202842712402344, 0.004229307174682617, 0.004193305969238281, 0.004187107086181641, 0.004221677780151367, 0.0042383670806884766, 0.004196882247924805, 0.0042226314544677734, 0.004186153411865234, 0.004232168197631836, 0.004210710525512695, 0.0041849613189697266, 0.00419926643371582, 0.004241466522216797, 0.004209995269775391, 0.004252195358276367, 0.004198312759399414, 0.004193544387817383, 0.004209995269775391, 0.004204511642456055, 0.004221200942993164, 0.004201173782348633, 0.004177093505859375, 0.004233121871948242, 0.004279375076293945, 0.00415349006652832, 0.004208087921142578, 0.004258632659912109, 0.004281044006347656, 0.004208564758300781, 0.004161834716796875, 0.0042111873626708984, 0.0041997432708740234, 0.004188060760498047, 0.004185676574707031, 0.004261493682861328, 0.004294395446777344, 0.00422358512878418, 0.004165172576904297, 0.004248142242431641, 0.004209756851196289, 0.004177093505859375, 0.004296302795410156, 0.004599094390869141, 0.0042552947998046875, 0.004216194152832031, 0.004204988479614258, 0.0041561126708984375, 0.004213571548461914, 0.0042285919189453125, 0.0042171478271484375, 0.004279613494873047, 0.0042226314544677734, 0.004221439361572266, 0.0042266845703125, 0.0041658878326416016, 0.0041887760162353516, 0.004184722900390625, 0.004245758056640625, 0.004215717315673828, 0.004178047180175781, 0.005193948745727539, 0.004509925842285156, 0.00426793098449707, 0.004633903503417969, 0.0042650699615478516, 0.0043201446533203125, 0.004262447357177734, 0.004247426986694336, 0.004196643829345703, 0.004290103912353516, 0.0042302608489990234, 0.004153728485107422, 0.004239082336425781, 0.004319429397583008, 0.004228353500366211, 0.004209756851196289, 0.004194498062133789, 0.00422978401184082, 0.0042192935943603516, 0.004227638244628906, 0.004218578338623047, 0.004277706146240234, 0.004271507263183594, 0.004240512847900391, 0.004282236099243164, 0.004244327545166016, 0.004224538803100586, 0.004182100296020508, 0.0042459964752197266, 0.004203081130981445, 0.0041806697845458984, 0.004187107086181641, 0.004301786422729492, 0.004249095916748047, 0.004222393035888672, 0.004243135452270508, 0.004193544387817383, 0.004239320755004883, 0.004208087921142578, 0.004251718521118164, 0.004227876663208008, 0.004181623458862305, 0.004208564758300781, 0.004251003265380859, 0.004233121871948242, 0.00422358512878418, 0.004189491271972656, 0.004289865493774414, 0.0042455196380615234, 0.004171609878540039, 0.004227161407470703, 0.004229545593261719, 0.0042209625244140625, 0.004233598709106445, 0.0042247772216796875, 0.004194974899291992, 0.004206418991088867, 0.00423884391784668, 0.004200935363769531, 0.004216194152832031, 0.004262685775756836, 0.004209995269775391, 0.004277944564819336, 0.00425267219543457, 0.004233360290527344, 0.004242897033691406, 0.004213571548461914, 0.004276275634765625, 0.004172325134277344, 0.004330873489379883, 0.004228353500366211, 0.0042209625244140625, 0.004266262054443359, 0.004233598709106445, 0.00424504280090332, 0.0042362213134765625, 0.004284381866455078, 0.004191398620605469, 0.004232168197631836, 0.004199504852294922, 0.004361391067504883, 0.004235744476318359, 0.004197359085083008, 0.004216194152832031, 0.004284381866455078, 0.004221916198730469, 0.004235267639160156, 0.004241943359375, 0.004304170608520508, 0.0042302608489990234, 0.004210948944091797, 0.0042171478271484375, 0.0043222904205322266, 0.0043070316314697266, 0.0042612552642822266, 0.004248142242431641, 0.004158735275268555, 0.004225969314575195, 0.004271507263183594, 0.004221677780151367, 0.004225730895996094, 0.004174709320068359, 0.0042798519134521484, 0.004224061965942383, 0.004196882247924805, 0.004241943359375, 0.004217386245727539, 0.004376888275146484, 0.004237174987792969, 0.004232168197631836, 0.0041730403900146484, 0.0042111873626708984, 0.004206657409667969, 0.004269599914550781, 0.00423884391784668, 0.004179239273071289, 0.004222869873046875, 0.004350185394287109, 0.004164457321166992, 0.004245281219482422, 0.004284381866455078, 0.0041887760162353516, 0.004237651824951172, 0.004171609878540039, 0.004270315170288086, 0.004254817962646484, 0.004201650619506836, 0.004250526428222656, 0.0042493343353271484, 0.004251003265380859, 0.0042455196380615234, 0.004178762435913086, 0.004250764846801758, 0.004244327545166016, 0.004242420196533203, 0.0042455196380615234, 0.00424957275390625, 0.0042035579681396484, 0.004250288009643555, 0.004281044006347656, 0.004194498062133789, 0.004229545593261719, 0.004199504852294922, 0.004291057586669922, 0.004242658615112305, 0.004207134246826172, 0.00426030158996582, 0.004221439361572266, 0.004231929779052734, 0.004223823547363281, 0.004320859909057617, 0.004269599914550781, 0.00426030158996582, 0.004256486892700195, 0.004372358322143555, 0.0042514801025390625, 0.00417780876159668, 0.0042629241943359375, 0.0042877197265625, 0.004209756851196289, 0.004216432571411133, 0.004438877105712891, 0.004280567169189453, 0.004265785217285156, 0.004410266876220703, 0.004240512847900391, 0.0042552947998046875, 0.004237174987792969, 0.00427556037902832, 0.004252910614013672, 0.004217624664306641, 0.00429844856262207, 0.004292011260986328, 0.00423121452331543, 0.0042493343353271484, 0.004261016845703125, 0.0042781829833984375, 0.004826784133911133, 0.004339933395385742, 0.004265785217285156, 0.004271984100341797, 0.004768848419189453, 0.0042231082916259766, 0.0042629241943359375, 0.004243135452270508, 0.004241228103637695, 0.004238605499267578, 0.004204750061035156, 0.004270076751708984, 0.0043375492095947266, 0.0042400360107421875, 0.0042629241943359375, 0.004304409027099609, 0.004193782806396484, 0.0042612552642822266, 0.004317760467529297, 0.0042095184326171875, 0.004258632659912109, 0.0042421817779541016, 0.004315853118896484, 0.004242420196533203, 0.0043451786041259766, 0.004283428192138672, 0.004221439361572266, 0.004249095916748047, 0.004332304000854492, 0.004278898239135742, 0.00420689582824707, 0.0042645931243896484, 0.00432276725769043, 0.004321575164794922, 0.004283905029296875, 0.004195213317871094, 0.004379987716674805, 0.004276275634765625, 0.004212141036987305, 0.0042531490325927734, 0.004340410232543945, 0.004305124282836914, 0.004247426986694336, 0.004294633865356445, 0.004189014434814453, 0.005023479461669922, 0.004320621490478516, 0.004258871078491211, 0.004263639450073242, 0.004864931106567383, 0.004370212554931641, 0.004317760467529297, 0.0043294429779052734, 0.004281759262084961, 0.0042154788970947266, 0.004266023635864258, 0.004274606704711914, 0.004269599914550781, 0.004258155822753906, 0.0042552947998046875, 0.0042264461517333984, 0.0042264461517333984, 0.004270315170288086, 0.0042819976806640625, 0.004250764846801758, 0.004213094711303711, 0.00432276725769043, 0.004275083541870117, 0.004195690155029297, 0.004286527633666992, 0.004290580749511719, 0.00434422492980957, 0.004239320755004883, 0.00419926643371582, 0.004350423812866211, 0.004303455352783203, 0.004297733306884766, 0.004263162612915039, 0.0042552947998046875, 0.0042574405670166016, 0.004312038421630859, 0.0042645931243896484, 0.004235029220581055, 0.004248857498168945, 0.004273653030395508, 0.004265546798706055, 0.0042879581451416016, 0.004232645034790039, 0.004317045211791992, 0.004189014434814453, 0.0042536258697509766, 0.00425410270690918, 0.0042572021484375, 0.004257678985595703, 0.004214048385620117, 0.004270076751708984, 0.004341840744018555, 0.004296541213989258, 0.004323244094848633, 0.0043027400970458984, 0.004231929779052734, 0.00428462028503418, 0.004361391067504883, 0.004278898239135742, 0.004339456558227539, 0.004271268844604492, 0.004239797592163086, 0.0042743682861328125, 0.004227638244628906, 0.004265308380126953, 0.004273414611816406, 0.004261970520019531, 0.0042629241943359375, 0.004349470138549805, 0.004230499267578125, 0.004295825958251953, 0.0044367313385009766, 0.0043103694915771484, 0.004395246505737305, 0.004278421401977539, 0.004255771636962891, 0.0042705535888671875, 0.004301786422729492, 0.004267454147338867, 0.004266500473022461, 0.00429224967956543, 0.0043370723724365234, 0.004302501678466797, 0.004258871078491211, 0.004268646240234375, 0.004323005676269531, 0.0042951107025146484, 0.004271268844604492, 0.0042722225189208984, 0.004250288009643555, 0.004285097122192383, 0.004259824752807617, 0.004419803619384766, 0.004283428192138672, 0.004287004470825195, 0.004315614700317383, 0.0042705535888671875, 0.004271030426025391, 0.00431513786315918, 0.004339694976806641, 0.004290342330932617, 0.0042531490325927734, 0.004290580749511719, 0.004247903823852539, 0.0043451786041259766, 0.0042476654052734375, 0.004311323165893555, 0.004286527633666992, 0.0042536258697509766, 0.004309415817260742, 0.004269838333129883, 0.0042226314544677734, 0.0042803287506103516, 0.00430607795715332, 0.004242420196533203, 0.004307270050048828, 0.004770755767822266, 0.0043103694915771484, 0.004363298416137695, 0.004376649856567383, 0.0043659210205078125, 0.004255771636962891, 0.004351615905761719, 0.004259824752807617, 0.004270315170288086, 0.004259586334228516, 0.004320383071899414, 0.004295825958251953, 0.004321098327636719, 0.004314422607421875, 0.004349708557128906, 0.004294872283935547, 0.004290580749511719, 0.0043163299560546875, 0.004231929779052734, 0.004254579544067383, 0.004323244094848633, 0.004239559173583984, 0.004289388656616211, 0.00425410270690918, 0.004320859909057617, 0.004266500473022461, 0.00422978401184082, 0.00430607795715332, 0.0043010711669921875, 0.004302978515625, 0.004302978515625, 0.0043027400970458984, 0.004263877868652344, 0.004271745681762695, 0.004320859909057617, 0.0042264461517333984, 0.004296064376831055, 0.004292488098144531, 0.004335880279541016, 0.004274129867553711, 0.004248619079589844, 0.004297494888305664, 0.004268646240234375, 0.00427699089050293, 0.004289388656616211, 0.004313468933105469, 0.0042629241943359375, 0.004339933395385742, 0.004360675811767578, 0.004299163818359375, 0.004408836364746094, 0.004265546798706055, 0.004336833953857422, 0.0042574405670166016, 0.004227161407470703, 0.004288434982299805, 0.004306793212890625, 0.0043528079986572266, 0.004434823989868164, 0.004315376281738281, 0.004243373870849609, 0.004255533218383789, 0.004309415817260742, 0.00421595573425293, 0.004278421401977539, 0.0042192935943603516, 0.004366159439086914, 0.004303693771362305, 0.0042383670806884766, 0.00429844856262207, 0.004263162612915039, 0.004297494888305664, 0.004359722137451172, 0.00431513786315918, 0.004235744476318359, 0.004375934600830078, 0.004384279251098633, 0.004247426986694336, 0.0042803287506103516, 0.004309177398681641, 0.0042724609375, 0.0042934417724609375, 0.0042324066162109375, 0.004307746887207031, 0.004309415817260742, 0.004296302795410156, 0.004277467727661133, 0.004311084747314453, 0.004246711730957031, 0.004294157028198242, 0.004324674606323242, 0.004306793212890625, 0.004317283630371094, 0.004300355911254883, 0.004309892654418945, 0.004282474517822266, 0.004301786422729492, 0.004343271255493164, 0.004274606704711914, 0.004279375076293945, 0.004296779632568359, 0.0042994022369384766, 0.0042362213134765625, 0.004296541213989258, 0.004335880279541016, 0.004399538040161133, 0.004302978515625, 0.004332780838012695, 0.004294395446777344, 0.004311323165893555, 0.004297733306884766, 0.00432586669921875, 0.00434112548828125, 0.00426173210144043, 0.004297733306884766, 0.004273653030395508, 0.004228830337524414, 0.004342794418334961, 0.004297971725463867, 0.004247188568115234, 0.004279136657714844, 0.004351139068603516, 0.004244804382324219, 0.004273891448974609, 0.00429844856262207, 0.004275321960449219, 0.0043065547943115234, 0.0042572021484375, 0.004302501678466797, 0.004547119140625, 0.004349470138549805, 0.004304409027099609, 0.00429081916809082, 0.004251956939697266, 0.004350900650024414, 0.0043027400970458984, 0.004250526428222656, 0.0042607784271240234, 0.004311561584472656, 0.004332780838012695, 0.0042841434478759766, 0.004296064376831055, 0.004244804382324219, 0.004431962966918945, 0.004312276840209961, 0.004328250885009766, 0.004284858703613281, 0.0042724609375, 0.00435638427734375, 0.00429081916809082, 0.0042705535888671875, 0.004431962966918945, 0.004317522048950195, 0.004293680191040039, 0.004286766052246094, 0.004343509674072266, 0.004273891448974609, 0.004323720932006836, 0.004381418228149414, 0.004300832748413086, 0.004302501678466797, 0.004236698150634766, 0.00440669059753418, 0.004299163818359375, 0.004269123077392578, 0.004313230514526367, 0.004314422607421875, 0.004272937774658203, 0.004332065582275391, 0.0042896270751953125, 0.004269599914550781, 0.004315853118896484, 0.004351615905761719, 0.0042307376861572266, 0.004294395446777344, 0.0042803287506103516, 0.004292726516723633, 0.004296779632568359, 0.004272937774658203, 0.0043201446533203125, 0.0042989253997802734, 0.0043218135833740234, 0.004312992095947266, 0.004297733306884766, 0.004240512847900391, 0.004263401031494141, 0.0043985843658447266, 0.004261970520019531, 0.0042917728424072266, 0.0043828487396240234, 0.0043413639068603516, 0.00430750846862793, 0.00424504280090332, 0.004326343536376953, 0.004302024841308594, 0.00428318977355957, 0.004307985305786133, 0.004305124282836914, 0.004338979721069336, 0.004378080368041992, 0.004339933395385742, 0.004262685775756836, 0.004366874694824219, 0.004395961761474609, 0.004388093948364258, 0.0043087005615234375, 0.004300832748413086, 0.004286766052246094, 0.004327535629272461, 0.004526615142822266, 0.004309177398681641, 0.004336118698120117, 0.0042629241943359375, 0.004426002502441406, 0.004317760467529297, 0.004265308380126953, 0.004335641860961914, 0.00432586669921875, 0.00440669059753418, 0.004343986511230469, 0.00440216064453125, 0.004319667816162109, 0.004623889923095703, 0.004346132278442383, 0.004324913024902344, 0.0043108463287353516, 0.0043659210205078125, 0.0043065547943115234, 0.004319667816162109, 0.004380702972412109, 0.004369974136352539, 0.004329681396484375, 0.004384279251098633, 0.004276275634765625, 0.0042798519134521484, 0.004317045211791992, 0.004443168640136719, 0.004319429397583008, 0.004269838333129883, 0.004398822784423828, 0.00432896614074707, 0.0042667388916015625, 0.004350423812866211, 0.004319190979003906, 0.00444793701171875, 0.004374980926513672, 0.004336833953857422, 0.004369258880615234, 0.004575967788696289, 0.0043299198150634766, 0.004293918609619141, 0.004287004470825195, 0.004354238510131836, 0.0042858123779296875, 0.004305124282836914, 0.004328727722167969, 0.004311800003051758, 0.0043070316314697266, 0.004279136657714844, 0.00430750846862793, 0.0042972564697265625, 0.004351139068603516, 0.004315853118896484, 0.004317283630371094, 0.004288196563720703, 0.0043184757232666016, 0.004317522048950195, 0.00426936149597168, 0.004347324371337891, 0.004349231719970703, 0.0043299198150634766, 0.0043125152587890625, 0.004317760467529297, 0.004266500473022461, 0.004347324371337891, 0.00461888313293457, 0.004320383071899414, 0.004380941390991211, 0.00431060791015625, 0.004357099533081055, 0.004339933395385742, 0.004263162612915039, 0.00444340705871582, 0.004304170608520508, 0.004404306411743164, 0.004349946975708008, 0.004364490509033203, 0.004279136657714844, 0.004332542419433594, 0.004381656646728516, 0.004355669021606445, 0.004321575164794922, 0.0043370723724365234, 0.0042951107025146484, 0.004331827163696289, 0.00440669059753418, 0.004288911819458008, 0.004330873489379883, 0.0043261051177978516, 0.004325151443481445, 0.004300832748413086, 0.004265546798706055, 0.004349708557128906, 0.0043489933013916016, 0.004597187042236328, 0.00435948371887207, 0.004350423812866211, 0.004675149917602539, 0.004416227340698242, 0.004354715347290039, 0.004320383071899414, 0.004375457763671875, 0.0043065547943115234, 0.004332065582275391, 0.004349470138549805, 0.004327297210693359, 0.004270792007446289, 0.004328250885009766, 0.0043735504150390625, 0.004353523254394531, 0.0044095516204833984, 0.004387378692626953, 0.0044095516204833984, 0.004336357116699219, 0.004377603530883789, 0.004319906234741211, 0.004347085952758789, 0.004349231719970703, 0.004410266876220703, 0.004334926605224609, 0.004307985305786133, 0.004431247711181641, 0.00432133674621582, 0.004271984100341797, 0.004384040832519531, 0.004374980926513672, 0.004386425018310547, 0.004355907440185547, 0.004343509674072266, 0.004378795623779297, 0.004335880279541016, 0.004330158233642578, 0.00439763069152832, 0.004359722137451172, 0.004358530044555664, 0.004355907440185547, 0.004358768463134766, 0.0043680667877197266, 0.004632711410522461, 0.0043125152587890625, 0.004401206970214844, 0.004290342330932617, 0.004330158233642578, 0.004359006881713867, 0.004302978515625, 0.004298210144042969, 0.004292488098144531, 0.004350900650024414, 0.004331827163696289, 0.004313230514526367, 0.004352569580078125, 0.0043408870697021484, 0.0043566226959228516, 0.004366874694824219, 0.004350900650024414, 0.004310131072998047, 0.004392385482788086, 0.004386186599731445, 0.004277229309082031, 0.004346370697021484, 0.004384517669677734, 0.00435328483581543, 0.004351377487182617, 0.00431513786315918, 0.00440669059753418, 0.0043337345123291016, 0.004346609115600586, 0.004352569580078125, 0.004312038421630859, 0.004274606704711914, 0.004397392272949219, 0.004357576370239258, 0.00452876091003418, 0.004370689392089844, 0.0043354034423828125, 0.00432896614074707, 0.004350185394287109, 0.004317760467529297, 0.004346609115600586, 0.004389286041259766, 0.0044362545013427734, 0.004341602325439453, 0.0043561458587646484, 0.004396200180053711, 0.004326343536376953, 0.004345417022705078, 0.004344940185546875, 0.0043163299560546875, 0.0043256282806396484, 0.004419088363647461, 0.00429844856262207, 0.004330158233642578, 0.004307985305786133, 0.00439763069152832, 0.004342317581176758, 0.004296541213989258, 0.004377126693725586, 0.004328012466430664, 0.00435638427734375, 0.004351139068603516, 0.004323482513427734, 0.004303693771362305, 0.0045011043548583984, 0.0043354034423828125, 0.0043256282806396484, 0.004323005676269531, 0.0043604373931884766, 0.00432586669921875, 0.004351377487182617, 0.004378557205200195, 0.004448413848876953, 0.005738496780395508, 0.004647731781005859, 0.004446744918823242, 0.0044171810150146484, 0.004918575286865234, 0.004430294036865234, 0.0044138431549072266, 0.00463104248046875, 0.0044405460357666016, 0.004369974136352539, 0.004341602325439453, 0.004479885101318359, 0.004385948181152344, 0.00435185432434082, 0.0042877197265625, 0.004454612731933594, 0.004336118698120117, 0.004517555236816406, 0.00444340705871582, 0.004394054412841797, 0.004305362701416016, 0.004467487335205078, 0.004382610321044922, 0.00431513786315918, 0.0043642520904541016, 0.004342794418334961, 0.0044629573822021484, 0.004471778869628906, 0.004340648651123047, 0.0043468475341796875, 0.004753828048706055, 0.0045642852783203125, 0.005399942398071289, 0.005125761032104492, 0.009627103805541992, 0.01358485221862793, 0.005498647689819336, 0.004965782165527344, 0.004729032516479492, 0.004534006118774414, 0.0045795440673828125, 0.00476384162902832, 0.004400730133056641, 0.0045087337493896484, 0.004461526870727539, 0.004349946975708008, 0.004395723342895508, 0.004415750503540039, 0.004445314407348633, 0.004473686218261719, 0.004560232162475586, 0.0043866634368896484, 0.004323482513427734, 0.004365205764770508, 0.004422903060913086, 0.0044097900390625, 0.004462718963623047, 0.004442453384399414, 0.004349470138549805, 0.004526853561401367, 0.004420280456542969, 0.004372835159301758, 0.004467010498046875, 0.0043849945068359375, 0.00455927848815918, 0.004503488540649414, 0.004482269287109375, 0.004341602325439453, 0.004534482955932617, 0.004509687423706055, 0.004454135894775391, 0.0044078826904296875, 0.004436492919921875, 0.004407167434692383, 0.004407644271850586, 0.004434823989868164, 0.0043871402740478516, 0.004476785659790039, 0.004456043243408203, 0.004367351531982422, 0.004459381103515625, 0.004448652267456055, 0.0044689178466796875, 0.004506349563598633, 0.004524946212768555, 0.004582405090332031, 0.0045011043548583984, 0.004454851150512695, 0.004430532455444336, 0.004346609115600586, 0.0044405460357666016, 0.0044825077056884766, 0.004477500915527344, 0.004479169845581055, 0.0043909549713134766, 0.0045146942138671875, 0.004431962966918945, 0.004472255706787109, 0.004369020462036133, 0.004401206970214844, 0.004533290863037109, 0.004401206970214844, 0.0044403076171875, 0.00446772575378418, 0.0043795108795166016, 0.004553318023681641, 0.0044476985931396484, 0.004397869110107422, 0.004393339157104492, 0.0044918060302734375, 0.004448890686035156, 0.004446744918823242, 0.004381656646728516, 0.004487037658691406, 0.004451274871826172, 0.00448918342590332, 0.00438237190246582, 0.004451751708984375, 0.00451207160949707, 0.004338741302490234, 0.004409074783325195, 0.00445866584777832, 0.004393815994262695, 0.004474163055419922, 0.004438638687133789, 0.004373073577880859, 0.0044286251068115234, 0.004532575607299805, 0.004378318786621094, 0.004450082778930664, 0.0045299530029296875, 0.0044858455657958984, 0.0044481754302978516, 0.004512310028076172, 0.004358530044555664, 0.0044901371002197266, 0.004457950592041016, 0.004523515701293945, 0.004576921463012695, 0.004425764083862305, 0.004488229751586914, 0.0044252872467041016, 0.004488229751586914, 0.0043942928314208984, 0.004503965377807617, 0.004507780075073242, 0.004448413848876953, 0.004430294036865234, 0.0044782161712646484, 0.00442814826965332, 0.004479885101318359, 0.004506111145019531, 0.004454135894775391, 0.00441431999206543, 0.004458189010620117, 0.006089687347412109, 0.004759788513183594, 0.008173465728759766, 0.011275529861450195, 0.005543708801269531, 0.005094051361083984, 0.004879474639892578, 0.004811286926269531, 0.00470423698425293, 0.0045812129974365234, 0.00438237190246582, 0.004429340362548828, 0.004516124725341797, 0.004392385482788086, 0.0043866634368896484, 0.004556417465209961, 0.0045359134674072266, 0.0043773651123046875, 0.004435539245605469, 0.0044155120849609375, 0.004452705383300781, 0.004396915435791016, 0.004461050033569336, 0.0044252872467041016, 0.004532337188720703, 0.004376888275146484, 0.00439453125, 0.004530191421508789, 0.004351139068603516, 0.004356861114501953, 0.004452943801879883, 0.004402637481689453, 0.004431486129760742, 0.004403352737426758, 0.004395723342895508, 0.0043909549713134766, 0.004460811614990234, 0.004313230514526367, 0.00441431999206543, 0.0043888092041015625, 0.004457712173461914, 0.004400730133056641, 0.005078315734863281, 0.004431962966918945, 0.00439906120300293, 0.004471778869628906, 0.004430294036865234, 0.004451274871826172, 0.004462242126464844, 0.004403829574584961, 0.004372119903564453, 0.004407167434692383, 0.004358053207397461, 0.004362583160400391, 0.004435539245605469, 0.004391670227050781, 0.0044097900390625, 0.0042955875396728516, 0.0044634342193603516, 0.00439000129699707, 0.004344940185546875, 0.004480600357055664, 0.004371166229248047, 0.0043871402740478516, 0.004446506500244141, 0.004357576370239258, 0.004335641860961914, 0.004462242126464844, 0.0044176578521728516, 0.00445103645324707, 0.004423856735229492, 0.0043582916259765625, 0.004391193389892578, 0.004415988922119141, 0.004463672637939453, 0.004434347152709961, 0.004492998123168945, 0.004357576370239258, 0.004326581954956055, 0.004415273666381836, 0.004608869552612305, 0.004361867904663086, 0.004384517669677734, 0.004471302032470703, 0.00432896614074707, 0.004471540451049805, 0.00437164306640625, 0.00435185432434082, 0.004388332366943359, 0.004393100738525391, 0.004411458969116211, 0.004641532897949219, 0.004371166229248047, 0.004342555999755859, 0.004410982131958008, 0.0044705867767333984, 0.004314899444580078, 0.004366636276245117, 0.004393815994262695, 0.004404306411743164, 0.004423856735229492, 0.004401206970214844, 0.004325151443481445, 0.00438380241394043, 0.004448652267456055, 0.0043184757232666016, 0.004363298416137695, 0.004500627517700195, 0.004418611526489258, 0.0043942928314208984, 0.004380941390991211, 0.004343748092651367, 0.00437617301940918, 0.004366874694824219, 0.004473447799682617, 0.004388332366943359, 0.004307270050048828, 0.004439353942871094, 0.004361152648925781, 0.004316091537475586, 0.004392147064208984, 0.004380464553833008, 0.00438380241394043, 0.004533529281616211, 0.00455927848815918, 0.004416227340698242, 0.00449061393737793, 0.004465818405151367, 0.0043642520904541016, 0.004426240921020508, 0.0044825077056884766, 0.004390716552734375, 0.004457712173461914, 0.0044019222259521484, 0.004331827163696289, 0.0044634342193603516, 0.004402637481689453, 0.0043833255767822266, 0.004405021667480469, 0.004390239715576172, 0.004401445388793945, 0.004468679428100586, 0.004456043243408203, 0.004388332366943359, 0.0044422149658203125, 0.004693508148193359, 0.004354000091552734, 0.004402875900268555, 0.004396915435791016, 0.0043964385986328125, 0.004459857940673828, 0.004424571990966797, 0.004716157913208008, 0.0044689178466796875, 0.004399299621582031, 0.004358768463134766, 0.004380702972412109, 0.004389524459838867, 0.00441741943359375, 0.004518985748291016, 0.004443168640136719, 0.0043621063232421875, 0.004445314407348633, 0.004437446594238281, 0.004342555999755859, 0.0044248104095458984, 0.004393100738525391, 0.004391193389892578, 0.004410266876220703, 0.0044002532958984375, 0.004492998123168945, 0.004488229751586914, 0.004395484924316406, 0.0043523311614990234, 0.004446983337402344, 0.00447392463684082, 0.004411220550537109, 0.004432201385498047, 0.0044138431549072266, 0.004323244094848633, 0.0044100284576416016, 0.004457235336303711, 0.004362344741821289, 0.004473447799682617, 0.004433393478393555, 0.004441261291503906, 0.004407167434692383, 0.004499673843383789, 0.004424095153808594, 0.004477977752685547, 0.0045168399810791016, 0.00442051887512207, 0.004437923431396484, 0.004476785659790039, 0.004452705383300781, 0.0044634342193603516, 0.004458427429199219, 0.004482746124267578, 0.004477262496948242, 0.004526376724243164, 0.0044460296630859375, 0.004457235336303711, 0.00447392463684082, 0.0044918060302734375, 0.004575967788696289, 0.004473447799682617, 0.004388093948364258, 0.0045278072357177734, 0.0045015811920166016, 0.004404544830322266, 0.004498720169067383, 0.004818439483642578, 0.0044841766357421875, 0.0045239925384521484, 0.00449061393737793, 0.004431962966918945, 0.004526853561401367, 0.004439592361450195, 0.004401683807373047, 0.004518985748291016, 0.004476070404052734, 0.004483461380004883, 0.004732370376586914, 0.0045032501220703125, 0.004478931427001953, 0.004461526870727539, 0.004458904266357422, 0.004494428634643555, 0.005179882049560547, 0.004532575607299805, 0.004482269287109375, 0.004446983337402344, 0.004453897476196289, 0.004456281661987305, 0.0044558048248291016, 0.004471778869628906, 0.004457712173461914, 0.00450444221496582, 0.004560232162475586, 0.004483222961425781, 0.004419565200805664, 0.004477500915527344, 0.004548311233520508, 0.00445556640625, 0.004474639892578125, 0.0044574737548828125, 0.0043981075286865234, 0.00450897216796875, 0.004500389099121094, 0.004570484161376953, 0.004499673843383789, 0.004487037658691406, 0.004442930221557617, 0.004462718963623047, 0.004462242126464844, 0.004446744918823242, 0.004555225372314453, 0.004488945007324219, 0.004489898681640625, 0.004518032073974609, 0.004495859146118164, 0.00451970100402832, 0.004468202590942383, 0.004512786865234375, 0.004518270492553711, 0.004473209381103516, 0.004447221755981445, 0.00458979606628418, 0.0044214725494384766, 0.004452705383300781, 0.0044977664947509766, 0.004407167434692383, 0.004630565643310547, 0.0044896602630615234, 0.004507541656494141, 0.004501819610595703, 0.004494428634643555, 0.0043985843658447266, 0.00447392463684082, 0.00454401969909668, 0.004431962966918945, 0.004483222961425781, 0.004478931427001953, 0.00461888313293457, 0.0045244693756103516, 0.0045015811920166016, 0.0045049190521240234, 0.0047054290771484375, 0.004483461380004883, 0.004438877105712891, 0.0045130252838134766, 0.004438877105712891, 0.004433870315551758, 0.004547834396362305, 0.00447845458984375, 0.0044748783111572266, 0.00444483757019043, 0.004454851150512695, 0.0044519901275634766, 0.004552364349365234, 0.0045070648193359375, 0.004415750503540039, 0.004565238952636719, 0.004488229751586914, 0.0047380924224853516, 0.004422426223754883, 0.004461526870727539, 0.0044667720794677734, 0.0044612884521484375, 0.004472970962524414, 0.004446268081665039, 0.004532814025878906, 0.004464864730834961, 0.004517316818237305, 0.00450444221496582, 0.005048513412475586, 0.004523038864135742, 0.004531383514404297, 0.00450897216796875, 0.0044972896575927734, 0.004469871520996094, 0.004530906677246094, 0.004553079605102539, 0.004423618316650391, 0.004522085189819336, 0.004456281661987305, 0.004515409469604492, 0.004492044448852539, 0.004495382308959961, 0.004428386688232422, 0.004570722579956055, 0.004583120346069336, 0.004496097564697266, 0.004485130310058594, 0.0044689178466796875, 0.00452113151550293, 0.004433155059814453, 0.004476785659790039, 0.004615068435668945, 0.0044689178466796875, 0.004504680633544922, 0.004498720169067383, 0.004416465759277344, 0.00447535514831543, 0.004553556442260742, 0.004442691802978516, 0.004475593566894531, 0.004506349563598633, 0.004499197006225586, 0.004541635513305664, 0.004509449005126953, 0.004440784454345703, 0.004552364349365234, 0.0044939517974853516, 0.0044367313385009766, 0.004491090774536133, 0.004491567611694336, 0.004475831985473633, 0.004588127136230469, 0.004470109939575195, 0.004423856735229492, 0.004548788070678711, 0.004507541656494141, 0.004477977752685547, 0.004488468170166016, 0.004476785659790039, 0.004460811614990234, 0.004479169845581055, 0.0044705867767333984, 0.004440784454345703, 0.004575252532958984, 0.004486083984375, 0.0044095516204833984, 0.004555702209472656, 0.004441738128662109, 0.0045261383056640625, 0.004550457000732422, 0.0044651031494140625, 0.004476070404052734, 0.00449824333190918, 0.0045070648193359375, 0.00453495979309082, 0.004597902297973633, 0.004535675048828125, 0.004551410675048828, 0.004446983337402344, 0.00448918342590332, 0.004500389099121094, 0.004509687423706055, 0.004511833190917969, 0.00451207160949707, 0.004454612731933594, 0.004461765289306641, 0.0045375823974609375, 0.004433870315551758, 0.005220651626586914, 0.004570960998535156, 0.0044820308685302734, 0.00452876091003418, 0.005014657974243164, 0.0045244693756103516, 0.004487037658691406, 0.004480600357055664, 0.0045206546783447266, 0.004438638687133789, 0.004500865936279297, 0.004592180252075195, 0.004435300827026367, 0.004500150680541992, 0.00455474853515625, 0.004509687423706055, 0.004508256912231445, 0.004515409469604492, 0.004454135894775391, 0.004597663879394531, 0.004502534866333008, 0.004402637481689453, 0.004464149475097656, 0.0044863224029541016, 0.004476070404052734, 0.004632711410522461, 0.004483461380004883, 0.0044403076171875, 0.004526615142822266, 0.004549503326416016, 0.004478931427001953, 0.004657268524169922, 0.0045261383056640625, 0.0045566558837890625, 0.004434347152709961, 0.004509687423706055, 0.004509449005126953, 0.004500150680541992, 0.004557371139526367, 0.004514455795288086, 0.00446629524230957, 0.004526376724243164, 0.004605770111083984, 0.004792690277099609, 0.0045549869537353516, 0.004526376724243164, 0.004827976226806641, 0.0046787261962890625, 0.004578351974487305, 0.004730224609375, 0.004443168640136719, 0.004474639892578125, 0.00453495979309082, 0.004492282867431641, 0.004584550857543945, 0.004505157470703125, 0.004462242126464844, 0.0044629573822021484, 0.0045659542083740234, 0.004435539245605469, 0.004524946212768555, 0.004544258117675781, 0.004496335983276367, 0.004498004913330078, 0.0045719146728515625, 0.004446744918823242, 0.004482746124267578, 0.004558086395263672, 0.0044362545013427734, 0.004582405090332031, 0.004514217376708984, 0.004503011703491211, 0.004492044448852539, 0.004494428634643555, 0.004437923431396484, 0.004580497741699219, 0.004580259323120117, 0.004437446594238281, 0.004591226577758789, 0.004515409469604492, 0.004448652267456055, 0.00452733039855957, 0.0044939517974853516, 0.004487037658691406, 0.004526615142822266, 0.0044994354248046875, 0.004498481750488281, 0.0044858455657958984, 0.0044863224029541016, 0.004522800445556641, 0.0045070648193359375, 0.004479646682739258, 0.004539966583251953, 0.004467010498046875, 0.004477739334106445, 0.004584312438964844, 0.0045013427734375, 0.004503488540649414, 0.004507303237915039, 0.004421234130859375, 0.004528045654296875, 0.00457310676574707, 0.004404306411743164, 0.004637002944946289, 0.004544734954833984, 0.004503965377807617, 0.0045490264892578125, 0.004504203796386719, 0.00447535514831543, 0.004553556442260742, 0.0045070648193359375, 0.0044422149658203125, 0.004479885101318359, 0.004603385925292969, 0.004522085189819336, 0.004542827606201172, 0.004523515701293945, 0.004430294036865234, 0.004544734954833984, 0.004532337188720703, 0.00463557243347168, 0.0044841766357421875, 0.004473209381103516, 0.004578351974487305, 0.0045011043548583984, 0.004533290863037109, 0.0045087337493896484, 0.004494428634643555, 0.004519224166870117, 0.004533529281616211, 0.00447845458984375, 0.004754304885864258, 0.0045223236083984375, 0.00445246696472168, 0.004568576812744141, 0.0044977664947509766, 0.004489898681640625, 0.004541635513305664, 0.004483699798583984, 0.004462003707885742, 0.004570484161376953, 0.0047149658203125, 0.004462718963623047, 0.004559755325317383, 0.004497528076171875, 0.004579305648803711, 0.0044651031494140625, 0.004481315612792969, 0.004556179046630859, 0.00451207160949707, 0.0045490264892578125, 0.004510641098022461, 0.004439830780029297, 0.004498004913330078, 0.004584312438964844, 0.004536628723144531, 0.004495859146118164, 0.004508018493652344, 0.004668235778808594, 0.004540443420410156, 0.0045278072357177734, 0.0044765472412109375, 0.004561662673950195, 0.0045392513275146484, 0.004559755325317383, 0.004543304443359375, 0.004492998123168945, 0.0045375823974609375, 0.0045680999755859375, 0.004548549652099609, 0.0045354366302490234, 0.004535198211669922, 0.004504680633544922, 0.004492998123168945, 0.004479885101318359, 0.004511117935180664, 0.0045354366302490234, 0.004476785659790039, 0.004500627517700195, 0.004497528076171875, 0.004545450210571289, 0.00449824333190918, 0.004541635513305664, 0.0044858455657958984, 0.004513740539550781, 0.004598379135131836, 0.004412412643432617, 0.004500150680541992, 0.004525184631347656, 0.0045130252838134766, 0.004591226577758789, 0.004514455795288086, 0.004448890686035156, 0.004578590393066406, 0.004475831985473633, 0.004442453384399414, 0.004584550857543945, 0.004546642303466797, 0.004518985748291016, 0.004522562026977539, 0.004549741744995117, 0.00482630729675293, 0.004721403121948242, 0.004549503326416016, 0.004506349563598633, 0.00446772575378418, 0.004532575607299805, 0.004569530487060547, 0.004574298858642578, 0.004591703414916992, 0.004489898681640625, 0.004548072814941406, 0.004487276077270508, 0.0045318603515625, 0.004482746124267578, 0.004586458206176758, 0.0045278072357177734, 0.00447392463684082, 0.004547119140625, 0.004511833190917969, 0.004593372344970703, 0.004660129547119141, 0.004561662673950195, 0.004507541656494141, 0.004509449005126953, 0.004530191421508789, 0.004507541656494141, 0.004466533660888672, 0.004521369934082031, 0.0045642852783203125, 0.004468441009521484, 0.004528522491455078, 0.0045206546783447266, 0.00452423095703125, 0.0045740604400634766, 0.004567146301269531, 0.004469871520996094, 0.00458979606628418, 0.0045320987701416016, 0.004518032073974609, 0.004534244537353516, 0.004529476165771484, 0.004492044448852539, 0.004565238952636719, 0.004488468170166016, 0.004474639892578125, 0.004680156707763672, 0.004561901092529297, 0.004533290863037109, 0.004561424255371094, 0.0045697689056396484, 0.004583597183227539, 0.004563570022583008, 0.004575490951538086, 0.0045185089111328125, 0.004576921463012695, 0.004520416259765625, 0.004561424255371094, 0.004517793655395508, 0.004593849182128906, 0.004568815231323242, 0.004569053649902344, 0.0045490264892578125, 0.004534244537353516, 0.004483699798583984, 0.004515409469604492, 0.004543304443359375, 0.004540681838989258, 0.004559755325317383, 0.004518985748291016, 0.0047969818115234375, 0.004464626312255859, 0.004558563232421875, 0.004559993743896484, 0.00447535514831543, 0.004586219787597656, 0.004524946212768555, 0.004505634307861328, 0.0045316219329833984, 0.0045163631439208984, 0.004473447799682617, 0.004603862762451172, 0.004524707794189453, 0.004511833190917969, 0.004506111145019531, 0.004522800445556641, 0.004508018493652344, 0.00461125373840332, 0.004505157470703125, 0.004491090774536133, 0.004691600799560547, 0.0045239925384521484, 0.004606008529663086, 0.00448298454284668, 0.004565238952636719, 0.004557371139526367, 0.004603385925292969, 0.004674196243286133, 0.004498481750488281, 0.0045397281646728516, 0.004603385925292969, 0.00455164909362793, 0.004511594772338867, 0.004556179046630859, 0.0045223236083984375, 0.004544258117675781, 0.0045201778411865234, 0.004475593566894531, 0.004549264907836914, 0.004555463790893555, 0.00449824333190918, 0.004517555236816406, 0.004512786865234375, 0.004508018493652344, 0.004511833190917969, 0.0045168399810791016, 0.00448298454284668, 0.004568576812744141, 0.004519939422607422, 0.004612922668457031, 0.004641532897949219, 0.004501819610595703, 0.004612445831298828, 0.004487037658691406, 0.004513978958129883, 0.004542827606201172, 0.004507780075073242, 0.00453639030456543, 0.0045125484466552734, 0.004508018493652344, 0.004518270492553711, 0.004582643508911133, 0.004448652267456055, 0.004572153091430664, 0.004626274108886719, 0.004553318023681641, 0.004549264907836914, 0.0045278072357177734, 0.004494667053222656, 0.004694461822509766, 0.004617214202880859, 0.004532575607299805, 0.004586219787597656, 0.004530906677246094, 0.004594564437866211, 0.004533290863037109, 0.004628181457519531, 0.004532575607299805, 0.0045604705810546875, 0.0044727325439453125, 0.004640817642211914, 0.0047206878662109375, 0.0045528411865234375, 0.0046253204345703125, 0.004572868347167969, 0.004604816436767578, 0.004509449005126953, 0.004558086395263672, 0.004531145095825195, 0.004537343978881836, 0.004518032073974609, 0.004631519317626953, 0.004521846771240234, 0.004601716995239258, 0.004610776901245117, 0.004554033279418945, 0.004575967788696289, 0.004585981369018555, 0.004558563232421875, 0.004580020904541016, 0.004530429840087891, 0.0045320987701416016, 0.004549980163574219, 0.004543781280517578, 0.0045588016510009766, 0.004544258117675781, 20.60214376449585]
t_12_lzf = [0.004929780960083008, 0.0038156509399414062, 0.003911733627319336, 0.004050493240356445, 0.004028797149658203, 0.003984212875366211, 0.003877878189086914, 0.003891468048095703, 0.003903627395629883, 0.005043506622314453, 0.004525423049926758, 0.00388336181640625, 0.004904747009277344, 0.00519871711730957, 0.005606889724731445, 0.005484104156494141, 0.00558161735534668, 0.005568742752075195, 0.00554347038269043, 0.005568504333496094, 0.0055654048919677734, 0.005498647689819336, 0.005564451217651367, 0.005421161651611328, 0.005053520202636719, 0.0056476593017578125, 0.0038330554962158203, 0.0037734508514404297, 0.0037915706634521484, 0.0039577484130859375, 0.005307674407958984, 0.005006074905395508, 0.0037682056427001953, 0.0036864280700683594, 0.003718852996826172, 0.0036878585815429688, 0.0038099288940429688, 0.003686666488647461, 0.0036821365356445312, 0.003702402114868164, 0.003650188446044922, 0.003745555877685547, 0.0036716461181640625, 0.0037200450897216797, 0.003676176071166992, 0.003634929656982422, 0.003692150115966797, 0.0036928653717041016, 0.0036857128143310547, 0.0036475658416748047, 0.003697633743286133, 0.003675699234008789, 0.0036661624908447266, 0.0036880970001220703, 0.0036363601684570312, 0.003682851791381836, 0.003634214401245117, 0.003670215606689453, 0.0036759376525878906, 0.003732919692993164, 0.0036873817443847656, 0.0036399364471435547, 0.0036721229553222656, 0.0036742687225341797, 0.003695249557495117, 0.0036344528198242188, 0.0036640167236328125, 0.0036363601684570312, 0.003687143325805664, 0.003673076629638672, 0.0036897659301757812, 0.003677845001220703, 0.0036399364471435547, 0.0036869049072265625, 0.0036780834197998047, 0.0036699771881103516, 0.0036389827728271484, 0.0036721229553222656, 0.0036716461181640625, 0.0036787986755371094, 0.0037260055541992188, 0.0036313533782958984, 0.0036721229553222656, 0.0036535263061523438, 0.0036814212799072266, 0.0036776065826416016, 0.0036852359771728516, 0.0036351680755615234, 0.0036847591400146484, 0.0036613941192626953, 0.0037255287170410156, 0.0037801265716552734, 0.0036325454711914062, 0.003699779510498047, 0.0036475658416748047, 0.003684520721435547, 0.003682851791381836, 0.0036725997924804688, 0.003724813461303711, 0.003696918487548828, 0.003672361373901367, 0.0036835670471191406, 0.0036749839782714844, 0.0036520957946777344, 0.003693819046020508, 0.003695964813232422, 0.00368499755859375, 0.0036745071411132812, 0.0036957263946533203, 0.0037589073181152344, 0.0036554336547851562, 0.0037336349487304688, 0.003753185272216797, 0.0037038326263427734, 0.003658294677734375, 0.0036840438842773438, 0.0036542415618896484, 0.003699064254760742, 0.003729104995727539, 0.0037691593170166016, 0.0036983489990234375, 0.0036542415618896484, 0.003698110580444336, 0.003695249557495117, 0.0036897659301757812, 0.0036547183990478516, 0.0037255287170410156, 0.003705263137817383, 0.0036933422088623047, 0.0036954879760742188, 0.0036516189575195312, 0.0037009716033935547, 0.0036525726318359375, 0.0036721229553222656, 0.0037031173706054688, 0.003681182861328125, 0.0036423206329345703, 0.0037152767181396484, 0.0036957263946533203, 0.0036950111389160156, 0.003690004348754883, 0.0036461353302001953, 0.0036852359771728516, 0.003652334213256836, 0.003697633743286133, 0.003770589828491211, 0.003700733184814453, 0.003696918487548828, 0.003647327423095703, 0.003690958023071289, 0.0037119388580322266, 0.0036764144897460938, 0.0036983489990234375, 0.0037479400634765625, 0.0036618709564208984, 0.0037305355072021484, 0.0036928653717041016, 0.0036482810974121094, 0.003687143325805664, 0.0036520957946777344, 0.003673076629638672, 0.003679513931274414, 0.003690958023071289, 0.003656625747680664, 0.003753662109375, 0.0037446022033691406, 0.004190921783447266, 0.004029035568237305, 0.004080295562744141, 0.003917217254638672, 0.004404783248901367, 0.0046503543853759766, 0.008309125900268555, 0.008644819259643555, 0.005231142044067383, 0.00494074821472168, 0.004426479339599609, 0.012221813201904297, 0.010361671447753906, 0.006964683532714844, 0.008132457733154297, 0.0070612430572509766, 0.0053751468658447266, 0.009238719940185547, 0.005740165710449219, 0.004227399826049805, 0.004088163375854492, 0.004071712493896484, 0.004076957702636719, 0.004059791564941406, 0.0039751529693603516, 0.0038089752197265625, 0.0037813186645507812, 0.003705739974975586, 0.0037391185760498047, 0.003657102584838867, 0.0036954879760742188, 0.0036475658416748047, 0.0036911964416503906, 0.003808736801147461, 0.0036954879760742188, 0.0037016868591308594, 0.0036628246307373047, 0.0037131309509277344, 0.0036602020263671875, 0.003857135772705078, 0.003683805465698242, 0.0037033557891845703, 0.0037195682525634766, 0.003660917282104492, 0.0037004947662353516, 0.0037009716033935547, 0.0036857128143310547, 0.003722667694091797, 0.0037147998809814453, 0.0036547183990478516, 0.003691434860229492, 0.003885507583618164, 0.0036787986755371094, 0.0037527084350585938, 0.0036618709564208984, 0.0036971569061279297, 0.003697633743286133, 0.0037114620208740234, 0.0036728382110595703, 0.003720998764038086, 0.0037093162536621094, 0.0036945343017578125, 0.0036973953247070312, 0.003656148910522461, 0.003711223602294922, 0.0036749839782714844, 0.003699779510498047, 0.0037026405334472656, 0.0037355422973632812, 0.003718852996826172, 0.0036721229553222656, 0.0038537979125976562, 0.003721952438354492, 0.0038673877716064453, 0.0036749839782714844, 0.003711700439453125, 0.003671884536743164, 0.0037927627563476562, 0.003703594207763672, 0.0037322044372558594, 0.0037169456481933594, 0.003664255142211914, 0.003705263137817383, 0.0037446022033691406, 0.0037810802459716797, 0.0037245750427246094, 0.0036704540252685547, 0.0037164688110351562, 0.0037391185760498047, 0.0037124156951904297, 0.003666400909423828, 0.003719806671142578, 0.003668546676635742, 0.0037021636962890625, 0.003758668899536133, 0.0036683082580566406, 0.0037164688110351562, 0.0036673545837402344, 0.0037136077880859375, 0.0037162303924560547, 0.003704547882080078, 0.0036602020263671875, 0.0037162303924560547, 0.003710508346557617, 0.003718852996826172, 0.0037102699279785156, 0.0036673545837402344, 0.0037322044372558594, 0.0036771297454833984, 0.003719329833984375, 0.003708362579345703, 0.003712177276611328, 0.0037577152252197266, 0.003682374954223633, 0.0037190914154052734, 0.0037140846252441406, 0.0036966800689697266, 0.0036759376525878906, 0.003711223602294922, 0.0037147998809814453, 0.003701925277709961, 0.0037527084350585938, 0.0036666393280029297, 0.0037050247192382812, 0.003667116165161133, 0.003691434860229492, 0.0037126541137695312, 0.003839731216430664, 0.0036678314208984375, 0.0037229061126708984, 0.0037326812744140625, 0.0037050247192382812, 0.0037081241607666016, 0.003767728805541992, 0.003706216812133789, 0.003661632537841797, 0.0036928653717041016, 0.0037050247192382812, 0.0037260055541992188, 0.0037679672241210938, 0.003665447235107422, 0.0036895275115966797, 0.00372314453125, 0.003707408905029297, 0.0036728382110595703, 0.003726482391357422, 0.0036554336547851562, 0.0037288665771484375, 0.0037729740142822266, 0.0037584304809570312, 0.003696918487548828, 0.0036733150482177734, 0.0037076473236083984, 0.0037245750427246094, 0.0037488937377929688, 0.003721952438354492, 0.0037620067596435547, 0.0037648677825927734, 0.0037615299224853516, 0.0037622451782226562, 0.003726482391357422, 0.0037620067596435547, 0.0037603378295898438, 0.003751516342163086, 0.00380706787109375, 0.0037078857421875, 0.0037517547607421875, 0.0037043094635009766, 0.003778219223022461, 0.003756999969482422, 0.0037615299224853516, 0.003697633743286133, 0.003788471221923828, 0.003791332244873047, 0.0037391185760498047, 0.00374603271484375, 0.0037050247192382812, 0.003770589828491211, 0.0037097930908203125, 0.0037627220153808594, 0.003914833068847656, 0.0037353038787841797, 0.003763437271118164, 0.0038199424743652344, 0.0037696361541748047, 0.0037479400634765625, 0.0038137435913085938, 0.0038683414459228516, 0.0037260055541992188, 0.0037572383880615234, 0.0037839412689208984, 0.0037527084350585938, 0.0037212371826171875, 0.0037660598754882812, 0.0037140846252441406, 0.003820180892944336, 0.0037660598754882812, 0.003791332244873047, 0.0037801265716552734, 0.003737926483154297, 0.0037610530853271484, 0.003770589828491211, 0.0037937164306640625, 0.0037834644317626953, 0.003713846206665039, 0.003770112991333008, 0.0038034915924072266, 0.0040950775146484375, 0.003801584243774414, 0.003860950469970703, 0.0037724971771240234, 0.003784656524658203, 0.0037682056427001953, 0.003761768341064453, 0.0037746429443359375, 0.0037424564361572266, 0.0037124156951904297, 0.0038068294525146484, 0.003762960433959961, 0.003773927688598633, 0.003766775131225586, 0.0037407875061035156, 0.003807544708251953, 0.0037293434143066406, 0.003763437271118164, 0.003806591033935547, 0.003718852996826172, 0.0037682056427001953, 0.0037314891815185547, 0.0037636756896972656, 0.0037679672241210938, 0.0037775039672851562, 0.0037703514099121094, 0.0037970542907714844, 0.0037641525268554688, 0.0037581920623779297, 0.003781557083129883, 0.0037338733673095703, 0.0037682056427001953, 0.003715991973876953, 0.0038852691650390625, 0.003766775131225586, 0.0037648677825927734, 0.003780841827392578, 0.003764629364013672, 0.0037190914154052734, 0.0037508010864257812, 0.0037887096405029297, 0.0038237571716308594, 0.003754138946533203, 0.0037217140197753906, 0.0037589073181152344, 0.0037314891815185547, 0.0037603378295898438, 0.003754854202270508, 0.003761768341064453, 0.00376129150390625, 0.003714323043823242, 0.003761768341064453, 0.0037648677825927734, 0.0037527084350585938, 0.0037140846252441406, 0.0037772655487060547, 0.003758668899536133, 0.0037887096405029297, 0.0037827491760253906, 0.003710031509399414, 0.0038137435913085938, 0.003733396530151367, 0.0037882328033447266, 0.0037589073181152344, 0.0037696361541748047, 0.0037589073181152344, 0.0037238597869873047, 0.0037598609924316406, 0.0037720203399658203, 0.003756999969482422, 0.0037877559661865234, 0.0037686824798583984, 0.003763437271118164, 0.0038127899169921875, 0.0037899017333984375, 0.003755331039428711, 0.0037717819213867188, 0.0037353038787841797, 0.0037589073181152344, 0.0037245750427246094, 0.003899812698364258, 0.0037233829498291016, 0.0037555694580078125, 0.003756284713745117, 0.003747224807739258, 0.0037224292755126953, 0.00382232666015625, 0.0038039684295654297, 0.003763914108276367, 0.0037627220153808594, 0.0037834644317626953, 0.003731966018676758, 0.003759622573852539, 0.003758668899536133, 0.0037927627563476562, 0.003832101821899414, 0.0037441253662109375, 0.0037963390350341797, 0.0037183761596679688, 0.0039043426513671875, 0.003778219223022461, 0.003782033920288086, 0.003797292709350586, 0.003730297088623047, 0.003771066665649414, 0.0037865638732910156, 0.003783702850341797, 0.00374603271484375, 0.0037689208984375, 0.003735780715942383, 0.003831624984741211, 0.0037827491760253906, 0.0037369728088378906, 0.0037796497344970703, 0.003747224807739258, 0.003785371780395508, 0.003797769546508789, 0.0038323402404785156, 0.003785371780395508, 0.0037353038787841797, 0.0037870407104492188, 0.003793478012084961, 0.0037994384765625, 0.003752470016479492, 0.003802776336669922, 0.003847837448120117, 0.003801584243774414, 0.0037813186645507812, 0.003738880157470703, 0.003767251968383789, 0.0037326812744140625, 0.0037832260131835938, 0.003853321075439453, 0.0038115978240966797, 0.0037686824798583984, 0.0037508010864257812, 0.0037958621978759766, 0.003792285919189453, 0.003774881362915039, 0.0038902759552001953, 0.0037550926208496094, 0.003820657730102539, 0.0037784576416015625, 0.003773927688598633, 0.003732919692993164, 0.003810882568359375, 0.0037457942962646484, 0.0038268566131591797, 0.003776073455810547, 0.003735065460205078, 0.003776073455810547, 0.0037412643432617188, 0.0037658214569091797, 0.003787517547607422, 0.003785848617553711, 0.0038177967071533203, 0.0037381649017333984, 0.0037851333618164062, 0.003798961639404297, 0.003766298294067383, 0.003751039505004883, 0.003799915313720703, 0.0037958621978759766, 0.0037822723388671875, 0.003773927688598633, 0.0037403106689453125, 0.0037992000579833984, 0.0037539005279541016, 0.0037832260131835938, 0.0038275718688964844, 0.0037508010864257812, 0.003780364990234375, 0.0037393569946289062, 0.003788471221923828, 0.0037784576416015625, 0.0037856101989746094, 0.0037550926208496094, 0.003924131393432617, 0.003785371780395508, 0.003782033920288086, 0.0037839412689208984, 0.003815174102783203, 0.003779172897338867, 0.0037310123443603516, 0.0038504600524902344, 0.0039014816284179688, 0.0037851333618164062, 0.0037801265716552734, 0.0037546157836914062, 0.003779888153076172, 0.0037822723388671875, 0.003889322280883789, 0.003774881362915039, 0.0037436485290527344, 0.003792285919189453, 0.003770589828491211, 0.0037848949432373047, 0.0037436485290527344, 0.003785371780395508, 0.0038061141967773438, 0.0037860870361328125, 0.0038213729858398438, 0.0037615299224853516, 0.00379180908203125, 0.003747701644897461, 0.003793954849243164, 0.003867626190185547, 0.0037415027618408203, 0.0037758350372314453, 0.003748178482055664, 0.003782033920288086, 0.0037903785705566406, 0.0037856101989746094, 0.003800630569458008, 0.003745555877685547, 0.0037872791290283203, 0.003785371780395508, 0.003798246383666992, 0.003727436065673828, 0.0037708282470703125, 0.0037584304809570312, 0.003873109817504883, 0.003795146942138672, 0.0037500858306884766, 0.0037822723388671875, 0.0037450790405273438, 0.003798246383666992, 0.0037703514099121094, 0.0037856101989746094, 0.0037775039672851562, 0.0037636756896972656, 0.003789663314819336, 0.0037734508514404297, 0.003782510757446289, 0.003736734390258789, 0.0037925243377685547, 0.0037980079650878906, 0.0037872791290283203, 0.003919839859008789, 0.00376129150390625, 0.003800630569458008, 0.003751993179321289, 0.0038030147552490234, 0.003844022750854492, 0.0038383007049560547, 0.0038118362426757812, 0.003756284713745117, 0.003796815872192383, 0.0037963390350341797, 0.003795146942138672, 0.0038187503814697266, 0.003797769546508789, 0.004081249237060547, 0.0038099288940429688, 0.003797292709350586, 0.0037572383880615234, 0.0038199424743652344, 0.0038154125213623047, 0.0037903785705566406, 0.003805398941040039, 0.003759145736694336, 0.0037958621978759766, 0.0037419795989990234, 0.003787517547607422, 0.0038671493530273438, 0.003795623779296875, 0.0038089752197265625, 0.003746509552001953, 0.0037796497344970703, 0.0037789344787597656, 0.0037882328033447266, 0.003818511962890625, 0.004395723342895508, 0.00410771369934082, 0.0038208961486816406, 0.0037899017333984375, 0.0038068294525146484, 0.0037384033203125, 0.003877401351928711, 0.003788471221923828, 0.0038149356842041016, 0.0037543773651123047, 0.00380706787109375, 0.0038061141967773438, 0.0038094520568847656, 0.0037970542907714844, 0.0037543773651123047, 0.0037877559661865234, 0.0037453174591064453, 0.003827333450317383, 0.0038514137268066406, 0.0038840770721435547, 0.0038008689880371094, 0.0038280487060546875, 0.003888368606567383, 0.003849029541015625, 0.003788471221923828, 0.003917217254638672, 0.003865957260131836, 0.00379180908203125, 0.0038001537322998047, 0.003793001174926758, 0.003757953643798828, 0.0037856101989746094, 0.003826618194580078, 0.0038056373596191406, 0.003813505172729492, 0.003754138946533203, 0.003782510757446289, 0.0037555694580078125, 0.003810405731201172, 0.003864288330078125, 0.0037496089935302734, 0.0037844181060791016, 0.0037496089935302734, 0.0038347244262695312, 0.0038051605224609375, 0.003797292709350586, 0.003799915313720703, 0.003757953643798828, 0.003793954849243164, 0.0038068294525146484, 0.0038115978240966797, 0.0038056373596191406, 0.003792285919189453, 0.003766775131225586, 0.0038640499114990234, 0.003811359405517578, 0.003765583038330078, 0.0037937164306640625, 0.003772735595703125, 0.0038042068481445312, 0.003808259963989258, 0.0037932395935058594, 0.0038077831268310547, 0.003765106201171875, 0.0038030147552490234, 0.003804445266723633, 0.0037822723388671875, 0.003757953643798828, 0.003834962844848633, 0.0038237571716308594, 0.0037877559661865234, 0.003794431686401367, 0.0038878917694091797, 0.0038299560546875, 0.0037708282470703125, 0.0038661956787109375, 0.003811359405517578, 0.0038471221923828125, 0.003821134567260742, 0.0037767887115478516, 0.0038030147552490234, 0.0038118362426757812, 0.0038115978240966797, 0.0038366317749023438, 0.0038101673126220703, 0.003930807113647461, 0.003835916519165039, 0.0038444995880126953, 0.0038421154022216797, 0.0038022994995117188, 0.0040013790130615234, 0.003835916519165039, 0.003862619400024414, 0.003814220428466797, 0.0038552284240722656, 0.0038461685180664062, 0.0038385391235351562, 0.0038633346557617188, 0.0038268566131591797, 0.0038690567016601562, 0.0037965774536132812, 0.0038928985595703125, 0.003927707672119141, 0.0038955211639404297, 0.0038712024688720703, 0.003810882568359375, 0.0038449764251708984, 0.003902435302734375, 0.0038046836853027344, 0.0038416385650634766, 0.0037992000579833984, 0.0038292407989501953, 0.0038564205169677734, 0.0038394927978515625, 0.0038733482360839844, 0.0037932395935058594, 0.003832101821899414, 0.003832101821899414, 0.003866910934448242, 0.0038046836853027344, 0.003844022750854492, 0.0038404464721679688, 0.003850221633911133, 0.003844022750854492, 0.003908395767211914, 0.0038542747497558594, 0.0037963390350341797, 0.0038814544677734375, 0.0038595199584960938, 0.0038862228393554688, 0.003834247589111328, 0.0037832260131835938, 0.0038259029388427734, 0.0038568973541259766, 0.0038466453552246094, 0.003854990005493164, 0.0038063526153564453, 0.0038604736328125, 0.0038306713104248047, 0.003838062286376953, 0.0038263797760009766, 0.0037844181060791016, 0.003819704055786133, 0.0038454532623291016, 0.003847837448120117, 0.0037789344787597656, 0.0038361549377441406, 0.0038335323333740234, 0.003851175308227539, 0.0038247108459472656, 0.0037994384765625, 0.0038292407989501953, 0.0037851333618164062, 0.0038480758666992188, 0.003907442092895508, 0.003806591033935547, 0.003824472427368164, 0.003787994384765625, 0.0038399696350097656, 0.0038411617279052734, 0.0038576126098632812, 0.0038602352142333984, 0.003793001174926758, 0.0038514137268066406, 0.003838777542114258, 0.0038671493530273438, 0.003784656524658203, 0.003841400146484375, 0.003840923309326172, 0.0038301944732666016, 0.0038394927978515625, 0.0037965774536132812, 0.0038292407989501953, 0.0038039684295654297, 0.0038895606994628906, 0.0038378238677978516, 0.0038759708404541016, 0.003885984420776367, 0.003807544708251953, 0.0038552284240722656, 0.003886699676513672, 0.003940105438232422, 0.00384521484375, 0.0038559436798095703, 0.003847360610961914, 0.00384521484375, 0.003839254379272461, 0.00384521484375, 0.0038292407989501953, 0.003853321075439453, 0.0038366317749023438, 0.0038526058197021484, 0.003800630569458008, 0.0038623809814453125, 0.003837108612060547, 0.0038483142852783203, 0.003831148147583008, 0.004402637481689453, 0.0039021968841552734, 0.0038573741912841797, 0.0038297176361083984, 0.004217863082885742, 0.003852367401123047, 0.003857135772705078, 0.0038759708404541016, 0.003796100616455078, 0.0038547515869140625, 0.0038323402404785156, 0.0038690567016601562, 0.0038118362426757812, 0.0038886070251464844, 0.0038471221923828125, 0.003850221633911133, 0.0038568973541259766, 0.003795146942138672, 0.003855466842651367, 0.0038013458251953125, 0.003893613815307617, 0.0038552284240722656, 0.0038225650787353516, 0.003863096237182617, 0.0038073062896728516, 0.0038444995880126953, 0.0038542747497558594, 0.0038771629333496094, 0.003846883773803711, 0.0038204193115234375, 0.003857851028442383, 0.003963470458984375, 0.003862619400024414, 0.003932952880859375, 0.0038299560546875, 0.003856182098388672, 0.00391077995300293, 0.003875732421875, 0.0038046836853027344, 0.0039327144622802734, 0.0038323402404785156, 0.0038192272186279297, 0.003869295120239258, 0.0038137435913085938, 0.0038955211639404297, 0.0038912296295166016, 0.003873586654663086, 0.003851652145385742, 0.0038101673126220703, 0.004067897796630859, 0.003970623016357422, 0.0039052963256835938, 0.0038492679595947266, 0.004040718078613281, 0.004113674163818359, 0.003943681716918945, 0.0038530826568603516, 0.003859996795654297, 0.003810882568359375, 0.0038483142852783203, 0.0038869380950927734, 0.0038504600524902344, 0.0038051605224609375, 0.0039033889770507812, 0.0038461685180664062, 0.003840923309326172, 0.0038492679595947266, 0.003829479217529297, 0.0038535594940185547, 0.0038564205169677734, 0.0038521289825439453, 0.003863096237182617, 0.0038225650787353516, 0.0038568973541259766, 0.0038156509399414062, 0.003867626190185547, 0.003916263580322266, 0.0038063526153564453, 0.003866910934448242, 0.003803730010986328, 0.003855466842651367, 0.0038535594940185547, 0.003957271575927734, 0.003859281539916992, 0.0038127899169921875, 0.0038635730743408203, 0.003850221633911133, 0.0038576126098632812, 0.003957509994506836, 0.003817319869995117, 0.0038881301879882812, 0.0038623809814453125, 0.0038967132568359375, 0.0038115978240966797, 0.0038595199584960938, 0.003927946090698242, 0.003856182098388672, 0.003862142562866211, 0.0038247108459472656, 0.003923654556274414, 0.0038492679595947266, 0.0039234161376953125, 0.0038573741912841797, 0.003816843032836914, 0.0038480758666992188, 0.0037987232208251953, 0.003857851028442383, 0.003900766372680664, 0.003818511962890625, 0.003859281539916992, 0.0038013458251953125, 0.0038700103759765625, 0.0038657188415527344, 0.0038564205169677734, 0.0038781166076660156, 0.0038051605224609375, 0.0038652420043945312, 0.003866434097290039, 0.003875732421875, 0.00382232666015625, 0.0038678646087646484, 0.0038678646087646484, 0.0038726329803466797, 0.0038466453552246094, 0.003810882568359375, 0.003850698471069336, 0.0038280487060546875, 0.003919124603271484, 0.003840923309326172, 0.0038673877716064453, 0.0038559436798095703, 0.003818035125732422, 0.0038738250732421875, 0.003915071487426758, 0.0039038658142089844, 0.0038754940032958984, 0.0038869380950927734, 0.0039103031158447266, 0.003857851028442383, 0.003952980041503906, 0.0038788318634033203, 0.0038280487060546875, 0.003889322280883789, 0.003877401351928711, 0.003912925720214844, 0.0038802623748779297, 0.003821134567260742, 0.003907442092895508, 0.0038568973541259766, 0.0038602352142333984, 0.003831624984741211, 0.003892183303833008, 0.004019975662231445, 0.003932476043701172, 0.0038595199584960938, 0.003826141357421875, 0.003873109817504883, 0.0038864612579345703, 0.003869295120239258, 0.00388336181640625, 0.0038368701934814453, 0.0038678646087646484, 0.003835916519165039, 0.00391077995300293, 0.0038487911224365234, 0.0038747787475585938, 0.004289388656616211, 0.003821134567260742, 0.0038819313049316406, 0.003905057907104492, 0.003826618194580078, 0.003868579864501953, 0.003830432891845703, 0.0038678646087646484, 0.0038595199584960938, 0.0038862228393554688, 0.0038657188415527344, 0.003833770751953125, 0.003863811492919922, 0.003857851028442383, 0.003887176513671875, 0.0038323402404785156, 0.003863096237182617, 0.0038640499114990234, 0.0038743019104003906, 0.003860950469970703, 0.003962278366088867, 0.0038852691650390625, 0.0038568973541259766, 0.0038604736328125, 0.0038678646087646484, 0.003934621810913086, 0.003878355026245117, 0.003824472427368164, 0.003931999206542969, 0.003864288330078125, 0.0038204193115234375, 0.0038797855377197266, 0.0038313865661621094, 0.003862619400024414, 0.003912210464477539, 0.0038356781005859375, 0.003877401351928711, 0.0038323402404785156, 0.003870248794555664, 0.003871440887451172, 0.0038857460021972656, 0.003889799118041992, 0.0038259029388427734, 0.003859996795654297, 0.0038590431213378906, 0.0038650035858154297, 0.0038454532623291016, 0.0038781166076660156, 0.003869295120239258, 0.0038793087005615234, 0.0038743019104003906, 0.0038290023803710938, 0.0038709640502929688, 0.003901243209838867, 0.0038604736328125, 0.003895282745361328, 0.0038461685180664062, 0.003875255584716797, 0.0038177967071533203, 0.003928422927856445, 0.003874063491821289, 0.0038237571716308594, 0.0038764476776123047, 0.0038406848907470703, 0.003873109817504883, 0.003928422927856445, 0.0038385391235351562, 0.003849506378173828, 0.0038306713104248047, 0.0038726329803466797, 0.0038771629333496094, 0.0038766860961914062, 0.0039899349212646484, 0.0038623809814453125, 0.0038580894470214844, 0.0038635730743408203, 0.003875732421875, 0.00394439697265625, 0.003850221633911133, 0.003894805908203125, 0.0038814544677734375, 0.003894805908203125, 0.003818988800048828, 0.0038690567016601562, 0.0038743019104003906, 0.003877401351928711, 0.003899812698364258, 0.003837108612060547, 0.003860950469970703, 0.0038259029388427734, 0.003925323486328125, 0.003883838653564453, 0.00392603874206543, 0.003869771957397461, 0.0038361549377441406, 0.003876924514770508, 0.003934144973754883, 0.0038247108459472656, 0.0038635730743408203, 0.003834247589111328, 0.0038712024688720703, 0.003885507583618164, 0.0039043426513671875, 0.003880023956298828, 0.003826141357421875, 0.0038690567016601562, 0.003895998001098633, 0.003883838653564453, 0.003874063491821289, 0.003827333450317383, 0.0038881301879882812, 0.0038802623748779297, 0.003879070281982422, 0.0038421154022216797, 0.0038788318634033203, 0.0038945674896240234, 0.0038874149322509766, 0.003872394561767578, 0.0038449764251708984, 0.0038716793060302734, 0.003852367401123047, 0.003943681716918945, 0.003882884979248047, 0.003947257995605469, 0.0039217472076416016, 0.003877878189086914, 0.0038874149322509766, 0.00398707389831543, 0.003914356231689453, 0.0038764476776123047, 0.003860950469970703, 0.004009723663330078, 0.0039861202239990234, 0.0038704872131347656, 0.003923177719116211, 0.0039021968841552734, 0.003977298736572266, 0.003926277160644531, 0.003923892974853516, 0.003913402557373047, 0.003877878189086914, 0.0039675235748291016, 0.003930568695068359, 0.00393223762512207, 0.003925800323486328, 0.0038805007934570312, 0.00392603874206543, 0.003918647766113281, 0.003940582275390625, 0.003937959671020508, 0.003875732421875, 0.003939390182495117, 0.003920316696166992, 0.0039157867431640625, 0.0038819313049316406, 0.004053831100463867, 0.004099845886230469, 0.003926753997802734, 0.0038907527923583984, 0.003916740417480469, 0.00404810905456543, 0.003911495208740234, 0.003925323486328125, 0.0038847923278808594, 0.003916025161743164, 0.0039212703704833984, 0.00392603874206543, 0.003935098648071289, 0.003872394561767578, 0.003912210464477539, 0.003925323486328125, 0.003941059112548828, 0.0039119720458984375, 0.0038857460021972656, 0.00397181510925293, 0.004014492034912109, 0.003920316696166992, 0.003954410552978516, 0.003906726837158203, 0.00397944450378418, 0.003917217254638672, 0.003951549530029297, 0.0039784908294677734, 0.003915548324584961, 0.0038805007934570312, 0.003941535949707031, 0.003933906555175781, 0.003922224044799805, 0.003932476043701172, 0.003875732421875, 0.003939628601074219, 0.003937959671020508, 0.003920793533325195, 0.003890514373779297, 0.003938436508178711, 0.003934144973754883, 0.003931999206542969, 0.0039060115814208984, 0.003876924514770508, 0.003937959671020508, 0.003925323486328125, 0.003937721252441406, 0.003931999206542969, 0.00389862060546875, 0.003911256790161133, 0.003931999206542969, 0.003921985626220703, 0.003908872604370117, 0.0038695335388183594, 0.003907680511474609, 0.0038995742797851562, 0.0039784908294677734, 0.003929615020751953, 0.0038950443267822266, 0.003923654556274414, 0.0038971900939941406, 0.0039768218994140625, 0.003921985626220703, 0.003877401351928711, 0.003931760787963867, 0.003911256790161133, 0.003928422927856445, 0.003978729248046875, 0.0038967132568359375, 0.003916501998901367, 0.003904104232788086, 0.004083395004272461, 0.0040204524993896484, 0.003909111022949219, 0.003927707672119141, 0.003966093063354492, 0.003933429718017578, 0.003916263580322266, 0.003929615020751953, 0.003935813903808594, 0.003898143768310547, 0.003935813903808594, 0.0039560794830322266, 0.00400853157043457, 0.003939628601074219, 0.003910064697265625, 0.00394439697265625, 0.003929853439331055, 0.0040814876556396484, 0.003958940505981445, 0.0039081573486328125, 0.003958702087402344, 0.003955364227294922, 0.003954887390136719, 0.0039484500885009766, 0.003901243209838867, 0.003940105438232422, 0.00391387939453125, 0.0039119720458984375, 0.003891468048095703, 0.004097938537597656, 0.003949403762817383, 0.004141330718994141, 0.003945827484130859, 0.003919363021850586, 0.0039882659912109375, 0.00393986701965332, 0.0039348602294921875, 0.003946542739868164, 0.003937959671020508, 0.00397181510925293, 0.003960847854614258, 0.0038993358612060547, 0.0039141178131103516, 0.0038979053497314453, 0.003931999206542969, 0.0039806365966796875, 0.0038940906524658203, 0.0039348602294921875, 0.0038940906524658203, 0.00393986701965332, 0.004133462905883789, 0.0039014816284179688, 0.003932952880859375, 0.0038962364196777344, 0.003938436508178711, 0.004028797149658203, 0.003912210464477539, 0.003936767578125, 0.003943443298339844, 0.0041081905364990234, 0.004313468933105469, 0.004458427429199219, 0.004291534423828125, 0.0042226314544677734, 0.0042955875396728516, 0.004261493682861328, 0.005339860916137695, 0.004807949066162109, 0.004251956939697266, 0.004960060119628906, 0.005827426910400391, 0.005189180374145508, 0.005211830139160156, 0.005044460296630859, 0.0051364898681640625, 0.005103111267089844, 0.005150794982910156, 0.0051157474517822266, 0.005104780197143555, 0.005161762237548828, 0.005279541015625, 0.005289554595947266, 0.005155801773071289, 0.004214286804199219, 0.0040967464447021484, 0.004350900650024414, 0.005244255065917969, 0.005796194076538086, 0.0040988922119140625, 0.004034519195556641, 0.0039653778076171875, 0.003969430923461914, 0.004015922546386719, 0.0039789676666259766, 0.004041910171508789, 0.00396275520324707, 0.0039043426513671875, 0.003978252410888672, 0.003947257995605469, 0.004052639007568359, 0.003951549530029297, 0.003935813903808594, 0.0038967132568359375, 0.003957271575927734, 0.003983020782470703, 0.003960847854614258, 0.00394129753112793, 0.0038971900939941406, 0.003942012786865234, 0.0038962364196777344, 0.0039975643157958984, 0.004029273986816406, 0.0038988590240478516, 0.00396275520324707, 0.00412440299987793, 0.00403594970703125, 0.003946781158447266, 0.004072427749633789, 0.004006385803222656, 0.003935575485229492, 0.004012107849121094, 0.0039501190185546875, 0.003901958465576172, 0.003930807113647461, 0.003917217254638672, 0.003991842269897461, 0.003930568695068359, 0.003906726837158203, 0.003959178924560547, 0.0039098262786865234, 0.003969430923461914, 0.003992557525634766, 0.0039048194885253906, 0.00393223762512207, 0.003914356231689453, 0.0039637088775634766, 0.0040056705474853516, 0.003912448883056641, 0.003941774368286133, 0.003963470458984375, 0.003965139389038086, 0.004004001617431641, 0.0039031505584716797, 0.003954172134399414, 0.0039136409759521484, 0.004019498825073242, 0.003991127014160156, 0.003972291946411133, 0.003964424133300781, 0.003998279571533203, 0.003968238830566406, 0.00395512580871582, 0.003960609436035156, 0.003942728042602539, 0.003918170928955078, 0.003968477249145508, 0.003968238830566406, 0.0040187835693359375, 0.0039403438568115234, 0.0039522647857666016, 0.0039691925048828125, 0.003941059112548828, 0.004076957702636719, 0.003968238830566406, 0.003939628601074219, 0.003945827484130859, 0.003969430923461914, 0.003976583480834961, 0.0039408206939697266, 0.003923177719116211, 0.003954410552978516, 0.003947734832763672, 0.003942728042602539, 0.0039556026458740234, 0.003926992416381836, 0.003962039947509766, 0.0039937496185302734, 0.0039501190185546875, 0.00395655632019043, 0.003934383392333984, 0.003949642181396484, 0.0039522647857666016, 0.003928184509277344, 0.003901243209838867, 0.003984928131103516, 0.0039522647857666016, 0.003952741622924805, 0.003929853439331055, 0.003911256790161133, 0.003972053527832031, 0.0039539337158203125, 0.0039441585540771484, 0.00397038459777832, 0.003957033157348633, 0.003954887390136719, 0.0040683746337890625, 0.004007577896118164, 0.00396728515625, 0.003908634185791016, 0.003975868225097656, 0.004087924957275391, 0.003972530364990234, 0.00398564338684082, 0.003921031951904297, 0.003963947296142578, 0.003972530364990234, 0.0039484500885009766, 0.0039730072021484375, 0.0039234161376953125, 0.00397491455078125, 0.003983497619628906, 0.0039713382720947266, 0.003937244415283203, 0.0040209293365478516, 0.003950595855712891, 0.003930807113647461, 0.004001617431640625, 0.003961801528930664, 0.003958940505981445, 0.003945827484130859, 0.003950595855712891, 0.003960847854614258, 0.003939151763916016, 0.003911495208740234, 0.003966808319091797, 0.003984928131103516, 0.0039942264556884766, 0.003941535949707031, 0.0039098262786865234, 0.003974437713623047, 0.003968238830566406, 0.003973484039306641, 0.0039823055267333984, 0.003883838653564453, 0.0039560794830322266, 0.003955364227294922, 0.003958463668823242, 0.0039637088775634766, 0.003970146179199219, 0.0040280818939208984, 0.003965616226196289, 0.003942966461181641, 0.003989219665527344, 0.0039136409759521484, 0.0039637088775634766, 0.0039293766021728516, 0.004053354263305664, 0.004081249237060547, 0.0039234161376953125, 0.00398707389831543, 0.004018306732177734, 0.0039691925048828125, 0.004010677337646484, 0.003917217254638672, 0.003950834274291992, 0.003923892974853516, 0.003975629806518555, 0.003957509994506836, 0.004019498825073242, 0.0039899349212646484, 0.003948688507080078, 0.003975868225097656, 0.00396418571472168, 0.004109621047973633, 0.003976345062255859, 0.003948211669921875, 0.00397181510925293, 0.003976583480834961, 0.00397181510925293, 0.003971576690673828, 0.0039250850677490234, 0.003966808319091797, 0.003963470458984375, 0.003974199295043945, 0.004118442535400391, 0.003945827484130859, 0.004133701324462891, 0.003983974456787109, 0.003984689712524414, 0.003972053527832031, 0.003965854644775391, 0.003958225250244141, 0.003957509994506836, 0.003964900970458984, 0.003982067108154297, 0.0039615631103515625, 0.003954887390136719, 0.003928661346435547, 0.003945112228393555, 0.003983736038208008, 0.003983736038208008, 0.003973245620727539, 0.003926515579223633, 0.003992795944213867, 0.003971099853515625, 0.003975868225097656, 0.004031658172607422, 0.00394892692565918, 0.00395655632019043, 0.00398707389831543, 0.004042863845825195, 0.004007101058959961, 0.003923654556274414, 0.003947257995605469, 0.004023313522338867, 0.0039517879486083984, 0.003981590270996094, 0.003940582275390625, 0.00397181510925293, 0.004024505615234375, 0.003937244415283203, 0.003985404968261719, 0.0039215087890625, 0.003953218460083008, 0.003980875015258789, 0.003983974456787109, 0.004015684127807617, 0.003960609436035156, 0.004003286361694336, 0.004011869430541992, 0.004023075103759766, 0.004004955291748047, 0.003964662551879883, 0.004016876220703125, 0.0040400028228759766, 0.004024982452392578, 0.004014492034912109, 0.0040187835693359375, 0.004014015197753906, 0.004072904586791992, 0.003973245620727539, 0.003997325897216797, 0.003985404968261719, 0.004033803939819336, 0.004091501235961914, 0.0039751529693603516, 0.0040209293365478516, 0.003991127014160156, 0.004022121429443359, 0.004063129425048828, 0.003988981246948242, 0.004003047943115234, 0.003961324691772461, 0.004014492034912109, 0.0040547847747802734, 0.004001617431640625, 0.004128694534301758, 0.004010200500488281, 0.004035234451293945, 0.004056692123413086, 0.004062652587890625, 0.004041433334350586, 0.0039615631103515625, 0.0040073394775390625, 0.004057407379150391, 0.0039861202239990234, 0.004058361053466797, 0.003967761993408203, 0.003996133804321289, 0.004097700119018555, 0.003991127014160156, 0.004029512405395508, 0.0039865970611572266, 0.004068851470947266, 0.0040128231048583984, 0.004031181335449219, 0.004006862640380859, 0.0039598941802978516, 0.0040628910064697266, 0.004043102264404297, 0.003988027572631836, 0.004004240036010742, 0.0039560794830322266, 0.004062652587890625, 0.004010200500488281, 0.003976106643676758, 0.004008054733276367, 0.003965139389038086, 0.004064321517944336, 0.004030942916870117, 0.003990650177001953, 0.00400853157043457, 0.0039784908294677734, 0.0040585994720458984, 0.0040247440338134766, 0.004010438919067383, 0.004047393798828125, 0.00399470329284668, 0.004097700119018555, 0.004044055938720703, 0.003996133804321289, 0.0040509700775146484, 0.003981828689575195, 0.0040552616119384766, 0.00404810905456543, 0.004011869430541992, 0.004105806350708008, 0.004050493240356445, 0.004056453704833984, 0.004100799560546875, 0.004067897796630859, 0.0040378570556640625, 0.0040378570556640625, 0.004025459289550781, 0.004014015197753906, 0.00397944450378418, 0.004041194915771484, 0.004090547561645508, 0.004025459289550781, 0.004033803939819336, 0.003992319107055664, 0.004050016403198242, 0.004021167755126953, 0.0040590763092041016, 0.0040531158447265625, 0.004002809524536133, 0.0040493011474609375, 0.00400996208190918, 0.00402069091796875, 0.004027128219604492, 0.003983974456787109, 0.0040438175201416016, 0.004014015197753906, 0.0040128231048583984, 0.004072904586791992, 0.003995180130004883, 0.0040435791015625, 0.004010200500488281, 0.004037618637084961, 0.0040302276611328125, 0.003991127014160156, 0.004052400588989258, 0.004029512405395508, 0.00400996208190918, 0.004035234451293945, 0.0040378570556640625, 0.004004955291748047, 0.004042148590087891, 0.004012584686279297, 0.004058122634887695, 0.004042625427246094, 0.003999471664428711, 0.004021406173706055, 0.004022836685180664, 0.004019498825073242, 0.0040705204010009766, 0.004093170166015625, 0.0040209293365478516, 0.004042148590087891, 0.004042148590087891, 0.004141807556152344, 0.003972291946411133, 0.004080533981323242, 0.004013538360595703, 0.0040433406829833984, 0.0040318965911865234, 0.003996849060058594, 0.004033565521240234, 0.0040357112884521484, 0.004044771194458008, 0.004063606262207031, 0.004049539566040039, 0.004026889801025391, 0.004008769989013672, 0.004041910171508789, 0.0040242671966552734, 0.0040013790130615234, 0.004025936126708984, 0.004022359848022461, 0.004041194915771484, 0.004113435745239258, 0.003997087478637695, 0.004045248031616211, 0.0040683746337890625, 0.003998994827270508, 0.004028797149658203, 0.003998517990112305, 0.004015922546386719, 0.004060029983520508, 0.0039975643157958984, 0.0040318965911865234, 0.003992557525634766, 0.0040323734283447266, 0.004074811935424805, 0.004001140594482422, 0.004034996032714844, 0.0039865970611572266, 0.004042387008666992, 0.00406956672668457, 0.003994464874267578, 0.00403141975402832, 0.0039920806884765625, 0.004024982452392578, 0.0040891170501708984, 0.004014015197753906, 0.004033565521240234, 0.004094362258911133, 0.0040988922119140625, 0.00402069091796875, 0.004079103469848633, 0.004037141799926758, 0.004002094268798828, 0.0040667057037353516, 0.0040378570556640625, 0.004000186920166016, 0.004045963287353516, 0.0039958953857421875, 0.00413823127746582, 0.004032611846923828, 0.00400543212890625, 0.0040476322174072266, 0.004054069519042969, 0.004212617874145508, 0.0040569305419921875, 0.004018068313598633, 0.004060506820678711, 0.004065990447998047, 0.004056215286254883, 0.00403285026550293, 0.004033803939819336, 0.004067659378051758, 0.00406336784362793, 0.004057884216308594, 0.0041158199310302734, 0.0040187835693359375, 0.004059314727783203, 0.004045963287353516, 0.004060506820678711, 0.004033803939819336, 0.003996610641479492, 0.004098415374755859, 0.004065275192260742, 0.00403285026550293, 0.004035234451293945, 0.004057168960571289, 0.004011631011962891, 0.004044771194458008, 0.004059553146362305, 0.004041194915771484, 0.004063606262207031, 0.004002571105957031, 0.004064798355102539, 0.004045724868774414, 0.004043102264404297, 0.004073619842529297, 0.00410008430480957, 0.004052639007568359, 0.004033565521240234, 0.0041484832763671875, 0.004059791564941406, 0.004017353057861328, 0.004030466079711914, 0.004029273986816406, 0.004057407379150391, 0.004038333892822266, 0.004033803939819336, 0.004057884216308594, 0.004052162170410156, 0.004055500030517578, 0.00403904914855957, 0.004029273986816406, 0.004042387008666992, 0.004077911376953125, 0.004013776779174805, 0.004042863845825195, 0.004053831100463867, 0.0040476322174072266, 0.004105091094970703, 0.0040111541748046875, 0.004057407379150391, 0.004012346267700195, 0.004048824310302734, 0.004092693328857422, 0.004004478454589844, 0.004053831100463867, 0.004025936126708984, 0.0041124820709228516, 0.004052877426147461, 0.00400543212890625, 0.004071474075317383, 0.004010200500488281, 0.004095315933227539, 0.004207134246826172, 0.004030704498291016, 0.004045724868774414, 0.00408172607421875, 0.0040569305419921875, 0.004064321517944336, 0.0040073394775390625, 0.004221916198730469, 0.004132986068725586, 0.004062652587890625, 0.004060983657836914, 0.004038095474243164, 0.004155158996582031, 0.004093170166015625, 0.004076242446899414, 0.004051685333251953, 0.004158735275268555, 0.004077434539794922, 0.00405120849609375, 0.0040590763092041016, 0.004056453704833984, 0.004056453704833984, 0.004027366638183594, 0.004062652587890625, 0.0040400028228759766, 0.004055023193359375, 0.0040683746337890625, 0.0040132999420166016, 0.0040552616119384766, 0.004057407379150391, 0.004070758819580078, 0.004048585891723633, 0.004079580307006836, 0.004058361053466797, 0.004055500030517578, 0.0041158199310302734, 0.004076242446899414, 0.004025936126708984, 0.00406336784362793, 0.004062652587890625, 0.0040700435638427734, 0.00405573844909668, 0.004011869430541992, 0.00407862663269043, 0.00410151481628418, 0.004023551940917969, 0.004045963287353516, 0.004025459289550781, 0.0040585994720458984, 0.0041086673736572266, 0.004006147384643555, 0.004057407379150391, 0.004000663757324219, 0.00410151481628418, 0.00405120849609375, 0.004027843475341797, 0.004072666168212891, 0.004024505615234375, 0.004147768020629883, 0.00408482551574707, 0.004033327102661133, 0.004069328308105469, 0.004210710525512695, 0.0041065216064453125, 0.004074573516845703, 0.004117488861083984, 0.004057645797729492, 0.004060506820678711, 0.004073381423950195, 0.0040645599365234375, 0.004019737243652344, 0.004051685333251953, 0.004446268081665039, 0.004095315933227539, 0.004030466079711914, 0.004101753234863281, 0.004012584686279297, 0.0040700435638427734, 0.0040509700775146484, 0.0041027069091796875, 0.00406956672668457, 0.0040187835693359375, 0.0040819644927978516, 0.0040607452392578125, 0.004056692123413086, 0.004070758819580078, 0.004009246826171875, 0.004068613052368164, 0.004072666168212891, 0.00406646728515625, 0.004050254821777344, 0.00461578369140625, 0.004101991653442383, 0.004134178161621094, 0.0040471553802490234, 0.004496574401855469, 0.004025936126708984, 0.004133462905883789, 0.004074811935424805, 0.004021406173706055, 0.004050731658935547, 0.0040624141693115234, 0.004072904586791992, 0.004139423370361328, 0.004036664962768555, 0.004064083099365234, 0.004080057144165039, 0.004068613052368164, 0.004068851470947266, 0.004034996032714844, 0.0040781497955322266, 0.004159212112426758, 0.0040705204010009766, 0.004067420959472656, 0.004099369049072266, 0.004075765609741211, 0.004065990447998047, 0.004118919372558594, 0.004076480865478516, 0.0041332244873046875, 0.004065513610839844, 0.0040585994720458984, 0.004057407379150391, 0.004065275192260742, 0.004076242446899414, 0.00403141975402832, 0.004059553146362305, 0.004069089889526367, 0.0040912628173828125, 0.0040628910064697266, 0.004019737243652344, 0.004052639007568359, 0.00407862663269043, 0.004114866256713867, 0.004107952117919922, 0.0040247440338134766, 0.004060983657836914, 0.004137992858886719, 0.00402069091796875, 0.0040585994720458984, 0.0040857791900634766, 0.004104137420654297, 0.0041790008544921875, 0.004137516021728516, 0.00415492057800293, 0.00406193733215332, 0.0041735172271728516, 0.004111051559448242, 0.004076957702636719, 0.00410771369934082, 0.0041103363037109375, 0.004148006439208984, 0.004120826721191406, 0.004119396209716797, 0.004118919372558594, 0.004115581512451172, 0.0041162967681884766, 0.004119157791137695, 0.004077434539794922, 0.004154205322265625, 0.004213094711303711, 0.004109621047973633, 0.0041408538818359375, 0.004220247268676758, 0.004099607467651367, 0.004130125045776367, 0.004122734069824219, 0.004142284393310547, 0.004128217697143555, 0.00408625602722168, 0.0041124820709228516, 0.004229307174682617, 0.004095792770385742, 0.004141092300415039, 0.004097461700439453, 0.0043184757232666016, 0.004164218902587891, 0.004091978073120117, 0.004132747650146484, 0.0041348934173583984, 0.004129171371459961, 0.004144906997680664, 0.004085063934326172, 0.004149675369262695, 0.0041370391845703125, 0.004144430160522461, 0.004117012023925781, 0.004128932952880859, 0.004150867462158203, 0.004143476486206055, 0.004134654998779297, 0.004118442535400391, 0.004171848297119141, 0.0041103363037109375, 0.0041468143463134766, 0.004118919372558594, 0.004153728485107422, 0.004136085510253906, 0.004078388214111328, 0.00411534309387207, 0.004168510437011719, 0.0040781497955322266, 0.004119873046875, 0.004077911376953125, 0.00415492057800293, 0.0041201114654541016, 0.004092216491699219, 0.004111051559448242, 0.004163980484008789, 0.004201173782348633, 0.004105091094970703, 0.0041637420654296875, 0.004137992858886719, 0.004118919372558594, 0.004134416580200195, 0.0041294097900390625, 0.004068613052368164, 0.00414276123046875, 0.0041332244873046875, 0.004126310348510742, 0.004118442535400391, 0.0041310787200927734, 0.0041120052337646484, 0.004141569137573242, 0.00411224365234375, 0.004143714904785156, 0.004164695739746094, 0.004082202911376953, 0.0041124820709228516, 0.0041201114654541016, 0.00413823127746582, 0.004121541976928711, 0.004099130630493164, 0.0041387081146240234, 0.004156589508056641, 0.004077911376953125, 0.0041234493255615234, 0.004101991653442383, 0.0041828155517578125, 0.004147529602050781, 0.004090785980224609, 0.004137992858886719, 0.004114866256713867, 0.004122734069824219, 0.004107236862182617, 0.004088640213012695, 0.004175662994384766, 0.004160165786743164, 0.004156351089477539, 0.004126310348510742, 0.004102945327758789, 0.004133939743041992, 0.004133462905883789, 0.0041272640228271484, 0.004118919372558594, 0.004121541976928711, 0.004088878631591797, 0.004236698150634766, 0.004124879837036133, 0.004225969314575195, 0.0041387081146240234, 0.004075765609741211, 0.0041332244873046875, 0.004175901412963867, 0.004088878631591797, 0.00412750244140625, 0.0040988922119140625, 0.004179716110229492, 0.004139900207519531, 0.004110097885131836, 0.0041103363037109375, 0.004119396209716797, 0.00412297248840332, 0.00414586067199707, 0.004670381546020508, 0.004226207733154297, 0.0041522979736328125, 0.004125118255615234, 0.0046443939208984375, 0.004220247268676758, 0.004233837127685547, 0.00424504280090332, 0.004254817962646484, 0.004239559173583984, 0.004191160202026367, 0.0041468143463134766, 0.004145622253417969, 0.004133462905883789, 0.0041010379791259766, 0.004156827926635742, 0.00418543815612793, 0.004099130630493164, 0.00415492057800293, 0.004124879837036133, 0.0041656494140625, 0.004131317138671875, 0.004109859466552734, 0.004131793975830078, 0.0041620731353759766, 0.004140377044677734, 0.00412440299987793, 0.004082679748535156, 0.00416254997253418, 0.004132986068725586, 0.004117250442504883, 0.0043332576751708984, 0.004255533218383789, 0.004324674606323242, 0.004252910614013672, 0.0042645931243896484, 0.0043141841888427734, 0.0042340755462646484, 0.004260540008544922, 0.004320859909057617, 0.004179716110229492, 0.004161834716796875, 0.0041353702545166016, 0.004161357879638672, 0.0041768550872802734, 0.004208564758300781, 0.004340171813964844, 0.004140615463256836, 0.004162788391113281, 0.004178047180175781, 0.004143953323364258, 0.0041620731353759766, 0.004106760025024414, 0.004140377044677734, 0.0041654109954833984, 0.004170894622802734, 0.0041310787200927734, 0.004102230072021484, 0.004208087921142578, 0.004239320755004883, 0.00410151481628418, 0.004160165786743164, 0.004134416580200195, 0.004203081130981445, 0.004157304763793945, 0.0040934085845947266, 0.0041294097900390625, 0.00410914421081543, 0.004174947738647461, 0.004172086715698242, 0.004105806350708008, 0.004126548767089844, 0.0041751861572265625, 0.0041387081146240234, 0.004159688949584961, 0.00411224365234375, 0.004152774810791016, 0.004135847091674805, 0.004163265228271484, 0.004244565963745117, 0.004152059555053711, 0.004186868667602539, 0.0041408538818359375, 0.004155874252319336, 0.0042459964752197266, 0.004148960113525391, 0.004092693328857422, 0.004156351089477539, 0.004197359085083008, 0.004136562347412109, 0.004157066345214844, 0.004094123840332031, 0.004210472106933594, 0.004132747650146484, 0.004114389419555664, 0.004152774810791016, 0.0041351318359375, 0.004174232482910156, 0.004158973693847656, 0.004118919372558594, 0.0041539669036865234, 0.004125833511352539, 0.004147529602050781, 0.004137516021728516, 0.004148244857788086, 0.004128217697143555, 0.0042171478271484375, 0.0041658878326416016, 0.004177570343017578, 0.004166603088378906, 0.004105329513549805, 0.0041484832763671875, 0.004194974899291992, 0.004125833511352539, 0.004273176193237305, 0.004131793975830078, 0.004186153411865234, 0.004166126251220703, 0.0041043758392333984, 0.0041446685791015625, 0.004201173782348633, 0.0041353702545166016, 0.004136085510253906, 0.004129171371459961, 0.00416254997253418, 0.0041539669036865234, 0.004141330718994141, 0.004273653030395508, 0.004191398620605469, 0.0042002201080322266, 0.0041577816009521484, 0.004139900207519531, 0.0041484832763671875, 0.0041353702545166016, 0.004105567932128906, 0.0041620731353759766, 0.0042264461517333984, 0.004098653793334961, 0.004209995269775391, 0.004141807556152344, 0.004201173782348633, 0.004139900207519531, 0.004270076751708984, 0.004167079925537109, 0.00417780876159668, 0.004145622253417969, 0.004146575927734375, 0.004102468490600586, 0.004175662994384766, 0.004156589508056641, 0.004160642623901367, 0.004150867462158203, 0.004192352294921875, 0.004148960113525391, 0.004141807556152344, 0.004158496856689453, 0.00414276123046875, 0.004171609878540039, 0.004124641418457031, 0.0041391849517822266, 0.004218339920043945, 0.004122257232666016, 0.004153013229370117, 0.0041272640228271484, 0.004161834716796875, 0.004200935363769531, 0.004113435745239258, 0.004160642623901367, 0.004109621047973633, 0.004229545593261719, 0.004164457321166992, 0.0041277408599853516, 0.004152774810791016, 0.004201173782348633, 0.004175901412963867, 0.0042378902435302734, 0.004201412200927734, 0.004261016845703125, 0.004128694534301758, 0.0041849613189697266, 0.0041501522064208984, 0.004157543182373047, 0.004148960113525391, 0.0041141510009765625, 0.004199028015136719, 0.004166126251220703, 0.004146575927734375, 0.0041348934173583984, 0.004133462905883789, 0.004168272018432617, 0.004139900207519531, 0.004154205322265625, 0.004124641418457031, 0.004155635833740234, 0.0041239261627197266, 0.004148244857788086, 0.004180908203125, 0.00415492057800293, 0.004143476486206055, 0.00413823127746582, 0.004144430160522461, 0.004201412200927734, 0.004110574722290039, 0.004148960113525391, 0.004117488861083984, 0.004195451736450195, 0.004148244857788086, 0.004124641418457031, 0.004171133041381836, 0.004175662994384766, 0.004168033599853516, 0.004165172576904297, 0.0041158199310302734, 0.004199504852294922, 0.0041697025299072266, 0.004158735275268555, 0.004166603088378906, 0.0044403076171875, 0.004191875457763672, 0.004170417785644531, 0.00419163703918457, 0.004171848297119141, 0.004161834716796875, 0.004197359085083008, 0.0042684078216552734, 0.004245758056640625, 0.004141569137573242, 0.0041697025299072266, 0.0041692256927490234, 0.004174709320068359, 0.0041697025299072266, 0.004129886627197266, 0.00414276123046875, 0.004173994064331055, 0.004177093505859375, 0.004198789596557617, 0.004149913787841797, 0.004163026809692383, 0.004174709320068359, 0.004186868667602539, 0.004227638244628906, 0.004148006439208984, 0.004104137420654297, 0.004153728485107422, 0.004258394241333008, 0.0041239261627197266, 0.0041429996490478516, 0.00412750244140625, 0.004197597503662109, 0.0041811466217041016, 0.004124879837036133, 0.004161357879638672, 0.0041539669036865234, 0.00417780876159668, 0.004180908203125, 0.004125356674194336, 0.00415492057800293, 0.004157066345214844, 0.004168272018432617, 0.004149913787841797, 0.004167318344116211, 0.0041370391845703125, 0.004166364669799805, 0.004167079925537109, 0.004216194152832031, 0.004159212112426758, 0.004120588302612305, 0.00416254997253418, 0.004209280014038086, 0.0041391849517822266, 0.004148960113525391, 0.0042438507080078125, 0.004246711730957031, 0.004212141036987305, 0.004190206527709961, 0.00416874885559082, 0.004252910614013672, 0.00420069694519043, 0.0041713714599609375, 0.004210233688354492, 0.0042877197265625, 0.00418400764465332, 0.004222393035888672, 0.004182338714599609, 0.004265308380126953, 0.004197835922241211, 0.004210710525512695, 0.004221916198730469, 0.0042574405670166016, 0.0042302608489990234, 0.004225015640258789, 0.004229068756103516, 0.004183053970336914, 0.00420689582824707, 0.004244804382324219, 0.0042417049407958984, 0.004227876663208008, 0.004198789596557617, 0.004455089569091797, 0.0042798519134521484, 0.0041904449462890625, 0.0042057037353515625, 0.004214048385620117, 0.004201412200927734, 0.004189014434814453, 0.0041730403900146484, 0.004209756851196289, 0.0042133331298828125, 0.004229545593261719, 0.0042188167572021484, 0.004200458526611328, 0.004173755645751953, 0.004220008850097656, 0.004242420196533203, 0.004212141036987305, 0.004215717315673828, 0.0041844844818115234, 0.00429844856262207, 0.004239559173583984, 0.004303455352783203, 0.004225730895996094, 0.004314422607421875, 0.0042116641998291016, 0.004239320755004883, 0.0042400360107421875, 0.004246950149536133, 0.004226207733154297, 0.00421905517578125, 0.004236936569213867, 0.004227876663208008, 0.004183053970336914, 0.004263162612915039, 0.004309654235839844, 0.0041844844818115234, 0.004219770431518555, 0.0041828155517578125, 0.004297971725463867, 0.00421595573425293, 0.004170656204223633, 0.00422358512878418, 0.004224061965942383, 0.004206657409667969, 0.00422215461730957, 0.004229068756103516, 0.004203081130981445, 0.004228353500366211, 0.004236698150634766, 0.004239082336425781, 0.004221677780151367, 0.004198551177978516, 0.004227638244628906, 0.004266977310180664, 0.004179716110229492, 0.004228353500366211, 0.004189491271972656, 0.004278659820556641, 0.004212856292724609, 0.004308223724365234, 0.004265308380126953, 0.004255533218383789, 0.004213571548461914, 0.004208564758300781, 0.004236936569213867, 0.004187822341918945, 0.004234790802001953, 0.004215240478515625, 0.004370689392089844, 0.004235982894897461, 0.0042934417724609375, 0.004286289215087891, 0.0042226314544677734, 0.004199504852294922, 0.004222869873046875, 0.004225015640258789, 0.00421452522277832, 0.0042133331298828125, 0.0041811466217041016, 0.0043337345123291016, 0.00423741340637207, 0.0042438507080078125, 0.00438237190246582, 0.004248619079589844, 0.004186868667602539, 0.004214763641357422, 0.004273414611816406, 0.004185914993286133, 0.004202365875244141, 0.004190683364868164, 0.004275083541870117, 0.004232883453369141, 0.004181385040283203, 0.004223346710205078, 0.004544496536254883, 0.004237174987792969, 0.00423884391784668, 0.004219532012939453, 0.004215717315673828, 0.004218578338623047, 0.004308223724365234, 0.004219532012939453, 0.0042247772216796875, 0.00418853759765625, 0.004286289215087891, 0.00421905517578125, 0.004218339920043945, 0.0042552947998046875, 0.004271984100341797, 0.004227161407470703, 0.0042231082916259766, 0.0042040348052978516, 0.0042111873626708984, 0.004231691360473633, 0.004251003265380859, 0.00434422492980957, 0.004237651824951172, 0.004266500473022461, 0.00422978401184082, 0.0043032169342041016, 0.004209280014038086, 0.0042383670806884766, 0.004224538803100586, 0.004240512847900391, 0.0042285919189453125, 0.004190206527709961, 0.004282474517822266, 0.004262447357177734, 0.004230976104736328, 0.004228115081787109, 0.004242420196533203, 0.004193305969238281, 0.004232645034790039, 0.00422358512878418, 0.0042552947998046875, 0.0042572021484375, 0.004217863082885742, 0.004299163818359375, 0.004230976104736328, 0.0042040348052978516, 0.004253387451171875, 0.0042536258697509766, 0.004231452941894531, 0.004231452941894531, 0.004206418991088867, 0.0042994022369384766, 0.00423431396484375, 0.004224300384521484, 0.004270076751708984, 0.00423884391784668, 0.004235744476318359, 0.00425410270690918, 0.0042607784271240234, 0.004192352294921875, 0.004228115081787109, 0.0042476654052734375, 0.004245281219482422, 0.004261970520019531, 0.004196882247924805, 0.0042760372161865234, 0.00424647331237793, 0.0042209625244140625, 0.004322528839111328, 0.004241228103637695, 0.00431370735168457, 0.004258155822753906, 0.004255771636962891, 0.004228115081787109, 0.004231691360473633, 0.004217386245727539, 0.004244565963745117, 0.004230976104736328, 0.004197597503662109, 0.0042438507080078125, 0.0042819976806640625, 0.0042209625244140625, 0.004247426986694336, 0.004238128662109375, 0.004256486892700195, 0.004252195358276367, 0.00420069694519043, 0.004260063171386719, 0.004265785217285156, 0.004223346710205078, 0.00425410270690918, 0.004267454147338867, 0.0042514801025390625, 0.004251241683959961, 0.004308462142944336, 0.004213809967041016, 0.0042455196380615234, 0.004235029220581055, 0.004307270050048828, 0.00424647331237793, 0.0042226314544677734, 0.004265785217285156, 0.004252195358276367, 0.004216432571411133, 0.004235267639160156, 0.004254341125488281, 0.0042116641998291016, 0.004244089126586914, 0.004250764846801758, 0.004269123077392578, 0.00423741340637207, 0.004209280014038086, 0.004248380661010742, 0.004283905029296875, 0.004201173782348633, 0.0042688846588134766, 0.0044672489166259766, 0.004347324371337891, 0.004248380661010742, 0.004198789596557617, 0.00426030158996582, 0.004235029220581055, 0.004974842071533203, 0.004308223724365234, 0.004319906234741211, 0.004209280014038086, 0.004251956939697266, 0.004269123077392578, 0.004254817962646484, 0.004277944564819336, 0.004326343536376953, 0.004244327545166016, 0.004242897033691406, 0.0042972564697265625, 0.0042057037353515625, 0.00423884391784668, 0.004354953765869141, 0.004343509674072266, 0.004268169403076172, 0.004270792007446289, 0.004278421401977539, 0.0043849945068359375, 0.004334449768066406, 0.004315614700317383, 0.004275321960449219, 0.004233121871948242, 0.004262447357177734, 0.004283905029296875, 0.00426793098449707, 0.004239082336425781, 0.004215240478515625, 0.00430607795715332, 0.004277229309082031, 0.004252433776855469, 0.004257678985595703, 0.004249095916748047, 0.004213094711303711, 0.0042612552642822266, 0.004317283630371094, 0.004200935363769531, 0.004266500473022461, 0.0042285919189453125, 0.004393577575683594, 0.00427699089050293, 0.004305601119995117, 0.00426793098449707, 0.0042572021484375, 0.004297018051147461, 0.004275083541870117, 0.004258155822753906, 0.004212856292724609, 0.004262685775756836, 0.004317760467529297, 0.0042226314544677734, 0.0042378902435302734, 0.004233598709106445, 0.004330158233642578, 0.004251003265380859, 0.0042285919189453125, 0.004273891448974609, 0.004329681396484375, 0.00429987907409668, 0.004285573959350586, 0.0042760372161865234, 0.004270792007446289, 0.004257678985595703, 0.004303932189941406, 0.0042264461517333984, 0.004254817962646484, 0.0042383670806884766, 0.004319429397583008, 0.004270076751708984, 0.004230499267578125, 0.004266262054443359, 0.004256010055541992, 0.004248380661010742, 0.004278421401977539, 0.004281520843505859, 0.004207134246826172, 0.004262685775756836, 0.004270792007446289, 0.004270315170288086, 0.004286766052246094, 0.004211902618408203, 0.005009174346923828, 0.0044972896575927734, 0.0042841434478759766, 0.004325389862060547, 0.004875898361206055, 0.004353046417236328, 0.004386186599731445, 0.00438690185546875, 0.004476070404052734, 0.004312276840209961, 0.00428318977355957, 0.004300832748413086, 0.004343748092651367, 0.004302501678466797, 0.0042417049407958984, 0.004377126693725586, 0.004301309585571289, 0.004273176193237305, 0.004265785217285156, 0.004511117935180664, 0.0042650699615478516, 0.004255056381225586, 0.004343509674072266, 0.0042877197265625, 0.0042688846588134766, 0.004218101501464844, 0.004300832748413086, 0.00435328483581543, 0.0042629241943359375, 0.004368305206298828, 0.0042476654052734375, 0.004351139068603516, 0.0042760372161865234, 0.004319190979003906, 0.004313230514526367, 0.004293680191040039, 0.0043146610260009766, 0.00423741340637207, 0.004350423812866211, 0.00422978401184082, 0.0042989253997802734, 0.004304647445678711, 0.004317283630371094, 0.004288911819458008, 0.004248619079589844, 0.004261016845703125, 0.004400968551635742, 0.004289388656616211, 0.004243373870849609, 0.004287242889404297, 0.004361152648925781, 0.004391193389892578, 0.004293680191040039, 0.004396915435791016, 0.004241943359375, 0.004269123077392578, 0.004221916198730469, 0.004282712936401367, 0.004278898239135742, 0.004257917404174805, 0.004287242889404297, 0.004260540008544922, 0.004259824752807617, 0.004297018051147461, 0.004342794418334961, 0.004240512847900391, 0.004266977310180664, 0.0042877197265625, 0.004286289215087891, 0.004273653030395508, 0.004231929779052734, 0.004302024841308594, 0.004304170608520508, 0.004274129867553711, 0.004282951354980469, 0.004281520843505859, 0.004274129867553711, 0.0042743682861328125, 0.004340410232543945, 0.00426173210144043, 0.0042765140533447266, 0.0043179988861083984, 0.0043261051177978516, 0.004289388656616211, 0.004235982894897461, 0.0042951107025146484, 0.00427699089050293, 0.004389286041259766, 0.004325389862060547, 0.0042896270751953125, 0.004256486892700195, 0.00430607795715332, 0.004359006881713867, 0.004288673400878906, 0.004314899444580078, 0.0043218135833740234, 0.004296064376831055, 0.004310131072998047, 0.00442957878112793, 0.0043604373931884766, 0.0043294429779052734, 0.004304647445678711, 0.004318952560424805, 0.004326581954956055, 0.0042722225189208984, 0.004354953765869141, 0.004323482513427734, 0.0042724609375, 0.004299640655517578, 0.004316568374633789, 0.004317283630371094, 0.004316091537475586, 0.004334688186645508, 0.0043163299560546875, 0.004307270050048828, 0.0043485164642333984, 0.004265546798706055, 0.0043299198150634766, 0.004263877868652344, 0.0043659210205078125, 0.004311323165893555, 0.004262685775756836, 0.004323005676269531, 0.00431513786315918, 0.004315376281738281, 0.004324197769165039, 0.004307985305786133, 0.004285573959350586, 0.004347562789916992, 0.004354000091552734, 0.00433659553527832, 0.0043103694915771484, 0.004324913024902344, 0.0043489933013916016, 0.004322528839111328, 0.00425267219543457, 0.00432133674621582, 0.0043048858642578125, 0.004339933395385742, 0.0043239593505859375, 0.004324197769165039, 0.00426936149597168, 0.004300594329833984, 0.00436854362487793, 0.004381895065307617, 0.004378080368041992, 0.0044748783111572266, 0.0043048858642578125, 0.004341602325439453, 0.004334688186645508, 0.004300117492675781, 0.004344463348388672, 0.004394054412841797, 0.004328250885009766, 0.004312992095947266, 0.004322052001953125, 0.004361629486083984, 0.004372835159301758, 0.0042765140533447266, 0.004323005676269531, 0.004296064376831055, 0.004309177398681641, 0.0043354034423828125, 0.0043294429779052734, 0.004263401031494141, 0.0042934417724609375, 0.004358768463134766, 0.00428318977355957, 0.004309892654418945, 0.0043506622314453125, 0.0043184757232666016, 0.004310131072998047, 0.004297971725463867, 0.004324674606323242, 0.004333972930908203, 0.004309892654418945, 0.0043697357177734375, 0.004334926605224609, 0.0042760372161865234, 0.004370212554931641, 0.004326343536376953, 0.004256010055541992, 0.004317522048950195, 0.004363298416137695, 0.004309654235839844, 0.004318714141845703, 0.0043261051177978516, 0.004270076751708984, 0.004372835159301758, 0.004382133483886719, 0.004297733306884766, 0.004432201385498047, 0.004395008087158203, 0.004381895065307617, 0.004315614700317383, 0.00434112548828125, 0.004372596740722656, 0.004324674606323242, 0.004322528839111328, 0.004345893859863281, 0.004349946975708008, 0.004309415817260742, 0.004340171813964844, 0.004365682601928711, 0.0042841434478759766, 0.004321575164794922, 0.0043528079986572266, 0.004319667816162109, 0.004369258880615234, 0.004327535629272461, 0.004322528839111328, 0.004311084747314453, 0.004332304000854492, 0.004362344741821289, 0.004323005676269531, 0.004324913024902344, 0.004404544830322266, 0.004372596740722656, 0.004303693771362305, 0.004328489303588867, 0.0043375492095947266, 0.004317522048950195, 0.0043430328369140625, 0.004338502883911133, 0.00428462028503418, 0.004324913024902344, 0.004380941390991211, 0.004274606704711914, 0.004345417022705078, 0.004392147064208984, 0.004336833953857422, 0.004327297210693359, 0.00431513786315918, 0.004323482513427734, 0.004348039627075195, 0.004350185394287109, 0.0043413639068603516, 0.0044019222259521484, 0.004395246505737305, 0.004368782043457031, 0.004309415817260742, 0.0042858123779296875, 0.004355669021606445, 0.004359722137451172, 0.004346132278442383, 0.004343271255493164, 0.0043239593505859375, 0.0043735504150390625, 0.004331827163696289, 0.004367351531982422, 0.004422426223754883, 0.004323482513427734, 0.0043277740478515625, 0.004336357116699219, 0.004305362701416016, 0.004279375076293945, 0.004378795623779297, 0.004325389862060547, 0.004332542419433594, 0.0043239593505859375, 0.004422664642333984, 0.004327297210693359, 0.004384279251098633, 0.0043392181396484375, 0.004293680191040039, 0.004347562789916992, 0.004307270050048828, 0.004343986511230469, 0.004340410232543945, 0.004361867904663086, 0.004299163818359375, 0.0043261051177978516, 0.0043790340423583984, 0.004296064376831055, 0.0043392181396484375, 0.00435185432434082, 0.004291057586669922, 0.004335641860961914, 0.0043184757232666016, 0.004361867904663086, 0.004317045211791992, 0.004317283630371094, 0.004354953765869141, 0.004426002502441406, 0.004355669021606445, 0.004402875900268555, 0.004346370697021484, 0.004294395446777344, 0.0043299198150634766, 0.004339456558227539, 0.004334211349487305, 0.004308938980102539, 0.004338741302490234, 0.004317760467529297, 0.0043222904205322266, 0.004396915435791016, 0.00431370735168457, 0.004336357116699219, 0.004313945770263672, 0.0043680667877197266, 0.0044116973876953125, 0.00440526008605957, 0.004375457763671875, 0.004345893859863281, 0.0043222904205322266, 0.004357814788818359, 0.004343986511230469, 0.0043070316314697266, 0.004395484924316406, 0.004315614700317383, 0.0042877197265625, 0.0043332576751708984, 0.004414796829223633, 0.004296302795410156, 0.004341840744018555, 0.0043544769287109375, 0.004337787628173828, 0.004327297210693359, 0.00434565544128418, 0.004297733306884766, 0.00432896614074707, 0.004331111907958984, 0.0043718814849853516, 0.004343748092651367, 0.0043125152587890625, 0.004388332366943359, 0.004352569580078125, 0.0042951107025146484, 0.004356861114501953, 0.004437446594238281, 0.0044519901275634766, 0.004368305206298828, 0.004349946975708008, 0.004324913024902344, 0.004362821578979492, 0.004374027252197266, 0.0042951107025146484, 0.00434112548828125, 0.004353523254394531, 0.004339933395385742, 0.0043544769287109375, 0.0043528079986572266, 0.004317045211791992, 0.0043849945068359375, 0.00488734245300293, 0.004365682601928711, 0.00436091423034668, 0.004412651062011719, 0.004365682601928711, 0.004388332366943359, 0.004397392272949219, 0.0043566226959228516, 0.004326343536376953, 0.004334211349487305, 0.004395723342895508, 0.00435328483581543, 0.004358530044555664, 0.004370689392089844, 0.004310131072998047, 0.004358053207397461, 0.004380226135253906, 0.004319429397583008, 0.004332780838012695, 0.004339456558227539, 0.004354238510131836, 0.004364728927612305, 0.004316568374633789, 0.004395246505737305, 0.004338264465332031, 0.004338264465332031, 0.0043544769287109375, 0.004347324371337891, 0.004306316375732422, 0.004380941390991211, 0.004342555999755859, 0.004431724548339844, 0.004576206207275391, 0.0043866634368896484, 0.0043675899505615234, 0.004329204559326172, 0.00442957878112793, 0.004297971725463867, 0.004381418228149414, 0.004742860794067383, 0.00436091423034668, 0.004359245300292969, 0.004350185394287109, 0.0043182373046875, 0.00437164306640625, 0.0043637752532958984, 0.004338502883911133, 0.004373073577880859, 0.004367828369140625, 0.0043714046478271484, 0.004316568374633789, 0.004369020462036133, 0.004361867904663086, 0.004348278045654297, 0.004369020462036133, 0.004355430603027344, 0.005209922790527344, 0.00449061393737793, 0.004387855529785156, 0.004305124282836914, 0.004709959030151367, 0.004314422607421875, 0.0043370723724365234, 0.00441741943359375, 0.0043201446533203125, 0.004328489303588867, 0.004305124282836914, 0.004426240921020508, 0.00439906120300293, 0.004305601119995117, 0.004383563995361328, 0.004393339157104492, 0.004390716552734375, 0.0043680667877197266, 0.0043408870697021484, 0.004302263259887695, 0.004503965377807617, 0.00449824333190918, 0.004404306411743164, 0.004359722137451172, 0.004378318786621094, 0.004398822784423828, 0.004346609115600586, 0.004355430603027344, 0.004314422607421875, 0.0043408870697021484, 0.0044214725494384766, 0.004339456558227539, 0.004443645477294922, 0.0043811798095703125, 0.004364490509033203, 0.004357576370239258, 0.004384040832519531, 0.004305601119995117, 0.004364490509033203, 0.004405498504638672, 0.0043065547943115234, 0.004357576370239258, 0.004326343536376953, 0.004435300827026367, 0.004361629486083984, 0.004324436187744141, 0.004444122314453125, 0.004396915435791016, 0.004364967346191406, 0.004588127136230469, 0.0044248104095458984, 0.004323005676269531, 0.004442453384399414, 0.0043659210205078125, 0.004336357116699219, 0.004441499710083008, 0.004327535629272461, 0.004365444183349609, 0.004428386688232422, 0.0043408870697021484, 0.004557371139526367, 0.004492521286010742, 0.004373788833618164, 0.004383087158203125, 0.004392862319946289, 0.00432133674621582, 0.004431962966918945, 0.004497051239013672, 0.004338502883911133, 0.004385709762573242, 0.004368782043457031, 0.00442957878112793, 0.00436091423034668, 0.0043795108795166016, 0.0043659210205078125, 0.00433659553527832, 0.004363298416137695, 0.004412412643432617, 0.004353523254394531, 0.004489898681640625, 0.0044019222259521484, 0.0043599605560302734, 0.004335165023803711, 0.00436854362487793, 0.004395484924316406, 0.004363298416137695, 0.0043447017669677734, 0.004373073577880859, 0.004347562789916992, 0.00443720817565918, 0.004426240921020508, 0.004314422607421875, 0.004403352737426758, 0.00437617301940918, 0.004378557205200195, 0.004416227340698242, 0.004359245300292969, 0.004325151443481445, 0.004334449768066406, 0.004430294036865234, 0.004334688186645508, 0.004401445388793945, 0.0043964385986328125, 0.00447845458984375, 0.004398822784423828, 0.0043718814849853516, 0.004410505294799805, 0.004370689392089844, 0.004415035247802734, 0.00440216064453125, 0.004392862319946289, 0.004360675811767578, 0.0046002864837646484, 0.004434108734130859, 0.00435638427734375, 0.00444340705871582, 0.004403352737426758, 0.004409313201904297, 0.004431724548339844, 0.0044040679931640625, 0.004345893859863281, 0.004520416259765625, 0.004400014877319336, 0.004399299621582031, 0.004419088363647461, 0.004401683807373047, 0.004392385482788086, 0.0044193267822265625, 0.004438877105712891, 0.004361152648925781, 0.0044786930084228516, 0.0044062137603759766, 0.0043582916259765625, 0.004416704177856445, 0.004412651062011719, 0.004423618316650391, 0.004396677017211914, 0.004396677017211914, 0.00438237190246582, 0.004418611526489258, 0.004456043243408203, 0.004374265670776367, 0.004389286041259766, 0.004398345947265625, 0.004400491714477539, 0.004428863525390625, 0.004421234130859375, 0.004408836364746094, 0.00442957878112793, 0.004469633102416992, 0.004376411437988281, 0.004418373107910156, 0.0044095516204833984, 0.004459381103515625, 0.004415750503540039, 0.0044367313385009766, 0.004362344741821289, 0.00449371337890625, 0.004580974578857422, 0.004399538040161133, 0.004433155059814453, 0.004414796829223633, 0.004422903060913086, 0.004410266876220703, 0.0044291019439697266, 0.004374265670776367, 0.004380941390991211, 0.004445791244506836, 0.004370689392089844, 0.004419565200805664, 0.004473447799682617, 0.004876375198364258, 0.004538536071777344, 0.004456520080566406, 0.004411220550537109, 0.0047762393951416016, 0.004453420639038086, 0.004405021667480469, 0.0044291019439697266, 0.0044252872467041016, 0.004434347152709961, 0.0044596195220947266, 0.00442194938659668, 0.0044176578521728516, 0.004416465759277344, 0.004502296447753906, 0.004411935806274414, 0.004437685012817383, 0.0044248104095458984, 0.00441431999206543, 0.004395961761474609, 0.004446744918823242, 0.004403829574584961, 0.0044248104095458984, 0.004456758499145508, 0.004385948181152344, 0.004404306411743164, 0.004439115524291992, 0.0044193267822265625, 0.004441261291503906, 0.0044405460357666016, 0.004373311996459961, 0.004599809646606445, 0.0045740604400634766, 0.004367828369140625, 0.004407405853271484, 0.0044710636138916016, 0.00447392463684082, 0.004424095153808594, 0.004476308822631836, 0.004381418228149414, 0.004423856735229492, 0.004494667053222656, 0.004422426223754883, 0.004951000213623047, 0.004509925842285156, 0.004422426223754883, 0.004487037658691406, 0.004445791244506836, 0.0044209957122802734, 0.004451274871826172, 0.004422426223754883, 0.004360198974609375, 0.004482746124267578, 0.004411458969116211, 0.0043888092041015625, 0.0044803619384765625, 0.004441022872924805, 0.004443168640136719, 0.004458904266357422, 0.004418611526489258, 0.0043735504150390625, 0.0044765472412109375, 0.004447221755981445, 0.004394054412841797, 0.004472494125366211, 0.0044248104095458984, 0.0044460296630859375, 0.004428386688232422, 0.004424571990966797, 0.0043718814849853516, 0.004489421844482422, 0.004433870315551758, 0.004386425018310547, 0.0044553279876708984, 0.004431247711181641, 0.004415988922119141, 0.0045986175537109375, 0.004452705383300781, 0.004414081573486328, 0.004479408264160156, 0.0044362545013427734, 0.004483699798583984, 0.0044977664947509766, 0.004409074783325195, 0.0044040679931640625, 0.004439353942871094, 0.004461526870727539, 0.004384756088256836, 0.00447535514831543, 0.0044248104095458984, 0.004382133483886719, 0.0044345855712890625, 0.0044612884521484375, 0.0044155120849609375, 0.0044918060302734375, 0.004434823989868164, 0.004391908645629883, 0.004445075988769531, 0.0044519901275634766, 0.004372596740722656, 0.004438161849975586, 0.00446009635925293, 0.004460573196411133, 0.00441431999206543, 0.004446983337402344, 0.004396200180053711, 0.004439830780029297, 0.004503011703491211, 0.00438237190246582, 0.0044286251068115234, 0.004439115524291992, 0.004518747329711914, 0.00444483757019043, 0.00450444221496582, 0.0043909549713134766, 0.0044405460357666016, 0.00448155403137207, 0.004430532455444336, 0.004409313201904297, 0.004445075988769531, 0.0043697357177734375, 0.00469970703125, 0.004465341567993164, 0.004427194595336914, 0.0044403076171875, 0.004426002502441406, 0.004434347152709961, 0.00446009635925293, 0.004425048828125, 0.004383087158203125, 0.004486560821533203, 0.004494190216064453, 0.004431247711181641, 0.004435539245605469, 0.004597663879394531, 0.00444793701171875, 0.004473447799682617, 0.004438161849975586, 0.004409313201904297, 0.004503011703491211, 0.004452705383300781, 0.004395484924316406, 0.004492998123168945, 0.0045278072357177734, 0.00442957878112793, 0.004470348358154297, 0.004450321197509766, 0.004400491714477539, 0.0045013427734375, 0.004441261291503906, 0.00439453125, 0.004433870315551758, 0.004412412643432617, 0.004450082778930664, 0.004449129104614258, 0.004433631896972656, 0.004408597946166992, 0.004527091979980469, 0.004453420639038086, 0.004400014877319336, 0.0044634342193603516, 0.004462242126464844, 0.004450798034667969, 0.004474163055419922, 0.004422903060913086, 0.004428386688232422, 0.004717588424682617, 0.004483222961425781, 0.004400014877319336, 0.004445075988769531, 0.004428386688232422, 0.00444483757019043, 0.004463911056518555, 0.00444340705871582, 0.004379987716674805, 0.0044972896575927734, 0.004426479339599609, 0.004408359527587891, 0.004475593566894531, 0.004410266876220703, 0.004434347152709961, 0.004476785659790039, 0.004471302032470703, 0.00439453125, 0.004494905471801758, 0.004454135894775391, 0.004397153854370117, 0.00446772575378418, 0.0044133663177490234, 0.004442930221557617, 0.004501819610595703, 0.004472017288208008, 0.004401445388793945, 0.00448155403137207, 0.004427671432495117, 0.004415988922119141, 0.0044672489166259766, 0.0044476985931396484, 0.004438638687133789, 0.004544496536254883, 0.004457712173461914, 0.004433155059814453, 0.004528045654296875, 0.004423856735229492, 0.004420280456542969, 0.004454374313354492, 0.0044193267822265625, 0.004454612731933594, 0.004465818405151367, 0.004432201385498047, 0.004426479339599609, 0.0046155452728271484, 0.004480600357055664, 0.004439592361450195, 0.004456758499145508, 0.004456758499145508, 0.004446983337402344, 0.004471540451049805, 0.004440784454345703, 0.0044078826904296875, 0.0045011043548583984, 0.0044384002685546875, 0.00441431999206543, 0.004454851150512695, 0.004426479339599609, 0.0045316219329833984, 0.004490375518798828, 0.004468441009521484, 0.004487037658691406, 0.004439353942871094, 0.004429340362548828, 0.0044782161712646484, 0.004409313201904297, 0.004441022872924805, 0.0044896602630615234, 0.0044019222259521484, 0.004444122314453125, 0.004445075988769531, 0.004456996917724609, 0.004441976547241211, 0.004441499710083008, 0.004425525665283203, 0.004460334777832031, 0.004499673843383789, 0.004408597946166992, 0.004433393478393555, 0.0044651031494140625, 0.004513263702392578, 0.004663944244384766, 0.004554033279418945, 0.0044591426849365234, 0.004530668258666992, 0.0045795440673828125, 0.004500150680541992, 0.004481077194213867, 0.00469207763671875, 0.004833221435546875, 0.004759311676025391, 0.004754066467285156, 0.004765033721923828, 0.004979372024536133, 0.005001068115234375, 0.005030393600463867, 0.005086421966552734, 0.005026817321777344, 0.004991054534912109, 0.005021333694458008, 0.00507807731628418, 0.005218982696533203, 0.0052242279052734375, 0.005199909210205078, 0.0052144527435302734, 0.005228281021118164, 0.0052683353424072266, 0.005205392837524414, 0.005229473114013672, 0.005288362503051758, 0.0055196285247802734, 0.005258083343505859, 0.005312442779541016, 0.0052263736724853516, 0.005257129669189453, 0.005259990692138672, 0.005281925201416016, 0.005212068557739258, 0.005253314971923828, 0.005249977111816406, 0.0052640438079833984, 0.005246162414550781, 0.005224704742431641, 0.005208015441894531, 0.005217552185058594, 0.005239963531494141, 0.005250692367553711, 0.005210161209106445, 0.005397796630859375, 0.005176544189453125, 0.00528407096862793, 0.0052449703216552734, 0.0052683353424072266, 0.005244016647338867, 0.005184173583984375, 0.005270957946777344, 0.005223751068115234, 0.005257368087768555, 0.005186796188354492, 0.005278825759887695, 0.005254030227661133, 0.0052335262298583984, 0.005248069763183594, 0.005242586135864258, 0.005247354507446289, 0.005227327346801758, 0.005234956741333008, 0.005354881286621094, 0.005184173583984375, 0.0052585601806640625, 0.005223512649536133, 0.005269527435302734, 0.005212068557739258, 0.005200862884521484, 0.005261898040771484, 0.0051991939544677734, 0.005269527435302734, 0.005214691162109375, 0.0052645206451416016, 0.0052530765533447266, 0.0052449703216552734, 0.005239963531494141, 0.0052356719970703125, 0.005235195159912109, 0.005518436431884766, 0.005404949188232422, 0.0054950714111328125, 0.005217790603637695, 0.0052297115325927734, 0.00524449348449707, 0.005308866500854492, 0.005233287811279297, 0.005182027816772461, 0.005398273468017578, 0.005342006683349609, 0.00528717041015625, 0.005480051040649414, 0.005285501480102539, 0.005269765853881836, 0.005292654037475586, 0.005319118499755859, 0.00532841682434082, 0.005219936370849609, 0.005281925201416016, 0.005387306213378906, 0.005455493927001953, 0.005450725555419922, 0.005235433578491211, 0.005359649658203125, 0.00528407096862793, 0.0053424835205078125, 0.005322456359863281, 0.005292177200317383, 0.005297422409057617, 0.005282163619995117, 0.005297183990478516, 0.0052683353424072266, 0.005312681198120117, 0.005263090133666992, 0.00527191162109375, 0.005314350128173828, 0.005232095718383789, 0.005287885665893555, 0.005536317825317383, 0.0053212642669677734, 0.005311250686645508, 0.005427360534667969, 0.005414247512817383, 0.005272626876831055, 0.005127906799316406, 0.005118131637573242, 0.005109071731567383, 0.0051381587982177734, 0.005271434783935547, 0.0052051544189453125, 0.005079030990600586, 0.0053441524505615234, 0.004859447479248047, 0.004919290542602539, 0.004820108413696289, 0.004858255386352539, 0.0048198699951171875, 0.004857778549194336, 0.004815101623535156, 0.004848480224609375, 0.004812002182006836, 0.005331754684448242, 0.005125284194946289, 0.004512786865234375, 0.004533529281616211, 0.004582405090332031, 0.004621267318725586, 0.0045244693756103516, 0.0044744014739990234, 0.0045168399810791016, 0.0045244693756103516, 0.004559755325317383, 0.004473686218261719, 0.00452423095703125, 0.004528045654296875, 0.00454258918762207, 0.004524707794189453, 0.004554271697998047, 0.004464864730834961, 0.0045316219329833984, 0.004780292510986328, 0.004503011703491211, 0.004514932632446289, 0.004530191421508789, 0.004518747329711914, 0.004517316818237305, 0.004530668258666992, 0.004467487335205078, 0.004567861557006836, 0.0045015811920166016, 0.004472017288208008, 0.004506826400756836, 0.00452733039855957, 0.004875898361206055, 0.004566192626953125, 0.004522800445556641, 0.004846334457397461, 0.00451207160949707, 0.0045318603515625, 0.004605531692504883, 0.004484653472900391, 0.004511356353759766, 0.004534721374511719, 0.004547834396362305, 0.0045511722564697266, 0.004558086395263672, 0.004462242126464844, 0.0045626163482666016, 0.004512786865234375, 0.0045013427734375, 0.004556179046630859, 0.004510164260864258, 0.004521846771240234, 0.004518747329711914, 0.004523515701293945, 0.004526615142822266, 0.004515409469604492, 0.004544734954833984, 0.004545688629150391, 0.004475116729736328, 0.004505634307861328, 0.0046253204345703125, 0.0044901371002197266, 0.004590511322021484, 0.004750967025756836, 0.004515409469604492, 0.0045435428619384766, 0.004536867141723633, 0.004491329193115234, 0.004597902297973633, 0.004594087600708008, 0.004474639892578125, 0.004572868347167969, 0.004506349563598633, 0.0045473575592041016, 0.004566669464111328, 0.004523515701293945, 0.004518985748291016, 0.004537820816040039, 0.004506349563598633, 0.004542350769042969, 0.004475116729736328, 0.0045316219329833984, 0.004559755325317383, 0.004473686218261719, 0.004532814025878906, 0.004559040069580078, 0.004587411880493164, 0.0045318603515625, 0.004573345184326172, 0.004469871520996094, 0.004536151885986328, 0.0045354366302490234, 0.004484653472900391, 0.004558086395263672, 0.004492044448852539, 0.004526615142822266, 0.0045316219329833984, 0.004535198211669922, 0.0044841766357421875, 0.0045833587646484375, 0.004527091979980469, 0.004476308822631836, 0.004509925842285156, 0.004512310028076172, 0.004556894302368164, 0.00449824333190918, 0.00452733039855957, 0.004794120788574219, 0.004587888717651367, 0.004525184631347656, 0.004579305648803711, 0.004559040069580078, 0.004579305648803711, 0.004542350769042969, 0.004488706588745117, 0.004515647888183594, 0.0045392513275146484, 0.004614830017089844, 0.0045566558837890625, 0.004531383514404297, 0.004461526870727539, 0.004608154296875, 0.00451350212097168, 0.004576683044433594, 0.004479646682739258, 0.004528045654296875, 0.004560947418212891, 0.004515409469604492, 0.004520893096923828, 0.004536867141723633, 0.004511833190917969, 0.0045659542083740234, 0.00451970100402832, 0.004498004913330078, 0.0045354366302490234, 0.004554271697998047, 0.004493236541748047, 0.0045244693756103516, 0.00451207160949707, 0.0045430660247802734, 0.004521369934082031, 0.00452113151550293, 0.0044689178466796875, 0.004566192626953125, 0.004540920257568359, 0.004476070404052734, 0.0045318603515625, 0.004506826400756836, 0.0045680999755859375, 0.004618167877197266, 0.004530429840087891, 0.004751920700073242, 0.004552364349365234, 0.004537105560302734, 0.0045604705810546875, 0.004513263702392578, 0.004523038864135742, 0.004601716995239258, 0.004526853561401367, 0.004527091979980469, 0.004554271697998047, 0.0045239925384521484, 0.004549264907836914, 0.0047457218170166016, 0.004529714584350586, 0.0045223236083984375, 0.0044825077056884766, 0.00453495979309082, 0.004600048065185547, 0.004481792449951172, 0.00453495979309082, 0.004541873931884766, 0.004530668258666992, 0.004938364028930664, 0.004800081253051758, 0.004630327224731445, 0.004555225372314453, 0.0045473575592041016, 0.004637718200683594, 0.004535198211669922, 0.0045092105865478516, 0.004556417465209961, 0.0045166015625, 0.0045337677001953125, 0.004513978958129883, 0.004527568817138672, 0.004594564437866211, 0.004478931427001953, 0.004559516906738281, 0.004544496536254883, 0.004535198211669922, 0.004528522491455078, 0.004594564437866211, 0.00450444221496582, 0.004577159881591797, 0.004772186279296875, 0.004492759704589844, 0.004544258117675781, 0.00455164909362793, 0.004568338394165039, 0.004566192626953125, 0.004530668258666992, 0.0045049190521240234, 0.0045871734619140625, 0.004555225372314453, 0.004541158676147461, 0.004498958587646484, 0.0045239925384521484, 0.0045986175537109375, 0.004500150680541992, 0.00455474853515625, 0.0045166015625, 0.004560947418212891, 0.004542350769042969, 0.004572153091430664, 0.004511594772338867, 0.004579305648803711, 0.004548549652099609, 0.004531383514404297, 0.0045909881591796875, 0.0045392513275146484, 0.004539966583251953, 0.0045282840728759766, 0.004548788070678711, 0.004507303237915039, 0.0046045780181884766, 0.004551887512207031, 0.004554271697998047, 0.004511117935180664, 0.004535675048828125, 0.0045964717864990234, 0.004544973373413086, 0.004556894302368164, 0.004604816436767578, 0.00457000732421875, 0.0045549869537353516, 0.0045528411865234375, 0.004520416259765625, 0.004679679870605469, 0.004683494567871094, 0.004531145095825195, 0.004568576812744141, 0.004567146301269531, 0.0046160221099853516, 0.0045359134674072266, 0.004544734954833984, 0.00455474853515625, 0.004561662673950195, 0.0045545101165771484, 0.004567861557006836, 0.0045168399810791016, 0.004538536071777344, 0.004708051681518555, 0.004502534866333008, 0.004560232162475586, 0.0045354366302490234, 0.0045087337493896484, 0.004530429840087891, 0.004556179046630859, 0.0045316219329833984, 0.004584074020385742, 0.004550457000732422, 0.004489898681640625, 0.00457310676574707, 0.0045740604400634766, 0.004590749740600586, 0.004516124725341797, 0.004517078399658203, 0.004557132720947266, 0.004560708999633789, 0.004535198211669922, 0.004555225372314453, 0.004494190216064453, 0.00459599494934082, 0.004586219787597656, 0.004492998123168945, 0.004572868347167969, 0.004568815231323242, 0.00455927848815918, 0.004544734954833984, 0.004560232162475586, 0.004561185836791992, 0.004835367202758789, 0.0045735836029052734, 0.004590034484863281, 0.004494905471801758, 0.004529476165771484, 0.004606485366821289, 0.004513978958129883, 0.004593849182128906, 0.004582881927490234, 0.004550933837890625, 0.004583597183227539, 0.004569053649902344, 0.004607677459716797, 0.004578590393066406, 0.004550457000732422, 0.004556179046630859, 0.0045697689056396484, 0.004587650299072266, 0.004557371139526367, 0.004570960998535156, 0.004576683044433594, 0.0045697689056396484, 0.004511356353759766, 0.004555940628051758, 0.004594087600708008, 0.004510164260864258, 0.00455784797668457, 0.0045659542083740234, 0.004576444625854492, 0.00455164909362793, 0.00456547737121582, 0.004532814025878906, 0.0046007633209228516, 0.0045545101165771484, 0.004515647888183594, 0.004591226577758789, 0.004590511322021484, 0.004630565643310547, 0.004505157470703125, 0.00456547737121582, 0.004561424255371094, 0.0045588016510009766, 0.00457000732421875, 0.004543781280517578, 0.004728555679321289, 0.004637002944946289, 0.004537820816040039, 0.0045735836029052734, 0.004541873931884766, 0.004590749740600586, 0.00457310676574707, 0.004584550857543945, 0.004566192626953125, 0.004537343978881836, 0.004613637924194336, 0.004594087600708008, 0.004570484161376953, 0.004534482955932617, 0.004555463790893555, 0.004639863967895508, 0.004549503326416016, 0.0045430660247802734, 0.004555702209472656, 0.004605770111083984, 0.004556179046630859, 0.004554033279418945, 0.004541158676147461, 0.004611968994140625, 0.004547595977783203, 0.0045354366302490234, 0.004567384719848633, 0.0045697689056396484, 0.004674434661865234, 0.004553079605102539, 0.004685401916503906, 0.0046465396881103516, 0.0046329498291015625, 0.004622936248779297, 0.004604339599609375, 0.004568576812744141, 0.004694223403930664, 0.004640817642211914, 0.0045928955078125, 0.00461578369140625, 0.004618167877197266, 0.004647016525268555, 0.004658937454223633, 0.004656314849853516, 0.0048694610595703125, 0.004618167877197266, 0.004667758941650391, 0.004614114761352539, 0.004586696624755859, 0.004656314849853516, 0.004602193832397461, 0.004614353179931641, 0.004575490951538086, 0.004665374755859375, 0.004645347595214844, 0.004602193832397461, 0.0048062801361083984, 0.004636287689208984, 0.0046007633209228516, 0.004633426666259766, 0.004625082015991211, 0.004712104797363281, 0.004647493362426758, 0.004623889923095703, 0.004644632339477539, 0.004591226577758789, 0.004617452621459961, 0.004688739776611328, 0.0045680999755859375, 0.004624366760253906, 0.004614591598510742, 0.0046215057373046875, 0.004652976989746094, 0.004654407501220703, 0.004629373550415039, 0.004620552062988281, 0.004616975784301758, 0.004657745361328125, 0.0046007633209228516, 0.004654407501220703, 0.004656314849853516, 0.004581928253173828, 0.004642486572265625, 0.004630565643310547, 0.0046389102935791016, 0.004620790481567383, 0.0046234130859375, 0.00487208366394043, 0.004614830017089844, 0.004591703414916992, 0.004869699478149414, 0.004625797271728516, 0.004708290100097656, 0.004618167877197266, 0.004571199417114258, 0.0046160221099853516, 0.004601478576660156, 0.004647970199584961, 0.004583120346069336, 0.004624128341674805, 0.004642486572265625, 0.0046138763427734375, 0.004655361175537109, 0.004624366760253906, 0.004594326019287109, 0.004701375961303711, 0.004647254943847656, 0.004642486572265625, 0.004584074020385742, 0.004621267318725586, 0.004660844802856445, 0.004571676254272461, 0.004606723785400391, 0.004658699035644531, 0.004627704620361328, 0.004649162292480469, 0.004853725433349609, 0.004615068435668945, 0.004744291305541992, 0.004622220993041992, 0.004654407501220703, 0.00461125373840332, 0.004824399948120117, 0.004703044891357422, 0.004599094390869141, 0.004628896713256836, 0.004622936248779297, 0.004639863967895508, 0.004634380340576172, 0.004652976989746094, 0.004881620407104492, 0.0046350955963134766, 0.0046613216400146484, 0.004656314849853516, 0.004594564437866211, 0.0047109127044677734, 0.0046498775482177734, 0.004584074020385742, 0.004642486572265625, 0.0046422481536865234, 0.004667997360229492, 0.0046024322509765625, 0.0046269893646240234, 0.004639863967895508, 0.00461125373840332, 0.004632234573364258, 0.004626274108886719, 0.0045855045318603516, 0.004710674285888672, 0.004608869552612305, 0.004642009735107422, 0.004582405090332031, 0.00462794303894043, 0.004685640335083008, 0.004578590393066406, 0.004643917083740234, 0.004647970199584961, 0.004621267318725586, 0.004668474197387695, 0.004627704620361328, 0.004586935043334961, 0.004678964614868164, 0.004649639129638672, 0.004633426666259766, 0.0046539306640625, 0.00462651252746582, 0.004723787307739258, 0.00462031364440918, 0.004637956619262695, 0.004652738571166992, 0.004613637924194336, 0.004636287689208984, 0.004620552062988281, 0.0049228668212890625, 0.0046613216400146484, 0.004646778106689453, 0.004656076431274414, 0.004590272903442383, 0.004683494567871094, 0.0049474239349365234, 0.004775524139404297, 0.004664421081542969, 0.00464940071105957, 0.004667758941650391, 0.0046236515045166016, 0.004636287689208984, 0.004644632339477539, 0.004658222198486328, 0.004711627960205078, 0.00462031364440918, 0.004615068435668945, 0.004716157913208008, 0.004643917083740234, 0.004642486572265625, 0.0045757293701171875, 0.004627227783203125, 0.005611419677734375, 0.004746913909912109, 0.00466465950012207, 0.005116939544677734, 0.004696846008300781, 0.0046274662017822266, 0.0046770572662353516, 0.0046198368072509766, 0.004594326019287109, 0.004698038101196289, 0.004654645919799805, 0.004647254943847656, 0.004651784896850586, 0.0046575069427490234, 0.004698753356933594, 0.004573822021484375, 0.004700899124145508, 0.0046656131744384766, 0.004636049270629883, 0.00484776496887207, 0.004666805267333984, 0.00471949577331543, 0.004668474197387695, 0.004638195037841797, 0.004696846008300781, 0.004651308059692383, 0.004708290100097656, 0.0046672821044921875, 0.004605293273925781, 0.004667520523071289, 0.004631519317626953, 0.004698276519775391, 0.0045795440673828125, 0.004666566848754883, 0.004629850387573242, 0.00461578369140625, 0.004634857177734375, 0.004633426666259766, 0.0045909881591796875, 0.004684925079345703, 0.004679203033447266, 0.0047304630279541016, 0.004634857177734375, 0.004650115966796875, 0.0046977996826171875, 0.004651546478271484, 0.0046999454498291016, 0.004663944244384766, 0.0046846866607666016, 0.00471806526184082, 0.0046460628509521484, 0.00463104248046875, 0.004772663116455078, 0.004639863967895508, 0.004659891128540039, 0.004592180252075195, 0.004665851593017578, 0.004656314849853516, 0.0046749114990234375, 0.0046617984771728516, 0.004656791687011719, 0.0047016143798828125, 0.004859209060668945, 0.004698514938354492, 0.004666566848754883, 0.004607677459716797, 0.004701137542724609, 0.0046787261962890625, 0.00462794303894043, 0.004661083221435547, 0.004660367965698242, 0.0047152042388916016, 0.004720211029052734, 0.0046350955963134766, 0.004854917526245117, 0.004650592803955078, 0.0047147274017333984, 0.004689216613769531, 0.004691362380981445, 0.004626035690307617, 0.0046803951263427734, 0.004721641540527344, 0.004622220993041992, 0.004786014556884766, 0.004662275314331055, 0.005781888961791992, 0.005043745040893555, 0.01598334312438965, 0.007942438125610352, 0.005082130432128906, 0.004954099655151367, 0.004980325698852539, 0.00477147102355957, 0.004772186279296875, 0.004826784133911133, 0.004845619201660156, 0.004700183868408203, 0.004837512969970703, 0.00478672981262207, 0.0046651363372802734, 0.0047359466552734375, 0.00493931770324707, 0.004921913146972656, 0.0046803951263427734, 0.004826068878173828, 0.00472712516784668, 0.00470423698425293, 0.0047686100006103516, 0.004695415496826172, 0.004697084426879883, 0.004652976989746094, 0.004773855209350586, 0.004745960235595703, 0.00469207763671875, 0.004730701446533203, 0.004714488983154297, 0.00484156608581543, 0.0046844482421875, 0.004763126373291016, 0.004693031311035156, 0.004808902740478516, 0.004742145538330078, 0.0047643184661865234, 0.0047397613525390625, 0.004743814468383789, 0.00465083122253418, 0.004796028137207031, 0.004723548889160156, 0.00482630729675293, 0.004719257354736328, 0.004782676696777344, 0.004725456237792969, 0.004725217819213867, 0.004679441452026367, 0.004701852798461914, 0.004803895950317383, 0.0046846866607666016, 0.004769086837768555, 0.004778861999511719, 0.004720449447631836, 0.004720211029052734, 0.0047113895416259766, 0.004728794097900391, 0.004975795745849609, 0.00474095344543457, 0.0047261714935302734, 0.0047414302825927734, 0.004759788513183594, 0.004765748977661133, 0.004750967025756836, 0.004693031311035156, 0.0047130584716796875, 0.004803895950317383, 0.004680633544921875, 0.004773378372192383, 0.0047435760498046875, 0.005502223968505859, 0.00481414794921875, 0.00478053092956543, 0.0051953792572021484, 0.004884958267211914, 0.004690408706665039, 0.004772663116455078, 0.004749774932861328, 0.0047304630279541016, 0.0047647953033447266, 0.004707813262939453, 0.0047948360443115234, 0.004777193069458008, 0.0047512054443359375, 0.004689693450927734, 0.004696369171142578, 0.004752635955810547, 0.004784107208251953, 0.004714250564575195, 0.004710674285888672, 0.0047533512115478516, 0.004876136779785156, 0.0046346187591552734, 0.004755735397338867, 0.004747629165649414, 0.004786968231201172, 0.004651069641113281, 0.004743099212646484, 0.004760265350341797, 0.004995584487915039, 0.0047779083251953125, 0.00482487678527832, 0.004815816879272461, 0.004739522933959961, 0.0047719478607177734, 0.00483393669128418, 0.004791259765625, 0.00480341911315918, 0.004777431488037109, 0.00479435920715332, 0.005443572998046875, 0.004823446273803711, 0.004785060882568359, 0.004809856414794922, 0.004734516143798828, 0.004816770553588867, 0.004755258560180664, 0.004691123962402344, 0.004797935485839844, 0.004727363586425781, 0.004767417907714844, 0.004747867584228516, 0.004767417907714844, 0.004812955856323242, 0.0047149658203125, 0.004763126373291016, 0.00475621223449707, 0.004717111587524414, 0.004847526550292969, 0.0047435760498046875, 0.004786491394042969, 0.0047473907470703125, 0.0048313140869140625, 0.004746198654174805, 0.0047757625579833984, 0.004711627960205078, 0.004811286926269531, 0.00477290153503418, 0.004977226257324219, 0.004900693893432617, 0.004755496978759766, 0.005040407180786133, 0.004698753356933594, 0.0048732757568359375, 0.0047490596771240234, 0.004866838455200195, 0.004698514938354492, 0.004760026931762695, 0.0048503875732421875, 0.0047495365142822266, 0.004854440689086914, 0.004777193069458008, 0.004815340042114258, 0.00472712516784668, 0.0047376155853271484, 0.0047149658203125, 0.0047109127044677734, 0.00480198860168457, 0.004848480224609375, 0.004863739013671875, 0.004841327667236328, 0.0048143863677978516, 0.004801511764526367, 0.004751920700073242, 0.004777431488037109, 0.004801034927368164, 0.004914522171020508, 0.004728555679321289, 0.004824399948120117, 0.004786968231201172, 0.004900217056274414, 0.00471949577331543, 0.004781246185302734, 0.0048067569732666016, 0.0048372745513916016, 0.004817962646484375, 0.004820346832275391, 0.0048024654388427734, 0.004791975021362305, 0.0047719478607177734, 0.004810810089111328, 0.004807472229003906, 0.004962444305419922, 0.004990339279174805, 0.004858970642089844, 0.00476837158203125, 0.004873991012573242, 0.00485539436340332, 0.004810810089111328, 0.004751443862915039, 0.0048868656158447266, 0.00487065315246582, 0.004854917526245117, 0.004830360412597656, 0.004945516586303711, 0.0048274993896484375, 0.004729509353637695, 0.004796266555786133, 0.0048444271087646484, 0.0048100948333740234, 0.004744052886962891, 0.004790782928466797, 0.004903316497802734, 0.004827976226806641, 0.004810810089111328, 0.004869222640991211, 0.0048198699951171875, 0.004782199859619141, 0.004808902740478516, 0.004786491394042969, 0.004785060882568359, 0.0049016475677490234, 0.004828453063964844, 0.004817962646484375, 0.004789113998413086, 0.004814863204956055, 0.004805803298950195, 0.004792213439941406, 0.004803657531738281, 0.0047607421875, 0.004917621612548828, 0.004790544509887695, 0.004770994186401367, 0.004779338836669922, 0.0050809383392333984, 0.004801273345947266, 0.004808664321899414, 0.004780292510986328, 0.004827737808227539, 0.00483250617980957, 0.004761457443237305, 0.004844188690185547, 0.004813194274902344, 0.004838228225708008, 0.00474095344543457, 0.004843235015869141, 0.004767417907714844, 0.004813194274902344, 0.00477290153503418, 0.004735708236694336, 0.004907846450805664, 0.004847049713134766, 0.004809379577636719, 0.00476527214050293, 0.004872322082519531, 0.004862785339355469, 0.0048296451568603516, 0.0047626495361328125, 0.004824399948120117, 0.004897594451904297, 0.00475621223449707, 0.004786252975463867, 0.004784107208251953, 0.004902839660644531, 0.0047681331634521484, 0.00483393669128418, 0.004832744598388672, 0.004778623580932617, 0.005127906799316406, 0.004895925521850586, 0.004836320877075195, 0.004793405532836914, 0.0048029422760009766, 0.0048828125, 0.004806041717529297, 0.005056142807006836, 0.0051021575927734375, 0.004853725433349609, 0.004804372787475586, 0.004796028137207031, 0.004796028137207031, 0.004838466644287109, 0.004765987396240234, 0.004877328872680664, 0.004862070083618164, 0.004822254180908203, 0.0047664642333984375, 0.00481724739074707, 0.004850864410400391, 0.0048291683197021484, 0.004866123199462891, 0.00480341911315918, 0.004862785339355469, 0.004861593246459961, 0.0048274993896484375, 0.0047588348388671875, 0.004950761795043945, 0.00473475456237793, 0.004795551300048828, 0.004839658737182617, 0.004857778549194336, 0.004816532135009766, 0.0048406124114990234, 0.004831790924072266, 0.004793643951416016, 0.004829883575439453, 0.004817485809326172, 0.004830837249755859, 0.004772186279296875, 0.004775524139404297, 0.004837512969970703, 0.0047953128814697266, 0.004823446273803711, 0.004841327667236328, 0.004816293716430664, 0.004763364791870117, 0.005084514617919922, 0.00507044792175293, 0.004810810089111328, 0.004915952682495117, 0.004880189895629883, 0.004837989807128906, 0.004739046096801758, 0.004976034164428711, 0.004820823669433594, 0.004968166351318359, 0.004761934280395508, 0.004918098449707031, 0.004792451858520508, 0.004806041717529297, 0.004714488983154297, 0.0047986507415771484, 0.004895925521850586, 0.004782438278198242, 0.004790782928466797, 0.0048809051513671875, 0.0047914981842041016, 0.004854440689086914, 0.004921674728393555, 0.005201578140258789, 0.004890918731689453, 0.004866123199462891, 0.004866361618041992, 0.004746675491333008, 0.004748821258544922, 0.004918098449707031, 0.004801750183105469, 0.004882335662841797, 0.004803895950317383, 0.004869222640991211, 0.004862546920776367, 0.004840850830078125, 0.004739046096801758, 0.004954814910888672, 0.004846096038818359, 0.004751443862915039, 0.004859209060668945, 0.004861116409301758, 0.0050504207611083984, 0.0048618316650390625, 0.004861116409301758, 0.004782438278198242, 0.0050013065338134766, 0.004934072494506836, 0.00479435920715332, 0.004891872406005859, 0.004878997802734375, 0.0047872066497802734, 0.004862070083618164, 0.004900693893432617, 0.0048694610595703125, 0.004845380783081055, 0.00481867790222168, 0.004833698272705078, 0.0047800540924072266, 0.0048675537109375, 0.004830837249755859, 0.004906177520751953, 0.00481867790222168, 0.004889011383056641, 0.004796266555786133, 0.00476527214050293, 0.004902362823486328, 0.004804849624633789, 0.004832267761230469, 0.004781246185302734, 0.004856109619140625, 0.0048542022705078125, 0.00479888916015625, 0.004834651947021484, 0.0048847198486328125, 0.004950523376464844, 0.004757404327392578, 0.004839420318603516, 0.004893302917480469, 0.0049059391021728516, 0.004834890365600586, 0.004818439483642578, 0.0048558712005615234, 0.005117654800415039, 0.004863262176513672, 0.004833698272705078, 0.004842281341552734, 0.004870414733886719, 0.004862070083618164, 0.004824399948120117, 0.004915714263916016, 0.004842996597290039, 0.004891157150268555, 0.004956245422363281, 0.004969120025634766, 0.00494694709777832, 0.004914522171020508, 0.005052804946899414, 0.0048274993896484375, 0.004830598831176758, 0.004893064498901367, 0.004847049713134766, 0.0047724246978759766, 0.0049245357513427734, 0.0048830509185791016, 0.004872322082519531, 0.004767417907714844, 0.0048961639404296875, 0.004856586456298828, 0.0048694610595703125, 0.004799842834472656, 0.004858255386352539, 0.004869222640991211, 0.004778385162353516, 0.00483393669128418, 0.004873752593994141, 0.004922389984130859, 0.00478363037109375, 0.004847288131713867, 0.004887104034423828, 0.0049016475677490234, 0.004870176315307617, 0.0048258304595947266, 0.004846096038818359, 0.005052089691162109, 0.0049593448638916016, 0.004878997802734375, 0.004816293716430664, 0.004799604415893555, 0.004829883575439453, 0.004822731018066406, 0.004840373992919922, 0.0047948360443115234, 0.004860401153564453, 0.0048220157623291016, 0.004843711853027344, 0.004872798919677734, 0.004872798919677734, 0.004853010177612305, 0.004779338836669922, 0.004946470260620117, 0.004877567291259766, 0.0048372745513916016, 0.004804372787475586, 0.00487971305847168, 0.0048046112060546875, 0.004956960678100586, 0.004831075668334961, 0.004822969436645508, 0.004859447479248047, 0.004855632781982422, 0.004799842834472656, 0.004803180694580078, 0.004947662353515625, 0.004932403564453125, 0.004876613616943359, 0.004828929901123047, 0.00488591194152832, 0.004828691482543945, 0.0048809051513671875, 0.00485992431640625, 0.004904747009277344, 0.004837512969970703, 0.004901409149169922, 0.0048313140869140625, 0.004925966262817383, 0.005041599273681641, 0.0048906803131103516, 0.0048296451568603516, 0.004883766174316406, 0.004899024963378906, 0.004853248596191406, 0.004880666732788086, 0.004878044128417969, 0.004901885986328125, 0.004878044128417969, 0.004807233810424805, 0.00485992431640625, 0.004947185516357422, 0.004872798919677734, 0.004805326461791992, 0.004983186721801758, 0.004921436309814453, 0.0048444271087646484, 0.004887580871582031, 0.004975080490112305, 0.004811286926269531, 0.00485539436340332, 0.004790067672729492, 0.00484013557434082, 0.004953861236572266, 0.00484776496887207, 0.0048215389251708984, 0.004837751388549805, 0.0049724578857421875, 0.004829883575439453, 0.0049250125885009766, 0.0048792362213134766, 0.004986286163330078, 0.004812479019165039, 0.004885673522949219, 0.0048868656158447266, 0.004786491394042969, 0.004822492599487305, 0.004880428314208984, 0.004846811294555664, 0.005092620849609375, 0.005071401596069336, 0.0048618316650390625, 0.004857063293457031, 0.004918575286865234, 0.0049381256103515625, 0.004973649978637695, 0.0049245357513427734, 0.004922389984130859, 0.00491642951965332, 0.004839658737182617, 0.005095958709716797, 0.004854440689086914, 0.0048825740814208984, 0.005017757415771484, 0.004850864410400391, 0.004865407943725586, 0.004761457443237305, 0.005004167556762695, 0.004909038543701172, 0.0048542022705078125, 0.004863739013671875, 0.0050356388092041016, 0.004919528961181641, 0.004927873611450195, 0.004807710647583008, 0.0049555301666259766, 0.00485539436340332, 0.0048770904541015625, 0.0048906803131103516, 0.004920005798339844, 0.004876136779785156, 0.004841327667236328, 0.004841327667236328, 0.004871368408203125, 0.00490260124206543, 0.0048427581787109375, 0.004808187484741211, 0.004825115203857422, 0.00494074821472168, 0.004760026931762695, 0.00497889518737793, 0.004974842071533203, 0.004909038543701172, 0.0048980712890625, 0.004830598831176758, 0.004958629608154297, 0.004885435104370117, 0.00486445426940918, 0.004900217056274414, 0.004844188690185547, 0.004926204681396484, 0.00497746467590332, 0.004901409149169922, 0.005010366439819336, 0.004990339279174805, 0.0049512386322021484, 0.004945278167724609, 0.004945516586303711, 0.004942417144775391, 0.004981279373168945, 0.004898548126220703, 0.005072116851806641, 0.004923820495605469, 0.004961252212524414, 0.004975318908691406, 0.004942178726196289, 0.004937648773193359, 0.0049474239349365234, 0.004930973052978516, 0.0049860477447509766, 0.004880666732788086, 0.004880189895629883, 0.004950046539306641, 0.0049285888671875, 0.004954338073730469, 0.004820823669433594, 0.004966259002685547, 0.0048944950103759766, 0.004860401153564453, 0.0049037933349609375, 0.005010128021240234, 0.005043506622314453, 0.0049190521240234375, 0.004853725433349609, 0.004990100860595703, 0.00496673583984375, 0.004946231842041016, 0.004910469055175781, 0.0050487518310546875, 0.0049173831939697266, 0.0050013065338134766, 0.0049092769622802734, 0.005185604095458984, 0.0049555301666259766, 0.004990100860595703, 0.0049211978912353516, 0.004976987838745117, 0.0049533843994140625, 0.0049860477447509766, 0.0048885345458984375, 0.00512385368347168, 0.004967927932739258, 0.004865884780883789, 0.0048656463623046875, 0.0050201416015625, 0.00496983528137207, 0.0049285888671875, 0.0049703121185302734, 0.00494384765625, 0.005012035369873047, 0.004878520965576172, 0.0048902034759521484, 0.004894733428955078, 0.004974365234375, 0.00487971305847168, 0.0048563480377197266, 0.004931449890136719, 0.004957437515258789, 0.0048084259033203125, 0.0049550533294677734, 0.004885673522949219, 0.005059719085693359, 0.005040407180786133, 0.0049800872802734375, 0.004962921142578125, 0.005011320114135742, 0.004883527755737305, 0.004962444305419922, 0.004931211471557617, 0.004927873611450195, 0.0048961639404296875, 0.004981040954589844, 0.004945039749145508, 0.004975080490112305, 0.004902362823486328, 0.0049440860748291016, 0.0049707889556884766, 0.004948139190673828, 0.004927158355712891, 0.004987478256225586, 0.004975795745849609, 0.00493621826171875, 0.005019426345825195, 0.0049626827239990234, 0.004958391189575195, 0.004967927932739258, 0.004945039749145508, 0.0049533843994140625, 0.004986286163330078, 0.0049402713775634766, 0.0049326419830322266, 0.004955291748046875, 0.0048830509185791016, 0.004923343658447266, 0.004891157150268555, 0.004846096038818359, 0.004912137985229492, 0.0049114227294921875, 0.0049571990966796875, 0.004945278167724609, 0.004927158355712891, 0.005017757415771484, 0.005107402801513672, 0.004960298538208008, 0.004895210266113281, 0.004930019378662109, 0.004956245422363281, 0.004963397979736328, 0.004904985427856445, 0.005025625228881836, 0.004917144775390625, 0.004971981048583984, 0.004822254180908203, 0.004988431930541992, 0.005031108856201172, 0.004996299743652344, 0.0048329830169677734, 0.004967212677001953, 0.004987955093383789, 0.004949092864990234, 0.0048847198486328125, 0.004992485046386719, 0.004952669143676758, 0.004933595657348633, 0.00492095947265625, 0.00502324104309082, 0.004934549331665039, 0.004973173141479492, 0.004930734634399414, 0.005000114440917969, 0.004949092864990234, 0.004957914352416992, 0.004933357238769531, 0.005023002624511719, 0.004927396774291992, 0.0049707889556884766, 0.004930019378662109, 0.004996299743652344, 0.004943132400512695, 0.00497746467590332, 0.004952192306518555, 0.005101203918457031, 0.005033254623413086, 0.0050203800201416016, 0.004907131195068359, 0.004979610443115234, 0.00493311882019043, 0.004965543746948242, 0.00490260124206543, 0.0050525665283203125, 0.004926443099975586, 0.004891395568847656, 0.005026817321777344, 0.004952907562255859, 0.004952430725097656, 0.004899263381958008, 0.005034446716308594, 0.004942893981933594, 0.0049517154693603516, 0.004934072494506836, 0.00502467155456543, 0.004926919937133789, 0.004983186721801758, 0.004951953887939453, 0.004984140396118164, 0.004900693893432617, 0.005005598068237305, 0.004950761795043945, 0.0049893856048583984, 21.529611349105835]
t_14_lzf = [0.004839897155761719, 0.003641366958618164, 0.0038471221923828125, 0.003820657730102539, 0.004106044769287109, 0.0039746761322021484, 0.0038831233978271484, 0.003863811492919922, 0.0038688182830810547, 0.003869295120239258, 0.005105018615722656, 0.006177425384521484, 0.0057756900787353516, 0.005425930023193359, 0.0051708221435546875, 0.005186319351196289, 0.0051500797271728516, 0.005268096923828125, 0.005147457122802734, 0.005350828170776367, 0.0050809383392333984, 0.005228519439697266, 0.0051844120025634766, 0.005117893218994141, 0.00525975227355957, 0.0052297115325927734, 0.005522251129150391, 0.0039501190185546875, 0.0037376880645751953, 0.004647970199584961, 0.0039713382720947266, 0.0047228336334228516, 0.005582332611083984, 0.003751993179321289, 0.0036292076110839844, 0.003648996353149414, 0.0036263465881347656, 0.003633737564086914, 0.003630399703979492, 0.0036046504974365234, 0.003570556640625, 0.0036313533782958984, 0.0036268234252929688, 0.003624439239501953, 0.0036230087280273438, 0.0035779476165771484, 0.0036165714263916016, 0.0036935806274414062, 0.003612995147705078, 0.00362396240234375, 0.0036165714263916016, 0.0035698413848876953, 0.0036406517028808594, 0.0035843849182128906, 0.003589630126953125, 0.0036449432373046875, 0.003583192825317383, 0.0036230087280273438, 0.003584146499633789, 0.0036220550537109375, 0.0036923885345458984, 0.0036101341247558594, 0.00368499755859375, 0.003662109375, 0.0035698413848876953, 0.0035996437072753906, 0.0036580562591552734, 0.003576040267944336, 0.003622770309448242, 0.0035867691040039062, 0.003612995147705078, 0.0036203861236572266, 0.0036208629608154297, 0.0035653114318847656, 0.003645658493041992, 0.003573894500732422, 0.003750324249267578, 0.00360870361328125, 0.0036644935607910156, 0.0036199092864990234, 0.003589630126953125, 0.0036356449127197266, 0.0036077499389648438, 0.0036046504974365234, 0.0035598278045654297, 0.003604888916015625, 0.003572702407836914, 0.0036344528198242188, 0.003875732421875, 0.0037157535552978516, 0.003648042678833008, 0.0035774707794189453, 0.0036432743072509766, 0.0036156177520751953, 0.003629446029663086, 0.003572702407836914, 0.0036153793334960938, 0.0035774707794189453, 0.0036051273345947266, 0.0036079883575439453, 0.003614664077758789, 0.003607511520385742, 0.003579854965209961, 0.003608226776123047, 0.003614187240600586, 0.0036106109619140625, 0.003576517105102539, 0.0036439895629882812, 0.003586292266845703, 0.0036187171936035156, 0.0036039352416992188, 0.003607034683227539, 0.003592967987060547, 0.0036199092864990234, 0.003608226776123047, 0.0036880970001220703, 0.0036199092864990234, 0.0037527084350585938, 0.0036957263946533203, 0.003587484359741211, 0.00362396240234375, 0.0036268234252929688, 0.0036170482635498047, 0.0035753250122070312, 0.003650665283203125, 0.0036127567291259766, 0.00362396240234375, 0.003629446029663086, 0.0035898685455322266, 0.0036194324493408203, 0.003574371337890625, 0.00363922119140625, 0.0036420822143554688, 0.0036704540252685547, 0.0035800933837890625, 0.003663778305053711, 0.003643035888671875, 0.003625631332397461, 0.0036149024963378906, 0.0035681724548339844, 0.0036172866821289062, 0.0035865306854248047, 0.003638029098510742, 0.0036127567291259766, 0.0036072731018066406, 0.0035943984985351562, 0.003622293472290039, 0.0036344528198242188, 0.003622770309448242, 0.0036296844482421875, 0.003592967987060547, 0.003635406494140625, 0.0035910606384277344, 0.003621339797973633, 0.0036232471466064453, 0.0036334991455078125, 0.0035843849182128906, 0.003635406494140625, 0.0036172866821289062, 0.003699779510498047, 0.003640890121459961, 0.003591775894165039, 0.003643512725830078, 0.0036001205444335938, 0.0036351680755615234, 0.0036139488220214844, 0.003637075424194336, 0.0035963058471679688, 0.0036394596099853516, 0.003564119338989258, 0.0037910938262939453, 0.0036814212799072266, 0.0036089420318603516, 0.0037841796875, 0.0036242008209228516, 0.0036160945892333984, 0.0040683746337890625, 0.004300117492675781, 0.004210710525512695, 0.004018068313598633, 0.003934144973754883, 0.008809089660644531, 0.00787353515625, 0.010035991668701172, 0.008716106414794922, 0.011248350143432617, 0.006907224655151367, 0.0053064823150634766, 0.010024070739746094, 0.00914144515991211, 0.00442194938659668, 0.003981113433837891, 0.0039861202239990234, 0.004060983657836914, 0.004000663757324219, 0.003976345062255859, 0.003833293914794922, 0.003759145736694336, 0.003646373748779297, 0.0036551952362060547, 0.0036630630493164062, 0.003619670867919922, 0.003572225570678711, 0.0036373138427734375, 0.0036351680755615234, 0.0036308765411376953, 0.0036058425903320312, 0.003641843795776367, 0.0036034584045410156, 0.0036253929138183594, 0.0036995410919189453, 0.003673076629638672, 0.0037322044372558594, 0.0036170482635498047, 0.003719806671142578, 0.003632783889770508, 0.003641843795776367, 0.0036058425903320312, 0.003654956817626953, 0.0036160945892333984, 0.0038673877716064453, 0.0037505626678466797, 0.0036122798919677734, 0.0036687850952148438, 0.0036041736602783203, 0.0036230087280273438, 0.003648519515991211, 0.003641843795776367, 0.0036287307739257812, 0.003620147705078125, 0.0036330223083496094, 0.0036470890045166016, 0.0036420822143554688, 0.0035858154296875, 0.003652811050415039, 0.0036301612854003906, 0.0036230087280273438, 0.003600597381591797, 0.003641366958618164, 0.0035851001739501953, 0.0036139488220214844, 0.0048716068267822266, 0.0037348270416259766, 0.003651142120361328, 0.0035948753356933594, 0.003968000411987305, 0.003655672073364258, 0.003643035888671875, 0.003650665283203125, 0.0036771297454833984, 0.0036292076110839844, 0.0035779476165771484, 0.0036792755126953125, 0.003643512725830078, 0.0036988258361816406, 0.0036673545837402344, 0.0036611557006835938, 0.003584146499633789, 0.003646373748779297, 0.0037233829498291016, 0.003588438034057617, 0.003640890121459961, 0.00360870361328125, 0.0036313533782958984, 0.003644704818725586, 0.0036482810974121094, 0.003707408905029297, 0.0036640167236328125, 0.003691434860229492, 0.003629446029663086, 0.0037136077880859375, 0.003602743148803711, 0.003655672073364258, 0.0036597251892089844, 0.003640890121459961, 0.0036559104919433594, 0.0036470890045166016, 0.0036115646362304688, 0.0036575794219970703, 0.003611326217651367, 0.003649473190307617, 0.0036797523498535156, 0.003614187240600586, 0.0036499500274658203, 0.0036149024963378906, 0.003665924072265625, 0.003640413284301758, 0.003646373748779297, 0.00360107421875, 0.003652811050415039, 0.003590106964111328, 0.0036249160766601562, 0.003699064254760742, 0.003593921661376953, 0.0036497116088867188, 0.003609895706176758, 0.0036346912384033203, 0.0036897659301757812, 0.0036821365356445312, 0.003583192825317383, 0.0036292076110839844, 0.003613710403442383, 0.0036725997924804688, 0.0036678314208984375, 0.0036041736602783203, 0.003632783889770508, 0.003605365753173828, 0.0036513805389404297, 0.003644227981567383, 0.0036530494689941406, 0.0035915374755859375, 0.003636598587036133, 0.003614664077758789, 0.0036606788635253906, 0.0036840438842773438, 0.0036025047302246094, 0.0036339759826660156, 0.003597259521484375, 0.0036635398864746094, 0.0036420822143554688, 0.00370025634765625, 0.0037326812744140625, 0.0036733150482177734, 0.0037953853607177734, 0.0036551952362060547, 0.0036859512329101562, 0.0036017894744873047, 0.0036623477935791016, 0.0036041736602783203, 0.0036504268646240234, 0.003633260726928711, 0.0036487579345703125, 0.0036287307739257812, 0.003656625747680664, 0.0036678314208984375, 0.0037403106689453125, 0.0036411285400390625, 0.0038754940032958984, 0.0036513805389404297, 0.0036020278930664062, 0.003634929656982422, 0.0036575794219970703, 0.003670930862426758, 0.003607034683227539, 0.0036602020263671875, 0.003670215606689453, 0.0036537647247314453, 0.0038034915924072266, 0.003636598587036133, 0.0036568641662597656, 0.0035982131958007812, 0.003672361373901367, 0.003656625747680664, 0.0036842823028564453, 0.003684520721435547, 0.003587007522583008, 0.0036487579345703125, 0.0036678314208984375, 0.003648519515991211, 0.00360870361328125, 0.003638744354248047, 0.003601551055908203, 0.0036516189575195312, 0.0036733150482177734, 0.00363922119140625, 0.0036499500274658203, 0.00360870361328125, 0.0036745071411132812, 0.0036535263061523438, 0.0036580562591552734, 0.003610849380493164, 0.003646373748779297, 0.003630399703979492, 0.0036563873291015625, 0.003647327423095703, 0.00368499755859375, 0.003726959228515625, 0.003609895706176758, 0.003802061080932617, 0.003683328628540039, 0.003650188446044922, 0.0036003589630126953, 0.003653287887573242, 0.003614187240600586, 0.003675699234008789, 0.003687143325805664, 0.003606081008911133, 0.003656625747680664, 0.0036163330078125, 0.003642559051513672, 0.0036482810974121094, 0.0036482810974121094, 0.003631591796875, 0.003652334213256836, 0.003600597381591797, 0.003657102584838867, 0.0037109851837158203, 0.003624439239501953, 0.003676176071166992, 0.003628253936767578, 0.003652334213256836, 0.003676891326904297, 0.003683328628540039, 0.0036253929138183594, 0.0036721229553222656, 0.003625631332397461, 0.003709077835083008, 0.003685474395751953, 0.0036461353302001953, 0.003663778305053711, 0.003622293472290039, 0.003676891326904297, 0.003667593002319336, 0.0036733150482177734, 0.003630399703979492, 0.003737211227416992, 0.0037086009979248047, 0.0036983489990234375, 0.003664731979370117, 0.003620624542236328, 0.003654003143310547, 0.0036182403564453125, 0.0036597251892089844, 0.0036573410034179688, 0.0036478042602539062, 0.003629446029663086, 0.0037126541137695312, 0.003659963607788086, 0.0036666393280029297, 0.0036847591400146484, 0.0036995410919189453, 0.0036721229553222656, 0.003640413284301758, 0.0037500858306884766, 0.00368499755859375, 0.0036575794219970703, 0.0036764144897460938, 0.0036163330078125, 0.0036695003509521484, 0.003666400909423828, 0.0036525726318359375, 0.0036165714263916016, 0.0036706924438476562, 0.0036280155181884766, 0.0036666393280029297, 0.0036573410034179688, 0.003671407699584961, 0.004170656204223633, 0.003736257553100586, 0.003682851791381836, 0.0037589073181152344, 0.003995180130004883, 0.003793001174926758, 0.0036690235137939453, 0.0036754608154296875, 0.003686666488647461, 0.0036797523498535156, 0.0036313533782958984, 0.003691434860229492, 0.003627300262451172, 0.0036716461181640625, 0.003670215606689453, 0.0036923885345458984, 0.0036704540252685547, 0.0036389827728271484, 0.0036649703979492188, 0.003649473190307617, 0.0036683082580566406, 0.0036461353302001953, 0.0036797523498535156, 0.003622293472290039, 0.003690004348754883, 0.003674745559692383, 0.0036764144897460938, 0.003667593002319336, 0.0036225318908691406, 0.003655672073364258, 0.003680706024169922, 0.0037009716033935547, 0.003627777099609375, 0.003672361373901367, 0.003609895706176758, 0.0036749839782714844, 0.003751039505004883, 0.0037081241607666016, 0.0036666393280029297, 0.0036857128143310547, 0.003770112991333008, 0.003697633743286133, 0.0036725997924804688, 0.0036149024963378906, 0.003685474395751953, 0.0036280155181884766, 0.003774881362915039, 0.003687143325805664, 0.0037069320678710938, 0.003670215606689453, 0.0036706924438476562, 0.003681659698486328, 0.004216909408569336, 0.0037009716033935547, 0.003671407699584961, 0.0036237239837646484, 0.0036857128143310547, 0.0036706924438476562, 0.0036704540252685547, 0.0036382675170898438, 0.003730297088623047, 0.003625631332397461, 0.003704071044921875, 0.0037412643432617188, 0.003643035888671875, 0.0036857128143310547, 0.0036499500274658203, 0.0037195682525634766, 0.0036678314208984375, 0.003703594207763672, 0.003644227981567383, 0.0036983489990234375, 0.0036313533782958984, 0.0037238597869873047, 0.003675699234008789, 0.0036449432373046875, 0.0036902427673339844, 0.003638744354248047, 0.003686666488647461, 0.003662586212158203, 0.0037009716033935547, 0.003653287887573242, 0.0036916732788085938, 0.0036847591400146484, 0.0036840438842773438, 0.003696918487548828, 0.0037169456481933594, 0.0036885738372802734, 0.0036389827728271484, 0.0036773681640625, 0.003710508346557617, 0.0036950111389160156, 0.00379180908203125, 0.003709077835083008, 0.0037860870361328125, 0.0037114620208740234, 0.003679037094116211, 0.003684520721435547, 0.0036907196044921875, 0.003759622573852539, 0.003676176071166992, 0.003746509552001953, 0.0036382675170898438, 0.0036885738372802734, 0.0036361217498779297, 0.0036821365356445312, 0.003670215606689453, 0.0036835670471191406, 0.0036416053771972656, 0.0036630630493164062, 0.0036509037017822266, 0.0037479400634765625, 0.0037021636962890625, 0.003624439239501953, 0.003676891326904297, 0.003640413284301758, 0.0036840438842773438, 0.0036797523498535156, 0.0036673545837402344, 0.003641366958618164, 0.003775358200073242, 0.0036826133728027344, 0.0037152767181396484, 0.0036699771881103516, 0.003639698028564453, 0.0036830902099609375, 0.0036504268646240234, 0.003691434860229492, 0.0036656856536865234, 0.003675699234008789, 0.0036933422088623047, 0.0036323070526123047, 0.0036859512329101562, 0.003685474395751953, 0.00366973876953125, 0.0036363601684570312, 0.0036988258361816406, 0.003668546676635742, 0.0036771297454833984, 0.0036818981170654297, 0.0036880970001220703, 0.0037183761596679688, 0.0036492347717285156, 0.0036814212799072266, 0.0036847591400146484, 0.003686189651489258, 0.0037457942962646484, 0.0037081241607666016, 0.003682374954223633, 0.0039048194885253906, 0.0036711692810058594, 0.0036420822143554688, 0.0036787986755371094, 0.003621339797973633, 0.0036749839782714844, 0.0036869049072265625, 0.0037147998809814453, 0.0037195682525634766, 0.0037097930908203125, 0.003679990768432617, 0.0037403106689453125, 0.0037224292755126953, 0.0038521289825439453, 0.0036725997924804688, 0.0036437511444091797, 0.0037147998809814453, 0.003698110580444336, 0.003691434860229492, 0.0036847591400146484, 0.003641843795776367, 0.0036771297454833984, 0.003699064254760742, 0.0037605762481689453, 0.003674745559692383, 0.0036878585815429688, 0.0036551952362060547, 0.0037441253662109375, 0.003699064254760742, 0.0036590099334716797, 0.003705263137817383, 0.0036525726318359375, 0.0036993026733398438, 0.003690481185913086, 0.0036695003509521484, 0.0036454200744628906, 0.0037305355072021484, 0.0036907196044921875, 0.004117250442504883, 0.003698110580444336, 0.0037415027618408203, 0.0037355422973632812, 0.0036542415618896484, 0.003692150115966797, 0.00377655029296875, 0.0036466121673583984, 0.003690481185913086, 0.003650665283203125, 0.0037050247192382812, 0.003682374954223633, 0.0037202835083007812, 0.0037467479705810547, 0.0037081241607666016, 0.003701448440551758, 0.0038433074951171875, 0.0036971569061279297, 0.003673076629638672, 0.003709077835083008, 0.003683328628540039, 0.0037093162536621094, 0.003679513931274414, 0.003700733184814453, 0.00372314453125, 0.0036792755126953125, 0.0037016868591308594, 0.00371551513671875, 0.003705263137817383, 0.003656148910522461, 0.0036911964416503906, 0.003643035888671875, 0.0036950111389160156, 0.003699779510498047, 0.0037043094635009766, 0.0037012100219726562, 0.0036606788635253906, 0.003679037094116211, 0.003702878952026367, 0.003712177276611328, 0.0036487579345703125, 0.0036945343017578125, 0.003661632537841797, 0.0037479400634765625, 0.003724813461303711, 0.003658294677734375, 0.003687620162963867, 0.003655672073364258, 0.0037233829498291016, 0.0037088394165039062, 0.0036933422088623047, 0.003644227981567383, 0.0037016868591308594, 0.0037093162536621094, 0.0037026405334472656, 0.003706693649291992, 0.003663301467895508, 0.0036962032318115234, 0.0036668777465820312, 0.0037026405334472656, 0.003704071044921875, 0.0037195682525634766, 0.0037088394165039062, 0.0036559104919433594, 0.0039005279541015625, 0.0037076473236083984, 0.003691434860229492, 0.003746509552001953, 0.003721475601196289, 0.0036711692810058594, 0.0038809776306152344, 0.0036933422088623047, 0.003663778305053711, 0.00370025634765625, 0.0036673545837402344, 0.003692150115966797, 0.003731250762939453, 0.0037043094635009766, 0.003668069839477539, 0.003720521926879883, 0.0036916732788085938, 0.0036897659301757812, 0.003712892532348633, 0.0036706924438476562, 0.003768444061279297, 0.0036590099334716797, 0.003690958023071289, 0.003704071044921875, 0.003720521926879883, 0.003697633743286133, 0.0036602020263671875, 0.003708362579345703, 0.0036919116973876953, 0.0037152767181396484, 0.0036542415618896484, 0.003710508346557617, 0.0036573410034179688, 0.003764629364013672, 0.0037069320678710938, 0.0036652088165283203, 0.0036928653717041016, 0.0036623477935791016, 0.0036907196044921875, 0.0037317276000976562, 0.003698587417602539, 0.003656148910522461, 0.0037078857421875, 0.0036957263946533203, 0.003699779510498047, 0.0037240982055664062, 0.0036652088165283203, 0.0036916732788085938, 0.003650188446044922, 0.0037300586700439453, 0.0037190914154052734, 0.003711700439453125, 0.0037140846252441406, 0.003657817840576172, 0.003720998764038086, 0.0037260055541992188, 0.003715038299560547, 0.0037522315979003906, 0.003715038299560547, 0.003682851791381836, 0.00390625, 0.003725767135620117, 0.0036678314208984375, 0.003723621368408203, 0.003751516342163086, 0.0037560462951660156, 0.003660440444946289, 0.0036907196044921875, 0.003701448440551758, 0.0037038326263427734, 0.0036962032318115234, 0.003745555877685547, 0.003726482391357422, 0.003721952438354492, 0.0037250518798828125, 0.0036728382110595703, 0.0037298202514648438, 0.003713846206665039, 0.0037088394165039062, 0.0037457942962646484, 0.0036842823028564453, 0.003740549087524414, 0.003662109375, 0.0037202835083007812, 0.003733396530151367, 0.0037469863891601562, 0.004122257232666016, 0.003699779510498047, 0.0037293434143066406, 0.0037250518798828125, 0.003735065460205078, 0.0036885738372802734, 0.0037093162536621094, 0.0037157535552978516, 0.003778219223022461, 0.003709554672241211, 0.003690958023071289, 0.0037016868591308594, 0.003679990768432617, 0.0037348270416259766, 0.0037169456481933594, 0.003702402114868164, 0.003708362579345703, 0.003679990768432617, 0.003734111785888672, 0.0037300586700439453, 0.0037343502044677734, 0.003663778305053711, 0.003706216812133789, 0.00370025634765625, 0.0037217140197753906, 0.003883838653564453, 0.0036749839782714844, 0.003713369369506836, 0.0038461685180664062, 0.003726959228515625, 0.0037124156951904297, 0.003706216812133789, 0.003710031509399414, 0.003735780715942383, 0.0037255287170410156, 0.0037283897399902344, 0.0037262439727783203, 0.0037648677825927734, 0.0037267208099365234, 0.003695249557495117, 0.0037245750427246094, 0.0037271976470947266, 0.003732919692993164, 0.003720521926879883, 0.0036988258361816406, 0.003712892532348633, 0.0037059783935546875, 0.0037186145782470703, 0.0036802291870117188, 0.0037288665771484375, 0.003682374954223633, 0.003777742385864258, 0.003744840621948242, 0.0036814212799072266, 0.0037279129028320312, 0.0036826133728027344, 0.003713846206665039, 0.003729104995727539, 0.0037517547607421875, 0.003768444061279297, 0.003702878952026367, 0.003713846206665039, 0.003725290298461914, 0.003725290298461914, 0.0037012100219726562, 0.003713846206665039, 0.003682374954223633, 0.00371551513671875, 0.0037949085235595703, 0.0036783218383789062, 0.003763914108276367, 0.00370025634765625, 0.0037152767181396484, 0.0037508010864257812, 0.0037615299224853516, 0.003679990768432617, 0.003744363784790039, 0.003761768341064453, 0.0038251876831054688, 0.003753185272216797, 0.0036759376525878906, 0.003705739974975586, 0.0038373470306396484, 0.0037627220153808594, 0.003787994384765625, 0.003675222396850586, 0.003712892532348633, 0.0036783218383789062, 0.0037317276000976562, 0.003726959228515625, 0.0037522315979003906, 0.003675222396850586, 0.003778696060180664, 0.0037686824798583984, 0.003969430923461914, 0.0037336349487304688, 0.0036935806274414062, 0.0037221908569335938, 0.0036923885345458984, 0.003765106201171875, 0.0037729740142822266, 0.0036933422088623047, 0.0037314891815185547, 0.0037069320678710938, 0.003848552703857422, 0.003751516342163086, 0.00395512580871582, 0.0037658214569091797, 0.0037746429443359375, 0.003759145736694336, 0.003740072250366211, 0.0037522315979003906, 0.003721952438354492, 0.003731966018676758, 0.0037260055541992188, 0.0037424564361572266, 0.0038127899169921875, 0.003680706024169922, 0.0037801265716552734, 0.0037026405334472656, 0.0037453174591064453, 0.003766775131225586, 0.0037841796875, 0.0037038326263427734, 0.0038411617279052734, 0.0037572383880615234, 0.003757476806640625, 0.0037543773651123047, 0.003759145736694336, 0.0037789344787597656, 0.003694772720336914, 0.0037412643432617188, 0.0038814544677734375, 0.0037288665771484375, 0.0037603378295898438, 0.0038290023803710938, 0.004055976867675781, 0.003745555877685547, 0.003756284713745117, 0.0037589073181152344, 0.003716707229614258, 0.003743886947631836, 0.0037660598754882812, 0.003747701644897461, 0.003705263137817383, 0.0037238597869873047, 0.003914833068847656, 0.0037822723388671875, 0.0037505626678466797, 0.003725767135620117, 0.003762483596801758, 0.0037393569946289062, 0.0037381649017333984, 0.003773212432861328, 0.0037469863891601562, 0.0037467479705810547, 0.003744840621948242, 0.003732442855834961, 0.003695964813232422, 0.0037555694580078125, 0.003691434860229492, 0.0037403106689453125, 0.003764629364013672, 0.003742694854736328, 0.003757953643798828, 0.0037047863006591797, 0.003748655319213867, 0.003731966018676758, 0.0038995742797851562, 0.0037233829498291016, 0.003765106201171875, 0.003746509552001953, 0.0037276744842529297, 0.0037539005279541016, 0.003699064254760742, 0.0037300586700439453, 0.0036797523498535156, 0.0037271976470947266, 0.003738880157470703, 0.0037605762481689453, 0.0037260055541992188, 0.0037140846252441406, 0.0037164688110351562, 0.003767251968383789, 0.0037245750427246094, 0.0037915706634521484, 0.0037453174591064453, 0.003759622573852539, 0.0038602352142333984, 0.003743886947631836, 0.0036857128143310547, 0.003756284713745117, 0.003709554672241211, 0.003740549087524414, 0.0037429332733154297, 0.0037631988525390625, 0.0037169456481933594, 0.0036950111389160156, 0.003736734390258789, 0.003745555877685547, 0.003735065460205078, 0.003769397735595703, 0.004011392593383789, 0.0037462711334228516, 0.0037500858306884766, 0.0040132999420166016, 0.0037376880645751953, 0.0037338733673095703, 0.003696441650390625, 0.0037178993225097656, 0.0037393569946289062, 0.003731250762939453, 0.00374603271484375, 0.0036933422088623047, 0.0037441253662109375, 0.003696441650390625, 0.003726959228515625, 0.003720998764038086, 0.003728151321411133, 0.003725767135620117, 0.0036966800689697266, 0.0037298202514648438, 0.0037374496459960938, 0.003743410110473633, 0.003702402114868164, 0.003740072250366211, 0.003690481185913086, 0.003768444061279297, 0.0037224292755126953, 0.003690481185913086, 0.0037453174591064453, 0.0036878585815429688, 0.0037164688110351562, 0.003747701644897461, 0.003721952438354492, 0.003705263137817383, 0.003765106201171875, 0.003832578659057617, 0.0038106441497802734, 0.0037627220153808594, 0.0037500858306884766, 0.003741741180419922, 0.003926515579223633, 0.00382232666015625, 0.003760814666748047, 0.003736257553100586, 0.0037298202514648438, 0.003752470016479492, 0.003738880157470703, 0.0037751197814941406, 0.0037767887115478516, 0.003794431686401367, 0.0037584304809570312, 0.0037505626678466797, 0.0037016868591308594, 0.003749847412109375, 0.0037508010864257812, 0.0037539005279541016, 0.003701925277709961, 0.0037598609924316406, 0.0037376880645751953, 0.0037610530853271484, 0.0037336349487304688, 0.0037109851837158203, 0.0037665367126464844, 0.0036983489990234375, 0.003720521926879883, 0.0038323402404785156, 0.003713846206665039, 0.0037517547607421875, 0.0037078857421875, 0.0037560462951660156, 0.003742218017578125, 0.0037565231323242188, 0.0036983489990234375, 0.0037496089935302734, 0.003753185272216797, 0.0037300586700439453, 0.003732442855834961, 0.003687143325805664, 0.003737926483154297, 0.003696441650390625, 0.0037229061126708984, 0.003729581832885742, 0.0037641525268554688, 0.0037326812744140625, 0.003694772720336914, 0.003752470016479492, 0.0037343502044677734, 0.0037462711334228516, 0.0036933422088623047, 0.003751993179321289, 0.0038449764251708984, 0.0037665367126464844, 0.003793954849243164, 0.0038013458251953125, 0.0037467479705810547, 0.0037102699279785156, 0.003748178482055664, 0.0038335323333740234, 0.0038461685180664062, 0.0037348270416259766, 0.0037517547607421875, 0.003711223602294922, 0.003772258758544922, 0.003765583038330078, 0.0037393569946289062, 0.003795146942138672, 0.003734588623046875, 0.0037031173706054688, 0.003747701644897461, 0.003685474395751953, 0.003747224807739258, 0.003743410110473633, 0.0037326812744140625, 0.0036880970001220703, 0.0037686824798583984, 0.0037453174591064453, 0.003747701644897461, 0.0037360191345214844, 0.0036852359771728516, 0.0037384033203125, 0.0036966800689697266, 0.0037288665771484375, 0.003795623779296875, 0.003710508346557617, 0.0037276744842529297, 0.0037126541137695312, 0.0037381649017333984, 0.0037698745727539062, 0.003732442855834961, 0.003702878952026367, 0.003802776336669922, 0.003801107406616211, 0.0037403106689453125, 0.003752470016479492, 0.003738880157470703, 0.003736734390258789, 0.004780292510986328, 0.0037343502044677734, 0.003743886947631836, 0.003743410110473633, 0.00421452522277832, 0.003715038299560547, 0.0037593841552734375, 0.003850221633911133, 0.0039653778076171875, 0.0037403106689453125, 0.0038428306579589844, 0.003763437271118164, 0.003789186477661133, 0.003749847412109375, 0.003784656524658203, 0.0037126541137695312, 0.003778696060180664, 0.0038743019104003906, 0.0038232803344726562, 0.003842592239379883, 0.003862619400024414, 0.00397801399230957, 0.003997325897216797, 0.0038623809814453125, 0.0038628578186035156, 0.003911256790161133, 0.0038793087005615234, 0.003823995590209961, 0.003851652145385742, 0.00385284423828125, 0.003945827484130859, 0.0038416385650634766, 0.0038700103759765625, 0.0038504600524902344, 0.004154682159423828, 0.0039098262786865234, 0.0038595199584960938, 0.003931760787963867, 0.004000663757324219, 0.0038611888885498047, 0.0038847923278808594, 0.0038530826568603516, 0.003915309906005859, 0.003894805908203125, 0.0038728713989257812, 0.0038971900939941406, 0.0038263797760009766, 0.003917217254638672, 0.0038695335388183594, 0.0038497447967529297, 0.0038928985595703125, 0.003949403762817383, 0.003833770751953125, 0.0038628578186035156, 0.0038759708404541016, 0.003847360610961914, 0.0038738250732421875, 0.003850698471069336, 0.003958463668823242, 0.003864765167236328, 0.003815889358520508, 0.003860950469970703, 0.0039615631103515625, 0.0038957595825195312, 0.0038466453552246094, 0.0038056373596191406, 0.0038836002349853516, 0.0038008689880371094, 0.0038433074951171875, 0.003949642181396484, 0.003829479217529297, 0.003856658935546875, 0.0038442611694335938, 0.0038471221923828125, 0.003874063491821289, 0.0038666725158691406, 0.0038604736328125, 0.003805398941040039, 0.0038328170776367188, 0.0038576126098632812, 0.003854036331176758, 0.003826618194580078, 0.003859996795654297, 0.003842592239379883, 0.0038497447967529297, 0.0038483142852783203, 0.0038330554962158203, 0.0038542747497558594, 0.0038497447967529297, 0.0038437843322753906, 0.0038754940032958984, 0.003828763961791992, 0.003841400146484375, 0.003822803497314453, 0.0038557052612304688, 0.003889322280883789, 0.003899812698364258, 0.0038912296295166016, 0.0038056373596191406, 0.0038471221923828125, 0.0038721561431884766, 0.003869295120239258, 0.003839731216430664, 0.0038101673126220703, 0.0038521289825439453, 0.0038759708404541016, 0.0038690567016601562, 0.003894329071044922, 0.003822803497314453, 0.0038716793060302734, 0.003973245620727539, 0.003912687301635742, 0.0038268566131591797, 0.003854513168334961, 0.003999948501586914, 0.003845691680908203, 0.003881692886352539, 0.0038335323333740234, 0.003871440887451172, 0.0038251876831054688, 0.003937721252441406, 0.003872394561767578, 0.003813028335571289, 0.0038902759552001953, 0.003802776336669922, 0.0038962364196777344, 0.003973245620727539, 0.003855466842651367, 0.003851652145385742, 0.0038313865661621094, 0.0038979053497314453, 0.003893613815307617, 0.0038678646087646484, 0.0038423538208007812, 0.00384521484375, 0.003855466842651367, 0.003873586654663086, 0.003838777542114258, 0.0038688182830810547, 0.0038046836853027344, 0.0038633346557617188, 0.003870725631713867, 0.003857135772705078, 0.003818511962890625, 0.0038704872131347656, 0.0038819313049316406, 0.003856658935546875, 0.0038781166076660156, 0.0038154125213623047, 0.0038614273071289062, 0.0038225650787353516, 0.0039196014404296875, 0.003876209259033203, 0.0037992000579833984, 0.0038747787475585938, 0.0038268566131591797, 0.0038657188415527344, 0.0039157867431640625, 0.003852367401123047, 0.003871917724609375, 0.0038573741912841797, 0.0038802623748779297, 0.003968715667724609, 0.003915548324584961, 0.003877401351928711, 0.003833770751953125, 0.003996133804321289, 0.003892660140991211, 0.0038771629333496094, 0.0038967132568359375, 0.003938913345336914, 0.0038595199584960938, 0.0038537979125976562, 0.0038526058197021484, 0.0038046836853027344, 0.003910064697265625, 0.003966569900512695, 0.003877401351928711, 0.003865480422973633, 0.0038347244262695312, 0.003864288330078125, 0.003851175308227539, 0.003912687301635742, 0.0038733482360839844, 0.0038251876831054688, 0.003889322280883789, 0.003822803497314453, 0.0038690567016601562, 0.003941774368286133, 0.0038285255432128906, 0.003869771957397461, 0.003861665725708008, 0.0038559436798095703, 0.003849029541015625, 0.003888368606567383, 0.0038652420043945312, 0.0038607120513916016, 0.003853321075439453, 0.003870725631713867, 0.003859281539916992, 0.00390934944152832, 0.0038344860076904297, 0.0038580894470214844, 0.003871440887451172, 0.003860950469970703, 0.003826618194580078, 0.0038747787475585938, 0.003882884979248047, 0.003870248794555664, 0.003894805908203125, 0.0038480758666992188, 0.003876924514770508, 0.0038471221923828125, 0.0039823055267333984, 0.003883838653564453, 0.0038466453552246094, 0.0039031505584716797, 0.00391387939453125, 0.003935098648071289, 0.003904581069946289, 0.0038576126098632812, 0.003914594650268555, 0.00382232666015625, 0.0038635730743408203, 0.0038712024688720703, 0.003923892974853516, 0.003874063491821289, 0.003820180892944336, 0.0038728713989257812, 0.0038437843322753906, 0.0039043426513671875, 0.0038902759552001953, 0.0038297176361083984, 0.0038492679595947266, 0.0038826465606689453, 0.0038805007934570312, 0.0038263797760009766, 0.0038738250732421875, 0.0038704872131347656, 0.0038785934448242188, 0.0039048194885253906, 0.0039005279541015625, 0.0038712024688720703, 0.0038781166076660156, 0.0038678646087646484, 0.003841400146484375, 0.003867626190185547, 0.0038771629333496094, 0.003881692886352539, 0.003834247589111328, 0.003916501998901367, 0.003885984420776367, 0.0038666725158691406, 0.003877401351928711, 0.0038726329803466797, 0.0038661956787109375, 0.0039098262786865234, 0.0039033889770507812, 0.00390934944152832, 0.003834962844848633, 0.0038909912109375, 0.003854513168334961, 0.00393986701965332, 0.0038785934448242188, 0.0038595199584960938, 0.0039958953857421875, 0.003873109817504883, 0.0051805973052978516, 0.0043010711669921875, 0.004200458526611328, 0.004721164703369141, 0.004292488098144531, 0.004251956939697266, 0.004342555999755859, 0.005349874496459961, 0.00663304328918457, 0.004311084747314453, 0.0055806636810302734, 0.00553441047668457, 0.005085468292236328, 0.005325794219970703, 0.005513191223144531, 0.005452871322631836, 0.005278110504150391, 0.0055522918701171875, 0.0053234100341796875, 0.00554656982421875, 0.0052297115325927734, 0.005158424377441406, 0.005095243453979492, 0.005032539367675781, 0.0043125152587890625, 0.0039043426513671875, 0.0040740966796875, 0.004339456558227539, 0.0051119327545166016, 0.005969524383544922, 0.004175901412963867, 0.0039784908294677734, 0.00392913818359375, 0.003949165344238281, 0.003912210464477539, 0.003911256790161133, 0.0039288997650146484, 0.003910541534423828, 0.003912687301635742, 0.003873586654663086, 0.003930807113647461, 0.004019737243652344, 0.003911733627319336, 0.00391840934753418, 0.003976345062255859, 0.00413203239440918, 0.003942728042602539, 0.003942012786865234, 0.0039365291595458984, 0.003881216049194336, 0.003942966461181641, 0.003922939300537109, 0.003939628601074219, 0.003926992416381836, 0.003857135772705078, 0.0039119720458984375, 0.003866434097290039, 0.0038847923278808594, 0.003953695297241211, 0.003879547119140625, 0.0039017200469970703, 0.003862142562866211, 0.003898143768310547, 0.003918170928955078, 0.003917217254638672, 0.003901243209838867, 0.0038475990295410156, 0.003884553909301758, 0.0039250850677490234, 0.00392603874206543, 0.003930807113647461, 0.003926277160644531, 0.0038869380950927734, 0.003933429718017578, 0.003945827484130859, 0.0039997100830078125, 0.003968238830566406, 0.00392460823059082, 0.003918886184692383, 0.003892183303833008, 0.003896474838256836, 0.003901243209838867, 0.003870248794555664, 0.004098415374755859, 0.003931999206542969, 0.003908395767211914, 0.003852367401123047, 0.003904104232788086, 0.0039060115814208984, 0.003910064697265625, 0.0039021968841552734, 0.0038712024688720703, 0.004026651382446289, 0.003917217254638672, 0.003904581069946289, 0.0039005279541015625, 0.003994464874267578, 0.003908634185791016, 0.003922939300537109, 0.003935575485229492, 0.003926992416381836, 0.0038635730743408203, 0.003920793533325195, 0.003892660140991211, 0.003927469253540039, 0.0038976669311523438, 0.0038526058197021484, 0.0039370059967041016, 0.0038623809814453125, 0.003943681716918945, 0.003945350646972656, 0.0038826465606689453, 0.0039463043212890625, 0.003855466842651367, 0.00394129753112793, 0.003958702087402344, 0.0038743019104003906, 0.0039141178131103516, 0.0038547515869140625, 0.0039055347442626953, 0.003935337066650391, 0.0039215087890625, 0.0039212703704833984, 0.003924369812011719, 0.0039081573486328125, 0.003912210464477539, 0.003925323486328125, 0.003902912139892578, 0.0038573741912841797, 0.003887176513671875, 0.0038979053497314453, 0.003904581069946289, 0.0038690567016601562, 0.0038957595825195312, 0.003915309906005859, 0.0039615631103515625, 0.003922224044799805, 0.003888368606567383, 0.0039370059967041016, 0.0039293766021728516, 0.003916740417480469, 0.0039052963256835938, 0.003872394561767578, 0.004022121429443359, 0.003996610641479492, 0.0039594173431396484, 0.003956317901611328, 0.0040013790130615234, 0.0039196014404296875, 0.00399327278137207, 0.003922939300537109, 0.004363298416137695, 0.003909111022949219, 0.0039310455322265625, 0.0039632320404052734, 0.003932476043701172, 0.004054546356201172, 0.0038928985595703125, 0.003930330276489258, 0.003961086273193359, 0.003952503204345703, 0.003913402557373047, 0.003923177719116211, 0.003931283950805664, 0.003930330276489258, 0.003958225250244141, 0.0038869380950927734, 0.0039327144622802734, 0.00393986701965332, 0.00391840934753418, 0.0046384334564208984, 0.0040283203125, 0.0039577484130859375, 0.003909111022949219, 0.004351615905761719, 0.003975391387939453, 0.003870725631713867, 0.00392913818359375, 0.0038957595825195312, 0.003915309906005859, 0.003941774368286133, 0.003873586654663086, 0.003920316696166992, 0.0039288997650146484, 0.003919363021850586, 0.0038535594940185547, 0.003905057907104492, 0.003966331481933594, 0.0039522647857666016, 0.003933906555175781, 0.0038785934448242188, 0.0039272308349609375, 0.003904581069946289, 0.004022359848022461, 0.00397038459777832, 0.0038673877716064453, 0.003912448883056641, 0.003935098648071289, 0.004029512405395508, 0.003935098648071289, 0.00393986701965332, 0.003951311111450195, 0.0039386749267578125, 0.003928661346435547, 0.003931999206542969, 0.003940582275390625, 0.003924131393432617, 0.0038802623748779297, 0.003977537155151367, 0.003934383392333984, 0.003885507583618164, 0.003883838653564453, 0.003844738006591797, 0.0039441585540771484, 0.003977537155151367, 0.003880023956298828, 0.003949165344238281, 0.003861665725708008, 0.003908395767211914, 0.003951072692871094, 0.0038733482360839844, 0.0039038658142089844, 0.0038902759552001953, 0.003930568695068359, 0.003923177719116211, 0.0039501190185546875, 0.00391840934753418, 0.0038847923278808594, 0.003919124603271484, 0.0039556026458740234, 0.003920316696166992, 0.00391697883605957, 0.0038623809814453125, 0.0038785934448242188, 0.003983020782470703, 0.004094839096069336, 0.004029989242553711, 0.003894329071044922, 0.003930568695068359, 0.0038900375366210938, 0.003912925720214844, 0.0038766860961914062, 0.0039043426513671875, 0.003936052322387695, 0.003981113433837891, 0.003921031951904297, 0.0039255619049072266, 0.003953456878662109, 0.003927946090698242, 0.004007577896118164, 0.0039055347442626953, 0.003907680511474609, 0.003926277160644531, 0.003922700881958008, 0.003887176513671875, 0.003930568695068359, 0.003937244415283203, 0.0039293766021728516, 0.003979206085205078, 0.0040934085845947266, 0.003920078277587891, 0.0039217472076416016, 0.003935098648071289, 0.003925800323486328, 0.00396275520324707, 0.003926515579223633, 0.003906965255737305, 0.003908395767211914, 0.003907203674316406, 0.004039764404296875, 0.004334926605224609, 0.0039746761322021484, 0.003907680511474609, 0.003935337066650391, 0.003961801528930664, 0.003932952880859375, 0.003935098648071289, 0.003942966461181641, 0.00391697883605957, 0.0039119720458984375, 0.0039556026458740234, 0.0039594173431396484, 0.003937721252441406, 0.00389862060546875, 0.003926992416381836, 0.003929615020751953, 0.0039081573486328125, 0.003912687301635742, 0.003952741622924805, 0.003948211669921875, 0.003872394561767578, 0.0039632320404052734, 0.003936052322387695, 0.003885030746459961, 0.0039327144622802734, 0.003988742828369141, 0.004012107849121094, 0.0039310455322265625, 0.0038776397705078125, 0.00402522087097168, 0.0038788318634033203, 0.003938436508178711, 0.004006862640380859, 0.003875732421875, 0.003945827484130859, 0.0038738250732421875, 0.003927707672119141, 0.003958225250244141, 0.0039119720458984375, 0.003958463668823242, 0.0038924217224121094, 0.0038993358612060547, 0.0039043426513671875, 0.003927946090698242, 0.003909587860107422, 0.00388336181640625, 0.003934144973754883, 0.003985404968261719, 0.0038995742797851562, 0.00394892692565918, 0.0038836002349853516, 0.003923892974853516, 0.003919124603271484, 0.003916501998901367, 0.0038938522338867188, 0.00396728515625, 0.0039157867431640625, 0.003854036331176758, 0.003925323486328125, 0.00390934944152832, 0.003956794738769531, 0.003936052322387695, 0.003881216049194336, 0.0039520263671875, 0.003901958465576172, 0.003937959671020508, 0.003978252410888672, 0.003887653350830078, 0.003925800323486328, 0.0038843154907226562, 0.003929853439331055, 0.003896474838256836, 0.003958702087402344, 0.003922939300537109, 0.003888845443725586, 0.003954648971557617, 0.004001140594482422, 0.0039730072021484375, 0.003916501998901367, 0.003893613815307617, 0.003927707672119141, 0.0040280818939208984, 0.003998994827270508, 0.0039157867431640625, 0.0038909912109375, 0.003934144973754883, 0.003960132598876953, 0.003922700881958008, 0.003942251205444336, 0.0038840770721435547, 0.003937721252441406, 0.0039119720458984375, 0.004594564437866211, 0.003972768783569336, 0.003926992416381836, 0.003929615020751953, 0.004363059997558594, 0.003937244415283203, 0.0039250850677490234, 0.0039048194885253906, 0.003914594650268555, 0.003931760787963867, 0.003923892974853516, 0.003989458084106445, 0.0039212703704833984, 0.003959178924560547, 0.003944873809814453, 0.003905773162841797, 0.0039060115814208984, 0.003942728042602539, 0.00392913818359375, 0.003945589065551758, 0.003922700881958008, 0.0039060115814208984, 0.003971099853515625, 0.003943681716918945, 0.003947734832763672, 0.003932476043701172, 0.003874540328979492, 0.003973245620727539, 0.0039708614349365234, 0.003924846649169922, 0.003929853439331055, 0.0039103031158447266, 0.003953456878662109, 0.0039997100830078125, 0.004052162170410156, 0.003949642181396484, 0.0039136409759521484, 0.00394439697265625, 0.003940105438232422, 0.004087686538696289, 0.003953695297241211, 0.0039556026458740234, 0.004038572311401367, 0.003991127014160156, 0.003973484039306641, 0.003962278366088867, 0.0039501190185546875, 0.003971099853515625, 0.004505157470703125, 0.003969669342041016, 0.003930330276489258, 0.0038962364196777344, 0.003925800323486328, 0.0039598941802978516, 0.003946542739868164, 0.003937721252441406, 0.00389862060546875, 0.00396418571472168, 0.003913164138793945, 0.004009246826171875, 0.003947019577026367, 0.003924131393432617, 0.004003763198852539, 0.003913402557373047, 0.003990650177001953, 0.003942251205444336, 0.0039157867431640625, 0.0039577484130859375, 0.003915548324584961, 0.003937721252441406, 0.00399327278137207, 0.003922462463378906, 0.003968477249145508, 0.003894805908203125, 0.0039217472076416016, 0.003996133804321289, 0.003891468048095703, 0.003945112228393555, 0.003919124603271484, 0.003918886184692383, 0.003939390182495117, 0.0039713382720947266, 0.003952741622924805, 0.003908634185791016, 0.0039577484130859375, 0.004056692123413086, 0.003954887390136719, 0.003955841064453125, 0.003952980041503906, 0.0039637088775634766, 0.004077434539794922, 0.003971576690673828, 0.003977298736572266, 0.0040013790130615234, 0.003966093063354492, 0.003962039947509766, 0.003936290740966797, 0.003984212875366211, 0.003933429718017578, 0.003948211669921875, 0.003938436508178711, 0.0039327144622802734, 0.003953695297241211, 0.0039327144622802734, 0.003938913345336914, 0.0039441585540771484, 0.003954172134399414, 0.0039746761322021484, 0.003925800323486328, 0.003931283950805664, 0.003949880599975586, 0.003978729248046875, 0.003926515579223633, 0.003964662551879883, 0.003945350646972656, 0.003941774368286133, 0.00394439697265625, 0.003928661346435547, 0.003987789154052734, 0.003924846649169922, 0.003959178924560547, 0.003946065902709961, 0.003927946090698242, 0.003957509994506836, 0.0039348602294921875, 0.003953695297241211, 0.003943204879760742, 0.003933429718017578, 0.003946542739868164, 0.003946542739868164, 0.003964662551879883, 0.00397181510925293, 0.003939628601074219, 0.003971099853515625, 0.003956317901611328, 0.004015922546386719, 0.003993511199951172, 0.003945112228393555, 0.003966808319091797, 0.0039522647857666016, 0.0040209293365478516, 0.0039675235748291016, 0.004011631011962891, 0.00396275520324707, 0.003902912139892578, 0.004005908966064453, 0.003987789154052734, 0.003911256790161133, 0.003949880599975586, 0.003923177719116211, 0.0041446685791015625, 0.003988742828369141, 0.003933429718017578, 0.003950834274291992, 0.003929615020751953, 0.0039980411529541016, 0.0039594173431396484, 0.003930568695068359, 0.003952503204345703, 0.00399327278137207, 0.004144430160522461, 0.003976345062255859, 0.003949880599975586, 0.003983259201049805, 0.003954172134399414, 0.003994941711425781, 0.003971576690673828, 0.0039403438568115234, 0.0040247440338134766, 0.00396275520324707, 0.004038810729980469, 0.003966569900512695, 0.003946542739868164, 0.0039713382720947266, 0.003952741622924805, 0.003976106643676758, 0.004038095474243164, 0.003951311111450195, 0.003954172134399414, 0.003927946090698242, 0.003954887390136719, 0.004006624221801758, 0.003927707672119141, 0.003943920135498047, 0.003934383392333984, 0.00393223762512207, 0.00418543815612793, 0.003928661346435547, 0.003943443298339844, 0.003951549530029297, 0.003962278366088867, 0.004184722900390625, 0.003951311111450195, 0.00394892692565918, 0.00391387939453125, 0.003952741622924805, 0.004014253616333008, 0.0039331912994384766, 0.003954172134399414, 0.003925323486328125, 0.003933429718017578, 0.0039479732513427734, 0.003977537155151367, 0.003995656967163086, 0.0038928985595703125, 0.003978729248046875, 0.003971099853515625, 0.00398564338684082, 0.0039865970611572266, 0.0039479732513427734, 0.003948211669921875, 0.003972053527832031, 0.003983259201049805, 0.003957033157348633, 0.0039217472076416016, 0.0039539337158203125, 0.004002809524536133, 0.003949403762817383, 0.0039675235748291016, 0.00392460823059082, 0.003937244415283203, 0.0039288997650146484, 0.003975391387939453, 0.004000663757324219, 0.003926277160644531, 0.003950834274291992, 0.003942012786865234, 0.003977537155151367, 0.004289388656616211, 0.003925800323486328, 0.003960847854614258, 0.00400853157043457, 0.00397181510925293, 0.003984928131103516, 0.003926753997802734, 0.003958702087402344, 0.004050016403198242, 0.004008054733276367, 0.003961086273193359, 0.0039136409759521484, 0.003945112228393555, 0.0040683746337890625, 0.003965139389038086, 0.003936052322387695, 0.003977775573730469, 0.003938198089599609, 0.003972053527832031, 0.0039598941802978516, 0.003920316696166992, 0.0039730072021484375, 0.00395655632019043, 0.004000663757324219, 0.004011869430541992, 0.003931522369384766, 0.00399470329284668, 0.00394439697265625, 0.003957509994506836, 0.003954887390136719, 0.003925323486328125, 0.003979682922363281, 0.0039637088775634766, 0.0039441585540771484, 0.003952503204345703, 0.003904581069946289, 0.003964900970458984, 0.003945350646972656, 0.003968000411987305, 0.003949403762817383, 0.00392913818359375, 0.0039615631103515625, 0.003979206085205078, 0.003968954086303711, 0.003965139389038086, 0.003951549530029297, 0.00395655632019043, 0.003909587860107422, 0.0040435791015625, 0.00397491455078125, 0.003925800323486328, 0.00394892692565918, 0.003922224044799805, 0.004047870635986328, 0.003946065902709961, 0.003927469253540039, 0.0039517879486083984, 0.003943204879760742, 0.0041141510009765625, 0.00398707389831543, 0.0039255619049072266, 0.004011631011962891, 0.003949403762817383, 0.00400233268737793, 0.004082679748535156, 0.003931999206542969, 0.003968000411987305, 0.003990650177001953, 0.0040454864501953125, 0.0040018558502197266, 0.0039310455322265625, 0.0039517879486083984, 0.00399470329284668, 0.003988981246948242, 0.004030466079711914, 0.003987789154052734, 0.00397944450378418, 0.003943920135498047, 0.0039441585540771484, 0.0040013790130615234, 0.003943920135498047, 0.003960132598876953, 0.003968954086303711, 0.003964424133300781, 0.003983736038208008, 0.003945589065551758, 0.0041980743408203125, 0.003998756408691406, 0.0039937496185302734, 0.003995180130004883, 0.0039424896240234375, 0.003988027572631836, 0.003972291946411133, 0.003977298736572266, 0.004004716873168945, 0.003969907760620117, 0.004004955291748047, 0.003971099853515625, 0.003966331481933594, 0.003988742828369141, 0.003951549530029297, 0.003977537155151367, 0.003976345062255859, 0.003985404968261719, 0.003983497619628906, 0.003930091857910156, 0.003985881805419922, 0.0039577484130859375, 0.003962516784667969, 0.0041046142578125, 0.003942012786865234, 0.003977060317993164, 0.004001140594482422, 0.003962993621826172, 0.004102230072021484, 0.003975391387939453, 0.00397944450378418, 0.004029035568237305, 0.003970623016357422, 0.003976345062255859, 0.003946542739868164, 0.00396728515625, 0.003966093063354492, 0.003991603851318359, 0.0039539337158203125, 0.004002571105957031, 0.00397181510925293, 0.003986358642578125, 0.003958225250244141, 0.00404667854309082, 0.0039997100830078125, 0.00402522087097168, 0.003984212875366211, 0.0039637088775634766, 0.003942012786865234, 0.004025459289550781, 0.003981828689575195, 0.0039615631103515625, 0.003987789154052734, 0.003949403762817383, 0.003965139389038086, 0.00394749641418457, 0.004007577896118164, 0.0039675235748291016, 0.003953456878662109, 0.003998994827270508, 0.003985404968261719, 0.004073143005371094, 0.004008769989013672, 0.004065990447998047, 0.004007101058959961, 0.003974199295043945, 0.003957986831665039, 0.003996610641479492, 0.003964900970458984, 0.004035234451293945, 0.004008054733276367, 0.0039408206939697266, 0.003990888595581055, 0.0040776729583740234, 0.004008293151855469, 0.003983736038208008, 0.003928661346435547, 0.00397038459777832, 0.004057168960571289, 0.0040149688720703125, 0.003997087478637695, 0.00393223762512207, 0.003981828689575195, 0.0039997100830078125, 0.0039920806884765625, 0.0039827823638916016, 0.0039293766021728516, 0.004029512405395508, 0.004162788391113281, 0.004057168960571289, 0.003983259201049805, 0.003930330276489258, 0.004022121429443359, 0.003989696502685547, 0.004030942916870117, 0.004000663757324219, 0.003992319107055664, 0.004060983657836914, 0.004022121429443359, 0.003999471664428711, 0.004011869430541992, 0.003973245620727539, 0.003995180130004883, 0.00399327278137207, 0.003997087478637695, 0.003972530364990234, 0.003945589065551758, 0.00402379035949707, 0.00397038459777832, 0.004006147384643555, 0.004021167755126953, 0.003933429718017578, 0.0040013790130615234, 0.004022121429443359, 0.003997802734375, 0.0039768218994140625, 0.0039408206939697266, 0.003970146179199219, 0.003983974456787109, 0.004017353057861328, 0.003997325897216797, 0.0039615631103515625, 0.00399327278137207, 0.004090070724487305, 0.004002094268798828, 0.003977775573730469, 0.003960132598876953, 0.0040209293365478516, 0.0041277408599853516, 0.0040204524993896484, 0.004010200500488281, 0.003952503204345703, 0.004025936126708984, 0.004000186920166016, 0.004012346267700195, 0.003993988037109375, 0.003958463668823242, 0.003997087478637695, 0.0040090084075927734, 0.004004955291748047, 0.004004001617431641, 0.003936052322387695, 0.00397181510925293, 0.003963470458984375, 0.003985166549682617, 0.003972053527832031, 0.003943204879760742, 0.004008054733276367, 0.003995656967163086, 0.0040035247802734375, 0.004000425338745117, 0.003964900970458984, 0.0039730072021484375, 0.003997802734375, 0.004027843475341797, 0.00397181510925293, 0.003930091857910156, 0.00402522087097168, 0.004067182540893555, 0.0040416717529296875, 0.004004955291748047, 0.004001140594482422, 0.003981351852416992, 0.004027128219604492, 0.004007577896118164, 0.003997087478637695, 0.003942728042602539, 0.0039865970611572266, 0.003987550735473633, 0.0039904117584228516, 0.0039958953857421875, 0.003943443298339844, 0.0039865970611572266, 0.004151344299316406, 0.003996372222900391, 0.003987312316894531, 0.0039517879486083984, 0.004010438919067383, 0.004129171371459961, 0.0040628910064697266, 0.004036664962768555, 0.003967761993408203, 0.004004955291748047, 0.0040471553802490234, 0.004006147384643555, 0.0040242671966552734, 0.003952980041503906, 0.00399470329284668, 0.004007101058959961, 0.004064321517944336, 0.004011631011962891, 0.003957509994506836, 0.0040056705474853516, 0.004015445709228516, 0.004019737243652344, 0.004033327102661133, 0.003946065902709961, 0.0040225982666015625, 0.003985166549682617, 0.004011869430541992, 0.003991603851318359, 0.003940105438232422, 0.003981828689575195, 0.004041433334350586, 0.004030942916870117, 0.0040130615234375, 0.003956317901611328, 0.0040130615234375, 0.004006147384643555, 0.003992795944213867, 0.003999233245849609, 0.003964424133300781, 0.003981590270996094, 0.003997802734375, 0.004019260406494141, 0.0039746761322021484, 0.003976106643676758, 0.004012346267700195, 0.00400233268737793, 0.00400996208190918, 0.003973245620727539, 0.003954172134399414, 0.004029273986816406, 0.004115581512451172, 0.004091501235961914, 0.004000425338745117, 0.003949403762817383, 0.004029035568237305, 0.004119157791137695, 0.0040738582611083984, 0.0040264129638671875, 0.0040264129638671875, 0.0040225982666015625, 0.004042387008666992, 0.004012107849121094, 0.00405430793762207, 0.0040454864501953125, 0.004010438919067383, 0.004073619842529297, 0.003966093063354492, 0.003995418548583984, 0.00396275520324707, 0.003979682922363281, 0.0040552616119384766, 0.003964900970458984, 0.003985881805419922, 0.003965139389038086, 0.003996849060058594, 0.00400543212890625, 0.004023313522338867, 0.004060506820678711, 0.004037380218505859, 0.00400853157043457, 0.0040471553802490234, 0.003964900970458984, 0.004052639007568359, 0.0040130615234375, 0.0040435791015625, 0.0040590763092041016, 0.0039751529693603516, 0.0039958953857421875, 0.003990888595581055, 0.004020214080810547, 0.004075765609741211, 0.0039560794830322266, 0.003988027572631836, 0.0039789676666259766, 0.004002094268798828, 0.004045724868774414, 0.0039501190185546875, 0.0039958953857421875, 0.0039975643157958984, 0.0040128231048583984, 0.004164934158325195, 0.0039958953857421875, 0.003978729248046875, 0.0040285587310791016, 0.003996372222900391, 0.004145145416259766, 0.004058837890625, 0.003995656967163086, 0.004025697708129883, 0.0040934085845947266, 0.004015684127807617, 0.0040569305419921875, 0.0040090084075927734, 0.003979921340942383, 0.004031181335449219, 0.004004716873168945, 0.0039942264556884766, 0.0040132999420166016, 0.003953695297241211, 0.004080295562744141, 0.004030942916870117, 0.003935337066650391, 0.004003286361694336, 0.003988027572631836, 0.004065513610839844, 0.0040035247802734375, 0.003960371017456055, 0.0040166378021240234, 0.003973722457885742, 0.004040956497192383, 0.004027843475341797, 0.00397038459777832, 0.004008769989013672, 0.0039730072021484375, 0.004082679748535156, 0.003995418548583984, 0.003988504409790039, 0.004019975662231445, 0.003975629806518555, 0.004076480865478516, 0.004012584686279297, 0.003991365432739258, 0.0040018558502197266, 0.0039637088775634766, 0.004048347473144531, 0.0040130615234375, 0.00397491455078125, 0.004027843475341797, 0.003989696502685547, 0.004136562347412109, 0.004044771194458008, 0.003996610641479492, 0.0039942264556884766, 0.003990650177001953, 0.004038333892822266, 0.004105806350708008, 0.004006862640380859, 0.003999948501586914, 0.00403594970703125, 0.004034280776977539, 0.004011869430541992, 0.003991127014160156, 0.0039823055267333984, 0.0040552616119384766, 0.0041904449462890625, 0.004091024398803711, 0.003968000411987305, 0.004004001617431641, 0.0040035247802734375, 0.004070758819580078, 0.004009723663330078, 0.003996610641479492, 0.0040128231048583984, 0.0041043758392333984, 0.0042400360107421875, 0.0040836334228515625, 0.003985881805419922, 0.004059553146362305, 0.0040264129638671875, 0.0040438175201416016, 0.004022121429443359, 0.003977060317993164, 0.0040857791900634766, 0.004029750823974609, 0.004021644592285156, 0.004021167755126953, 0.003964900970458984, 0.004068613052368164, 0.004029035568237305, 0.00405573844909668, 0.004075765609741211, 0.0040132999420166016, 0.0040090084075927734, 0.004014015197753906, 0.004015445709228516, 0.004040718078613281, 0.004033327102661133, 0.003981113433837891, 0.004033565521240234, 0.004123210906982422, 0.004082202911376953, 0.004022121429443359, 0.004030942916870117, 0.00400853157043457, 0.004029273986816406, 0.004164218902587891, 0.004016876220703125, 0.00399470329284668, 0.004019260406494141, 0.004012346267700195, 0.004012346267700195, 0.004036426544189453, 0.003955364227294922, 0.004024982452392578, 0.004027605056762695, 0.004079580307006836, 0.00403285026550293, 0.003993988037109375, 0.00402379035949707, 0.004030704498291016, 0.004029989242553711, 0.004022121429443359, 0.003975629806518555, 0.0040454864501953125, 0.004004716873168945, 0.004048347473144531, 0.004023551940917969, 0.003996610641479492, 0.004012584686279297, 0.004042863845825195, 0.004061222076416016, 0.004125118255615234, 0.004007577896118164, 0.004034757614135742, 0.004057884216308594, 0.00402379035949707, 0.004041910171508789, 0.003991127014160156, 0.004036903381347656, 0.004229545593261719, 0.004047393798828125, 0.004061698913574219, 0.004033327102661133, 0.004022121429443359, 0.0040285587310791016, 0.004033803939819336, 0.003998279571533203, 0.003971099853515625, 0.004049539566040039, 0.004144906997680664, 0.004065752029418945, 0.003993511199951172, 0.003983974456787109, 0.004039764404296875, 0.004073143005371094, 0.0041391849517822266, 0.004039764404296875, 0.003987789154052734, 0.004042863845825195, 0.004015684127807617, 0.004044771194458008, 0.004011631011962891, 0.0040130615234375, 0.004045009613037109, 0.004083395004272461, 0.004037618637084961, 0.0040090084075927734, 0.004080295562744141, 0.003961324691772461, 0.00402522087097168, 0.00402522087097168, 0.004011869430541992, 0.004050254821777344, 0.004013538360595703, 0.004032611846923828, 0.004065513610839844, 0.004047870635986328, 0.004043102264404297, 0.003989458084106445, 0.00402379035949707, 0.004082441329956055, 0.004060983657836914, 0.004020214080810547, 0.004012346267700195, 0.004031181335449219, 0.004044055938720703, 0.004020214080810547, 0.004041910171508789, 0.003964424133300781, 0.004062175750732422, 0.00404810905456543, 0.004021644592285156, 0.003989219665527344, 0.004021406173706055, 0.004027843475341797, 0.004027605056762695, 0.004105567932128906, 0.004080057144165039, 0.004076480865478516, 0.004058361053466797, 0.004112958908081055, 0.004068851470947266, 0.0040285587310791016, 0.00401759147644043, 0.004180908203125, 0.004130363464355469, 0.0040242671966552734, 0.0040132999420166016, 0.0040318965911865234, 0.004049777984619141, 0.004112958908081055, 0.004035234451293945, 0.004110813140869141, 0.004022359848022461, 0.004076480865478516, 0.0040416717529296875, 0.004002809524536133, 0.004036426544189453, 0.003978252410888672, 0.0041234493255615234, 0.004060268402099609, 0.003982067108154297, 0.004033327102661133, 0.0039708614349365234, 0.0041370391845703125, 0.004060506820678711, 0.003992795944213867, 0.004052639007568359, 0.004019975662231445, 0.004048824310302734, 0.004058122634887695, 0.00399470329284668, 0.004017829895019531, 0.004037618637084961, 0.004060029983520508, 0.004052400588989258, 0.0039937496185302734, 0.004051685333251953, 0.004056215286254883, 0.004068136215209961, 0.004059314727783203, 0.0040094852447509766, 0.0040874481201171875, 0.004041194915771484, 0.004021644592285156, 0.0040740966796875, 0.004013538360595703, 0.004167318344116211, 0.0040667057037353516, 0.004041194915771484, 0.00404047966003418, 0.004005908966064453, 0.004040241241455078, 0.0040323734283447266, 0.004204750061035156, 0.004048585891723633, 0.004135608673095703, 0.0040166378021240234, 0.004040718078613281, 0.004021644592285156, 0.00407862663269043, 0.004057168960571289, 0.00399017333984375, 0.00400233268737793, 0.004067659378051758, 0.004078388214111328, 0.0040128231048583984, 0.004006624221801758, 0.004032135009765625, 0.004046201705932617, 0.004047870635986328, 0.004058361053466797, 0.0039997100830078125, 0.004021883010864258, 0.004076480865478516, 0.004062175750732422, 0.004043102264404297, 0.004015684127807617, 0.00404810905456543, 0.004038095474243164, 0.00406646728515625, 0.003999948501586914, 0.004041433334350586, 0.004006624221801758, 0.004093647003173828, 0.004003047943115234, 0.0040700435638427734, 0.004019260406494141, 0.004021406173706055, 0.00411677360534668, 0.004050016403198242, 0.00403904914855957, 0.0040247440338134766, 0.004038810729980469, 0.004072666168212891, 0.004027605056762695, 0.00404667854309082, 0.004087686538696289, 0.004068136215209961, 0.00407719612121582, 0.004042387008666992, 0.004050254821777344, 0.00401616096496582, 0.004183292388916016, 0.004058837890625, 0.004001140594482422, 0.0040967464447021484, 0.00399327278137207, 0.004087209701538086, 0.0040242671966552734, 0.003992319107055664, 0.004074811935424805, 0.004267215728759766, 0.004065275192260742, 0.004071474075317383, 0.004039287567138672, 0.004028797149658203, 0.004085063934326172, 0.00403141975402832, 0.0040645599365234375, 0.00403904914855957, 0.004109382629394531, 0.00405120849609375, 0.004088640213012695, 0.004044294357299805, 0.004030704498291016, 0.004042863845825195, 0.004084110260009766, 0.0040204524993896484, 0.004064083099365234, 0.004065513610839844, 0.00406336784362793, 0.00406336784362793, 0.004044532775878906, 0.004075050354003906, 0.004060268402099609, 0.003996610641479492, 0.004082202911376953, 0.004024982452392578, 0.004049539566040039, 0.004103183746337891, 0.004006862640380859, 0.004069089889526367, 0.004067420959472656, 0.004092216491699219, 0.00405573844909668, 0.004127979278564453, 0.004027605056762695, 0.004108905792236328, 0.004087209701538086, 0.0040509700775146484, 0.004032135009765625, 0.004161357879638672, 0.004131793975830078, 0.004029035568237305, 0.004031658172607422, 0.004006624221801758, 0.004048824310302734, 0.004124641418457031, 0.004011869430541992, 0.004077911376953125, 0.003996372222900391, 0.004132509231567383, 0.004063606262207031, 0.0040056705474853516, 0.004082679748535156, 0.0040225982666015625, 0.004088163375854492, 0.004036664962768555, 0.004021406173706055, 0.004037618637084961, 0.0040132999420166016, 0.004111289978027344, 0.0040283203125, 0.0040454864501953125, 0.004052639007568359, 0.004107475280761719, 0.004009246826171875, 0.004004716873168945, 0.004057884216308594, 0.004083156585693359, 0.0041196346282958984, 0.004046916961669922, 0.004066944122314453, 0.004037141799926758, 0.004124164581298828, 0.004053831100463867, 0.0040166378021240234, 0.004038810729980469, 0.004004001617431641, 0.0041217803955078125, 0.0040454864501953125, 0.0040128231048583984, 0.0041086673736572266, 0.004001617431640625, 0.004178762435913086, 0.004079341888427734, 0.004003763198852539, 0.004037141799926758, 0.004076242446899414, 0.004136323928833008, 0.00405573844909668, 0.0041081905364990234, 0.004059791564941406, 0.004059791564941406, 0.0040435791015625, 0.004063844680786133, 0.004016876220703125, 0.004049062728881836, 0.004070758819580078, 0.004061222076416016, 0.0040705204010009766, 0.00400853157043457, 0.004096508026123047, 0.004063844680786133, 0.004083395004272461, 0.004086494445800781, 0.004035234451293945, 0.004024982452392578, 0.00407719612121582, 0.004056692123413086, 0.0040280818939208984, 0.004106760025024414, 0.004029989242553711, 0.0040514469146728516, 0.00406336784362793, 0.004095554351806641, 0.0040700435638427734, 0.004031658172607422, 0.004065513610839844, 0.004117012023925781, 0.004058361053466797, 0.0040531158447265625, 0.004074811935424805, 0.004078388214111328, 0.004075527191162109, 0.004053354263305664, 0.004059791564941406, 0.004038572311401367, 0.004241228103637695, 0.004253387451171875, 0.0041463375091552734, 0.0041577816009521484, 0.004136562347412109, 0.004357099533081055, 0.004171133041381836, 0.004147529602050781, 0.004207611083984375, 0.004143238067626953, 0.00437474250793457, 0.00417017936706543, 0.004239559173583984, 0.0046269893646240234, 0.004264354705810547, 0.0041561126708984375, 0.004240751266479492, 0.0041751861572265625, 0.0042934417724609375, 0.004251718521118164, 0.004201650619506836, 0.004170656204223633, 0.004156589508056641, 0.004179239273071289, 0.0041522979736328125, 0.004225254058837891, 0.0041882991790771484, 0.004132986068725586, 0.004155397415161133, 0.004134178161621094, 0.004192352294921875, 0.0049571990966796875, 0.004379987716674805, 0.004202365875244141, 0.004177093505859375, 0.004499197006225586, 0.004338264465332031, 0.004296541213989258, 0.004293918609619141, 0.004221677780151367, 0.004182100296020508, 0.004191398620605469, 0.004122018814086914, 0.004151821136474609, 0.004158496856689453, 0.004190206527709961, 0.0041348934173583984, 0.004173755645751953, 0.004143714904785156, 0.004213571548461914, 0.0042133331298828125, 0.004128456115722656, 0.004187345504760742, 0.004354000091552734, 0.004097700119018555, 0.0041806697845458984, 0.004195690155029297, 0.004321575164794922, 0.004193782806396484, 0.004224538803100586, 0.00414729118347168, 0.0042264461517333984, 0.0041811466217041016, 0.004131317138671875, 0.004197359085083008, 0.004266500473022461, 0.004156589508056641, 0.0041463375091552734, 0.004198789596557617, 0.004153013229370117, 0.00421905517578125, 0.004149913787841797, 0.004173994064331055, 0.004157304763793945, 0.004192352294921875, 0.004172325134277344, 0.00418543815612793, 0.004204273223876953, 0.00417780876159668, 0.0041751861572265625, 0.0041637420654296875, 0.004198789596557617, 0.004160404205322266, 0.004175662994384766, 0.004244089126586914, 0.004156589508056641, 0.004196643829345703, 0.004109382629394531, 0.004280805587768555, 0.004174947738647461, 0.004134416580200195, 0.0041942596435546875, 0.0042226314544677734, 0.004233598709106445, 0.004151105880737305, 0.004083395004272461, 0.004147052764892578, 0.004193782806396484, 0.004185914993286133, 0.004131793975830078, 0.004266262054443359, 0.004143238067626953, 0.004179239273071289, 0.004217624664306641, 0.004210710525512695, 0.004204273223876953, 0.004193544387817383, 0.0042684078216552734, 0.004271030426025391, 0.004148006439208984, 0.00420069694519043, 0.0042111873626708984, 0.00416111946105957, 0.004177570343017578, 0.004090547561645508, 0.0043718814849853516, 0.00421595573425293, 0.004138469696044922, 0.00422358512878418, 0.004202604293823242, 0.004162311553955078, 0.0041675567626953125, 0.004160165786743164, 0.004342794418334961, 0.0042209625244140625, 0.0041544437408447266, 0.004151582717895508, 0.00420689582824707, 0.0041615962982177734, 0.0041501522064208984, 0.004135608673095703, 0.004252195358276367, 0.004202842712402344, 0.004142284393310547, 0.004220485687255859, 0.004175662994384766, 0.004156827926635742, 0.004206180572509766, 0.004141807556152344, 0.004173994064331055, 0.004168033599853516, 0.004184246063232422, 0.004181623458862305, 0.004188060760498047, 0.0040967464447021484, 0.004186153411865234, 0.004183530807495117, 0.0042264461517333984, 0.004254341125488281, 0.004185199737548828, 0.004266023635864258, 0.004179716110229492, 0.00419926643371582, 0.004207134246826172, 0.004184722900390625, 0.004288673400878906, 0.004147052764892578, 0.004118680953979492, 0.004161834716796875, 0.004156589508056641, 0.004197597503662109, 0.004155397415161133, 0.004194736480712891, 0.004195213317871094, 0.004196882247924805, 0.004172325134277344, 0.0041866302490234375, 0.004130125045776367, 0.004109621047973633, 0.0041544437408447266, 0.004251718521118164, 0.0041773319244384766, 0.004178762435913086, 0.0041506290435791016, 0.004176616668701172, 0.004208087921142578, 0.004132509231567383, 0.004140377044677734, 0.004174947738647461, 0.00415492057800293, 0.004169464111328125, 0.0041925907135009766, 0.004176139831542969, 0.004187822341918945, 0.0042111873626708984, 0.0041768550872802734, 0.004179477691650391, 0.0041391849517822266, 0.00417017936706543, 0.004155635833740234, 0.004201412200927734, 0.0041751861572265625, 0.004160165786743164, 0.004157543182373047, 0.004264354705810547, 0.0041501522064208984, 0.00427699089050293, 0.004160642623901367, 0.004250764846801758, 0.004229307174682617, 0.004163503646850586, 0.004163265228271484, 0.004187583923339844, 0.004257917404174805, 0.00418400764465332, 0.004212617874145508, 0.004143238067626953, 0.004170417785644531, 0.004196882247924805, 0.004174709320068359, 0.004167079925537109, 0.004793643951416016, 0.00426173210144043, 0.004183292388916016, 0.0041506290435791016, 0.0046536922454833984, 0.004254341125488281, 0.004164695739746094, 0.004203081130981445, 0.004154682159423828, 0.00423884391784668, 0.004233360290527344, 0.004178762435913086, 0.0042459964752197266, 0.004154205322265625, 0.004214763641357422, 0.004150867462158203, 0.004263877868652344, 0.004227399826049805, 0.004136800765991211, 0.00419926643371582, 0.004185914993286133, 0.004175662994384766, 0.004186868667602539, 0.00414276123046875, 0.004241943359375, 0.004199981689453125, 0.004210472106933594, 0.004191160202026367, 0.004185199737548828, 0.004117727279663086, 0.004204511642456055, 0.004151344299316406, 0.004239797592163086, 0.004250764846801758, 0.0041539669036865234, 0.0042133331298828125, 0.004336118698120117, 0.004125356674194336, 0.004172563552856445, 0.004282474517822266, 0.004254579544067383, 0.004183053970336914, 0.004189729690551758, 0.004195451736450195, 0.004184722900390625, 0.004459381103515625, 0.0045964717864990234, 0.004239320755004883, 0.004194736480712891, 0.004231452941894531, 0.004235982894897461, 0.0041768550872802734, 0.004190683364868164, 0.004174232482910156, 0.004297971725463867, 0.0042057037353515625, 0.004136562347412109, 0.0042035579681396484, 0.0042572021484375, 0.004283428192138672, 0.0042345523834228516, 0.004226207733154297, 0.004156351089477539, 0.004204750061035156, 0.0041849613189697266, 0.0042324066162109375, 0.004195451736450195, 0.00410914421081543, 0.004178762435913086, 0.0042536258697509766, 0.004205942153930664, 0.004208803176879883, 0.004158496856689453, 0.004277944564819336, 0.004176616668701172, 0.004186153411865234, 0.00420069694519043, 0.00419926643371582, 0.004232168197631836, 0.004191398620605469, 0.004267215728759766, 0.004198789596557617, 0.004254579544067383, 0.004255056381225586, 0.004202842712402344, 0.004178285598754883, 0.004227399826049805, 0.0042972564697265625, 0.004227638244628906, 0.004247903823852539, 0.004202842712402344, 0.004212141036987305, 0.004301786422729492, 0.004174232482910156, 0.0041468143463134766, 0.004144430160522461, 0.004230976104736328, 0.004189014434814453, 0.004200935363769531, 0.00429534912109375, 0.004188060760498047, 0.0042688846588134766, 0.00422358512878418, 0.004228830337524414, 0.00421452522277832, 0.004213809967041016, 0.0042078495025634766, 0.004223346710205078, 0.004172325134277344, 0.00422215461730957, 0.00417780876159668, 0.004311800003051758, 0.0042302608489990234, 0.004112958908081055, 0.004217863082885742, 0.004213094711303711, 0.004288911819458008, 0.004190921783447266, 0.0042874813079833984, 0.004135608673095703, 0.0041811466217041016, 0.004153251647949219, 0.004204988479614258, 0.004207611083984375, 0.0041882991790771484, 0.004187583923339844, 0.004191398620605469, 0.004332304000854492, 0.004203319549560547, 0.004250764846801758, 0.004193305969238281, 0.004188060760498047, 0.004197597503662109, 0.004182100296020508, 0.004267454147338867, 0.004161357879638672, 0.004235029220581055, 0.0042421817779541016, 0.004185914993286133, 0.004219532012939453, 0.004211902618408203, 0.004247903823852539, 0.00445556640625, 0.004232883453369141, 0.004216194152832031, 0.0042612552642822266, 0.004159688949584961, 0.0041849613189697266, 0.004266977310180664, 0.00419306755065918, 0.004221916198730469, 0.004503726959228516, 0.0042765140533447266, 0.00421142578125, 0.0042002201080322266, 0.0042896270751953125, 0.004231691360473633, 0.00419306755065918, 0.004191398620605469, 0.0042378902435302734, 0.004265785217285156, 0.0042192935943603516, 0.004122018814086914, 0.0042879581451416016, 0.004212379455566406, 0.004206657409667969, 0.00419926643371582, 0.004204511642456055, 0.0041391849517822266, 0.0041844844818115234, 0.004224300384521484, 0.0042400360107421875, 0.004235982894897461, 0.004136562347412109, 0.004271984100341797, 0.004374504089355469, 0.004154682159423828, 0.004179239273071289, 0.004267215728759766, 0.004212617874145508, 0.004210948944091797, 0.0042498111724853516, 0.00430750846862793, 0.004203081130981445, 0.004262208938598633, 0.004250049591064453, 0.004218339920043945, 0.004205226898193359, 0.004207611083984375, 0.004285335540771484, 0.004201173782348633, 0.0042057037353515625, 0.0041866302490234375, 0.004296779632568359, 0.004237174987792969, 0.004248142242431641, 0.004220485687255859, 0.0041959285736083984, 0.004210233688354492, 0.00423884391784668, 0.004245281219482422, 0.0041468143463134766, 0.004218101501464844, 0.004237651824951172, 0.0042724609375, 0.004205226898193359, 0.00420069694519043, 0.004265308380126953, 0.004262685775756836, 0.004214763641357422, 0.004189014434814453, 0.0041310787200927734, 0.004271507263183594, 0.0042724609375, 0.004166603088378906, 0.0042111873626708984, 0.004272937774658203, 0.00420689582824707, 0.004214763641357422, 0.004279375076293945, 0.004205226898193359, 0.00423741340637207, 0.004288196563720703, 0.004231929779052734, 0.004247426986694336, 0.004183053970336914, 0.004203081130981445, 0.004258394241333008, 0.004206180572509766, 0.004320621490478516, 0.004304409027099609, 0.004277229309082031, 0.004213809967041016, 0.004178047180175781, 0.004248857498168945, 0.0041980743408203125, 0.0042192935943603516, 0.0042896270751953125, 0.004247426986694336, 0.004198312759399414, 0.004239082336425781, 0.0043354034423828125, 0.004136562347412109, 0.00418400764465332, 0.004183530807495117, 0.004307985305786133, 0.004288434982299805, 0.004163503646850586, 0.004218339920043945, 0.004286527633666992, 0.004231452941894531, 0.004210233688354492, 0.004217386245727539, 0.004220485687255859, 0.004205465316772461, 0.00423741340637207, 0.0041942596435546875, 0.004179716110229492, 0.004137754440307617, 0.004225969314575195, 0.004296064376831055, 0.004178285598754883, 0.004224061965942383, 0.004180192947387695, 0.0042645931243896484, 0.0042459964752197266, 0.004213571548461914, 0.004259586334228516, 0.004327535629272461, 0.004450798034667969, 0.00426483154296875, 0.0042841434478759766, 0.004182577133178711, 0.004244327545166016, 0.004336118698120117, 0.004197120666503906, 0.00435185432434082, 0.004250049591064453, 0.00424957275390625, 0.004313945770263672, 0.0042455196380615234, 0.004247426986694336, 0.004322052001953125, 0.004240512847900391, 0.004239320755004883, 0.004282236099243164, 0.004204750061035156, 0.004192352294921875, 0.004255771636962891, 0.00423741340637207, 0.0042340755462646484, 0.004186868667602539, 0.0042705535888671875, 0.004316091537475586, 0.004178762435913086, 0.004461050033569336, 0.004355907440185547, 0.004218339920043945, 0.004204988479614258, 0.004148006439208984, 0.004258632659912109, 0.004248619079589844, 0.0042400360107421875, 0.004266500473022461, 0.0042591094970703125, 0.004190921783447266, 0.004251956939697266, 0.00429844856262207, 0.004217624664306641, 0.004278898239135742, 0.004215717315673828, 0.004278659820556641, 0.004226207733154297, 0.004187822341918945, 0.004305839538574219, 0.0042650699615478516, 0.004332065582275391, 0.004229545593261719, 0.00431060791015625, 0.004247426986694336, 0.0043070316314697266, 0.0043354034423828125, 0.004208087921142578, 0.004356861114501953, 0.0042476654052734375, 0.004348039627075195, 0.004260063171386719, 0.0041790008544921875, 0.004292964935302734, 0.004238128662109375, 0.004265546798706055, 0.004277706146240234, 0.004274129867553711, 0.004206418991088867, 0.004190921783447266, 0.004221200942993164, 0.004267692565917969, 0.004282236099243164, 0.00420689582824707, 0.0042765140533447266, 0.004290580749511719, 0.004248857498168945, 0.004171609878540039, 0.0042798519134521484, 0.004250288009643555, 0.004271745681762695, 0.004162311553955078, 0.004287004470825195, 0.004244327545166016, 0.004227399826049805, 0.004378795623779297, 0.004247426986694336, 0.004236936569213867, 0.004212379455566406, 0.004330635070800781, 0.004244804382324219, 0.004267454147338867, 0.004288911819458008, 0.004206418991088867, 0.004252195358276367, 0.0041921138763427734, 0.00422358512878418, 0.004260540008544922, 0.004404783248901367, 0.004281520843505859, 0.004286289215087891, 0.0041882991790771484, 0.004226207733154297, 0.0042531490325927734, 0.004203081130981445, 0.00439453125, 0.004269838333129883, 0.004296302795410156, 0.00426483154296875, 0.004211902618408203, 0.004224300384521484, 0.004251956939697266, 0.004209041595458984, 0.004481077194213867, 0.004252433776855469, 0.004192829132080078, 0.0042209625244140625, 0.004306316375732422, 0.004189252853393555, 0.004233598709106445, 0.004185676574707031, 0.004469394683837891, 0.004351139068603516, 0.00434422492980957, 0.004276275634765625, 0.0042760372161865234, 0.00424957275390625, 0.004241943359375, 0.0043125152587890625, 0.0042421817779541016, 0.004275083541870117, 0.0041964054107666016, 0.004270076751708984, 0.0042514801025390625, 0.004189968109130859, 0.004278898239135742, 0.00422978401184082, 0.004206657409667969, 0.004247426986694336, 0.004281044006347656, 0.0042572021484375, 0.004233598709106445, 0.0041811466217041016, 0.004277706146240234, 0.004278659820556641, 0.004393339157104492, 0.004261016845703125, 0.0042684078216552734, 0.004224538803100586, 0.004512310028076172, 0.00424957275390625, 0.00421142578125, 0.004292726516723633, 0.004216670989990234, 0.004276752471923828, 0.004262208938598633, 0.004216670989990234, 0.004300832748413086, 0.004252195358276367, 0.004244327545166016, 0.004227876663208008, 0.004222869873046875, 0.004238128662109375, 0.0042591094970703125, 0.004324913024902344, 0.004239797592163086, 0.004220008850097656, 0.004211902618408203, 0.004284858703613281, 0.004274129867553711, 0.004205942153930664, 0.004242658615112305, 0.004311084747314453, 0.004282236099243164, 0.004230022430419922, 0.0042612552642822266, 0.00424647331237793, 0.0044515132904052734, 0.0043909549713134766, 0.004366397857666016, 0.0042760372161865234, 0.004294395446777344, 0.004225969314575195, 0.004198789596557617, 0.0042569637298583984, 0.0043179988861083984, 0.0041964054107666016, 0.004283905029296875, 0.00419926643371582, 0.004332542419433594, 0.004290580749511719, 0.004200458526611328, 0.004380464553833008, 0.004263639450073242, 0.0042877197265625, 0.004246711730957031, 0.004255056381225586, 0.004231929779052734, 0.004265308380126953, 0.0044291019439697266, 0.004240274429321289, 0.004247903823852539, 0.004225492477416992, 0.0043294429779052734, 0.004252910614013672, 0.004202842712402344, 0.004277706146240234, 0.004575967788696289, 0.004261970520019531, 0.004549264907836914, 0.0043680667877197266, 0.004179954528808594, 0.004282951354980469, 0.004349708557128906, 0.004222869873046875, 0.0042765140533447266, 0.004221677780151367, 0.004317522048950195, 0.004247903823852539, 0.0042192935943603516, 0.004265308380126953, 0.004244565963745117, 0.004265785217285156, 0.00426483154296875, 0.0043621063232421875, 0.0042493343353271484, 0.004210472106933594, 0.0043182373046875, 0.0041959285736083984, 0.004258155822753906, 0.004198551177978516, 0.0043337345123291016, 0.004313468933105469, 0.004228115081787109, 0.004245758056640625, 0.004279375076293945, 0.004281759262084961, 0.0042459964752197266, 0.0043599605560302734, 0.004269599914550781, 0.004271745681762695, 0.004284381866455078, 0.004181623458862305, 0.004285097122192383, 0.004239320755004883, 0.004331827163696289, 0.00443267822265625, 0.004197359085083008, 0.004339694976806641, 0.004243373870849609, 0.004266977310180664, 0.004290103912353516, 0.004276275634765625, 0.004329204559326172, 0.004200935363769531, 0.004293203353881836, 0.0042858123779296875, 0.004213571548461914, 0.0042531490325927734, 0.004265785217285156, 0.004252910614013672, 0.004219770431518555, 0.004198789596557617, 0.004264116287231445, 0.004355430603027344, 0.004270792007446289, 0.004258632659912109, 0.004231691360473633, 0.00424647331237793, 0.004236936569213867, 0.004341840744018555, 0.0042209625244140625, 0.0042514801025390625, 0.0041468143463134766, 0.004346132278442383, 0.004354000091552734, 0.0042417049407958984, 0.004252910614013672, 0.004244804382324219, 0.004235982894897461, 0.0042493343353271484, 0.00428318977355957, 0.004189014434814453, 0.004263401031494141, 0.004355430603027344, 0.004304647445678711, 0.004296064376831055, 0.0042285919189453125, 0.004355192184448242, 0.004289865493774414, 0.004201173782348633, 0.004308462142944336, 0.0042858123779296875, 0.004399776458740234, 0.004293203353881836, 0.004413604736328125, 0.0041866302490234375, 0.004258394241333008, 0.00429224967956543, 0.004275321960449219, 0.0042667388916015625, 0.004186153411865234, 0.004322052001953125, 0.004330873489379883, 0.004278421401977539, 0.004309892654418945, 0.004290580749511719, 0.004270076751708984, 0.004255056381225586, 0.004370927810668945, 0.004244804382324219, 0.00431060791015625, 0.004339933395385742, 0.004233598709106445, 0.004277944564819336, 0.004252195358276367, 0.00431370735168457, 0.004273176193237305, 0.004217386245727539, 0.004258155822753906, 0.00434422492980957, 0.004278421401977539, 0.004299640655517578, 0.004263401031494141, 0.004216432571411133, 0.004305601119995117, 0.004354238510131836, 0.00423884391784668, 0.004244804382324219, 0.004229068756103516, 0.0043561458587646484, 0.004243135452270508, 0.004369020462036133, 0.004300594329833984, 0.004248619079589844, 0.004304170608520508, 0.0042476654052734375, 0.00431370735168457, 0.004221439361572266, 0.0042726993560791016, 0.004441261291503906, 0.004255533218383789, 0.004224300384521484, 0.004241228103637695, 0.0042803287506103516, 0.0042803287506103516, 0.004209756851196289, 0.004492998123168945, 0.0042455196380615234, 0.004259824752807617, 0.004299163818359375, 0.004265785217285156, 0.004245758056640625, 0.004274129867553711, 0.0043332576751708984, 0.004311323165893555, 0.0044994354248046875, 0.004330873489379883, 0.0043261051177978516, 0.004321575164794922, 0.00423741340637207, 0.004331350326538086, 0.00433802604675293, 0.004274845123291016, 0.004287242889404297, 0.0043294429779052734, 0.0042438507080078125, 0.00429081916809082, 0.004383087158203125, 0.004321098327636719, 0.004286289215087891, 0.0042476654052734375, 0.004249095916748047, 0.004302263259887695, 0.004256010055541992, 0.004306316375732422, 0.004288196563720703, 0.004359006881713867, 0.004352569580078125, 0.0043239593505859375, 0.004259824752807617, 0.004311800003051758, 0.0042989253997802734, 0.0042266845703125, 0.004265546798706055, 0.0042726993560791016, 0.004372835159301758, 0.004283428192138672, 0.004269123077392578, 0.00430750846862793, 0.004251718521118164, 0.004263162612915039, 0.00429844856262207, 0.004301548004150391, 0.0042340755462646484, 0.00432133674621582, 0.004275798797607422, 0.004205465316772461, 0.004299163818359375, 0.004329204559326172, 0.00429081916809082, 0.004283428192138672, 0.0042264461517333984, 0.004299640655517578, 0.004320621490478516, 0.00434112548828125, 0.004294157028198242, 0.004281282424926758, 0.00427556037902832, 0.004264354705810547, 0.004314422607421875, 0.004257678985595703, 0.004367828369140625, 0.004265308380126953, 0.004292726516723633, 0.004323244094848633, 0.004240989685058594, 0.004278898239135742, 0.004334449768066406, 0.0043201446533203125, 0.004302978515625, 0.00428318977355957, 0.004247188568115234, 0.0042955875396728516, 0.0043659210205078125, 0.004341840744018555, 0.004340648651123047, 0.004314899444580078, 0.004281759262084961, 0.00432896614074707, 0.004383563995361328, 0.004289388656616211, 0.004280805587768555, 0.004408359527587891, 0.004303455352783203, 0.0043294429779052734, 0.004238605499267578, 0.004363059997558594, 0.0043141841888427734, 0.0042722225189208984, 0.0043828487396240234, 0.004307746887207031, 0.004277229309082031, 0.0043108463287353516, 0.0043811798095703125, 0.004284858703613281, 0.004307746887207031, 0.004349470138549805, 0.0042760372161865234, 0.004288196563720703, 0.004288911819458008, 0.004410266876220703, 0.004311561584472656, 0.004263162612915039, 0.004311084747314453, 0.004355192184448242, 0.004334449768066406, 0.004298686981201172, 0.004287242889404297, 0.004274845123291016, 0.004282951354980469, 0.00434112548828125, 0.004277229309082031, 0.004416465759277344, 0.004335880279541016, 0.004300117492675781, 0.004289150238037109, 0.004273653030395508, 0.0043332576751708984, 0.00435948371887207, 0.0043871402740478516, 0.004430294036865234, 0.004310131072998047, 0.0042247772216796875, 0.004370927810668945, 0.0043642520904541016, 0.004314899444580078, 0.004359722137451172, 0.0043261051177978516, 0.0044095516204833984, 0.004320383071899414, 0.004300832748413086, 0.004312753677368164, 0.004312276840209961, 0.004561185836791992, 0.004302263259887695, 0.004324436187744141, 0.004360198974609375, 0.004240989685058594, 0.004287004470825195, 0.0042896270751953125, 0.00439453125, 0.004315614700317383, 0.004228830337524414, 0.004312038421630859, 0.004323005676269531, 0.0043299198150634766, 0.004285097122192383, 0.00426173210144043, 0.004264354705810547, 0.004301548004150391, 0.004327058792114258, 0.004292964935302734, 0.004342794418334961, 0.004227161407470703, 0.0043985843658447266, 0.00428462028503418, 0.0042345523834228516, 0.004343271255493164, 0.004366874694824219, 0.0043392181396484375, 0.004349231719970703, 0.004269123077392578, 0.0042574405670166016, 0.0043182373046875, 0.004345417022705078, 0.004266262054443359, 0.004293918609619141, 0.004416942596435547, 0.004277467727661133, 0.004274606704711914, 0.004387855529785156, 0.004265546798706055, 0.00430750846862793, 0.004332304000854492, 0.004323482513427734, 0.004480600357055664, 0.004333972930908203, 0.004395484924316406, 0.004317760467529297, 0.00432896614074707, 0.0043487548828125, 0.004313468933105469, 0.004292011260986328, 0.004324197769165039, 0.004350185394287109, 0.004305362701416016, 0.00433802604675293, 0.004369258880615234, 0.0042493343353271484, 0.004333972930908203, 0.0042951107025146484, 0.00436091423034668, 0.0042572021484375, 0.004286289215087891, 0.004371166229248047, 0.004286289215087891, 0.0043261051177978516, 0.004347801208496094, 0.004364728927612305, 0.00429534912109375, 0.004369020462036133, 0.004398822784423828, 0.004252910614013672, 0.004324674606323242, 0.004378795623779297, 0.004318952560424805, 0.004410982131958008, 0.004411220550537109, 0.0043544769287109375, 0.0043430328369140625, 0.004448890686035156, 0.004292726516723633, 0.004315614700317383, 0.004376649856567383, 0.004399776458740234, 0.004343986511230469, 0.004284381866455078, 0.004321098327636719, 0.004329681396484375, 0.004312992095947266, 0.004310131072998047, 0.004416465759277344, 0.0044095516204833984, 0.0043523311614990234, 0.004384279251098633, 0.0042989253997802734, 0.0043182373046875, 0.004446744918823242, 0.0043849945068359375, 0.004332780838012695, 0.0043108463287353516, 0.0042417049407958984, 0.00431513786315918, 0.00432133674621582, 0.004302024841308594, 0.004542827606201172, 0.004327535629272461, 0.004403591156005859, 0.004326820373535156, 0.004288673400878906, 0.004357099533081055, 0.0043032169342041016, 0.004305601119995117, 0.004396200180053711, 0.004305601119995117, 0.004287242889404297, 0.004290103912353516, 0.004394054412841797, 0.004309177398681641, 0.0043315887451171875, 0.0043566226959228516, 0.0042912960052490234, 0.0044057369232177734, 0.004355907440185547, 0.0043103694915771484, 0.004376649856567383, 0.004334688186645508, 0.0043315887451171875, 0.004346370697021484, 0.0043926239013671875, 0.0044078826904296875, 0.0043718814849853516, 0.004296302795410156, 0.004637241363525391, 0.00436854362487793, 0.004302501678466797, 0.0043468475341796875, 0.0044269561767578125, 0.004347324371337891, 0.0043985843658447266, 0.004372835159301758, 0.004300832748413086, 0.004312038421630859, 0.004496097564697266, 0.004326820373535156, 0.00437617301940918, 0.004370927810668945, 0.004278421401977539, 0.004344463348388672, 0.004347801208496094, 0.004306316375732422, 0.004305124282836914, 0.004320383071899414, 0.004332304000854492, 0.004277706146240234, 0.004283905029296875, 0.004364013671875, 0.004301786422729492, 0.0042803287506103516, 0.004335880279541016, 0.004506587982177734, 0.0043637752532958984, 0.0043773651123046875, 0.004456758499145508, 0.00438690185546875, 0.0043752193450927734, 0.004390716552734375, 0.004663705825805664, 0.004412174224853516, 0.004326581954956055, 0.004277467727661133, 0.0043108463287353516, 0.004403591156005859, 0.004292964935302734, 0.0043447017669677734, 0.004397153854370117, 0.004373073577880859, 0.0043392181396484375, 0.004354238510131836, 0.004297494888305664, 0.004291534423828125, 0.004299640655517578, 0.004333019256591797, 0.004456281661987305, 0.004353523254394531, 0.004476070404052734, 0.004304170608520508, 0.004319667816162109, 0.004360198974609375, 0.004374504089355469, 0.0043163299560546875, 0.004361629486083984, 0.0043087005615234375, 0.004269599914550781, 0.004349708557128906, 0.004398345947265625, 0.004297733306884766, 0.00432896614074707, 0.0043408870697021484, 0.004311323165893555, 0.004325151443481445, 0.004297971725463867, 0.004323244094848633, 0.004326820373535156, 0.004342317581176758, 0.004347801208496094, 0.004329681396484375, 0.004289388656616211, 0.004395961761474609, 0.004305362701416016, 0.004284381866455078, 0.004355192184448242, 0.004435062408447266, 0.004355192184448242, 0.004359722137451172, 0.004350185394287109, 0.004346370697021484, 0.004358768463134766, 0.004406929016113281, 0.004325151443481445, 0.00442051887512207, 0.004484653472900391, 0.004339456558227539, 0.004412651062011719, 0.004406929016113281, 0.0044062137603759766, 0.004355192184448242, 0.004359722137451172, 0.004723310470581055, 0.004576921463012695, 0.004439830780029297, 0.004305601119995117, 0.004331350326538086, 0.004377603530883789, 0.004332780838012695, 0.004366874694824219, 0.0043299198150634766, 0.004385232925415039, 0.0043261051177978516, 0.004283905029296875, 0.004366636276245117, 0.004353046417236328, 0.004308462142944336, 0.004355669021606445, 0.004317045211791992, 0.004353761672973633, 0.0053136348724365234, 0.004441022872924805, 0.0044574737548828125, 0.004659891128540039, 0.004646778106689453, 0.0044231414794921875, 0.004305839538574219, 0.004389047622680664, 0.00439906120300293, 0.004403352737426758, 0.004356861114501953, 0.004287242889404297, 0.004393339157104492, 0.004346132278442383, 0.0043065547943115234, 0.004353046417236328, 0.004373073577880859, 0.004456520080566406, 0.004369258880615234, 0.004334449768066406, 0.004385709762573242, 0.004319906234741211, 0.004406452178955078, 0.00428318977355957, 0.004309892654418945, 0.004324197769165039, 0.004323244094848633, 0.0043487548828125, 0.004418373107910156, 0.004404306411743164, 0.004355669021606445, 0.004393100738525391, 0.004324913024902344, 0.004302263259887695, 0.004349946975708008, 0.004439830780029297, 0.004350423812866211, 0.00431513786315918, 0.004359722137451172, 0.004373311996459961, 0.004334211349487305, 0.004418849945068359, 0.004374265670776367, 0.004319906234741211, 0.004395008087158203, 0.004372596740722656, 0.004314899444580078, 0.0043544769287109375, 0.0044133663177490234, 0.004338979721069336, 0.004330873489379883, 0.004366397857666016, 0.004345417022705078, 0.0043888092041015625, 0.004502058029174805, 0.004426002502441406, 0.004402875900268555, 0.004400014877319336, 0.004377841949462891, 0.0043299198150634766, 0.004374980926513672, 0.004330635070800781, 0.004398345947265625, 0.004464864730834961, 0.004294157028198242, 0.004363059997558594, 0.004472017288208008, 0.004475116729736328, 0.004346609115600586, 0.0042874813079833984, 0.00435185432434082, 0.0044040679931640625, 0.004356861114501953, 0.0043926239013671875, 0.0044708251953125, 0.004375457763671875, 0.004492521286010742, 0.004343271255493164, 0.004350185394287109, 0.004363059997558594, 0.004487276077270508, 0.004472017288208008, 0.004440784454345703, 0.004370212554931641, 0.004391670227050781, 0.004415750503540039, 0.004431009292602539, 0.004383563995361328, 0.004330873489379883, 0.00439763069152832, 0.0043184757232666016, 0.004349470138549805, 0.004395008087158203, 0.0043408870697021484, 0.00432586669921875, 0.004366159439086914, 0.004358768463134766, 0.0043337345123291016, 0.0044307708740234375, 0.004403114318847656, 0.004312276840209961, 0.004372119903564453, 0.004410505294799805, 0.004292964935302734, 0.0043447017669677734, 0.004401206970214844, 0.004377603530883789, 0.00434112548828125, 0.0043735504150390625, 0.0043642520904541016, 0.0043430328369140625, 0.0044596195220947266, 0.00440216064453125, 0.0043528079986572266, 0.004338979721069336, 0.004317045211791992, 0.004362821578979492, 0.004370689392089844, 0.0043964385986328125, 0.004341602325439453, 0.004490852355957031, 0.00440526008605957, 0.0043756961822509766, 0.004352569580078125, 0.004408359527587891, 0.0043811798095703125, 0.0043370723724365234, 0.0043337345123291016, 0.004436016082763672, 0.004361391067504883, 0.00438237190246582, 0.004366874694824219, 0.00430607795715332, 0.004350185394287109, 0.0043621063232421875, 0.004328727722167969, 0.004431724548339844, 0.004355430603027344, 0.004358053207397461, 0.004390239715576172, 0.0043485164642333984, 0.004364490509033203, 0.004348278045654297, 0.004494190216064453, 0.004449605941772461, 0.004397869110107422, 0.004369020462036133, 0.0043756961822509766, 0.004353523254394531, 0.00434112548828125, 0.004325151443481445, 0.0043299198150634766, 0.004392147064208984, 0.004415273666381836, 0.0043370723724365234, 0.004356861114501953, 0.0043947696685791016, 0.004441261291503906, 0.004362821578979492, 0.004360675811767578, 0.004303693771362305, 0.0043392181396484375, 0.004438638687133789, 0.004306316375732422, 0.004338502883911133, 0.004343986511230469, 0.004528045654296875, 0.004486799240112305, 0.004365682601928711, 0.004330873489379883, 0.004361152648925781, 0.004406929016113281, 0.0051839351654052734, 0.004457950592041016, 0.004450798034667969, 0.004913330078125, 0.004422903060913086, 0.004370927810668945, 0.0043277740478515625, 0.004342794418334961, 0.0044901371002197266, 0.00442051887512207, 0.004379749298095703, 0.00445556640625, 0.004334688186645508, 0.0043811798095703125, 0.004354715347290039, 0.004380464553833008, 0.0043375492095947266, 0.004338502883911133, 0.004360675811767578, 0.004382610321044922, 0.004334211349487305, 0.00440216064453125, 0.004342079162597656, 0.004299640655517578, 0.004420280456542969, 0.0043714046478271484, 0.004446506500244141, 0.00442957878112793, 0.004444599151611328, 0.004334449768066406, 0.004457235336303711, 0.00451970100402832, 0.004305839538574219, 0.0043256282806396484, 0.004403829574584961, 0.0043926239013671875, 0.004332780838012695, 0.0044193267822265625, 0.004361391067504883, 0.004517555236816406, 0.004466056823730469, 0.004449129104614258, 0.00433349609375, 0.005026340484619141, 0.004439592361450195, 0.004496335983276367, 0.0043833255767822266, 0.0043659210205078125, 0.00441431999206543, 0.004337310791015625, 0.004341602325439453, 0.004536628723144531, 0.004420280456542969, 0.004351615905761719, 0.004338741302490234, 0.004426717758178711, 0.004388332366943359, 0.004418134689331055, 0.00445866584777832, 0.004334211349487305, 0.004395723342895508, 0.004388093948364258, 0.004385709762573242, 0.004431247711181641, 0.004413604736328125, 0.00438690185546875, 0.0043871402740478516, 0.0044291019439697266, 0.004374504089355469, 0.004395246505737305, 0.004420757293701172, 0.0043871402740478516, 0.004401683807373047, 0.004339456558227539, 0.004326820373535156, 0.004362344741821289, 0.0044634342193603516, 0.004469871520996094, 0.004441976547241211, 0.004446268081665039, 0.00445246696472168, 0.004354238510131836, 0.004312753677368164, 0.004380702972412109, 0.004453182220458984, 0.004474163055419922, 0.004403829574584961, 0.004387855529785156, 0.004324674606323242, 0.004425764083862305, 0.00443577766418457, 0.0043489933013916016, 0.00443267822265625, 0.004372596740722656, 0.0044498443603515625, 0.004423856735229492, 0.004425048828125, 0.004325389862060547, 0.004414081573486328, 0.004476308822631836, 0.004420042037963867, 0.0044078826904296875, 0.0043582916259765625, 0.004370689392089844, 0.00435328483581543, 0.004369974136352539, 0.004345893859863281, 0.0043942928314208984, 0.004408836364746094, 0.004345893859863281, 0.004419803619384766, 0.004418373107910156, 0.0044062137603759766, 0.004389047622680664, 0.004367351531982422, 0.004332304000854492, 0.0043811798095703125, 0.004403352737426758, 0.004390239715576172, 0.004427194595336914, 0.00445103645324707, 0.004536867141723633, 0.004487514495849609, 0.004392385482788086, 0.004416704177856445, 0.004349708557128906, 0.004458427429199219, 0.004308462142944336, 0.004370689392089844, 0.004363536834716797, 0.0045659542083740234, 0.0043675899505615234, 0.004370212554931641, 0.004504680633544922, 0.004391908645629883, 0.004405498504638672, 0.0045948028564453125, 0.004477739334106445, 0.004400014877319336, 0.004441499710083008, 0.004464864730834961, 0.004351377487182617, 0.00438690185546875, 0.004619121551513672, 0.004440784454345703, 0.004457235336303711, 0.004423618316650391, 0.004432201385498047, 0.004446983337402344, 0.004327535629272461, 0.004344463348388672, 0.0044443607330322266, 0.004467487335205078, 0.00439453125, 0.004378557205200195, 0.004359722137451172, 0.004349231719970703, 0.004409074783325195, 0.0043544769287109375, 0.004436969757080078, 0.004563331604003906, 0.004513740539550781, 0.004405975341796875, 0.004446744918823242, 0.004414081573486328, 0.004362821578979492, 0.0045506954193115234, 0.004431962966918945, 0.004317283630371094, 0.004395484924316406, 0.004348278045654297, 0.004377841949462891, 0.004404783248901367, 0.0043468475341796875, 0.004343986511230469, 0.004469871520996094, 0.0045125484466552734, 0.004409313201904297, 0.004446506500244141, 0.00439906120300293, 0.004342794418334961, 0.0044097900390625, 0.004412412643432617, 0.00438690185546875, 0.004407167434692383, 0.0044367313385009766, 0.004320621490478516, 0.004362583160400391, 0.004402875900268555, 0.004372835159301758, 0.004384040832519531, 0.004373073577880859, 0.00436854362487793, 0.004358768463134766, 0.004511117935180664, 0.004346370697021484, 0.004374980926513672, 0.004495143890380859, 0.004498958587646484, 0.004437685012817383, 0.00441741943359375, 0.004322052001953125, 0.00443267822265625, 0.004496097564697266, 0.004663944244384766, 0.004490852355957031, 0.004441976547241211, 0.004504203796386719, 0.004472494125366211, 0.004462718963623047, 0.004336357116699219, 0.004416465759277344, 0.0045964717864990234, 0.004465579986572266, 0.0044193267822265625, 0.004399776458740234, 0.00439763069152832, 0.0044307708740234375, 0.0044596195220947266, 0.004380464553833008, 0.004456043243408203, 0.0046422481536865234, 0.0043261051177978516, 0.004448652267456055, 0.004370450973510742, 0.0044231414794921875, 0.0044176578521728516, 0.004372358322143555, 0.004335165023803711, 0.00439000129699707, 0.004425525665283203, 0.004560708999633789, 0.004452943801879883, 0.004473447799682617, 0.004370689392089844, 0.004441261291503906, 0.004456281661987305, 0.004374265670776367, 0.00439000129699707, 0.0045166015625, 0.004364013671875, 0.00446772575378418, 0.0043375492095947266, 0.004496574401855469, 0.004410982131958008, 0.0043315887451171875, 0.004441499710083008, 0.004407167434692383, 0.004380464553833008, 0.004385471343994141, 0.004387378692626953, 0.0043642520904541016, 0.00445103645324707, 0.0044422149658203125, 0.0043942928314208984, 0.00439906120300293, 0.004354715347290039, 0.00449824333190918, 0.004389762878417969, 0.004414081573486328, 0.004461765289306641, 0.0045397281646728516, 0.004509925842285156, 0.00442051887512207, 0.004553794860839844, 0.004499673843383789, 0.00457310676574707, 0.004553079605102539, 0.0045206546783447266, 0.004496574401855469, 0.004705667495727539, 0.004526615142822266, 0.004546403884887695, 0.004550933837890625, 0.004450082778930664, 0.004563570022583008, 0.004488945007324219, 0.0045299530029296875, 0.004626750946044922, 0.004495859146118164, 0.00446009635925293, 0.004598140716552734, 0.004893302917480469, 0.00465083122253418, 0.004481077194213867, 0.0045337677001953125, 0.004579782485961914, 0.004511833190917969, 0.004567623138427734, 0.004453182220458984, 0.004494905471801758, 0.004474639892578125, 0.004511117935180664, 0.004537105560302734, 0.004468202590942383, 0.0044612884521484375, 0.0045435428619384766, 0.004590272903442383, 0.00439000129699707, 0.0044765472412109375, 0.004497051239013672, 0.004571437835693359, 0.0044994354248046875, 0.004571199417114258, 0.004414081573486328, 0.0046234130859375, 0.004569053649902344, 0.004560232162475586, 0.004533290863037109, 0.004497051239013672, 0.004572868347167969, 0.00461888313293457, 0.0044515132904052734, 0.004400014877319336, 0.004544496536254883, 0.004492282867431641, 0.004397153854370117, 0.004471302032470703, 0.004506111145019531, 0.004525423049926758, 0.00449061393737793, 0.00451207160949707, 0.00446009635925293, 0.00449371337890625, 0.0044972896575927734, 0.004565000534057617, 0.004439115524291992, 0.004465341567993164, 0.004599809646606445, 0.004480123519897461, 0.0044820308685302734, 0.0045053958892822266, 0.004540443420410156, 0.004603862762451172, 0.004652500152587891, 0.004475831985473633, 0.004574298858642578, 0.004559993743896484, 0.0044782161712646484, 0.004477024078369141, 0.00453495979309082, 0.004525423049926758, 0.004545688629150391, 0.004423856735229492, 0.004456043243408203, 0.004656791687011719, 0.004492759704589844, 0.004426002502441406, 0.004526615142822266, 0.004578113555908203, 0.004544258117675781, 0.0045413970947265625, 0.004498481750488281, 0.004518032073974609, 0.004695892333984375, 0.004570722579956055, 0.004479169845581055, 0.0045013427734375, 0.004523515701293945, 0.004785060882568359, 0.004590511322021484, 0.004600048065185547, 0.004506826400756836, 0.004508256912231445, 0.004483461380004883, 0.0045490264892578125, 0.00453495979309082, 0.004656314849853516, 0.004872798919677734, 0.004537820816040039, 0.004496097564697266, 0.004513263702392578, 0.0045855045318603516, 0.004464149475097656, 0.004542350769042969, 0.004560708999633789, 0.004598140716552734, 0.004526853561401367, 0.004644632339477539, 0.004491090774536133, 0.004610300064086914, 0.00455164909362793, 0.004489898681640625, 0.00448298454284668, 0.00454258918762207, 0.004556179046630859, 0.004506349563598633, 0.0046253204345703125, 0.004518985748291016, 0.004697084426879883, 0.0045795440673828125, 0.004502058029174805, 0.004469871520996094, 0.004503965377807617, 0.004530191421508789, 0.004456996917724609, 0.004539966583251953, 0.004503488540649414, 0.0045855045318603516, 0.004508018493652344, 0.004530429840087891, 0.004503726959228516, 0.004560947418212891, 0.004556417465209961, 0.004447221755981445, 0.00456690788269043, 0.004529237747192383, 0.0045413970947265625, 0.00453639030456543, 0.004473209381103516, 0.0045664310455322266, 0.004584074020385742, 0.004497051239013672, 0.004534006118774414, 0.004583835601806641, 0.004536151885986328, 0.0045948028564453125, 0.004511833190917969, 0.004533290863037109, 0.0044744014739990234, 0.004549503326416016, 0.004546165466308594, 0.004571437835693359, 0.004428863525390625, 0.004500150680541992, 0.004569292068481445, 0.00449824333190918, 0.004578828811645508, 0.0045201778411865234, 0.004513978958129883, 0.004539966583251953, 0.004569053649902344, 0.004496335983276367, 0.004669904708862305, 0.0045201778411865234, 0.004446268081665039, 0.0046100616455078125, 0.004539966583251953, 0.004601240158081055, 0.004628419876098633, 0.004536867141723633, 0.0045623779296875, 0.004595756530761719, 0.004514217376708984, 0.004626035690307617, 0.0045354366302490234, 0.004554271697998047, 0.00458073616027832, 0.00455474853515625, 0.004510164260864258, 0.004590034484863281, 0.004536151885986328, 0.004539966583251953, 0.004530429840087891, 0.004425764083862305, 0.0046062469482421875, 0.004540920257568359, 0.004513978958129883, 0.004605531692504883, 0.004549741744995117, 0.004618167877197266, 0.004526376724243164, 0.0045320987701416016, 0.004599809646606445, 0.004557609558105469, 0.004571437835693359, 0.0046007633209228516, 0.004499673843383789, 0.0046117305755615234, 0.004568576812744141, 0.004517793655395508, 0.004556417465209961, 0.004510164260864258, 0.004565000534057617, 0.0045642852783203125, 0.004476308822631836, 0.004491090774536133, 0.004887580871582031, 0.00458526611328125, 0.004515647888183594, 0.0044934749603271484, 0.004506826400756836, 0.004626035690307617, 0.004490852355957031, 0.0045888423919677734, 0.004624843597412109, 0.004647254943847656, 0.00457310676574707, 0.004513263702392578, 0.00453495979309082, 0.004630565643310547, 0.004515409469604492, 0.00447392463684082, 0.0046122074127197266, 0.0045282840728759766, 0.0045490264892578125, 0.004533052444458008, 0.0045549869537353516, 0.004530191421508789, 0.004494905471801758, 0.004760265350341797, 0.004637956619262695, 0.004725217819213867, 0.004591941833496094, 0.004536628723144531, 0.004462480545043945, 0.004559040069580078, 0.004509687423706055, 0.004684925079345703, 0.0046198368072509766, 0.00454258918762207, 0.004541158676147461, 0.004576683044433594, 0.004525423049926758, 0.004579067230224609, 0.004593849182128906, 0.004515886306762695, 0.00459742546081543, 0.004476308822631836, 0.004572629928588867, 0.004591226577758789, 0.004816532135009766, 0.004570484161376953, 0.004564523696899414, 0.0045773983001708984, 0.004595041275024414, 0.004611015319824219, 0.0044708251953125, 0.004534244537353516, 0.004528045654296875, 0.004706859588623047, 0.004584550857543945, 0.004627704620361328, 0.004584312438964844, 0.00454258918762207, 0.004518032073974609, 0.0046083927154541016, 0.004530906677246094, 0.004613161087036133, 0.004537820816040039, 0.004561185836791992, 0.004630565643310547, 0.004502534866333008, 0.00453948974609375, 0.004659891128540039, 0.004523277282714844, 0.004625797271728516, 0.0045893192291259766, 0.004479169845581055, 0.004608631134033203, 0.004540205001831055, 0.004557609558105469, 0.004553079605102539, 0.004544258117675781, 0.004559993743896484, 0.004570960998535156, 0.004552364349365234, 0.004530906677246094, 0.004552364349365234, 0.006183624267578125, 0.0046520233154296875, 0.004595041275024414, 0.004933357238769531, 0.0046498775482177734, 0.0046977996826171875, 0.004656076431274414, 0.004877805709838867, 0.004529714584350586, 0.004532337188720703, 0.004735708236694336, 0.0045435428619384766, 0.004590034484863281, 0.004580497741699219, 0.004597187042236328, 0.004618644714355469, 0.0045359134674072266, 0.004555940628051758, 0.004599571228027344, 0.004698276519775391, 0.004621028900146484, 0.0046541690826416016, 0.004518032073974609, 0.004587650299072266, 0.004469871520996094, 0.004624366760253906, 0.00475001335144043, 0.004599571228027344, 0.00466609001159668, 0.004530429840087891, 0.004513740539550781, 0.004593610763549805, 0.0045549869537353516, 0.004520893096923828, 0.004554271697998047, 0.004579305648803711, 0.0046482086181640625, 0.004518747329711914, 0.0045318603515625, 0.00463557243347168, 0.0045452117919921875, 0.00449371337890625, 0.00458979606628418, 0.004665374755859375, 0.004664421081542969, 0.004564523696899414, 0.0046122074127197266, 0.004598855972290039, 0.004569530487060547, 0.004712581634521484, 0.0045659542083740234, 0.004560947418212891, 0.004594326019287109, 0.004483699798583984, 0.004580259323120117, 0.004618644714355469, 0.004667520523071289, 0.004571676254272461, 0.004667043685913086, 0.004566669464111328, 0.0046422481536865234, 0.004584789276123047, 0.004534006118774414, 0.0046346187591552734, 0.004546642303466797, 0.004594087600708008, 0.004514455795288086, 0.004529237747192383, 0.004617452621459961, 0.00457000732421875, 0.004526615142822266, 0.0045473575592041016, 0.004578351974487305, 0.00457000732421875, 0.0046160221099853516, 0.004511594772338867, 0.004590272903442383, 0.004542112350463867, 0.004561185836791992, 0.004540681838989258, 0.004553794860839844, 0.0047130584716796875, 0.004628896713256836, 0.004561424255371094, 0.004576206207275391, 0.004588603973388672, 0.004529714584350586, 0.0046384334564208984, 0.004667043685913086, 0.004631757736206055, 0.004596233367919922, 0.004712104797363281, 0.00473332405090332, 0.004647731781005859, 0.004514217376708984, 0.004575490951538086, 0.004576921463012695, 0.004563808441162109, 0.0045125484466552734, 0.004542112350463867, 0.004583120346069336, 0.004583120346069336, 0.00472569465637207, 0.004637718200683594, 0.004547834396362305, 0.004580020904541016, 0.0045812129974365234, 0.004530191421508789, 0.004618644714355469, 0.004574298858642578, 0.004648685455322266, 0.004638671875, 0.0045239925384521484, 0.004617214202880859, 0.0044786930084228516, 0.004594326019287109, 0.00461125373840332, 0.0045545101165771484, 0.004554271697998047, 0.004537105560302734, 0.004586458206176758, 0.00459742546081543, 0.004558563232421875, 0.004570960998535156, 0.004611492156982422, 0.00462031364440918, 0.004584074020385742, 0.00458073616027832, 0.004526853561401367, 0.0045969486236572266, 0.004534006118774414, 0.004579782485961914, 0.004598379135131836, 0.004519939422607422, 0.004693031311035156, 0.004691123962402344, 0.004667043685913086, 0.00461125373840332, 0.004557609558105469, 0.004530668258666992, 0.004515647888183594, 0.0045757293701171875, 0.00447535514831543, 0.004612922668457031, 0.004555225372314453, 0.0045735836029052734, 0.004718303680419922, 0.004533529281616211, 0.004652261734008789, 0.004556417465209961, 0.004723548889160156, 0.004694700241088867, 0.00457763671875, 0.004580259323120117, 0.004530668258666992, 0.004599809646606445, 0.004618644714355469, 0.004645586013793945, 0.004717350006103516, 0.004510164260864258, 0.0046443939208984375, 0.004646778106689453, 0.004545688629150391, 0.004537105560302734, 0.004567384719848633, 0.0045468807220458984, 0.0046427249908447266, 0.004679679870605469, 0.004590749740600586, 0.004627704620361328, 0.0045087337493896484, 0.004551887512207031, 0.0045740604400634766, 0.004559993743896484, 0.004624128341674805, 0.004492044448852539, 0.00457310676574707, 0.004615068435668945, 0.00470423698425293, 0.004676342010498047, 0.004680156707763672, 0.004539012908935547, 0.0046176910400390625, 0.004526376724243164, 0.004560708999633789, 0.004602909088134766, 0.004634380340576172, 0.004700183868408203, 0.0045185089111328125, 0.004738807678222656, 0.004642009735107422, 0.004534244537353516, 0.004566192626953125, 0.004554271697998047, 0.0045092105865478516, 0.004702091217041016, 0.004647731781005859, 0.004613161087036133, 0.0045773983001708984, 0.0045490264892578125, 0.004526376724243164, 0.004583835601806641, 0.004653215408325195, 0.0046825408935546875, 0.004576921463012695, 0.004616498947143555, 0.004584312438964844, 0.004525661468505859, 0.004685878753662109, 0.004577159881591797, 0.004669904708862305, 0.004665851593017578, 0.00464940071105957, 0.004729509353637695, 0.004565715789794922, 0.0046422481536865234, 0.004636526107788086, 0.004530668258666992, 0.0045435428619384766, 0.004578113555908203, 0.0045506954193115234, 0.0045773983001708984, 0.004578590393066406, 0.004743337631225586, 0.004582405090332031, 0.004778146743774414, 0.004621982574462891, 0.00452733039855957, 0.004578590393066406, 0.004564523696899414, 0.0046460628509521484, 0.004610300064086914, 0.00460505485534668, 0.004708528518676758, 0.0047261714935302734, 0.004653453826904297, 0.004590749740600586, 0.0047686100006103516, 0.00473475456237793, 0.004656553268432617, 0.004553794860839844, 0.004647970199584961, 0.004562854766845703, 0.004637479782104492, 0.0046939849853515625, 0.004555702209472656, 0.004549264907836914, 0.004828929901123047, 0.0047266483306884766, 0.004644632339477539, 0.00452113151550293, 0.004609107971191406, 0.00466156005859375, 0.00451970100402832, 0.004559755325317383, 0.004572391510009766, 0.00457310676574707, 0.004589080810546875, 0.004533052444458008, 0.00454401969909668, 0.004639387130737305, 0.004567384719848633, 0.004674196243286133, 0.0047740936279296875, 0.004606485366821289, 0.004654407501220703, 0.004968404769897461, 0.004593849182128906, 0.004562854766845703, 0.004614830017089844, 0.004652261734008789, 0.004546403884887695, 0.00452876091003418, 0.004582881927490234, 0.004590034484863281, 0.0045125484466552734, 0.004773378372192383, 0.004624366760253906, 0.004721879959106445, 0.0045070648193359375, 0.004593610763549805, 0.004593849182128906, 0.004563808441162109, 0.004599094390869141, 0.0046389102935791016, 0.004582881927490234, 0.004629850387573242, 0.004725456237792969, 0.004565715789794922, 20.559864282608032]

ax_3[n].plot(t_12_none[:-1], label=f"chunk size 4096, None")
ax_3[n].plot(t_14_none[:-1], label=f"chunk size 16384, None")
ax_3[n].plot(t_12_gzip[:-1], label=f"chunk size 4096, gzip")
ax_3[n].plot(t_14_gzip[:-1], label=f"chunk size 16384, gzip")
ax_3[n].plot(t_12_lzf[:-1], label=f"chunk size 4096, lzf")
ax_3[n].plot(t_14_lzf[:-1], label=f"chunk size 16384, lzf")
ax_3[n].legend(loc='center left')
ax_3[n].set_title('No versioning')
    
plt.xlabel("Number of transactions")
plt.ylabel("Time (in seconds)")
plt.show()

This behaviour is similar to what we got in the `test_large_fraction_changes_sparse` case, but with larger transaction numbers there is a slight increase in the time required to write new versions as the file size grows.

[Back to top](#home)

<a id='test4'></a>

# Test 4: Mostly appends (dense)

In [ ]:
testname = "test_mostly_appends_dense"

For this case, we have tested the following number of transactions:

```python
num_transactions_4 = [50, 100, 200]
```

Change `num_transactions_4`, `exponents_4` and `compression_4` as desired:

In [ ]:
num_transactions_4 = [25, 50, 100, 200]
exponents_4 = [12, 14]
compression_4 = [None, "gzip", "lzf"]

In [ ]:
testcase = performance_tests.test_mostly_appends_dense(path=path,
                                                       num_transactions=num_transactions_4, 
                                                       exponents=exponents_4, 
                                                       compression=compression_4)

testcase_4, msg = testcase.create_files()
if msg:
    print(msg)
t_sizes_4 = [test['theoretical_sizes'] for test in testcase.tests[-len(num_transactions_4):]]

If you wish to save the results for later, execute the cell below:

In [ ]:
#testcase.save(testcase_4)

### Reading previously computed tests

To open an existing `.json` file, use

In [ ]:
#with open(f"{testname}.json", "r") as json_in:
#    testcase_4 = json.load(json_in)
#t_sizes_4 = None

## Analysis

Let's show the size information in a graph:

In [ ]:
num_transactions_4 = [test['num_transactions'] for test in testcase_4]
chunk_sizes_4 = [test['chunk_size'] for test in testcase_4]
compression_4 = [test['compression'] for test in testcase_4]

filesizes_4 = np.array([test['size'] for test in testcase_4])
sizelabels_4 = np.array([test['size_label'] for test in testcase_4])

n = len(set(num_transactions_4))
ncs = len(set(chunk_sizes_4))
ncomp = len(set(compression_4))

fig_mostly_appends_dense = plt.figure(figsize=(14,10))

if t_sizes_4:
    plt.plot(num_transactions_4[:n], t_sizes_4, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_4[:n], 
                 filesizes_4[start+j*n:start+(j+1)*n], 
                 '*--', ms=12,
                 label=f"Chunk size {chunk_sizes_4[start+j*n]}, {compression_4[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
plt.xticks(num_transactions_4[:n])
plt.yticks(filesizes_4[-n:], sizelabels_4[-n:])
plt.grid()
plt.show()

Changing the view to a logarithmic scale, we have the following:

In [ ]:
fig_mostly_appends_dense_log = plt.figure(figsize=(14,10))

if t_sizes_4:
    plt.loglog(num_transactions_4[:n], t_sizes_4, 'o--', ms=5, color='magenta', label="Theoretical file size")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.loglog(num_transactions_4[:n], 
                   filesizes_4[start+j*n:start+(j+1)*n], 
                   '*--', ms=12, 
                   label=f"Chunk size {chunk_sizes_4[start+j*n]}, {compression_4[start]}")

plt.xlabel("Transactions")
plt.title(f"{testname}")
plt.legend()
plt.grid()
plt.xticks(num_transactions_4[:n], num_transactions_4[:n])
plt.yticks(filesizes_4[-n:], sizelabels_4[-n:])
plt.minorticks_off()
plt.show()

### Comparing compression algorithms

For each chunk size that we chose to test, let's compare the file sizes corresponding to each compression algorithm that we used.

In [ ]:
fig_comp_4, ax_comp_4 = plt.subplots(ncs, figsize=(10,8), sharey=True)
fig_comp_4.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_comp_4[j].loglog(num_transactions_4[:n],
                            filesizes_4[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"{compression_4[start]}")
        ax_comp_4[j].legend()
        ax_comp_4[j].set_title(f"Chunk Size {chunk_sizes_4[start+j*n]}")
        ax_comp_4[j].set_xticks(num_transactions_4[:n])
        ax_comp_4[j].set_xticklabels(num_transactions_4[:n])
        ax_comp_4[j].set_yticks(filesizes_4[-n:])
        ax_comp_4[j].set_yticklabels(sizelabels_4[-n:])
        ax_comp_4[j].minorticks_off()
        ax_comp_4[j].grid()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

### Comparing chunk sizes

Now, for each choice of compression algorithm, we compare different chunk sizes.

In [ ]:
fig_chunks_4, ax_chunks_4 = plt.subplots(ncomp, figsize=(10,10), sharey=True)
fig_chunks_4.suptitle(f"{testname}: File sizes")

for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        ax_chunks_4[i].loglog(num_transactions_4[:n],
                            filesizes_4[start+j*n:start+(j+1)*n],
                            '*--', ms=12, 
                            label=f"Chunk size {chunk_sizes_4[start+j*n]}")
        ax_chunks_4[i].legend()
        ax_chunks_4[i].set_title(f"Compression: {compression_4[start]}")
        ax_chunks_4[i].set_xticks(num_transactions_4[:n])
        ax_chunks_4[i].set_xticklabels(num_transactions_4[:n])
        ax_chunks_4[i].set_yticks(filesizes_4[-n:])
        ax_chunks_4[i].set_yticklabels(sizelabels_4[-n:])
        ax_chunks_4[i].minorticks_off()
        ax_chunks_4[i].grid()

plt.xlabel("Transactions")
plt.suptitle(f"{testname}")
plt.show()

## Creation times

If we look at the creation times for these files, we have something like this:

In [ ]:
t_write_4 = np.array([test['t_write'][-1] for test in testcase_4])

fig_mostly_appends_dense_times = plt.figure(figsize=(10,8))
for i in range(ncomp):
    start = i*ncs*n
    for j in range(ncs):
        plt.plot(num_transactions_4[:n], 
                 filesizes_4[start+j*n:start+(j+1)*n], 
                 '*--', ms=12, 
                 label=f"Chunk size {chunk_sizes_4[start+j*n]}, {compression_4[start]}")
        
plt.plot(200, 2.318243980407715, 'ko', ms=10, label=f"Chunk size 4096, None, No versioning")
plt.plot(200, 2.117753267288208, 'o', ms=10, label=f"Chunk size 16384, None, No versioning")
plt.plot(200, 15.863131284713745, 'o', ms=10, label=f"Chunk size 4096, gzip, No versioning")
plt.plot(200, 21.600487232208252, 'o', ms=10, label=f"Chunk size 16384, gzip, No versioning")
plt.plot(200, 3.988503932952881, 'o', ms=10, label=f"Chunk size 4096, lzf, No versioning")
plt.plot(200, 3.9731950759887695, 'o', ms=10, label=f"Chunk size 16384, lzf, No versioning")

plt.xlabel("Transactions")
plt.title(f"{testname} - creation times in seconds")
plt.legend()
plt.xticks(num_transactions_4[:n])
plt.show()

Again, the graphs below show, for each fixed number of transactions, the time required to add new versions as the file is created.

In [ ]:
fig_times_4, ax_4 = plt.subplots(n+1, figsize=(14,18))
fig_times_4.suptitle(f"{testname}: time to write each new version")

for i in range(n):
    for test in testcase_4:
        if test['num_transactions'] == num_transactions_4[i]:
            t_write = np.array(test['t_write'][:-1])
            ax_4[i].plot(t_write, 
                         label=f"chunk size {test['chunk_size']}, {test['compression']}")
            ax_4[i].legend(loc='center left')
            
t_12_none = [0.005438804626464844, 0.003951549530029297, 0.0039670467376708984, 0.0039179325103759766, 0.003828763961791992, 0.003949165344238281, 0.004018545150756836, 0.003923177719116211, 0.004056692123413086, 0.0041043758392333984, 0.004675388336181641, 0.004052400588989258, 0.004063844680786133, 0.00563812255859375, 0.00532984733581543, 0.005133867263793945, 0.005067110061645508, 0.005059242248535156, 0.005429267883300781, 0.00609588623046875, 0.0054340362548828125, 0.005124807357788086, 0.0050852298736572266, 0.005201101303100586, 0.005255699157714844, 0.005125999450683594, 0.004335641860961914, 0.003972768783569336, 0.00404047966003418, 0.004115104675292969, 0.0043506622314453125, 0.006365060806274414, 0.00475001335144043, 0.004373073577880859, 0.0072824954986572266, 0.007182121276855469, 0.006766080856323242, 0.005095243453979492, 0.004848003387451172, 0.004364013671875, 0.004247426986694336, 0.004233598709106445, 0.0043032169342041016, 0.004299163818359375, 0.0043201446533203125, 0.0044133663177490234, 0.004600048065185547, 0.004395008087158203, 0.004460334777832031, 0.004390239715576172, 0.00445246696472168, 0.004514217376708984, 0.0046024322509765625, 0.004647493362426758, 0.004587411880493164, 0.004879474639892578, 0.004693031311035156, 0.004721403121948242, 0.004849433898925781, 0.004662275314331055, 0.0047261714935302734, 0.004743814468383789, 0.004804849624633789, 0.004938364028930664, 0.004869222640991211, 0.0048868656158447266, 0.004898786544799805, 0.004956483840942383, 0.0050694942474365234, 0.00510859489440918, 0.005018949508666992, 0.0051920413970947266, 0.005239725112915039, 0.005265712738037109, 0.0051767826080322266, 0.005174875259399414, 0.005313396453857422, 0.005344390869140625, 0.005539655685424805, 0.005535602569580078, 0.00538945198059082, 0.005713462829589844, 0.0066165924072265625, 0.006857395172119141, 0.006665945053100586, 0.006735563278198242, 0.006769657135009766, 0.0063707828521728516, 0.0062830448150634766, 0.0065419673919677734, 0.006289005279541016, 0.006722211837768555, 0.006500959396362305, 0.006895303726196289, 0.006615161895751953, 0.006937980651855469, 0.006642341613769531, 0.007595062255859375, 0.007843971252441406, 0.0075299739837646484, 0.007918357849121094, 0.007513761520385742, 0.008041620254516602, 0.007318973541259766, 0.008188962936401367, 0.007803440093994141, 0.010311126708984375, 0.010341405868530273, 0.009794950485229492, 0.011211633682250977, 0.01074361801147461, 0.010708808898925781, 0.012063980102539062, 0.011737346649169922, 0.012218475341796875, 0.013749122619628906, 0.024964094161987305, 0.018387794494628906, 0.024524450302124023, 0.029205799102783203, 0.019881725311279297, 0.01245260238647461, 0.011987447738647461, 0.012950897216796875, 0.012918710708618164, 0.012939929962158203, 0.013449907302856445, 0.013852834701538086, 0.013837575912475586, 0.014124393463134766, 0.014367818832397461, 0.014563560485839844, 0.014736413955688477, 0.014783620834350586, 0.014830589294433594, 0.01511383056640625, 0.015217304229736328, 0.015835285186767578, 0.014815568923950195, 0.016353845596313477, 0.015672922134399414, 0.015836238861083984, 0.0160982608795166, 0.016228437423706055, 0.01638484001159668, 0.016506433486938477, 0.016748905181884766, 0.016818761825561523, 0.01694631576538086, 0.01717376708984375, 0.018145322799682617, 0.024765968322753906, 0.017691373825073242, 0.01768636703491211, 0.01781749725341797, 0.018011808395385742, 0.026352643966674805, 0.022960901260375977, 0.01940298080444336, 0.01959371566772461, 0.019666433334350586, 0.01985788345336914, 0.020035266876220703, 0.020304203033447266, 0.02033686637878418, 0.020566940307617188, 0.0206451416015625, 0.020866870880126953, 0.02023458480834961, 0.020165443420410156, 0.020457983016967773, 0.020584583282470703, 0.020773887634277344, 0.028786182403564453, 0.021442890167236328, 0.020444154739379883, 0.020479202270507812, 0.020667314529418945, 0.0207669734954834, 0.021342039108276367, 0.021187543869018555, 0.021331071853637695, 0.02140498161315918, 0.021672487258911133, 0.021844863891601562, 0.02201676368713379, 0.02208876609802246, 0.02232837677001953, 0.022336721420288086, 0.022527456283569336, 0.022705793380737305, 0.023065805435180664, 0.023076534271240234, 0.023266077041625977, 0.023492813110351562, 0.028560400009155273, 0.027880430221557617, 0.025265216827392578, 0.02536320686340332, 0.025438308715820312, 2.318243980407715]
t_14_none = [0.004990100860595703, 0.00370025634765625, 0.0038318634033203125, 0.003692150115966797, 0.0036356449127197266, 0.003838062286376953, 0.003844738006591797, 0.003774881362915039, 0.003782510757446289, 0.003803253173828125, 0.0044612884521484375, 0.005806446075439453, 0.006129741668701172, 0.005214214324951172, 0.004731655120849609, 0.00413060188293457, 0.004225492477416992, 0.004486083984375, 0.0042417049407958984, 0.0042994022369384766, 0.004260540008544922, 0.004318952560424805, 0.0043909549713134766, 0.004312276840209961, 0.004347085952758789, 0.00439906120300293, 0.004357814788818359, 0.0040247440338134766, 0.003862619400024414, 0.0039980411529541016, 0.00439000129699707, 0.0053408145904541016, 0.0055997371673583984, 0.0041332244873046875, 0.004058122634887695, 0.004027605056762695, 0.0041348934173583984, 0.004035234451293945, 0.004099607467651367, 0.004122018814086914, 0.004349470138549805, 0.0041751861572265625, 0.004152536392211914, 0.004185914993286133, 0.004320859909057617, 0.0043947696685791016, 0.004311084747314453, 0.004265785217285156, 0.004311084747314453, 0.0043184757232666016, 0.004384279251098633, 0.004365205764770508, 0.0044803619384765625, 0.004381656646728516, 0.004481077194213867, 0.004410505294799805, 0.004443168640136719, 0.004471540451049805, 0.004502058029174805, 0.0044248104095458984, 0.004555940628051758, 0.004498720169067383, 0.004628419876098633, 0.004602670669555664, 0.004624128341674805, 0.00457763671875, 0.004662036895751953, 0.00472259521484375, 0.0047380924224853516, 0.005285024642944336, 0.00494384765625, 0.004929780960083008, 0.004903554916381836, 0.0049190521240234375, 0.004964113235473633, 0.004923105239868164, 0.00629115104675293, 0.005647897720336914, 0.006927490234375, 0.015180110931396484, 0.014491558074951172, 0.010172605514526367, 0.013298273086547852, 0.016260862350463867, 0.013032913208007812, 0.0058786869049072266, 0.005822896957397461, 0.005810737609863281, 0.005747556686401367, 0.005749225616455078, 0.0056917667388916016, 0.005773305892944336, 0.005805015563964844, 0.005817413330078125, 0.005857229232788086, 0.00570368766784668, 0.0057332515716552734, 0.0057163238525390625, 0.005984306335449219, 0.006302595138549805, 0.0060155391693115234, 0.006232500076293945, 0.006039142608642578, 0.0060961246490478516, 0.006266117095947266, 0.006989240646362305, 0.007616519927978516, 0.007536888122558594, 0.008083105087280273, 0.0077207088470458984, 0.007991790771484375, 0.007951736450195312, 0.007949352264404297, 0.008511543273925781, 0.008323907852172852, 0.008819103240966797, 0.008534669876098633, 0.008453130722045898, 0.008671998977661133, 0.008925199508666992, 0.008925914764404297, 0.00916147232055664, 0.009526491165161133, 0.009450912475585938, 0.009165763854980469, 0.009467363357543945, 0.009658336639404297, 0.009705781936645508, 0.009983062744140625, 0.009778738021850586, 0.009818792343139648, 0.01023411750793457, 0.010463476181030273, 0.010014533996582031, 0.010033607482910156, 0.010451555252075195, 0.010307073593139648, 0.010434389114379883, 0.010892391204833984, 0.014606952667236328, 0.028780460357666016, 0.016382932662963867, 0.020708322525024414, 0.011212348937988281, 0.010921716690063477, 0.011440038681030273, 0.011395931243896484, 0.012813329696655273, 0.03631091117858887, 0.044525146484375, 0.031031370162963867, 0.04678845405578613, 0.032189130783081055, 0.03350973129272461, 0.03548765182495117, 0.042226552963256836, 0.024011850357055664, 0.03892230987548828, 0.01334834098815918, 0.013066768646240234, 0.01576089859008789, 0.01596522331237793, 0.015569210052490234, 0.015620231628417969, 0.015863418579101562, 0.015807151794433594, 0.016237735748291016, 0.016104459762573242, 0.01638054847717285, 0.016407489776611328, 0.016764163970947266, 0.01638484001159668, 0.016732215881347656, 0.01694631576538086, 0.01734185218811035, 0.017531871795654297, 0.017764568328857422, 0.01770162582397461, 0.017717599868774414, 0.018051862716674805, 0.01807117462158203, 0.028198719024658203, 0.018577098846435547, 0.018051862716674805, 0.018157243728637695, 0.01809382438659668, 0.01826000213623047, 0.01847219467163086, 0.019032716751098633, 0.019295454025268555, 0.0188751220703125, 0.019167661666870117, 0.01968836784362793, 0.019621610641479492, 0.019629716873168945, 0.019894123077392578, 0.01947641372680664, 0.019899368286132812, 0.01987314224243164, 0.019742965698242188, 2.117753267288208]
t_12_gzip = [0.004727363586425781, 0.00363922119140625, 0.003749370574951172, 0.003692150115966797, 0.003694295883178711, 0.0037429332733154297, 0.003713846206665039, 0.003673076629638672, 0.0038962364196777344, 0.003809213638305664, 0.003795146942138672, 0.004307746887207031, 0.0040013790130615234, 0.006106376647949219, 0.00470423698425293, 0.0047779083251953125, 0.004897594451904297, 0.0048291683197021484, 0.004324674606323242, 0.0047147274017333984, 0.004858732223510742, 0.004882335662841797, 0.004666805267333984, 0.0041120052337646484, 0.0040853023529052734, 0.004524707794189453, 0.004847526550292969, 0.005062103271484375, 0.004633188247680664, 0.004538297653198242, 0.004012584686279297, 0.016528844833374023, 0.010407209396362305, 0.00428462028503418, 0.004243135452270508, 0.004265546798706055, 0.004273891448974609, 0.0042819976806640625, 0.004566192626953125, 0.004312038421630859, 0.004189491271972656, 0.004163980484008789, 0.0042858123779296875, 0.004174709320068359, 0.004268169403076172, 0.00444483757019043, 0.004271745681762695, 0.004369020462036133, 0.004449129104614258, 0.004398345947265625, 0.0043544769287109375, 0.0043714046478271484, 0.004476785659790039, 0.00444483757019043, 0.004395723342895508, 0.004546403884887695, 0.004533052444458008, 0.004498720169067383, 0.004561185836791992, 0.0045909881591796875, 0.00469970703125, 0.0046961307525634766, 0.0046977996826171875, 0.00478053092956543, 0.0047647953033447266, 0.005078792572021484, 0.004972219467163086, 0.00490117073059082, 0.004850864410400391, 0.004975080490112305, 0.0050318241119384766, 0.00504302978515625, 0.005005359649658203, 0.005028724670410156, 0.005065441131591797, 0.005173921585083008, 0.00515294075012207, 0.0052297115325927734, 0.005390167236328125, 0.0052585601806640625, 0.005282163619995117, 0.005313396453857422, 0.005350589752197266, 0.005370616912841797, 0.005522251129150391, 0.0057811737060546875, 0.005571603775024414, 0.005618095397949219, 0.005723237991333008, 0.005746603012084961, 0.0057904720306396484, 0.00578761100769043, 0.0057947635650634766, 0.005847454071044922, 0.005931854248046875, 0.005992412567138672, 0.006061077117919922, 0.006134033203125, 0.006115436553955078, 0.0061719417572021484, 0.006387948989868164, 0.006277322769165039, 0.006178140640258789, 0.006424903869628906, 0.007373809814453125, 0.010608434677124023, 0.023322582244873047, 0.031034231185913086, 0.043831825256347656, 0.07163453102111816, 0.04822969436645508, 0.04831743240356445, 0.06162118911743164, 0.05909919738769531, 0.06485700607299805, 0.06493473052978516, 0.06528997421264648, 0.08198070526123047, 0.10296010971069336, 0.09701180458068848, 0.09700775146484375, 0.10025548934936523, 0.10781359672546387, 0.10431075096130371, 0.10426020622253418, 0.10645008087158203, 0.10685014724731445, 0.11697053909301758, 0.11113810539245605, 0.11200881004333496, 0.11546158790588379, 0.1273820400238037, 0.1230461597442627, 0.1269826889038086, 0.12906479835510254, 0.13301515579223633, 0.13196921348571777, 0.13494873046875, 0.14006900787353516, 0.14101362228393555, 0.14153838157653809, 0.14689254760742188, 0.15050768852233887, 0.14568614959716797, 0.14580082893371582, 0.1534738540649414, 0.14908695220947266, 0.1511063575744629, 0.1508629322052002, 0.16205954551696777, 0.15612268447875977, 0.15784692764282227, 0.1639554500579834, 0.16106820106506348, 0.16404938697814941, 0.16907596588134766, 0.16787290573120117, 0.16911792755126953, 0.19305944442749023, 0.18614697456359863, 0.18318939208984375, 0.17853593826293945, 0.17779827117919922, 0.18496131896972656, 0.1837902069091797, 0.18782711029052734, 0.1987752914428711, 0.19536828994750977, 0.20338082313537598, 0.19734954833984375, 0.19782805442810059, 0.19785690307617188, 0.19672155380249023, 0.2078237533569336, 0.22399497032165527, 0.21216368675231934, 0.21937203407287598, 0.21140718460083008, 0.22104382514953613, 0.21892285346984863, 0.2266840934753418, 0.22591423988342285, 0.23158907890319824, 0.227949857711792, 0.22841763496398926, 0.2384958267211914, 0.23509955406188965, 0.23911619186401367, 0.2409675121307373, 0.24990200996398926, 0.24210333824157715, 0.24985814094543457, 0.2439708709716797, 0.2580552101135254, 0.25502872467041016, 0.26095104217529297, 0.25511765480041504, 0.2624988555908203, 0.26048946380615234, 0.2754096984863281, 15.863131284713745]
t_14_gzip = [0.004693269729614258, 0.00345611572265625, 0.0038826465606689453, 0.003789186477661133, 0.003713369369506836, 0.0038423538208007812, 0.0036237239837646484, 0.0036666393280029297, 0.0036847591400146484, 0.003707408905029297, 0.0037734508514404297, 0.0052754878997802734, 0.004065990447998047, 0.004486799240112305, 0.005158901214599609, 0.004712581634521484, 0.004953622817993164, 0.004696369171142578, 0.0049877166748046875, 0.005084991455078125, 0.004965782165527344, 0.004951000213623047, 0.004942417144775391, 0.004855155944824219, 0.005101919174194336, 0.005037069320678711, 0.005088090896606445, 0.00444483757019043, 0.003911495208740234, 0.004264354705810547, 0.0049610137939453125, 0.0054569244384765625, 0.0064373016357421875, 0.005346775054931641, 0.007003068923950195, 0.008733749389648438, 0.010215044021606445, 0.010423660278320312, 0.005241870880126953, 0.009280681610107422, 0.010267972946166992, 0.011477231979370117, 0.009639501571655273, 0.011068105697631836, 0.005317211151123047, 0.004555940628051758, 0.0044498443603515625, 0.004488706588745117, 0.00451350212097168, 0.004480600357055664, 0.004523754119873047, 0.004498720169067383, 0.004325389862060547, 0.0043792724609375, 0.004441738128662109, 0.004381895065307617, 0.0044367313385009766, 0.004416227340698242, 0.004437446594238281, 0.004557609558105469, 0.004609823226928711, 0.004657745361328125, 0.004574775695800781, 0.004574775695800781, 0.004777431488037109, 0.004742860794067383, 0.004590272903442383, 0.004773855209350586, 0.0049097537994384766, 0.004896402359008789, 0.0048983097076416016, 0.005001544952392578, 0.00484156608581543, 0.0049571990966796875, 0.0049686431884765625, 0.005068063735961914, 0.004947662353515625, 0.005101203918457031, 0.0051381587982177734, 0.005212545394897461, 0.005148649215698242, 0.0053904056549072266, 0.005308628082275391, 0.005350351333618164, 0.005442619323730469, 0.005366086959838867, 0.0053997039794921875, 0.0055086612701416016, 0.005502462387084961, 0.005572319030761719, 0.005720376968383789, 0.005598783493041992, 0.005845785140991211, 0.005689144134521484, 0.0057141780853271484, 0.005865335464477539, 0.0059719085693359375, 0.0059413909912109375, 0.005984067916870117, 0.005872964859008789, 0.006009340286254883, 0.006117820739746094, 0.0061452388763427734, 0.006018877029418945, 0.012264490127563477, 0.034027099609375, 0.07711052894592285, 0.10954022407531738, 0.061640024185180664, 0.06488466262817383, 0.07532405853271484, 0.061960697174072266, 0.07664012908935547, 0.12492680549621582, 0.1297602653503418, 0.11186933517456055, 0.12694501876831055, 0.13276219367980957, 0.1349353790283203, 0.13822674751281738, 0.1346292495727539, 0.14695382118225098, 0.14291143417358398, 0.1408369541168213, 0.14243793487548828, 0.15088915824890137, 0.1536102294921875, 0.15506196022033691, 0.166642427444458, 0.15906238555908203, 0.158203125, 0.1744217872619629, 0.16745305061340332, 0.16689085960388184, 0.17270207405090332, 0.17084145545959473, 0.16823673248291016, 0.19536733627319336, 0.18143153190612793, 0.1869034767150879, 0.1833817958831787, 0.18593430519104004, 0.1875777244567871, 0.1947329044342041, 0.1977252960205078, 0.209425687789917, 0.1974809169769287, 0.21469712257385254, 0.21185612678527832, 0.2160048484802246, 0.21351885795593262, 0.20863723754882812, 0.2179100513458252, 0.2127392292022705, 0.2385098934173584, 0.2228076457977295, 0.22456574440002441, 0.22356772422790527, 0.23282742500305176, 0.2544701099395752, 0.23727202415466309, 0.24796843528747559, 0.24651288986206055, 0.24067306518554688, 0.2682178020477295, 0.25079870223999023, 0.2563474178314209, 0.27389097213745117, 0.26083898544311523, 0.28403663635253906, 0.2833380699157715, 0.273054838180542, 0.26691198348999023, 0.29428553581237793, 0.2842826843261719, 0.3023512363433838, 0.28610730171203613, 0.28348588943481445, 0.28180384635925293, 0.3032090663909912, 0.30673813819885254, 0.31662726402282715, 0.3134489059448242, 0.31517815589904785, 0.318096399307251, 0.3040027618408203, 0.31390833854675293, 0.32845640182495117, 0.34217405319213867, 0.3387329578399658, 0.33047938346862793, 0.33347439765930176, 0.3292050361633301, 0.3412151336669922, 0.34862542152404785, 0.33614063262939453, 0.3518080711364746, 0.37036728858947754, 0.37462568283081055, 0.3508174419403076, 21.600487232208252]
t_12_lzf = [0.004712820053100586, 0.0035114288330078125, 0.0036263465881347656, 0.00356292724609375, 0.003803253173828125, 0.0038330554962158203, 0.0035598278045654297, 0.0036296844482421875, 0.003653287887573242, 0.003679990768432617, 0.0037724971771240234, 0.00620722770690918, 0.005317211151123047, 0.004961490631103516, 0.006325721740722656, 0.004868268966674805, 0.009664535522460938, 0.007977724075317383, 0.004678487777709961, 0.00450897216796875, 0.004485130310058594, 0.004616260528564453, 0.00449371337890625, 0.00452113151550293, 0.004572391510009766, 0.0045626163482666016, 0.0046198368072509766, 0.004638195037841797, 0.0040721893310546875, 0.004002809524536133, 0.003990650177001953, 0.0042111873626708984, 0.0070209503173828125, 0.0048406124114990234, 0.004268169403076172, 0.004077434539794922, 0.004045963287353516, 0.004076242446899414, 0.004067897796630859, 0.004267454147338867, 0.004118919372558594, 0.004114866256713867, 0.004194974899291992, 0.004174470901489258, 0.004213571548461914, 0.004238605499267578, 0.004255056381225586, 0.00425267219543457, 0.004230976104736328, 0.004349231719970703, 0.004333972930908203, 0.004347324371337891, 0.004334688186645508, 0.004408121109008789, 0.004430294036865234, 0.0043904781341552734, 0.004499912261962891, 0.004613161087036133, 0.004573345184326172, 0.004518032073974609, 0.0046117305755615234, 0.004591464996337891, 0.004723072052001953, 0.0046308040618896484, 0.004652261734008789, 0.004811286926269531, 0.004729270935058594, 0.004868268966674805, 0.005379438400268555, 0.004869699478149414, 0.0048980712890625, 0.0050008296966552734, 0.004994630813598633, 0.005118370056152344, 0.00513911247253418, 0.005078792572021484, 0.005166769027709961, 0.005270957946777344, 0.005371809005737305, 0.005236148834228516, 0.005376577377319336, 0.005301952362060547, 0.005290031433105469, 0.005438327789306641, 0.00541377067565918, 0.005801200866699219, 0.005556344985961914, 0.005643606185913086, 0.005585193634033203, 0.005685567855834961, 0.005655527114868164, 0.005684375762939453, 0.005688905715942383, 0.005891561508178711, 0.005904674530029297, 0.00587916374206543, 0.0060787200927734375, 0.0060002803802490234, 0.006036043167114258, 0.006142139434814453, 0.006075859069824219, 0.00625157356262207, 0.006215095520019531, 0.006291627883911133, 0.007398843765258789, 0.008364439010620117, 0.010295867919921875, 0.011233329772949219, 0.012421607971191406, 0.01470494270324707, 0.020822525024414062, 0.039312124252319336, 0.048372745513916016, 0.016988277435302734, 0.017446041107177734, 0.017400026321411133, 0.01732778549194336, 0.01861119270324707, 0.018787145614624023, 0.0236513614654541, 0.0239412784576416, 0.024024248123168945, 0.024809598922729492, 0.024871826171875, 0.024747371673583984, 0.02539205551147461, 0.025490999221801758, 0.02563190460205078, 0.02655649185180664, 0.026568889617919922, 0.027072906494140625, 0.027124881744384766, 0.027234315872192383, 0.02774357795715332, 0.028185606002807617, 0.0400235652923584, 0.02907562255859375, 0.02921271324157715, 0.029178857803344727, 0.02928924560546875, 0.028881311416625977, 0.029021501541137695, 0.02953314781188965, 0.029719829559326172, 0.03003859519958496, 0.0312960147857666, 0.03164052963256836, 0.031903982162475586, 0.031618356704711914, 0.0334780216217041, 0.0507199764251709, 0.034897565841674805, 0.03503298759460449, 0.03558015823364258, 0.03569173812866211, 0.035269737243652344, 0.035277366638183594, 0.03566884994506836, 0.03585457801818848, 0.036351919174194336, 0.03675985336303711, 0.040377140045166016, 0.05057382583618164, 0.04165530204772949, 0.04131674766540527, 0.041445255279541016, 0.04154229164123535, 0.041990041732788086, 0.04207611083984375, 0.04220175743103027, 0.04274249076843262, 0.04314374923706055, 0.043462514877319336, 0.0442042350769043, 0.05234503746032715, 0.0459599494934082, 0.04591798782348633, 0.04589700698852539, 0.0456080436706543, 0.046024322509765625, 0.04624533653259277, 0.04735898971557617, 0.04727983474731445, 0.0638418197631836, 0.050966739654541016, 0.0519101619720459, 0.05217599868774414, 0.0521085262298584, 0.05289196968078613, 0.053552865982055664, 0.05365872383117676, 0.05478620529174805, 0.06772160530090332, 0.05577707290649414, 0.054299354553222656, 0.05529141426086426, 0.0543673038482666, 0.05476021766662598, 0.052773237228393555, 0.05477333068847656, 3.988503932952881]
t_14_lzf = [0.004578590393066406, 0.0034782886505126953, 0.003605365753173828, 0.0035941600799560547, 0.0037970542907714844, 0.0036242008209228516, 0.003644227981567383, 0.0036263465881347656, 0.0036313533782958984, 0.003656148910522461, 0.0036907196044921875, 0.004145145416259766, 0.0036580562591552734, 0.003707408905029297, 0.005067110061645508, 0.0040552616119384766, 0.005217552185058594, 0.0045206546783447266, 0.007444143295288086, 0.008994579315185547, 0.005160331726074219, 0.005011558532714844, 0.004871845245361328, 0.005027055740356445, 0.004945516586303711, 0.0050466060638427734, 0.005052328109741211, 0.005114316940307617, 0.004586219787597656, 0.0040280818939208984, 0.003937721252441406, 0.004225492477416992, 0.004297733306884766, 0.006700038909912109, 0.004182577133178711, 0.004064083099365234, 0.004152059555053711, 0.004073381423950195, 0.004033088684082031, 0.004075288772583008, 0.004016876220703125, 0.004178285598754883, 0.0040585994720458984, 0.00410771369934082, 0.0041675567626953125, 0.004181861877441406, 0.004187583923339844, 0.004151105880737305, 0.004291057586669922, 0.004259347915649414, 0.004212856292724609, 0.0042498111724853516, 0.00431370735168457, 0.0043299198150634766, 0.004267215728759766, 0.004410743713378906, 0.0043795108795166016, 0.00446009635925293, 0.004436492919921875, 0.004479169845581055, 0.004512310028076172, 0.004581451416015625, 0.004557371139526367, 0.004588127136230469, 0.004594087600708008, 0.004606723785400391, 0.004624128341674805, 0.0048024654388427734, 0.004654645919799805, 0.0047512054443359375, 0.004798412322998047, 0.004820108413696289, 0.004802227020263672, 0.004849672317504883, 0.004957675933837891, 0.0048980712890625, 0.00490570068359375, 0.004977226257324219, 0.005051851272583008, 0.004982709884643555, 0.00525665283203125, 0.0050961971282958984, 0.0052912235260009766, 0.00515437126159668, 0.00514674186706543, 0.005309104919433594, 0.005224704742431641, 0.0053615570068359375, 0.005295991897583008, 0.005412578582763672, 0.0054857730865478516, 0.005390167236328125, 0.005491971969604492, 0.005503177642822266, 0.005609273910522461, 0.005571842193603516, 0.0056650638580322266, 0.005676984786987305, 0.005819082260131836, 0.006008625030517578, 0.005940675735473633, 0.005949735641479492, 0.005875825881958008, 0.00604248046875, 0.007013559341430664, 0.01074838638305664, 0.014194726943969727, 0.014648199081420898, 0.016812801361083984, 0.017378568649291992, 0.019471168518066406, 0.03907346725463867, 0.0462033748626709, 0.03940439224243164, 0.022314786911010742, 0.02247929573059082, 0.02239513397216797, 0.022486448287963867, 0.022394418716430664, 0.023528099060058594, 0.022681713104248047, 0.023406267166137695, 0.024243831634521484, 0.023988962173461914, 0.024251699447631836, 0.02526998519897461, 0.0251009464263916, 0.02519965171813965, 0.025841951370239258, 0.02676534652709961, 0.026293039321899414, 0.027167320251464844, 0.027114391326904297, 0.038854360580444336, 0.02685546875, 0.02704763412475586, 0.026973485946655273, 0.028784513473510742, 0.030249834060668945, 0.027971506118774414, 0.028329849243164062, 0.028764963150024414, 0.0287477970123291, 0.02962660789489746, 0.029816389083862305, 0.029857158660888672, 0.030391216278076172, 0.03114461898803711, 0.030467748641967773, 0.043514251708984375, 0.034889936447143555, 0.03161931037902832, 0.03212690353393555, 0.03219294548034668, 0.032921552658081055, 0.0340273380279541, 0.032520294189453125, 0.03774833679199219, 0.03654193878173828, 0.03956270217895508, 0.03840470314025879, 0.038256168365478516, 0.05469822883605957, 0.04179835319519043, 0.04180645942687988, 0.04241776466369629, 0.041252851486206055, 0.041272640228271484, 0.041144609451293945, 0.04311347007751465, 0.04259777069091797, 0.04399752616882324, 0.04400062561035156, 0.057297468185424805, 0.04416465759277344, 0.04344630241394043, 0.04122638702392578, 0.040827035903930664, 0.04460000991821289, 0.0469052791595459, 0.04648947715759277, 0.043650150299072266, 0.04402279853820801, 0.059778690338134766, 0.05059099197387695, 0.04904651641845703, 0.04850363731384277, 0.0479428768157959, 0.04715895652770996, 0.04762864112854004, 0.04952502250671387, 0.04895138740539551, 0.05133676528930664, 0.0632016658782959, 0.05532526969909668, 0.05186939239501953, 0.05316948890686035, 0.05720233917236328, 0.07781791687011719, 0.06397700309753418, 3.9731950759887695]

ax_4[n].plot(t_12_none[:-1], label=f"chunk size 4096, None")
ax_4[n].plot(t_14_none[:-1], label=f"chunk size 16384, None")
ax_4[n].plot(t_12_gzip[:-1], label=f"chunk size 4096, gzip")
ax_4[n].plot(t_14_gzip[:-1], label=f"chunk size 16384, gzip")
ax_4[n].plot(t_12_lzf[:-1], label=f"chunk size 4096, lzf")
ax_4[n].plot(t_14_lzf[:-1], label=f"chunk size 16384, lzf")
ax_4[n].legend(loc='center left')    
ax_4[n].set_title('No versioning')
    
plt.xlabel("Number of transactions")
plt.ylabel("Time (in seconds)")
plt.show()

The behaviour is similar to what we observed in `mostly_appends_sparse`.

[Back to top](#home)

## Understanding each file

Each versioned HDF5 file contains 3 datasets per version:
- `key0`, an array of `int64`
- `key1`, an array of `int64`
- `val`, an array of `float64`
plus metadata about groups, datasets and versions.

This means that each file has  

```
nversions * 24 * arraysize + metadata
```
bytes of information.

<a id='standard'></a>
## Standard parameters

- `test_large_fraction_changes_sparse`: 
    - `num_rows_initial = 5000`
    - `num_rows_per_append = 10`
    - `num_inserts = 10`
    - `num_deletes = 10`
    - `num_changes = 1000`
- `test_small_fraction_changes_sparse`
    - `num_rows_initial = 5000`
    - `num_rows_per_append = 10`
    - `num_inserts = 10`
    - `num_deletes = 10`
    - `num_changes = 10`
- `test_mostly_appends_sparse`:
    - `num_rows_initial = 1000`
    - `num_rows_per_append = 1000`
    - `num_inserts = 10`
    - `num_deletes = 10`
    - `num_changes = 10`  
- `test_mostly_appends_dense`
    - `num_rows_initial_0 = 30`
    - `num_rows_initial_1 = 30`
    - `num_rows_per_append_0 = 1`
    - `num_inserts_0 = 1`
    - `num_inserts_1 = 10`
    - `num_deletes_0 = 1`
    - `num_deletes_1 = 1`
    - `num_changes = 10`